<h1><b>Datasets cleaning</h1></b>

> Here I will clean the previously created dataframes 

In [325]:
import pandas as pd
import numpy as np
import os

<h3><b>Stats dataframes</h3></b>

In [326]:
#import all the datasets
stats_defensive = pd.read_csv(os.path.join('defensive','stats_defensive.csv')) 
stats_detailed = pd.read_csv(os.path.join('detailed','stats_detailed.csv')) 
stats_offensive = pd.read_csv(os.path.join('offensive','stats_offensive.csv')) 
stats_passing = pd.read_csv(os.path.join('passing','stats_passing.csv')) 
stats_summary = pd.read_csv(os.path.join('summary','stats_summary.csv')) 

In [327]:
#merge dataframes (the dataframes have the same player order name, meaning that is sufficient to combine them)

#remove name and league
stats_offensive.drop(stats_offensive.columns[[0,6]], axis = 1, inplace = True) 
stats_passing.drop(stats_passing.columns[[0,6]], axis = 1, inplace = True) 
stats_defensive.drop(stats_defensive.columns[[0,9]], axis = 1, inplace = True) 
stats_detailed.drop(stats_detailed.columns[[0,5]], axis = 1, inplace = True) 

#rename Drb to differentiate between defensive and offensive stats
stats_defensive.rename(columns={'Drb': 'Drb_past'}, inplace = True)
stats_offensive.rename(columns={'Drb': 'Drb_per_game'}, inplace = True)

df = pd.concat([stats_summary, stats_defensive, stats_offensive, stats_detailed, stats_passing], axis=1)

In [328]:
df.shape

(10356, 40)

In [329]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10356 entries, 0 to 10355
Data columns (total 40 columns):
Name              10356 non-null object
Team              10356 non-null object
Age               10326 non-null float64
Main_role         10325 non-null object
Secondary_role    2381 non-null object
third_role        70 non-null object
Apps              10356 non-null object
Mins              10356 non-null int64
Goals             10356 non-null object
Assists           10356 non-null object
Yel               10356 non-null object
Red               10356 non-null object
SpG               10356 non-null object
PS%               10356 non-null object
AerialsWon        10356 non-null object
MotM              10356 non-null object
Rating            10356 non-null float64
League            10356 non-null object
Tackles           10356 non-null object
Inter             10356 non-null object
Fouls             10356 non-null object
Offsides          10356 non-null object
Clear         

In [330]:
#Apps contains also the number of times a player started from the bench
#this should be separated from Apps
app = df.Apps.str.split('(',expand=True) 
app.set_axis(['Apps', 'start_bench'], axis = 1, inplace=True)
app['start_bench'] = app['start_bench'].str.replace(')','')

In [331]:
#Insert new coluns in df_clean
df['Apps'] = app['Apps']
df.insert(7,'start_bench', app['start_bench'])

In [332]:
#Main_role, Secondary_role, third_role columns may sometimes report Goalkeeper, Defender, Midfielder, Forward instead of GK, D(C), M(C) and FW

df['Main_role'] = df['Main_role'].str.replace('Goalkeeper','GK')
df['Main_role'] = df['Main_role'].str.replace('Defender','D(C)')
df['Main_role'] = df['Main_role'].str.replace('Midfielder','M(C)')
df['Main_role'] = df['Main_role'].str.replace('Forward','FW')

df['Secondary_role'] = df['Secondary_role'].str.replace('Goalkeeper','GK')
df['Secondary_role'] = df['Secondary_role'].str.replace('Defender','D(C)')
df['Secondary_role'] = df['Secondary_role'].str.replace('Midfielder','M(C)')
df['Secondary_role'] = df['Secondary_role'].str.replace('Forward','FW')

df['third_role'] = df['third_role'].str.replace('Goalkeeper','GK')
df['third_role'] = df['third_role'].str.replace('Defender','D(C)')
df['third_role'] = df['third_role'].str.replace('Midfielder','M(C)')
df['third_role'] = df['third_role'].str.replace('Forward','FW')

In [333]:
pd.DataFrame(df.Main_role.value_counts())

,Main_role
D(C),1677
FW,1402
M(C),1257
GK,783
DMC,678
D(L),614
D(R),549
AM(CLR),429
AM(LR),382
D(CR),315


In [334]:
#check who are the players marked as 'substitute'
df[df['Main_role'].str.contains('Substitute', na=False)]

,Name,Team,Age,Main_role,Secondary_role,third_role,Apps,start_bench,Mins,Goals,...,UnsTch,Total_shots,OutOfBox,SixYardBox,PenaltyArea,KeyP,AvgP,Crosses,LongB,ThrB
10249,Pascal Steinwender,Paderborn,0.0,Substitute,NaN,NaN,0,1,1,-,...,-,-,-,-,-,1,2,-,-,-
10256,Jannis Kleeberg,Eintracht Braunschweig,0.0,Substitute,NaN,NaN,0,1,3,-,...,-,-,-,-,-,-,3,-,-,-
10327,Adrian Schlagbauer,Wuerzburger Kickers,0.0,Substitute,NaN,NaN,0,1,1,-,...,-,-,-,-,-,-,-,-,-,-
10348,Noah Baumann,Erzgebirge Aue,0.0,Substitute,NaN,NaN,0,1,2,-,...,-,-,-,-,-,-,1,-,-,-


In [335]:
#I checked on transfermarkt for the solution
#df.set_value(10249, 'Main_role', 'M(R)')
df.loc[10249, 'Main_role']='M(R)'
df.loc[10256, 'Main_role']='D(L)'
df.loc[10327, 'Main_role']='M(C)'
df.loc[10348, 'Main_role']='M(C)'

In [336]:
#here I need to split each role in the corresponding position(center, left or righ)
position_main = df.Main_role.str.split('(',expand=True) 
position_main.set_axis(['Main_role', 'position'], axis = 1, inplace=True)
position_main['position'] = position_main['position'].str.replace(')','')
position_main.head(5)

,Main_role,position
0,AM,C
1,M,CLR
2,AM,CL
3,M,CLR
4,DMC,None


In [337]:
#fill the nan in the position column
position_main['position'] = position_main['position'].fillna('C')#the rows where position is nan are GK, DMC and FW i.e. positions that are center (C)

In [338]:
#check if nan are present also in Main_role
position_main[position_main.isna().any(axis=1)]

,Main_role,position
8589,NaN,C
8704,NaN,C
8728,NaN,C
8752,NaN,C
8772,NaN,C
8825,NaN,C
8828,NaN,C
8838,NaN,C
8850,NaN,C
8873,NaN,C


In [339]:
#here I check 1 by 1 one the above indexes and I compare with the information reported in trasfermakt
#print(df.loc[8589])
position_main.loc[8589, 'Main_role']='GK'
position_main.loc[8704, 'Main_role']='M(C)'
position_main.loc[8728, 'Main_role']='DMC'
position_main.loc[8752, 'Main_role']='C(RLC)'
position_main.loc[8772, 'Main_role']='D(L)'
position_main.loc[8825, 'Main_role']='DMC'
position_main.loc[8828, 'Main_role']='D(C)'
position_main.loc[8838, 'Main_role']='D(C)'
position_main.loc[8850, 'Main_role']='DMC'
position_main.loc[8873, 'Main_role']='D(R)'
position_main.loc[8879, 'Main_role']='D(C)'
position_main.loc[8887, 'Main_role']='M(L)'
position_main.loc[8894, 'Main_role']='FW'
position_main.loc[8898, 'Main_role']='GK'
position_main.loc[8935, 'Main_role']='D(R)'
position_main.loc[8944, 'Main_role']='FW'
position_main.loc[8955, 'Main_role']='D(R)'
position_main.loc[8961, 'Main_role']='DMC'
position_main.loc[9013, 'Main_role']='D(C)'
position_main.loc[9049, 'Main_role']='M(C)'
position_main.loc[9058, 'Main_role']='DMC'
position_main.loc[9100, 'Main_role']='FW'
position_main.loc[9144, 'Main_role']='D(C)'
position_main.loc[9155, 'Main_role']='FW'
position_main.loc[9180, 'Main_role']='FW'
position_main.loc[9183, 'Main_role']='FW'
position_main.loc[9208, 'Main_role']='C'
position_main.loc[9210, 'Main_role']='M(R)'
position_main.loc[9238, 'Main_role']='FW'
position_main.loc[9276, 'Main_role']='M(L)'
position_main.loc[9278, 'Main_role']='D(L)'

In [340]:
position_main.head(5)

,Main_role,position
0,AM,C
1,M,CLR
2,AM,CL
3,M,CLR
4,DMC,C


In [341]:
# one-hot encode position
position_main['is_pos_c'] = position_main['position'].map(lambda x:1 if 'C' in x else 0)
position_main['is_pos_l'] = position_main['position'].map(lambda x:1 if 'L' in x else 0)
position_main['is_pos_r'] = position_main['position'].map(lambda x:1 if 'R' in x else 0)
position_main.head(5)

,Main_role,position,is_pos_c,is_pos_l,is_pos_r
0,AM,C,1,0,0
1,M,CLR,1,1,1
2,AM,CL,1,1,0
3,M,CLR,1,1,1
4,DMC,C,1,0,0


In [342]:
#insert position_main columns in df
df = df.drop('Main_role', axis = 1)#remove main role from df
position_main = position_main.drop('position', axis = 1)#remove position from position_main
df = pd.concat([df, position_main], axis = 1)

In [343]:
#repeat the same for Secondary_role and third_role

#split each role in the corresponding position(center, left or righ)
position_secondary = df.Secondary_role.str.split('(',expand=True) 
position_secondary.set_axis(['secondary_role', 'position'], axis = 1, inplace=True)
position_secondary['position'] = position_secondary['position'].str.replace(')','')
position_secondary['secondary_role'] = position_secondary['secondary_role'].fillna('None')
position_secondary['position'] = position_secondary['position'].fillna('None')

#one-hot encoding (FW DMC and M are central roles)
position_secondary.loc[position_secondary.secondary_role == 'FW', 'position'] = 'C'
position_secondary.loc[position_secondary.secondary_role == 'DMC', 'position'] = 'C'
position_secondary['is_pos_sec_c'] = position_secondary['position'].map(lambda x:1 if 'C' in x else 0)
position_secondary['is_pos_sec_l'] = position_secondary['position'].map(lambda x:1 if 'L' in x else 0)
position_secondary['is_pos_sec_r'] = position_secondary['position'].map(lambda x:1 if 'R' in x else 0)

In [344]:
position_secondary.head(10)

,secondary_role,position,is_pos_sec_c,is_pos_sec_l,is_pos_sec_r
0,FW,C,1,0,0
1,FW,C,1,0,0
2,None,None,0,0,0
3,None,None,0,0,0
4,None,None,0,0,0
5,None,None,0,0,0
6,FW,C,1,0,0
7,None,None,0,0,0
8,M,R,0,0,1
9,FW,C,1,0,0


In [345]:
#insert position_secondary columns in df
df = df.drop('Secondary_role', axis = 1)#remove Secondary_role from df
position_secondary = position_secondary.drop('position', axis = 1)#remove position from position_secondary
df = pd.concat([df, position_secondary], axis = 1)

In [346]:
#repeat the same for third_role

#split each role in the corresponding position(center, left or righ)
third_role = df.third_role.str.split('(',expand=True) 
third_role.set_axis(['third_role', 'position'], axis = 1, inplace=True)
third_role['position'] = third_role['position'].str.replace(')','')
third_role['third_role'] = third_role['third_role'].fillna('None')
third_role['position'] = third_role['position'].fillna('None')

#one-hot encoding (FW DMC and M are central roles)
third_role.loc[third_role.third_role == 'FW', 'position'] = 'C'
third_role['is_pos_third_c'] = third_role['position'].map(lambda x:1 if 'C' in x else 0)
third_role['is_pos_third_l'] = third_role['position'].map(lambda x:1 if 'L' in x else 0)
third_role['is_pos_third_r'] = third_role['position'].map(lambda x:1 if 'R' in x else 0)

In [347]:
#insert third_role columns in df
df = df.drop('third_role', axis = 1)#remove third_role from df
third_role  = third_role.drop('position', axis = 1)#remove position from third_role 
df = pd.concat([df, third_role], axis = 1)

In [348]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10356 entries, 0 to 10355
Data columns (total 50 columns):
Name              10356 non-null object
Team              10356 non-null object
Age               10326 non-null float64
Apps              10356 non-null object
start_bench       8111 non-null object
Mins              10356 non-null int64
Goals             10356 non-null object
Assists           10356 non-null object
Yel               10356 non-null object
Red               10356 non-null object
SpG               10356 non-null object
PS%               10356 non-null object
AerialsWon        10356 non-null object
MotM              10356 non-null object
Rating            10356 non-null float64
League            10356 non-null object
Tackles           10356 non-null object
Inter             10356 non-null object
Fouls             10356 non-null object
Offsides          10356 non-null object
Clear             10356 non-null object
Drb_past          10356 non-null object
Blocks     

Fill missing value in Age column

In [349]:
#check nan in Age
df[df['Age'].isnull()].index.tolist()

[8704,
 8728,
 8752,
 8772,
 8825,
 8828,
 8838,
 8850,
 8873,
 8879,
 8887,
 8894,
 8898,
 8935,
 8944,
 8955,
 8961,
 9013,
 9049,
 9058,
 9100,
 9144,
 9155,
 9180,
 9183,
 9208,
 9210,
 9238,
 9276,
 9278]

In [350]:
#fill nan (I searcherd on trasnfermakt one-by-one)
#print(df.loc[9278])

In [351]:
#fill nan
df.loc[8704, 'Age']= 23
df.loc[8728, 'Age']= 27
df.loc[8752, 'Age']= 31
df.loc[8772, 'Age']= 28
df.loc[8825, 'Age']= 31
df.loc[8828, 'Age']= 29
df.loc[8838, 'Age']= 23
df.loc[8850, 'Age']= 32
df.loc[8873, 'Age']= 32
df.loc[8879, 'Age']= 22
df.loc[8887, 'Age']= 33
df.loc[8894, 'Age']= 29
df.loc[8898, 'Age']= 31
df.loc[8935, 'Age']= 32
df.loc[8944, 'Age']= 29
df.loc[8955, 'Age']= 30
df.loc[8961, 'Age']= 29
df.loc[9013, 'Age']= 24
df.loc[9049, 'Age']= 24
df.loc[9058, 'Age']= 30
df.loc[9100, 'Age']= 30
df.loc[9144, 'Age']= 37
df.loc[9155, 'Age']= 25
df.loc[9180, 'Age']= 29
df.loc[9183, 'Age']= 31
df.loc[9208, 'Age']= 26
df.loc[9210, 'Age']= 22
df.loc[9238, 'Age']= 34
df.loc[9276, 'Age']= 25
df.loc[9278, 'Age']= 29

In [352]:
#check age
df['Age'].unique()

array([27., 29., 25., 26., 22., 30., 31., 24., 28., 23., 21., 34., 20.,
       35., 19., 36., 33., 32., 18., 17., 37., 38., 39., 40., 43., 15.,
       16., 42.,  0., 41.])

Some players have 0 as age which is clearly a typo

In [353]:
#here again I have to manually check on transfermarkt
df[df['Age']==0].index.tolist()

[3054,
 6005,
 6547,
 6606,
 6614,
 6623,
 6661,
 6674,
 7080,
 7138,
 7362,
 7560,
 7574,
 7721,
 7738,
 7753,
 7754,
 7780,
 7787,
 7793,
 7806,
 7807,
 7812,
 7821,
 7822,
 7824,
 7829,
 7835,
 7837,
 7842,
 7935,
 8247,
 8448,
 8456,
 8471,
 8476,
 8490,
 8494,
 8502,
 8511,
 8512,
 8514,
 8515,
 8518,
 8535,
 8542,
 8543,
 8547,
 8563,
 8570,
 10249,
 10256,
 10327,
 10348]

In [354]:
#fill 0 with the real age
#print(df.loc[10348])
df.loc[3054, 'Age']= 20
df.loc[6005, 'Age']= 28
df.loc[6547, 'Age']= 17
df.loc[6606, 'Age']= 18
df.loc[6614, 'Age']= 19
df.loc[6623, 'Age']= 19
df.loc[6661, 'Age']= 20
df.loc[6674, 'Age']= 19
df.loc[7080, 'Age']= 16
df.loc[7138, 'Age']= 18
df.loc[7362, 'Age']= 18
df.loc[7560, 'Age']= 19
df.loc[7574, 'Age']= 17
df.loc[7721, 'Age']= 19
df.loc[7738, 'Age']= 19
df.loc[7753, 'Age']= 19
df.loc[7754, 'Age']= 17
df.loc[7780, 'Age']= 16
df.loc[7787, 'Age']= 18
df.loc[7793, 'Age']= 20
df.loc[7806, 'Age']= 19
df.loc[7807, 'Age']= 18
df.loc[7812, 'Age']= 18
df.loc[7821, 'Age']= 17
df.loc[7822, 'Age']= 19
df.loc[7824, 'Age']= 18
df.loc[7829, 'Age']= 18
df.loc[7835, 'Age']= 17
df.loc[7837, 'Age']= 17
df.loc[7842, 'Age']= 18
df.loc[7935, 'Age']= 19
df.loc[8247, 'Age']= 19
df.loc[8448, 'Age']= 18
df.loc[8456, 'Age']= 18
df.loc[8471, 'Age']= 24
df.loc[8476, 'Age']= 22
df.loc[8490, 'Age']= 18
df.loc[8494, 'Age']= 19
df.loc[8502, 'Age']= 17
df.loc[8511, 'Age']= 20
df.loc[8512, 'Age']= 19
df.loc[8514, 'Age']= 18
df.loc[8515, 'Age']= 18
df.loc[8518, 'Age']= 16
df.loc[8535, 'Age']= 20
df.loc[8542, 'Age']= 17
df.loc[8543, 'Age']= 18
df.loc[8547, 'Age']= 19
df.loc[8563, 'Age']= 17
df.loc[8570, 'Age']= 18
df.loc[10249, 'Age']= 24
df.loc[10256, 'Age']= 19
df.loc[10327, 'Age']= 18
df.loc[10348, 'Age']= 19

In [355]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10356 entries, 0 to 10355
Data columns (total 50 columns):
Name              10356 non-null object
Team              10356 non-null object
Age               10356 non-null float64
Apps              10356 non-null object
start_bench       8111 non-null object
Mins              10356 non-null int64
Goals             10356 non-null object
Assists           10356 non-null object
Yel               10356 non-null object
Red               10356 non-null object
SpG               10356 non-null object
PS%               10356 non-null object
AerialsWon        10356 non-null object
MotM              10356 non-null object
Rating            10356 non-null float64
League            10356 non-null object
Tackles           10356 non-null object
Inter             10356 non-null object
Fouls             10356 non-null object
Offsides          10356 non-null object
Clear             10356 non-null object
Drb_past          10356 non-null object
Blocks     

In [356]:
#if nan fill start bench with 0
df['start_bench'] = df['start_bench'].fillna(0)

In [357]:
#consistency check of Apps and min. Indeed, Apps might be 0 if they always played but starting from the bench
#replace 0 in Apps columns with the value in start_bench
df['Apps'] = np.where(df['Apps'] == 0, df['start_bench'], df['Apps'])

In [358]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10356 entries, 0 to 10355
Data columns (total 50 columns):
Name              10356 non-null object
Team              10356 non-null object
Age               10356 non-null float64
Apps              10356 non-null object
start_bench       10356 non-null object
Mins              10356 non-null int64
Goals             10356 non-null object
Assists           10356 non-null object
Yel               10356 non-null object
Red               10356 non-null object
SpG               10356 non-null object
PS%               10356 non-null object
AerialsWon        10356 non-null object
MotM              10356 non-null object
Rating            10356 non-null float64
League            10356 non-null object
Tackles           10356 non-null object
Inter             10356 non-null object
Fouls             10356 non-null object
Offsides          10356 non-null object
Clear             10356 non-null object
Drb_past          10356 non-null object
Blocks    

There are some columns where an empty value is '-' by default. Must be replaced with 0.

In [359]:
df.replace({'-': 0}, inplace=True)

In [360]:
df['Age'].unique()

array([27., 29., 25., 26., 22., 30., 31., 24., 28., 23., 21., 34., 20.,
       35., 19., 36., 33., 32., 18., 17., 37., 38., 39., 40., 43., 15.,
       16., 42., 41.])

Convert all the columns that should be numeric

In [361]:
df['Age'] = pd.to_numeric(df['Age'])
df['Apps'] = pd.to_numeric(df['Apps'])
df['start_bench'] = pd.to_numeric(df['start_bench'])
df['Goals'] = pd.to_numeric(df['Goals'])
df['Assists'] = pd.to_numeric(df['Assists'])
df['Yel'] = pd.to_numeric(df['Yel'])
df['Red'] = pd.to_numeric(df['Red'])
df['SpG'] = pd.to_numeric(df['SpG'])
df['PS%'] = pd.to_numeric(df['PS%'])
df['AerialsWon'] = pd.to_numeric(df['AerialsWon'])
df['MotM'] = pd.to_numeric(df['MotM'])
df['Tackles'] = pd.to_numeric(df['Tackles'])
df['Inter'] = pd.to_numeric(df['Inter'])
df['Fouls'] = pd.to_numeric(df['Fouls'])
df['Offsides'] = pd.to_numeric(df['Offsides'])
df['Clear'] = pd.to_numeric(df['Clear'])
df['Drb_past'] = pd.to_numeric(df['Drb_past'])
df['Blocks'] = pd.to_numeric(df['Blocks'])
df['OwnG'] = pd.to_numeric(df['OwnG'])
df['Drb_per_game'] = pd.to_numeric(df['Drb_per_game'])
df['Fouled'] = pd.to_numeric(df['Fouled'])
df['Off'] = pd.to_numeric(df['Off'])
df['Disp'] = pd.to_numeric(df['Disp'])
df['UnsTch'] = pd.to_numeric(df['UnsTch'])
df['Total_shots'] = pd.to_numeric(df['Total_shots'])
df['OutOfBox'] = pd.to_numeric(df['OutOfBox'])
df['SixYardBox'] = pd.to_numeric(df['SixYardBox'])
df['PenaltyArea'] = pd.to_numeric(df['PenaltyArea'])
df['KeyP'] = pd.to_numeric(df['KeyP'])
df['AvgP'] = pd.to_numeric(df['AvgP'])
df['Crosses'] = pd.to_numeric(df['Crosses'])
df['LongB'] = pd.to_numeric(df['LongB'])
df['ThrB'] = pd.to_numeric(df['ThrB'])
df['Age'] = df['Age'].astype(int)#Age is int

In [362]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10356 entries, 0 to 10355
Data columns (total 50 columns):
Name              10356 non-null object
Team              10356 non-null object
Age               10356 non-null int32
Apps              10356 non-null int64
start_bench       10356 non-null int64
Mins              10356 non-null int64
Goals             10356 non-null int64
Assists           10356 non-null int64
Yel               10356 non-null int64
Red               10356 non-null int64
SpG               10356 non-null float64
PS%               10356 non-null float64
AerialsWon        10356 non-null float64
MotM              10356 non-null int64
Rating            10356 non-null float64
League            10356 non-null object
Tackles           10356 non-null float64
Inter             10356 non-null float64
Fouls             10356 non-null float64
Offsides          10356 non-null float64
Clear             10356 non-null float64
Drb_past          10356 non-null float64
Blocks    

In [363]:
#sometimes there are repeated rows because some players were sold the team after few league games or were sent on load to another team.
#to avoid those duplicated rows I will only consider the data from the last team where they played
df = df.sort_values(by='Apps', ascending=False)
df = df.drop_duplicates(subset='Name', keep="first")

In [364]:
df.shape

(9526, 50)

In [365]:
#save the df
df.to_csv('df.csv', index=False)


________________________

<h3><b>Prices dataframe</h3></b>

__________________

In [366]:
prices = pd.read_csv(os.path.join('prices','prices.csv')) 
prices.head(5)

,name,data of birth,nationality,height,foot,joined,contract until,market value
0,EdersonEdersonGoalkeepe,"Aug 17, 1993","['Brazil', 'Portugal']","1,88 m",left,"Jul 1, 2017","Jun 30, 2025",€50.00m
1,Zack Steffen,"Apr 2, 1995",['United States'],"1,91 m",right,"Jul 9, 2019","Jun 30, 2023",€6.00m
2,Rúben Dias,"May 14, 1997",['Portugal'],"1,87 m",right,"Sep 29, 2020","Jun 30, 2026",€75.00m
3,Aymeric Laporte,"May 27, 1994","['Spain', 'France']","1,89 m",left,"Jan 30, 2018","Jun 30, 2025",€45.00m
4,Nathan Aké,"Feb 18, 1995","['Netherlands', ""Cote d'Ivoire""]","1,80 m",left,"Aug 5, 2020","Jun 30, 2025",€32.00m


In [367]:
#rename columns
prices.rename(columns={'data of birth':'date_of_birth', 'contract until':'contract_until', 'market value':'market_value'}, inplace=True)

In prices all the columns type are objects.

In [368]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10192 entries, 0 to 10191
Data columns (total 8 columns):
name              10189 non-null object
date_of_birth     10183 non-null object
nationality       10192 non-null object
height            10192 non-null object
foot              10192 non-null object
joined            10192 non-null object
contract_until    10192 non-null object
market_value      10192 non-null object
dtypes: object(8)
memory usage: 637.1+ KB


In [369]:
#clean and split nationality
prices['nationality'] = prices['nationality'].str.replace('[','')
prices['nationality'] = prices['nationality'].str.replace(']','')
prices[['nationality_1','nationality_2', 'nationality_3']] = prices.nationality.str.split(',',expand=True,)

#drop nationality
prices = prices.drop('nationality', axis = 1)
prices.head(5)

,name,date_of_birth,height,foot,joined,contract_until,market_value,nationality_1,nationality_2,nationality_3
0,EdersonEdersonGoalkeepe,"Aug 17, 1993","1,88 m",left,"Jul 1, 2017","Jun 30, 2025",€50.00m,'Brazil','Portugal',None
1,Zack Steffen,"Apr 2, 1995","1,91 m",right,"Jul 9, 2019","Jun 30, 2023",€6.00m,'United States',None,None
2,Rúben Dias,"May 14, 1997","1,87 m",right,"Sep 29, 2020","Jun 30, 2026",€75.00m,'Portugal',None,None
3,Aymeric Laporte,"May 27, 1994","1,89 m",left,"Jan 30, 2018","Jun 30, 2025",€45.00m,'Spain','France',None
4,Nathan Aké,"Feb 18, 1995","1,80 m",left,"Aug 5, 2020","Jun 30, 2025",€32.00m,'Netherlands',"""Cote d'Ivoire""",None


In [370]:
#fill nan
prices['nationality_2'] = prices['nationality_2'].fillna('None')
prices['nationality_3'] = prices['nationality_3'].fillna('None')

In [371]:
#convert date of birth, joined, contract to datetime
#before I need to clean the column from uncessary characters

prices['date_of_birth'] = prices['date_of_birth'].str.replace(' ','/')
prices['date_of_birth'] = prices['date_of_birth'].str.replace(',','')
prices['date_of_birth'] = pd.to_datetime(prices['date_of_birth'], errors='coerce')

prices['joined'] = prices['joined'].str.replace(' ','/')
prices['joined'] = prices['joined'].str.replace(',','')
prices['joined'] = pd.to_datetime(prices['joined'], errors='coerce')

prices['contract_until'] = prices['contract_until'].str.replace(' ','/')
prices['contract_until'] = prices['contract_until'].str.replace(',','')
prices['contract_until'] = pd.to_datetime(prices['contract_until'], errors='coerce')

prices.head()

,name,date_of_birth,height,foot,joined,contract_until,market_value,nationality_1,nationality_2,nationality_3
0,EdersonEdersonGoalkeepe,1993-08-17,"1,88 m",left,2017-07-01,2025-06-30,€50.00m,'Brazil','Portugal',None
1,Zack Steffen,1995-04-02,"1,91 m",right,2019-07-09,2023-06-30,€6.00m,'United States',None,None
2,Rúben Dias,1997-05-14,"1,87 m",right,2020-09-29,2026-06-30,€75.00m,'Portugal',None,None
3,Aymeric Laporte,1994-05-27,"1,89 m",left,2018-01-30,2025-06-30,€45.00m,'Spain','France',None
4,Nathan Aké,1995-02-18,"1,80 m",left,2020-08-05,2025-06-30,€32.00m,'Netherlands',"""Cote d'Ivoire""",None


Date of birt missing values will be replaced with an arbitrary date corresponding to the mean from df['age']

In [372]:
prices.loc[prices.date_of_birth.isnull()].shape

(10, 10)

In [373]:
df['Age'].mean()#I will use 26 as mean for imputation corresponding to the arbitrary 1995-01-01

25.968192315767375

In [374]:
#date of birth imputation
prices['date_of_birth'] = prices['date_of_birth'].fillna('1995-01-01')
prices['date_of_birth'] = pd.to_datetime(prices['date_of_birth'])

Some players contract lenght is unknown (no joined or contract_until values). Those missing values will be replaced with the median 'joined' and 'contract_until' date.

In [375]:
print(prices.loc[prices.joined.isnull()].shape)
print(prices.loc[prices.contract_until.isnull()].shape)

(298, 10)
(721, 10)


In [376]:
prices['joined'].astype('datetime64[ns]').quantile(0.5, interpolation="midpoint")

Timestamp('2019-12-19 00:00:00')

In [377]:
prices['contract_until'].astype('datetime64[ns]').quantile(0.5, interpolation="midpoint")

Timestamp('2022-06-30 00:00:00')

In [378]:
#imputation
prices['joined'] = prices['joined'].fillna('2019-12-19')
prices['joined'] = pd.to_datetime(prices['joined'])

prices['contract_until'] = prices['contract_until'].fillna('2022-06-30')
prices['contract_until'] = pd.to_datetime(prices['contract_until'])

There are some player for which the date dominant foot is unknown. In the general population 81% of people are favoring the right foot (source wikipedia https://en.wikipedia.org/wiki/Laterality) while in this sample there are 6566 right foot and 2361 left foot players resulting in ~73.5 right foot players (both foot players are excluded from the calculation because full ambidextrous people are extremely rare and most of those actually show a preference and best skilld with one of the two foot). Here I will make an imputation randomly assigning left or right respectively to 27% and 73% of the players with no foot reported.

In [379]:
pd.DataFrame(prices.foot.value_counts())

,foot
right,6566
left,2361
-,903
both,362


In [380]:
#replace '-' with nan
prices['foot'] = prices['foot'].replace('-', np.nan)

#replace with random right/left [80%, 20%]
import random
np.random.seed(seed=0)
prices['foot'] = prices['foot'].fillna(pd.Series(np.random.choice(['right', 'left'], 
                                                                  p=[0.8, 0.2], size=len(prices))))

Correct the 'height' variable format

In [381]:
prices['height'] = prices['height'].str.replace('m','')
prices['height'] = prices['height'].str.replace(',','.')
prices['height'] = prices['height'].str.replace(' ','')
prices['height'] = prices['height'].str.replace(';','.')
prices['height'] = prices['height'].replace('', np.nan)

In [382]:
#replace nan with the mean 
prices['height'] = prices['height'].astype(float)#convert to float
prices['height'].mean()

1.8201046322633077

In [383]:
prices['height'] = prices['height'].fillna(1.8)

The prices from the scraping contain some characters (m = milion, th = thousands) + the currency

In [384]:
prices['market_value'] = prices['market_value'].str.replace('€','')
prices['market_value'] = prices['market_value'].str.replace('.',',')
prices['market_value'] = prices['market_value'].str.replace('m','0000.000')
prices['market_value'] = prices['market_value'].str.replace('Th.','000.000')
prices['market_value'] = prices['market_value'].str.replace(u'\xa0\xa0',u'')#remove the unicode
prices['market_value'] = prices['market_value'].str.replace(u'\xa0',u'')#remove the unicode
prices['market_value'] = prices['market_value'].replace(u'-\xa0', np.nan)# turn empty to nan

In [385]:
prices['market_value'] = prices['market_value'].str.replace(',','')

There are 596 players with no market_value. Those are probably players with a low reputation and the price should be reasonably low. I will fill those values with with the lowest market_value in the dataframe.

In [386]:
prices.market_value.replace({'-': np.nan}, inplace=True)
prices['market_value'] = prices['market_value'].astype(float)#convert to float
print('lowest price:', prices.market_value.min())

lowest price: 25000.0


In [387]:
#fill nan
prices['market_value'] = prices['market_value'].fillna(25000.0)

In [388]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10192 entries, 0 to 10191
Data columns (total 10 columns):
name              10189 non-null object
date_of_birth     10192 non-null datetime64[ns]
height            10192 non-null float64
foot              10192 non-null object
joined            10192 non-null datetime64[ns]
contract_until    10192 non-null datetime64[ns]
market_value      10192 non-null float64
nationality_1     10192 non-null object
nationality_2     10192 non-null object
nationality_3     10192 non-null object
dtypes: datetime64[ns](3), float64(2), object(5)
memory usage: 796.3+ KB


In the name column there is unwanted text due to the scraping

In [389]:
#prices.name.unique().tolist() ##with this line I briefly checked what are the possible typos (most of them...)
prices['name'] = prices['name'].str.replace(u'\xa0','')

prices['name'] = prices['name'].str.replace('Goalkeepe','')
prices['name'] = prices['name'].str.replace(u'\xa0Goalkeepe','')

prices['name'] = prices['name'].str.replace('Left-Bac','')
prices['name'] = prices['name'].str.replace(u'\xa0Left-Bac','')

prices['name'] = prices['name'].str.replace('Right-Bac','')
prices['name'] = prices['name'].str.replace(u'\xa0Right-Bac','')

prices['name'] = prices['name'].str.replace('Centre-Bac','')
prices['name'] = prices['name'].str.replace(u'\xa0Centre-Bac','')

prices['name'] = prices['name'].str.replace('Defensive Midfiel','')
prices['name'] = prices['name'].str.replace(u'\xa0Defensive Midfiel','')

prices['name'] = prices['name'].str.replace('Left Winge','')
prices['name'] = prices['name'].str.replace(u'\xa0Left Winge','')

prices['name'] = prices['name'].str.replace('Right Winge','')
prices['name'] = prices['name'].str.replace(u'\xa0Right Winge','')

prices['name'] = prices['name'].str.replace('Central Midfiel','')
prices['name'] = prices['name'].str.replace(u'\xa0Central Midfiel','')

prices['name'] = prices['name'].str.replace('Attacking Midfiel','')
prices['name'] = prices['name'].str.replace(u'\xa0Attacking Midfiel','')

prices['name'] = prices['name'].str.replace('Second Strike','')
prices['name'] = prices['name'].str.replace(u'\xa0Second Strike','')

prices['name'] = prices['name'].str.replace('Centre-Forwar','')
prices['name'] = prices['name'].str.replace(u'\xa0Centre-Forwar','')

A further issue is that some names are repeated twice in the same cell

Drop rows where the name is nan

In [390]:
prices.dropna(subset = ['name'], inplace=True)

In [391]:
#save the df
prices.to_csv('prices.csv', index=False)

In [392]:
prices.sort_values(by='name', ascending=True).head(10)

,name,date_of_birth,height,foot,joined,contract_until,market_value,nationality_1,nationality_2,nationality_3
7932,AJ Leitch-Smith,1990-03-06,1.73,right,2018-08-08,2021-06-30,300000.0,'England',None,None
6702,Aapo Halme,1998-05-22,1.96,right,2019-07-03,2022-06-30,800000.0,'Finland',None,None
5886,Aaron Appindangoyé,1992-02-20,1.84,right,2019-07-01,2023-06-30,1500000.0,'Gabon',None,None
5815,Aaron Boupendza,1996-08-07,1.80,left,2020-08-05,2022-06-30,8500000.0,'Gabon',None,None
6972,Aaron Chapman,1990-05-29,2.03,right,2020-10-07,2021-05-31,250000.0,'England',None,None
8280,Aaron Collins,1997-05-27,1.78,right,2019-07-01,2021-06-30,75000.0,'Wales',None,None
336,Aaron Connolly,2000-01-28,1.75,right,2019-07-01,2024-06-30,7000000.0,'Ireland',None,None
350,Aaron Cresswell,1989-12-15,1.70,left,2014-07-03,2023-06-30,5000000.0,'England',None,None
7208,Aaron Drinan,1998-05-06,1.82,left,2018-01-04,2022-06-30,100000.0,'Ireland',None,None
8290,Aaron Hayden,1997-01-16,1.85,right,2019-07-29,2022-06-30,25000.0,'England',None,None


In [393]:
df.sort_values(by='Name', ascending=True).head(10)

,Name,Team,Age,Apps,start_bench,Mins,Goals,Assists,Yel,Red,...,is_pos_l,is_pos_r,secondary_role,is_pos_sec_c,is_pos_sec_l,is_pos_sec_r,third_role,is_pos_third_c,is_pos_third_l,is_pos_third_r
8311,A-Jay Leitch-Smith,Morecambe,31,1,4,113,1,0,1,0,...,0,0,None,0,0,0,None,0,0,0
5208,AJ Delagarza JR,New England,33,0,1,36,0,0,0,0,...,1,1,None,0,0,0,None,0,0,0
6465,Aapo Halme,Barnsley,23,3,15,654,0,1,4,0,...,0,0,DMC,1,0,0,None,0,0,0
5573,Aaron Appindangoyé,Sivasspor,29,13,0,1148,0,0,3,0,...,0,0,None,0,0,0,None,0,0,0
3810,Aaron Bastiaans,VVV-Venlo,19,0,1,22,0,0,0,0,...,0,0,None,0,0,0,None,0,0,0
6933,Aaron Chapman,Motherwell,31,4,2,448,0,0,0,0,...,0,0,None,0,0,0,None,0,0,0
7997,Aaron Collins,Forest Green,24,39,8,3542,11,7,6,0,...,0,0,FW,1,0,0,None,0,0,0
421,Aaron Connolly,Brighton,21,9,8,793,2,1,0,0,...,0,0,None,0,0,0,None,0,0,0
140,Aaron Cresswell,West Ham,31,36,0,3172,0,8,3,0,...,1,0,M,0,1,0,None,0,0,0
7707,Aaron Drinan,Ipswich,23,6,16,724,1,0,2,0,...,0,0,None,0,0,0,None,0,0,0


_________________________

<h3><b>Merge stats and prices dataframes</h3></b>

__________________________

Reload and merge the 2 data frames

In [424]:
df = pd.read_csv(os.path.join('df.csv')) 
prices = pd.read_csv(os.path.join('prices.csv')) 
print('shape df:', df.shape)
print('shape prices:', prices.shape)

shape df: (9526, 50)
shape prices: (10189, 10)


There are 2 problems. 
1- the dataframes don't have the same length (this is likely due to the fact that transfermarkt and whoscored may diverge in considering the youngsters as squad players)
2- the player's name might differ across the 2 dataframe (players have multiple names, there are some errors in the transfermarkt names...)

In [425]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10189 entries, 0 to 10188
Data columns (total 10 columns):
name              10189 non-null object
date_of_birth     10189 non-null object
height            10189 non-null float64
foot              10189 non-null object
joined            10189 non-null object
contract_until    10189 non-null object
market_value      10189 non-null float64
nationality_1     10189 non-null object
nationality_2     10189 non-null object
nationality_3     10189 non-null object
dtypes: float64(2), object(8)
memory usage: 796.1+ KB


In [426]:
prices['market_value'] = prices['market_value'].astype(float)

In [427]:
prices = prices.sort_values(by='market_value', ascending=False)
prices = prices.drop_duplicates(subset='name', keep="first")

Unfortunately I didn't find any solutions about how to ensure that the players are exactly the same in the two dataframe. I attempted different solutions (remove all player names that are not repeated or are using algorithms with fuzzywuzzy or dedupe libraries) but the amount of rules to be specified and syntax differences (e.g. russian or turkish players don't have a unique way to be translated) made me loss too much data. Accordigly I made a visual inspection on excel and removed as above the players who not present in both dataframes.

In [428]:
df = df[~df.Name.str.contains('AJ Delagarza JR')]
df = df[~df.Name.str.contains('Aaron Bastiaans')]
df = df[~df.Name.str.contains('Driscoll')]
df = df[~df.Name.str.contains('Aaron Rowe')]
df = df[~df.Name.str.contains('Aaron Pressley')]
df = df[~df.Name.str.contains('Aatif Chahechouhe')]
df = df[~df.Name.str.contains('Abdoulaye Sylla')]
df = df[~df.Name.str.contains('Abdul Ajagun')]
df = df[~df.Name.str.contains('Aaron Martin')]
prices = prices[~prices.name.str.contains('Aaron Martin')]
df = df[~df.Name.str.contains('Aaron Martin')]
prices = prices[~prices.name.str.contains('Aaron Martin')]
prices = prices[~prices.name.str.contains('Abdoulaye Diallo')]
prices = prices[~prices.name.str.contains('Abraham Rodriguez')]
df = df[~df.Name.str.contains('Abdoulaye Sidibé')]
df = df[~df.Name.str.contains('Abdul Mohammed Kadiri')]
df = df[~df.Name.str.contains('Abdul Wahab Ibrahim')]
df = df[~df.Name.str.contains('Abdurrahim Dursun')]
df = df[~df.Name.str.contains('Abner Felipe')]
df = df[~df.Name.str.contains('Abner Vinicius')]
df = df[~df.Name.str.contains('Abo Eisa')]
df = df[~df.Name.str.contains('Abou Ouattara')]
df = df[~df.Name.str.contains('Adem Dogan')]
df = df[~df.Name.str.contains('Adler Nascimento')]
df = df[~df.Name.str.contains('Admiral Muskwe')]
df = df[~df.Name.str.contains('Adrian Schlagbauer')]
df = df[~df.Name.str.contains('Adrian Szoke')]
df = df[~df.Name.str.contains('Adrien Louveau')]
df = df[~df.Name.str.contains('Adrián Calello')]
df = df[~df.Name.str.contains('Adrián Cubas')]
df = df[~df.Name.str.contains('Agustin Hausch')]
df = df[~df.Name.str.contains('Agustín Aleo')]
df = df[~df.Name.str.contains('Agustín Allione')]
df = df[~df.Name.str.contains('Agustín Bolivar')]
df = df[~df.Name.str.contains('Agustín Manzur')]
df = df[~df.Name.str.contains('Ahmed Hegazi')]
df = df[~df.Name.str.contains('Ahmed Isaiah')]
df = df[~df.Name.str.contains('Ahmet Yilmaz Calik')]
df = df[~df.Name.str.contains('Ajani Burchall')]
df = df[~df.Name.str.contains('Ajibola Alese')]
df = df[~df.Name.str.contains('Akhmed Alibekov')]
df = df[~df.Name.str.contains('Akmal Bakhtiyarov')]
df = df[~df.Name.str.contains('Adrián Spörle')]
df = df[~df.Name.str.contains('Alberto Cifuentes')]
df = df[~df.Name.str.contains('Aleks Matsukatov')]
df = df[~df.Name.str.contains('Aleksandr Chernikov')]
df = df[~df.Name.str.contains('Aleksandr Gapechkin')]
df = df[~df.Name.str.contains('Aleksandre Karapetian')]
df = df[~df.Name.str.contains('Aleksey Rybin')]
df = df[~df.Name.str.contains('Alessandro Di Pardo')]
df = df[~df.Name.str.contains('Adam Lewis')]
df = df[~df.Name.str.contains('Adam Roscrow')]
df = df[~df.Name.str.contains('Adam Montgomery')]
df = df[~df.Name.str.contains('Adam Zrelák')]
df = df[~df.Name.str.contains('Adama Noss Traoré')]
df = df[~df.Name.str.contains('Adama Diakhaby')]
df = df[~df.Name.str.contains('Adan George')]
df = df[~df.Name.str.contains('Alex Rodriguez')]
df = df[~df.Name.str.contains('Alex Scott')]
df = df[~df.Name.str.contains('Alexander Barboza')]
df = df[~df.Name.str.contains('Alexander Bernabei')]
df = df[~df.Name.str.contains('Alexander Ferguson')]
df = df[~df.Name.str.contains('Alexander Jeremejeff')]
df = df[~df.Name.str.contains('Alexander Jones')]
df = df[~df.Name.str.contains('Alexander Lomovitskiy')]
df = df[~df.Name.str.contains('Alexander Maes')]
df = df[~df.Name.str.contains('Alexander Selikhov')]
df = df[~df.Name.str.contains('Alexander Sobolev')]
df = df[~df.Name.str.contains('Alexander Tettey')]
df = df[~df.Name.str.contains('Alexandre Jankewitz')]
df = df[~df.Name.str.contains('Alexandros Gogic')]
df = df[~df.Name.str.contains('Alexei Coselev')]
df = df[~df.Name.str.contains('Alexey Nikitin')]
df = df[~df.Name.str.contains('Alexis Arias')]
df = df[~df.Name.str.contains('Alexis Flips')]
df = df[~df.Name.str.contains('Alexis Gamboa')]
df = df[~df.Name.str.contains('Alexis de Sart')]
df = df[~df.Name.str.contains('Alfonso Parot')]
df = df[~df.Name.str.contains('Alimami Gory')]
df = df[~df.Name.str.contains('Alistair McCann')]
df = df[~df.Name.str.contains('Alistair Smith')]
df = df[~df.Name.str.contains('Alister Crawford')]
df = df[~df.Name.str.contains('Allan Tchaptchet')]
df = df[~df.Name.str.contains('Amari Miller')]
df = df[~df.Name.str.contains('Anas Ouahim')]
df = df[~df.Name.str.contains('Anatoliy Nemchenko')]
df = df[~df.Name.str.contains('Anco Jansen')]
df = df[~df.Name.str.contains('Andre Becker')]
df = df[~df.Name.str.contains('Andre Biyogo Poko')]
df = df[~df.Name.str.contains('Andre Hoffmann')]
df = df[~df.Name.str.contains('Andrea Adorante')]
df = df[~df.Name.str.contains('Andrea Danzi')]
df = df[~df.Name.str.contains('Andrei Ratiu')]
df = df[~df.Name.str.contains('Andrew Cannon')]
df = df[~df.Name.str.contains('Andrew Considine')]
df = df[~df.Name.str.contains('Andrew Dallas')]
df = df[~df.Name.str.contains('Andrey Bokovoy')]
df = df[~df.Name.str.contains('Andrija Vukcevic')]
df = df[~df.Name.str.contains('André Paulo')]
df = df[~df.Name.str.contains('Andrés Vombergar')]
df = df[~df.Name.str.contains('Andy Diouf')]
df = df[~df.Name.str.contains('Angelo Gabrielli')]
df = df[~df.Name.str.contains('Ansou Sow')]
df = df[~df.Name.str.contains('Ante Coric')]
df = df[~df.Name.str.contains('Anthony Descotte')]
df = df[~df.Name.str.contains('Anthony Elanga')]
df = df[~df.Name.str.contains('Anthony Lozano')]
df = df[~df.Name.str.contains('Anton Kilin')]
df = df[~df.Name.str.contains('Antonio Blanco')]
df = df[~df.Name.str.contains('Antonio Palumbo')]
df = df[~df.Name.str.contains('Antony Silva')]
df = df[~df.Name.str.contains('Antwoine Hackford')]
df = df[~df.Name.str.contains('Arcanjo')]
df = df[~df.Name.str.contains('Ari Freyr Skulason')]
df = df[~df.Name.str.contains('Aritz Arambarri')]
df = df[~df.Name.str.contains('Armando Sadiku')]
df = df[~df.Name.str.contains('Armen Manucharyan')]
df = df[~df.Name.str.contains('Armstrong Okoflex')]
df = df[~df.Name.str.contains('Artem Shabolin')]
df = df[~df.Name.str.contains('Arthur Read')]
df = df[~df.Name.str.contains('Artur Sokhiev')]
df = df[~df.Name.str.contains('Arturo Mina')]
df = df[~df.Name.str.contains('Austin Samuels')]
df = df[~df.Name.str.contains('Aymen Souda')]
df = df[~df.Name.str.contains('Aziz Bouhaddouz')]
df = df[~df.Name.str.contains('Azubuike Okechukwu')]
df = df[~df.Name.str.contains('Aïmen Moueffek')]
df = df[~df.Name.str.contains('Badou Ndiaye')]
df = df[~df.Name.str.contains('Baptiste Gabard')]
df = df[~df.Name.str.contains('Barry Baggley')]
df = df[~df.Name.str.contains('Bartug Elmaz')]
df = df[~df.Name.str.contains('Bassala Sambou')]
df = df[~df.Name.str.contains('Bastos')]
df = df[~df.Name.str.contains('Bautista Merlini')]
df = df[~df.Name.str.contains('Ben Richards-Everton')]
df = df[~df.Name.str.contains('Ben Scholte')]
df = df[~df.Name.str.contains('Ben Sheaf')]
df = df[~df.Name.str.contains('Beni Redzic')]
df = df[~df.Name.str.contains('Bilel Mohsni')]
df = df[~df.Name.str.contains('Bjorn Meijer')]
df = df[~df.Name.str.contains('Bobby Grant')]
df = df[~df.Name.str.contains('Bobby Thomas')]
df = df[~df.Name.str.contains('Bora Kadioglu')]
df = df[~df.Name.str.contains('Boris Tashchy')]
df = df[~df.Name.str.contains('Bradley Guzan')]
df = df[~df.Name.str.contains('Bradley Holmes')]
df = df[~df.Name.str.contains('Brahim Darri')]
df = df[~df.Name.str.contains('Braian Ojeda')]
df = df[~df.Name.str.contains('Brandon Pierrick')]
df = df[~df.Name.str.contains('Branislav Ninaj')]
df = df[~df.Name.str.contains('Brecht Dejaegere')]
df = df[~df.Name.str.contains('Brian Fernández')]
df = df[~df.Name.str.contains('Bruno Xadas')]
df = df[~df.Name.str.contains('Augustus Kargbo')]
df = df[~df.Name.str.contains('Aurelien Scheidler')]
df = df[~df.Name.str.contains('Berat Kalkan')]
df = df[~df.Name.str.contains('Bradley Collins')]
df = df[~df.Name.str.contains('Brooklyn Ilunga')]
df = df[~df.Name.str.contains('Burak Süleyman')]
df = df[~df.Name.str.contains('Caleb Watts')]
df = df[~df.Name.str.contains('Callum Connolly')]
df = df[~df.Name.str.contains('Callum Morton')]
df = df[~df.Name.str.contains('Callum Slattery')]
df = df[~df.Name.str.contains('Callum Wright')]
df = df[~df.Name.str.contains('Calvin Ramsey')]
df = df[~df.Name.str.contains('Cameron Cresswell')]
df = df[~df.Name.str.contains('Carl Tremarco')]
df = df[~df.Name.str.contains('Carlos Auzqui')]
df = df[~df.Name.str.contains('Carlos Embalo')]
df = df[~df.Name.str.contains('Cedric Omoigui')]
df = df[~df.Name.str.contains('Carney Chukwuemeka')]
df = df[~df.Name.str.contains('Cebrail Karayel')]
df = df[~df.Name.str.contains('Chaka Traorè')]
df = df[~df.Name.str.contains('Charles Costes')]
df = df[~df.Name.str.contains('Charlie Caton')]
df = df[~df.Name.str.contains('Chimuanya Ugochukwu')]
df = df[~df.Name.str.contains('Chris Brunt')]
df = df[~df.Name.str.contains('Christian Almeida')]
df = df[~df.Name.str.contains('Christian Chimino')]
df = df[~df.Name.str.contains('Christian Cueva')]
df = df[~df.Name.str.contains('Christian Maggio')]
df = df[~df.Name.str.contains('Christian Norton')]
df = df[~df.Name.str.contains('Christopher Kane')]
df = df[~df.Name.str.contains('Ciaran McKenna')]
df = df[~df.Name.str.contains('Cifu')]
df = df[~df.Name.str.contains('Claudio Spinelli')]
df = df[~df.Name.str.contains('Claudio Villagra')]
df = df[~df.Name.str.contains('Colin Doyle')]
df = df[~df.Name.str.contains('Connor Malley')]
df = df[~df.Name.str.contains('Cote')]
df = df[~df.Name.str.contains('Cristian Battocchio')]
df = df[~df.Name.str.contains('Cristian Díaz')]
df = df[~df.Name.str.contains('Cristian Tassano')]
df = df[~df.Name.str.contains('Dan McNamara')]
df = df[~df.Name.str.contains('Danila Emelyanov')]
df = df[~df.Name.str.contains('Danny Ballard')]
df = df[~df.Name.str.contains('Danny Guthrie')]
df = df[~df.Name.str.contains('Danny Preston')]
df = df[~df.Name.str.contains('Cristian Chávez')]
df = df[~df.Name.str.contains('Batallini')]
df = df[~df.Name.str.contains('Dane Scarlett')]
df = df[~df.Name.str.contains('Daniel Arzani')]
df = df[~df.Name.str.contains('Daniel Bramall')]
df = df[~df.Name.str.contains('Daniel Butterworth')]
df = df[~df.Name.str.contains('Daniel Cárdenas')]
df = df[~df.Name.str.contains('Daniel Csoka')]
df = df[~df.Name.str.contains('Daniel Fox')]
df = df[~df.Name.str.contains('Daniel Gazdag')]
df = df[~df.Name.str.contains('Daniel Neil')]
df = df[~df.Name.str.contains('Daniel Osvaldo')]
df = df[~df.Name.str.contains('Daniel Plomer')]
df = df[~df.Name.str.contains('Daniel Phillips')]
df = df[~df.Name.str.contains('Dante Rigo')]
df = df[~df.Name.str.contains('Dany Cure')]
df = df[~df.Name.str.contains('Dardo Miloc')]
df = df[~df.Name.str.contains('Dario Sarmiento')]
df = df[~df.Name.str.contains('Juan Brunet')]
df = df[~df.Name.str.contains('Dudu Santos')]
df = df[~df.Name.str.contains('ÃÃ±igo Lekue')]
df = df[~df.Name.str.contains('ÃÃ±igo Martinez')]
df = df[~df.Name.str.contains('Adam Phillips')]
df = df[~df.Name.str.contains('Adam Randell')]
df = df[~df.Name.str.contains('Ahmet Calik')]
df = df[~df.Name.str.contains('Ahmet Canbaz')]
df = df[~df.Name.str.contains('Alan Carius')]
df = df[~df.Name.str.contains('Alejandro Cabrera')]
df = df[~df.Name.str.contains('Alejandro Cantero')]
df = df[~df.Name.str.contains('Aleksandr Golovnya')]
df = df[~df.Name.str.contains('Aleksandr Lomovitskiy')]
df = df[~df.Name.str.contains('Aleksandr Smirnov')]
df = df[~df.Name.str.contains('Aleksandr Sobolev')]
df = df[~df.Name.str.contains('Ãlex Blesa')]
df = df[~df.Name.str.contains('Alfie McCalmont')]
df = df[~df.Name.str.contains('Alfie Whiteman')]
df = df[~df.Name.str.contains('Alireza Beiranvand')]
df = df[~df.Name.str.contains('Alireza Jahanbakhsh')]
df = df[~df.Name.str.contains('Allan-Romeo Nyom')]
df = df[~df.Name.str.contains('Alonzo Engwanda')]
df = df[~df.Name.str.contains('Ãlvaro Bastida')]
df = df[~df.Name.str.contains('Ãlvaro Bravo')]
df = df[~df.Name.str.contains('Ãlvaro Gimenez')]
df = df[~df.Name.str.contains('Ãlvaro Quezada')]
df = df[~df.Name.str.contains('Ãlvaro Tejero')]
df = df[~df.Name.str.contains('Alvas Powell')]
df = df[~df.Name.str.contains('Alvi Adilkhanov')]
df = df[~df.Name.str.contains('Amir Abrashi')]
df = df[~df.Name.str.contains('Anders Trondsen')]
df = df[~df.Name.str.contains('Ãngelo Martino')]
df = df[~df.Name.str.contains('Ãngelo Preciado')]
df = df[~df.Name.str.contains('Ãngelo Sagal')]
df = df[~df.Name.str.contains('Anthony Ujah')]
df = df[~df.Name.str.contains('Anthony Uzodimma')]
df = df[~df.Name.str.contains('Anton Mitryushkin')]
df = df[~df.Name.str.contains('Anton Piskunov')]
df = df[~df.Name.str.contains('Antonio Raimondo')]
df = df[~df.Name.str.contains('Antonio Rosati')]
df = df[~df.Name.str.contains('Archie Meekison')]
df = df[~df.Name.str.contains('Arda Kizildag')]
df = df[~df.Name.str.contains('Aristote Nsiala')]
df = df[~df.Name.str.contains('Aristoteles Romero')]
df = df[~df.Name.str.contains('Ãsak Thorvaldsson')]
df = df[~df.Name.str.contains('Ayaz Guliev')]
df = df[~df.Name.str.contains('Aykut Ozer')]
df = df[~df.Name.str.contains('Batuhan Ãœnsal')]
df = df[~df.Name.str.contains('Batuhan Unsal')]
df = df[~df.Name.str.contains('Beckham Sunderland')]
df = df[~df.Name.str.contains('Beka Mikeltadze')]
df = df[~df.Name.str.contains('Ben Jackson')]
df = df[~df.Name.str.contains('Beni Makouana')]
df = df[~df.Name.str.contains('Berk Cetin')]
df = df[~df.Name.str.contains('Berk Yildiz')]
df = df[~df.Name.str.contains('Bernardo ADDDDDDDDDDD')]
df = df[~df.Name.str.contains('Braian Aguirre')]
df = df[~df.Name.str.contains('Braian Ãlvarez')]
df = df[~df.Name.str.contains('Braian Cufre')]
df = df[~df.Name.str.contains('Braian Sanchez')]
df = df[~df.Name.str.contains('BrayanBrayan')]
df = df[~df.Name.str.contains('Braydon Manu')]
df = df[~df.Name.str.contains('Bret Halsey')]
df = df[~df.Name.str.contains('Brett McGavin')]
df = df[~df.Name.str.contains('Brian Idowu')]
df = df[~df.Name.str.contains('Brian Lluy')]
df = df[~df.Name.str.contains('Brian Negro')]
df = df[~df.Name.str.contains('Brian Nievas')]
df = df[~df.Name.str.contains('BruninhoBruninho')]
df = df[~df.Name.str.contains('Bruno Andrade')]
df = df[~df.Name.str.contains('Bruno MendezBruno Mendez')]
df = df[~df.Name.str.contains('Bruno Miguel Rodrigues')]
df = df[~df.Name.str.contains('Bryce Hosannah')]
df = df[~df.Name.str.contains('Bryce Washington')]
df = df[~df.Name.str.contains('BuenoBueno')]
df = df[~df.Name.str.contains('Buiu')]
df = df[~df.Name.str.contains('CaduCadu*')]
df = df[~df.Name.str.contains('CafuCafu')]
df = df[~df.Name.str.contains('Cain Fara')]
df = df[~df.Name.str.contains('Callum Jones')]
df = df[~df.Name.str.contains('Callum King-Harmes')]
df = df[~df.Name.str.contains('Camacho')]
df = df[~df.Name.str.contains('Cameron Antwi')]
df = df[~df.Name.str.contains('Cammy Ballantyne')]
df = df[~df.Name.str.contains('Cammy Smith')]
df = df[~df.Name.str.contains('Cenk Sahin')]
df = df[~df.Name.str.contains('Cesar IbaÃ±ez')]
df = df[~df.Name.str.contains('Charalampos Makridis')]
df = df[~df.Name.str.contains('Charlie Owens')]
df = df[~df.Name.str.contains('Charly Jan')]
df = df[~df.Name.str.contains('Chase Gasper')]
df = df[~df.Name.str.contains('Christopher Garcia')]
df = df[~df.Name.str.contains('Christopher Gloster')]
df = df[~df.Name.str.contains('Clint Irwin')]
df = df[~df.Name.str.contains('Clint Leemans')]
df = df[~df.Name.str.contains('Connor Shanks')]
df = df[~df.Name.str.contains('Connor Taylor')]
df = df[~df.Name.str.contains('Couhaib Driouech')]
df = df[~df.Name.str.contains('Courtney Baker-Richardson')]
df = df[~df.Name.str.contains('Damian Perez')]
df = df[~df.Name.str.contains('Damian Schmidt')]
df = df[~df.Name.str.contains('Daniel Kemp')]
df = df[~df.Name.str.contains('Daniel Leadbitter')]
df = df[~df.Name.str.contains('Daniel Vega')]
df = df[~df.Name.str.contains('Daniel Vukovic')]
df = df[~df.Name.str.contains('Danil Prucev')]
df = df[~df.Name.str.contains('Danil Prutsev')]
df = df[~df.Name.str.contains('Danilo Fernandes')]
df = df[~df.Name.str.contains('Danilo OrtizDavid Henen')]
df = df[~df.Name.str.contains('David Jensen')]
df = df[~df.Name.str.contains('David Junior Hoilett')]
df = df[~df.Name.str.contains('David Karaev')]
df = df[~df.Name.str.contains('David Kong')]
df = df[~df.Name.str.contains('David Lennart Philipp')]
df = df[~df.Name.str.contains('David Loera')]
df = df[~df.Name.str.contains('David Martin')]
df = df[~df.Name.str.contains('David Pavelka')]
df = df[~df.Name.str.contains('David Philipp')]
df = df[~df.Name.str.contains('David Sualehe')]
df = df[~df.Name.str.contains('David Tavares')]
df = df[~df.Name.str.contains('Davide Faraoni')]
df = df[~df.Name.str.contains('Davide Ghislandi')]
df = df[~df.Name.str.contains('Davide Masella')]
df = df[~df.Name.str.contains('Dayro Moreno')]
df = df[~df.Name.str.contains('Declan Drysdale')]
df = df[~df.Name.str.contains('Deian Veron')]
df = df[~df.Name.str.contains('Dejan Galjen')]
df = df[~df.Name.str.contains('Dejan Kerkez')]
df = df[~df.Name.str.contains('Deji Sotona')]
df = df[~df.Name.str.contains('Delano Vianello')]
df = df[~df.Name.str.contains('Demba Ba')]
df = df[~df.Name.str.contains('Denis Adamov')]
df = df[~df.Name.str.contains('Denis Franchi')]
df = df[~df.Name.str.contains('Denis Petric')]
df = df[~df.Name.str.contains('Dennis Adeniran')]
df = df[~df.Name.str.contains('Dennis Gorka')]
df = df[~df.Name.str.contains('Dennis Jastrzembski')]
df = df[~df.Name.str.contains('Dennis Politic')]
df = df[~df.Name.str.contains('Denys Bain')]
df = df[~df.Name.str.contains('Denzel Jubitana')]
df = df[~df.Name.str.contains('Derek Dodson')]
df = df[~df.Name.str.contains('DerickDerick')]
df = df[~df.Name.str.contains('Derrick Tshimanga')]
df = df[~df.Name.str.contains('Desevio Payne')]
df = df[~df.Name.str.contains('Destiny Udogie')]
df = df[~df.Name.str.contains('Dialy Ndiaye')]
df = df[~df.Name.str.contains('Diant Ramaj')]
df = df[~df.Name.str.contains('DidiDidi')]
df = df[~df.Name.str.contains('Diego Braghieri')]
df = df[~df.Name.str.contains('Diego Calcaterra')]
df = df[~df.Name.str.contains('Diego Churin')]
df = df[~df.Name.str.contains('Diego FerreiraDiego Ferreira')]
df = df[~df.Name.str.contains('Diego Lopes')]
df = df[~df.Name.str.contains('Diego Mendoza')]
df = df[~df.Name.str.contains('Diego Novaretti')]
df = df[~df.Name.str.contains('Diogo Teixeira')]
df = df[~df.Name.str.contains('Dion McGhee')]
df = df[~df.Name.str.contains('Dion Moise Sahi')]
df = df[~df.Name.str.contains('Dion Sanderson')]
df = df[~df.Name.str.contains('Dior Angus')]
df = df[~df.Name.str.contains('Dixon Renteria')]
df = df[~df.Name.str.contains('Donis Avdijaj')]
df = df[~df.Name.str.contains('Douglas Friedrich')]
df = df[~df.Name.str.contains('Duncan Turnbull')]
df = df[~df.Name.str.contains('Duncan Turnbull')]
df = df[~df.Name.str.contains('Dusan Jovancic')]
df = df[~df.Name.str.contains('Dyego Sousa')]
df = df[~df.Name.str.contains('Dylan Asonganyi')]
df = df[~df.Name.str.contains('Dylan Borrero')]
df = df[~df.Name.str.contains('Dylan Castanheira')]
df = df[~df.Name.str.contains('Dylan Levitt')]
df = df[~df.Name.str.contains('Dylan Mottley-Henry')]
df = df[~df.Name.str.contains('Dylan Reid')]
df = df[~df.Name.str.contains('Dylan Vente')]
df = df[~df.Name.str.contains('EÃ±aut Mendia')]
df = df[~df.Name.str.contains('Ebrima Darboe')]
df = df[~df.Name.str.contains('Eddie Nolan')]
df = df[~df.Name.str.contains('Edgar Castillo')]
df = df[~df.Name.str.contains('Edison Duarte')]
df = df[~df.Name.str.contains('Edoardo Bove')]
df = df[~df.Name.str.contains('Edwin Banguera')]
df = df[~df.Name.str.contains('Edwin Cardona')]
df = df[~df.Name.str.contains('Edwin Mosquera')]
df = df[~df.Name.str.contains('Egemen Pehlivan')]
df = df[~df.Name.str.contains('Egor Generalov')]
df = df[~df.Name.str.contains('Egor Prutsev')]
df = df[~df.Name.str.contains('Ekrem Kilicarslan')]
df = df[~df.Name.str.contains('El Mami Tetah')]
df = df[~df.Name.str.contains('Elbasan Rashani')]
df = df[~df.Name.str.contains('Eldin Jakupovic')]
df = df[~df.Name.str.contains('Elia Caprile')]
df = df[~df.Name.str.contains('Elias Herzig')]
df = df[~df.Name.str.contains('Elias Manoel')]
df = df[~df.Name.str.contains('Elias Oubella')]
df = df[~df.Name.str.contains('Elias Pereyra')]
df = df[~df.Name.str.contains('Elliot Anderson')]
df = df[~df.Name.str.contains('Elliott Bolton')]
df = df[~df.Name.str.contains('Elliott Bonds')]
df = df[~df.Name.str.contains('Ellis Simms')]
df = df[~df.Name.str.contains('ElzoElzo')]
df = df[~df.Name.str.contains('Emanuel Herrera')]
df = df[~df.Name.str.contains('Emanuel Insua')]
df = df[~df.Name.str.contains('Emanuel Olivera')]
df = df[~df.Name.str.contains('Emanuel Rosales')]
df = df[~df.Name.str.contains('Emir Senocak')]
df = df[~df.Name.str.contains('Emirhan Kascioglu')]
df = df[~df.Name.str.contains('Emmanuel Boateng')]
df = df[~df.Name.str.contains('Emmanuel Gyabuaa')]
df = df[~df.Name.str.contains('Emmanuel Hackman')]
df = df[~df.Name.str.contains('Emmanuel Idem')]
df = df[~df.Name.str.contains('Emmanuel Monthe')]
df = df[~df.Name.str.contains('Emmanuel Ochoa')]
df = df[~df.Name.str.contains('Emmanuel Ojeda')]
df = df[~df.Name.str.contains('Emeka Obi')]
df = df[~df.Name.str.contains('Emerson Royal')]
df = df[~df.Name.str.contains('Emiliano Insua')]
df = df[~df.Name.str.contains('Emiliano Vecchio')]
df = df[~df.Name.str.contains('Emin Bayram')]
df = df[~df.Name.str.contains('Emrah Bassan')]
df = df[~df.Name.str.contains('Emre Bilgin')]
df = df[~df.Name.str.contains('Emre Colak')]
df = df[~df.Name.str.contains('Emre Karaal')]
df = df[~df.Name.str.contains('Enes Karakus')]
df = df[~df.Name.str.contains('Enes Sancar Sahin')]
df = df[~df.Name.str.contains('Ennio van der Gouw')]
df = df[~df.Name.str.contains('Enrico Hernandez')]
df = df[~df.Name.str.contains('Enver Cenk Sahin')]
df = df[~df.Name.str.contains('Enzo Coacci')]
df = df[~df.Name.str.contains('Enzo Copetti')]
df = df[~df.Name.str.contains('Enzo Fernandez')]
df = df[~df.Name.str.contains('Eray Iscan')]
df = df[~df.Name.str.contains('Erdin Demir')]
df = df[~df.Name.str.contains('Erdogan Kaya')]
df = df[~df.Name.str.contains('Eren Elmali')]
df = df[~df.Name.str.contains('Eren Kiryolcu')]
df = df[~df.Name.str.contains('Eric Bocat')]
df = df[~df.Name.str.contains('Eric Calvillo')]
df = df[~df.Name.str.contains('Eric DickEric Dick')]
df = df[~df.Name.str.contains('Eric Bocat')]
df = df[~df.Name.str.contains('Eric Calvillo')]
df = df[~df.Name.str.contains('Eric Ebimbe')]
df = df[~df.Name.str.contains('Eric Junior Bocat')]
df = df[~df.Name.str.contains('Eric Lichaj')]
df = df[~df.Name.str.contains('Eric Lopez')]
df = df[~df.Name.str.contains('Eric MezaEric Meza')]
df = df[~df.Name.str.contains('Eric Miller')]
df = df[~df.Name.str.contains('Erick Ferigra')]
df = df[~df.Name.str.contains('Ernest Boahene')]
df = df[~df.Name.str.contains('Erwan Belhadji')]
df = df[~df.Name.str.contains('Esteban Conde')]
df = df[~df.Name.str.contains('Esteban Rolon')]
df = df[~df.Name.str.contains('Ethan Bartlow')]
df = df[~df.Name.str.contains('Ethan Johnston')]
df = df[~df.Name.str.contains('Ethan Laird')]
df = df[~df.Name.str.contains('Etienne Vaessen')]
df = df[~df.Name.str.contains('Eugenio Pizzuto')]
df = df[~df.Name.str.contains('Evan BushEvan Bush')]
df = df[~df.Name.str.contains('Evan Newton')]
df = df[~df.Name.str.contains('Evann Guessand')]
df = df[~df.Name.str.contains('EveraldoEveraldo')]
df = df[~df.Name.str.contains('Evert Linthorst')]
df = df[~df.Name.str.contains('Everton Felipe')]
df = df[~df.Name.str.contains('Ezequiel Cerica')]
df = df[~df.Name.str.contains('Ezequiel Cirigliano')]
df = df[~df.Name.str.contains('Ezequiel Edison Rescaldani')]
df = df[~df.Name.str.contains('Ezequiel Fernandez')]
df = df[~df.Name.str.contains('Ezequiel Piovi')]
df = df[~df.Name.str.contains('Ezequiel Schelotto')]
df = df[~df.Name.str.contains('Ezequiel Unsain')]
df = df[~df.Name.str.contains('FaÃ¯z Selemani')]
df = df[~df.Name.str.contains('Fabian Cubero')]
df = df[~df.Name.str.contains('Fabian de Keijzer')]
df = df[~df.Name.str.contains('Fabian Kalig')]
df = df[~df.Name.str.contains('Fabian Noguera')]
df = df[~df.Name.str.contains('Fabien Farnolle')]
df = df[~df.Name.str.contains('Fabio Abreu')]
df = df[~df.Name.str.contains('Fabio Carvalho')]
df = df[~df.Name.str.contains('Fabio China')]
df = df[~df.Name.str.contains('Fabio ChinaFabio China')]
df = df[~df.Name.str.contains('Fabio Pereyra')]
df = df[~df.Name.str.contains('Fabio Pisacane')]
df = df[~df.Name.str.contains('Fabio SantosFabio Santos')]
df = df[~df.Name.str.contains('Fabio Schneider')]
df = df[~df.Name.str.contains('Fabio Vazquez')]
df = df[~df.Name.str.contains('Fabrice Hartmann')]
df = df[~df.Name.str.contains('Fabricio Dominguez')]
df = df[~df.Name.str.contains('Fabricio Fontanini')]
df = df[~df.Name.str.contains('FabriFabri')]
df = df[~df.Name.str.contains('Fabrizio Caligara')]
df = df[~df.Name.str.contains('Facundo Altamirano')]
df = df[~df.Name.str.contains('Facundo Barboza')]
df = df[~df.Name.str.contains('Facundo Bertoglio')]
df = df[~df.Name.str.contains('Facundo Cambeses')]
df = df[~df.Name.str.contains('Facundo Castet')]
df = df[~df.Name.str.contains('Facundo Curuchet')]
df = df[~df.Name.str.contains('Facundo Fernandez')]
df = df[~df.Name.str.contains('Facundo Ferrero')]
df = df[~df.Name.str.contains('Facundo Garces')]
df = df[~df.Name.str.contains('Facundo Mater')]
df = df[~df.Name.str.contains('Facundo Melivilo')]
df = df[~df.Name.str.contains('Facundo Pons')]
df = df[~df.Name.str.contains('Facundo Sanchez')]
df = df[~df.Name.str.contains('Facundo Tobares')]
df = df[~df.Name.str.contains('Faiq Bolkiah')]
df = df[~df.Name.str.contains('Faiz Selemani')]
df = df[~df.Name.str.contains('Falcao')]
df = df[~df.Name.str.contains('Fali')]
df = df[~df.Name.str.contains('FaliFali')]
df = df[~df.Name.str.contains('Famana Quizera')]
df = df[~df.Name.str.contains('Fatih Yigit Sanliturk')]
df = df[~df.Name.str.contains('Favio Cabral')]
df = df[~df.Name.str.contains('Fede Varela')]
df = df[~df.Name.str.contains('Federico Andueza')]
df = df[~df.Name.str.contains('Federico Bonansea')]
df = df[~df.Name.str.contains('Federico Brancolini')]
df = df[~df.Name.str.contains('Federico Gattoni')]
df = df[~df.Name.str.contains('Federico Gino')]
df = df[~df.Name.str.contains('Federico Girotti')]
df = df[~df.Name.str.contains('Federico Lanzillotta')]
df = df[~df.Name.str.contains('Federico Mancuello')]
df = df[~df.Name.str.contains('Federico Martinez')]
df = df[~df.Name.str.contains('Federico Palacios')]
df = df[~df.Name.str.contains('Federico Ricci')]
df = df[~df.Name.str.contains('Federico Vera')]
df = df[~df.Name.str.contains('Federico Vietto')]
df = df[~df.Name.str.contains('Federico Vismara')]
df = df[~df.Name.str.contains('Felipe Aguilar')]
df = df[~df.Name.str.contains('Felipe Martins')]
df = df[~df.Name.str.contains('Felipe PeÃ±a Biafore')]
df = df[~df.Name.str.contains('Felipe Santana')]
df = df[~df.Name.str.contains('Felipe Valencia')]
df = df[~df.Name.str.contains('Felipe VizeuFelipe Vizeu')]
df = df[~df.Name.str.contains('Felix Correia')]
df = df[~df.Name.str.contains('Felix Irorere')]
df = df[~df.Name.str.contains('Femi Azeez')]
df = df[~df.Name.str.contains('Femi Seriki')]
df = df[~df.Name.str.contains('Fer NiÃ±oFer NiÃ±o')]
df = df[~df.Name.str.contains('Ferhat Erdogen')]
df = df[~df.Name.str.contains('Ferhat GÃ¶rgulu')]
df = df[~df.Name.str.contains('Ferhat Kaplan')]
df = df[~df.Name.str.contains('Fernando Andrade')]
df = df[~df.Name.str.contains('Fernando Caixeta')]
df = df[~df.Name.str.contains('Fernando Canesin')]
df = df[~df.Name.str.contains('Fernando Coniglio')]
df = df[~df.Name.str.contains('Fernando Cosciuc')]
df = df[~df.Name.str.contains('Fernando Evangelista')]
df = df[~df.Name.str.contains('Fernando Gago')]
df = df[~df.Name.str.contains('Fernando Luna')]
df = df[~df.Name.str.contains('Fernando Meza')]
df = df[~df.Name.str.contains('Fernando Monetti')]
df = df[~df.Name.str.contains('Fernando NiÃ±o')]
df = df[~df.Name.str.contains('Fernando Pellegrino')]
df = df[~df.Name.str.contains('Fernando Reges')]
df = df[~df.Name.str.contains('Fernando Roman')]
df = df[~df.Name.str.contains('Fernando Sobral')]
df = df[~df.Name.str.contains('Fernando Tobio')]
df = df[~df.Name.str.contains('FerreiraFerreira')]
df = df[~df.Name.str.contains('Festus Arthur')]
df = df[~df.Name.str.contains('Festy Ebosele')]
df = df[~df.Name.str.contains('Filippo Rinaldi')]
df = df[~df.Name.str.contains('Filippo Romagna')]
df = df[~df.Name.str.contains('Fin Stevens')]
df = df[~df.Name.str.contains('Finlay Hurford-Lockett')]
df = df[~df.Name.str.contains('Finley Craske')]
df = df[~df.Name.str.contains('Finley Shrimpton')]
df = df[~df.Name.str.contains('Finley Stevens')]
df = df[~df.Name.str.contains('Finn Azaz')]
df = df[~df.Name.str.contains('Florent da Silva')]
df = df[~df.Name.str.contains('Florent Hoti')]
df = df[~df.Name.str.contains('Florian Flecker')]
df = df[~df.Name.str.contains('Florian Kamberi')]
df = df[~df.Name.str.contains('Florian Monzon')]
df = df[~df.Name.str.contains('Florian Stritzel')]
df = df[~df.Name.str.contains('Florin Andone')]
df = df[~df.Name.str.contains('Foster Langsdorf')]
df = df[~df.Name.str.contains('Francesco Rossi')]
df = df[~df.Name.str.contains('Francisco Apaolaza')]
df = df[~df.Name.str.contains('Francisco Flores')]
df = df[~df.Name.str.contains('Francisco Fydriszewski')]
df = df[~df.Name.str.contains('Francisco Geraldes')]
df = df[~df.Name.str.contains('Francisco Ilarregui')]
df = df[~df.Name.str.contains('Francisco Manenti')]
df = df[~df.Name.str.contains('Francisco Meixedo')]
df = df[~df.Name.str.contains('Francisco Montero')]
df = df[~df.Name.str.contains('Francisco Ramirez')]
df = df[~df.Name.str.contains('Francisco Silva')]
df = df[~df.Name.str.contains('Franco Baldassarra')]
df = df[~df.Name.str.contains('Franco Di Santo')]
df = df[~df.Name.str.contains('Franco Escobar')]
df = df[~df.Name.str.contains('Franco Godoy')]
df = df[~df.Name.str.contains('Franco Gonzalez')]
df = df[~df.Name.str.contains('Franco Leys')]
df = df[~df.Name.str.contains('Franco Negri')]
df = df[~df.Name.str.contains('Franco Orozco')]
df = df[~df.Name.str.contains('Franco Paredes')]
df = df[~df.Name.str.contains('Franco Perez')]
df = df[~df.Name.str.contains('Franco Perinciolo')]
df = df[~df.Name.str.contains('Franco Pizzicanella')]
df = df[~df.Name.str.contains('Franco Quiroz')]
df = df[~df.Name.str.contains('Franco Rivasseau')]
df = df[~df.Name.str.contains('Franco Tongya')]
df = df[~df.Name.str.contains('Francois Marquet')]
df = df[~df.Name.str.contains('Francois-Xavier Fumu Tamuzo')]
df = df[~df.Name.str.contains('Frank Fielding')]
df = df[~df.Name.str.contains('Frank Tsadjout')]
df = df[~df.Name.str.contains('Frank Vincent')]
df = df[~df.Name.str.contains('Franko Kovacevic')]
df = df[~df.Name.str.contains('Fraser Murray')]
df = df[~df.Name.str.contains('Fred Emmings')]
df = df[~df.Name.str.contains('Frederik Alves')]
df = df[~df.Name.str.contains('Fredrik Oppegard')]
df = df[~df.Name.str.contains('Furkan Beklevic')]
df = df[~df.Name.str.contains('Furkan Kara')]
df = df[~df.Name.str.contains('Fynn OttoFynn Otto')]
df = df[~df.Name.str.contains('Gabriel Barbosa')]
df = df[~df.Name.str.contains('Gabriel Batista')]
df = df[~df.Name.str.contains('Gabriel Boschilia')]
df = df[~df.Name.str.contains('Gabriel Carrasco')]
df = df[~df.Name.str.contains('Gabriel Chapeco')]
df = df[~df.Name.str.contains('Gabriel Compagnucci')]
df = df[~df.Name.str.contains('Gabriel Dias')]
df = df[~df.Name.str.contains('Gabriel Diaz')]
df = df[~df.Name.str.contains('Gabriel Esparza')]
df = df[~df.Name.str.contains('Gabriel Falcao')]
df = df[~df.Name.str.contains('Gabriel Hachen')]
df = df[~df.Name.str.contains('Gabriel Pires')]
df = df[~df.Name.str.contains('Gabriel Rojas')]
df = df[~df.Name.str.contains('Gabriel Slonina')]
df = df[~df.Name.str.contains('Gabriel Veiga')]
df = df[~df.Name.str.contains('Gabriel VeronGabriel Veron')]
df = df[~df.Name.str.contains('Gaetan Poussin')]
df = df[~df.Name.str.contains('GalenoGalenoLeft Midfiel')]
df = df[~df.Name.str.contains('Gamid Agalarov')]
df = df[~df.Name.str.contains('GansoGanso')]
df = df[~df.Name.str.contains('Gary Maley')]
df = df[~df.Name.str.contains('Gary Roberts')]
df = df[~df.Name.str.contains('Gary Sawyer')]
df = df[~df.Name.str.contains('Gaston Ãlvarez Suarez')]
df = df[~df.Name.str.contains('Gaston Benavidez')]
df = df[~df.Name.str.contains('Gaston Diaz')]
df = df[~df.Name.str.contains('Gaston Fernandez')]
df = df[~df.Name.str.contains('Gaston Gerzel')]
df = df[~df.Name.str.contains('Gaston Gomez')]
df = df[~df.Name.str.contains('Gaston Gonzalez')]
df = df[~df.Name.str.contains('Gaston Suso')]
df = df[~df.Name.str.contains('Gaston Torres')]
df = df[~df.Name.str.contains('Gavin Bazunu')]
df = df[~df.Name.str.contains('Gedion Zelalem')]
df = df[~df.Name.str.contains('GeilsonGeilson')]
df = df[~df.Name.str.contains('Gela Zaseev')]
df = df[~df.Name.str.contains('George Acosta')]
df = df[~df.Name.str.contains('George Asomani')]
df = df[~df.Name.str.contains('George Campbell')]
df = df[~df.Name.str.contains('George Fochive')]
df = df[~df.Name.str.contains('George Hirst')]
df = df[~df.Name.str.contains('George Melkadze')]
df = df[~df.Name.str.contains('George Nugent')]
df = df[~df.Name.str.contains('Georges Mandjeck')]
df = df[~df.Name.str.contains('Georgi Dzhikiya')]
df = df[~df.Name.str.contains('Getterson')]
df = df[~df.Name.str.contains('Giacomo Ricci')]
df = df[~df.Name.str.contains('Giacomo Vrioni')]
df = df[~df.Name.str.contains('Gian-Luca Itter')]
df = df[~df.Name.str.contains('Gian-Luca Waldschmidt')]
df = df[~df.Name.str.contains('Giancarlo Gonzalez')]
df = df[~df.Name.str.contains('Gianfranco Facchineri')]
df = df[~df.Name.str.contains('Giannelli Imbula')]
df = df[~df.Name.str.contains('Gianni Mollo')]
df = df[~df.Name.str.contains('Giorgi Kochorashvili')]
df = df[~df.Name.str.contains('Giovanni Crociata')]
df = df[~df.Name.str.contains('Giuseppe Mastinu')]
df = df[~df.Name.str.contains('Gjoko Zajkov')]
df = df[~df.Name.str.contains('Gokhan Kardes')]
df = df[~df.Name.str.contains('Gonzalo Asis')]
df = df[~df.Name.str.contains('Gonzalo Castellani')]
df = df[~df.Name.str.contains('Gonzalo GoÃ±i')]
df = df[~df.Name.str.contains('Gonzalo Jara')]
df = df[~df.Name.str.contains('Gonzalo Maroni')]
df = df[~df.Name.str.contains('Gonzalo Rodriguez')]
df = df[~df.Name.str.contains('Gonzalo Veron')]
df = df[~df.Name.str.contains('Grace Bokake')]
df = df[~df.Name.str.contains('Graham Smith')]
df = df[~df.Name.str.contains('Grant Lillard')]
df = df[~df.Name.str.contains('Grayson Barber')]
df = df[~df.Name.str.contains('Greg Ranjitsingh')]
df = df[~df.Name.str.contains('Griffin Yow')]
df = df[~df.Name.str.contains('Guido Mainero')]
df = df[~df.Name.str.contains('Guido Vadala')]
df = df[~df.Name.str.contains('Guilherme Camacho')]
df = df[~df.Name.str.contains('Guilherme GuedesGuilherme Guedes')]
df = df[~df.Name.str.contains('Guilherme Lazaroni')]
df = df[~df.Name.str.contains('Guilherme Mantuan')]
df = df[~df.Name.str.contains('Guilherme Oliveira')]
df = df[~df.Name.str.contains('Guillem Molina')]
df = df[~df.Name.str.contains('Guillermo Celis')]
df = df[~df.Name.str.contains('Guillermo Fernandez')]
df = df[~df.Name.str.contains('Guillermo Gozalvez')]
df = df[~df.Name.str.contains('Guram Tetrashvili')]
df = df[~df.Name.str.contains('GustavinhoGustavinho')]
df = df[~df.Name.str.contains('Gustavo ApisGustavo Apis')]
df = df[~df.Name.str.contains('Gustavo BlancoGustavo Blanco')]
df = df[~df.Name.str.contains('Gustavo Canto')]
df = df[~df.Name.str.contains('Gustavo CoutinhoGustavo Coutinho')]
df = df[~df.Name.str.contains('Gustavo Garcia')]
df = df[~df.Name.str.contains('Gustavo Henrique')]
df = df[~df.Name.str.contains('Gustavo Mosquito')]
df = df[~df.Name.str.contains('Gustavo NescauGustavo Nescau')]
df = df[~df.Name.str.contains('Gustavo SilvaGustavo Silva')]
df = df[~df.Name.str.contains('Guy Mbenza')]
df = df[~df.Name.str.contains('Haqi Osman')]
df = df[~df.Name.str.contains('Harlem Gnohere')]
df = df[~df.Name.str.contains('Harry Clarke')]
df = df[~df.Name.str.contains('Harry Draper')]
df = df[~df.Name.str.contains('Harry Isted')]
df = df[~df.Name.str.contains('Harry Lewis')]
df = df[~df.Name.str.contains('Harry McHugh')]
df = df[~df.Name.str.contains('Harry Paton')]
df = df[~df.Name.str.contains('Harry Vaughan')]
df = df[~df.Name.str.contains('Harry Wood')]
df = df[~df.Name.str.contains('Harry Wright')]
df = df[~df.Name.str.contains('Harvey Bunker')]
df = df[~df.Name.str.contains('Harvey Cartwright')]
df = df[~df.Name.str.contains('Harvey Elliott')]
df = df[~df.Name.str.contains('Harvey Rew')]
df = df[~df.Name.str.contains('Harvey White')]
df = df[~df.Name.str.contains('Hayden Carter')]
df = df[~df.Name.str.contains('Hayden Hackney')]
df = df[~df.Name.str.contains('Hayden Muller')]
df = df[~df.Name.str.contains('Haydon Roberts')]
df = df[~df.Name.str.contains('Helder Barbosa')]
df = df[~df.Name.str.contains('Henning Matriciani')]
df = df[~df.Name.str.contains('Henri Saivet')]
df = df[~df.Name.str.contains('Henrich Ravas')]
df = df[~df.Name.str.contains('Henrique Trevisan')]
df = df[~df.Name.str.contains('Hernan Burbano')]
df = df[~df.Name.str.contains('Hernan Lamberti')]
df = df[~df.Name.str.contains('Hernan Lopez')]
df = df[~df.Name.str.contains('Hicham Mahou')]
df = df[~df.Name.str.contains('Hugo Novoa')]
df = df[~df.Name.str.contains('Hugo Rodallega')]
df = df[~df.Name.str.contains('Hugo Silveira')]
df = df[~df.Name.str.contains('Hugo Sotelo')]
df = df[~df.Name.str.contains('Hugo SouzaHugo Souza')]
df = df[~df.Name.str.contains('Iago MaidanaIago Maidana')]
df = df[~df.Name.str.contains('Ian Escobar')]
df = df[~df.Name.str.contains('Ian FrayIan Fray')]
df = df[~df.Name.str.contains('Ian Hoffmann')]
df = df[~df.Name.str.contains('Ian Maatsen')]
df = df[~df.Name.str.contains('Ian Poveda')]
df = df[~df.Name.str.contains('Ian Poveda-Ocampo')]
df = df[~df.Name.str.contains('IbrahimaIbrahima')]
df = df[~df.Name.str.contains('Idris Umaev')]
df = df[~df.Name.str.contains('Idrissa Dioh')]
df = df[~df.Name.str.contains('Idrissa Sylla')]
df = df[~df.Name.str.contains('Ifunanyachi Achara')]
df = df[~df.Name.str.contains('Ignacio Miramon')]
df = df[~df.Name.str.contains('Ignacio Russo')]
df = df[~df.Name.str.contains('Ignacio Schor')]
df = df[~df.Name.str.contains('Igor Lucatelli')]
df = df[~df.Name.str.contains('Igor Obukhov')]
df = df[~df.Name.str.contains('Ilhan Depe')]
df = df[~df.Name.str.contains('Ilia Shkurin')]
df = df[~df.Name.str.contains('Iliman Ndiaye')]
df = df[~df.Name.str.contains('Isaac Olaofe')]
df = df[~df.Name.str.contains('Isaac Vassell')]
df = df[~df.Name.str.contains('Isaiah Brown')]
df = df[~df.Name.str.contains('Jakub Niemczyk')]
df = df[~df.Name.str.contains('Jakub Szumski')]
df = df[~df.Name.str.contains('Jalen Neal')]
df = df[~df.Name.str.contains('James Clark')]
df = df[~df.Name.str.contains('James Dodd')]
df = df[~df.Name.str.contains('James Garner')]
df = df[~df.Name.str.contains('James Gullan')]
df = df[~df.Name.str.contains('James Marcinkowski')]
df = df[~df.Name.str.contains('James Montgomery')]
df = df[~df.Name.str.contains('James Morton')]
df = df[~df.Name.str.contains('James Olayinka')]
df = df[~df.Name.str.contains('James Pardington')]
df = df[~df.Name.str.contains('James Rowland')]
df = df[~df.Name.str.contains('James Vennings')]
df = df[~df.Name.str.contains('Jamie Armstrong')]
df = df[~df.Name.str.contains('Jamie Barjonas')]
df = df[~df.Name.str.contains('Jamie Cumming')]
df = df[~df.Name.str.contains('Jamie Fielding')]
df = df[~df.Name.str.contains('Jamie Gullan')]
df = df[~df.Name.str.contains('Jamie Mascoll')]
df = df[~df.Name.str.contains('Jamie Semple')]
df = df[~df.Name.str.contains('Jamie Smith')]
df = df[~df.Name.str.contains('Jamie Soule')]
df = df[~df.Name.str.contains('Jamil Awoyejo')]
df = df[~df.Name.str.contains('Jan Bekkema')]
df = df[~df.Name.str.contains('Jan Carlos Hurtado')]
df = df[~df.Name.str.contains('Jan de Boer')]
df = df[~df.Name.str.contains('Jan Hurtado')]
df = df[~df.Name.str.contains('Jaquite')]
df = df[~df.Name.str.contains('Jarno Westerman')]
df = df[~df.Name.str.contains('Jaroslaw Jach')]
df = df[~df.Name.str.contains('Jaroslaw Niezgoda')]
df = df[~df.Name.str.contains('Jarvis Cleal')]
df = df[~df.Name.str.contains('Jason Cummings')]
df = df[~df.Name.str.contains('Jay-Dee Geusens')]
df = df[~df.Name.str.contains('Jayden Bennetts')]
df = df[~df.Name.str.contains('Jayden Campbell')]
df = df[~df.Name.str.contains('Jayden Mitchell-Lawson')]
df = df[~df.Name.str.contains('Jayden Onen')]
df = df[~df.Name.str.contains('Jayden Reid')]
df = df[~df.Name.str.contains('Jayden Richardson')]
df = df[~df.Name.str.contains('Jayson Molumby')]
df = df[~df.Name.str.contains('Jeison Chala')]
df = df[~df.Name.str.contains('Jeison Lucumi')]
df = df[~df.Name.str.contains('Jeizon Ramirez')]
df = df[~df.Name.str.contains('Jelle Duin')]
df = df[~df.Name.str.contains('Jens Jonsson')]
df = df[~df.Name.str.contains('Jeong Woo-Yeong')]
df = df[~df.Name.str.contains('Jeredy Hilterman')]
df = df[~df.Name.str.contains('Jeremy Antonisse')]
df = df[~df.Name.str.contains('Jeremy Helmer')]
df = df[~df.Name.str.contains('Jeremy Kelly')]
df = df[~df.Name.str.contains('Jeremy Perbet')]
df = df[~df.Name.str.contains('Jeremy Van Mullem')]
df = df[~df.Name.str.contains('Jerome Greaves')]
df = df[~df.Name.str.contains('Jerome Opoku')]
df = df[~df.Name.str.contains('Jerome Sinclair')]
df = df[~df.Name.str.contains('Jeronimo Pourtau')]
df = df[~df.Name.str.contains('Jesus Ferreira')]
df = df[~df.Name.str.contains('Jim McNulty')]
df = df[~df.Name.str.contains('Jimmy Knowles')]
df = df[~df.Name.str.contains('Jimmy McNulty')]
df = df[~df.Name.str.contains('Jiri Skalak')]
df = df[~df.Name.str.contains('Joao Jaquite')]
df = df[~df.Name.str.contains('Joao Klauss')]
df = df[~df.Name.str.contains('Joao Mendes')]
df = df[~df.Name.str.contains('Joao Monteiro')]
df = df[~df.Name.str.contains('Joao Queiros')]
df = df[~df.Name.str.contains('Joaquin Arzura')]
df = df[~df.Name.str.contains('Joaquin Blazquez')]
df = df[~df.Name.str.contains('Joaquin Hass')]
df = df[~df.Name.str.contains('Joaquin Mendive')]
df = df[~df.Name.str.contains('Joaquin Varela')]
df = df[~df.Name.str.contains('Jobi McAnuff')]
df = df[~df.Name.str.contains('Jock Curran')]
df = df[~df.Name.str.contains('Jodi Jones')]
df = df[~df.Name.str.contains('Jody Lukoki')]
df = df[~df.Name.str.contains('Joe Adams')]
df = df[~df.Name.str.contains('Joe Foulkes')]
df = df[~df.Name.str.contains('Joe Fryer')]
df = df[~df.Name.str.contains('Joe Garner')]
df = df[~df.Name.str.contains('Joe Gelhardt')]
df = df[~df.Name.str.contains('Joe Grayson')]
df = df[~df.Name.str.contains('Joe Hilton')]
df = df[~df.Name.str.contains('Joe Leesley')]
df = df[~df.Name.str.contains('Joel Coleman')]
df = df[~df.Name.str.contains('Joey Jones')]
df = df[~df.Name.str.contains('Johan Carbonero')]
df = df[~df.Name.str.contains('John Brooks')]
df = df[~df.Name.str.contains('John Donald')]
df = df[~df.Name.str.contains('John Nekadio')]
df = df[~df.Name.str.contains('John Patrick')]
df = df[~df.Name.str.contains('John Robertson')]
df = df[~df.Name.str.contains('John Yeboah')]
df = df[~df.Name.str.contains('Kacper Mialkowski')]
df = df[~df.Name.str.contains('Kacper Urbanski')]
df = df[~df.Name.str.contains('Kagan Moradaoglu')]
df = df[~df.Name.str.contains('Kai Koreniuk')]
df = df[~df.Name.str.contains('Kai McKenzie-Lyle')]
df = df[~df.Name.str.contains('Kane Crichlow')]
df = df[~df.Name.str.contains('Kang-in Lee')]
df = df[~df.Name.str.contains('Karamoko Dembele')]
df = df[~df.Name.str.contains('Kastriot Dermaku')]
df = df[~df.Name.str.contains('Katuku Tshimanga')]
df = df[~df.Name.str.contains('Kayden Pierre')]
df = df[~df.Name.str.contains('Kayne Ramsay')]
df = df[~df.Name.str.contains('Kazaiah Sterling')]
df = df[~df.Name.str.contains('Keaghan Jacobs')]
df = df[~df.Name.str.contains('Keanu Staude')]
df = df[~df.Name.str.contains('Keito Nakamura')]
df = df[~df.Name.str.contains('Kelechi Nwakali')]
df = df[~df.Name.str.contains('Kelland Watts')]
df = df[~df.Name.str.contains('Kelsey Mooney')]
df = df[~df.Name.str.contains('Kemal Ruzgar')]
df = df[~df.Name.str.contains('Kendall McIntosh')]
df = df[~df.Name.str.contains('Kenneth Dougall')]
df = df[~df.Name.str.contains('Kenneth Kronholm')]
df = df[~df.Name.str.contains('Kenny Coker')]
df = df[~df.Name.str.contains('Kenny Dougall')]
df = df[~df.Name.str.contains('Kenny Lala')]
df = df[~df.Name.str.contains('Kenny Nagera')]
df = df[~df.Name.str.contains('Kevin Berkoe')]
df = df[~df.Name.str.contains('Kevin Boma')]
df = df[~df.Name.str.contains('Kevin Goden')]
df = df[~df.Name.str.contains('Kevin Harr')]
df = df[~df.Name.str.contains('Kevin Ibrahim')]
df = df[~df.Name.str.contains('Kevin Isa Luna')]
df = df[~df.Name.str.contains('Kevin Lankford')]
df = df[~df.Name.str.contains('Kevin McDonald')]
df = df[~df.Name.str.contains('Kevin Molino')]
df = df[~df.Name.str.contains('Kevin Nickler')]
df = df[~df.Name.str.contains('Kevin Silva')]
df = df[~df.Name.str.contains('Kevin Zenon')]
df = df[~df.Name.str.contains('Khalil Caraballo')]
df = df[~df.Name.str.contains('Kieran Agard')]
df = df[~df.Name.str.contains('Kieran Ngwenya')]
df = df[~df.Name.str.contains('Kim Moon-Hwan')]
df = df[~df.Name.str.contains('Kingsley Fobi')]
df = df[~df.Name.str.contains('Kiril Despodov')]
df = df[~df.Name.str.contains('Kirill Dontsov')]
df = df[~df.Name.str.contains('Kirill Kolesnichenko')]
df = df[~df.Name.str.contains('Kirill Kosarev')]
df = df[~df.Name.str.contains('Kirill Kravtsov')]
df = df[~df.Name.str.contains('Koba Koindredi')]
df = df[~df.Name.str.contains('Kwasi Wriedt')]
df = df[~df.Name.str.contains('Kyle Connell')]
df = df[~df.Name.str.contains('Kyle Hayde')]
df = df[~df.Name.str.contains('Kyle Morton')]
df = df[~df.Name.str.contains('Kyle Taylor')]
df = df[~df.Name.str.contains('Kyle Zobeck')]
df = df[~df.Name.str.contains('Laercio Solda')]
df = df[~df.Name.str.contains('Lameck Banda')]
df = df[~df.Name.str.contains('Lamine Sherif')]
df = df[~df.Name.str.contains('Landry Dimata')]
df = df[~df.Name.str.contains('Lars Coveliers')]
df = df[~df.Name.str.contains('Lars Lokotsch')]
df = df[~df.Name.str.contains('Laurenz Beckemeyer')]
df = df[~df.Name.str.contains('Lautaro Blanco')]
df = df[~df.Name.str.contains('Lautaro Chavez')]
df = df[~df.Name.str.contains('Lautaro de Leon')]
df = df[~df.Name.str.contains('Lautaro Escalante')]
df = df[~df.Name.str.contains('Lautaro Geminiani')]
df = df[~df.Name.str.contains('Lautaro Giaccone')]
df = df[~df.Name.str.contains('Lautaro Montoya')]
df = df[~df.Name.str.contains('Lautaro Morales')]
df = df[~df.Name.str.contains('Lautaro Parisi')]
df = df[~df.Name.str.contains('Lautaro Rinaldi')]
df = df[~df.Name.str.contains('Lazaros Lamprou')]
df = df[~df.Name.str.contains('Leandro Barcia')]
df = df[~df.Name.str.contains('Leandro Barreiro')]
df = df[~df.Name.str.contains('Leandro Barreiro Martins')]
df = df[~df.Name.str.contains('Leandro CarvalhoLeandro Carvalho')]
df = df[~df.Name.str.contains('Leandro Contin')]
df = df[~df.Name.str.contains('Leandro Fernandez')]
df = df[~df.Name.str.contains('Leandro Finochietto')]
df = df[~df.Name.str.contains('Leandro Grimi')]
df = df[~df.Name.str.contains('Leandro MatheusLeandro Matheus')]
df = df[~df.Name.str.contains('Leandro Putaro')]
df = df[~df.Name.str.contains('Leandro Silva')]
df = df[~df.Name.str.contains('Leandro Vella')]
df = df[~df.Name.str.contains('Lebo Mothiba')]
df = df[~df.Name.str.contains('Lee Kang-In')]
df = df[~df.Name.str.contains('Lenny Lacroix')]
df = df[~df.Name.str.contains('Lentini Caciano')]
df = df[~df.Name.str.contains('Leo Fuhr Hjelde')]
df = df[~df.Name.str.contains('Leo Gomes')]
df = df[~df.Name.str.contains('Leonardo Gil')]
df = df[~df.Name.str.contains('Leonardo Jara')]
df = df[~df.Name.str.contains('Leonardo Realpe')]
df = df[~df.Name.str.contains('Leonardo Sequeira')]
df = df[~df.Name.str.contains('Leonardo Villalba')]
df = df[~df.Name.str.contains('LeonardoLeonardo')]
df = df[~df.Name.str.contains('Leonel Ferroni')]
df = df[~df.Name.str.contains('Leonel Galeano')]
df = df[~df.Name.str.contains('Leonel Gonzalez')]
df = df[~df.Name.str.contains('Leroy Kwadwo')]
df = df[~df.Name.str.contains('Levi Malangu')]
df = df[~df.Name.str.contains('Levi Ntumba')]
df = df[~df.Name.str.contains('Lewis Bradley')]
df = df[~df.Name.str.contains('Lewis Gibson')]
df = df[~df.Name.str.contains('Lewis Hardcastle')]
df = df[~df.Name.str.contains('Lewis Johnson')]
df = df[~df.Name.str.contains('Lewis Richardson')]
df = df[~df.Name.str.contains('Lewis Simper')]
df = df[~df.Name.str.contains('Lewis Wilson')]
df = df[~df.Name.str.contains('Lewis Young')]
df = df[~df.Name.str.contains('Lex-Tyger Lobinger')]
df = df[~df.Name.str.contains('Liam Armstrong')]
df = df[~df.Name.str.contains('Liam Cross')]
df = df[~df.Name.str.contains('Liam Delap')]
df = df[~df.Name.str.contains('Liam Edwards')]
df = df[~df.Name.str.contains('Liam Gibbs')]
df = df[~df.Name.str.contains('Liam Lightfoot')]
df = df[~df.Name.str.contains('Liam Millar')]
df = df[~df.Name.str.contains('Liam Waldock')]
df = df[~df.Name.str.contains('Logan Ketterer')]
df = df[~df.Name.str.contains('Logan Ross')]
df = df[~df.Name.str.contains('Loic Ritiere')]
df = df[~df.Name.str.contains('Lois Diony')]
df = df[~df.Name.str.contains('Lorenzo Colombo')]
df = df[~df.Name.str.contains('Lorenzo Del Pinto')]
df = df[~df.Name.str.contains('Lorenzo Faravelli')]
df = df[~df.Name.str.contains('Lorenzo Melgarejo')]
df = df[~df.Name.str.contains('Lorhan')]
df = df[~df.Name.str.contains('Louie Watson')]
df = df[~df.Name.str.contains('Louis Boyd')]
df = df[~df.Name.str.contains('Louis Britton')]
df = df[~df.Name.str.contains('Louis Jordan Beyer')]
df = df[~df.Name.str.contains('Louis Morison')]
df = df[~df.Name.str.contains('Louis Moult')]
df = df[~df.Name.str.contains('Lovro Bizjak')]
df = df[~df.Name.str.contains('Luan Garcia')]
df = df[~df.Name.str.contains('Luan Santos')]
df = df[~df.Name.str.contains('Luan Silva')]
df = df[~df.Name.str.contains('Lubomir Tupta')]
df = df[~df.Name.str.contains('Luca Andrada')]
df = df[~df.Name.str.contains('Luca Antei')]
df = df[~df.Name.str.contains('Luca Connell')]
df = df[~df.Name.str.contains('Luca Itter')]
df = df[~df.Name.str.contains('Luca Lewis')]
df = df[~df.Name.str.contains('Luca Mora')]
df = df[~df.Name.str.contains('Luca Orellano')]
df = df[~df.Name.str.contains('Luca Pfeiffer')]
df = df[~df.Name.str.contains('Luca Philipp')]
df = df[~df.Name.str.contains('Luca Plogmann')]
df = df[~df.Name.str.contains('Luca Sangalli')]
df = df[~df.Name.str.contains('Luca Schuler')]
df = df[~df.Name.str.contains('Luca Siligardi')]
df = df[~df.Name.str.contains('Luca Unbehaun')]
df = df[~df.Name.str.contains('Luca Waldschmidt')]
df = df[~df.Name.str.contains('Luca Zander')]
df = df[~df.Name.str.contains('Luca-Milan Zander')]
df = df[~df.Name.str.contains('Lucao')]
df = df[~df.Name.str.contains('LucaoLucaoattac')]
df = df[~df.Name.str.contains('Lucas')]
df = df[~df.Name.str.contains('Lucas Beltran')]
df = df[~df.Name.str.contains('Lucas Besozzi')]
df = df[~df.Name.str.contains('Lucas Brochero')]
df = df[~df.Name.str.contains('Lucas Chevalier')]
df = df[~df.Name.str.contains('Lucas Di Yorio')]
df = df[~df.Name.str.contains('Lucas Dias')]
df = df[~df.Name.str.contains('Lucas EstevesLucas Esteves')]
df = df[~df.Name.str.contains('Lucas FonsecaLucas Fonseca')]
df = df[~df.Name.str.contains('Lucas GabrielLucas Gabriel')]
df = df[~df.Name.str.contains('Lucas HalterLucas Halter')]
df = df[~df.Name.str.contains('Lucas Kal')]
df = df[~df.Name.str.contains('Lucas KalLucas Kal')]
df = df[~df.Name.str.contains('Lucas Kruspzky')]
df = df[~df.Name.str.contains('Lucas Lima')]
df = df[~df.Name.str.contains('Lucas LourenÃ§oLucas LourenÃ§o')]
df = df[~df.Name.str.contains('Lucas Mancinelli')]
df = df[~df.Name.str.contains('Lucas Menossi')]
df = df[~df.Name.str.contains('Lucas Orellano')]
df = df[~df.Name.str.contains('Lucas Perri')]
df = df[~df.Name.str.contains('Lucas Pirard')]
df = df[~df.Name.str.contains('Lucas Rios')]
df = df[~df.Name.str.contains('Lucas Robertone')]
df = df[~df.Name.str.contains('Lucas Suarez')]
df = df[~df.Name.str.contains('Lucas Tagliapietra')]
df = df[~df.Name.str.contains('Lucas Tomlinson')]
df = df[~df.Name.str.contains('Lucas VenutoLucas Venuto')]
df = df[~df.Name.str.contains('Lucas Viatri')]
df = df[~df.Name.str.contains('Lucas Villarruel')]
df = df[~df.Name.str.contains('LuccaLucca')]
df = df[~df.Name.str.contains('Luciano Cingolani')]
df = df[~df.Name.str.contains('Luciano Ferreyra')]
df = df[~df.Name.str.contains('Luciano Gondou')]
df = df[~df.Name.str.contains('Luciano Pons')]
df = df[~df.Name.str.contains('Luciano Recalde')]
df = df[~df.Name.str.contains('Luciano Squadrone')]
df = df[~df.Name.str.contains('Luciano Vietto')]
df = df[~df.Name.str.contains('Ludovic Butelle')]
df = df[~df.Name.str.contains('Ludvig Ohman')]
df = df[~df.Name.str.contains('Luis Adlard')]
df = df[~df.Name.str.contains('Luis Binks')]
df = df[~df.Name.str.contains('Luis Caicedo')]
df = df[~df.Name.str.contains('Luis Coordes')]
df = df[~df.Name.str.contains('Luis Felipe')]
df = df[~df.Name.str.contains('Luis Garnier')]
df = df[~df.Name.str.contains('Luis Leal')]
df = df[~df.Name.str.contains('Luis Manuel Orejuela')]
df = df[~df.Name.str.contains('Luis Sequeira')]
df = df[~df.Name.str.contains('Luis Unsain')]
df = df[~df.Name.str.contains('Luis Vazquez')]
df = df[~df.Name.str.contains('Luiz Otavio')]
df = df[~df.Name.str.contains('Luiz PhellypeLuiz Phellype')]
df = df[~df.Name.str.contains('Lukas Lerager')]
df = df[~df.Name.str.contains('Lukas Zima')]
df = df[~df.Name.str.contains('Luke Bolton')]
df = df[~df.Name.str.contains('Luke Brennan')]
df = df[~df.Name.str.contains('Luke Haines')]
df = df[~df.Name.str.contains('Luke Hallett')]
df = df[~df.Name.str.contains('Luke Matheson')]
df = df[~df.Name.str.contains('Luke McNally')]
df = df[~df.Name.str.contains('Mace Goodridge')]
df = df[~df.Name.str.contains('Maciej Dabrowski')]
df = df[~df.Name.str.contains('Mal Benning')]
df = df[~df.Name.str.contains('Malamine Doumbouya')]
df = df[~df.Name.str.contains('Malcolm Barcola')]
df = df[~df.Name.str.contains('Malcom Braida')]
df = df[~df.Name.str.contains('Mamor Niang')]
df = df[~df.Name.str.contains('ManchaMancha')]
df = df[~df.Name.str.contains('Mandela Egbo')]
df = df[~df.Name.str.contains('Mandela Keita')]
df = df[~df.Name.str.contains('Manny Monthe')]
df = df[~df.Name.str.contains('Manny Oyeleke')]
df = df[~df.Name.str.contains('ManoelManoel')]
df = df[~df.Name.str.contains('Manuel Cabit')]
df = df[~df.Name.str.contains('Manuel Capasso')]
df = df[~df.Name.str.contains('Manuel da Costa')]
df = df[~df.Name.str.contains('Manuel Llano')]
df = df[~df.Name.str.contains('Manuel Namora')]
df = df[~df.Name.str.contains('Manuel Nieto')]
df = df[~df.Name.str.contains('Manuel Panaro')]
df = df[~df.Name.str.contains('Manuel Perez')]
df = df[~df.Name.str.contains('Manuel Vicentini')]
df = df[~df.Name.str.contains('Marc Auge')]
df = df[~df.Name.str.contains('Marc Baro')]
df = df[~df.Name.str.contains('Marc Cardona')]
df = df[~df.Name.str.contains('Marc Guehi')]
df = df[~df.Name.str.contains('Marc Pugh')]
df = df[~df.Name.str.contains('Marcel Johnen')]
df = df[~df.Name.str.contains('Marcel Ritzmaier')]
df = df[~df.Name.str.contains('Marcel Schmelzer')]
df = df[~df.Name.str.contains('Marcello Trotta')]
df = df[~df.Name.str.contains('Marcelo Boeck')]
df = df[~df.Name.str.contains('Marcelo Estigarribia')]
df = df[~df.Name.str.contains('Marcelo Freites')]
df = df[~df.Name.str.contains('Marcelo Goiano')]
df = df[~df.Name.str.contains('Marcelo Toscano')]
df = df[~df.Name.str.contains('Marciano Aziz')]
df = df[~df.Name.str.contains('Marcin Bulka')]
df = df[~df.Name.str.contains('Marcio Mossoro')]
df = df[~df.Name.str.contains('Marco Alia')]
df = df[~df.Name.str.contains('Marco Aratore')]
df = df[~df.Name.str.contains('Marco Benassi')]
df = df[~df.Name.str.contains('Marco Bertini')]
df = df[~df.Name.str.contains('Marco Borgnino')]
df = df[~df.Name.str.contains('Marco Delgado')]
df = df[~df.Name.str.contains('Marco Di Cesare')]
df = df[~df.Name.str.contains('Marco Djuricin')]
df = df[~df.Name.str.contains('Marco Faraoni')]
df = df[~df.Name.str.contains('Marco Ludivin Majouga')]
df = df[~df.Name.str.contains('Marco Majouga')]
df = df[~df.Name.str.contains('Marcos Rouzies')]
df = df[~df.Name.str.contains('Marcos Sanchez')]
df = df[~df.Name.str.contains('Marcus Ferkranus')]
df = df[~df.Name.str.contains('Marcus McGuane')]
df = df[~df.Name.str.contains('Marian Shved')]
df = df[~df.Name.str.contains('Mariano Bareiro')]
df = df[~df.Name.str.contains('Mariano Diaz')]
df = df[~df.Name.str.contains('Mariano Vazquez')]
df = df[~df.Name.str.contains('Mario Situm')]
df = df[~df.Name.str.contains('Mario Suver')]
df = df[~df.Name.str.contains('Marius Liesegang')]
df = df[~df.Name.str.contains('Mark Gillespie')]
df = df[~df.Name.str.contains('Mark Kosznovszky')]
df = df[~df.Name.str.contains('Mark McGuinness')]
df = df[~df.Name.str.contains('Marlon Fossey')]
df = df[~df.Name.str.contains('Marlon Hairston')]
df = df[~df.Name.str.contains('Marlon Mustapha')]
df = df[~df.Name.str.contains('Marlon Santos')]
df = df[~df.Name.str.contains('Marouan Azarkan')]
df = df[~df.Name.str.contains('Marquinhos Calazans')]
df = df[~df.Name.str.contains('Martin Samuelsen')]
df = df[~df.Name.str.contains('Martin Skrtel')]
df = df[~df.Name.str.contains('Martin Smith')]
df = df[~df.Name.str.contains('Marvin Dewaele')]
df = df[~df.Name.str.contains('Marvin Obuz')]
df = df[~df.Name.str.contains('Marvin Park')]
df = df[~df.Name.str.contains('Marvin Tshibuabua')]
df = df[~df.Name.str.contains('Maryan Shved')]
df = df[~df.Name.str.contains('Mason Stajduhar')]
df = df[~df.Name.str.contains('Massimo Giamattei')]
df = df[~df.Name.str.contains('Mat Ryan')]
df = df[~df.Name.str.contains('Matej Kovar')]
df = df[~df.Name.str.contains('Matej Oravec')]
df = df[~df.Name.str.contains('Mateo Coronel')]
df = df[~df.Name.str.contains('Mateo Mamani')]
df = df[~df.Name.str.contains('Mateo Tanlongo')]
df = df[~df.Name.str.contains('Matheo Vroman')]
df = df[~df.Name.str.contains('Matheus Clemente')]
df = df[~df.Name.str.contains('Matheus Donelli')]
df = df[~df.Name.str.contains('Matheus Fernandes')]
df = df[~df.Name.str.contains('Matheus Oliveira')]
df = df[~df.Name.str.contains('Matheuzinho')]
df = df[~df.Name.str.contains('Mathew Hudson')]
df = df[~df.Name.str.contains('Mathew Ryan')]
df = df[~df.Name.str.contains('Mathias Suarez')]
df = df[~df.Name.str.contains('Mathias Wittek')]
df = df[~df.Name.str.contains('Mathieu Dossevi')]
df = df[~df.Name.str.contains('Mathys Saban')]
df = df[~df.Name.str.contains('Matias Belloso')]
df = df[~df.Name.str.contains('Matias Borgogno')]
df = df[~df.Name.str.contains('Matias Cordoba')]
df = df[~df.Name.str.contains('Matias De los Santos')]
df = df[~df.Name.str.contains('Matias de los Santos')]
df = df[~df.Name.str.contains('Matias Escudero')]
df = df[~df.Name.str.contains('Matias Fritzler')]
df = df[~df.Name.str.contains('Matias Garcia')]
df = df[~df.Name.str.contains('Matias Gomez')]
df = df[~df.Name.str.contains('Matias Gonzalez')]
df = df[~df.Name.str.contains('Matias Laba')]
df = df[~df.Name.str.contains('Matias Molina')]
df = df[~df.Name.str.contains('Matias Moya')]
df = df[~df.Name.str.contains('Matias Palavecino')]
df = df[~df.Name.str.contains('Matias Pellegrini')]
df = df[~df.Name.str.contains('Matias Pisano')]
df = df[~df.Name.str.contains('Matias Ramirez')]
df = df[~df.Name.str.contains('Matias Rodriguez')]
df = df[~df.Name.str.contains('Matias Sosa')]
df = df[~df.Name.str.contains('Matias Tagliamonte')]
df = df[~df.Name.str.contains('Matias Tissera')]
df = df[~df.Name.str.contains('Matias Villarreal')]
df = df[~df.Name.str.contains('Matis Carvalho')]
df = df[~df.Name.str.contains('Matisse Didden')]
df = df[~df.Name.str.contains('Matko Miljevic')]
df = df[~df.Name.str.contains('Matt Bersano')]
df = df[~df.Name.str.contains('Matt Clarke')]
df = df[~df.Name.str.contains('Matt Dolan')]
df = df[~df.Name.str.contains('Matt Freese')]
df = df[~df.Name.str.contains('Matt Gilks')]
df = df[~df.Name.str.contains('Matt Hundley')]
df = df[~df.Name.str.contains('Matt Lampson')]
df = df[~df.Name.str.contains('Matt Palmer')]
df = df[~df.Name.str.contains('Matt Warburton')]
df = df[~df.Name.str.contains('Matt Young')]
df = df[~df.Name.str.contains('Matteo Campagna')]
df = df[~df.Name.str.contains('Matteo Scozzarella')]
df = df[~df.Name.str.contains('Matthew Alexander')]
df = df[~df.Name.str.contains('Matthew Cash')]
df = df[~df.Name.str.contains('Matthew Clarke')]
df = df[~df.Name.str.contains('Matthew Dolan')]
df = df[~df.Name.str.contains('Matthew Foulds')]
df = df[~df.Name.str.contains('Matthew Gilks')]
df = df[~df.Name.str.contains('Matthew Millar')]
df = df[~df.Name.str.contains('Matthew Palmer')]
df = df[~df.Name.str.contains('Matthew Platt')]
df = df[~df.Name.str.contains('Matthew Pollock')]
df = df[~df.Name.str.contains('Matthew Smith')]
df = df[~df.Name.str.contains('Matthew Virtue-Thick')]
df = df[~df.Name.str.contains('Matthew Warburton')]
df = df[~df.Name.str.contains('Matthias Heiland')]
df = df[~df.Name.str.contains('Matthias Verreth')]
df = df[~df.Name.str.contains('Matthieu Dossevi')]
df = df[~df.Name.str.contains('Matthieu Saunier')]
df = df[~df.Name.str.contains('Matthis Abline')]
df = df[~df.Name.str.contains('Mattia Pagliuca')]
df = df[~df.Name.str.contains('Mattias Johansson')]
df = df[~df.Name.str.contains('Mattie Pollock')]
df = df[~df.Name.str.contains('Matty Cash')]
df = df[~df.Name.str.contains('Matty Foulds')]
df = df[~df.Name.str.contains('Matty Platt')]
df = df[~df.Name.str.contains('Matty Virtue')]
df = df[~df.Name.str.contains('Matty Wolfe')]
df = df[~df.Name.str.contains('Matz Sels')]
df = df[~df.Name.str.contains('Matz SelsMatz Sels')]
df = df[~df.Name.str.contains('Mauricio')]
df = df[~df.Name.str.contains('Mauricio Arboleda')]
df = df[~df.Name.str.contains('Mauricio Caranta')]
df = df[~df.Name.str.contains('Mauricio Cuero')]
df = df[~df.Name.str.contains('Mauricio Duarte')]
df = df[~df.Name.str.contains('Mauricio Guzman')]
df = df[~df.Name.str.contains('Mauricio Kozlinski')]
df = df[~df.Name.str.contains('MauricioMauricio')]
df = df[~df.Name.str.contains('Maurizio Pochettino')]
df = df[~df.Name.str.contains('Mauro Barraza')]
df = df[~df.Name.str.contains('Mauro Da Luz')]
df = df[~df.Name.str.contains('Mauro dos Santos')]
df = df[~df.Name.str.contains('Mauro Luna Diale')]
df = df[~df.Name.str.contains('Mauro Ortiz')]
df = df[~df.Name.str.contains('Mauro Osores')]
df = df[~df.Name.str.contains('Mauro Valiente')]
df = df[~df.Name.str.contains('Max Harris')]
df = df[~df.Name.str.contains('Max Hunt')]
df = df[~df.Name.str.contains('Max Kilman')]
df = df[~df.Name.str.contains('Maxi Gomez')]
df = df[~df.Name.str.contains('Maxi Moralez')]
df = df[~df.Name.str.contains('Maxim De Cuyper')]
df = df[~df.Name.str.contains('Maxim Deman')]
df = df[~df.Name.str.contains('Maxime Delanghe')]
df = df[~df.Name.str.contains('Maxime Wenssens')]
df = df[~df.Name.str.contains('Maximilian Kilman')]
df = df[~df.Name.str.contains('Maximiliano Caire')]
df = df[~df.Name.str.contains('Maximiliano Cavallotti')]
df = df[~df.Name.str.contains('Maximiliano Cuadra')]
df = df[~df.Name.str.contains('Maximiliano Gagliardo')]
df = df[~df.Name.str.contains('Maximiliano Gomez')]
df = df[~df.Name.str.contains('Maximiliano Luayza')]
df = df[~df.Name.str.contains('Maximiliano Moralez')]
df = df[~df.Name.str.contains('Maximiliano Rogoski')]
df = df[~df.Name.str.contains('MaykeMayke')]
df = df[~df.Name.str.contains('Mees Bakker')]
df = df[~df.Name.str.contains('Mehdi Abeid')]
df = df[~df.Name.str.contains('Mehmet Ali Ulaman')]
df = df[~df.Name.str.contains('Michael Chacon')]
df = df[~df.Name.str.contains('Michael Dawson')]
df = df[~df.Name.str.contains('Michael DeShields')]
df = df[~df.Name.str.contains('Michael Edwards')]
df = df[~df.Name.str.contains('Michael Fabrie')]
df = df[~df.Name.str.contains('Michael Hefele')]
df = df[~df.Name.str.contains('Michael Johnston')]
df = df[~df.Name.str.contains('Michael Jones')]
df = df[~df.Name.str.contains('Michael Kelly')]
df = df[~df.Name.str.contains('Michael Klass')]
df = df[~df.Name.str.contains('Michael Krmencik')]
df = df[~df.Name.str.contains('Michael Lennon')]
df = df[~df.Name.str.contains('Michael Nelson')]
df = df[~df.Name.str.contains('Michael Ruth')]
df = df[~df.Name.str.contains('Michael Santos')]
df = df[~df.Name.str.contains('Michael Schultz')]
df = df[~df.Name.str.contains('Michal Karbownik')]
df = df[~df.Name.str.contains('Michal Sadilek')]
df = df[~df.Name.str.contains('Michel Araujo')]
df = df[~df.Name.str.contains('Michel Balikwisha')]
df = df[~df.Name.str.contains('Michel Herrero')]
df = df[~df.Name.str.contains('Michel Macedo')]
df = df[~df.Name.str.contains('Michel-Ange Balikwisha')]
df = df[~df.Name.str.contains('Miguel Atienza')]
df = df[~df.Name.str.contains('Miguel Cifuentes')]
df = df[~df.Name.str.contains('Miguel Jacquet')]
df = df[~df.Name.str.contains('Miguel Van Damme')]
df = df[~df.Name.str.contains('MiguelMiguel')]
df = df[~df.Name.str.contains('Mihai-Alexandru Dobre')]
df = df[~df.Name.str.contains('Mihail Levashov')]
df = df[~df.Name.str.contains('Mikael Soisalo')]
df = df[~df.Name.str.contains('Mikalay Signevich')]
df = df[~df.Name.str.contains('Mike Hauptmeijer')]
df = df[~df.Name.str.contains('Mike Jones')]
df = df[~df.Name.str.contains('Mike Tresor')]
df = df[~df.Name.str.contains('Mike Tresor Ndayishimiye')]
df = df[~df.Name.str.contains('Mikhail Ageev')]
df = df[~df.Name.str.contains('Mikhail Levashov')]
df = df[~df.Name.str.contains('Mikkel Diskerud')]
df = df[~df.Name.str.contains('Milan De Mey')]
df = df[~df.Name.str.contains('Milan Iloski')]
df = df[~df.Name.str.contains('Milan Tucic')]
df = df[~df.Name.str.contains('Miles Mitchell-Nelson')]
df = df[~df.Name.str.contains('Mirkan Aydin')]
df = df[~df.Name.str.contains('Mitch Pinnock')]
df = df[~df.Name.str.contains('Mitchell Clark')]
df = df[~df.Name.str.contains('Mitchell Curry')]
df = df[~df.Name.str.contains('Mitchell Pinnock')]
df = df[~df.Name.str.contains('Mitchell Roberts')]
df = df[~df.Name.str.contains('Mitchy Ntelo')]
df = df[~df.Name.str.contains('Mitsuki Saito')]
df = df[~df.Name.str.contains('Mix Diskerud')]
df = df[~df.Name.str.contains('Mo AdamsMo Adams')]
df = df[~df.Name.str.contains('Modou Diagne')]
df = df[~df.Name.str.contains('Mohamed Ali-Cho')]
df = df[~df.Name.str.contains('Mohamed Badamosi')]
df = df[~df.Name.str.contains('Mohamed Bouchouari')]
df = df[~df.Name.str.contains('Mohamed Maouche')]
df = df[~df.Name.str.contains('Mohamed Rayhi')]
df = df[~df.Name.str.contains('Mohamed Taabouni')]
df = df[~df.Name.str.contains('Mohammed Sangare')]
df = df[~df.Name.str.contains('Moise Sakava')]
df = df[~df.Name.str.contains('Moises Caicedo')]
df = df[~df.Name.str.contains('Morgan Rogers')]
df = df[~df.Name.str.contains('Moritz Leitner')]
df = df[~df.Name.str.contains('Moritz RÃ¶mling')]
df = df[~df.Name.str.contains('Moritz Schulze')]
df = df[~df.Name.str.contains('Moritz-Broni Kwarteng')]
df = df[~df.Name.str.contains('Mostapha El Kabir')]
df = df[~df.Name.str.contains('Mouez Hassen')]
df = df[~df.Name.str.contains('Mousa Al-Tamari')]
df = df[~df.Name.str.contains('Mousa Suleiman')]
df = df[~df.Name.str.contains('Moussa Wague')]
df = df[~df.Name.str.contains('Moustafa Ava Dongo')]
df = df[~df.Name.str.contains('Mucahit Albayrak')]
df = df[~df.Name.str.contains('Mucahit Can Akcay')]
df = df[~df.Name.str.contains('Muhamed Badamosi')]
df = df[~df.Name.str.contains('Muhamed Besic')]
df = df[~df.Name.str.contains('Muhammadu Faal')]
df = df[~df.Name.str.contains('Muhammed Badamosi')]
df = df[~df.Name.str.contains('Muhammed Egemen Pehlivan')]
df = df[~df.Name.str.contains('Muhammed Emin Sarikaya')]
df = df[~df.Name.str.contains('Muhammed Eren Kiryolcu')]
df = df[~df.Name.str.contains('Muhammed Kerem Akturkoglu')]
df = df[~df.Name.str.contains('Muhammed Sarikaya')]
df = df[~df.Name.str.contains('Muhammed Uzun')]
df = df[~df.Name.str.contains('Muhayer Oktay')]
df = df[~df.Name.str.contains('Mukhammad Sultanov')]
df = df[~df.Name.str.contains('Munir Mohamedi')]
df = df[~df.Name.str.contains('Nano Mesa')]
df = df[~df.Name.str.contains('Naoufal Bannis')]
df = df[~df.Name.str.contains('Nassim Ahmed')]
df = df[~df.Name.str.contains('Nassim Chadli')]
df = df[~df.Name.str.contains('Nathan Baxter')]
df = df[~df.Name.str.contains('Nathan Broadhead')]
df = df[~df.Name.str.contains('Nathan Cameron')]
df = df[~df.Name.str.contains('Nathan Harriel')]
df = df[~df.Name.str.contains('Nathan Markelo')]
df = df[~df.Name.str.contains('Navajo Bakboord')]
df = df[~df.Name.str.contains('Nayair Tiknizyan')]
df = df[~df.Name.str.contains('Nayel Mehssatou')]
df = df[~df.Name.str.contains('Nazareno Solis')]
df = df[~df.Name.str.contains('NeiltonNeilton')]
df = df[~df.Name.str.contains('Neri Bandiera')]
df = df[~df.Name.str.contains('Neri Cardozo')]
df = df[~df.Name.str.contains('Nestor Ortigoza')]
df = df[~df.Name.str.contains('Neven Subotic')]
df = df[~df.Name.str.contains('Neyder Lozano')]
df = df[~df.Name.str.contains('Niall Huggins')]
df = df[~df.Name.str.contains('Nicholas DePuy')]
df = df[~df.Name.str.contains('Nicholas Ioannou')]
df = df[~df.Name.str.contains('Nicholas Lawrence Anwan')]
df = df[~df.Name.str.contains('Nicholas Slonina')]
df = df[~df.Name.str.contains('Nick DePuy')]
df = df[~df.Name.str.contains('Nick Shinton')]
df = df[~df.Name.str.contains('Nicky Hernandez')]
df = df[~df.Name.str.contains('Nicky Law')]
df = df[~df.Name.str.contains('Niclas Thiede')]
df = df[~df.Name.str.contains('Nicolas Ferreyra')]
df = df[~df.Name.str.contains('Nicolas Figal')]
df = df[~df.Name.str.contains('Nicolas Franco')]
df = df[~df.Name.str.contains('Nicolas Gaitan')]
df = df[~df.Name.str.contains('Nicolas Gimenez')]
df = df[~df.Name.str.contains('Nicolas Lamendola')]
df = df[~df.Name.str.contains('Nicolas Linares')]
df = df[~df.Name.str.contains('Nicolas Mana')]
df = df[~df.Name.str.contains('Nicolas Mazzola')]
df = df[~df.Name.str.contains('Nicolas Messiniti')]
df = df[~df.Name.str.contains('Nicolas Morgantini')]
df = df[~df.Name.str.contains('Nicolas Romat')]
df = df[~df.Name.str.contains('Nicolas Romero')]
df = df[~df.Name.str.contains('Nicolas Schiappacasse')]
df = df[~df.Name.str.contains('Nicolas Thaller')]
df = df[~df.Name.str.contains('Nicolas Tie')]
df = df[~df.Name.str.contains('Nikell Touglo')]
df = df[~df.Name.str.contains('Nikita Belousov')]
df = df[~df.Name.str.contains('Nikita Chagrov')]
df = df[~df.Name.str.contains('Nikita Chicherin')]
df = df[~df.Name.str.contains('Nikita Iosifov')]
df = df[~df.Name.str.contains('Nikita Kalugin')]
df = df[~df.Name.str.contains('Nikita Repin')]
df = df[~df.Name.str.contains('Niklas Klinger')]
df = df[~df.Name.str.contains('Niklo Dailly')]
df = df[~df.Name.str.contains('Nikola Katic')]
df = df[~df.Name.str.contains('Nikolai Kemlein')]
df = df[~df.Name.str.contains('Nikolai Rasskazov')]
df = df[~df.Name.str.contains('Nikolai Rehnen')]
df = df[~df.Name.str.contains('Nikolai Zolotov')]
df = df[~df.Name.str.contains('Nikolaos Kainourgios')]
df = df[~df.Name.str.contains('Nikolaos Karelis')]
df = df[~df.Name.str.contains('Nikolaos Zografakis')]
df = df[~df.Name.str.contains('Nikolay Giorgobiani')]
df = df[~df.Name.str.contains('Nikolay Rasskazov')]
df = df[~df.Name.str.contains('Nikolay Titkov')]
df = df[~df.Name.str.contains('Nikolay Zlobin')]
df = df[~df.Name.str.contains('Nikos Kainourgios')]
df = df[~df.Name.str.contains('Nils De Wilde')]
df = df[~df.Name.str.contains('Nino ParaibaNino Paraiba')]
df = df[~df.Name.str.contains('Nkosi Burgess')]
df = df[~df.Name.str.contains('Nnamdi Chinonso Offor')]
df = df[~df.Name.str.contains('Noah Awuku')]
df = df[~df.Name.str.contains('Noah Baumann')]
df = df[~df.Name.str.contains('Noah Ohio')]
df = df[~df.Name.str.contains('Noah Smerdon')]
df = df[~df.Name.str.contains('Nunoo Sarpei')]
df = df[~df.Name.str.contains('Nya Kirby')]
df = df[~df.Name.str.contains('Oladapo Afolayan')]
df = df[~df.Name.str.contains('Oleg Baklov')]
df = df[~df.Name.str.contains('Oleg Danchenko')]
df = df[~df.Name.str.contains('Oleg Dzantiev')]
df = df[~df.Name.str.contains('Oleg Nikolaev')]
df = df[~df.Name.str.contains('Oleg Reabciuk')]
df = df[~df.Name.str.contains('Oleksandr Kaplienko')]
df = df[~df.Name.str.contains('Oli Finney')]
df = df[~df.Name.str.contains('Oliver Banks')]
df = df[~df.Name.str.contains('Oliver Coker')]
df = df[~df.Name.str.contains('Oliver Finney')]
df = df[~df.Name.str.contains('Oliver Lancashire')]
df = df[~df.Name.str.contains('Oliver Myny')]
df = df[~df.Name.str.contains('Oliver Norburn')]
df = df[~df.Name.str.contains('Oliver Turton')]
df = df[~df.Name.str.contains('Olivier Myny')]
df = df[~df.Name.str.contains('Olivier Thill')]
df = df[~df.Name.str.contains('Ollie Banks')]
df = df[~df.Name.str.contains('Ollie Battersby')]
df = df[~df.Name.str.contains('Ollie Norburn')]
df = df[~df.Name.str.contains('Ollie Turton')]
df = df[~df.Name.str.contains('Ollie Younger')]
df = df[~df.Name.str.contains('Olly Lancashire')]
df = df[~df.Name.str.contains('Olly Lee')]
df = df[~df.Name.str.contains('Olufela Olomola')]
df = df[~df.Name.str.contains('Omar Khailoti')]
df = df[~df.Name.str.contains('Omer Beyaz')]
df = df[~df.Name.str.contains('Omer Faruk Beyaz')]
df = df[~df.Name.str.contains('Omer Hanin')]
df = df[~df.Name.str.contains('Omer Sismanoglu')]
df = df[~df.Name.str.contains('Onur Kaya')]
df = df[~df.Name.str.contains('Onur KayaOnur Kaya')]
df = df[~df.Name.str.contains('Opi Edwards')]
df = df[~df.Name.str.contains('Oregan Ravet')]
df = df[~df.Name.str.contains('Orestis Karnezis')]
df = df[~df.Name.str.contains('Oriol Rey')]
df = df[~df.Name.str.contains('Osama Rashid')]
df = df[~df.Name.str.contains('Oscar Benitez')]
df = df[~df.Name.str.contains('Oscar Linner')]
df = df[~df.Name.str.contains('Oscar Ruiz')]
df = df[~df.Name.str.contains('Oscar SchÃ¶nfelder')]
df = df[~df.Name.str.contains('Oskar Buur')]
df = df[~df.Name.str.contains('Ouasim Bouy')]
df = df[~df.Name.str.contains('Oumar Niasse')]
df = df[~df.Name.str.contains('Pablo Aranda')]
df = df[~df.Name.str.contains('Pablo Cortizo')]
df = df[~df.Name.str.contains('Pablo Cuadra')]
df = df[~df.Name.str.contains('Pablo Insua')]
df = df[~df.Name.str.contains('Pablo Rosales')]
df = df[~df.Name.str.contains('Pablo Sabbag')]
df = df[~df.Name.str.contains('Pablo Velazquez')]
df = df[~df.Name.str.contains('Paddy Madden')]
df = df[~df.Name.str.contains('Paddy McNair')]
df = df[~df.Name.str.contains('Panagiotis Armenakas')]
df = df[~df.Name.str.contains('Panos Armenakas')]
df = df[~df.Name.str.contains('Pantelis Chatzidiakos')]
df = df[~df.Name.str.contains('Pantelis Hatzidiakos')]
df = df[~df.Name.str.contains('Paolo Bartolomei')]
df = df[~df.Name.str.contains('Paolo Guerrero')]
df = df[~df.Name.str.contains('Paolo Hurtado')]
df = df[~df.Name.str.contains('Parker Siegfried')]
df = df[~df.Name.str.contains('Pascal Steinwender')]
df = df[~df.Name.str.contains('Patric Klandt')]
df = df[~df.Name.str.contains('Patricio Cucchi')]
df = df[~df.Name.str.contains('Patricio Matricardi')]
df = df[~df.Name.str.contains('Patricio Ostachuk')]
df = df[~df.Name.str.contains('Patricio Toranzo')]
df = df[~df.Name.str.contains('Patrick Jones')]
df = df[~df.Name.str.contains('Patrick McNair')]
df = df[~df.Name.str.contains('Paul Grave')]
df = df[~df.Name.str.contains('Paul-Philipp Besong')]
df = df[~df.Name.str.contains('Pavel Kaloshin')]
df = df[~df.Name.str.contains('Pavel Karasev')]
df = df[~df.Name.str.contains('Pedro Lucas')]
df = df[~df.Name.str.contains('Pedro Ojeda')]
df = df[~df.Name.str.contains('Pedro Perotti')]
df = df[~df.Name.str.contains('Pepe Bonet')]
df = df[~df.Name.str.contains('Per Kristian Bratveit')]
df = df[~df.Name.str.contains('Peru Nolaskoain')]
df = df[~df.Name.str.contains('Peter Urminsky')]
df = df[~df.Name.str.contains('Petrus Boumal')]
df = df[~df.Name.str.contains('Phil Jones')]
df = df[~df.Name.str.contains('Philip Gameli Awuku')]
df = df[~df.Name.str.contains('Philip Mayaka')]
df = df[~df.Name.str.contains('Philip Turpitz')]
df = df[~df.Name.str.contains('Philip Yeboah Ankrah')]
df = df[~df.Name.str.contains('Philipp Heerwagen')]
df = df[~df.Name.str.contains('Philipp Mwene')]
df = df[~df.Name.str.contains('Philipp Tschauner')]
df = df[~df.Name.str.contains('Philippe Sandler')]
df = df[~df.Name.str.contains('Phillipp Mwene')]
df = df[~df.Name.str.contains('Pier Graziano Gori')]
df = df[~df.Name.str.contains('Piero Hincapie')]
df = df[~df.Name.str.contains('Pierre-Yves Polomat')]
df = df[~df.Name.str.contains('PJ Morrison')]
df = df[~df.Name.str.contains('Pol Garcia')]
df = df[~df.Name.str.contains('Racine Coly')]
df = df[~df.Name.str.contains('Rafa De Vita')]
df = df[~df.Name.str.contains('Rafael Borre')]
df = df[~df.Name.str.contains('Rafael Cabral')]
df = df[~df.Name.str.contains('Rafael Ferrario')]
df = df[~df.Name.str.contains('Rafael Ribeiro')]
df = df[~df.Name.str.contains('Rafael Sangiovani')]
df = df[~df.Name.str.contains('Ramiro')]
df = df[~df.Name.str.contains('Ramiro Enrique')]
df = df[~df.Name.str.contains('Ramiro Luna')]
df = df[~df.Name.str.contains('Ramiro Macagno')]
df = df[~df.Name.str.contains('Ramon Arias')]
df = df[~df.Name.str.contains('Ramon Azeez')]
df = df[~df.Name.str.contains('Ramon Folch')]
df = df[~df.Name.str.contains('Ramon ten Hove')]
df = df[~df.Name.str.contains('Raul Aguilera')]
df = df[~df.Name.str.contains('Raul Carnero')]
df = df[~df.Name.str.contains('Raul Lozano')]
df = df[~df.Name.str.contains('Raul Moro')]
df = df[~df.Name.str.contains('Raul Navas')]
df = df[~df.Name.str.contains('Raulzinho')]
df = df[~df.Name.str.contains('Rayan Philippe')]
df = df[~df.Name.str.contains('Rayan Raveloson')]
df = df[~df.Name.str.contains('Rayan Souici')]
df = df[~df.Name.str.contains('Rayhaan Tulloch')]
df = df[~df.Name.str.contains('Raynner Silva')]
df = df[~df.Name.str.contains('Reagan Ogle')]
df = df[~df.Name.str.contains('Recep Niyaz')]
df = df[~df.Name.str.contains('Reda Akbib')]
df = df[~df.Name.str.contains('Reda Khadra')]
df = df[~df.Name.str.contains('Reece Grego-Cox')]
df = df[~df.Name.str.contains('Reece McAlear')]
df = df[~df.Name.str.contains('Regan Slater')]
df = df[~df.Name.str.contains('Renat Dadashov')]
df = df[~df.Name.str.contains('Rene Gilmartin')]
df = df[~df.Name.str.contains('Renzo Bacchia')]
df = df[~df.Name.str.contains('Renzo Giampaoli')]
df = df[~df.Name.str.contains('Renzo Saravia')]
df = df[~df.Name.str.contains('Renzo Tesuri')]
df = df[~df.Name.str.contains('Reuben Reid')]
df = df[~df.Name.str.contains('Richard Prieto')]
df = df[~df.Name.str.contains('Richard Rodrigues')]
df = df[~df.Name.str.contains('Richard Towell')]
df = df[~df.Name.str.contains('Rida Zouhir')]
df = df[~df.Name.str.contains('Rik Vercauteren')]
df = df[~df.Name.str.contains('Rio Hope-Gund')]
df = df[~df.Name.str.contains('Robert Apter')]
df = df[~df.Name.str.contains('Robert Arboleda')]
df = df[~df.Name.str.contains('Robert Castellanos')]
df = df[~df.Name.str.contains('Robert Dickie')]
df = df[~df.Name.str.contains('Robert Hall')]
df = df[~df.Name.str.contains('Robert Piris da Motta')]
df = df[~df.Name.str.contains('Robert Voloder')]
df = df[~df.Name.str.contains('Robert Zwinkels')]
df = df[~df.Name.str.contains('Roberto Bochi')]
df = df[~df.Name.str.contains('Roberto Correa')]
df = df[~df.Name.str.contains('Roberto Jimenez')]
df = df[~df.Name.str.contains('Roberto Navarro')]
df = df[~df.Name.str.contains('Robin Jalving')]
df = df[~df.Name.str.contains('Rodolfo Cardoso')]
df = df[~df.Name.str.contains('RodolfoRodolfo')]
df = df[~df.Name.str.contains('Rodri Sanchez')]
df = df[~df.Name.str.contains('Rodrigo DouradoRodrigo Dourado')]
df = df[~df.Name.str.contains('Rogerio Santos')]
df = df[~df.Name.str.contains('Romain Amalfitano')]
df = df[~df.Name.str.contains('Roman Martinez')]
df = df[~df.Name.str.contains('Romano Postema')]
df = df[~df.Name.str.contains('Ross Doohan')]
df = df[~df.Name.str.contains('Ross Graham')]
df = df[~df.Name.str.contains('Ross MacIver')]
df = df[~df.Name.str.contains('Ross Munro')]
df = df[~df.Name.str.contains('Ross Sinclair')]
df = df[~df.Name.str.contains('Ruben Botta')]
df = df[~df.Name.str.contains('Ruben Del Campo')]
df = df[~df.Name.str.contains('Ruben Fonseca')]
df = df[~df.Name.str.contains('Ruben Pardo')]
df = df[~df.Name.str.contains('Rubin Wilson')]
df = df[~df.Name.str.contains('Rumarn Burrell')]
df = df[~df.Name.str.contains('Ruwen Werthmuller')]
df = df[~df.Name.str.contains('Ryan Campbell-Gordon')]
df = df[~df.Name.str.contains('Ryan Cassidy')]
df = df[~df.Name.str.contains('Ryan Cooney')]
df = df[~df.Name.str.contains('Ryan Duncan')]
df = df[~df.Name.str.contains('Ryan Edmondson')]
df = df[~df.Name.str.contains('Ryan Giles')]
df = df[~df.Name.str.contains('Ryan Hillier')]
df = df[~df.Name.str.contains('Ryan Jones')]
df = df[~df.Name.str.contains('Ryan Longman')]
df = df[~df.Name.str.contains('Ryan Meara')]
df = df[~df.Name.str.contains('Ryan Rydel')]
df = df[~df.Name.str.contains('Ryan Scully')]
df = df[~df.Name.str.contains('Ryan Stirk')]
df = df[~df.Name.str.contains('Saba Sazonov')]
df = df[~df.Name.str.contains('Sabit Abdulai')]
df = df[~df.Name.str.contains('Sacha Bastien')]
df = df[~df.Name.str.contains('Sacha Delaye')]
df = df[~df.Name.str.contains('Said Ahmed Said')]
df = df[~df.Name.str.contains('Said Akhmaev')]
df = df[~df.Name.str.contains('Said Bakari')]
df = df[~df.Name.str.contains('Said-Ali Akhmaev')]
df = df[~df.Name.str.contains('Saidou Sow')]
df = df[~df.Name.str.contains('Saif-Eddine Khaoui')]
df = df[~df.Name.str.contains('Saikou Janneh')]
df = df[~df.Name.str.contains('Salim Ben Seghir')]
df = df[~df.Name.str.contains('Saliou Sane')]
df = df[~df.Name.str.contains('Sam Bell')]
df = df[~df.Name.str.contains('Sam Cartwright')]
df = df[~df.Name.str.contains('Sam Folarin')]
df = df[~df.Name.str.contains('Sam Junqua')]
df = df[~df.Name.str.contains('Sam McQueen')]
df = df[~df.Name.str.contains('Sam Morsy')]
df = df[~df.Name.str.contains('Sam Pearson')]
df = df[~df.Name.str.contains('Sam Sargeant')]
df = df[~df.Name.str.contains('Samed Onur')]
df = df[~df.Name.str.contains('Sami Gokhan Altiparmak')]
df = df[~df.Name.str.contains('Samy Bourard')]
df = df[~df.Name.str.contains('Samy Kehli')]
df = df[~df.Name.str.contains('Samy Mmaee')]
df = df[~df.Name.str.contains('Samy Morsy')]
df = df[~df.Name.str.contains('Santiago Caseres')]
df = df[~df.Name.str.contains('Santiago Colombatto')]
df = df[~df.Name.str.contains('Santiago Gallucci')]
df = df[~df.Name.str.contains('Santiago Garcia')]
df = df[~df.Name.str.contains('Santiago Rodriguez')]
df = df[~df.Name.str.contains('Santiago Silva')]
df = df[~df.Name.str.contains('Santiago Simon')]
df = df[~df.Name.str.contains('Santiago Trellez')]
df = df[~df.Name.str.contains('Scott Caldwell')]
df = df[~df.Name.str.contains('Scott Fox')]
df = df[~df.Name.str.contains('Scott Moloney')]
df = df[~df.Name.str.contains('Scott Pollock')]
df = df[~df.Name.str.contains('Scott Robertson')]
df = df[~df.Name.str.contains('Scott Smith')]
df = df[~df.Name.str.contains('Sebastian Anderson')]
df = df[~df.Name.str.contains('Sebastian Assis')]
df = df[~df.Name.str.contains('Sebastian Blanco')]
df = df[~df.Name.str.contains('Sebastian Breza')]
df = df[~df.Name.str.contains('Sebastian Dubarbier')]
df = df[~df.Name.str.contains('Sebastian Meza')]
df = df[~df.Name.str.contains('Sebastian Ramirez')]
df = df[~df.Name.str.contains('Sebastian Sosa')]
df = df[~df.Name.str.contains('Sebastian Soto')]
df = df[~df.Name.str.contains('Sebastien Thill')]
df = df[~df.Name.str.contains('Sega Coulibaly')]
df = df[~df.Name.str.contains('Sekou Diawara')]
df = df[~df.Name.str.contains('Sepe Elye Wahi')]
df = df[~df.Name.str.contains('Serdar Poyraz')]
df = df[~df.Name.str.contains('Serdar Saatci')]
df = df[~df.Name.str.contains('Sergei Borodin')]
df = df[~df.Name.str.contains('Sergei Makarov')]
df = df[~df.Name.str.contains('Sergei Ryzhikov')]
df = df[~df.Name.str.contains('Sergey Chibisov')]
df = df[~df.Name.str.contains('Sergey Makarov')]
df = df[~df.Name.str.contains('Sergi Garcia')]
df = df[~df.Name.str.contains('Sergio Barcia')]
df = df[~df.Name.str.contains('Sergio Carreira')]
df = df[~df.Name.str.contains('Sergio Cubero')]
df = df[~df.Name.str.contains('Sergio Gonzalez')]
df = df[~df.Name.str.contains('Sergio Ojeda')]
df = df[~df.Name.str.contains('Sergio Quiroga')]
df = df[~df.Name.str.contains('Sergio Romero')]
df = df[~df.Name.str.contains('Sergio Vittor')]
df = df[~df.Name.str.contains('Sergiy Rybalka')]
df = df[~df.Name.str.contains('Seung-Ho Paik')]
df = df[~df.Name.str.contains('Seung-woo Lee')]
df = df[~df.Name.str.contains('Shay Logan')]
df = df[~df.Name.str.contains('Shilow Tracey')]
df = df[~df.Name.str.contains('Shola Shoretire')]
df = df[~df.Name.str.contains('Shoya Nakajima')]
df = df[~df.Name.str.contains('Shurandy Sambo')]
df = df[~df.Name.str.contains('Sibiry Keita')]
df = df[~df.Name.str.contains('Siebren Lathouwers')]
df = df[~df.Name.str.contains('Silas Gnaka')]
df = df[~df.Name.str.contains('Silas Katompa Mvumpa')]
df = df[~df.Name.str.contains('Silas Wamangituka')]
df = df[~df.Name.str.contains('Silas Zehnder')]
df = df[~df.Name.str.contains('Silvinho Esajas')]
df = df[~df.Name.str.contains('Simon Deli')]
df = df[~df.Name.str.contains('Simon Moore')]
df = df[~df.Name.str.contains('Simon Ngapandouetnbu')]
df = df[~df.Name.str.contains('Simon Rhein')]
df = df[~df.Name.str.contains('Simone Aresti')]
df = df[~df.Name.str.contains('Simone Edera')]
df = df[~df.Name.str.contains('Simone Farelli')]
df = df[~df.Name.str.contains('Simone Rabbi')]
df = df[~df.Name.str.contains('Siro Rosane')]
df = df[~df.Name.str.contains('Sofian Chakla')]
df = df[~df.Name.str.contains('Sofiane Bouzian')]
df = df[~df.Name.str.contains('Soufiane Chakla')]
df = df[~df.Name.str.contains('Souleymane Aw')]
df = df[~df.Name.str.contains('Spencer Richey')]
df = df[~df.Name.str.contains('Stan van Dijck')]
df = df[~df.Name.str.contains('Stanislav Ivanov')]
df = df[~df.Name.str.contains('Stanislav Krapukhin')]
df = df[~df.Name.str.contains('Stanislav Shopov')]
df = df[~df.Name.str.contains('Steeve-Mike Eboa Ebongue')]
df = df[~df.Name.str.contains('Stefan Drljaca')]
df = df[~df.Name.str.contains('Stefan Milosevic')]
df = df[~df.Name.str.contains('Stefan van der Lei')]
df = df[~df.Name.str.contains('Stefano Callegari')]
df = df[~df.Name.str.contains('Stepan Oganesyan')]
df = df[~df.Name.str.contains('Stephane Paul Keller')]
df = df[~df.Name.str.contains('Stephen Duke-McKenna')]
df = df[~df.Name.str.contains('Stephen Henderson')]
df = df[~df.Name.str.contains('Stephen Kelly')]
df = df[~df.Name.str.contains('Stephen Mallan')]
df = df[~df.Name.str.contains('Steve Birnbaum')]
df = df[~df.Name.str.contains('Steven Beitashour')]
df = df[~df.Name.str.contains('Steven Defour')]
df = df[~df.Name.str.contains('Stiven Mendoza')]
df = df[~df.Name.str.contains('Stiven Plaza')]
df = df[~df.Name.str.contains('Suk-jae Lee')]
df = df[~df.Name.str.contains('Suleyman Cebeci')]
df = df[~df.Name.str.contains('Taylor Harwood-Bellis')]
df = df[~df.Name.str.contains('Taylor Richards')]
df = df[~df.Name.str.contains('Taylor Seymour')]
df = df[~df.Name.str.contains('Teddy Bartouche')]
df = df[~df.Name.str.contains('Teddy Boulhendi')]
df = df[~df.Name.str.contains('Teden Mengi')]
df = df[~df.Name.str.contains('Tennai Watson')]
df = df[~df.Name.str.contains('Thakgalo Leshabela')]
df = df[~df.Name.str.contains('TharlisTharlis')]
df = df[~df.Name.str.contains('Theo Corbeanu')]
df = df[~df.Name.str.contains('Theo Leoni')]
df = df[~df.Name.str.contains('Thiago Lopes')]
df = df[~df.Name.str.contains('Thiago Maia')]
df = df[~df.Name.str.contains('Thiago Santana')]
df = df[~df.Name.str.contains('Thibang Phete')]
df = df[~df.Name.str.contains('Thibault Tamas')]
df = df[~df.Name.str.contains('Thomas Allan')]
df = df[~df.Name.str.contains('Thomas Basila')]
df = df[~df.Name.str.contains('Thomas Callens')]
df = df[~df.Name.str.contains('Thomas Eastman')]
df = df[~df.Name.str.contains('Thomas Edwards')]
df = df[~df.Name.str.contains('Thomas Hamer')]
df = df[~df.Name.str.contains('Thomas Hasal')]
df = df[~df.Name.str.contains('Thomas McGill')]
df = df[~df.Name.str.contains('Thomas Mehew')]
df = df[~df.Name.str.contains('Thomas Poll')]
df = df[~df.Name.str.contains('Thomas Roberts')]
df = df[~df.Name.str.contains('Thomas Stagg')]
df = df[~df.Name.str.contains('Thomas Toure')]
df = df[~df.Name.str.contains('Thomas Williams')]
df = df[~df.Name.str.contains('Tiago Banega')]
df = df[~df.Name.str.contains('Tiago Ilori')]
df = df[~df.Name.str.contains('Tiago Lopes')]
df = df[~df.Name.str.contains('Tiago Palacios')]
df = df[~df.Name.str.contains('Tibet Oniz')]
df = df[~df.Name.str.contains('Tiernan Parker')]
df = df[~df.Name.str.contains('Tim Coremans')]
df = df[~df.Name.str.contains('Tim Hall')]
df = df[~df.Name.str.contains('Tim Lemperle')]
df = df[~df.Name.str.contains('Tim Maciejewski')]
df = df[~df.Name.str.contains('Timothe Nkada')]
df = df[~df.Name.str.contains('Timothy Eyoma')]
df = df[~df.Name.str.contains('Titas Krapikas')]
df = df[~df.Name.str.contains('Tivonge Rushesha')]
df = df[~df.Name.str.contains('Tobe Leysen')]
df = df[~df.Name.str.contains('Tobias Raschl')]
df = df[~df.Name.str.contains('Tobias Zarate')]
df = df[~df.Name.str.contains('Toby Bancroft')]
df = df[~df.Name.str.contains('Tolaji Bola')]
df = df[~df.Name.str.contains('Tolga Topaloglu')]
df = df[~df.Name.str.contains('Tom Bloxham')]
df = df[~df.Name.str.contains('Tom Chamberlain')]
df = df[~df.Name.str.contains('Tom Collins')]
df = df[~df.Name.str.contains('Tom Eastman')]
df = df[~df.Name.str.contains('Tom Edwards')]
df = df[~df.Name.str.contains('Tom Heaton')]
df = df[~df.Name.str.contains('Tom Hendriks')]
df = df[~df.Name.str.contains('Tom Mickel')]
df = df[~df.Name.str.contains('Tom ScottTom Scott')]
df = df[~df.Name.str.contains('Tom Van Hyfte')]
df = df[~df.Name.str.contains('Tom Weilandt')]
df = df[~df.Name.str.contains('Tom White')]
df = df[~df.Name.str.contains('Tom Wilson')]
df = df[~df.Name.str.contains('Tomas Andrade')]
df = df[~df.Name.str.contains('Tomas Cardona')]
df = df[~df.Name.str.contains('Tomas Cavanagh')]
df = df[~df.Name.str.contains('Tomas Durso')]
df = df[~df.Name.str.contains('Tomas Escalante')]
df = df[~df.Name.str.contains('Tomas Fernandez')]
df = df[~df.Name.str.contains('Tomas Koubek')]
df = df[~df.Name.str.contains('Tomas Marchiori')]
df = df[~df.Name.str.contains('Tomas Martinez')]
df = df[~df.Name.str.contains('Tomas Ortiz')]
df = df[~df.Name.str.contains('Tomas Romero')]
df = df[~df.Name.str.contains('Tomas Rukas')]
df = df[~df.Name.str.contains('Tomas Sandoval')]
df = df[~df.Name.str.contains('Tomas Silva')]
df = df[~df.Name.str.contains('Tommy Conway')]
df = df[~df.Name.str.contains('Tommy Elphick')]
df = df[~df.Name.str.contains('Toni Datkovic')]
df = df[~df.Name.str.contains('Toni Sunjic')]
df = df[~df.Name.str.contains('Tony Alfaro')]
df = df[~df.Name.str.contains('Torben Musel')]
df = df[~df.Name.str.contains('Toto Nsiala')]
df = df[~df.Name.str.contains('Trae Coyle')]
df = df[~df.Name.str.contains('Tristan Dingome')]
df = df[~df.Name.str.contains('Trova Boni')]
df = df[~df.Name.str.contains('Troy Parrott')]
df = df[~df.Name.str.contains('Tunde Owolabi')]
df = df[~df.Name.str.contains('Tunji Akinola')]
df = df[~df.Name.str.contains('Tyler Burey')]
df = df[~df.Name.str.contains('Tyler Cordner')]
df = df[~df.Name.str.contains('Tyler Freeman')]
df = df[~df.Name.str.contains('Tyler French')]
df = df[~df.Name.str.contains('Tyler Magloire')]
df = df[~df.Name.str.contains('Tyler Smith')]
df = df[~df.Name.str.contains('Tyler Wolff')]
df = df[~df.Name.str.contains('Tyreik Wright')]
df = df[~df.Name.str.contains('Tyrese Fornah')]
df = df[~df.Name.str.contains('Tyrese Francois')]
df = df[~df.Name.str.contains('Tyrese Omotoye')]
df = df[~df.Name.str.contains('Tyrone Marsh')]
df = df[~df.Name.str.contains('Tyrone Tormin')]
df = df[~df.Name.str.contains('Tyreece Simpson')]
df = df[~df.Name.str.contains('Ui-jo Hwang')]
df = df[~df.Name.str.contains('Uillian Correia')]
df = df[~df.Name.str.contains('Ulises Segura')]
df = df[~df.Name.str.contains('Umut Bozok')]
df = df[~df.Name.str.contains('Unai Arietaleanizbeaskoa')]
df = df[~df.Name.str.contains('Unai Dufur')]
df = df[~df.Name.str.contains('Uri Rosell')]
df = df[~df.Name.str.contains('Uriel Ramirez')]
df = df[~df.Name.str.contains('Urko Gonzalez')]
df = df[~df.Name.str.contains('Valentin Larralde')]
df = df[~df.Name.str.contains('Valentin Mihaila')]
df = df[~df.Name.str.contains('Valeriu Ciuperca')]
df = df[~df.Name.str.contains('Varazdat Haroyan')]
df = df[~df.Name.str.contains('Vedad Ibisevic')]
df = df[~df.Name.str.contains('Veron Parkes')]
df = df[~df.Name.str.contains('Vic Chambaere')]
df = df[~df.Name.str.contains('Vicente Esquerdo')]
df = df[~df.Name.str.contains('Victor Bobsin')]
df = df[~df.Name.str.contains('Victor Camarasa')]
df = df[~df.Name.str.contains('Victor Chust')]
df = df[~df.Name.str.contains('Victor Jensen')]
df = df[~df.Name.str.contains('Victor Mollejo')]
df = df[~df.Name.str.contains('Victor Ramis')]
df = df[~df.Name.str.contains('Vincent Muller')]
df = df[~df.Name.str.contains('Vincent Schippers')]
df = df[~df.Name.str.contains('Vincenzo Millico')]
df = df[~df.Name.str.contains('Vinicius Zanocelo')]
df = df[~df.Name.str.contains('Virgil Gomis')]
df = df[~df.Name.str.contains('Virgiliu Postolachi')]
df = df[~df.Name.str.contains('Vitor Bueno')]
df = df[~df.Name.str.contains('Vittorio Parigini')]
df = df[~df.Name.str.contains('VladimirVladimir')]
df = df[~df.Name.str.contains('Vladislav Cherny')]
df = df[~df.Name.str.contains('Vladislav Torop')]
df = df[~df.Name.str.contains('Vlasdilav Yakovlev')]
df = df[~df.Name.str.contains('Vukasin Jovanovic')]
df = df[~df.Name.str.contains('Vyacheslav Litvinov')]
df = df[~df.Name.str.contains('Walter Kannemann')]
df = df[~df.Name.str.contains('Walter Montoya')]
df = df[~df.Name.str.contains('Welinton Junior')]
df = df[~df.Name.str.contains('Wenderson Galeno')]
df = df[~df.Name.str.contains('Wes Foderingham')]
df = df[~df.Name.str.contains('Wes McDonald')]
df = df[~df.Name.str.contains('Wesley McDonald')]
df = df[~df.Name.str.contains('Wesley Ribeiro')]
df = df[~df.Name.str.contains('Wilfredo Rivera')]
df = df[~df.Name.str.contains('Will Bapaga')]
df = df[~df.Name.str.contains('Will Collar')]
df = df[~df.Name.str.contains('Will Dennis')]
df = df[~df.Name.str.contains('Will Grigg')]
df = df[~df.Name.str.contains('Will Huffer')]
df = df[~df.Name.str.contains('Will Pulisic')]
df = df[~df.Name.str.contains('Will Smallbone')]
df = df[~df.Name.str.contains('Will Sutton')]
df = df[~df.Name.str.contains('Will Swan')]
df = df[~df.Name.str.contains('William Evans')]
df = df[~df.Name.str.contains('William Fish')]
df = df[~df.Name.str.contains('William Forrester')]
df = df[~df.Name.str.contains('William Grigg')]
df = df[~df.Name.str.contains('William Hondermarck')]
df = df[~df.Name.str.contains('Wouter Biebauw')]
df = df[~df.Name.str.contains('Wouter George')]
df = df[~df.Name.str.contains('Yanis Ammour')]
df = df[~df.Name.str.contains('Yanis Lhery')]
df = df[~df.Name.str.contains('Yanis Saidani')]
df = df[~df.Name.str.contains('Yann Bodiger')]
df = df[~df.Name.str.contains('Yannick Brugger')]
df = df[~df.Name.str.contains('YanYan')]
df = df[~df.Name.str.contains('Yasin Davus')]
df = df[~df.Name.str.contains('Yasin Oztekin')]
df = df[~df.Name.str.contains('Yassin Fekir')]
df = df[~df.Name.str.contains('Yassin Oukili')]
df = df[~df.Name.str.contains('Yassine Bounou')]
df = df[~df.Name.str.contains('Yavuz Bugra Boyar')]
df = df[~df.Name.str.contains('Yavuz Ulas Genc')]
df = df[~df.Name.str.contains('Yaya Toure')]
df = df[~df.Name.str.contains('Yayah Kallon')]
df = df[~df.Name.str.contains('Yeiler Goez')]
df = df[~df.Name.str.contains('Yeison Gordillo')]
df = df[~df.Name.str.contains('Yildirim Mert Ã‡etin')]
df = df[~df.Name.str.contains('Yoan Cardinale')]
df = df[~df.Name.str.contains('Yoann Etienne')]
df = df[~df.Name.str.contains('Yohan Croizet')]
df = df[~df.Name.str.contains('Yonathan Cabral')]
df = df[~df.Name.str.contains('Younes Belhanda')]
df = df[~df.Name.str.contains('Youri Schoonderwaldt')]
df = df[~df.Name.str.contains('Zvonimir Sarlija')]
df = df[~df.Name.str.contains('Zlatko Tripic')]
df = df[~df.Name.str.contains('Ziyad Al-Oyouni')]
df = df[~df.Name.str.contains('Zito Luvumbo')]
df = df[~df.Name.str.contains('Zeno Rossi')]
df = df[~df.Name.str.contains('Zeki Yildirim')]
df = df[~df.Name.str.contains('Zeki Fryers')]
df = df[~df.Name.str.contains('Zeki Celik')]
df = df[~df.Name.str.contains('Zehn Mohammed')]
df = df[~df.Name.str.contains('Zech Medley')]
df = df[~df.Name.str.contains('Ze Oliveira')]
df = df[~df.Name.str.contains('Ze IvaldoZe Ivaldo')]
df = df[~df.Name.str.contains('Ze GabrielZe Gabriel')]
df = df[~df.Name.str.contains('Zdenek Zlamal')]
df = df[~df.Name.str.contains('Zanka')]
df = df[~df.Name.str.contains('Zakaria Diallo')]
df = df[~df.Name.str.contains('Zak Dearnley')]
df = df[~df.Name.str.contains('Zaid Romero')]
df = df[~df.Name.str.contains('Zaid Al Hussaini')]
df = df[~df.Name.str.contains('Zachary Dearnley')]
df = df[~df.Name.str.contains('Zach Jeacock')]
df = df[~df.Name.str.contains('Zac Williams')]
df = df[~df.Name.str.contains('Zac SmithZac Smith')]
df = df[~df.Name.str.contains('Dimitri Chistyakov')]
df = df[~df.Name.str.contains('Dimitri German')]
df = df[~df.Name.str.contains('Dimitri Khomich')]
df = df[~df.Name.str.contains('Dimitri Nazarov')]
df = df[~df.Name.str.contains('Dimitri Poloz')]
df = df[~df.Name.str.contains('Dimitri Rybchinskiy')]
df = df[~df.Name.str.contains('Dimitri Shomko')]
df = df[~df.Name.str.contains('Dimitri Skopintsev')]
df = df[~df.Name.str.contains('Dimitri Stotskiy')]
df = df[~df.Name.str.contains('Dimitri Sysuev')]
df = df[~df.Name.str.contains('Dimitri Tikhiy')]
df = df[~df.Name.str.contains('Dimitri Zhivoglyadov')]
df = df[~df.Name.str.contains('Dino Visser')]
df = df[~df.Name.str.contains('Diogo BarbosaDiogo Barbosa')]
df = df[~df.Name.str.contains('Dixon Renteria')]
df = df[~df.Name.str.contains('Djibril Diani')]
df = df[~df.Name.str.contains('Dmitri Efremov')]
df = df[~df.Name.str.contains('Dmitri German')]
df = df[~df.Name.str.contains('Dmitri Markitesov')]
df = df[~df.Name.str.contains('Dmitri Rybchinsky')]
df = df[~df.Name.str.contains('Dmitri Skopintsev')]
df = df[~df.Name.str.contains('Dmitri Sysuev')]
df = df[~df.Name.str.contains('Dmitri Tikhiy')]
df = df[~df.Name.str.contains('Dmitri Vershkov')]
df = df[~df.Name.str.contains('Dmitrii Merenchukov')]
df = df[~df.Name.str.contains('Dmitriy Chistyakov')]
df = df[~df.Name.str.contains('Dmitriy Poloz')]
df = df[~df.Name.str.contains('Dmitriy Shomko')]
df = df[~df.Name.str.contains('Dmitriy Stotskiy')]
df = df[~df.Name.str.contains('Dmitriy Tarasov')]
df = df[~df.Name.str.contains('Dmitry Zhivoglyadov')]
df = df[~df.Name.str.contains('DodoDodo')]
df = df[~df.Name.str.contains('Dominik Reimann')]
df = df[~df.Name.str.contains('Dominik Szoboszlai')]
df = df[~df.Name.str.contains('Dragan Lausberg')]
df = df[~df.Name.str.contains('Drake Callender')]
df = df[~df.Name.str.contains('Drissa Camara')]
df = df[~df.Name.str.contains('EÃ±aut Mendia')]
df = df[~df.Name.str.contains('Edgar Pacheco')]
df = df[~df.Name.str.contains('Edi Semedo')]
df = df[~df.Name.str.contains('EdimarEdimar')]
df = df[~df.Name.str.contains('Edson MontaÃ±o')]
df = df[~df.Name.str.contains('EdsonEdson')]
df = df[~df.Name.str.contains('Eduardo dos Santos Haesler')]
df = df[~df.Name.str.contains('Edwar Lopez')]
df = df[~df.Name.str.contains('Elias Pereyra')]
df = df[~df.Name.str.contains('EliasElias')]
df = df[~df.Name.str.contains('Elton Kabangu')]
df = df[~df.Name.str.contains('Elye WahiElye Wahi')]
df = df[~df.Name.str.contains('Emanuel Insua')]
df = df[~df.Name.str.contains('EmanuelEmanuel')]
df = df[~df.Name.str.contains('Emil Riis Jakobsen')]
df = df[~df.Name.str.contains('Emil RiisEmil Riis')]
df = df[~df.Name.str.contains('Emiliano Gonzalez')]
df = df[~df.Name.str.contains('Emiliano Insua')]
df = df[~df.Name.str.contains('Emre Pehlivan')]
df = df[~df.Name.str.contains('Enzo YbaÃ±ez')]
df = df[~df.Name.str.contains('Eric Lopez')]
df = df[~df.Name.str.contains('ErickErick')]
df = df[~df.Name.str.contains('Erickson Gallardo')]
df = df[~df.Name.str.contains('Erik DueÃ±as')]
df = df[~df.Name.str.contains('Ertugrul Cetin')]
df = df[~df.Name.str.contains('Esteban Rolon')]
df = df[~df.Name.str.contains('Eunan O\'Kane')]
df = df[~df.Name.str.contains('Evan N\'Dicka')]
df = df[~df.Name.str.contains('Exequiel Zeballos')]
df = df[~df.Name.str.contains('Ezekiel Fryers')]
df = df[~df.Name.str.contains('Ezequiel Cerica')]
df = df[~df.Name.str.contains('Ezequiel Fernandez')]
df = df[~df.Name.str.contains('Fabian Cubero')]
df = df[~df.Name.str.contains('Fabian Noguera')]
df = df[~df.Name.str.contains('Fabio Abreu')]
df = df[~df.Name.str.contains('Fabio ChinaFabio China')]
df = df[~df.Name.str.contains('Fabio SantosFabio Santos')]
df = df[~df.Name.str.contains('Fabio Vazquez')]
df = df[~df.Name.str.contains('Fabricio Dominguez')]
df = df[~df.Name.str.contains('Facundo Fernandez')]
df = df[~df.Name.str.contains('Facundo Garces')]
df = df[~df.Name.str.contains('Facundo Sanchez')]
df = df[~df.Name.str.contains('Faiz Selemani')]
df = df[~df.Name.str.contains('Federico Martinez')]
df = df[~df.Name.str.contains('Fehmi KocFehmi Koc')]
df = df[~df.Name.str.contains('Fehmi Mert Gunok')]
df = df[~df.Name.str.contains('Fernando Roman')]
df = df[~df.Name.str.contains('Ferris N\'Goma')]
df = df[~df.Name.str.contains('Francesco D\'Alterio')]
df = df[~df.Name.str.contains('Francisco Ramirez')]
df = df[~df.Name.str.contains('Franco Gonzalez')]
df = df[~df.Name.str.contains('Franco Perez')]
df = df[~df.Name.str.contains('Gabriel AlvesGabriel Alves')]
df = df[~df.Name.str.contains('Gabriel Chapeco')]
df = df[~df.Name.str.contains('Gabriel Diaz')]
df = df[~df.Name.str.contains('Gabriel Falcao')]
df = df[~df.Name.str.contains('Gabriel Graciani')]
df = df[~df.Name.str.contains('Gabriel NogaGabriel Noga')]
df = df[~df.Name.str.contains('Gabriel PereiraGabriel Pereira')]
df = df[~df.Name.str.contains('Gaetan Poussin')]
df = df[~df.Name.str.contains('Gaston Benavidez')]
df = df[~df.Name.str.contains('Gaston Diaz')]
df = df[~df.Name.str.contains('Gaston Fernandez')]
df = df[~df.Name.str.contains('Gaston Gerzel')]
df = df[~df.Name.str.contains('Gaston Gomez')]
df = df[~df.Name.str.contains('Gaston Gonzalez')]
df = df[~df.Name.str.contains('Gaston Suso')]
df = df[~df.Name.str.contains('Gaston Torres')]
df = df[~df.Name.str.contains('George Nurse')]
df = df[~df.Name.str.contains('Gerald Sithole')]
df = df[~df.Name.str.contains('Giles Phillips')]
df = df[~df.Name.str.contains('Gonzalo Abrego')]
df = df[~df.Name.str.contains('Gonzalo Asis')]
df = df[~df.Name.str.contains('Gonzalo Rodriguez')]
df = df[~df.Name.str.contains('Gonzalo Veron')]
df = df[~df.Name.str.contains('Gorkem Can')]
df = df[~df.Name.str.contains('Guido Vadala')]
df = df[~df.Name.str.contains('Guillermo Fernandez')]
df = df[~df.Name.str.contains('Gustavo Ramalho')]
df = df[~df.Name.str.contains('Guven Yalcin')]
df = df[~df.Name.str.contains('Habib Keita')]
df = df[~df.Name.str.contains('Hamza Masoudi')]
df = df[~df.Name.str.contains('Hannibal Mejbri')]
df = df[~df.Name.str.contains('Hans Nicolussi Caviglia')]
df = df[~df.Name.str.contains('Harrison Paton')]
df = df[~df.Name.str.contains('Hatem Elhamed')]
df = df[~df.Name.str.contains('HeberHeber')]
df = df[~df.Name.str.contains('Hector Canteros')]
df = df[~df.Name.str.contains('HelioHelio')]
df = df[~df.Name.str.contains('HenriHenri')]
df = df[~df.Name.str.contains('Henry Burnett')]
df = df[~df.Name.str.contains('Hernan Burbano')]
df = df[~df.Name.str.contains('Hernan Lamberti')]
df = df[~df.Name.str.contains('Hernan Lopez')]
df = df[~df.Name.str.contains('Heung-min Son')]
df = df[~df.Name.str.contains('HiagoHiago')]
df = df[~df.Name.str.contains('Hugo Ekitike')]
df = df[~df.Name.str.contains('Hugo Silva')]
df = df[~df.Name.str.contains('Huseyin Biler')]
df = df[~df.Name.str.contains('Ibrahima Camara')]
df = df[~df.Name.str.contains('Ignacio Cechi')]
df = df[~df.Name.str.contains('Ignacio Gariglio')]
df = df[~df.Name.str.contains('Ignacio Lago')]
df = df[~df.Name.str.contains('Ignacio Maestro Puch')]
df = df[~df.Name.str.contains('Ignacio Miramon')]
df = df[~df.Name.str.contains('Igor CariusIgor Carius')]
df = df[~df.Name.str.contains('Igor JuliaoIgor Juliao')]
df = df[~df.Name.str.contains('Igor Julio')]
df = df[~df.Name.str.contains('Igor TorresIgor Torres')]
df = df[~df.Name.str.contains('Ike OparaIke Opara')]
df = df[~df.Name.str.contains('Il-lok Yun')]
df = df[~df.Name.str.contains('Ilya Shkurin')]
df = df[~df.Name.str.contains('Ilya Vorobjov')]
df = df[~df.Name.str.contains('Imanol Machuca')]
df = df[~df.Name.str.contains('In-beom Hwang')]
df = df[~df.Name.str.contains('Indiana Vassilev')]
df = df[~df.Name.str.contains('Innes Cameron')]
df = df[~df.Name.str.contains('Ionut Nedelcearu')]
df = df[~df.Name.str.contains('Irakoze Donasiyano')]
df = df[~df.Name.str.contains('Isaac Boehmer')]
df = df[~df.Name.str.contains('Isaac Hutchinson')]
df = df[~df.Name.str.contains('Isaac Karamoko')]
df = df[~df.Name.str.contains('IsaqueIsaque')]
df = df[~df.Name.str.contains('Islamzhan Nasyrov')]
df = df[~df.Name.str.contains('Isma Ruiz')]
df = df[~df.Name.str.contains('Ismael Saibari')]
df = df[~df.Name.str.contains('Ismael Sow')]
df = df[~df.Name.str.contains('Ismail Aissati')]
df = df[~df.Name.str.contains('Ismail Yuksek')]
df = df[~df.Name.str.contains('Israel Damonte')]
df = df[~df.Name.str.contains('Ivan Angulo')]
df = df[~df.Name.str.contains('Ivan Arboleda')]
df = df[~df.Name.str.contains('Ivan Budachev')]
df = df[~df.Name.str.contains('Ivan Kuzmichev')]
df = df[~df.Name.str.contains('IvoneiIvonei')]
df = df[~df.Name.str.contains('Iyenoma Udogie')]
df = df[~df.Name.str.contains('Jabez Makanda')]
df = df[~df.Name.str.contains('Jack Bolton')]
df = df[~df.Name.str.contains('Jack Burroughs')]
df = df[~df.Name.str.contains('Jack Chambers')]
df = df[~df.Name.str.contains('Jack Currie')]
df = df[~df.Name.str.contains('Jack Davies')]
df = df[~df.Name.str.contains('Jack de Vries')]
df = df[~df.Name.str.contains('Jack Hindle')]
df = df[~df.Name.str.contains('Jack Madelin')]
df = df[~df.Name.str.contains('Jack Newman')]
df = df[~df.Name.str.contains('Jack Rodwell')]
df = df[~df.Name.str.contains('Jack Roles')]
df = df[~df.Name.str.contains('Jack Ruddy')]
df = df[~df.Name.str.contains('Jack SimsJack Sims')]
df = df[~df.Name.str.contains('Jack Stretton')]
df = df[~df.Name.str.contains('Jack Vale')]
df = df[~df.Name.str.contains('Jack Young')]
df = df[~df.Name.str.contains('Jackson')]
df = df[~df.Name.str.contains('Jackson Conway')]
df = df[~df.Name.str.contains('Jacob Akanyirige')]
df = df[~df.Name.str.contains('Jacob Bancroft')]
df = df[~df.Name.str.contains('Jacob Barrett Laursen')]
df = df[~df.Name.str.contains('Jacob Gratton')]
df = df[~df.Name.str.contains('Jacob Greene')]
df = df[~df.Name.str.contains('Jacob Laursen')]
df = df[~df.Name.str.contains('Jacopo Dezi')]
df = df[~df.Name.str.contains('Jacopo Segre')]
df = df[~df.Name.str.contains('Jaden Philogene-Bidace')]
df = df[~df.Name.str.contains('JadsomJadsom')]
df = df[~df.Name.str.contains('Jae-ik Lee')]
df = df[~df.Name.str.contains('JailsonJailson')]
df = df[~df.Name.str.contains('Jaime Alvarado')]
df = df[~df.Name.str.contains('Jaimy Kroesen')]
df = df[~df.Name.str.contains('JairJair')]
df = df[~df.Name.str.contains('Jake Davidson')]
df = df[~df.Name.str.contains('Jake HullJake Hull')]
df = df[~df.Name.str.contains('Jake PeckJake Peck')]
df = df[~df.Name.str.contains('Jake Scrimshaw')]
df = df[~df.Name.str.contains('Jake Turner')]
df = df[~df.Name.str.contains('Jake Vokins')]
df = df[~df.Name.str.contains('Jakob Busk')]
df = df[~df.Name.str.contains('Jan Olschowsky')]
df = df[~df.Name.str.contains('Jan RasJan Ras')]
df = df[~df.Name.str.contains('JandersonJanderson')]
df = df[~df.Name.str.contains('JandreiJandrei')]
df = df[~df.Name.str.contains('Janeiler Rivas')]
df = df[~df.Name.str.contains('Jaquite')]
df = df[~df.Name.str.contains('Jason Meerstadt')]
df = df[~df.Name.str.contains('Jason Pendant')]
df = df[~df.Name.str.contains('Jason Steele')]
df = df[~df.Name.str.contains('Jay Henderson')]
df = df[~df.Name.str.contains('Jean Zimmer')]
df = df[~df.Name.str.contains('Jean-Aniel Assi')]
df = df[~df.Name.str.contains('Jean-Eudes Aholou')]
df = df[~df.Name.str.contains('Jeff Dewsnup')]
df = df[~df.Name.str.contains('Jeffrey de Lange')]
df = df[~df.Name.str.contains('Jeison Chala')]
df = df[~df.Name.str.contains('Jeison Lucumi')]
df = df[~df.Name.str.contains('Jeizon Ramirez')]
df = df[~df.Name.str.contains('JemersonJemerson')]
df = df[~df.Name.str.contains('JenisonJenison')]
df = df[~df.Name.str.contains('Jens Castrop')]
df = df[~df.Name.str.contains('Jens Craenmehr')]
df = df[~df.Name.str.contains('Jens Grahl')]
df = df[~df.Name.str.contains('Jeremy Perbet')]
df = df[~df.Name.str.contains('Jeronimo Pourtau')]
df = df[~df.Name.str.contains('Jesus Ferreira')]
df = df[~df.Name.str.contains('Jetro Willems')]
df = df[~df.Name.str.contains('Jevan Anderson')]
df = df[~df.Name.str.contains('Jhonatan Candia')]
df = df[~df.Name.str.contains('Jhonathan Agudelo')]
df = df[~df.Name.str.contains('JhonnathanJhonnathan')]
df = df[~df.Name.str.contains('Jimmy Adrian Kaparos')]
df = df[~df.Name.str.contains('Jimmy Kaparos')]
df = df[~df.Name.str.contains('Joachim Imbrechts')]
df = df[~df.Name.str.contains('Joan Mazzaco')]
df = df[~df.Name.str.contains('Joao Carlos Teixeira')]
df = df[~df.Name.str.contains('Joao CarlosJoao Carlos')]
df = df[~df.Name.str.contains('Joao Ferreira')]
df = df[~df.Name.str.contains('Joao GomesJoao Gomes')]
df = df[~df.Name.str.contains('Joao Jaquite')]
df = df[~df.Name.str.contains('Joao Klauss')]
df = df[~df.Name.str.contains('Joao LopesJoao Lopes')]
df = df[~df.Name.str.contains('Joao MarioJoao MarioRight Midfiel')]
df = df[~df.Name.str.contains('Joao Monteiro')]
df = df[~df.Name.str.contains('Joao Queiros')]
df = df[~df.Name.str.contains('Joao Rodriguez')]
df = df[~df.Name.str.contains('Joao Teixeira')]
df = df[~df.Name.str.contains('Joao VictorJoao Victor')]
df = df[~df.Name.str.contains('Joaquin Arzura')]
df = df[~df.Name.str.contains('Joaquin Blazquez')]
df = df[~df.Name.str.contains('Joaquin Hass')]
df = df[~df.Name.str.contains('Joaquin Mendive')]
df = df[~df.Name.str.contains('Joaquin Novillo')]
df = df[~df.Name.str.contains('Joaquin Varela')]
df = df[~df.Name.str.contains('JobsonJobson')]
df = df[~df.Name.str.contains('Joe Bendik')]
df = df[~df.Name.str.contains('Joe DodooJoe Dodoo')]
df = df[~df.Name.str.contains('Joe DunneJoe Dunne')]
df = df[~df.Name.str.contains('Joe HartJoe Hart')]
df = df[~df.Name.str.contains('Joe Maguire')]
df = df[~df.Name.str.contains('Joe Mason')]
df = df[~df.Name.str.contains('Joe NealJoe Neal')]
df = df[~df.Name.str.contains('Joe Rafferty')]
df = df[~df.Name.str.contains('Joe Scally')]
df = df[~df.Name.str.contains('Joe Willock')]
df = df[~df.Name.str.contains('Joe Woodiwiss')]
df = df[~df.Name.str.contains('Joel Tagueu')]
df = df[~df.Name.str.contains('Joey DeZart')]
df = df[~df.Name.str.contains('Johannes Eggestein')]
df = df[~df.Name.str.contains('John Anthony Brooks')]
df = df[~df.Name.str.contains('John Joe Patrick Finn')]
df = df[~df.Name.str.contains('John KennedyJohn Kennedy')]
df = df[~df.Name.str.contains('JohnJohn')]
df = df[~df.Name.str.contains('JohnnyJohnny')]
df = df[~df.Name.str.contains('JoilsonJoilson')]
df = df[~df.Name.str.contains('JoJo')]
df = df[~df.Name.str.contains('Jomaine Consbruch')]
df = df[~df.Name.str.contains('Jon BellJon Bell')]
df = df[~df.Name.str.contains('Jon Flanagan')]
df = df[~df.Name.str.contains('Jon Pacheco Dozagarat')]
df = df[~df.Name.str.contains('Jon Stead')]
df = df[~df.Name.str.contains('Jonah Osabutey')]
df = df[~df.Name.str.contains('Jonas Aguirre')]
df = df[~df.Name.str.contains('Jonas Carls')]
df = df[~df.Name.str.contains('JonasJonas')]
df = df[~df.Name.str.contains('Jonatan Gomez')]
df = df[~df.Name.str.contains('Jonatan Maidana')]
df = df[~df.Name.str.contains('Jonathan Bell')]
df = df[~df.Name.str.contains('Jonathan Bolingi')]
df = df[~df.Name.str.contains('Jonathan Cabral')]
df = df[~df.Name.str.contains('Jonathan Howson')]
df = df[~df.Name.str.contains('Jonathan Klinsmann')]
df = df[~df.Name.str.contains('Jonathan Mulder')]
df = df[~df.Name.str.contains('Jonathan Russell')]
df = df[~df.Name.str.contains('Jonathan Torres')]
df = df[~df.Name.str.contains('Jonathan Zacaria')]
df = df[~df.Name.str.contains('Jonny Howson')]
df = df[~df.Name.str.contains('Jonny Otto')]
df = df[~df.Name.str.contains('Jorben Vanhulle')]
df = df[~df.Name.str.contains('Jordan Barnett')]
df = df[~df.Name.str.contains('Jordan Beyer')]
df = df[~df.Name.str.contains('Jordan Eastham')]
df = df[~df.Name.str.contains('Jordan Flores')]
df = df[~df.Name.str.contains('Jordan Garrick')]
df = df[~df.Name.str.contains('Jordan Green')]
df = df[~df.Name.str.contains('Jordan Harvey')]
df = df[~df.Name.str.contains('Jordan Holsgrove')]
df = df[~df.Name.str.contains('Jordan Michael Flores')]
df = df[~df.Name.str.contains('Jordan Perruzza')]
df = df[~df.Name.str.contains('Jordan Roberts')]
df = df[~df.Name.str.contains('Jordan Thomas')]
df = df[~df.Name.str.contains('Jordan van der Gaag')]
df = df[~df.Name.str.contains('JordanJordan')]
df = df[~df.Name.str.contains('Jordon Garrick')]
df = df[~df.Name.str.contains('Jordon Thompson')]
df = df[~df.Name.str.contains('Jordy Schelfhout')]
df = df[~df.Name.str.contains('Jordy Wehrmann')]
df = df[~df.Name.str.contains('Joren DomJoren Dom')]
df = df[~df.Name.str.contains('Jores Rahou')]
df = df[~df.Name.str.contains('Jorge Aruga')]
df = df[~df.Name.str.contains('Jorge Caceres')]
df = df[~df.Name.str.contains('Jorge De Olivera')]
df = df[~df.Name.str.contains('Jorge Figal')]
df = df[~df.Name.str.contains('Jorge Moreira')]
df = df[~df.Name.str.contains('Jorge Ortega')]
df = df[~df.Name.str.contains('Jorge Ortiz')]
df = df[~df.Name.str.contains('Jorge Pereira')]
df = df[~df.Name.str.contains('Jorge Pereyra Diaz')]
df = df[~df.Name.str.contains('Jorge Saenz')]
df = df[~df.Name.str.contains('Joris Gnagnon')]
df = df[~df.Name.str.contains('Jorn Berkhout')]
df = df[~df.Name.str.contains('Joroen Dom')]
df = df[~df.Name.str.contains('Jorrit Smeets')]
df = df[~df.Name.str.contains('Jose Cifuentes')]
df = df[~df.Name.str.contains('Jose Devecchi')]
df = df[~df.Name.str.contains('Jose Gimenez')]
df = df[~df.Name.str.contains('Jose Gonzalez')]
df = df[~df.Name.str.contains('Jose Luis Fernandez')]
df = df[~df.Name.str.contains('Jose Luis Gomez')]
df = df[~df.Name.str.contains('Jose Luis Rodriguez')]
df = df[~df.Name.str.contains('Jose Luis Sinisterra')]
df = df[~df.Name.str.contains('Jose Manuel Fontan')]
df = df[~df.Name.str.contains('Jose Manuel Lopez')]
df = df[~df.Name.str.contains('Jose Marcelo')]
df = df[~df.Name.str.contains('Jose Mari')]
df = df[~df.Name.str.contains('Jose Maria Gimenez')]
df = df[~df.Name.str.contains('Jose MariJose Mari')]
df = df[~df.Name.str.contains('Jose Martinez')]
df = df[~df.Name.str.contains('Jose Moya')]
df = df[~df.Name.str.contains('JoryJory')]
df = df[~df.Name.str.contains('JosefJosef')]
df = df[~df.Name.str.contains('Joseph Amoah')]
df = df[~df.Name.str.contains('Joseph Collinge')]
df = df[~df.Name.str.contains('Joseph Dodoo')]
df = df[~df.Name.str.contains('Joseph Hungbo')]
df = df[~df.Name.str.contains('Joseph Maguire')]
df = df[~df.Name.str.contains('Joseph Marie Minala')]
df = df[~df.Name.str.contains('Joseph Martin Paintsil')]
df = df[~df.Name.str.contains('Joseph Paintsil')]
df = df[~df.Name.str.contains('Joseph Rafferty')]
df = df[~df.Name.str.contains('Joseph Willock')]
df = df[~df.Name.str.contains('Josete Miranda')]
df = df[~df.Name.str.contains('Josh Andrews')]
df = df[~df.Name.str.contains('Josh Atencio')]
df = df[~df.Name.str.contains('Josh Bauer')]
df = df[~df.Name.str.contains('Josh Coburn')]
df = df[~df.Name.str.contains('Josh EarlJosh Earl')]
df = df[~df.Name.str.contains('Josh Emmanuel')]
df = df[~df.Name.str.contains('Josh Ginnelly')]
df = df[~df.Name.str.contains('Josh Gould')]
df = df[~df.Name.str.contains('Josh KeyJosh Key')]
df = df[~df.Name.str.contains('Josh Knight')]
df = df[~df.Name.str.contains('Josh Lillis')]
df = df[~df.Name.str.contains('Josh Sargent')]
df = df[~df.Name.str.contains('Josh Vickers')]
df = df[~df.Name.str.contains('Joshua Atencio')]
df = df[~df.Name.str.contains('Joshua Earl')]
df = df[~df.Name.str.contains('Joshua Emmanuel')]
df = df[~df.Name.str.contains('Joshua Griffiths')]
df = df[~df.Name.str.contains('Joshua Key')]
df = df[~df.Name.str.contains('Joshua McPake')]
df = df[~df.Name.str.contains('Joshua Render')]
df = df[~df.Name.str.contains('Joshua Sargent')]
df = df[~df.Name.str.contains('Joshua Wehking')]
df = df[~df.Name.str.contains('Jospin Nshimirimana')]
df = df[~df.Name.str.contains('Josue Ayala')]
df = df[~df.Name.str.contains('Jota Peleteiro')]
df = df[~df.Name.str.contains('Jovanny Bolivar')]
df = df[~df.Name.str.contains('Joye Dezart')]
df = df[~df.Name.str.contains('JT Marcinkowski')]
df = df[~df.Name.str.contains('Juan Ãlvarez')]
df = df[~df.Name.str.contains('Juan Antonini')]
df = df[~df.Name.str.contains('Juan Bolado')]
df = df[~df.Name.str.contains('Juan Brunetta')]
df = df[~df.Name.str.contains('Juan Carlos Portillo')]
df = df[~df.Name.str.contains('Juan Castilla')]
df = df[~df.Name.str.contains('Juan Cavallaro')]
df = df[~df.Name.str.contains('Juan Caviglia')]
df = df[~df.Name.str.contains('Juan Cruz Komar')]
df = df[~df.Name.str.contains('Juan Espinola')]
df = df[~df.Name.str.contains('Juan Franzoni')]
df = df[~df.Name.str.contains('Juan Fuentes')]
df = df[~df.Name.str.contains('Juan Gonzalez')]
df = df[~df.Name.str.contains('Juan Iglesias')]
df = df[~df.Name.str.contains('Juan Ignacio Mendez')]
df = df[~df.Name.str.contains('Juan Ignacio Nardoni')]
df = df[~df.Name.str.contains('Juan Infante')]
df = df[~df.Name.str.contains('Juan Jose Caceres')]
df = df[~df.Name.str.contains('Juan Kaprof')]
df = df[~df.Name.str.contains('Juan Komar')]
df = df[~df.Name.str.contains('Juan Lucero')]
df = df[~df.Name.str.contains('Juan Manuel Cruz')]
df = df[~df.Name.str.contains('Juan Manuel Cuesta')]
df = df[~df.Name.str.contains('Juan Manuel Insaurralde')]
df = df[~df.Name.str.contains('Juan Martin Lucero')]
df = df[~df.Name.str.contains('Juan Miritello')]
df = df[~df.Name.str.contains('Juan Nardoni')]
df = df[~df.Name.str.contains('Juan Pablo Barinaga')]
df = df[~df.Name.str.contains('Juan Pablo Freytes')]
df = df[~df.Name.str.contains('Juan Pablo Ramirez')]
df = df[~df.Name.str.contains('Juan Pablo Romero')]
df = df[~df.Name.str.contains('Juan Pacchini')]
df = df[~df.Name.str.contains('Juan Roman Pucheta')]
df = df[~df.Name.str.contains('Juan Sforza')]
df = df[~df.Name.str.contains('Juan Vieyra')]
df = df[~df.Name.str.contains('Juan Zurbriggen')]
df = df[~df.Name.str.contains('Jugurtha Hamroun')]
df = df[~df.Name.str.contains('Jules-Anthony Vilsaint')]
df = df[~df.Name.str.contains('Julian Aude')]
df = df[~df.Name.str.contains('Julian Chicco')]
df = df[~df.Name.str.contains('Julian Dunn')]
df = df[~df.Name.str.contains('Julian Frommann')]
df = df[~df.Name.str.contains('Julian Krahl')]
df = df[~df.Name.str.contains('Julian Lopez')]
df = df[~df.Name.str.contains('Julian Malatini')]
df = df[~df.Name.str.contains('Julian Marcioni')]
df = df[~df.Name.str.contains('Julian Navas')]
df = df[~df.Name.str.contains('Julian Pollersbeck')]
df = df[~df.Name.str.contains('Julian Schieber')]
df = df[~df.Name.str.contains('Julian Velazquez')]
df = df[~df.Name.str.contains('Julio Gonzalez')]
df = df[~df.Name.str.contains('Julio Villalba')]
df = df[~df.Name.str.contains('Juninho Capixaba')]
df = df[~df.Name.str.contains('JunioJunio')]
df = df[~df.Name.str.contains('Junior Alonso')]
df = df[~df.Name.str.contains('Junior AlonsoJunior Alonso')]
df = df[~df.Name.str.contains('Junior Dina Ebimbe')]
df = df[~df.Name.str.contains('Junior Hoilett')]
df = df[~df.Name.str.contains('Justin Che')]
df = df[~df.Name.str.contains('Justin Haak')]
df = df[~df.Name.str.contains('Justin Johnson')]
df = df[~df.Name.str.contains('Justin Rennicks')]
df = df[~df.Name.str.contains('Justin vom Steeg')]
df = df[~df.Name.str.contains('Justo Giani')]
df = df[~df.Name.str.contains('Karim Zedadka')]
df = df[~df.Name.str.contains('Kartal Kayra Yilmaz')]
df = df[~df.Name.str.contains('Kasper Lunding')]
df = df[~df.Name.str.contains('Kasper Lunding Jakobsen')]
df = df[~df.Name.str.contains('Kemal Ruzgar')]
df = df[~df.Name.str.contains('KennedyKennedy')]
df = df[~df.Name.str.contains('Kevin Boma')]
df = df[~df.Name.str.contains('Kevin MalthusKevin Malthus')]
df = df[~df.Name.str.contains('Kevin Zenon')]
df = df[~df.Name.str.contains('Kibe')]
df = df[~df.Name.str.contains('Kiernan Dewsbury-Hall')]
df = df[~df.Name.str.contains('KikasKikas')]
df = df[~df.Name.str.contains('Koke Vegas')]
df = df[~df.Name.str.contains('Kosuke Nakamura')]
df = df[~df.Name.str.contains('Kouadio Dabila')]
df = df[~df.Name.str.contains('Kouadio-Yves Dabila')]
df = df[~df.Name.str.contains('Kuki Zalazar')]
df = df[~df.Name.str.contains('Kuku Emmanuel Fidelis')]
df = df[~df.Name.str.contains('Kwasi Okyere Wriedt')]
df = df[~df.Name.str.contains('LaercioLaercio')]
df = df[~df.Name.str.contains('Lars Lukas Mai')]
df = df[~df.Name.str.contains('Lautaro de Leon')]
df = df[~df.Name.str.contains('LazaroLazaro')]
df = df[~df.Name.str.contains('Leandro Contin')]
df = df[~df.Name.str.contains('Leandro Fernandez')]
df = df[~df.Name.str.contains('Lee GrantLee Grant')]
df = df[~df.Name.str.contains('Lee Seung-Woo')]
df = df[~df.Name.str.contains('Leo BorgesLeo Borges')]
df = df[~df.Name.str.contains('Leo Gomes')]
df = df[~df.Name.str.contains('Leo NatelLeo Natel')]
df = df[~df.Name.str.contains('Leo Ostigard')]
df = df[~df.Name.str.contains('Leo PassosLeo Passos')]
df = df[~df.Name.str.contains('Leo SantosLeo Santos')]
df = df[~df.Name.str.contains('Leon Chambers-Parillon')]
df = df[~df.Name.str.contains('Leon Sabua')]
df = df[~df.Name.str.contains('Leon Schaffran')]
df = df[~df.Name.str.contains('Leon Thomson King')]
df = df[~df.Name.str.contains('Leonel Gonzalez')]
df = df[~df.Name.str.contains('Levi Ntumba')]
df = df[~df.Name.str.contains('LiggerLigger')]
df = df[~df.Name.str.contains('Lillo Guarneri')]
df = df[~df.Name.str.contains('LinconLincon')]
df = df[~df.Name.str.contains('Lino Kasten')]
df = df[~df.Name.str.contains('Lion Lauberbach')]
df = df[~df.Name.str.contains('Lisandro Alzugaray')]
df = df[~df.Name.str.contains('Logan Delaurier-Chaubet')]
df = df[~df.Name.str.contains('LohanLohan')]
df = df[~df.Name.str.contains('Lois Diony')]
df = df[~df.Name.str.contains('Luan PolliLuan Polli')]
df = df[~df.Name.str.contains('Luca Martinez Dupuy')]
df = df[~df.Name.str.contains('Ludvig Ohman')]
df = df[~df.Name.str.contains('Luis Leal')]
df = df[~df.Name.str.contains('Luis Miguel Rodriguez')]
df = df[~df.Name.str.contains('Luis Ojeda')]
df = df[~df.Name.str.contains('Luis PhelipeLuis Phelipe')]
df = df[~df.Name.str.contains('Luis Rodriguez')]
df = df[~df.Name.str.contains('Luis Vazquez')]
df = df[~df.Name.str.contains('Luiz Otavio')]
df = df[~df.Name.str.contains('Lukas Fahrnberger')]
df = df[~df.Name.str.contains('Lukas Hinterseer')]
df = df[~df.Name.str.contains('Lukas Hornicek')]
df = df[~df.Name.str.contains('LumorLumor')]
df = df[~df.Name.str.contains('Mads Bidstrup')]
df = df[~df.Name.str.contains('Maikel van der Werff')]
df = df[~df.Name.str.contains('Majid Hosseini')]
df = df[~df.Name.str.contains('Maksim Kutovoi')]
df = df[~df.Name.str.contains('Maksim Martusevich')]
df = df[~df.Name.str.contains('Maksim Turischev')]
df = df[~df.Name.str.contains('Maksim Volodko')]
df = df[~df.Name.str.contains('Marc Auge')]
df = df[~df.Name.str.contains('Marc Baro')]
df = df[~df.Name.str.contains('Marc Olivier Doue')]
df = df[~df.Name.str.contains('Marcelo Meli')]
df = df[~df.Name.str.contains('Marcelo RyanMarcelo Ryan')]
df = df[~df.Name.str.contains('Marcio AzevedoMarcio Azevedo')]
df = df[~df.Name.str.contains('Marcio Mossoro')]
df = df[~df.Name.str.contains('Marco Torsiglieri')]
df = df[~df.Name.str.contains('Marcos Abreliano')]
df = df[~df.Name.str.contains('Marcos AndreMarcos Andre')]
df = df[~df.Name.str.contains('Marcos Angeleri')]
df = df[~df.Name.str.contains('Marcos BritoMarcos Brito')]
df = df[~df.Name.str.contains('Marcos de Sousa')]
df = df[~df.Name.str.contains('Marcos Landaburu')]
df = df[~df.Name.str.contains('Marcos Ledesma')]
df = df[~df.Name.str.contains('Marcos PauloMarcos Paulo')]
df = df[~df.Name.str.contains('Marcos Peano')]
df = df[~df.Name.str.contains('Marcos RochaMarcos Rocha')]
df = df[~df.Name.str.contains('Marcos Sanchez')]
df = df[~df.Name.str.contains('Mariano Diaz')]
df = df[~df.Name.str.contains('Mariano Vazquez')]
df = df[~df.Name.str.contains('Mark Kosznovszky')]
df = df[~df.Name.str.contains('Markus Schubert')]
df = df[~df.Name.str.contains('Marky Delgado')]
df = df[~df.Name.str.contains('Marley Ake')]
df = df[~df.Name.str.contains('Marley Marshall')]
df = df[~df.Name.str.contains('Martin Garay')]
df = df[~df.Name.str.contains('Martin Garcia')]
df = df[~df.Name.str.contains('Matheo Vroman')]
df = df[~df.Name.str.contains('Matheus FerrazMatheus Ferraz')]
df = df[~df.Name.str.contains('Matheus Kayser')]
df = df[~df.Name.str.contains('Matheus Martins')]
df = df[~df.Name.str.contains('Matheus Mascarenhas')]
df = df[~df.Name.str.contains('Matheus MendesMatheus Mendes')]
df = df[~df.Name.str.contains('Matheus NogueiraMatheus Nogueira')]
df = df[~df.Name.str.contains('Matheus Teixeira')]
df = df[~df.Name.str.contains('Mathias Abero')]
df = df[~df.Name.str.contains('Mathias Olivera')]
df = df[~df.Name.str.contains('Mathias Suarez')]
df = df[~df.Name.str.contains('Matias Belloso')]
df = df[~df.Name.str.contains('Matias Borgogno')]
df = df[~df.Name.str.contains('Matias Cordoba')]
df = df[~df.Name.str.contains('Matias Escudero')]
df = df[~df.Name.str.contains('Matias Fritzler')]
df = df[~df.Name.str.contains('Matias Garcia')]
df = df[~df.Name.str.contains('Matias Gomez')]
df = df[~df.Name.str.contains('Matias Gonzalez')]
df = df[~df.Name.str.contains('Matias Laba')]
df = df[~df.Name.str.contains('Matias Moya')]
df = df[~df.Name.str.contains('Matias Palavecino')]
df = df[~df.Name.str.contains('Matias Pellegrini')]
df = df[~df.Name.str.contains('Matias Pisano')]
df = df[~df.Name.str.contains('Matias Ramirez')]
df = df[~df.Name.str.contains('Matias Rodriguez')]
df = df[~df.Name.str.contains('Matias Sosa')]
df = df[~df.Name.str.contains('Matias Tagliamonte')]
df = df[~df.Name.str.contains('Matias Tissera')]
df = df[~df.Name.str.contains('Matias Villarreal')]
df = df[~df.Name.str.contains('Mauro Ortiz')]
df = df[~df.Name.str.contains('Max ClarkMax Clark')]
df = df[~df.Name.str.contains('Maxwell Haygarth')]
df = df[~df.Name.str.contains('Maycon CleitonMaycon Cleiton')]
df = df[~df.Name.str.contains('Mehmet Can Aydin')]
df = df[~df.Name.str.contains('Mehmet Erdem Ugurlu')]
df = df[~df.Name.str.contains('Mehmet Goktug Bakirbas')]
df = df[~df.Name.str.contains('Mehmet Umut Nayir')]
df = df[~df.Name.str.contains('Melvin Minter')]
df = df[~df.Name.str.contains('Melvin Ramusovic')]
df = df[~df.Name.str.contains('Melvin Sitti')]
df = df[~df.Name.str.contains('Meme Placca Fessou')]
df = df[~df.Name.str.contains('Menno Koch')]
df = df[~df.Name.str.contains('Mert Cetin')]
df = df[~df.Name.str.contains('Mert Gunok')]
df = df[~df.Name.str.contains('Mert KulaMert Kula')]
df = df[~df.Name.str.contains('Mert Sarikus')]
df = df[~df.Name.str.contains('Merveille Goblet')]
df = df[~df.Name.str.contains('Micah Obiero')]
df = df[~df.Name.str.contains('Michael Chacon')]
df = df[~df.Name.str.contains('Michael Lang')]
df = df[~df.Name.str.contains('Michel Araujo')]
df = df[~df.Name.str.contains('Michel Herrero')]
df = df[~df.Name.str.contains('Michel Macedo')]
df = df[~df.Name.str.contains('MichelMichel')]
df = df[~df.Name.str.contains('Miguel Barbieri')]
df = df[~df.Name.str.contains('Miguel Berry')]
df = df[~df.Name.str.contains('Mikey Ambrose')]
df = df[~df.Name.str.contains('Mikey Johnston')]
df = df[~df.Name.str.contains('Mohamed-Ali Cho')]
df = df[~df.Name.str.contains('Mohammed Kamara')]
df = df[~df.Name.str.contains('Mohammed Kesselly Kamara')]
df = df[~df.Name.str.contains('Mohammed Rabiu Al-Hassan')]
df = df[~df.Name.str.contains('Moises Caicedo')]
df = df[~df.Name.str.contains('Morgan Boyes')]
df = df[~df.Name.str.contains('Morgan Feeney')]
df = df[~df.Name.str.contains('Morgan Ferrier')]
df = df[~df.Name.str.contains('Morgan Penfold')]
df = df[~df.Name.str.contains('Mustafa Seyhan')]
df = df[~df.Name.str.contains('Mustafa Yumlu')]
df = df[~df.Name.str.contains('Mustapha Bundu')]
df = df[~df.Name.str.contains('Mustapha Olagunju')]
df = df[~df.Name.str.contains('Muzaffer Kablan')]
df = df[~df.Name.str.contains('Myles Judd')]
df = df[~df.Name.str.contains('Nacho Fernandez')]
df = df[~df.Name.str.contains('Nacho Martinez')]
df = df[~df.Name.str.contains('Nadir Ciftci')]
df = df[~df.Name.str.contains('Nadir Zeineddin')]
df = df[~df.Name.str.contains('Nahuel Arena')]
df = df[~df.Name.str.contains('Nahuel Banegas')]
df = df[~df.Name.str.contains('Nahuel Bustos')]
df = df[~df.Name.str.contains('Nahuel Iribarren')]
df = df[~df.Name.str.contains('Nair Tiknizyan')]
df = df[~df.Name.str.contains('Nando Pijnaker')]
df = df[~df.Name.str.contains('Nelson Iseguan')]
df = df[~df.Name.str.contains('Nestor Ortigoza')]
df = df[~df.Name.str.contains('Nico Gordon')]
df = df[~df.Name.str.contains('Nico Williams')]
df = df[~df.Name.str.contains('Nicola Borghetto')]
df = df[~df.Name.str.contains('Nicola Ravaglia')]
df = df[~df.Name.str.contains('Nicola Zalewski')]
df = df[~df.Name.str.contains('Nicolas Andereggen')]
df = df[~df.Name.str.contains('Nicolas Avellaneda')]
df = df[~df.Name.str.contains('Nicolas Blandi')]
df = df[~df.Name.str.contains('Nicolas Burgy')]
df = df[~df.Name.str.contains('Nicolas Contin')]
df = df[~df.Name.str.contains('Nicolas Correa')]
df = df[~df.Name.str.contains('Nicolas Domingo')]
df = df[~df.Name.str.contains('Nicolas Femia')]
df = df[~df.Name.str.contains('Nicolas Fernandez Mercau')]
df = df[~df.Name.str.contains('Nicolas Ferreyra')]
df = df[~df.Name.str.contains('Nicolas Figal')]
df = df[~df.Name.str.contains('Nicolas Gimenez')]
df = df[~df.Name.str.contains('Nicolas Lamendola')]
df = df[~df.Name.str.contains('Nicolas Linares')]
df = df[~df.Name.str.contains('Nicolas Mana')]
df = df[~df.Name.str.contains('Nicolas Mazzola')]
df = df[~df.Name.str.contains('Nicolas Messiniti')]
df = df[~df.Name.str.contains('Nicolas Morgantini')]
df = df[~df.Name.str.contains('Nicolas Romat')]
df = df[~df.Name.str.contains('Nicolas Romero')]
df = df[~df.Name.str.contains('Nicolas Schiappacasse')]
df = df[~df.Name.str.contains('Nicolas Thaller')]
df = df[~df.Name.str.contains('Nicolas Zalazar')]
df = df[~df.Name.str.contains('Nicolo Zaniolo')]
df = df[~df.Name.str.contains('Nigel Bertrams')]
df = df[~df.Name.str.contains('Nino ParaibaNino Paraiba')]
df = df[~df.Name.str.contains('O\'Shane Stewart')]
df = df[~df.Name.str.contains('Obite Evan Ndicka')]
df = df[~df.Name.str.contains('Odion Ighalo')]
df = df[~df.Name.str.contains('Odise Roshi')]
df = df[~df.Name.str.contains('Ody AlfaOdy Alfa')]
df = df[~df.Name.str.contains('Oguzhan Capar')]
df = df[~df.Name.str.contains('Oier Sanjurjo')]
df = df[~df.Name.str.contains('OierOier')]
df = df[~df.Name.str.contains('Okechukwu Azubuike')]
df = df[~df.Name.str.contains('Owen Windsor')]
df = df[~df.Name.str.contains('Owura Edwards')]
df = df[~df.Name.str.contains('Ozan Can Oruc')]
df = df[~df.Name.str.contains('Ozzie Cisneros')]
df = df[~df.Name.str.contains('Pablo DyegoPablo Dyego')]
df = df[~df.Name.str.contains('Pablo MaiaPablo Maia')]
df = df[~df.Name.str.contains('Pablo Minissale')]
df = df[~df.Name.str.contains('Pablo OroPablo Oro')]
df = df[~df.Name.str.contains('Pablo PardalPablo Pardal')]
df = df[~df.Name.str.contains('Pablo Velazquez')]
df = df[~df.Name.str.contains('Paulo MirandaPaulo Miranda')]
df = df[~df.Name.str.contains('Paulo VictorPaulo Victor')]
df = df[~df.Name.str.contains('Pedro RangelPedro Rangel')]
df = df[~df.Name.str.contains('Pedro Rocha')]
df = df[~df.Name.str.contains('PeglowPeglow')]
df = df[~df.Name.str.contains('PerottiPerotti')]
df = df[~df.Name.str.contains('Philip Turpitz')]
df = df[~df.Name.str.contains('Piero Hincapie')]
df = df[~df.Name.str.contains('Pol Garcia')]
df = df[~df.Name.str.contains('PopPop')]
df = df[~df.Name.str.contains('Praxedes')]
df = df[~df.Name.str.contains('Quentin Westberg')]
df = df[~df.Name.str.contains('Radoslaw Murawski')]
df = df[~df.Name.str.contains('Rafa SilvaRafa SilvaRight Midfiel')]
df = df[~df.Name.str.contains('Rafael Borre')]
df = df[~df.Name.str.contains('Rafael Garcia')]
df = df[~df.Name.str.contains('Rafael GrampolaRafael Grampola')]
df = df[~df.Name.str.contains('Rafael Longuine')]
df = df[~df.Name.str.contains('Ramon Arias')]
df = df[~df.Name.str.contains('Ramon Azeez')]
df = df[~df.Name.str.contains('Ramon Folch')]
df = df[~df.Name.str.contains('Ramon ten Hove')]
df = df[~df.Name.str.contains('RanielRaniel')]
df = df[~df.Name.str.contains('Raul Carnero')]
df = df[~df.Name.str.contains('Raul Lozano')]
df = df[~df.Name.str.contains('Raul Navas')]
df = df[~df.Name.str.contains('Raul Silva')]
df = df[~df.Name.str.contains('Ravel Morrison')]
df = df[~df.Name.str.contains('Reinaldo Lenis')]
df = df[~df.Name.str.contains('Renan Ribeiro')]
df = df[~df.Name.str.contains('Reynaldo')]
df = df[~df.Name.str.contains('Ricardo Velho')]
df = df[~df.Name.str.contains('Richecard Richard')]
df = df[~df.Name.str.contains('Richie Towell')]
df = df[~df.Name.str.contains('Rob Dickie')]
df = df[~df.Name.str.contains('Rob Elliot')]
df = df[~df.Name.str.contains('Rob HallRob Hall')]
df = df[~df.Name.str.contains('Robby McCrorie')]
df = df[~df.Name.str.contains('Rober Correa')]
df = df[~df.Name.str.contains('RobersonRoberson')]
df = df[~df.Name.str.contains('Roberto Jimenez')]
df = df[~df.Name.str.contains('Robinho')]
df = df[~df.Name.str.contains('Robson ReisRobson Reis')]
df = df[~df.Name.str.contains('RodineiRodinei')]
df = df[~df.Name.str.contains('Rodri Sanchez')]
df = df[~df.Name.str.contains('Rodrigo Arciero')]
df = df[~df.Name.str.contains('Rodrigo Gomez')]
df = df[~df.Name.str.contains('Rodrigo Herrera')]
df = df[~df.Name.str.contains('Rodrigo LindosoRodrigo Lindoso')]
df = df[~df.Name.str.contains('Rodrigo MoledoRodrigo Moledo')]
df = df[~df.Name.str.contains('Rodrigo Nestor')]
df = df[~df.Name.str.contains('Rodrigo Salinas')]
df = df[~df.Name.str.contains('Rodrigo Sanchez')]
df = df[~df.Name.str.contains('Rodrigo VarandaRodrigo Varanda')]
df = df[~df.Name.str.contains('Rogerio Santos')]
df = df[~df.Name.str.contains('Rolando Garcia')]
df = df[~df.Name.str.contains('Roman Eremenko')]
df = df[~df.Name.str.contains('Roman Martinez')]
df = df[~df.Name.str.contains('Romulo Otero')]
df = df[~df.Name.str.contains('Ron-Thorben Hoffmann')]
df = df[~df.Name.str.contains('Ruben Botta')]
df = df[~df.Name.str.contains('Ruben Fonseca')]
df = df[~df.Name.str.contains('Ruben Pardo')]
df = df[~df.Name.str.contains('Ruwen Werthmuller')]
df = df[~df.Name.str.contains('Saba Lobjanidze')]
df = df[~df.Name.str.contains('Saba Lobzhanidze')]
df = df[~df.Name.str.contains('Said Bakari')]
df = df[~df.Name.str.contains('Saidou Sow')]
df = df[~df.Name.str.contains('Saliou Sane')]
df = df[~df.Name.str.contains('Salva Ferrer')]
df = df[~df.Name.str.contains('Salvador Ferrer')]
df = df[~df.Name.str.contains('Sam BoothSam Booth')]
df = df[~df.Name.str.contains('Sam ByramSam Byram')]
df = df[~df.Name.str.contains('Sammie McLeod')]
df = df[~df.Name.str.contains('Sammy Bossut')]
df = df[~df.Name.str.contains('Sander Coopman')]
df = df[~df.Name.str.contains('Santiago Caseres')]
df = df[~df.Name.str.contains('Santiago Garcia')]
df = df[~df.Name.str.contains('Santiago Rodriguez')]
df = df[~df.Name.str.contains('Santiago Simon')]
df = df[~df.Name.str.contains('Santiago Trellez')]
df = df[~df.Name.str.contains('Santos Borre')]
df = df[~df.Name.str.contains('SaraviaSaravia')]
df = df[~df.Name.str.contains('Sarpreet Singh')]
df = df[~df.Name.str.contains('Saul Salcedo')]
df = df[~df.Name.str.contains('Sean Slaven')]
df = df[~df.Name.str.contains('Sebastian Assis')]
df = df[~df.Name.str.contains('Sebastian Blanco')]
df = df[~df.Name.str.contains('Sebastian Dubarbier')]
df = df[~df.Name.str.contains('Sebastian Meza')]
df = df[~df.Name.str.contains('Sebastian Ramirez')]
df = df[~df.Name.str.contains('Sebastian Sosa')]
df = df[~df.Name.str.contains('Sebastien Thill')]
df = df[~df.Name.str.contains('Sega Coulibaly')]
df = df[~df.Name.str.contains('Senne Lammens')]
df = df[~df.Name.str.contains('Sergi Garcia')]
df = df[~df.Name.str.contains('Sergio Benito')]
df = df[~df.Name.str.contains('Sergio Gonzalez')]
df = df[~df.Name.str.contains('Seydou SySeydou Sy')]
df = df[~df.Name.str.contains('Seyed Majid Hosseini')]
df = df[~df.Name.str.contains('Shadrach Ogie')]
df = df[~df.Name.str.contains('Shahriar Moghanlou')]
df = df[~df.Name.str.contains('Sidnei Tavares')]
df = df[~df.Name.str.contains('Siro Rosane')]
df = df[~df.Name.str.contains('SivaldoSivaldo')]
df = df[~df.Name.str.contains('Slobodan Rajkovic')]
df = df[~df.Name.str.contains('Son Heung-Min')]
df = df[~df.Name.str.contains('Sonny CoxSonny Cox')]
df = df[~df.Name.str.contains('Sonny Graham')]
df = df[~df.Name.str.contains('Sori ManeSori Mane')]
df = df[~df.Name.str.contains('Soslan Takazov')]
df = df[~df.Name.str.contains('Suleyman Cebeci')]
df = df[~df.Name.str.contains('Sylian Mokono')]
df = df[~df.Name.str.contains('TabataTabata')]
df = df[~df.Name.str.contains('Tahvon Campbell')]
df = df[~df.Name.str.contains('TallesTalles')]
df = df[~df.Name.str.contains('Tariq Uwakwe')]
df = df[~df.Name.str.contains('Tein Troost')]
df = df[~df.Name.str.contains('Telmo ArcanjoTelmo Arcanjo')]
df = df[~df.Name.str.contains('Theo Leoni')]
df = df[~df.Name.str.contains('Thiago RibeiroThiago Ribeiro')]
df = df[~df.Name.str.contains('Thomas O\'Connor')]
df = df[~df.Name.str.contains('Thomas Ortega')]
df = df[~df.Name.str.contains('Thomas Ouwejan')]
df = df[~df.Name.str.contains('Thomas Toure')]
df = df[~df.Name.str.contains('ThulerThuler')]
df = df[~df.Name.str.contains('Thuram')]
df = df[~df.Name.str.contains('ThyllerThyller')]
df = df[~df.Name.str.contains('TiagoTiago')]
df = df[~df.Name.str.contains('Tibet Oniz')]
df = df[~df.Name.str.contains('Timmy Abraham')]
df = df[~df.Name.str.contains('Timothe Nkada')]
df = df[~df.Name.str.contains('Tobias Zarate')]
df = df[~df.Name.str.contains('Tom HamerTom Hamer')]
df = df[~df.Name.str.contains('Tomas Andrade')]
df = df[~df.Name.str.contains('Tomas Cardona')]
df = df[~df.Name.str.contains('Tomas Cavanagh')]
df = df[~df.Name.str.contains('Tomas Durso')]
df = df[~df.Name.str.contains('Tomas Escalante')]
df = df[~df.Name.str.contains('Tomas Fernandez')]
df = df[~df.Name.str.contains('Tomas Marchiori')]
df = df[~df.Name.str.contains('Tomas Martinez')]
df = df[~df.Name.str.contains('Tomas Ortiz')]
df = df[~df.Name.str.contains('Tomas Romero')]
df = df[~df.Name.str.contains('Tomas Sandoval')]
df = df[~df.Name.str.contains('Tomas Silva')]
df = df[~df.Name.str.contains('TQ AddyTQ Addy')]
df = df[~df.Name.str.contains('Tyreece John-Jules')]
df = df[~df.Name.str.contains('Ugo Bertelli')]
df = df[~df.Name.str.contains('Uriel Ramirez')]
df = df[~df.Name.str.contains('Vani Da Silva')]
df = df[~df.Name.str.contains('Vaso Vasic')]
df = df[~df.Name.str.contains('Vaughn Covil')]
df = df[~df.Name.str.contains('Victor Camarasa')]
df = df[~df.Name.str.contains('Victor FerrazVictor Ferraz')]
df = df[~df.Name.str.contains('Victor KawaVictor Kawaattac')]
df = df[~df.Name.str.contains('Victor YanVictor Yan')]
df = df[~df.Name.str.contains('VinaVina')]
df = df[~df.Name.str.contains('Vincent Muller')]
df = df[~df.Name.str.contains('Vinicius BalieiroVinicius Balieiro')]
df = df[~df.Name.str.contains('Vinicius FoguinhoVinicius Foguinho')]
df = df[~df.Name.str.contains('Vinicius MingottiVinicius Mingotti')]
df = df[~df.Name.str.contains('Vinicius SilvestreVinicius Silvestre')]
df = df[~df.Name.str.contains('Vitor Hugodefenc')]
df = df[~df.Name.str.contains('Vitor JacareVitor Jacare')]
df = df[~df.Name.str.contains('Vitor RicardoVitor Ricardo')]
df = df[~df.Name.str.contains('WalberWalber')]
df = df[~df.Name.str.contains('WalceWalce')]
df = df[~df.Name.str.contains('WallisonWallison')]
df = df[~df.Name.str.contains('Walter Andrade')]
df = df[~df.Name.str.contains('Walter Kannemann')]
df = df[~df.Name.str.contains('Warner Hahn')]
df = df[~df.Name.str.contains('Wayne Hennessey')]
df = df[~df.Name.str.contains('Wayne Rooney')]
df = df[~df.Name.str.contains('Welinton Junior')]
df = df[~df.Name.str.contains('Will VintWill Vint')]
df = df[~df.Name.str.contains('William MatheusWilliam Matheus')]
df = df[~df.Name.str.contains('William OliveiraWilliam Oliveira')]
df = df[~df.Name.str.contains('William Simba')]
df = df[~df.Name.str.contains('William Smallbone')]
df = df[~df.Name.str.contains('William Togui')]
df = df[~df.Name.str.contains('Williams Barlasina')]
df = df[~df.Name.str.contains('Willian MaranhaoWillian Maranhao')]
df = df[~df.Name.str.contains('Wim Vanmarsenille')]
df = df[~df.Name.str.contains('Wiris Gustavo')]
df = df[~df.Name.str.contains('Wisdom Amey')]
df = df[~df.Name.str.contains('Woo-yeong Jeong')]
df = df[~df.Name.str.contains('XadasXadas')]
df = df[~df.Name.str.contains('Xander Severina')]
df = df[~df.Name.str.contains('Xandro Schenk')]
df = df[~df.Name.str.contains('XavierXavier')]
df = df[~df.Name.str.contains('Yahya Soumare')]
df = df[~df.Name.str.contains('Yair Arismendi')]
df = df[~df.Name.str.contains('Yan SasseYan Sasse')]
df = df[~df.Name.str.contains('Yanis Lhery')]
df = df[~df.Name.str.contains('Yann M\'Vila')]
df = df[~df.Name.str.contains('Yasin Oztekin')]
df = df[~df.Name.str.contains('Yaya Soumare')]
df = df[~df.Name.str.contains('Yildirim Mert Ã‡etin')]
df = df[~df.Name.str.contains('Younes Belhanda')]
df = df[~df.Name.str.contains('Youssef Ait Bennasser')]
df = df[~df.Name.str.contains('Youssuf Sylla')]
df = df[~df.Name.str.contains('Yuki Kobayashi')]
df = df[~df.Name.str.contains('Yun Il-Lok')]
df = df[~df.Name.str.contains('Yunus Bahadir')]
df = df[~df.Name.str.contains('Yusuf Ozdemir')]




df['Name'] = df['Name'].str.replace('Aaron-Salem Boupendza','Aaron Boupendza')
df['Name'] = df['Name'].str.replace('Abdallahi Mahmoud','Abdallah Ali Mohamed')
df['Name'] = df['Name'].str.replace('Abdel Jalil Zaim Idriss Medioub','Abdel Medioub')
df['Name'] = df['Name'].str.replace('Abdenasser El Khayati','Nasser El Khayati')
df['Name'] = df['Name'].str.replace('Abdoulaye Diaby','Abdoulay Diaby')
df['Name'] = df['Name'].str.replace('Abdoulaye Ba','Abdoulaye Ba')
df['Name'] = df['Name'].str.replace('Abdoulaye Bamba','Abdoulaye Bamba')
df['Name'] = df['Name'].str.replace('Abdulia Sanyang','Abdoulie Sanyang')
df['Name'] = df['Name'].str.replace('Abdul Manaf Nurudeen','Abdul Nurudeen')
df['Name'] = df['Name'].str.replace('douglasdouglas','douglas')
df['Name'] = df['Name'].str.replace('ewertonewerton','ewerton')
df['Name'] = df['Name'].str.replace('Amad ','Amad Diallo')
df['Name'] = df['Name'].str.replace('Abdullahi Ibrahim Alhassan','Ibrahim Abdullahi Alhassan')
df['Name'] = df['Name'].str.replace('Adrien Pérez','Adrien Perez')


prices = prices[~prices.name.str.contains('Abdulkadir Tasdan')]
prices = prices[~prices.name.str.contains('Abdullah Celik')]
prices = prices[~prices.name.str.contains('Abel Argañaraz')]
prices = prices[~prices.name.str.contains('Abel Bustos')]
prices = prices[~prices.name.str.contains('Abou Ouattara')]
prices = prices[~prices.name.str.contains('Abu Danladi')]
prices = prices[~prices.name.str.contains('Aaron Schoenfeld')]
prices = prices[~prices.name.str.contains('Adalberto Peñaranda')]
prices = prices[~prices.name.str.contains('Adam Forshaw')]
prices = prices[~prices.name.str.contains('Adi Yussuf')]
prices = prices[~prices.name.str.contains('AdryelsonAdryelson')]
prices = prices[~prices.name.str.contains('Adrián Spörle')]
prices = prices[~prices.name.str.contains('Adrián Szőke')]
prices = prices[~prices.name.str.contains('Agustín Mulet')]
prices = prices[~prices.name.str.contains('Agustín Palavecino')]
prices = prices[~prices.name.str.contains('Agustín Pastorelli')]
prices = prices[~prices.name.str.contains('Agustín Rogel')]
prices = prices[~prices.name.str.contains('Agustín Sienra')]
prices = prices[~prices.name.str.contains('Agustín Sández')]
prices = prices[~prices.name.str.contains('Ahmet Kivanc')]
prices = prices[~prices.name.str.contains('Ahmetcan Kaplan')]
prices = prices[~prices.name.str.contains('Aidan Morris')]
prices = prices[~prices.name.str.contains('Aimen Moueffek')]
prices = prices[~prices.name.str.contains('Agostino Spina')]
prices = prices[~prices.name.str.contains('Airton')]
prices = prices[~prices.name.str.contains('Alan MendezAlan Mendezattac')]
prices = prices[~prices.name.str.contains('Alan SantosAlan Santos')]
prices = prices[~prices.name.str.contains('Adrián Arregui')]
prices = prices[~prices.name.str.contains('Alan Varela')]
prices = prices[~prices.name.str.contains('AlanAlan')]
prices = prices[~prices.name.str.contains('AlanzinhoAlanzinho')]
prices = prices[~prices.name.str.contains('Alec KannAlec Kann')]
prices = prices[~prices.name.str.contains('Alejandro Cantero')]
prices = prices[~prices.name.str.contains('Alejandro Maciel')]
prices = prices[~prices.name.str.contains('Alejandro Medina')]
prices = prices[~prices.name.str.contains('Alejandro Rivero')]
prices = prices[~prices.name.str.contains('Aleksandr Melikhov')]
prices = prices[~prices.name.str.contains('Aleksandr Selikhov')]
prices = prices[~prices.name.str.contains('Aleksandr Tashaev')]
prices = prices[~prices.name.str.contains('Aleksey Nikitin')]
prices = prices[~prices.name.str.contains('Aleksey Pomerko')]
prices = prices[~prices.name.str.contains('AlerrandroAlerrandro')]
prices = prices[~prices.name.str.contains('Alex Fojticek')]
prices = prices[~prices.name.str.contains('Alex Gogic')]
prices = prices[~prices.name.str.contains('Alex Gorrin')]
prices = prices[~prices.name.str.contains('Alex Jones')]
prices = prices[~prices.name.str.contains('Alex Monis')]
prices = prices[~prices.name.str.contains('Zé Vitor')]
prices = prices[~prices.name.str.contains('Örjan Nyland')]
prices = prices[~prices.name.str.contains('Adam Kelsey')]
prices = prices[~prices.name.str.contains('Adam King')]
prices = prices[~prices.name.str.contains('Adam Mackinnon')]
prices = prices[~prices.name.str.contains('Adam Oudjani')]
prices = prices[~prices.name.str.contains('Adam Senior')]
prices = prices[~prices.name.str.contains('AdrielAdriel')]
prices = prices[~prices.name.str.contains('Alex Vallejo')]
prices = prices[~prices.name.str.contains('AlexAlex')]
prices = prices[~prices.name.str.contains('Alexander Vogler')]
prices = prices[~prices.name.str.contains('Alexander Weidinger')]
prices = prices[~prices.name.str.contains('Alexandre Guedes')]
prices = prices[~prices.name.str.contains('Alexandre Ippolito')]
prices = prices[~prices.name.str.contains('Alexandre Letellier')]
prices = prices[~prices.name.str.contains('Alexandre Phliponeau')]
prices = prices[~prices.name.str.contains('Alexandre TamAlexandre Tam')]
prices = prices[~prices.name.str.contains('Alexandro Bernabéi')]
prices = prices[~prices.name.str.contains('Alexandru Dobre')]
prices = prices[~prices.name.str.contains('Alexis Andre')]
prices = prices[~prices.name.str.contains('Alexis De Sart')]
prices = prices[~prices.name.str.contains('Alexis Messidoro')]
prices = prices[~prices.name.str.contains('Alexis Sabella')]
prices = prices[~prices.name.str.contains('Alexis Sosa')]
prices = prices[~prices.name.str.contains('Alfonso Ocampo-Chávez')]
prices = prices[~prices.name.str.contains('Ali AdnanAli Adnan')]
prices = prices[~prices.name.str.contains('Alhassane Keita')]
prices = prices[~prices.name.str.contains('Ali AkmanAli Akman')]
prices = prices[~prices.name.str.contains('Ali Crawford')]
prices = prices[~prices.name.str.contains('Ali McCann')]
prices = prices[~prices.name.str.contains('Alican Özfesli')]
prices = prices[~prices.name.str.contains('Aljaz Ivacic')]
prices = prices[~prices.name.str.contains('Allan Delferriere')]
prices = prices[~prices.name.str.contains('Allan Rodriguez')]
prices = prices[~prices.name.str.contains('Ally Taylor')]
prices = prices[~prices.name.str.contains('Aminu Umar')]
prices = prices[~prices.name.str.contains('Anderson CearáAnderson Ceará')]
prices = prices[~prices.name.str.contains('Anderson PaixãoAnderson Paixão')]
prices = prices[~prices.name.str.contains('Andi Koshi')]
prices = prices[~prices.name.str.contains('AndersonAnderson')]
prices = prices[~prices.name.str.contains('Andre Rawls')]
prices = prices[~prices.name.str.contains('Andreas Lukse')]
prices = prices[~prices.name.str.contains('Andreas Söndergaard')]
prices = prices[~prices.name.str.contains('Andrew Putna')]
prices = prices[~prices.name.str.contains('Andrew Tarbell')]
prices = prices[~prices.name.str.contains('Andrew Thomas')]
prices = prices[~prices.name.str.contains('Andriy Lunin')]
prices = prices[~prices.name.str.contains('André Becker')]
prices = prices[~prices.name.str.contains('Filipe Da Silva Mendes')]
prices = prices[~prices.name.str.contains('André Hoffmann')]
prices = prices[~prices.name.str.contains('André TrindadeAndré Trindade')]
prices = prices[~prices.name.str.contains('AndréAndré')]
prices = prices[~prices.name.str.contains('Andrés Cubas')]
prices = prices[~prices.name.str.contains('Andrés Desábato')]
prices = prices[~prices.name.str.contains('Andrés Prieto')]
prices = prices[~prices.name.str.contains('Andy Cannon')]
prices = prices[~prices.name.str.contains('Andy Considine')]
prices = prices[~prices.name.str.contains('Andy Dales')]
prices = prices[~prices.name.str.contains('Andy Dallas')]
prices = prices[~prices.name.str.contains('Andy Firth')]
prices = prices[~prices.name.str.contains('Andy Lonergan')]
prices = prices[~prices.name.str.contains('Angelo Preciado')]
prices = prices[~prices.name.str.contains('Angelo da Costa')]
prices = prices[~prices.name.str.contains('Anil Dikmen')]
prices = prices[~prices.name.str.contains('AnilsonAnilson')]
prices = prices[~prices.name.str.contains('Anthony Cheshire')]
prices = prices[~prices.name.str.contains('Anthony Limbombe')]
prices = prices[~prices.name.str.contains('Antoine Lejoly')]
prices = prices[~prices.name.str.contains('Antonio Alfaro')]
prices = prices[~prices.name.str.contains('Antonio Donnarumma')]
prices = prices[~prices.name.str.contains('Antonio Galeano')]
prices = prices[~prices.name.str.contains('Antonio Valera')]
prices = prices[~prices.name.str.contains('Antonis Aidonis')]
prices = prices[~prices.name.str.contains('Antonis Stergiakis')]
prices = prices[~prices.name.str.contains('Andrej Lukic')]
prices = prices[~prices.name.str.contains('Anwan Lawrence')]
prices = prices[~prices.name.str.contains('Arbenit Xhemajli')]
prices = prices[~prices.name.str.contains('Archie Jones')]
prices = prices[~prices.name.str.contains('Archie Procter')]
prices = prices[~prices.name.str.contains('Arda Akbulut')]
prices = prices[~prices.name.str.contains('Arda Kizildag')]
prices = prices[~prices.name.str.contains('Ariel Harush')]
prices = prices[~prices.name.str.contains('Arnau Fàbrega')]
prices = prices[~prices.name.str.contains('Arne Schulz')]
prices = prices[~prices.name.str.contains('Arno Valkenaers')]
prices = prices[~prices.name.str.contains('Artem Rebrov')]
prices = prices[~prices.name.str.contains('Arthur GazzeArthur Gazze')]
prices = prices[~prices.name.str.contains('Allan Nyom')]
prices = prices[~prices.name.str.contains('Arthur Gomes')]
prices = prices[~prices.name.str.contains('Arthur Henrique')]
prices = prices[~prices.name.str.contains('Ashley Maynard-Brewer')]
prices = prices[~prices.name.str.contains('Asim Aksungur')]
prices = prices[~prices.name.str.contains('Aston Oxborough')]
prices = prices[~prices.name.str.contains('Atakan Cankaya')]
prices = prices[~prices.name.str.contains('Augustin Drakpe')]
prices = prices[~prices.name.str.contains('Augustine Williams')]
prices = prices[~prices.name.str.contains('Augusto Aguirre')]
prices = prices[~prices.name.str.contains('Augusto Schott')]
prices = prices[~prices.name.str.contains('AuremirAuremir')]
prices = prices[~prices.name.str.contains('Ayrton Costa')]
prices = prices[~prices.name.str.contains('Aziel Jackson')]
prices = prices[~prices.name.str.contains('Babacar Dione')]
prices = prices[~prices.name.str.contains('Ballou Tabla')]
prices = prices[~prices.name.str.contains('Bamba Dieng')]
prices = prices[~prices.name.str.contains('Baris Ekincier')]
prices = prices[~prices.name.str.contains('Bastian Allgeier')]
prices = prices[~prices.name.str.contains('Ben Chalton')]
prices = prices[~prices.name.str.contains('Ben HartBen Hart')]
prices = prices[~prices.name.str.contains('Ben HoughBen Hough')]
prices = prices[~prices.name.str.contains('Ben Hughes')]
prices = prices[~prices.name.str.contains('Ben Lundgaard')]
prices = prices[~prices.name.str.contains('Ben MinesBen Mines')]
prices = prices[~prices.name.str.contains('Ben Williams')]
prices = prices[~prices.name.str.contains('Ben Williamson')]
prices = prices[~prices.name.str.contains('Ben Worman')]
prices = prices[~prices.name.str.contains('Benedikt Grawe')]
prices = prices[~prices.name.str.contains('Benjamin Borasi')]
prices = prices[~prices.name.str.contains('Benjamin Hübner')]
prices = prices[~prices.name.str.contains('Benjamin Leneis')]
prices = prices[~prices.name.str.contains('Benjamin Mbunga Kimpioka')]
prices = prices[~prices.name.str.contains('Benjamin Uphoff')]
prices = prices[~prices.name.str.contains('Kuscevic')]
prices = prices[~prices.name.str.contains('Bento Estrela')]
prices = prices[~prices.name.str.contains('BentoBento')]
prices = prices[~prices.name.str.contains('Berat Kalkan')]
prices = prices[~prices.name.str.contains('BetinhoBetinho')]
prices = prices[~prices.name.str.contains('Bilal Bayazit')]
prices = prices[~prices.name.str.contains('Bilal Ceylan')]
prices = prices[~prices.name.str.contains('Bill Lathouwers')]
prices = prices[~prices.name.str.contains('Billy Johnson')]
prices = prices[~prices.name.str.contains('Billy Sass-Davies')]
prices = prices[~prices.name.str.contains('Bjorn Meijer')]
prices = prices[~prices.name.str.contains('Blondy Nna Noukeu')]
prices = prices[~prices.name.str.contains('Bobby Burns')]
prices = prices[~prices.name.str.contains('Bode Davis')]
prices = prices[~prices.name.str.contains('BonoBono')]
prices = prices[~prices.name.str.contains('Bora AdamBora Adam')]
prices = prices[~prices.name.str.contains('Boris Lambert')]
prices = prices[~prices.name.str.contains('Boris Rotenberg')]
prices = prices[~prices.name.str.contains('Boris Radunović')]
prices = prices[~prices.name.str.contains('Boubacar Fall')]
prices = prices[~prices.name.str.contains('Brad Collins')]
prices = prices[~prices.name.str.contains('Brad Guzan')]
prices = prices[~prices.name.str.contains('Brad Knighton')]
prices = prices[~prices.name.str.contains('Bradley Collins')]
prices = prices[~prices.name.str.contains('Bradley Wade')]
prices = prices[~prices.name.str.contains('Braian Salvareschi')]
prices = prices[~prices.name.str.contains('Bram Verbong')]
prices = prices[~prices.name.str.contains('Brandan Craig')]
prices = prices[~prices.name.str.contains('Brandon Austin')]
prices = prices[~prices.name.str.contains('Brendan Galloway')]
prices = prices[~prices.name.str.contains('Brendon Shabani')]
prices = prices[~prices.name.str.contains('Brennan Camp')]
prices = prices[~prices.name.str.contains('Breno LopesBreno Lopes')]
prices = prices[~prices.name.str.contains('Bruno Amione')]
prices = prices[~prices.name.str.contains('Bruno Andrade')]
prices = prices[~prices.name.str.contains('Bruno MarquesBruno Marques')]
prices = prices[~prices.name.str.contains('Bruno PachecoBruno Pacheco')]
prices = prices[~prices.name.str.contains('Bruno Palazzo')]
prices = prices[~prices.name.str.contains('Bruno Praxedes')]
prices = prices[~prices.name.str.contains('Bruno Rodrigues')]
prices = prices[~prices.name.str.contains('Bruno RodriguesBruno Rodrigues')]
prices = prices[~prices.name.str.contains('Bruno Valdez')]
prices = prices[~prices.name.str.contains('Bryan IdowuBryan Idowu')]
prices = prices[~prices.name.str.contains('Bryan Meredith')]
prices = prices[~prices.name.str.contains('Bugra Cagiran')]
prices = prices[~prices.name.str.contains('BuiúBuiú')]
prices = prices[~prices.name.str.contains('Aurélien Collin')]
prices = prices[~prices.name.str.contains('Beni Redzic')]
prices = prices[~prices.name.str.contains('Bjorn Engels')]
prices = prices[~prices.name.str.contains('Brooks Thompson')]
prices = prices[~prices.name.str.contains('Bruno Sepúlveda')]
prices = prices[~prices.name.str.contains('Cagtay Kurukalip')]
prices = prices[~prices.name.str.contains('Cagri Güney')]
prices = prices[~prices.name.str.contains('Caio FelipeCaio Felipe')]
prices = prices[~prices.name.str.contains('Caio MotaCaio Mota')]
prices = prices[~prices.name.str.contains('CaiqueCaique')]
prices = prices[~prices.name.str.contains('CalebeCalebe')]
prices = prices[~prices.name.str.contains('CalegariCalegari')]
prices = prices[~prices.name.str.contains('Callum Brain')]
prices = prices[~prices.name.str.contains('Callum Hawkins')]
prices = prices[~prices.name.str.contains('Callum Taylor')]
prices = prices[~prices.name.str.contains('Cameron Gregory')]
prices = prices[~prices.name.str.contains('Cameron Painter')]
prices = prices[~prices.name.str.contains('CarlinhosCarlinhos')]
prices = prices[~prices.name.str.contains('Carlo Boukhalfa')]
prices = prices[~prices.name.str.contains('Carlo Lattanzio')]
prices = prices[~prices.name.str.contains('Carlos AlbertoCarlos Alberto')]
prices = prices[~prices.name.str.contains('Carlos Rentería')]
prices = prices[~prices.name.str.contains('Carlos Rotondi')]
prices = prices[~prices.name.str.contains('Carlos Terán')]
prices = prices[~prices.name.str.contains('Cameron Yates')]
prices = prices[~prices.name.str.contains('Caíque França')]
prices = prices[~prices.name.str.contains('Cem Türkmen')]
prices = prices[~prices.name.str.contains('Cesar SousaCesar Sousa*')]
prices = prices[~prices.name.str.contains('Charles Dunne')]
prices = prices[~prices.name.str.contains('Charles Hagan')]
prices = prices[~prices.name.str.contains('Charlie Barks')]
prices = prices[~prices.name.str.contains('Charlie Barnes')]
prices = prices[~prices.name.str.contains('Charlie Birch')]
prices = prices[~prices.name.str.contains('Charlie Seaman')]
prices = prices[~prices.name.str.contains('Charly Musonda J')]
prices = prices[~prices.name.str.contains('Chase Gasper')]
prices = prices[~prices.name.str.contains('Cheick Diabate')]
prices = prices[~prices.name.str.contains('Chico GeraldesChico Geraldes')]
prices = prices[~prices.name.str.contains('Chinonso Offor')]
prices = prices[~prices.name.str.contains('Choco Lozano')]
prices = prices[~prices.name.str.contains('ChrigorChrigor')]
prices = prices[~prices.name.str.contains('Chris Kane')]
prices = prices[~prices.name.str.contains('Chris Mochrie')]
prices = prices[~prices.name.str.contains('Chris Odoi-Atsem')]
prices = prices[~prices.name.str.contains('Christian Früchtl')]
prices = prices[~prices.name.str.contains('Christian Walton')]
prices = prices[~prices.name.str.contains('ChristianChristian')]
prices = prices[~prices.name.str.contains('Ciaran Brennan')]
prices = prices[~prices.name.str.contains('Cisse Sandra')]
prices = prices[~prices.name.str.contains('ClebersonCleberson')]
prices = prices[~prices.name.str.contains('Claudio Pombo')]
prices = prices[~prices.name.str.contains('Claudio RamosClaudio Ramos')]
prices = prices[~prices.name.str.contains('Claudio Yacob')]
prices = prices[~prices.name.str.contains('Cody Mizell')]
prices = prices[~prices.name.str.contains('Cole Turner')]
prices = prices[~prices.name.str.contains('Colin Coosemans')]
prices = prices[~prices.name.str.contains('Collin Smith')]
prices = prices[~prices.name.str.contains('Collin Verfurth')]
prices = prices[~prices.name.str.contains('Connor Wickham')]
prices = prices[~prices.name.str.contains('Connor van den Berg')]
prices = prices[~prices.name.str.contains('Corey Panter')]
prices = prices[~prices.name.str.contains('Corey Whitely')]
prices = prices[~prices.name.str.contains('Corie Andrews')]
prices = prices[~prices.name.str.contains('Craig Samson')]
prices = prices[~prices.name.str.contains('Cristian Chávez')]
prices = prices[~prices.name.str.contains('Cristian Colmán')]
prices = prices[~prices.name.str.contains('Cristian González')]
prices = prices[~prices.name.str.contains('Cristian Marcial')]
prices = prices[~prices.name.str.contains('Cristian Medina')]
prices = prices[~prices.name.str.contains('Cristian Parano')]
prices = prices[~prices.name.str.contains('Cristian Pavón')]
prices = prices[~prices.name.str.contains('Cristian Rivero')]
prices = prices[~prices.name.str.contains('Curtis Anderson')]
prices = prices[~prices.name.str.contains('Curtis Lyle')]
prices = prices[~prices.name.str.contains('Cy Goddard')]
prices = prices[~prices.name.str.contains('CÃ©sar Haydar')]
prices = prices[~prices.name.str.contains('César Pinares')]
prices = prices[~prices.name.str.contains('César Taborda')]
prices = prices[~prices.name.str.contains('Damiano Pecile')]
prices = prices[~prices.name.str.contains('Damien Rivera')]
prices = prices[~prices.name.str.contains('Dan BowryDan Bowry')]
prices = prices[~prices.name.str.contains('Dan KempDan Kemp')]
prices = prices[~prices.name.str.contains('Dan Trickett-Smith')]
prices = prices[~prices.name.str.contains('Dani Cárdenas')]
prices = prices[~prices.name.str.contains('Dani Martín')]
prices = prices[~prices.name.str.contains('Danijel Petkovic')]
prices = prices[~prices.name.str.contains('Danilo AvelarDanilo Avelar')]
prices = prices[~prices.name.str.contains('Danilo BarcelosDanilo Barcelos')]
prices = prices[~prices.name.str.contains('Danilo Boza')]
prices = prices[~prices.name.str.contains('Danilo GomesDanilo Gomes')]
prices = prices[~prices.name.str.contains('Claudio Riaño')]
prices = prices[~prices.name.str.contains('Clinton Mola')]
prices = prices[~prices.name.str.contains('Cléber Reis')]
prices = prices[~prices.name.str.contains('Haydar')]
prices = prices[~prices.name.str.contains('Dani Alves')]
prices = prices[~prices.name.str.contains('Daniel Mesenhöler')]
prices = prices[~prices.name.str.contains('Daniel Rios')]
prices = prices[~prices.name.str.contains('Daniel Saggiomo')]
prices = prices[~prices.name.str.contains('Daniele De Rossi')]
prices = prices[~prices.name.str.contains('Danilo Acosta')]
prices = prices[~prices.name.str.contains('DarlanDarlan')]
prices = prices[~prices.name.str.contains('Stevie Mallan')]
prices = prices[~prices.name.str.contains('Dominik Kotarski')]
prices = prices[~prices.name.str.contains('Liam Smyth')]
prices = prices[~prices.name.str.contains('Ewerthon')]
prices = prices[~prices.name.str.contains('Konrad de la Fuente')]
prices = prices[~prices.name.str.contains('Rafa VieiraRafa Vieira')]




prices['name'] = prices['name'].str.replace('João LucasJoão Lucas','João Lucas')
prices['name'] = prices['name'].str.replace('AllanAllan','Allan')
prices['name'] = prices['name'].str.replace('AlissonAlisson','Alisson')
prices['name'] = prices['name'].str.replace('Carlos EduardoCarlos Eduardo','Carlos Eduardo')
prices['name'] = prices['name'].str.replace('DanielDaniel','Daniel')
prices['name'] = prices['name'].str.replace('DaniloDanilo','Danilo')
prices['name'] = prices['name'].str.replace('CostinhaCostinha','Costinha')
prices['name'] = prices['name'].str.replace('DuduDudu','Dudu')
prices['name'] = prices['name'].str.replace('CharlesCharles','Charles')
prices['name'] = prices['name'].str.replace('CharlesCharles','Charles')
prices['name'] = prices['name'].str.replace('EduardoEduardo','Eduardo')
prices['name'] = prices['name'].str.replace('FabinhoFabinho','Fabinho')
prices['name'] = prices['name'].str.replace('FelipeFelipe','Felipe')
prices['name'] = prices['name'].str.replace('FernandinhoFernandinho','Fernandinho')
prices['name'] = prices['name'].str.replace('FredFred','Fred')
prices['name'] = prices['name'].str.replace('FábioFábio','Fábio')
prices['name'] = prices['name'].str.replace('Gabriel SilvaGabriel Silva','Gabriel Silva')
prices['name'] = prices['name'].str.replace('GabrielGabriel','Gabriel')
prices['name'] = prices['name'].str.replace('GilbertoGilberto','Gilberto')
prices['name'] = prices['name'].str.replace('GuilhermeGuilherme','Guilherme')
prices['name'] = prices['name'].str.replace('JorginhoJorginho','Jorginho')
prices['name'] = prices['name'].str.replace('João LucasJoão Lucas','João Lucas')
prices['name'] = prices['name'].str.replace('JuninhoJuninho','Juninho')
prices['name'] = prices['name'].str.replace('LuanLuan','Luan')
prices['name'] = prices['name'].str.replace('Luiz FelipeLuiz Felipe','Luiz Felipe')
prices['name'] = prices['name'].str.replace('Luiz FernandoLuiz Fernando','Luiz Fernando')
prices['name'] = prices['name'].str.replace('Luiz GustavoLuiz Gustavo','Luiz Gustavo')
prices['name'] = prices['name'].str.replace('Luiz HenriqueLuiz Henrique','Luiz Henrique')
prices['name'] = prices['name'].str.replace('Luiz OtávioLuiz Otávio','Luiz Otávio')
prices['name'] = prices['name'].str.replace('Léo GomesLéo Gomes','Léo Gomes')
prices['name'] = prices['name'].str.replace('Léo PereiraLéo Pereira','Léo Pereira')
prices['name'] = prices['name'].str.replace('MarceloMarcelo','Marcelo')
prices['name'] = prices['name'].str.replace('MarlonMarlon','Marlon')
prices['name'] = prices['name'].str.replace('MarquinhosMarquinhos','Marquinhos')
prices['name'] = prices['name'].str.replace('MatheusMatheus','Matheus')
prices['name'] = prices['name'].str.replace('MichaelMichael','Michael')
prices['name'] = prices['name'].str.replace('MatheuzinhoMatheuzinho','Matheuzinho')
prices['name'] = prices['name'].str.replace('MichelMichel','Michel')
prices['name'] = prices['name'].str.replace('MikaelMikael','Mikael')
prices['name'] = prices['name'].str.replace('NathanNathan','Nathan')
prices['name'] = prices['name'].str.replace('NicolasNicolas','Nicolas')
prices['name'] = prices['name'].str.replace('NinoNino','Nino')
prices['name'] = prices['name'].str.replace('OtávioOtávio','Otávio')
prices['name'] = prices['name'].str.replace('PabloPablo','Pablo')
prices['name'] = prices['name'].str.replace('PatricPatric','Patric')
prices['name'] = prices['name'].str.replace('PatrickPatrick','Patrick')
prices['name'] = prices['name'].str.replace('PaulinhoPaulinho','Paulinho')
prices['name'] = prices['name'].str.replace('PedrinhoPedrinho','Pedrinho')
prices['name'] = prices['name'].str.replace('Pedro HenriquePedro Henrique','Pedro Henrique')
prices['name'] = prices['name'].str.replace('Pedro LucasPedro Lucas','Pedro Lucas')
prices['name'] = prices['name'].str.replace('PedroPedro','Pedro')
prices['name'] = prices['name'].str.replace('PepêPepê','Pepê')
prices['name'] = prices['name'].str.replace('RafaelRafael','Rafael')
prices['name'] = prices['name'].str.replace('RafinhaRafinha','Rafinha')
prices['name'] = prices['name'].str.replace('RamonRamon','Ramon')
prices['name'] = prices['name'].str.replace('ReinaldoReinaldo','Reinaldo')
prices['name'] = prices['name'].str.replace('RicardinhoRicardinho','Ricardinho')
prices['name'] = prices['name'].str.replace('RichardRichard','Richard')
prices['name'] = prices['name'].str.replace('RobertoRoberto','Roberto')
prices['name'] = prices['name'].str.replace('RobsonRobson','Robson')
prices['name'] = prices['name'].str.replace('RodrigoRodrigo','Rodrigo')
prices['name'] = prices['name'].str.replace('RonaldoRonaldo','Ronaldo')
prices['name'] = prices['name'].str.replace('RuanRuan','Ruan')
prices['name'] = prices['name'].str.replace('SabinoSabino','Sabino')
prices['name'] = prices['name'].str.replace('Sam LongSam Long','Sam Long')
prices['name'] = prices['name'].str.replace('VagnerVagner','Vagner')
prices['name'] = prices['name'].str.replace('VitinhoVitinho','Vitinho')
prices['name'] = prices['name'].str.replace('WandersonWanderson','Wanderson')
prices['name'] = prices['name'].str.replace('WesleyWesley','Wesley')
prices['name'] = prices['name'].str.replace('WilliamWilliam','William')
prices['name'] = prices['name'].str.replace('Zé VitorZé Vitor','Zé Vitor')
prices['name'] = prices['name'].str.replace('ÉderÉder','Éder')
prices['name'] = prices['name'].str.replace('ÉderÉder','Éder')
prices['name'] = prices['name'].str.replace('ÉversonÉverson','Éverson')
prices['name'] = prices['name'].str.replace('Abel RuizAbel Ruiz','Abel Ruiz')
prices['name'] = prices['name'].str.replace('AbnerAbner','Abner')
prices = prices[~prices.name.str.contains('Abner')]
prices['name'] = prices['name'].str.replace('Mo EisaMo Eisa','Mo Eisa')
prices['name'] = prices['name'].str.replace('Abu OgogoAbu Ogogo','Abu Ogogo')
prices['name'] = prices['name'].str.replace('Mo Eisa','Mohamed Eisa')
prices['name'] = prices['name'].str.replace('Zié OuattaraZié Ouattara','Zié Ouattara')
prices['name'] = prices['name'].str.replace('Adam IdahAdam Idah','Adam Idah')
prices['name'] = prices['name'].str.replace('Adam KingAdam King','Adam King')
prices['name'] = prices['name'].str.replace('Adam LongAdam Long','Adam Long')
prices['name'] = prices['name'].str.replace('Adam MayAdam May','Adam May')
prices['name'] = prices['name'].str.replace('Andy KingAndy King','Andy King')
prices['name'] = prices['name'].str.replace('AdemirAdemir','Ademir')
prices['name'] = prices['name'].str.replace('AderlanAderlan','Aderlan')
prices['name'] = prices['name'].str.replace('Adil RamiAdil Rami','Adil Rami')
prices['name'] = prices['name'].str.replace('AdrianoAdriano','Adriano')
prices['name'] = prices['name'].str.replace('AdriánAdrián','Adrián')
prices['name'] = prices['name'].str.replace('AdsonAdson','Adson')
prices['name'] = prices['name'].str.replace('Afonso SousaAfonso SousaLeft Midfiel','Afonso Sousa')
prices['name'] = prices['name'].str.replace('AirtonAirton','Airton')
prices['name'] = prices['name'].str.replace('Alan EmpereurAlan Empereur','Alan Empereur')
prices['name'] = prices['name'].str.replace('Alan RuizAlan Ruiz','Alan Ruiz')
prices['name'] = prices['name'].str.replace('Álex Berenguer','Alex Berenguer')
prices['name'] = prices['name'].str.replace('Alex MacDonald','Alexander MacDonald')
prices['name'] = prices['name'].str.replace('Alfa SemedoAlfa Semedo','Alfa Semedo')
prices['name'] = prices['name'].str.replace('Alireza BeiranvandAlireza Beiranvand','Alireza Beiranvand')
prices['name'] = prices['name'].str.replace('AlisonAlison','Alison')
prices['name'] = prices['name'].str.replace('Alireza Jahanbakhsh','Alireza Safar Beiranvand')
prices['name'] = prices['name'].str.replace('Al MusratiAl Musrati','Al Musrati')
prices['name'] = prices['name'].str.replace('Andrey Galabinov','Andrej Galabinov')
prices['name'] = prices['name'].str.replace('Andrey Murnin','Andrei Murnin')
prices['name'] = prices['name'].str.replace('Andrey Panyukov','Andrei Panyukov')
prices['name'] = prices['name'].str.replace('Andy Fisher','Andrew Fisher')
prices['name'] = prices['name'].str.replace('AntonyAntony','Antony')
prices['name'] = prices['name'].str.replace('Vitorino Antunes','Antunes')
prices['name'] = prices['name'].str.replace('António FilipeAntónio Filipe','António Filipe')
prices['name'] = prices['name'].str.replace('Arda Özcimen','Arda Ozcimen')
prices['name'] = prices['name'].str.replace('Artem Arkhipov','Artyom Arkhipov')
prices['name'] = prices['name'].str.replace('ArthurArthur','Arthur')
prices['name'] = prices['name'].str.replace('Arthur GomesArthur Gomes','Arthur Gomes')
prices['name'] = prices['name'].str.replace('Arthur HenriqueArthur Henrique','Arthur Henrique')
prices['name'] = prices['name'].str.replace('ArturArtur','Artur')
prices['name'] = prices['name'].str.replace('Ash BakerAsh Baker','Ashley Baker')
prices['name'] = prices['name'].str.replace('Aurélien Tchouaméni','Aurelien Tchouameni')
prices['name'] = prices['name'].str.replace('Auro J','Auro')
prices['name'] = prices['name'].str.replace('Bahadir Öztürk','Bahadir Ozturk')
prices['name'] = prices['name'].str.replace('Bengali-Fodé Koita','Bangaly-Fodé Koita')
prices['name'] = prices['name'].str.replace('Béni Badibanga','Beni Badibanga')
prices['name'] = prices['name'].str.replace('Ben Whiteman','Benjamin Whiteman')
prices['name'] = prices['name'].str.replace('Benoît Costil','Benoît Costil')
prices['name'] = prices['name'].str.replace('BernardBernard','Bernard')
prices['name'] = prices['name'].str.replace('Louis Chadwick','Billy Chadwick')
prices['name'] = prices['name'].str.replace('Björn','Bjorn')
prices['name'] = prices['name'].str.replace('Rafael Bracali','Bracali')
prices['name'] = prices['name'].str.replace('Ibrahim Diarra','Brahima Diarra')
prices['name'] = prices['name'].str.replace('BuraBura','Bura')
prices['name'] = prices['name'].str.replace('BurguiBurgui','Burgui')
prices['name'] = prices['name'].str.replace('Bünyamin Balci','Bunyamin Balci')
prices['name'] = prices['name'].str.replace('Bülent Cevahir','Bulent Cevahir')
prices['name'] = prices['name'].str.replace('CamiloCamilo','Camilo')
prices['name'] = prices['name'].str.replace('PonckPonck','Carlos Ponck')
prices['name'] = prices['name'].str.replace('CarraçaCarraça','Carraça')
prices['name'] = prices['name'].str.replace('CasagrandeCasagrande*','Casagrande')
prices['name'] = prices['name'].str.replace('Cédric Gogoua','Cedric Gogoua')
prices['name'] = prices['name'].str.replace('Chey Dunkley','Cheyenne Dunkley')
prices['name'] = prices['name'].str.replace('ChicoChico','Chico')
prices['name'] = prices['name'].str.replace('Cláudio','Claudio')
prices['name'] = prices['name'].str.replace('Clément','Clement')
prices['name'] = prices['name'].str.replace('Cristhian Cásseres J','Cristian Cásseres Jr')
prices['name'] = prices['name'].str.replace('CÃ¡ssio ScheidCÃ¡ssio Scheid','CÃ¡ssio Scheid')
prices['name'] = prices['name'].str.replace('Dan AgyeiDan Agyei','Daniel Agyei')
prices['name'] = prices['name'].str.replace('Dan Barlaser','Daniel Barlaser')
prices['name'] = prices['name'].str.replace('Dan Crowley','Daniel Crowley')
prices['name'] = prices['name'].str.replace('Dan HappeDan Happe','Daniel Happe')
prices['name'] = prices['name'].str.replace('Dani Parejo','Daniel Parejo')
prices['name'] = prices['name'].str.replace('Danil Krugovoy','Daniil Krugovoy')
prices['name'] = prices['name'].str.replace('Danny Whitehall','Daniel Whitehall')
prices['name'] = prices['name'].str.replace('DanteDante','Dante')
prices['name'] = prices['name'].str.replace('Dario Van den Buijs','Dario van den Buijs')
prices['name'] = prices['name'].str.replace('Mike Fondop','Mike Fondop-Talom')
prices['name'] = prices['name'].str.replace('Tom Dallison','Tom Dallison-Lisbon')
prices['name'] = prices['name'].str.replace('Lucas EvangelistaLucas Evangelista','Lucas Evangelista')
prices['name'] = prices['name'].str.replace('Flamarion JuniorFlamarion Junior','Flamarion Junior')
prices['name'] = prices['name'].str.replace('RonaldRonald','Ronald')
prices['name'] = prices['name'].str.replace('DeyversonDeyverson','Deyverson')
prices['name'] = prices['name'].str.replace('Cristiano RonaldoCristiano Ronaldo','Cristiano Ronaldo')
prices['name'] = prices['name'].str.replace('Dani OlmoDani Olmo','Dani Olmo')
prices['name'] = prices['name'].str.replace('Dan NdoyeDan Ndoye','Dan Ndoye')
prices['name'] = prices['name'].str.replace('Dani RabaDani Raba','Dani Raba')
prices['name'] = prices['name'].str.replace('Pape Cheikh','Pape Cheikh Diop Gueye')
prices['name'] = prices['name'].str.replace('Felipe AzevedoFelipe Azevedo','Felipe Azevedo')
prices['name'] = prices['name'].str.replace('Stefan Ortega','Stefan Ortega Moreno')
prices['name'] = prices['name'].str.replace('Matheus CavichioliMatheus Cavichioli','Matheus Cavichioli')
prices['name'] = prices['name'].str.replace('Francisco TeixeiraFrancisco Teixeira','Francisco Teixeira')
prices['name'] = prices['name'].str.replace('Derrick Etienne J','Derrick Etienne')
prices['name'] = prices['name'].str.replace('Pedro SáPedro Sá','Pedro Sá')
prices['name'] = prices['name'].str.replace('Pape Cheikh','Pape Cheikh Diop Gueye')
prices['name'] = prices['name'].str.replace('LeandrinhoLeandrinho','Leandrinho')
prices['name'] = prices['name'].str.replace('Carlos ViníciusCarlos Vinícius','Carlos Vinícius')
prices['name'] = prices['name'].str.replace('Carlos JúniorCarlos Júnior','Carlos Júnior')
prices['name'] = prices['name'].str.replace('ChiquinhoChiquinho','Chiquinho')
prices['name'] = prices['name'].str.replace('ClaudemirClaudemir','Claudemir')
prices['name'] = prices['name'].str.replace('Claudio FalcãoClaudio Falcão','Claudio Falcão')
prices['name'] = prices['name'].str.replace('Claudio RamosClaudio Ramos','Claudio Ramos')
prices['name'] = prices['name'].str.replace('Cássio ScheidCássio Scheid','Cássio Scheid')
prices['name'] = prices['name'].str.replace('Daniel GuimarãesDaniel Guimarães','Daniel Guimarães')
prices['name'] = prices['name'].str.replace('DanielzinhoDanielzinho','Danielzinho')
prices['name'] = prices['name'].str.replace('Danilo FernandesDanilo Fernandes','Danilo Fernandes')
prices['name'] = prices['name'].str.replace('Danny HenriquesDanny Henriques','Danny Henriques')
prices['name'] = prices['name'].str.replace('Derik LacerdaDerik Lacerda','Derik Lacerda')
prices['name'] = prices['name'].str.replace('Douglas FriedrichDouglas Friedrich','Douglas Friedrich')
prices['name'] = prices['name'].str.replace('Eduardo BauermannEduardo Bauermann','Eduardo Bauermann')
prices['name'] = prices['name'].str.replace('Eduardo ManchaEduardo Mancha','Eduardo Mancha')
prices['name'] = prices['name'].str.replace('Elias ManoelElias Manoel','Elias Manoel')
prices['name'] = prices['name'].str.replace('Eloy RoomEloy Room','Eloy Room')
prices['name'] = prices['name'].str.replace('Emerson RoyalEmerson Royal','Emerson Royal')
prices['name'] = prices['name'].str.replace('Erik DurmErik Durm','Erik Durm')
prices['name'] = prices['name'].str.replace('Everton FelipeEverton Felipe','Everton Felipe')
prices['name'] = prices['name'].str.replace('EwerthonEwerthon','Ewerthon')
prices['name'] = prices['name'].str.replace('EwertonEwerton','Ewerton')
prices['name'] = prices['name'].str.replace('Fabrício BrunoFabrício Bruno','Fabrício Bruno')
prices['name'] = prices['name'].str.replace('Ezequiel CeruttiEzequiel Cerutti','Ezequiel Cerutti')
prices['name'] = prices['name'].str.replace('Ezequiel CeruttiEzequiel Cerutti','Ezequiel Cerutti')
prices['name'] = prices['name'].str.replace('Fabrício BaianoFabrício Baiano','Fabrício Baiano')
prices['name'] = prices['name'].str.replace('Fabrício IsidoroFabrício Isidoro','Fabrício Isidoro')
prices['name'] = prices['name'].str.replace('Felipe AlvesFelipe Alves','Felipe Alves')
prices['name'] = prices['name'].str.replace('Felipe AndersonFelipe AndersonLeft Midfiel','Felipe Anderson')
prices['name'] = prices['name'].str.replace('Felipe MarquesFelipe Marques','Felipe Marques')
prices['name'] = prices['name'].str.replace('Felipe JonatanFelipe Jonatan','Felipe Jonatan')
prices['name'] = prices['name'].str.replace('Felipe SantanaFelipe Santana','Felipe Santana')
prices['name'] = prices['name'].str.replace('Fernando AndradeFernando Andrade','Fernando Andrade')
prices['name'] = prices['name'].str.replace('Fernando CaixetaFernando Caixeta','Fernando Caixeta')
prices['name'] = prices['name'].str.replace('Fernando CanesinFernando Canesin','Fernando Canesin')
prices['name'] = prices['name'].str.replace('Fernando FonsecaFernando Fonseca','Fernando Fonseca')
prices['name'] = prices['name'].str.replace('Fernando MiguelFernando Miguel','Fernando Miguel')
prices['name'] = prices['name'].str.replace('Fernando RegesFernando Reges','Fernando Reges')
prices['name'] = prices['name'].str.replace('Fernando SobralFernando Sobral','Fernando Sobral')
prices['name'] = prices['name'].str.replace('Filipe AugustoFilipe Augusto','Filipe Augusto')
prices['name'] = prices['name'].str.replace('FransérgioFransérgio','Fransérgio')
prices['name'] = prices['name'].str.replace('Pape Cheikh','Pape Cheikh Diop Gueye')
prices['name'] = prices['name'].str.replace('Joe Wollacott','Jojo Wollacott')
prices['name'] = prices['name'].str.replace('Jonny Maxted','Jon Maxted')
prices['name'] = prices['name'].str.replace('Matheus SilvaMatheus Silva','Matheus Silva')
prices['name'] = prices['name'].str.replace('Caio HenriqueCaio Henrique','Caio Henrique')
prices['name'] = prices['name'].str.replace('Thiago AndradeThiago Andrade','Thiago Andrade')
prices['name'] = prices['name'].str.replace('Gonçalo SilvaGonçalo Silva','Gonçalo Silva')
prices['name'] = prices['name'].str.replace('Gustavo HenriqueGustavo Henrique','Gustavo Henrique')
prices['name'] = prices['name'].str.replace('Scott FoxScott Fox','Scott Fox')
prices['name'] = prices['name'].str.replace('Anderson OliveiraAnderson Oliveira','Anderson Oliveira')
prices['name'] = prices['name'].str.replace('Gonçalo InácioGonçalo Inácio','Gonçalo Inácio')
prices['name'] = prices['name'].str.replace('Diego CostaDiego Costa','Diego Costa')
prices['name'] = prices['name'].str.replace('Rodrigo GomesRodrigo Gomes*','Rodrigo Gomes')
prices['name'] = prices['name'].str.replace('WelingtonWelington','Welington')
prices['name'] = prices['name'].str.replace('Rui FonteRui Fonte','Rui Fonte')
prices['name'] = prices['name'].str.replace('Daniel GuedesDaniel Guedes','Daniel Guedes')
prices['name'] = prices['name'].str.replace('GuedesGuedes','Guedes')
prices['name'] = prices['name'].str.replace('Marcelo BenevenutoMarcelo Benevenuto','Marcelo Benevenuto')
prices['name'] = prices['name'].str.replace('Raphael VeigaRaphael Veiga','Raphael Veiga')
prices['name'] = prices['name'].str.replace('Patrick de PaulaPatrick de Paula','Patrick de Paula')
prices['name'] = prices['name'].str.replace('KlausKlaus','Klaus')
prices['name'] = prices['name'].str.replace('Jota GonçalvesJota Gonçalves','Jota Gonçalves')
prices['name'] = prices['name'].str.replace('MikeMike','Mike')
prices['name'] = prices['name'].str.replace('Juninho CapixabaJuninho Capixaba','Juninho Capixaba')
prices['name'] = prices['name'].str.replace('TaisonTaison','Taison')
prices['name'] = prices['name'].str.replace('Léo SenaLéo Sena','Léo Sena')
prices['name'] = prices['name'].str.replace('WalterWalter','Walter')
prices['name'] = prices['name'].str.replace('Matheus TeixeiraMatheus Teixeira','Matheus Teixeira')
prices['name'] = prices['name'].str.replace('Rodolfo CardosoRodolfo Cardoso','Rodolfo Cardoso')
prices['name'] = prices['name'].str.replace('IagoIago','Iago')
prices['name'] = prices['name'].str.replace('AriAri','Ari')
prices['name'] = prices['name'].str.replace('Everton LuizEverton Luiz','Everton Luiz')
prices['name'] = prices['name'].str.replace('Wellington PaulistaWellington Paulista','Wellington Paulista')
prices['name'] = prices['name'].str.replace('PepePepe','Pepe')
prices['name'] = prices['name'].str.replace('PepeluPepelu','Pepelu')
prices['name'] = prices['name'].str.replace('RonyRony','Rony')
prices['name'] = prices['name'].str.replace('PeléPelé','Pelé')
prices['name'] = prices['name'].str.replace('MiullenMiullen','Miullen')
prices['name'] = prices['name'].str.replace('Kaio JorgeKaio Jorge','Kaio Jorge')
prices['name'] = prices['name'].str.replace('KaikyKaiky','Kaiky')
prices['name'] = prices['name'].str.replace('Chico RamosChico Ramos','Chico Ramos')
prices['name'] = prices['name'].str.replace('Rony LopesRony Lopes','Rony Lopes')
prices['name'] = prices['name'].str.replace('Rony LopesRony Lopes','Rony Lopes')
prices['name'] = prices['name'].str.replace('ShaylonShaylonRight Midfiel','Shaylon')
prices['name'] = prices['name'].str.replace('Tom PettTom PettLeft Midfiel','Tom Pett')
prices['name'] = prices['name'].str.replace('Wes BurnsWes BurnsRight Midfiel','Wes Burns')
prices['name'] = prices['name'].str.replace('ÉvertonÉvertonLeft Midfiel','Éverton')
prices['name'] = prices['name'].str.replace('CajuCaju','Caju')
prices['name'] = prices['name'].str.replace('KaioKaio','Kaio')
prices['name'] = prices['name'].str.replace('NaldoNaldo','Naldo')
prices['name'] = prices['name'].str.replace('Felix AguFelix Agu','Felix Agu')
prices['name'] = prices['name'].str.replace('Felipe MeloFelipe Melo','Felipe Melo')
prices['name'] = prices['name'].str.replace('Felipe MeloFelipe Melo','Felipe Melo')
prices['name'] = prices['name'].str.replace('Diego AlvesDiego Alves','Diego Alves')
prices['name'] = prices['name'].str.replace('Nelson BalongoNelson Balongo','Nelson Balongo')
prices['name'] = prices['name'].str.replace('Talles MagnoTalles Magno','Talles Magno')
prices['name'] = prices['name'].str.replace('Nathan SilvaNathan Silva','Nathan Silva')
prices['name'] = prices['name'].str.replace('Ayrton LucasAyrton Lucas','Ayrton Lucas')
prices['name'] = prices['name'].str.replace('Yago PikachuYago Pikachu','Yago Pikachu')
prices['name'] = prices['name'].str.replace('Pedro AugustoPedro Augusto','Pedro Augusto')
prices['name'] = prices['name'].str.replace('Rafael ThyereRafael Thyere','Rafael ThyereRafael Thyere')
prices['name'] = prices['name'].str.replace('Pedro NaressiPedro Naressi','Pedro Naressi')
prices['name'] = prices['name'].str.replace('Júnior MoraisJúnior Morais','Júnior MoraisJúnior Morais')
prices['name'] = prices['name'].str.replace('Gabriel LacerdaGabriel Lacerda','Gabriel Lacerda')
prices['name'] = prices['name'].str.replace('Pape Cheikh Diop Gueye Diop Gueye Diop Gueye Diop Gueye','Pape Cheikh Diop Gueye')
prices['name'] = prices['name'].str.replace('Gabriel SaraGabriel Sara','Gabriel Sara')
prices['name'] = prices['name'].str.replace('JoãozinhoJoãozinho','Joãozinho')
prices['name'] = prices['name'].str.replace('Matheus BahiaMatheus Bahia','Matheus Bahia')
prices['name'] = prices['name'].str.replace('MarinhoMarinho','Marinho')
prices['name'] = prices['name'].str.replace('Raul GustavoRaul Gustavo','Raul Gustavo')
prices['name'] = prices['name'].str.replace('RaulRaul','Raul')
prices['name'] = prices['name'].str.replace('RaulRaul','Raul')
prices['name'] = prices['name'].str.replace('RaulzinhoRaulzinho','Raulzinho')
prices['name'] = prices['name'].str.replace('Matheus GaldezaniMatheus Galdezani','Matheus Galdezani')
prices['name'] = prices['name'].str.replace('João FélixJoão Félix','João Félix')
prices['name'] = prices['name'].str.replace('Gabriel PaulistaGabriel Paulista','Gabriel Paulista')
prices['name'] = prices['name'].str.replace('Zé RobertoZé Roberto','Zé Roberto')
prices['name'] = prices['name'].str.replace('YtaloYtalo','Ytalo')
prices['name'] = prices['name'].str.replace('KaykyKayky','Kayky')
prices['name'] = prices['name'].str.replace('Ciro RiusCiro Rius','Ciro Rius')
prices['name'] = prices['name'].str.replace('Tom EavesTom Eaves','Tom Eaves')
prices['name'] = prices['name'].str.replace('Tom EavesTom Eaves','Tom Eaves')
prices['name'] = prices['name'].str.replace('Patrick de LuccaPatrick de Lucca','Patrick de Lucca')


#prices['name'] = prices['name'].str.replace('Álex','Alex')#replace all Álex in Alex
#prices['name'] = prices['name'].str.replace('Özgür','Ozgur')
#prices['name'] = prices['name'].str.replace('Ömer','Omer')


df['Name'] = df['Name'].str.replace('Joseph Alfred Duncan','Alfred Duncan')
df['Name'] = df['Name'].str.replace('Alkhaly Momo Cissé','Momo Cissé')
df['Name'] = df['Name'].str.replace('Babajide David Akintola','David Akintola')
df['Name'] = df['Name'].str.replace('Baralhas','Gabriel Baralhas')
df['Name'] = df['Name'].str.replace('Benoit Poulain','Benoît Poulain')
df['Name'] = df['Name'].str.replace('Bertrand Yves Baraye','Yves Baraye')
df['Name'] = df['Name'].str.replace('Bjørn','Bjorn')
df['Name'] = df['Name'].str.replace('Clément','Clement')
df['Name'] = df['Name'].str.replace('Cristián Borja','Cristian Borja')
df['Name'] = df['Name'].str.replace('Cristián Insaurralde','Cristian Insaurralde')
df['Name'] = df['Name'].str.replace('Daniel Fernandes','Daniel Heuer Fernandes')
df['Name'] = df['Name'].str.replace('Mateu Morey','Mateu Morey Bauza')
df['Name'] = df['Name'].str.replace('Arnaut Danjuma Groeneveld','Arnaut Danjuma')
df['Name'] = df['Name'].str.replace('Rodrigo Martins Gomes','Rodrigo Gomes')


prices['name'] = prices['name'].str.replace('LorhanLorhanmidfiel','Lorhan')
prices['name'] = prices['name'].str.replace('Édermidfiel','Éder')
prices['name'] = prices['name'].str.replace('FalcaoFalcao','Falcao')
prices['name'] = prices['name'].str.replace('Gabriel FalcãoGabriel Falcão','Gabriel Falcão')
prices['name'] = prices['name'].str.replace('Fábio VieiraFábio Vieira','Fábio Vieira')
prices['name'] = prices['name'].str.replace('Léo VieiraLéo Vieira','Léo Vieira')
prices['name'] = prices['name'].str.replace('Alex VigoAlex Vigo','Alex Vigo')
prices['name'] = prices['name'].str.replace('Dele AlliDele Alli','Dele Alli')
prices['name'] = prices['name'].str.replace('Rai VloetRai Vloet','Rai Vloet')
prices['name'] = prices['name'].str.replace('Sam SmithSam Smith','Sam Smith')
prices['name'] = prices['name'].str.replace('Zak JulesZak Jules','Zak Jules')
prices['name'] = prices['name'].str.replace('Ivan IlicIvan Ilic','Ivan Ilic')
prices['name'] = prices['name'].str.replace('Sam PerrySam Perry','Sam Perry')
prices['name'] = prices['name'].str.replace('Marc BolaMarc Bola','Marc Bola')
prices['name'] = prices['name'].str.replace('Drew MoorDrew Moor','Drew Moor')
prices['name'] = prices['name'].str.replace('Nick PopeNick Pope','Nick Pope')
prices['name'] = prices['name'].str.replace('Lee BurgeLee Burge','Lee Burge')
prices['name'] = prices['name'].str.replace('Will DeanWill Dean','Will Dean')
prices['name'] = prices['name'].str.replace('Percy TauPercy Tau','Percy Tau')
prices['name'] = prices['name'].str.replace('Guus TilGuus Til','Guus Til')



prices['name'] = prices['name'].str.replace('MarafonaMarafona','Marafona')
prices['name'] = prices['name'].str.replace('Matt JayMatt Jay','Matt Jay')
prices['name'] = prices['name'].str.replace('FabrícioFabrício','Fabrício')
prices['name'] = prices['name'].str.replace('RochinhaRochinha','Rochinha')
prices['name'] = prices['name'].str.replace('Ali SoweAli Sowe','Ali Sowe')
prices['name'] = prices['name'].str.replace('Sam LingSam Ling','Sam Ling')
prices['name'] = prices['name'].str.replace('WevertonWeverton','Weverton')
prices['name'] = prices['name'].str.replace('WeversonWeverson','Weverson')
prices['name'] = prices['name'].str.replace('Rob HuntRob Hunt','Rob Hunt')
prices['name'] = prices['name'].str.replace('LourencyLourency','Lourency')
prices['name'] = prices['name'].str.replace('Gabriel DiasGabriel Dias','Gabriel Dias')
prices['name'] = prices['name'].str.replace('Gabriel ChapecóGabriel Chapecó','Gabriel Chapecó')
prices['name'] = prices['name'].str.replace('Tom LeesTom Lees','Tom Lees')
prices['name'] = prices['name'].str.replace('CleitonCleiton','Cleiton')
prices['name'] = prices['name'].str.replace('DouglasDouglas','Douglas')
prices['name'] = prices['name'].str.replace('RodrygoRodrygo','Rodrygo')
prices['name'] = prices['name'].str.replace('VitinhaVitinha','Vitinha')
prices['name'] = prices['name'].str.replace('RamiresRamires','Ramires')
prices['name'] = prices['name'].str.replace('DjaninyDjaniny','Djaniny')
prices['name'] = prices['name'].str.replace('MirandaMiranda','Miranda')
prices['name'] = prices['name'].str.replace('HelinhoHelinho','Helinho')
prices['name'] = prices['name'].str.replace('LyancoLyanco','Lyanco')
prices['name'] = prices['name'].str.replace('JardelJardel','Jardel')
prices['name'] = prices['name'].str.replace('RenanRenan','Renan')
prices['name'] = prices['name'].str.replace('RoniRoni','Roni')
prices['name'] = prices['name'].str.replace('RodriguesRodrigues','Rodrigues')
prices['name'] = prices['name'].str.replace('JoaquimJoaquim','Joaquim')
prices['name'] = prices['name'].str.replace('JacksonJackson','Jackson')
prices['name'] = prices['name'].str.replace('EzequielEzequiel','Ezequiel')
prices['name'] = prices['name'].str.replace('LucianoLuciano','Luciano')
prices['name'] = prices['name'].str.replace('GustavoGustavo','Gustavo')
prices['name'] = prices['name'].str.replace('Luis HenriqueLuis Henrique','Luis Henrique')
prices['name'] = prices['name'].str.replace('IgnácioIgnácio','Ignácio')
prices['name'] = prices['name'].str.replace('Pedro GeromelPedro Geromel','Geromel')
prices['name'] = prices['name'].str.replace('Pape Cheikh Diop Gueye Diop Gueye Diop Gueye','Pape Cheikh Diop Gueye')
prices['name'] = prices['name'].str.replace('Pape Habib Gueye','Pape Habib Gueye')
prices['name'] = prices['name'].str.replace('Dominik Oroz','Alois Dominik Oroz')
prices['name'] = prices['name'].str.replace('Sylla SowSylla Sow','Sylla Sow')
prices['name'] = prices['name'].str.replace('Guilherme AranaGuilherme Arana','Guilherme Arana')
prices['name'] = prices['name'].str.replace('Mads Juel Andersen','Mads Andersen')
prices['name'] = prices['name'].str.replace('Dogan Gölpek','Dogan Can Golpek')
prices['name'] = prices['name'].str.replace('Mathew Stevens','Matt Stevens')
prices['name'] = prices['name'].str.replace('Sheldon Bateau','Sheldon Louis Bateau')
prices['name'] = prices['name'].str.replace('David Costa','David Da Costa')
prices['name'] = prices['name'].str.replace('Matty Kennedy','Matthew Kennedy')
prices['name'] = prices['name'].str.replace('Lucas SilvaLucas Silva','Lucas Silva')
prices['name'] = prices['name'].str.replace('Marcos LeonardoMarcos Leonardo','Marcos Leonardo')
prices['name'] = prices['name'].str.replace('Daníel Leó Grétarsson','Daniel Gretarsson')
prices['name'] = prices['name'].str.replace('MattheusMattheus','Mattheus')
prices['name'] = prices['name'].str.replace('Paulinho MocelinPaulinho Mocelin','Paulinho Mocelin')
prices['name'] = prices['name'].str.replace('Joshua John','Josua John')
prices['name'] = prices['name'].str.replace('Alex DeJohn','Alex De John')
prices['name'] = prices['name'].str.replace('Matheus BabiMatheus Babi','Matheus Babi')
prices['name'] = prices['name'].str.replace('João GonçalvesJoão Gonçalves','João Gonçalves')
prices['name'] = prices['name'].str.replace('Lautaro Giannetti','Lautaro Gianetti')
prices['name'] = prices['name'].str.replace('Marlon FreitasMarlon Freitas','Marlon Freitas')
prices['name'] = prices['name'].str.replace('Rodrigo CaioRodrigo Caio','Rodrigo Caio')
prices['name'] = prices['name'].str.replace('ShowShow','Show')
prices['name'] = prices['name'].str.replace('NonatoNonato','Nonato')
prices['name'] = prices['name'].str.replace('Paulo OtávioPaulo Otávio','Paulo Otávio')
prices['name'] = prices['name'].str.replace('JosemaJosema','Josema')
prices['name'] = prices['name'].str.replace('Marcos Guilherme','Marcos Guilherme')
prices['name'] = prices['name'].str.replace('Hernâni InfandeHernâni Infande*','Hernâni Infande')
prices['name'] = prices['name'].str.replace('Junior BrandãoJunior Brandão','Junior Brandão')
prices['name'] = prices['name'].str.replace('Ricardo RyllerRicardo Ryller','Ricardo Ryller')
prices['name'] = prices['name'].str.replace('Munir El HaddadiMunir El Haddadi','Munir El Haddadi')
prices['name'] = prices['name'].str.replace('Mohamed ElnenyMohamed Elneny','Mohamed Elneny')
prices['name'] = prices['name'].str.replace('Vitor HugoVitor Hugodefenc','Vitor Hugodefenc')
prices['name'] = prices['name'].str.replace('Vitor MendesVitor Mendes','Vitor Mendes')
prices['name'] = prices['name'].str.replace('Vitor HugoVitor Hugo','Vitor Hugo')
prices['name'] = prices['name'].str.replace('Vítor OliveiraVítor Oliveira*','Vítor Oliveira')
prices['name'] = prices['name'].str.replace('Gabriel NovaesGabriel Novaes','Gabriel Novaes')
prices['name'] = prices['name'].str.replace('RoneiRonei','Ronei')
prices['name'] = prices['name'].str.replace('RodriRodri','Rodri')
prices['name'] = prices['name'].str.replace('Jovane CabralJovane Cabral','Jovane Cabral')
prices['name'] = prices['name'].str.replace('Mohamed DiabyMohamed Diaby','Mohamed Diaby')
prices['name'] = prices['name'].str.replace('Bruno MeloBruno Melo','Bruno Melo')
prices['name'] = prices['name'].str.replace('Filipe MeloFilipe Melo','Filipe Melo')
prices['name'] = prices['name'].str.replace('Dan N','Daniel')
prices['name'] = prices['name'].str.replace('Dimitrios Giannoulis','Dimitris Giannoulis')
prices['name'] = prices['name'].str.replace('Dmitri','Dmitriy ')
prices['name'] = prices['name'].str.replace('Özgür Sert','Ozgur Sert')
prices['name'] = prices['name'].str.replace('Dario van den Buijs','Dario Van den Buijs')
prices['name'] = prices['name'].str.replace('DavidDavid','David')
prices['name'] = prices['name'].str.replace('DenisDenis','Denis')
prices['name'] = prices['name'].str.replace('Denis Will Poha','Denis Poha')
prices['name'] = prices['name'].str.replace('PedriPedri','Pedri')
prices['name'] = prices['name'].str.replace('GalegoGalego','Jefferson Galego')
prices['name'] = prices['name'].str.replace('Renan GuedesRenan Guedes','Renan Guedes')
prices['name'] = prices['name'].str.replace('Dmitriy','Dimitri ')


prices = prices[~prices.name.str.contains('Alexis Pérez')]
df = df[~df.Name.str.contains('Alfon')]

prices = prices[~prices.name.str.contains('ÃÃ±igo Lekue')]
prices = prices[~prices.name.str.contains('ÃÃ±igo Martinez')]
prices = prices[~prices.name.str.contains('Adam Phillips')]
prices = prices[~prices.name.str.contains('Adam Randell')]
prices = prices[~prices.name.str.contains('Ahmet Calik')]
prices = prices[~prices.name.str.contains('Ahmet Canbaz')]
prices = prices[~prices.name.str.contains('Alan Carius')]
prices = prices[~prices.name.str.contains('Alejandro Cabrera')]
prices = prices[~prices.name.str.contains('Alejandro Cantero')]
prices = prices[~prices.name.str.contains('Aleksandr Golovnya')]
prices = prices[~prices.name.str.contains('Aleksandr Lomovitskiy')]
prices = prices[~prices.name.str.contains('Aleksandr Smirnov')]
prices = prices[~prices.name.str.contains('Aleksandr Sobolev')]
prices = prices[~prices.name.str.contains('Ãlex Blesa')]
prices = prices[~prices.name.str.contains('Alfie McCalmont')]
prices = prices[~prices.name.str.contains('Alfie Whiteman')]
prices = prices[~prices.name.str.contains('Alireza Beiranvand')]
prices = prices[~prices.name.str.contains('Alireza Jahanbakhsh')]
prices = prices[~prices.name.str.contains('Allan-Romeo Nyom')]
prices = prices[~prices.name.str.contains('Alonzo Engwanda')]
prices = prices[~prices.name.str.contains('Ãlvaro Bastida')]
prices = prices[~prices.name.str.contains('Ãlvaro Bravo')]
prices = prices[~prices.name.str.contains('Ãlvaro Gimenez')]
prices = prices[~prices.name.str.contains('Ãlvaro Quezada')]
prices = prices[~prices.name.str.contains('Ãlvaro Tejero')]
prices = prices[~prices.name.str.contains('Alvas Powell')]
prices = prices[~prices.name.str.contains('Alvi Adilkhanov')]
prices = prices[~prices.name.str.contains('Amir Abrashi')]
prices = prices[~prices.name.str.contains('Anders Trondsen')]
prices = prices[~prices.name.str.contains('Ãngelo Martino')]
prices = prices[~prices.name.str.contains('Ãngelo Preciado')]
prices = prices[~prices.name.str.contains('Ãngelo Sagal')]
prices = prices[~prices.name.str.contains('Anthony Ujah')]
prices = prices[~prices.name.str.contains('Anthony Uzodimma')]
prices = prices[~prices.name.str.contains('Anton Mitryushkin')]
prices = prices[~prices.name.str.contains('Anton Piskunov')]
prices = prices[~prices.name.str.contains('Antonio Raimondo')]
prices = prices[~prices.name.str.contains('Antonio Rosati')]
prices = prices[~prices.name.str.contains('Archie Meekison')]
prices = prices[~prices.name.str.contains('Arda Kizildag')]
prices = prices[~prices.name.str.contains('Aristote Nsiala')]
prices = prices[~prices.name.str.contains('Aristoteles Romero')]
prices = prices[~prices.name.str.contains('Ãsak Thorvaldsson')]
prices = prices[~prices.name.str.contains('Ayaz Guliev')]
prices = prices[~prices.name.str.contains('Aykut Ozer')]
prices = prices[~prices.name.str.contains('Batuhan Ãœnsal')]
prices = prices[~prices.name.str.contains('Batuhan Unsal')]
prices = prices[~prices.name.str.contains('Beckham Sunderland')]
prices = prices[~prices.name.str.contains('Beka Mikeltadze')]
prices = prices[~prices.name.str.contains('Ben Jackson')]
prices = prices[~prices.name.str.contains('Beni Makouana')]
prices = prices[~prices.name.str.contains('Berk Cetin')]
prices = prices[~prices.name.str.contains('Berk Yildiz')]
prices = prices[~prices.name.str.contains('Bernardo ADDDDDDDDDDD')]
prices = prices[~prices.name.str.contains('Braian Aguirre')]
prices = prices[~prices.name.str.contains('Braian Ãlvarez')]
prices = prices[~prices.name.str.contains('Braian Cufre')]
prices = prices[~prices.name.str.contains('Braian Sanchez')]
prices = prices[~prices.name.str.contains('BrayanBrayan')]
prices = prices[~prices.name.str.contains('Braydon Manu')]
prices = prices[~prices.name.str.contains('Bret Halsey')]
prices = prices[~prices.name.str.contains('Brett McGavin')]
prices = prices[~prices.name.str.contains('Brian Idowu')]
prices = prices[~prices.name.str.contains('Brian Lluy')]
prices = prices[~prices.name.str.contains('Brian Negro')]
prices = prices[~prices.name.str.contains('Brian Nievas')]
prices = prices[~prices.name.str.contains('BruninhoBruninho')]
prices = prices[~prices.name.str.contains('Bruno Andrade')]
prices = prices[~prices.name.str.contains('Bruno MendezBruno Mendez')]
prices = prices[~prices.name.str.contains('Bruno Miguel Rodrigues')]
prices = prices[~prices.name.str.contains('Bryce Hosannah')]
prices = prices[~prices.name.str.contains('Bryce Washington')]
prices = prices[~prices.name.str.contains('BuenoBueno')]
prices = prices[~prices.name.str.contains('Buiu')]
prices = prices[~prices.name.str.contains('CaduCadu*')]
prices = prices[~prices.name.str.contains('CafuCafu')]
prices = prices[~prices.name.str.contains('Cain Fara')]
prices = prices[~prices.name.str.contains('Callum Jones')]
prices = prices[~prices.name.str.contains('Callum King-Harmes')]
prices = prices[~prices.name.str.contains('Camacho')]
prices = prices[~prices.name.str.contains('Cameron Antwi')]
prices = prices[~prices.name.str.contains('Cammy Ballantyne')]
prices = prices[~prices.name.str.contains('Cammy Smith')]
prices = prices[~prices.name.str.contains('Cenk Sahin')]
prices = prices[~prices.name.str.contains('Cesar IbaÃ±ez')]
prices = prices[~prices.name.str.contains('Charalampos Makridis')]
prices = prices[~prices.name.str.contains('Charlie Owens')]
prices = prices[~prices.name.str.contains('Charly Jan')]
prices = prices[~prices.name.str.contains('Chase Gasper')]
prices = prices[~prices.name.str.contains('Christopher Garcia')]
prices = prices[~prices.name.str.contains('Christopher Gloster')]
prices = prices[~prices.name.str.contains('Clint Irwin')]
prices = prices[~prices.name.str.contains('Clint Leemans')]
prices = prices[~prices.name.str.contains('Connor Shanks')]
prices = prices[~prices.name.str.contains('Connor Taylor')]
prices = prices[~prices.name.str.contains('Couhaib Driouech')]
prices = prices[~prices.name.str.contains('Courtney Baker-Richardson')]
prices = prices[~prices.name.str.contains('Damian Perez')]
prices = prices[~prices.name.str.contains('Damian Schmidt')]
prices = prices[~prices.name.str.contains('Daniel Kemp')]
prices = prices[~prices.name.str.contains('Daniel Leadbitter')]
prices = prices[~prices.name.str.contains('Daniel Vega')]
prices = prices[~prices.name.str.contains('Daniel Vukovic')]
prices = prices[~prices.name.str.contains('Danil Prucev')]
prices = prices[~prices.name.str.contains('Danil Prutsev')]
prices = prices[~prices.name.str.contains('Danilo Fernandes')]
prices = prices[~prices.name.str.contains('Danilo OrtizDavid Henen')]
prices = prices[~prices.name.str.contains('David Jensen')]
prices = prices[~prices.name.str.contains('David Junior Hoilett')]
prices = prices[~prices.name.str.contains('David Karaev')]
prices = prices[~prices.name.str.contains('David Kong')]
prices = prices[~prices.name.str.contains('David Lennart Philipp')]
prices = prices[~prices.name.str.contains('David Loera')]
prices = prices[~prices.name.str.contains('David Martin')]
prices = prices[~prices.name.str.contains('David Pavelka')]
prices = prices[~prices.name.str.contains('David Philipp')]
prices = prices[~prices.name.str.contains('David Sualehe')]
prices = prices[~prices.name.str.contains('David Tavares')]
prices = prices[~prices.name.str.contains('Davide Faraoni')]
prices = prices[~prices.name.str.contains('Davide Ghislandi')]
prices = prices[~prices.name.str.contains('Davide Masella')]
prices = prices[~prices.name.str.contains('Dayro Moreno')]
prices = prices[~prices.name.str.contains('Declan Drysdale')]
prices = prices[~prices.name.str.contains('Deian Veron')]
prices = prices[~prices.name.str.contains('Dejan Galjen')]
prices = prices[~prices.name.str.contains('Dejan Kerkez')]
prices = prices[~prices.name.str.contains('Deji Sotona')]
prices = prices[~prices.name.str.contains('Delano Vianello')]
prices = prices[~prices.name.str.contains('Demba Ba')]
prices = prices[~prices.name.str.contains('Denis Adamov')]
prices = prices[~prices.name.str.contains('Denis Franchi')]
prices = prices[~prices.name.str.contains('Denis Petric')]
prices = prices[~prices.name.str.contains('Dennis Adeniran')]
prices = prices[~prices.name.str.contains('Dennis Gorka')]
prices = prices[~prices.name.str.contains('Dennis Jastrzembski')]
prices = prices[~prices.name.str.contains('Dennis Politic')]
prices = prices[~prices.name.str.contains('Denys Bain')]
prices = prices[~prices.name.str.contains('Denzel Jubitana')]
prices = prices[~prices.name.str.contains('Derek Dodson')]
prices = prices[~prices.name.str.contains('DerickDerick')]
prices = prices[~prices.name.str.contains('Derrick Tshimanga')]
prices = prices[~prices.name.str.contains('Desevio Payne')]
prices = prices[~prices.name.str.contains('Destiny Udogie')]
prices = prices[~prices.name.str.contains('Dialy Ndiaye')]
prices = prices[~prices.name.str.contains('Diant Ramaj')]
prices = prices[~prices.name.str.contains('DidiDidi')]
prices = prices[~prices.name.str.contains('Diego Braghieri')]
prices = prices[~prices.name.str.contains('Diego Calcaterra')]
prices = prices[~prices.name.str.contains('Diego Churin')]
prices = prices[~prices.name.str.contains('Diego FerreiraDiego Ferreira')]
prices = prices[~prices.name.str.contains('Diego Lopes')]
prices = prices[~prices.name.str.contains('Diego Mendoza')]
prices = prices[~prices.name.str.contains('Diego Novaretti')]
prices = prices[~prices.name.str.contains('Diogo Teixeira')]
prices = prices[~prices.name.str.contains('Dion McGhee')]
prices = prices[~prices.name.str.contains('Dion Moise Sahi')]
prices = prices[~prices.name.str.contains('Dion Sanderson')]
prices = prices[~prices.name.str.contains('Dior Angus')]
prices = prices[~prices.name.str.contains('Dixon Renteria')]
prices = prices[~prices.name.str.contains('Donis Avdijaj')]
prices = prices[~prices.name.str.contains('Douglas Friedrich')]
prices = prices[~prices.name.str.contains('Duncan Turnbull')]
prices = prices[~prices.name.str.contains('Duncan Turnbull')]
prices = prices[~prices.name.str.contains('Dusan Jovancic')]
prices = prices[~prices.name.str.contains('Dyego Sousa')]
prices = prices[~prices.name.str.contains('Dylan Asonganyi')]
prices = prices[~prices.name.str.contains('Dylan Borrero')]
prices = prices[~prices.name.str.contains('Dylan Castanheira')]
prices = prices[~prices.name.str.contains('Dylan Levitt')]
prices = prices[~prices.name.str.contains('Dylan Mottley-Henry')]
prices = prices[~prices.name.str.contains('Dylan Reid')]
prices = prices[~prices.name.str.contains('Dylan Vente')]
prices = prices[~prices.name.str.contains('EÃ±aut Mendia')]
prices = prices[~prices.name.str.contains('Ebrima Darboe')]
prices = prices[~prices.name.str.contains('Eddie Nolan')]
prices = prices[~prices.name.str.contains('Edgar Castillo')]
prices = prices[~prices.name.str.contains('Edison Duarte')]
prices = prices[~prices.name.str.contains('Edoardo Bove')]
prices = prices[~prices.name.str.contains('Edwin Banguera')]
prices = prices[~prices.name.str.contains('Edwin Cardona')]
prices = prices[~prices.name.str.contains('Edwin Mosquera')]
prices = prices[~prices.name.str.contains('Egemen Pehlivan')]
prices = prices[~prices.name.str.contains('Egor Generalov')]
prices = prices[~prices.name.str.contains('Egor Prutsev')]
prices = prices[~prices.name.str.contains('Ekrem Kilicarslan')]
prices = prices[~prices.name.str.contains('El Mami Tetah')]
prices = prices[~prices.name.str.contains('Elbasan Rashani')]
prices = prices[~prices.name.str.contains('Eldin Jakupovic')]
prices = prices[~prices.name.str.contains('Elia Caprile')]
prices = prices[~prices.name.str.contains('Elias Herzig')]
prices = prices[~prices.name.str.contains('Elias Manoel')]
prices = prices[~prices.name.str.contains('Elias Oubella')]
prices = prices[~prices.name.str.contains('Elias Pereyra')]
prices = prices[~prices.name.str.contains('Elliot Anderson')]
prices = prices[~prices.name.str.contains('Elliott Bolton')]
prices = prices[~prices.name.str.contains('Elliott Bonds')]
prices = prices[~prices.name.str.contains('Ellis Simms')]
prices = prices[~prices.name.str.contains('ElzoElzo')]
prices = prices[~prices.name.str.contains('Emanuel Herrera')]
prices = prices[~prices.name.str.contains('Emanuel Insua')]
prices = prices[~prices.name.str.contains('Emanuel Olivera')]
prices = prices[~prices.name.str.contains('Emanuel Rosales')]
prices = prices[~prices.name.str.contains('Emir Senocak')]
prices = prices[~prices.name.str.contains('Emirhan Kascioglu')]
prices = prices[~prices.name.str.contains('Emmanuel Boateng')]
prices = prices[~prices.name.str.contains('Emmanuel Gyabuaa')]
prices = prices[~prices.name.str.contains('Emmanuel Hackman')]
prices = prices[~prices.name.str.contains('Emmanuel Idem')]
prices = prices[~prices.name.str.contains('Emmanuel Monthe')]
prices = prices[~prices.name.str.contains('Emmanuel Ochoa')]
prices = prices[~prices.name.str.contains('Emmanuel Ojeda')]
prices = prices[~prices.name.str.contains('Emeka Obi')]
prices = prices[~prices.name.str.contains('Emerson Royal')]
prices = prices[~prices.name.str.contains('Emiliano Insua')]
prices = prices[~prices.name.str.contains('Emiliano Vecchio')]
prices = prices[~prices.name.str.contains('Emin Bayram')]
prices = prices[~prices.name.str.contains('Emrah Bassan')]
prices = prices[~prices.name.str.contains('Emre Bilgin')]
prices = prices[~prices.name.str.contains('Emre Colak')]
prices = prices[~prices.name.str.contains('Emre Karaal')]
prices = prices[~prices.name.str.contains('Enes Karakus')]
prices = prices[~prices.name.str.contains('Enes Sancar Sahin')]
prices = prices[~prices.name.str.contains('Ennio van der Gouw')]
prices = prices[~prices.name.str.contains('Enrico Hernandez')]
prices = prices[~prices.name.str.contains('Enver Cenk Sahin')]
prices = prices[~prices.name.str.contains('Enzo Coacci')]
prices = prices[~prices.name.str.contains('Enzo Copetti')]
prices = prices[~prices.name.str.contains('Enzo Fernandez')]
prices = prices[~prices.name.str.contains('Eray Iscan')]
prices = prices[~prices.name.str.contains('Erdin Demir')]
prices = prices[~prices.name.str.contains('Erdogan Kaya')]
prices = prices[~prices.name.str.contains('Eren Elmali')]
prices = prices[~prices.name.str.contains('Eren Kiryolcu')]
prices = prices[~prices.name.str.contains('Eric Bocat')]
prices = prices[~prices.name.str.contains('Eric Calvillo')]
prices = prices[~prices.name.str.contains('Eric DickEric Dick')]
prices = prices[~prices.name.str.contains('Eric Bocat')]
prices = prices[~prices.name.str.contains('Eric Calvillo')]
prices = prices[~prices.name.str.contains('Eric Ebimbe')]
prices = prices[~prices.name.str.contains('Eric Junior Bocat')]
prices = prices[~prices.name.str.contains('Eric Lichaj')]
prices = prices[~prices.name.str.contains('Eric Lopez')]
prices = prices[~prices.name.str.contains('Eric MezaEric Meza')]
prices = prices[~prices.name.str.contains('Eric Miller')]
prices = prices[~prices.name.str.contains('Erick Ferigra')]
prices = prices[~prices.name.str.contains('Ernest Boahene')]
prices = prices[~prices.name.str.contains('Erwan Belhadji')]
prices = prices[~prices.name.str.contains('Esteban Conde')]
prices = prices[~prices.name.str.contains('Esteban Rolon')]
prices = prices[~prices.name.str.contains('Ethan Bartlow')]
prices = prices[~prices.name.str.contains('Ethan Johnston')]
prices = prices[~prices.name.str.contains('Ethan Laird')]
prices = prices[~prices.name.str.contains('Etienne Vaessen')]
prices = prices[~prices.name.str.contains('Eugenio Pizzuto')]
prices = prices[~prices.name.str.contains('Evan BushEvan Bush')]
prices = prices[~prices.name.str.contains('Evan Newton')]
prices = prices[~prices.name.str.contains('Evann Guessand')]
prices = prices[~prices.name.str.contains('EveraldoEveraldo')]
prices = prices[~prices.name.str.contains('Evert Linthorst')]
prices = prices[~prices.name.str.contains('Everton Felipe')]
prices = prices[~prices.name.str.contains('Ezequiel Cerica')]
prices = prices[~prices.name.str.contains('Ezequiel Cirigliano')]
prices = prices[~prices.name.str.contains('Ezequiel Edison Rescaldani')]
prices = prices[~prices.name.str.contains('Ezequiel Fernandez')]
prices = prices[~prices.name.str.contains('Ezequiel Piovi')]
prices = prices[~prices.name.str.contains('Ezequiel Schelotto')]
prices = prices[~prices.name.str.contains('Ezequiel Unsain')]
prices = prices[~prices.name.str.contains('FaÃ¯z Selemani')]
prices = prices[~prices.name.str.contains('Fabian Cubero')]
prices = prices[~prices.name.str.contains('Fabian de Keijzer')]
prices = prices[~prices.name.str.contains('Fabian Kalig')]
prices = prices[~prices.name.str.contains('Fabian Noguera')]
prices = prices[~prices.name.str.contains('Fabien Farnolle')]
prices = prices[~prices.name.str.contains('Fabio Abreu')]
prices = prices[~prices.name.str.contains('Fabio Carvalho')]
prices = prices[~prices.name.str.contains('Fabio China')]
prices = prices[~prices.name.str.contains('Fabio ChinaFabio China')]
prices = prices[~prices.name.str.contains('Fabio Pereyra')]
prices = prices[~prices.name.str.contains('Fabio Pisacane')]
prices = prices[~prices.name.str.contains('Fabio SantosFabio Santos')]
prices = prices[~prices.name.str.contains('Fabio Schneider')]
prices = prices[~prices.name.str.contains('Fabio Vazquez')]
prices = prices[~prices.name.str.contains('Fabrice Hartmann')]
prices = prices[~prices.name.str.contains('Fabricio Dominguez')]
prices = prices[~prices.name.str.contains('Fabricio Fontanini')]
prices = prices[~prices.name.str.contains('FabriFabri')]
prices = prices[~prices.name.str.contains('Fabrizio Caligara')]
prices = prices[~prices.name.str.contains('Facundo Altamirano')]
prices = prices[~prices.name.str.contains('Facundo Barboza')]
prices = prices[~prices.name.str.contains('Facundo Bertoglio')]
prices = prices[~prices.name.str.contains('Facundo Cambeses')]
prices = prices[~prices.name.str.contains('Facundo Castet')]
prices = prices[~prices.name.str.contains('Facundo Curuchet')]
prices = prices[~prices.name.str.contains('Facundo Fernandez')]
prices = prices[~prices.name.str.contains('Facundo Ferrero')]
prices = prices[~prices.name.str.contains('Facundo Garces')]
prices = prices[~prices.name.str.contains('Facundo Mater')]
prices = prices[~prices.name.str.contains('Facundo Melivilo')]
prices = prices[~prices.name.str.contains('Facundo Pons')]
prices = prices[~prices.name.str.contains('Facundo Sanchez')]
prices = prices[~prices.name.str.contains('Facundo Tobares')]
prices = prices[~prices.name.str.contains('Faiq Bolkiah')]
prices = prices[~prices.name.str.contains('Faiz Selemani')]
prices = prices[~prices.name.str.contains('Falcao')]
prices = prices[~prices.name.str.contains('Fali')]
prices = prices[~prices.name.str.contains('FaliFali')]
prices = prices[~prices.name.str.contains('Famana Quizera')]
prices = prices[~prices.name.str.contains('Fatih Yigit Sanliturk')]
prices = prices[~prices.name.str.contains('Favio Cabral')]
prices = prices[~prices.name.str.contains('Fede Varela')]
prices = prices[~prices.name.str.contains('Federico Andueza')]
prices = prices[~prices.name.str.contains('Federico Bonansea')]
prices = prices[~prices.name.str.contains('Federico Brancolini')]
prices = prices[~prices.name.str.contains('Federico Gattoni')]
prices = prices[~prices.name.str.contains('Federico Gino')]
prices = prices[~prices.name.str.contains('Federico Girotti')]
prices = prices[~prices.name.str.contains('Federico Lanzillotta')]
prices = prices[~prices.name.str.contains('Federico Mancuello')]
prices = prices[~prices.name.str.contains('Federico Martinez')]
prices = prices[~prices.name.str.contains('Federico Palacios')]
prices = prices[~prices.name.str.contains('Federico Ricci')]
prices = prices[~prices.name.str.contains('Federico Vera')]
prices = prices[~prices.name.str.contains('Federico Vietto')]
prices = prices[~prices.name.str.contains('Federico Vismara')]
prices = prices[~prices.name.str.contains('Felipe Aguilar')]
prices = prices[~prices.name.str.contains('Felipe Martins')]
prices = prices[~prices.name.str.contains('Felipe PeÃ±a Biafore')]
prices = prices[~prices.name.str.contains('Felipe Santana')]
prices = prices[~prices.name.str.contains('Felipe Valencia')]
prices = prices[~prices.name.str.contains('Felipe VizeuFelipe Vizeu')]
prices = prices[~prices.name.str.contains('Felix Correia')]
prices = prices[~prices.name.str.contains('Felix Irorere')]
prices = prices[~prices.name.str.contains('Femi Azeez')]
prices = prices[~prices.name.str.contains('Femi Seriki')]
prices = prices[~prices.name.str.contains('Fer NiÃ±oFer NiÃ±o')]
prices = prices[~prices.name.str.contains('Ferhat Erdogen')]
prices = prices[~prices.name.str.contains('Ferhat GÃ¶rgulu')]
prices = prices[~prices.name.str.contains('Ferhat Kaplan')]
prices = prices[~prices.name.str.contains('Fernando Andrade')]
prices = prices[~prices.name.str.contains('Fernando Caixeta')]
prices = prices[~prices.name.str.contains('Fernando Canesin')]
prices = prices[~prices.name.str.contains('Fernando Coniglio')]
prices = prices[~prices.name.str.contains('Fernando Cosciuc')]
prices = prices[~prices.name.str.contains('Fernando Evangelista')]
prices = prices[~prices.name.str.contains('Fernando Gago')]
prices = prices[~prices.name.str.contains('Fernando Luna')]
prices = prices[~prices.name.str.contains('Fernando Meza')]
prices = prices[~prices.name.str.contains('Fernando Monetti')]
prices = prices[~prices.name.str.contains('Fernando NiÃ±o')]
prices = prices[~prices.name.str.contains('Fernando Pellegrino')]
prices = prices[~prices.name.str.contains('Fernando Reges')]
prices = prices[~prices.name.str.contains('Fernando Roman')]
prices = prices[~prices.name.str.contains('Fernando Sobral')]
prices = prices[~prices.name.str.contains('Fernando Tobio')]
prices = prices[~prices.name.str.contains('FerreiraFerreira')]
prices = prices[~prices.name.str.contains('Festus Arthur')]
prices = prices[~prices.name.str.contains('Festy Ebosele')]
prices = prices[~prices.name.str.contains('Filippo Rinaldi')]
prices = prices[~prices.name.str.contains('Filippo Romagna')]
prices = prices[~prices.name.str.contains('Fin Stevens')]
prices = prices[~prices.name.str.contains('Finlay Hurford-Lockett')]
prices = prices[~prices.name.str.contains('Finley Craske')]
prices = prices[~prices.name.str.contains('Finley Shrimpton')]
prices = prices[~prices.name.str.contains('Finley Stevens')]
prices = prices[~prices.name.str.contains('Finn Azaz')]
prices = prices[~prices.name.str.contains('Florent da Silva')]
prices = prices[~prices.name.str.contains('Florent Hoti')]
prices = prices[~prices.name.str.contains('Florian Flecker')]
prices = prices[~prices.name.str.contains('Florian Kamberi')]
prices = prices[~prices.name.str.contains('Florian Monzon')]
prices = prices[~prices.name.str.contains('Florian Stritzel')]
prices = prices[~prices.name.str.contains('Florin Andone')]
prices = prices[~prices.name.str.contains('Foster Langsdorf')]
prices = prices[~prices.name.str.contains('Francesco Rossi')]
prices = prices[~prices.name.str.contains('Francisco Apaolaza')]
prices = prices[~prices.name.str.contains('Francisco Flores')]
prices = prices[~prices.name.str.contains('Francisco Fydriszewski')]
prices = prices[~prices.name.str.contains('Francisco Geraldes')]
prices = prices[~prices.name.str.contains('Francisco Ilarregui')]
prices = prices[~prices.name.str.contains('Francisco Manenti')]
prices = prices[~prices.name.str.contains('Francisco Meixedo')]
prices = prices[~prices.name.str.contains('Francisco Montero')]
prices = prices[~prices.name.str.contains('Francisco Ramirez')]
prices = prices[~prices.name.str.contains('Francisco Silva')]
prices = prices[~prices.name.str.contains('Franco Baldassarra')]
prices = prices[~prices.name.str.contains('Franco Di Santo')]
prices = prices[~prices.name.str.contains('Franco Escobar')]
prices = prices[~prices.name.str.contains('Franco Godoy')]
prices = prices[~prices.name.str.contains('Franco Gonzalez')]
prices = prices[~prices.name.str.contains('Franco Leys')]
prices = prices[~prices.name.str.contains('Franco Negri')]
prices = prices[~prices.name.str.contains('Franco Orozco')]
prices = prices[~prices.name.str.contains('Franco Paredes')]
prices = prices[~prices.name.str.contains('Franco Perez')]
prices = prices[~prices.name.str.contains('Franco Perinciolo')]
prices = prices[~prices.name.str.contains('Franco Pizzicanella')]
prices = prices[~prices.name.str.contains('Franco Quiroz')]
prices = prices[~prices.name.str.contains('Franco Rivasseau')]
prices = prices[~prices.name.str.contains('Franco Tongya')]
prices = prices[~prices.name.str.contains('Francois Marquet')]
prices = prices[~prices.name.str.contains('Francois-Xavier Fumu Tamuzo')]
prices = prices[~prices.name.str.contains('Frank Fielding')]
prices = prices[~prices.name.str.contains('Frank Tsadjout')]
prices = prices[~prices.name.str.contains('Frank Vincent')]
prices = prices[~prices.name.str.contains('Franko Kovacevic')]
prices = prices[~prices.name.str.contains('Fraser Murray')]
prices = prices[~prices.name.str.contains('Fred Emmings')]
prices = prices[~prices.name.str.contains('Frederik Alves')]
prices = prices[~prices.name.str.contains('Fredrik Oppegard')]
prices = prices[~prices.name.str.contains('Furkan Beklevic')]
prices = prices[~prices.name.str.contains('Furkan Kara')]
prices = prices[~prices.name.str.contains('Fynn OttoFynn Otto')]
prices = prices[~prices.name.str.contains('Gabriel Barbosa')]
prices = prices[~prices.name.str.contains('Gabriel Batista')]
prices = prices[~prices.name.str.contains('Gabriel Boschilia')]
prices = prices[~prices.name.str.contains('Gabriel Carrasco')]
prices = prices[~prices.name.str.contains('Gabriel Chapeco')]
prices = prices[~prices.name.str.contains('Gabriel Compagnucci')]
prices = prices[~prices.name.str.contains('Gabriel Dias')]
prices = prices[~prices.name.str.contains('Gabriel Diaz')]
prices = prices[~prices.name.str.contains('Gabriel Esparza')]
prices = prices[~prices.name.str.contains('Gabriel Falcao')]
prices = prices[~prices.name.str.contains('Gabriel Hachen')]
prices = prices[~prices.name.str.contains('Gabriel Pires')]
prices = prices[~prices.name.str.contains('Gabriel Rojas')]
prices = prices[~prices.name.str.contains('Gabriel Slonina')]
prices = prices[~prices.name.str.contains('Gabriel Veiga')]
prices = prices[~prices.name.str.contains('Gabriel VeronGabriel Veron')]
prices = prices[~prices.name.str.contains('Gaetan Poussin')]
prices = prices[~prices.name.str.contains('GalenoGalenoLeft Midfiel')]
prices = prices[~prices.name.str.contains('Gamid Agalarov')]
prices = prices[~prices.name.str.contains('GansoGanso')]
prices = prices[~prices.name.str.contains('Gary Maley')]
prices = prices[~prices.name.str.contains('Gary Roberts')]
prices = prices[~prices.name.str.contains('Gary Sawyer')]
prices = prices[~prices.name.str.contains('Gaston Ãlvarez Suarez')]
prices = prices[~prices.name.str.contains('Gaston Benavidez')]
prices = prices[~prices.name.str.contains('Gaston Diaz')]
prices = prices[~prices.name.str.contains('Gaston Fernandez')]
prices = prices[~prices.name.str.contains('Gaston Gerzel')]
prices = prices[~prices.name.str.contains('Gaston Gomez')]
prices = prices[~prices.name.str.contains('Gaston Gonzalez')]
prices = prices[~prices.name.str.contains('Gaston Suso')]
prices = prices[~prices.name.str.contains('Gaston Torres')]
prices = prices[~prices.name.str.contains('Gavin Bazunu')]
prices = prices[~prices.name.str.contains('Gedion Zelalem')]
prices = prices[~prices.name.str.contains('GeilsonGeilson')]
prices = prices[~prices.name.str.contains('Gela Zaseev')]
prices = prices[~prices.name.str.contains('George Acosta')]
prices = prices[~prices.name.str.contains('George Asomani')]
prices = prices[~prices.name.str.contains('George Campbell')]
prices = prices[~prices.name.str.contains('George Fochive')]
prices = prices[~prices.name.str.contains('George Hirst')]
prices = prices[~prices.name.str.contains('George Melkadze')]
prices = prices[~prices.name.str.contains('George Nugent')]
prices = prices[~prices.name.str.contains('Georges Mandjeck')]
prices = prices[~prices.name.str.contains('Georgi Dzhikiya')]
prices = prices[~prices.name.str.contains('Getterson')]
prices = prices[~prices.name.str.contains('Giacomo Ricci')]
prices = prices[~prices.name.str.contains('Giacomo Vrioni')]
prices = prices[~prices.name.str.contains('Gian-Luca Itter')]
prices = prices[~prices.name.str.contains('Gian-Luca Waldschmidt')]
prices = prices[~prices.name.str.contains('Giancarlo Gonzalez')]
prices = prices[~prices.name.str.contains('Gianfranco Facchineri')]
prices = prices[~prices.name.str.contains('Giannelli Imbula')]
prices = prices[~prices.name.str.contains('Gianni Mollo')]
prices = prices[~prices.name.str.contains('Giorgi Kochorashvili')]
prices = prices[~prices.name.str.contains('Giovanni Crociata')]
prices = prices[~prices.name.str.contains('Giuseppe Mastinu')]
prices = prices[~prices.name.str.contains('Gjoko Zajkov')]
prices = prices[~prices.name.str.contains('Gokhan Kardes')]
prices = prices[~prices.name.str.contains('Gonzalo Asis')]
prices = prices[~prices.name.str.contains('Gonzalo Castellani')]
prices = prices[~prices.name.str.contains('Gonzalo GoÃ±i')]
prices = prices[~prices.name.str.contains('Gonzalo Jara')]
prices = prices[~prices.name.str.contains('Gonzalo Maroni')]
prices = prices[~prices.name.str.contains('Gonzalo Rodriguez')]
prices = prices[~prices.name.str.contains('Gonzalo Veron')]
prices = prices[~prices.name.str.contains('Grace Bokake')]
prices = prices[~prices.name.str.contains('Graham Smith')]
prices = prices[~prices.name.str.contains('Grant Lillard')]
prices = prices[~prices.name.str.contains('Grayson Barber')]
prices = prices[~prices.name.str.contains('Greg Ranjitsingh')]
prices = prices[~prices.name.str.contains('Griffin Yow')]
prices = prices[~prices.name.str.contains('Guido Mainero')]
prices = prices[~prices.name.str.contains('Guido Vadala')]
prices = prices[~prices.name.str.contains('Guilherme Camacho')]
prices = prices[~prices.name.str.contains('Guilherme GuedesGuilherme Guedes')]
prices = prices[~prices.name.str.contains('Guilherme Lazaroni')]
prices = prices[~prices.name.str.contains('Guilherme Mantuan')]
prices = prices[~prices.name.str.contains('Guilherme Oliveira')]
prices = prices[~prices.name.str.contains('Guillem Molina')]
prices = prices[~prices.name.str.contains('Guillermo Celis')]
prices = prices[~prices.name.str.contains('Guillermo Fernandez')]
prices = prices[~prices.name.str.contains('Guillermo Gozalvez')]
prices = prices[~prices.name.str.contains('Guram Tetrashvili')]
prices = prices[~prices.name.str.contains('GustavinhoGustavinho')]
prices = prices[~prices.name.str.contains('Gustavo ApisGustavo Apis')]
prices = prices[~prices.name.str.contains('Gustavo BlancoGustavo Blanco')]
prices = prices[~prices.name.str.contains('Gustavo Canto')]
prices = prices[~prices.name.str.contains('Gustavo CoutinhoGustavo Coutinho')]
prices = prices[~prices.name.str.contains('Gustavo Garcia')]
prices = prices[~prices.name.str.contains('Gustavo Henrique')]
prices = prices[~prices.name.str.contains('Gustavo Mosquito')]
prices = prices[~prices.name.str.contains('Gustavo NescauGustavo Nescau')]
prices = prices[~prices.name.str.contains('Gustavo SilvaGustavo Silva')]
prices = prices[~prices.name.str.contains('Guy Mbenza')]
prices = prices[~prices.name.str.contains('Haqi Osman')]
prices = prices[~prices.name.str.contains('Harlem Gnohere')]
prices = prices[~prices.name.str.contains('Harry Clarke')]
prices = prices[~prices.name.str.contains('Harry Draper')]
prices = prices[~prices.name.str.contains('Harry Isted')]
prices = prices[~prices.name.str.contains('Harry Lewis')]
prices = prices[~prices.name.str.contains('Harry McHugh')]
prices = prices[~prices.name.str.contains('Harry Paton')]
prices = prices[~prices.name.str.contains('Harry Vaughan')]
prices = prices[~prices.name.str.contains('Harry Wood')]
prices = prices[~prices.name.str.contains('Harry Wright')]
prices = prices[~prices.name.str.contains('Harvey Bunker')]
prices = prices[~prices.name.str.contains('Harvey Cartwright')]
prices = prices[~prices.name.str.contains('Harvey Elliott')]
prices = prices[~prices.name.str.contains('Harvey Rew')]
prices = prices[~prices.name.str.contains('Harvey White')]
prices = prices[~prices.name.str.contains('Hayden Carter')]
prices = prices[~prices.name.str.contains('Hayden Hackney')]
prices = prices[~prices.name.str.contains('Hayden Muller')]
prices = prices[~prices.name.str.contains('Haydon Roberts')]
prices = prices[~prices.name.str.contains('Helder Barbosa')]
prices = prices[~prices.name.str.contains('Henning Matriciani')]
prices = prices[~prices.name.str.contains('Henri Saivet')]
prices = prices[~prices.name.str.contains('Henrich Ravas')]
prices = prices[~prices.name.str.contains('Henrique Trevisan')]
prices = prices[~prices.name.str.contains('Hernan Burbano')]
prices = prices[~prices.name.str.contains('Hernan Lamberti')]
prices = prices[~prices.name.str.contains('Hernan Lopez')]
prices = prices[~prices.name.str.contains('Hicham Mahou')]
prices = prices[~prices.name.str.contains('Hugo Novoa')]
prices = prices[~prices.name.str.contains('Hugo Rodallega')]
prices = prices[~prices.name.str.contains('Hugo Silveira')]
prices = prices[~prices.name.str.contains('Hugo Sotelo')]
prices = prices[~prices.name.str.contains('Hugo SouzaHugo Souza')]
prices = prices[~prices.name.str.contains('Iago MaidanaIago Maidana')]
prices = prices[~prices.name.str.contains('Ian Escobar')]
prices = prices[~prices.name.str.contains('Ian FrayIan Fray')]
prices = prices[~prices.name.str.contains('Ian Hoffmann')]
prices = prices[~prices.name.str.contains('Ian Maatsen')]
prices = prices[~prices.name.str.contains('Ian Poveda')]
prices = prices[~prices.name.str.contains('Ian Poveda-Ocampo')]
prices = prices[~prices.name.str.contains('IbrahimaIbrahima')]
prices = prices[~prices.name.str.contains('Idris Umaev')]
prices = prices[~prices.name.str.contains('Idrissa Dioh')]
prices = prices[~prices.name.str.contains('Idrissa Sylla')]
prices = prices[~prices.name.str.contains('Ifunanyachi Achara')]
prices = prices[~prices.name.str.contains('Ignacio Miramon')]
prices = prices[~prices.name.str.contains('Ignacio Russo')]
prices = prices[~prices.name.str.contains('Ignacio Schor')]
prices = prices[~prices.name.str.contains('Igor Lucatelli')]
prices = prices[~prices.name.str.contains('Igor Obukhov')]
prices = prices[~prices.name.str.contains('Ilhan Depe')]
prices = prices[~prices.name.str.contains('Ilia Shkurin')]
prices = prices[~prices.name.str.contains('Iliman Ndiaye')]
prices = prices[~prices.name.str.contains('Isaac Olaofe')]
prices = prices[~prices.name.str.contains('Isaac Vassell')]
prices = prices[~prices.name.str.contains('Isaiah Brown')]
prices = prices[~prices.name.str.contains('Jakub Niemczyk')]
prices = prices[~prices.name.str.contains('Jakub Szumski')]
prices = prices[~prices.name.str.contains('Jalen Neal')]
prices = prices[~prices.name.str.contains('James Clark')]
prices = prices[~prices.name.str.contains('James Dodd')]
prices = prices[~prices.name.str.contains('James Garner')]
prices = prices[~prices.name.str.contains('James Gullan')]
prices = prices[~prices.name.str.contains('James Marcinkowski')]
prices = prices[~prices.name.str.contains('James Montgomery')]
prices = prices[~prices.name.str.contains('James Morton')]
prices = prices[~prices.name.str.contains('James Olayinka')]
prices = prices[~prices.name.str.contains('James Pardington')]
prices = prices[~prices.name.str.contains('James Rowland')]
prices = prices[~prices.name.str.contains('James Vennings')]
prices = prices[~prices.name.str.contains('Jamie Armstrong')]
prices = prices[~prices.name.str.contains('Jamie Barjonas')]
prices = prices[~prices.name.str.contains('Jamie Cumming')]
prices = prices[~prices.name.str.contains('Jamie Fielding')]
prices = prices[~prices.name.str.contains('Jamie Gullan')]
prices = prices[~prices.name.str.contains('Jamie Mascoll')]
prices = prices[~prices.name.str.contains('Jamie Semple')]
prices = prices[~prices.name.str.contains('Jamie Smith')]
prices = prices[~prices.name.str.contains('Jamie Soule')]
prices = prices[~prices.name.str.contains('Jamil Awoyejo')]
prices = prices[~prices.name.str.contains('Jan Bekkema')]
prices = prices[~prices.name.str.contains('Jan Carlos Hurtado')]
prices = prices[~prices.name.str.contains('Jan de Boer')]
prices = prices[~prices.name.str.contains('Jan Hurtado')]
prices = prices[~prices.name.str.contains('Jaquite')]
prices = prices[~prices.name.str.contains('Jarno Westerman')]
prices = prices[~prices.name.str.contains('Jaroslaw Jach')]
prices = prices[~prices.name.str.contains('Jaroslaw Niezgoda')]
prices = prices[~prices.name.str.contains('Jarvis Cleal')]
prices = prices[~prices.name.str.contains('Jason Cummings')]
prices = prices[~prices.name.str.contains('Jay-Dee Geusens')]
prices = prices[~prices.name.str.contains('Jayden Bennetts')]
prices = prices[~prices.name.str.contains('Jayden Campbell')]
prices = prices[~prices.name.str.contains('Jayden Mitchell-Lawson')]
prices = prices[~prices.name.str.contains('Jayden Onen')]
prices = prices[~prices.name.str.contains('Jayden Reid')]
prices = prices[~prices.name.str.contains('Jayden Richardson')]
prices = prices[~prices.name.str.contains('Jayson Molumby')]
prices = prices[~prices.name.str.contains('Jeison Chala')]
prices = prices[~prices.name.str.contains('Jeison Lucumi')]
prices = prices[~prices.name.str.contains('Jeizon Ramirez')]
prices = prices[~prices.name.str.contains('Jelle Duin')]
prices = prices[~prices.name.str.contains('Jens Jonsson')]
prices = prices[~prices.name.str.contains('Jeong Woo-Yeong')]
prices = prices[~prices.name.str.contains('Jeredy Hilterman')]
prices = prices[~prices.name.str.contains('Jeremy Antonisse')]
prices = prices[~prices.name.str.contains('Jeremy Helmer')]
prices = prices[~prices.name.str.contains('Jeremy Kelly')]
prices = prices[~prices.name.str.contains('Jeremy Perbet')]
prices = prices[~prices.name.str.contains('Jeremy Van Mullem')]
prices = prices[~prices.name.str.contains('Jerome Greaves')]
prices = prices[~prices.name.str.contains('Jerome Opoku')]
prices = prices[~prices.name.str.contains('Jerome Sinclair')]
prices = prices[~prices.name.str.contains('Jeronimo Pourtau')]
prices = prices[~prices.name.str.contains('Jesus Ferreira')]
prices = prices[~prices.name.str.contains('Jim McNulty')]
prices = prices[~prices.name.str.contains('Jimmy Knowles')]
prices = prices[~prices.name.str.contains('Jimmy McNulty')]
prices = prices[~prices.name.str.contains('Jiri Skalak')]
prices = prices[~prices.name.str.contains('Joao Jaquite')]
prices = prices[~prices.name.str.contains('Joao Klauss')]
prices = prices[~prices.name.str.contains('Joao Mendes')]
prices = prices[~prices.name.str.contains('Joao Monteiro')]
prices = prices[~prices.name.str.contains('Joao Queiros')]
prices = prices[~prices.name.str.contains('Joaquin Arzura')]
prices = prices[~prices.name.str.contains('Joaquin Blazquez')]
prices = prices[~prices.name.str.contains('Joaquin Hass')]
prices = prices[~prices.name.str.contains('Joaquin Mendive')]
prices = prices[~prices.name.str.contains('Joaquin Varela')]
prices = prices[~prices.name.str.contains('Jobi McAnuff')]
prices = prices[~prices.name.str.contains('Jock Curran')]
prices = prices[~prices.name.str.contains('Jodi Jones')]
prices = prices[~prices.name.str.contains('Jody Lukoki')]
prices = prices[~prices.name.str.contains('Joe Adams')]
prices = prices[~prices.name.str.contains('Joe Foulkes')]
prices = prices[~prices.name.str.contains('Joe Fryer')]
prices = prices[~prices.name.str.contains('Joe Garner')]
prices = prices[~prices.name.str.contains('Joe Gelhardt')]
prices = prices[~prices.name.str.contains('Joe Grayson')]
prices = prices[~prices.name.str.contains('Joe Hilton')]
prices = prices[~prices.name.str.contains('Joe Leesley')]
prices = prices[~prices.name.str.contains('Joel Coleman')]
prices = prices[~prices.name.str.contains('Joey Jones')]
prices = prices[~prices.name.str.contains('Johan Carbonero')]
prices = prices[~prices.name.str.contains('John Brooks')]
prices = prices[~prices.name.str.contains('John Donald')]
prices = prices[~prices.name.str.contains('John Nekadio')]
prices = prices[~prices.name.str.contains('John Patrick')]
prices = prices[~prices.name.str.contains('John Robertson')]
prices = prices[~prices.name.str.contains('John Yeboah')]
prices = prices[~prices.name.str.contains('Kacper Mialkowski')]
prices = prices[~prices.name.str.contains('Kacper Urbanski')]
prices = prices[~prices.name.str.contains('Kagan Moradaoglu')]
prices = prices[~prices.name.str.contains('Kai Koreniuk')]
prices = prices[~prices.name.str.contains('Kai McKenzie-Lyle')]
prices = prices[~prices.name.str.contains('Kane Crichlow')]
prices = prices[~prices.name.str.contains('Kang-in Lee')]
prices = prices[~prices.name.str.contains('Karamoko Dembele')]
prices = prices[~prices.name.str.contains('Kastriot Dermaku')]
prices = prices[~prices.name.str.contains('Katuku Tshimanga')]
prices = prices[~prices.name.str.contains('Kayden Pierre')]
prices = prices[~prices.name.str.contains('Kayne Ramsay')]
prices = prices[~prices.name.str.contains('Kazaiah Sterling')]
prices = prices[~prices.name.str.contains('Keaghan Jacobs')]
prices = prices[~prices.name.str.contains('Keanu Staude')]
prices = prices[~prices.name.str.contains('Keito Nakamura')]
prices = prices[~prices.name.str.contains('Kelechi Nwakali')]
prices = prices[~prices.name.str.contains('Kelland Watts')]
prices = prices[~prices.name.str.contains('Kelsey Mooney')]
prices = prices[~prices.name.str.contains('Kemal Ruzgar')]
prices = prices[~prices.name.str.contains('Kendall McIntosh')]
prices = prices[~prices.name.str.contains('Kenneth Dougall')]
prices = prices[~prices.name.str.contains('Kenneth Kronholm')]
prices = prices[~prices.name.str.contains('Kenny Coker')]
prices = prices[~prices.name.str.contains('Kenny Dougall')]
prices = prices[~prices.name.str.contains('Kenny Lala')]
prices = prices[~prices.name.str.contains('Kenny Nagera')]
prices = prices[~prices.name.str.contains('Kevin Berkoe')]
prices = prices[~prices.name.str.contains('Kevin Boma')]
prices = prices[~prices.name.str.contains('Kevin Goden')]
prices = prices[~prices.name.str.contains('Kevin Harr')]
prices = prices[~prices.name.str.contains('Kevin Ibrahim')]
prices = prices[~prices.name.str.contains('Kevin Isa Luna')]
prices = prices[~prices.name.str.contains('Kevin Lankford')]
prices = prices[~prices.name.str.contains('Kevin McDonald')]
prices = prices[~prices.name.str.contains('Kevin Molino')]
prices = prices[~prices.name.str.contains('Kevin Nickler')]
prices = prices[~prices.name.str.contains('Kevin Silva')]
prices = prices[~prices.name.str.contains('Kevin Zenon')]
prices = prices[~prices.name.str.contains('Khalil Caraballo')]
prices = prices[~prices.name.str.contains('Kieran Agard')]
prices = prices[~prices.name.str.contains('Kieran Ngwenya')]
prices = prices[~prices.name.str.contains('Kim Moon-Hwan')]
prices = prices[~prices.name.str.contains('Kingsley Fobi')]
prices = prices[~prices.name.str.contains('Kiril Despodov')]
prices = prices[~prices.name.str.contains('Kirill Dontsov')]
prices = prices[~prices.name.str.contains('Kirill Kolesnichenko')]
prices = prices[~prices.name.str.contains('Kirill Kosarev')]
prices = prices[~prices.name.str.contains('Kirill Kravtsov')]
prices = prices[~prices.name.str.contains('Koba Koindredi')]
prices = prices[~prices.name.str.contains('Kwasi Wriedt')]
prices = prices[~prices.name.str.contains('Kyle Connell')]
prices = prices[~prices.name.str.contains('Kyle Hayde')]
prices = prices[~prices.name.str.contains('Kyle Morton')]
prices = prices[~prices.name.str.contains('Kyle Taylor')]
prices = prices[~prices.name.str.contains('Kyle Zobeck')]
prices = prices[~prices.name.str.contains('Laercio Solda')]
prices = prices[~prices.name.str.contains('Lameck Banda')]
prices = prices[~prices.name.str.contains('Lamine Sherif')]
prices = prices[~prices.name.str.contains('Landry Dimata')]
prices = prices[~prices.name.str.contains('Lars Coveliers')]
prices = prices[~prices.name.str.contains('Lars Lokotsch')]
prices = prices[~prices.name.str.contains('Laurenz Beckemeyer')]
prices = prices[~prices.name.str.contains('Lautaro Blanco')]
prices = prices[~prices.name.str.contains('Lautaro Chavez')]
prices = prices[~prices.name.str.contains('Lautaro de Leon')]
prices = prices[~prices.name.str.contains('Lautaro Escalante')]
prices = prices[~prices.name.str.contains('Lautaro Geminiani')]
prices = prices[~prices.name.str.contains('Lautaro Giaccone')]
prices = prices[~prices.name.str.contains('Lautaro Montoya')]
prices = prices[~prices.name.str.contains('Lautaro Morales')]
prices = prices[~prices.name.str.contains('Lautaro Parisi')]
prices = prices[~prices.name.str.contains('Lautaro Rinaldi')]
prices = prices[~prices.name.str.contains('Lazaros Lamprou')]
prices = prices[~prices.name.str.contains('Leandro Barcia')]
prices = prices[~prices.name.str.contains('Leandro Barreiro')]
prices = prices[~prices.name.str.contains('Leandro Barreiro Martins')]
prices = prices[~prices.name.str.contains('Leandro CarvalhoLeandro Carvalho')]
prices = prices[~prices.name.str.contains('Leandro Contin')]
prices = prices[~prices.name.str.contains('Leandro Fernandez')]
prices = prices[~prices.name.str.contains('Leandro Finochietto')]
prices = prices[~prices.name.str.contains('Leandro Grimi')]
prices = prices[~prices.name.str.contains('Leandro MatheusLeandro Matheus')]
prices = prices[~prices.name.str.contains('Leandro Putaro')]
prices = prices[~prices.name.str.contains('Leandro Silva')]
prices = prices[~prices.name.str.contains('Leandro Vella')]
prices = prices[~prices.name.str.contains('Lebo Mothiba')]
prices = prices[~prices.name.str.contains('Lee Kang-In')]
prices = prices[~prices.name.str.contains('Lenny Lacroix')]
prices = prices[~prices.name.str.contains('Lentini Caciano')]
prices = prices[~prices.name.str.contains('Leo Fuhr Hjelde')]
prices = prices[~prices.name.str.contains('Leo Gomes')]
prices = prices[~prices.name.str.contains('Leonardo Gil')]
prices = prices[~prices.name.str.contains('Leonardo Jara')]
prices = prices[~prices.name.str.contains('Leonardo Realpe')]
prices = prices[~prices.name.str.contains('Leonardo Sequeira')]
prices = prices[~prices.name.str.contains('Leonardo Villalba')]
prices = prices[~prices.name.str.contains('LeonardoLeonardo')]
prices = prices[~prices.name.str.contains('Leonel Ferroni')]
prices = prices[~prices.name.str.contains('Leonel Galeano')]
prices = prices[~prices.name.str.contains('Leonel Gonzalez')]
prices = prices[~prices.name.str.contains('Leroy Kwadwo')]
prices = prices[~prices.name.str.contains('Levi Malangu')]
prices = prices[~prices.name.str.contains('Levi Ntumba')]
prices = prices[~prices.name.str.contains('Lewis Bradley')]
prices = prices[~prices.name.str.contains('Lewis Gibson')]
prices = prices[~prices.name.str.contains('Lewis Hardcastle')]
prices = prices[~prices.name.str.contains('Lewis Johnson')]
prices = prices[~prices.name.str.contains('Lewis Richardson')]
prices = prices[~prices.name.str.contains('Lewis Simper')]
prices = prices[~prices.name.str.contains('Lewis Wilson')]
prices = prices[~prices.name.str.contains('Lewis Young')]
prices = prices[~prices.name.str.contains('Lex-Tyger Lobinger')]
prices = prices[~prices.name.str.contains('Liam Armstrong')]
prices = prices[~prices.name.str.contains('Liam Cross')]
prices = prices[~prices.name.str.contains('Liam Delap')]
prices = prices[~prices.name.str.contains('Liam Edwards')]
prices = prices[~prices.name.str.contains('Liam Gibbs')]
prices = prices[~prices.name.str.contains('Liam Lightfoot')]
prices = prices[~prices.name.str.contains('Liam Millar')]
prices = prices[~prices.name.str.contains('Liam Waldock')]
prices = prices[~prices.name.str.contains('Logan Ketterer')]
prices = prices[~prices.name.str.contains('Logan Ross')]
prices = prices[~prices.name.str.contains('Loic Ritiere')]
prices = prices[~prices.name.str.contains('Lois Diony')]
prices = prices[~prices.name.str.contains('Lorenzo Colombo')]
prices = prices[~prices.name.str.contains('Lorenzo Del Pinto')]
prices = prices[~prices.name.str.contains('Lorenzo Faravelli')]
prices = prices[~prices.name.str.contains('Lorenzo Melgarejo')]
prices = prices[~prices.name.str.contains('Lorhan')]
prices = prices[~prices.name.str.contains('Louie Watson')]
prices = prices[~prices.name.str.contains('Louis Boyd')]
prices = prices[~prices.name.str.contains('Louis Britton')]
prices = prices[~prices.name.str.contains('Louis Jordan Beyer')]
prices = prices[~prices.name.str.contains('Louis Morison')]
prices = prices[~prices.name.str.contains('Louis Moult')]
prices = prices[~prices.name.str.contains('Lovro Bizjak')]
prices = prices[~prices.name.str.contains('Luan Garcia')]
prices = prices[~prices.name.str.contains('Luan Santos')]
prices = prices[~prices.name.str.contains('Luan Silva')]
prices = prices[~prices.name.str.contains('Lubomir Tupta')]
prices = prices[~prices.name.str.contains('Luca Andrada')]
prices = prices[~prices.name.str.contains('Luca Antei')]
prices = prices[~prices.name.str.contains('Luca Connell')]
prices = prices[~prices.name.str.contains('Luca Itter')]
prices = prices[~prices.name.str.contains('Luca Lewis')]
prices = prices[~prices.name.str.contains('Luca Mora')]
prices = prices[~prices.name.str.contains('Luca Orellano')]
prices = prices[~prices.name.str.contains('Luca Pfeiffer')]
prices = prices[~prices.name.str.contains('Luca Philipp')]
prices = prices[~prices.name.str.contains('Luca Plogmann')]
prices = prices[~prices.name.str.contains('Luca Sangalli')]
prices = prices[~prices.name.str.contains('Luca Schuler')]
prices = prices[~prices.name.str.contains('Luca Siligardi')]
prices = prices[~prices.name.str.contains('Luca Unbehaun')]
prices = prices[~prices.name.str.contains('Luca Waldschmidt')]
prices = prices[~prices.name.str.contains('Luca Zander')]
prices = prices[~prices.name.str.contains('Luca-Milan Zander')]
prices = prices[~prices.name.str.contains('Lucao')]
prices = prices[~prices.name.str.contains('LucaoLucaoattac')]
prices = prices[~prices.name.str.contains('Lucas')]
prices = prices[~prices.name.str.contains('Lucas Beltran')]
prices = prices[~prices.name.str.contains('Lucas Besozzi')]
prices = prices[~prices.name.str.contains('Lucas Brochero')]
prices = prices[~prices.name.str.contains('Lucas Chevalier')]
prices = prices[~prices.name.str.contains('Lucas Di Yorio')]
prices = prices[~prices.name.str.contains('Lucas Dias')]
prices = prices[~prices.name.str.contains('Lucas EstevesLucas Esteves')]
prices = prices[~prices.name.str.contains('Lucas FonsecaLucas Fonseca')]
prices = prices[~prices.name.str.contains('Lucas GabrielLucas Gabriel')]
prices = prices[~prices.name.str.contains('Lucas HalterLucas Halter')]
prices = prices[~prices.name.str.contains('Lucas Kal')]
prices = prices[~prices.name.str.contains('Lucas KalLucas Kal')]
prices = prices[~prices.name.str.contains('Lucas Kruspzky')]
prices = prices[~prices.name.str.contains('Lucas Lima')]
prices = prices[~prices.name.str.contains('Lucas LourenÃ§oLucas LourenÃ§o')]
prices = prices[~prices.name.str.contains('Lucas Mancinelli')]
prices = prices[~prices.name.str.contains('Lucas Menossi')]
prices = prices[~prices.name.str.contains('Lucas Orellano')]
prices = prices[~prices.name.str.contains('Lucas Perri')]
prices = prices[~prices.name.str.contains('Lucas Pirard')]
prices = prices[~prices.name.str.contains('Lucas Rios')]
prices = prices[~prices.name.str.contains('Lucas Robertone')]
prices = prices[~prices.name.str.contains('Lucas Suarez')]
prices = prices[~prices.name.str.contains('Lucas Tagliapietra')]
prices = prices[~prices.name.str.contains('Lucas Tomlinson')]
prices = prices[~prices.name.str.contains('Lucas VenutoLucas Venuto')]
prices = prices[~prices.name.str.contains('Lucas Viatri')]
prices = prices[~prices.name.str.contains('Lucas Villarruel')]
prices = prices[~prices.name.str.contains('LuccaLucca')]
prices = prices[~prices.name.str.contains('Luciano Cingolani')]
prices = prices[~prices.name.str.contains('Luciano Ferreyra')]
prices = prices[~prices.name.str.contains('Luciano Gondou')]
prices = prices[~prices.name.str.contains('Luciano Pons')]
prices = prices[~prices.name.str.contains('Luciano Recalde')]
prices = prices[~prices.name.str.contains('Luciano Squadrone')]
prices = prices[~prices.name.str.contains('Luciano Vietto')]
prices = prices[~prices.name.str.contains('Ludovic Butelle')]
prices = prices[~prices.name.str.contains('Ludvig Ohman')]
prices = prices[~prices.name.str.contains('Luis Adlard')]
prices = prices[~prices.name.str.contains('Luis Binks')]
prices = prices[~prices.name.str.contains('Luis Caicedo')]
prices = prices[~prices.name.str.contains('Luis Coordes')]
prices = prices[~prices.name.str.contains('Luis Felipe')]
prices = prices[~prices.name.str.contains('Luis Garnier')]
prices = prices[~prices.name.str.contains('Luis Leal')]
prices = prices[~prices.name.str.contains('Luis Manuel Orejuela')]
prices = prices[~prices.name.str.contains('Luis Sequeira')]
prices = prices[~prices.name.str.contains('Luis Unsain')]
prices = prices[~prices.name.str.contains('Luis Vazquez')]
prices = prices[~prices.name.str.contains('Luiz Otavio')]
prices = prices[~prices.name.str.contains('Luiz PhellypeLuiz Phellype')]
prices = prices[~prices.name.str.contains('Lukas Lerager')]
prices = prices[~prices.name.str.contains('Lukas Zima')]
prices = prices[~prices.name.str.contains('Luke Bolton')]
prices = prices[~prices.name.str.contains('Luke Brennan')]
prices = prices[~prices.name.str.contains('Luke Haines')]
prices = prices[~prices.name.str.contains('Luke Hallett')]
prices = prices[~prices.name.str.contains('Luke Matheson')]
prices = prices[~prices.name.str.contains('Luke McNally')]
prices = prices[~prices.name.str.contains('Mace Goodridge')]
prices = prices[~prices.name.str.contains('Maciej Dabrowski')]
prices = prices[~prices.name.str.contains('Mal Benning')]
prices = prices[~prices.name.str.contains('Malamine Doumbouya')]
prices = prices[~prices.name.str.contains('Malcolm Barcola')]
prices = prices[~prices.name.str.contains('Malcom Braida')]
prices = prices[~prices.name.str.contains('Mamor Niang')]
prices = prices[~prices.name.str.contains('ManchaMancha')]
prices = prices[~prices.name.str.contains('Mandela Egbo')]
prices = prices[~prices.name.str.contains('Mandela Keita')]
prices = prices[~prices.name.str.contains('Manny Monthe')]
prices = prices[~prices.name.str.contains('Manny Oyeleke')]
prices = prices[~prices.name.str.contains('ManoelManoel')]
prices = prices[~prices.name.str.contains('Manuel Cabit')]
prices = prices[~prices.name.str.contains('Manuel Capasso')]
prices = prices[~prices.name.str.contains('Manuel da Costa')]
prices = prices[~prices.name.str.contains('Manuel Llano')]
prices = prices[~prices.name.str.contains('Manuel Namora')]
prices = prices[~prices.name.str.contains('Manuel Nieto')]
prices = prices[~prices.name.str.contains('Manuel Panaro')]
prices = prices[~prices.name.str.contains('Manuel Perez')]
prices = prices[~prices.name.str.contains('Manuel Vicentini')]
prices = prices[~prices.name.str.contains('Marc Auge')]
prices = prices[~prices.name.str.contains('Marc Baro')]
prices = prices[~prices.name.str.contains('Marc Cardona')]
prices = prices[~prices.name.str.contains('Marc Guehi')]
prices = prices[~prices.name.str.contains('Marc Pugh')]
prices = prices[~prices.name.str.contains('Marcel Johnen')]
prices = prices[~prices.name.str.contains('Marcel Ritzmaier')]
prices = prices[~prices.name.str.contains('Marcel Schmelzer')]
prices = prices[~prices.name.str.contains('Marcello Trotta')]
prices = prices[~prices.name.str.contains('Marcelo Boeck')]
prices = prices[~prices.name.str.contains('Marcelo Estigarribia')]
prices = prices[~prices.name.str.contains('Marcelo Freites')]
prices = prices[~prices.name.str.contains('Marcelo Goiano')]
prices = prices[~prices.name.str.contains('Marcelo Toscano')]
prices = prices[~prices.name.str.contains('Marciano Aziz')]
prices = prices[~prices.name.str.contains('Marcin Bulka')]
prices = prices[~prices.name.str.contains('Marcio Mossoro')]
prices = prices[~prices.name.str.contains('Marco Alia')]
prices = prices[~prices.name.str.contains('Marco Aratore')]
prices = prices[~prices.name.str.contains('Marco Benassi')]
prices = prices[~prices.name.str.contains('Marco Bertini')]
prices = prices[~prices.name.str.contains('Marco Borgnino')]
prices = prices[~prices.name.str.contains('Marco Delgado')]
prices = prices[~prices.name.str.contains('Marco Di Cesare')]
prices = prices[~prices.name.str.contains('Marco Djuricin')]
prices = prices[~prices.name.str.contains('Marco Faraoni')]
prices = prices[~prices.name.str.contains('Marco Ludivin Majouga')]
prices = prices[~prices.name.str.contains('Marco Majouga')]
prices = prices[~prices.name.str.contains('Marcos Rouzies')]
prices = prices[~prices.name.str.contains('Marcos Sanchez')]
prices = prices[~prices.name.str.contains('Marcus Ferkranus')]
prices = prices[~prices.name.str.contains('Marcus McGuane')]
prices = prices[~prices.name.str.contains('Marian Shved')]
prices = prices[~prices.name.str.contains('Mariano Bareiro')]
prices = prices[~prices.name.str.contains('Mariano Diaz')]
prices = prices[~prices.name.str.contains('Mariano Vazquez')]
prices = prices[~prices.name.str.contains('Mario Situm')]
prices = prices[~prices.name.str.contains('Mario Suver')]
prices = prices[~prices.name.str.contains('Marius Liesegang')]
prices = prices[~prices.name.str.contains('Mark Gillespie')]
prices = prices[~prices.name.str.contains('Mark Kosznovszky')]
prices = prices[~prices.name.str.contains('Mark McGuinness')]
prices = prices[~prices.name.str.contains('Marlon Fossey')]
prices = prices[~prices.name.str.contains('Marlon Hairston')]
prices = prices[~prices.name.str.contains('Marlon Mustapha')]
prices = prices[~prices.name.str.contains('Marlon Santos')]
prices = prices[~prices.name.str.contains('Marouan Azarkan')]
prices = prices[~prices.name.str.contains('Marquinhos Calazans')]
prices = prices[~prices.name.str.contains('Martin Samuelsen')]
prices = prices[~prices.name.str.contains('Martin Skrtel')]
prices = prices[~prices.name.str.contains('Martin Smith')]
prices = prices[~prices.name.str.contains('Marvin Dewaele')]
prices = prices[~prices.name.str.contains('Marvin Obuz')]
prices = prices[~prices.name.str.contains('Marvin Park')]
prices = prices[~prices.name.str.contains('Marvin Tshibuabua')]
prices = prices[~prices.name.str.contains('Maryan Shved')]
prices = prices[~prices.name.str.contains('Mason Stajduhar')]
prices = prices[~prices.name.str.contains('Massimo Giamattei')]
prices = prices[~prices.name.str.contains('Mat Ryan')]
prices = prices[~prices.name.str.contains('Matej Kovar')]
prices = prices[~prices.name.str.contains('Matej Oravec')]
prices = prices[~prices.name.str.contains('Mateo Coronel')]
prices = prices[~prices.name.str.contains('Mateo Mamani')]
prices = prices[~prices.name.str.contains('Mateo Tanlongo')]
prices = prices[~prices.name.str.contains('Matheo Vroman')]
prices = prices[~prices.name.str.contains('Matheus Clemente')]
prices = prices[~prices.name.str.contains('Matheus Donelli')]
prices = prices[~prices.name.str.contains('Matheus Fernandes')]
prices = prices[~prices.name.str.contains('Matheus Oliveira')]
prices = prices[~prices.name.str.contains('Matheuzinho')]
prices = prices[~prices.name.str.contains('Mathew Hudson')]
prices = prices[~prices.name.str.contains('Mathew Ryan')]
prices = prices[~prices.name.str.contains('Mathias Suarez')]
prices = prices[~prices.name.str.contains('Mathias Wittek')]
prices = prices[~prices.name.str.contains('Mathieu Dossevi')]
prices = prices[~prices.name.str.contains('Mathys Saban')]
prices = prices[~prices.name.str.contains('Matias Belloso')]
prices = prices[~prices.name.str.contains('Matias Borgogno')]
prices = prices[~prices.name.str.contains('Matias Cordoba')]
prices = prices[~prices.name.str.contains('Matias De los Santos')]
prices = prices[~prices.name.str.contains('Matias de los Santos')]
prices = prices[~prices.name.str.contains('Matias Escudero')]
prices = prices[~prices.name.str.contains('Matias Fritzler')]
prices = prices[~prices.name.str.contains('Matias Garcia')]
prices = prices[~prices.name.str.contains('Matias Gomez')]
prices = prices[~prices.name.str.contains('Matias Gonzalez')]
prices = prices[~prices.name.str.contains('Matias Laba')]
prices = prices[~prices.name.str.contains('Matias Molina')]
prices = prices[~prices.name.str.contains('Matias Moya')]
prices = prices[~prices.name.str.contains('Matias Palavecino')]
prices = prices[~prices.name.str.contains('Matias Pellegrini')]
prices = prices[~prices.name.str.contains('Matias Pisano')]
prices = prices[~prices.name.str.contains('Matias Ramirez')]
prices = prices[~prices.name.str.contains('Matias Rodriguez')]
prices = prices[~prices.name.str.contains('Matias Sosa')]
prices = prices[~prices.name.str.contains('Matias Tagliamonte')]
prices = prices[~prices.name.str.contains('Matias Tissera')]
prices = prices[~prices.name.str.contains('Matias Villarreal')]
prices = prices[~prices.name.str.contains('Matis Carvalho')]
prices = prices[~prices.name.str.contains('Matisse Didden')]
prices = prices[~prices.name.str.contains('Matko Miljevic')]
prices = prices[~prices.name.str.contains('Matt Bersano')]
prices = prices[~prices.name.str.contains('Matt Clarke')]
prices = prices[~prices.name.str.contains('Matt Dolan')]
prices = prices[~prices.name.str.contains('Matt Freese')]
prices = prices[~prices.name.str.contains('Matt Gilks')]
prices = prices[~prices.name.str.contains('Matt Hundley')]
prices = prices[~prices.name.str.contains('Matt Lampson')]
prices = prices[~prices.name.str.contains('Matt Palmer')]
prices = prices[~prices.name.str.contains('Matt Warburton')]
prices = prices[~prices.name.str.contains('Matt Young')]
prices = prices[~prices.name.str.contains('Matteo Campagna')]
prices = prices[~prices.name.str.contains('Matteo Scozzarella')]
prices = prices[~prices.name.str.contains('Matthew Alexander')]
prices = prices[~prices.name.str.contains('Matthew Cash')]
prices = prices[~prices.name.str.contains('Matthew Clarke')]
prices = prices[~prices.name.str.contains('Matthew Dolan')]
prices = prices[~prices.name.str.contains('Matthew Foulds')]
prices = prices[~prices.name.str.contains('Matthew Gilks')]
prices = prices[~prices.name.str.contains('Matthew Millar')]
prices = prices[~prices.name.str.contains('Matthew Palmer')]
prices = prices[~prices.name.str.contains('Matthew Platt')]
prices = prices[~prices.name.str.contains('Matthew Pollock')]
prices = prices[~prices.name.str.contains('Matthew Smith')]
prices = prices[~prices.name.str.contains('Matthew Virtue-Thick')]
prices = prices[~prices.name.str.contains('Matthew Warburton')]
prices = prices[~prices.name.str.contains('Matthias Heiland')]
prices = prices[~prices.name.str.contains('Matthias Verreth')]
prices = prices[~prices.name.str.contains('Matthieu Dossevi')]
prices = prices[~prices.name.str.contains('Matthieu Saunier')]
prices = prices[~prices.name.str.contains('Matthis Abline')]
prices = prices[~prices.name.str.contains('Mattia Pagliuca')]
prices = prices[~prices.name.str.contains('Mattias Johansson')]
prices = prices[~prices.name.str.contains('Mattie Pollock')]
prices = prices[~prices.name.str.contains('Matty Cash')]
prices = prices[~prices.name.str.contains('Matty Foulds')]
prices = prices[~prices.name.str.contains('Matty Platt')]
prices = prices[~prices.name.str.contains('Matty Virtue')]
prices = prices[~prices.name.str.contains('Matty Wolfe')]
prices = prices[~prices.name.str.contains('Matz Sels')]
prices = prices[~prices.name.str.contains('Matz SelsMatz Sels')]
prices = prices[~prices.name.str.contains('Mauricio')]
prices = prices[~prices.name.str.contains('Mauricio Arboleda')]
prices = prices[~prices.name.str.contains('Mauricio Caranta')]
prices = prices[~prices.name.str.contains('Mauricio Cuero')]
prices = prices[~prices.name.str.contains('Mauricio Duarte')]
prices = prices[~prices.name.str.contains('Mauricio Guzman')]
prices = prices[~prices.name.str.contains('Mauricio Kozlinski')]
prices = prices[~prices.name.str.contains('MauricioMauricio')]
prices = prices[~prices.name.str.contains('Maurizio Pochettino')]
prices = prices[~prices.name.str.contains('Mauro Barraza')]
prices = prices[~prices.name.str.contains('Mauro Da Luz')]
prices = prices[~prices.name.str.contains('Mauro dos Santos')]
prices = prices[~prices.name.str.contains('Mauro Luna Diale')]
prices = prices[~prices.name.str.contains('Mauro Ortiz')]
prices = prices[~prices.name.str.contains('Mauro Osores')]
prices = prices[~prices.name.str.contains('Mauro Valiente')]
prices = prices[~prices.name.str.contains('Max Harris')]
prices = prices[~prices.name.str.contains('Max Hunt')]
prices = prices[~prices.name.str.contains('Max Kilman')]
prices = prices[~prices.name.str.contains('Maxi Gomez')]
prices = prices[~prices.name.str.contains('Maxi Moralez')]
prices = prices[~prices.name.str.contains('Maxim De Cuyper')]
prices = prices[~prices.name.str.contains('Maxim Deman')]
prices = prices[~prices.name.str.contains('Maxime Delanghe')]
prices = prices[~prices.name.str.contains('Maxime Wenssens')]
prices = prices[~prices.name.str.contains('Maximilian Kilman')]
prices = prices[~prices.name.str.contains('Maximiliano Caire')]
prices = prices[~prices.name.str.contains('Maximiliano Cavallotti')]
prices = prices[~prices.name.str.contains('Maximiliano Cuadra')]
prices = prices[~prices.name.str.contains('Maximiliano Gagliardo')]
prices = prices[~prices.name.str.contains('Maximiliano Gomez')]
prices = prices[~prices.name.str.contains('Maximiliano Luayza')]
prices = prices[~prices.name.str.contains('Maximiliano Moralez')]
prices = prices[~prices.name.str.contains('Maximiliano Rogoski')]
prices = prices[~prices.name.str.contains('MaykeMayke')]
prices = prices[~prices.name.str.contains('Mees Bakker')]
prices = prices[~prices.name.str.contains('Mehdi Abeid')]
prices = prices[~prices.name.str.contains('Mehmet Ali Ulaman')]
prices = prices[~prices.name.str.contains('Michael Chacon')]
prices = prices[~prices.name.str.contains('Michael Dawson')]
prices = prices[~prices.name.str.contains('Michael DeShields')]
prices = prices[~prices.name.str.contains('Michael Edwards')]
prices = prices[~prices.name.str.contains('Michael Fabrie')]
prices = prices[~prices.name.str.contains('Michael Hefele')]
prices = prices[~prices.name.str.contains('Michael Johnston')]
prices = prices[~prices.name.str.contains('Michael Jones')]
prices = prices[~prices.name.str.contains('Michael Kelly')]
prices = prices[~prices.name.str.contains('Michael Klass')]
prices = prices[~prices.name.str.contains('Michael Krmencik')]
prices = prices[~prices.name.str.contains('Michael Lennon')]
prices = prices[~prices.name.str.contains('Michael Nelson')]
prices = prices[~prices.name.str.contains('Michael Ruth')]
prices = prices[~prices.name.str.contains('Michael Santos')]
prices = prices[~prices.name.str.contains('Michael Schultz')]
prices = prices[~prices.name.str.contains('Michal Karbownik')]
prices = prices[~prices.name.str.contains('Michal Sadilek')]
prices = prices[~prices.name.str.contains('Michel Araujo')]
prices = prices[~prices.name.str.contains('Michel Balikwisha')]
prices = prices[~prices.name.str.contains('Michel Herrero')]
prices = prices[~prices.name.str.contains('Michel Macedo')]
prices = prices[~prices.name.str.contains('Michel-Ange Balikwisha')]
prices = prices[~prices.name.str.contains('Miguel Atienza')]
prices = prices[~prices.name.str.contains('Miguel Cifuentes')]
prices = prices[~prices.name.str.contains('Miguel Jacquet')]
prices = prices[~prices.name.str.contains('Miguel Van Damme')]
prices = prices[~prices.name.str.contains('MiguelMiguel')]
prices = prices[~prices.name.str.contains('Mihai-Alexandru Dobre')]
prices = prices[~prices.name.str.contains('Mihail Levashov')]
prices = prices[~prices.name.str.contains('Mikael Soisalo')]
prices = prices[~prices.name.str.contains('Mikalay Signevich')]
prices = prices[~prices.name.str.contains('Mike Hauptmeijer')]
prices = prices[~prices.name.str.contains('Mike Jones')]
prices = prices[~prices.name.str.contains('Mike Tresor')]
prices = prices[~prices.name.str.contains('Mike Tresor Ndayishimiye')]
prices = prices[~prices.name.str.contains('Mikhail Ageev')]
prices = prices[~prices.name.str.contains('Mikhail Levashov')]
prices = prices[~prices.name.str.contains('Mikkel Diskerud')]
prices = prices[~prices.name.str.contains('Milan De Mey')]
prices = prices[~prices.name.str.contains('Milan Iloski')]
prices = prices[~prices.name.str.contains('Milan Tucic')]
prices = prices[~prices.name.str.contains('Miles Mitchell-Nelson')]
prices = prices[~prices.name.str.contains('Mirkan Aydin')]
prices = prices[~prices.name.str.contains('Mitch Pinnock')]
prices = prices[~prices.name.str.contains('Mitchell Clark')]
prices = prices[~prices.name.str.contains('Mitchell Curry')]
prices = prices[~prices.name.str.contains('Mitchell Pinnock')]
prices = prices[~prices.name.str.contains('Mitchell Roberts')]
prices = prices[~prices.name.str.contains('Mitchy Ntelo')]
prices = prices[~prices.name.str.contains('Mitsuki Saito')]
prices = prices[~prices.name.str.contains('Mix Diskerud')]
prices = prices[~prices.name.str.contains('Mo AdamsMo Adams')]
prices = prices[~prices.name.str.contains('Modou Diagne')]
prices = prices[~prices.name.str.contains('Mohamed Ali-Cho')]
prices = prices[~prices.name.str.contains('Mohamed Badamosi')]
prices = prices[~prices.name.str.contains('Mohamed Bouchouari')]
prices = prices[~prices.name.str.contains('Mohamed Maouche')]
prices = prices[~prices.name.str.contains('Mohamed Rayhi')]
prices = prices[~prices.name.str.contains('Mohamed Taabouni')]
prices = prices[~prices.name.str.contains('Mohammed Sangare')]
prices = prices[~prices.name.str.contains('Moise Sakava')]
prices = prices[~prices.name.str.contains('Moises Caicedo')]
prices = prices[~prices.name.str.contains('Morgan Rogers')]
prices = prices[~prices.name.str.contains('Moritz Leitner')]
prices = prices[~prices.name.str.contains('Moritz RÃ¶mling')]
prices = prices[~prices.name.str.contains('Moritz Schulze')]
prices = prices[~prices.name.str.contains('Moritz-Broni Kwarteng')]
prices = prices[~prices.name.str.contains('Mostapha El Kabir')]
prices = prices[~prices.name.str.contains('Mouez Hassen')]
prices = prices[~prices.name.str.contains('Mousa Al-Tamari')]
prices = prices[~prices.name.str.contains('Mousa Suleiman')]
prices = prices[~prices.name.str.contains('Moussa Wague')]
prices = prices[~prices.name.str.contains('Moustafa Ava Dongo')]
prices = prices[~prices.name.str.contains('Mucahit Albayrak')]
prices = prices[~prices.name.str.contains('Mucahit Can Akcay')]
prices = prices[~prices.name.str.contains('Muhamed Badamosi')]
prices = prices[~prices.name.str.contains('Muhamed Besic')]
prices = prices[~prices.name.str.contains('Muhammadu Faal')]
prices = prices[~prices.name.str.contains('Muhammed Badamosi')]
prices = prices[~prices.name.str.contains('Muhammed Egemen Pehlivan')]
prices = prices[~prices.name.str.contains('Muhammed Emin Sarikaya')]
prices = prices[~prices.name.str.contains('Muhammed Eren Kiryolcu')]
prices = prices[~prices.name.str.contains('Muhammed Kerem Akturkoglu')]
prices = prices[~prices.name.str.contains('Muhammed Sarikaya')]
prices = prices[~prices.name.str.contains('Muhammed Uzun')]
prices = prices[~prices.name.str.contains('Muhayer Oktay')]
prices = prices[~prices.name.str.contains('Mukhammad Sultanov')]
prices = prices[~prices.name.str.contains('Munir Mohamedi')]
prices = prices[~prices.name.str.contains('Nano Mesa')]
prices = prices[~prices.name.str.contains('Naoufal Bannis')]
prices = prices[~prices.name.str.contains('Nassim Ahmed')]
prices = prices[~prices.name.str.contains('Nassim Chadli')]
prices = prices[~prices.name.str.contains('Nathan Baxter')]
prices = prices[~prices.name.str.contains('Nathan Broadhead')]
prices = prices[~prices.name.str.contains('Nathan Cameron')]
prices = prices[~prices.name.str.contains('Nathan Harriel')]
prices = prices[~prices.name.str.contains('Nathan Markelo')]
prices = prices[~prices.name.str.contains('Navajo Bakboord')]
prices = prices[~prices.name.str.contains('Nayair Tiknizyan')]
prices = prices[~prices.name.str.contains('Nayel Mehssatou')]
prices = prices[~prices.name.str.contains('Nazareno Solis')]
prices = prices[~prices.name.str.contains('NeiltonNeilton')]
prices = prices[~prices.name.str.contains('Neri Bandiera')]
prices = prices[~prices.name.str.contains('Neri Cardozo')]
prices = prices[~prices.name.str.contains('Nestor Ortigoza')]
prices = prices[~prices.name.str.contains('Neven Subotic')]
prices = prices[~prices.name.str.contains('Neyder Lozano')]
prices = prices[~prices.name.str.contains('Nezar S')]
prices = prices[~prices.name.str.contains('Niall Huggins')]
prices = prices[~prices.name.str.contains('Nicholas DePuy')]
prices = prices[~prices.name.str.contains('Nicholas Ioannou')]
prices = prices[~prices.name.str.contains('Nicholas Lawrence Anwan')]
prices = prices[~prices.name.str.contains('Nicholas Slonina')]
prices = prices[~prices.name.str.contains('Nick DePuy')]
prices = prices[~prices.name.str.contains('Nick Shinton')]
prices = prices[~prices.name.str.contains('Nicky Hernandez')]
prices = prices[~prices.name.str.contains('Nicky Law')]
prices = prices[~prices.name.str.contains('Niclas Thiede')]
prices = prices[~prices.name.str.contains('Nicolas Ferreyra')]
prices = prices[~prices.name.str.contains('Nicolas Figal')]
prices = prices[~prices.name.str.contains('Nicolas Franco')]
prices = prices[~prices.name.str.contains('Nicolas Gaitan')]
prices = prices[~prices.name.str.contains('Nicolas Gimenez')]
prices = prices[~prices.name.str.contains('Nicolas Lamendola')]
prices = prices[~prices.name.str.contains('Nicolas Linares')]
prices = prices[~prices.name.str.contains('Nicolas Mana')]
prices = prices[~prices.name.str.contains('Nicolas Mazzola')]
prices = prices[~prices.name.str.contains('Nicolas Messiniti')]
prices = prices[~prices.name.str.contains('Nicolas Morgantini')]
prices = prices[~prices.name.str.contains('Nicolas Romat')]
prices = prices[~prices.name.str.contains('Nicolas Romero')]
prices = prices[~prices.name.str.contains('Nicolas Schiappacasse')]
prices = prices[~prices.name.str.contains('Nicolas Thaller')]
prices = prices[~prices.name.str.contains('Nicolas Tie')]
prices = prices[~prices.name.str.contains('Nikell Touglo')]
prices = prices[~prices.name.str.contains('Nikita Belousov')]
prices = prices[~prices.name.str.contains('Nikita Chagrov')]
prices = prices[~prices.name.str.contains('Nikita Chicherin')]
prices = prices[~prices.name.str.contains('Nikita Iosifov')]
prices = prices[~prices.name.str.contains('Nikita Kalugin')]
prices = prices[~prices.name.str.contains('Nikita Repin')]
prices = prices[~prices.name.str.contains('Niklas Klinger')]
prices = prices[~prices.name.str.contains('Niklo Dailly')]
prices = prices[~prices.name.str.contains('Nikola Katic')]
prices = prices[~prices.name.str.contains('Nikolai Kemlein')]
prices = prices[~prices.name.str.contains('Nikolai Rasskazov')]
prices = prices[~prices.name.str.contains('Nikolai Rehnen')]
prices = prices[~prices.name.str.contains('Nikolai Zolotov')]
prices = prices[~prices.name.str.contains('Nikolaos Kainourgios')]
prices = prices[~prices.name.str.contains('Nikolaos Karelis')]
prices = prices[~prices.name.str.contains('Nikolaos Zografakis')]
prices = prices[~prices.name.str.contains('Nikolay Giorgobiani')]
prices = prices[~prices.name.str.contains('Nikolay Rasskazov')]
prices = prices[~prices.name.str.contains('Nikolay Titkov')]
prices = prices[~prices.name.str.contains('Nikolay Zlobin')]
prices = prices[~prices.name.str.contains('Nikos Kainourgios')]
prices = prices[~prices.name.str.contains('Nils De Wilde')]
prices = prices[~prices.name.str.contains('Nino ParaibaNino Paraiba')]
prices = prices[~prices.name.str.contains('Nkosi Burgess')]
prices = prices[~prices.name.str.contains('Nnamdi Chinonso Offor')]
prices = prices[~prices.name.str.contains('Noah Awuku')]
prices = prices[~prices.name.str.contains('Noah Baumann')]
prices = prices[~prices.name.str.contains('Noah Ohio')]
prices = prices[~prices.name.str.contains('Noah Smerdon')]
prices = prices[~prices.name.str.contains('Nunoo Sarpei')]
prices = prices[~prices.name.str.contains('Nya Kirby')]
prices = prices[~prices.name.str.contains('Oladapo Afolayan')]
prices = prices[~prices.name.str.contains('Oleg Baklov')]
prices = prices[~prices.name.str.contains('Oleg Danchenko')]
prices = prices[~prices.name.str.contains('Oleg Dzantiev')]
prices = prices[~prices.name.str.contains('Oleg Nikolaev')]
prices = prices[~prices.name.str.contains('Oleg Reabciuk')]
prices = prices[~prices.name.str.contains('Oleksandr Kaplienko')]
prices = prices[~prices.name.str.contains('Oli Finney')]
prices = prices[~prices.name.str.contains('Oliver Banks')]
prices = prices[~prices.name.str.contains('Oliver Coker')]
prices = prices[~prices.name.str.contains('Oliver Finney')]
prices = prices[~prices.name.str.contains('Oliver Lancashire')]
prices = prices[~prices.name.str.contains('Oliver Myny')]
prices = prices[~prices.name.str.contains('Oliver Norburn')]
prices = prices[~prices.name.str.contains('Oliver Turton')]
prices = prices[~prices.name.str.contains('Olivier Myny')]
prices = prices[~prices.name.str.contains('Olivier Thill')]
prices = prices[~prices.name.str.contains('Ollie Banks')]
prices = prices[~prices.name.str.contains('Ollie Battersby')]
prices = prices[~prices.name.str.contains('Ollie Norburn')]
prices = prices[~prices.name.str.contains('Ollie Turton')]
prices = prices[~prices.name.str.contains('Ollie Younger')]
prices = prices[~prices.name.str.contains('Olly Lancashire')]
prices = prices[~prices.name.str.contains('Olly Lee')]
prices = prices[~prices.name.str.contains('Olufela Olomola')]
prices = prices[~prices.name.str.contains('Omar Khailoti')]
prices = prices[~prices.name.str.contains('Omer Beyaz')]
prices = prices[~prices.name.str.contains('Omer Faruk Beyaz')]
prices = prices[~prices.name.str.contains('Omer Hanin')]
prices = prices[~prices.name.str.contains('Omer Sismanoglu')]
prices = prices[~prices.name.str.contains('Onur Kaya')]
prices = prices[~prices.name.str.contains('Onur KayaOnur Kaya')]
prices = prices[~prices.name.str.contains('Opi Edwards')]
prices = prices[~prices.name.str.contains('Oregan Ravet')]
prices = prices[~prices.name.str.contains('Orestis Karnezis')]
prices = prices[~prices.name.str.contains('Oriol Rey')]
prices = prices[~prices.name.str.contains('Osama Rashid')]
prices = prices[~prices.name.str.contains('Oscar Benitez')]
prices = prices[~prices.name.str.contains('Oscar Linner')]
prices = prices[~prices.name.str.contains('Oscar Ruiz')]
prices = prices[~prices.name.str.contains('Oscar SchÃ¶nfelder')]
prices = prices[~prices.name.str.contains('Oskar Buur')]
prices = prices[~prices.name.str.contains('Ouasim Bouy')]
prices = prices[~prices.name.str.contains('Oumar Niasse')]
prices = prices[~prices.name.str.contains('Pablo Aranda')]
prices = prices[~prices.name.str.contains('Pablo Cortizo')]
prices = prices[~prices.name.str.contains('Pablo Cuadra')]
prices = prices[~prices.name.str.contains('Pablo Insua')]
prices = prices[~prices.name.str.contains('Pablo Rosales')]
prices = prices[~prices.name.str.contains('Pablo Sabbag')]
prices = prices[~prices.name.str.contains('Pablo Velazquez')]
prices = prices[~prices.name.str.contains('Paddy Madden')]
prices = prices[~prices.name.str.contains('Paddy McNair')]
prices = prices[~prices.name.str.contains('Panagiotis Armenakas')]
prices = prices[~prices.name.str.contains('Panos Armenakas')]
prices = prices[~prices.name.str.contains('Pantelis Chatzidiakos')]
prices = prices[~prices.name.str.contains('Pantelis Hatzidiakos')]
prices = prices[~prices.name.str.contains('Paolo Bartolomei')]
prices = prices[~prices.name.str.contains('Paolo Guerrero')]
prices = prices[~prices.name.str.contains('Paolo Hurtado')]
prices = prices[~prices.name.str.contains('Parker Siegfried')]
prices = prices[~prices.name.str.contains('Pascal Steinwender')]
prices = prices[~prices.name.str.contains('Patric Klandt')]
prices = prices[~prices.name.str.contains('Patricio Cucchi')]
prices = prices[~prices.name.str.contains('Patricio Matricardi')]
prices = prices[~prices.name.str.contains('Patricio Ostachuk')]
prices = prices[~prices.name.str.contains('Patricio Toranzo')]
prices = prices[~prices.name.str.contains('Patrick Jones')]
prices = prices[~prices.name.str.contains('Patrick McNair')]
prices = prices[~prices.name.str.contains('Paul Grave')]
prices = prices[~prices.name.str.contains('Paul-Philipp Besong')]
prices = prices[~prices.name.str.contains('Pavel Kaloshin')]
prices = prices[~prices.name.str.contains('Pavel Karasev')]
prices = prices[~prices.name.str.contains('Pedro Lucas')]
prices = prices[~prices.name.str.contains('Pedro Ojeda')]
prices = prices[~prices.name.str.contains('Pedro Perotti')]
prices = prices[~prices.name.str.contains('Pepe Bonet')]
prices = prices[~prices.name.str.contains('Per Kristian Bratveit')]
prices = prices[~prices.name.str.contains('Peru Nolaskoain')]
prices = prices[~prices.name.str.contains('Peter Urminsky')]
prices = prices[~prices.name.str.contains('Petrus Boumal')]
prices = prices[~prices.name.str.contains('Phil Jones')]
prices = prices[~prices.name.str.contains('Philip Gameli Awuku')]
prices = prices[~prices.name.str.contains('Philip Mayaka')]
prices = prices[~prices.name.str.contains('Philip Turpitz')]
prices = prices[~prices.name.str.contains('Philip Yeboah Ankrah')]
prices = prices[~prices.name.str.contains('Philipp Heerwagen')]
prices = prices[~prices.name.str.contains('Philipp Mwene')]
prices = prices[~prices.name.str.contains('Philipp Tschauner')]
prices = prices[~prices.name.str.contains('Philippe Sandler')]
prices = prices[~prices.name.str.contains('Phillipp Mwene')]
prices = prices[~prices.name.str.contains('Pier Graziano Gori')]
prices = prices[~prices.name.str.contains('Piero Hincapie')]
prices = prices[~prices.name.str.contains('Pierre-Yves Polomat')]
prices = prices[~prices.name.str.contains('PJ Morrison')]
prices = prices[~prices.name.str.contains('Pol Garcia')]
prices = prices[~prices.name.str.contains('Racine Coly')]
prices = prices[~prices.name.str.contains('Rafa De Vita')]
prices = prices[~prices.name.str.contains('Rafael Borre')]
prices = prices[~prices.name.str.contains('Rafael Cabral')]
prices = prices[~prices.name.str.contains('Rafael Ferrario')]
prices = prices[~prices.name.str.contains('Rafael Ribeiro')]
prices = prices[~prices.name.str.contains('Rafael Sangiovani')]
prices = prices[~prices.name.str.contains('Ramiro')]
prices = prices[~prices.name.str.contains('Ramiro Enrique')]
prices = prices[~prices.name.str.contains('Ramiro Luna')]
prices = prices[~prices.name.str.contains('Ramiro Macagno')]
prices = prices[~prices.name.str.contains('Ramon Arias')]
prices = prices[~prices.name.str.contains('Ramon Azeez')]
prices = prices[~prices.name.str.contains('Ramon Folch')]
prices = prices[~prices.name.str.contains('Ramon ten Hove')]
prices = prices[~prices.name.str.contains('Raul Aguilera')]
prices = prices[~prices.name.str.contains('Raul Carnero')]
prices = prices[~prices.name.str.contains('Raul Lozano')]
prices = prices[~prices.name.str.contains('Raul Moro')]
prices = prices[~prices.name.str.contains('Raul Navas')]
prices = prices[~prices.name.str.contains('Raulzinho')]
prices = prices[~prices.name.str.contains('Rayan Philippe')]
prices = prices[~prices.name.str.contains('Rayan Raveloson')]
prices = prices[~prices.name.str.contains('Rayan Souici')]
prices = prices[~prices.name.str.contains('Rayhaan Tulloch')]
prices = prices[~prices.name.str.contains('Raynner Silva')]
prices = prices[~prices.name.str.contains('Reagan Ogle')]
prices = prices[~prices.name.str.contains('Recep Niyaz')]
prices = prices[~prices.name.str.contains('Reda Akbib')]
prices = prices[~prices.name.str.contains('Reda Khadra')]
prices = prices[~prices.name.str.contains('Reece Grego-Cox')]
prices = prices[~prices.name.str.contains('Reece McAlear')]
prices = prices[~prices.name.str.contains('Regan Slater')]
prices = prices[~prices.name.str.contains('Renat Dadashov')]
prices = prices[~prices.name.str.contains('Rene Gilmartin')]
prices = prices[~prices.name.str.contains('Renzo Bacchia')]
prices = prices[~prices.name.str.contains('Renzo Giampaoli')]
prices = prices[~prices.name.str.contains('Renzo Saravia')]
prices = prices[~prices.name.str.contains('Renzo Tesuri')]
prices = prices[~prices.name.str.contains('Reuben Reid')]
prices = prices[~prices.name.str.contains('Richard Prieto')]
prices = prices[~prices.name.str.contains('Richard Rodrigues')]
prices = prices[~prices.name.str.contains('Richard Towell')]
prices = prices[~prices.name.str.contains('Rida Zouhir')]
prices = prices[~prices.name.str.contains('Rik Vercauteren')]
prices = prices[~prices.name.str.contains('Rio Hope-Gund')]
prices = prices[~prices.name.str.contains('Robert Apter')]
prices = prices[~prices.name.str.contains('Robert Arboleda')]
prices = prices[~prices.name.str.contains('Robert Castellanos')]
prices = prices[~prices.name.str.contains('Robert Dickie')]
prices = prices[~prices.name.str.contains('Robert Hall')]
prices = prices[~prices.name.str.contains('Robert Piris da Motta')]
prices = prices[~prices.name.str.contains('Robert Voloder')]
prices = prices[~prices.name.str.contains('Robert Zwinkels')]
prices = prices[~prices.name.str.contains('Roberto Bochi')]
prices = prices[~prices.name.str.contains('Roberto Correa')]
prices = prices[~prices.name.str.contains('Roberto Jimenez')]
prices = prices[~prices.name.str.contains('Roberto Navarro')]
prices = prices[~prices.name.str.contains('Robin Jalving')]
prices = prices[~prices.name.str.contains('Rodolfo Cardoso')]
prices = prices[~prices.name.str.contains('RodolfoRodolfo')]
prices = prices[~prices.name.str.contains('Rodri Sanchez')]
prices = prices[~prices.name.str.contains('Rodrigo DouradoRodrigo Dourado')]
prices = prices[~prices.name.str.contains('Rogerio Santos')]
prices = prices[~prices.name.str.contains('Romain Amalfitano')]
prices = prices[~prices.name.str.contains('Roman Martinez')]
prices = prices[~prices.name.str.contains('Romano Postema')]
prices = prices[~prices.name.str.contains('Ross Doohan')]
prices = prices[~prices.name.str.contains('Ross Graham')]
prices = prices[~prices.name.str.contains('Ross MacIver')]
prices = prices[~prices.name.str.contains('Ross Munro')]
prices = prices[~prices.name.str.contains('Ross Sinclair')]
prices = prices[~prices.name.str.contains('Ruben Botta')]
prices = prices[~prices.name.str.contains('Ruben Del Campo')]
prices = prices[~prices.name.str.contains('Ruben Fonseca')]
prices = prices[~prices.name.str.contains('Ruben Pardo')]
prices = prices[~prices.name.str.contains('Rubin Wilson')]
prices = prices[~prices.name.str.contains('Rumarn Burrell')]
prices = prices[~prices.name.str.contains('Ruwen Werthmuller')]
prices = prices[~prices.name.str.contains('Ryan Campbell-Gordon')]
prices = prices[~prices.name.str.contains('Ryan Cassidy')]
prices = prices[~prices.name.str.contains('Ryan Cooney')]
prices = prices[~prices.name.str.contains('Ryan Duncan')]
prices = prices[~prices.name.str.contains('Ryan Edmondson')]
prices = prices[~prices.name.str.contains('Ryan Giles')]
prices = prices[~prices.name.str.contains('Ryan Hillier')]
prices = prices[~prices.name.str.contains('Ryan Jones')]
prices = prices[~prices.name.str.contains('Ryan Longman')]
prices = prices[~prices.name.str.contains('Ryan Meara')]
prices = prices[~prices.name.str.contains('Ryan Rydel')]
prices = prices[~prices.name.str.contains('Ryan Scully')]
prices = prices[~prices.name.str.contains('Ryan Stirk')]
prices = prices[~prices.name.str.contains('Saba Sazonov')]
prices = prices[~prices.name.str.contains('Sabit Abdulai')]
prices = prices[~prices.name.str.contains('Sacha Bastien')]
prices = prices[~prices.name.str.contains('Sacha Delaye')]
prices = prices[~prices.name.str.contains('Said Ahmed Said')]
prices = prices[~prices.name.str.contains('Said Akhmaev')]
prices = prices[~prices.name.str.contains('Said Bakari')]
prices = prices[~prices.name.str.contains('Said-Ali Akhmaev')]
prices = prices[~prices.name.str.contains('Saidou Sow')]
prices = prices[~prices.name.str.contains('Saif-Eddine Khaoui')]
prices = prices[~prices.name.str.contains('Saikou Janneh')]
prices = prices[~prices.name.str.contains('Salim Ben Seghir')]
prices = prices[~prices.name.str.contains('Saliou Sane')]
prices = prices[~prices.name.str.contains('Sam Bell')]
prices = prices[~prices.name.str.contains('Sam Cartwright')]
prices = prices[~prices.name.str.contains('Sam Folarin')]
prices = prices[~prices.name.str.contains('Sam Junqua')]
prices = prices[~prices.name.str.contains('Sam McQueen')]
prices = prices[~prices.name.str.contains('Sam Morsy')]
prices = prices[~prices.name.str.contains('Sam Pearson')]
prices = prices[~prices.name.str.contains('Sam Sargeant')]
prices = prices[~prices.name.str.contains('Samed Onur')]
prices = prices[~prices.name.str.contains('Sami Gokhan Altiparmak')]
prices = prices[~prices.name.str.contains('Samy Bourard')]
prices = prices[~prices.name.str.contains('Samy Kehli')]
prices = prices[~prices.name.str.contains('Samy Mmaee')]
prices = prices[~prices.name.str.contains('Samy Morsy')]
prices = prices[~prices.name.str.contains('Santiago Caseres')]
prices = prices[~prices.name.str.contains('Santiago Colombatto')]
prices = prices[~prices.name.str.contains('Santiago Gallucci')]
prices = prices[~prices.name.str.contains('Santiago Garcia')]
prices = prices[~prices.name.str.contains('Santiago Rodriguez')]
prices = prices[~prices.name.str.contains('Santiago Silva')]
prices = prices[~prices.name.str.contains('Santiago Simon')]
prices = prices[~prices.name.str.contains('Santiago Trellez')]
prices = prices[~prices.name.str.contains('Scott Caldwell')]
prices = prices[~prices.name.str.contains('Scott Fox')]
prices = prices[~prices.name.str.contains('Scott Moloney')]
prices = prices[~prices.name.str.contains('Scott Pollock')]
prices = prices[~prices.name.str.contains('Scott Robertson')]
prices = prices[~prices.name.str.contains('Scott Smith')]
prices = prices[~prices.name.str.contains('Sebastian Anderson')]
prices = prices[~prices.name.str.contains('Sebastian Assis')]
prices = prices[~prices.name.str.contains('Sebastian Blanco')]
prices = prices[~prices.name.str.contains('Sebastian Breza')]
prices = prices[~prices.name.str.contains('Sebastian Dubarbier')]
prices = prices[~prices.name.str.contains('Sebastian Meza')]
prices = prices[~prices.name.str.contains('Sebastian Ramirez')]
prices = prices[~prices.name.str.contains('Sebastian Sosa')]
prices = prices[~prices.name.str.contains('Sebastian Soto')]
prices = prices[~prices.name.str.contains('Sebastien Thill')]
prices = prices[~prices.name.str.contains('Sega Coulibaly')]
prices = prices[~prices.name.str.contains('Sekou Diawara')]
prices = prices[~prices.name.str.contains('Sepe Elye Wahi')]
prices = prices[~prices.name.str.contains('Serdar Poyraz')]
prices = prices[~prices.name.str.contains('Serdar Saatci')]
prices = prices[~prices.name.str.contains('Sergei Borodin')]
prices = prices[~prices.name.str.contains('Sergei Makarov')]
prices = prices[~prices.name.str.contains('Sergei Ryzhikov')]
prices = prices[~prices.name.str.contains('Sergey Chibisov')]
prices = prices[~prices.name.str.contains('Sergey Makarov')]
prices = prices[~prices.name.str.contains('Sergi Garcia')]
prices = prices[~prices.name.str.contains('Sergio Barcia')]
prices = prices[~prices.name.str.contains('Sergio Carreira')]
prices = prices[~prices.name.str.contains('Sergio Cubero')]
prices = prices[~prices.name.str.contains('Sergio Gonzalez')]
prices = prices[~prices.name.str.contains('Sergio Ojeda')]
prices = prices[~prices.name.str.contains('Sergio Quiroga')]
prices = prices[~prices.name.str.contains('Sergio Romero')]
prices = prices[~prices.name.str.contains('Sergio Vittor')]
prices = prices[~prices.name.str.contains('Sergiy Rybalka')]
prices = prices[~prices.name.str.contains('Seung-Ho Paik')]
prices = prices[~prices.name.str.contains('Seung-woo Lee')]
prices = prices[~prices.name.str.contains('Shay Logan')]
prices = prices[~prices.name.str.contains('Shilow Tracey')]
prices = prices[~prices.name.str.contains('Shola Shoretire')]
prices = prices[~prices.name.str.contains('Shoya Nakajima')]
prices = prices[~prices.name.str.contains('Shurandy Sambo')]
prices = prices[~prices.name.str.contains('Sibiry Keita')]
prices = prices[~prices.name.str.contains('Siebren Lathouwers')]
prices = prices[~prices.name.str.contains('Silas Gnaka')]
prices = prices[~prices.name.str.contains('Silas Katompa Mvumpa')]
prices = prices[~prices.name.str.contains('Silas Wamangituka')]
prices = prices[~prices.name.str.contains('Silas Zehnder')]
prices = prices[~prices.name.str.contains('Silvinho Esajas')]
prices = prices[~prices.name.str.contains('Simon Deli')]
prices = prices[~prices.name.str.contains('Simon Moore')]
prices = prices[~prices.name.str.contains('Simon Ngapandouetnbu')]
prices = prices[~prices.name.str.contains('Simon Rhein')]
prices = prices[~prices.name.str.contains('Simone Aresti')]
prices = prices[~prices.name.str.contains('Simone Edera')]
prices = prices[~prices.name.str.contains('Simone Farelli')]
prices = prices[~prices.name.str.contains('Simone Rabbi')]
prices = prices[~prices.name.str.contains('Siro Rosane')]
prices = prices[~prices.name.str.contains('Sofian Chakla')]
prices = prices[~prices.name.str.contains('Sofiane Bouzian')]
prices = prices[~prices.name.str.contains('Soufiane Chakla')]
prices = prices[~prices.name.str.contains('Souleymane Aw')]
prices = prices[~prices.name.str.contains('Spencer Richey')]
prices = prices[~prices.name.str.contains('Stan van Dijck')]
prices = prices[~prices.name.str.contains('Stanislav Ivanov')]
prices = prices[~prices.name.str.contains('Stanislav Krapukhin')]
prices = prices[~prices.name.str.contains('Stanislav Shopov')]
prices = prices[~prices.name.str.contains('Steeve-Mike Eboa Ebongue')]
prices = prices[~prices.name.str.contains('Stefan Drljaca')]
prices = prices[~prices.name.str.contains('Stefan Milosevic')]
prices = prices[~prices.name.str.contains('Stefan van der Lei')]
prices = prices[~prices.name.str.contains('Stefano Callegari')]
prices = prices[~prices.name.str.contains('Stepan Oganesyan')]
prices = prices[~prices.name.str.contains('Stephane Paul Keller')]
prices = prices[~prices.name.str.contains('Stephen Duke-McKenna')]
prices = prices[~prices.name.str.contains('Stephen Henderson')]
prices = prices[~prices.name.str.contains('Stephen Kelly')]
prices = prices[~prices.name.str.contains('Stephen Mallan')]
prices = prices[~prices.name.str.contains('Steve Birnbaum')]
prices = prices[~prices.name.str.contains('Steven Beitashour')]
prices = prices[~prices.name.str.contains('Steven Defour')]
prices = prices[~prices.name.str.contains('Stiven Mendoza')]
prices = prices[~prices.name.str.contains('Stiven Plaza')]
prices = prices[~prices.name.str.contains('Suk-jae Lee')]
prices = prices[~prices.name.str.contains('Suleyman Cebeci')]
prices = prices[~prices.name.str.contains('Taylor Harwood-Bellis')]
prices = prices[~prices.name.str.contains('Taylor Richards')]
prices = prices[~prices.name.str.contains('Taylor Seymour')]
prices = prices[~prices.name.str.contains('Teddy Bartouche')]
prices = prices[~prices.name.str.contains('Teddy Boulhendi')]
prices = prices[~prices.name.str.contains('Teden Mengi')]
prices = prices[~prices.name.str.contains('Tennai Watson')]
prices = prices[~prices.name.str.contains('Thakgalo Leshabela')]
prices = prices[~prices.name.str.contains('TharlisTharlis')]
prices = prices[~prices.name.str.contains('Theo Corbeanu')]
prices = prices[~prices.name.str.contains('Theo Leoni')]
prices = prices[~prices.name.str.contains('Thiago Lopes')]
prices = prices[~prices.name.str.contains('Thiago Maia')]
prices = prices[~prices.name.str.contains('Thiago Santana')]
prices = prices[~prices.name.str.contains('Thibang Phete')]
prices = prices[~prices.name.str.contains('Thibault Tamas')]
prices = prices[~prices.name.str.contains('Thomas Allan')]
prices = prices[~prices.name.str.contains('Thomas Basila')]
prices = prices[~prices.name.str.contains('Thomas Callens')]
prices = prices[~prices.name.str.contains('Thomas Eastman')]
prices = prices[~prices.name.str.contains('Thomas Edwards')]
prices = prices[~prices.name.str.contains('Thomas Hamer')]
prices = prices[~prices.name.str.contains('Thomas Hasal')]
prices = prices[~prices.name.str.contains('Thomas McGill')]
prices = prices[~prices.name.str.contains('Thomas Mehew')]
prices = prices[~prices.name.str.contains('Thomas O')]
prices = prices[~prices.name.str.contains('Thomas Poll')]
prices = prices[~prices.name.str.contains('Thomas Roberts')]
prices = prices[~prices.name.str.contains('Thomas Stagg')]
prices = prices[~prices.name.str.contains('Thomas Toure')]
prices = prices[~prices.name.str.contains('Thomas Williams')]
prices = prices[~prices.name.str.contains('Tiago Banega')]
prices = prices[~prices.name.str.contains('Tiago Ilori')]
prices = prices[~prices.name.str.contains('Tiago Lopes')]
prices = prices[~prices.name.str.contains('Tiago Palacios')]
prices = prices[~prices.name.str.contains('Tibet Oniz')]
prices = prices[~prices.name.str.contains('Tiernan Parker')]
prices = prices[~prices.name.str.contains('Tim Coremans')]
prices = prices[~prices.name.str.contains('Tim Hall')]
prices = prices[~prices.name.str.contains('Tim Lemperle')]
prices = prices[~prices.name.str.contains('Tim Maciejewski')]
prices = prices[~prices.name.str.contains('Timothe Nkada')]
prices = prices[~prices.name.str.contains('Timothy Eyoma')]
prices = prices[~prices.name.str.contains('Titas Krapikas')]
prices = prices[~prices.name.str.contains('Tivonge Rushesha')]
prices = prices[~prices.name.str.contains('Tobe Leysen')]
prices = prices[~prices.name.str.contains('Tobias Raschl')]
prices = prices[~prices.name.str.contains('Tobias Zarate')]
prices = prices[~prices.name.str.contains('Toby Bancroft')]
prices = prices[~prices.name.str.contains('Tolaji Bola')]
prices = prices[~prices.name.str.contains('Tolga Topaloglu')]
prices = prices[~prices.name.str.contains('Tom Bloxham')]
prices = prices[~prices.name.str.contains('Tom Chamberlain')]
prices = prices[~prices.name.str.contains('Tom Collins')]
prices = prices[~prices.name.str.contains('Tom Eastman')]
prices = prices[~prices.name.str.contains('Tom Edwards')]
prices = prices[~prices.name.str.contains('Tom Heaton')]
prices = prices[~prices.name.str.contains('Tom Hendriks')]
prices = prices[~prices.name.str.contains('Tom Mickel')]
prices = prices[~prices.name.str.contains('Tom ScottTom Scott')]
prices = prices[~prices.name.str.contains('Tom Van Hyfte')]
prices = prices[~prices.name.str.contains('Tom Weilandt')]
prices = prices[~prices.name.str.contains('Tom White')]
prices = prices[~prices.name.str.contains('Tom Wilson')]
prices = prices[~prices.name.str.contains('Tomas Andrade')]
prices = prices[~prices.name.str.contains('Tomas Cardona')]
prices = prices[~prices.name.str.contains('Tomas Cavanagh')]
prices = prices[~prices.name.str.contains('Tomas Durso')]
prices = prices[~prices.name.str.contains('Tomas Escalante')]
prices = prices[~prices.name.str.contains('Tomas Fernandez')]
prices = prices[~prices.name.str.contains('Tomas Koubek')]
prices = prices[~prices.name.str.contains('Tomas Marchiori')]
prices = prices[~prices.name.str.contains('Tomas Martinez')]
prices = prices[~prices.name.str.contains('Tomas Ortiz')]
prices = prices[~prices.name.str.contains('Tomas Romero')]
prices = prices[~prices.name.str.contains('Tomas Rukas')]
prices = prices[~prices.name.str.contains('Tomas Sandoval')]
prices = prices[~prices.name.str.contains('Tomas Silva')]
prices = prices[~prices.name.str.contains('Tommy Conway')]
prices = prices[~prices.name.str.contains('Tommy Elphick')]
prices = prices[~prices.name.str.contains('Toni Datkovic')]
prices = prices[~prices.name.str.contains('Toni Sunjic')]
prices = prices[~prices.name.str.contains('Tony Alfaro')]
prices = prices[~prices.name.str.contains('Torben Musel')]
prices = prices[~prices.name.str.contains('Toto Nsiala')]
prices = prices[~prices.name.str.contains('Trae Coyle')]
prices = prices[~prices.name.str.contains('Tristan Dingome')]
prices = prices[~prices.name.str.contains('Trova Boni')]
prices = prices[~prices.name.str.contains('Troy Parrott')]
prices = prices[~prices.name.str.contains('Tunde Owolabi')]
prices = prices[~prices.name.str.contains('Tunji Akinola')]
prices = prices[~prices.name.str.contains('Tyler Burey')]
prices = prices[~prices.name.str.contains('Tyler Cordner')]
prices = prices[~prices.name.str.contains('Tyler Freeman')]
prices = prices[~prices.name.str.contains('Tyler French')]
prices = prices[~prices.name.str.contains('Tyler Magloire')]
prices = prices[~prices.name.str.contains('Tyler Smith')]
prices = prices[~prices.name.str.contains('Tyler Wolff')]
prices = prices[~prices.name.str.contains('Tyreik Wright')]
prices = prices[~prices.name.str.contains('Tyrese Fornah')]
prices = prices[~prices.name.str.contains('Tyrese Francois')]
prices = prices[~prices.name.str.contains('Tyrese Omotoye')]
prices = prices[~prices.name.str.contains('Tyrone Marsh')]
prices = prices[~prices.name.str.contains('Tyrone Tormin')]
prices = prices[~prices.name.str.contains('Tyreece Simpson')]
prices = prices[~prices.name.str.contains('Ui-jo Hwang')]
prices = prices[~prices.name.str.contains('Uillian Correia')]
prices = prices[~prices.name.str.contains('Ulises Segura')]
prices = prices[~prices.name.str.contains('Umut Bozok')]
prices = prices[~prices.name.str.contains('Unai Arietaleanizbeaskoa')]
prices = prices[~prices.name.str.contains('Unai Dufur')]
prices = prices[~prices.name.str.contains('Uri Rosell')]
prices = prices[~prices.name.str.contains('Uriel Ramirez')]
prices = prices[~prices.name.str.contains('Urko Gonzalez')]
prices = prices[~prices.name.str.contains('Valentin Larralde')]
prices = prices[~prices.name.str.contains('Valentin Mihaila')]
prices = prices[~prices.name.str.contains('Valeriu Ciuperca')]
prices = prices[~prices.name.str.contains('Varazdat Haroyan')]
prices = prices[~prices.name.str.contains('Vedad Ibisevic')]
prices = prices[~prices.name.str.contains('Veron Parkes')]
prices = prices[~prices.name.str.contains('Vic Chambaere')]
prices = prices[~prices.name.str.contains('Vicente Esquerdo')]
prices = prices[~prices.name.str.contains('Victor Bobsin')]
prices = prices[~prices.name.str.contains('Victor Camarasa')]
prices = prices[~prices.name.str.contains('Victor Chust')]
prices = prices[~prices.name.str.contains('Victor Jensen')]
prices = prices[~prices.name.str.contains('Victor Mollejo')]
prices = prices[~prices.name.str.contains('Victor Ramis')]
prices = prices[~prices.name.str.contains('Vincent Muller')]
prices = prices[~prices.name.str.contains('Vincent Schippers')]
prices = prices[~prices.name.str.contains('Vincenzo Millico')]
prices = prices[~prices.name.str.contains('Vinicius Zanocelo')]
prices = prices[~prices.name.str.contains('Virgil Gomis')]
prices = prices[~prices.name.str.contains('Virgiliu Postolachi')]
prices = prices[~prices.name.str.contains('Vitor Bueno')]
prices = prices[~prices.name.str.contains('Vittorio Parigini')]
prices = prices[~prices.name.str.contains('VladimirVladimir')]
prices = prices[~prices.name.str.contains('Vladislav Cherny')]
prices = prices[~prices.name.str.contains('Vladislav Torop')]
prices = prices[~prices.name.str.contains('Vlasdilav Yakovlev')]
prices = prices[~prices.name.str.contains('Vukasin Jovanovic')]
prices = prices[~prices.name.str.contains('Vyacheslav Litvinov')]
prices = prices[~prices.name.str.contains('Walter Kannemann')]
prices = prices[~prices.name.str.contains('Walter Montoya')]
prices = prices[~prices.name.str.contains('Welinton Junior')]
prices = prices[~prices.name.str.contains('Wenderson Galeno')]
prices = prices[~prices.name.str.contains('Wes Foderingham')]
prices = prices[~prices.name.str.contains('Wes McDonald')]
prices = prices[~prices.name.str.contains('Wesley McDonald')]
prices = prices[~prices.name.str.contains('Wesley Ribeiro')]
prices = prices[~prices.name.str.contains('Wilfredo Rivera')]
prices = prices[~prices.name.str.contains('Will Bapaga')]
prices = prices[~prices.name.str.contains('Will Collar')]
prices = prices[~prices.name.str.contains('Will Dennis')]
prices = prices[~prices.name.str.contains('Will Grigg')]
prices = prices[~prices.name.str.contains('Will Huffer')]
prices = prices[~prices.name.str.contains('Will Pulisic')]
prices = prices[~prices.name.str.contains('Will Smallbone')]
prices = prices[~prices.name.str.contains('Will Sutton')]
prices = prices[~prices.name.str.contains('Will Swan')]
prices = prices[~prices.name.str.contains('William Evans')]
prices = prices[~prices.name.str.contains('William Fish')]
prices = prices[~prices.name.str.contains('William Forrester')]
prices = prices[~prices.name.str.contains('William Grigg')]
prices = prices[~prices.name.str.contains('William Hondermarck')]
prices = prices[~prices.name.str.contains('Wouter Biebauw')]
prices = prices[~prices.name.str.contains('Wouter George')]
prices = prices[~prices.name.str.contains('Yanis Ammour')]
prices = prices[~prices.name.str.contains('Yanis Lhery')]
prices = prices[~prices.name.str.contains('Yanis Saidani')]
prices = prices[~prices.name.str.contains('Yann Bodiger')]
prices = prices[~prices.name.str.contains('Yann M')]
prices = prices[~prices.name.str.contains('Yannick Brugger')]
prices = prices[~prices.name.str.contains('YanYan')]
prices = prices[~prices.name.str.contains('Yasin Davus')]
prices = prices[~prices.name.str.contains('Yasin Oztekin')]
prices = prices[~prices.name.str.contains('Yassin Fekir')]
prices = prices[~prices.name.str.contains('Yassin Oukili')]
prices = prices[~prices.name.str.contains('Yassine Bounou')]
prices = prices[~prices.name.str.contains('Yavuz Bugra Boyar')]
prices = prices[~prices.name.str.contains('Yavuz Ulas Genc')]
prices = prices[~prices.name.str.contains('Yaya Toure')]
prices = prices[~prices.name.str.contains('Yayah Kallon')]
prices = prices[~prices.name.str.contains('Yeiler Goez')]
prices = prices[~prices.name.str.contains('Yeison Gordillo')]
prices = prices[~prices.name.str.contains('Yildirim Mert Ã‡etin')]
prices = prices[~prices.name.str.contains('Yoan Cardinale')]
prices = prices[~prices.name.str.contains('Yoann Etienne')]
prices = prices[~prices.name.str.contains('Yohan Croizet')]
prices = prices[~prices.name.str.contains('Yonathan Cabral')]
prices = prices[~prices.name.str.contains('Younes Belhanda')]
prices = prices[~prices.name.str.contains('Youri Schoonderwaldt')]
prices = prices[~prices.name.str.contains('Zvonimir Sarlija')]
prices = prices[~prices.name.str.contains('Zlatko Tripic')]
prices = prices[~prices.name.str.contains('Ziyad Al-Oyouni')]
prices = prices[~prices.name.str.contains('Zito Luvumbo')]
prices = prices[~prices.name.str.contains('Zeno Rossi')]
prices = prices[~prices.name.str.contains('Zeki Yildirim')]
prices = prices[~prices.name.str.contains('Zeki Fryers')]
prices = prices[~prices.name.str.contains('Zeki Celik')]
prices = prices[~prices.name.str.contains('Zehn Mohammed')]
prices = prices[~prices.name.str.contains('Zech Medley')]
prices = prices[~prices.name.str.contains('Ze Oliveira')]
prices = prices[~prices.name.str.contains('Ze IvaldoZe Ivaldo')]
prices = prices[~prices.name.str.contains('Ze GabrielZe Gabriel')]
prices = prices[~prices.name.str.contains('Zdenek Zlamal')]
prices = prices[~prices.name.str.contains('Zanka')]
prices = prices[~prices.name.str.contains('Zakaria Diallo')]
prices = prices[~prices.name.str.contains('Zak Dearnley')]
prices = prices[~prices.name.str.contains('Zaid Romero')]
prices = prices[~prices.name.str.contains('Zaid Al Hussaini')]
prices = prices[~prices.name.str.contains('Zachary Dearnley')]
prices = prices[~prices.name.str.contains('Zach Jeacock')]
prices = prices[~prices.name.str.contains('Zac Williams')]
prices = prices[~prices.name.str.contains('Zac SmithZac Smith')]
prices = prices[~prices.name.str.contains('Dimitri Chistyakov')]
prices = prices[~prices.name.str.contains('Dimitri German')]
prices = prices[~prices.name.str.contains('Dimitri Khomich')]
prices = prices[~prices.name.str.contains('Dimitri Nazarov')]
prices = prices[~prices.name.str.contains('Dimitri Poloz')]
prices = prices[~prices.name.str.contains('Dimitri Rybchinskiy')]
prices = prices[~prices.name.str.contains('Dimitri Shomko')]
prices = prices[~prices.name.str.contains('Dimitri Skopintsev')]
prices = prices[~prices.name.str.contains('Dimitri Stotskiy')]
prices = prices[~prices.name.str.contains('Dimitri Sysuev')]
prices = prices[~prices.name.str.contains('Dimitri Tikhiy')]
prices = prices[~prices.name.str.contains('Dimitri Zhivoglyadov')]
prices = prices[~prices.name.str.contains('Dino Visser')]
prices = prices[~prices.name.str.contains('Diogo BarbosaDiogo Barbosa')]
prices = prices[~prices.name.str.contains('Dixon Renteria')]
prices = prices[~prices.name.str.contains('Djibril Diani')]
prices = prices[~prices.name.str.contains('Dmitri Efremov')]
prices = prices[~prices.name.str.contains('Dmitri German')]
prices = prices[~prices.name.str.contains('Dmitri Markitesov')]
prices = prices[~prices.name.str.contains('Dmitri Rybchinsky')]
prices = prices[~prices.name.str.contains('Dmitri Skopintsev')]
prices = prices[~prices.name.str.contains('Dmitri Sysuev')]
prices = prices[~prices.name.str.contains('Dmitri Tikhiy')]
prices = prices[~prices.name.str.contains('Dmitri Vershkov')]
prices = prices[~prices.name.str.contains('Dmitrii Merenchukov')]
prices = prices[~prices.name.str.contains('Dmitriy Chistyakov')]
prices = prices[~prices.name.str.contains('Dmitriy Poloz')]
prices = prices[~prices.name.str.contains('Dmitriy Shomko')]
prices = prices[~prices.name.str.contains('Dmitriy Stotskiy')]
prices = prices[~prices.name.str.contains('Dmitriy Tarasov')]
prices = prices[~prices.name.str.contains('Dmitry Zhivoglyadov')]
prices = prices[~prices.name.str.contains('DodoDodo')]
prices = prices[~prices.name.str.contains('Dominik Reimann')]
prices = prices[~prices.name.str.contains('Dominik Szoboszlai')]
prices = prices[~prices.name.str.contains('Dragan Lausberg')]
prices = prices[~prices.name.str.contains('Drake Callender')]
prices = prices[~prices.name.str.contains('Drissa Camara')]
prices = prices[~prices.name.str.contains('EÃ±aut Mendia')]
prices = prices[~prices.name.str.contains('Edgar Pacheco')]
prices = prices[~prices.name.str.contains('Edi Semedo')]
prices = prices[~prices.name.str.contains('EdimarEdimar')]
prices = prices[~prices.name.str.contains('Edson MontaÃ±o')]
prices = prices[~prices.name.str.contains('EdsonEdson')]
prices = prices[~prices.name.str.contains('Eduardo dos Santos Haesler')]
prices = prices[~prices.name.str.contains('Edwar Lopez')]
prices = prices[~prices.name.str.contains('Elias Pereyra')]
prices = prices[~prices.name.str.contains('EliasElias')]
prices = prices[~prices.name.str.contains('Elton Kabangu')]
prices = prices[~prices.name.str.contains('Elye WahiElye Wahi')]
prices = prices[~prices.name.str.contains('Emanuel Insua')]
prices = prices[~prices.name.str.contains('EmanuelEmanuel')]
prices = prices[~prices.name.str.contains('Emil Riis Jakobsen')]
prices = prices[~prices.name.str.contains('Emil RiisEmil Riis')]
prices = prices[~prices.name.str.contains('Emiliano Gonzalez')]
prices = prices[~prices.name.str.contains('Emiliano Insua')]
prices = prices[~prices.name.str.contains('Emre Pehlivan')]
prices = prices[~prices.name.str.contains('Enzo YbaÃ±ez')]
prices = prices[~prices.name.str.contains('Eric Lopez')]
prices = prices[~prices.name.str.contains('ErickErick')]
prices = prices[~prices.name.str.contains('Erickson Gallardo')]
prices = prices[~prices.name.str.contains('Erik DueÃ±as')]
prices = prices[~prices.name.str.contains('Ertugrul Cetin')]
prices = prices[~prices.name.str.contains('Esteban Rolon')]
prices = prices[~prices.name.str.contains('Eunan O\'Kane')]
prices = prices[~prices.name.str.contains('Evan N\'Dicka')]
prices = prices[~prices.name.str.contains('Exequiel Zeballos')]
prices = prices[~prices.name.str.contains('Ezekiel Fryers')]
prices = prices[~prices.name.str.contains('Ezequiel Cerica')]
prices = prices[~prices.name.str.contains('Ezequiel Fernandez')]
prices = prices[~prices.name.str.contains('Fabian Cubero')]
prices = prices[~prices.name.str.contains('Fabian Noguera')]
prices = prices[~prices.name.str.contains('Fabio Abreu')]
prices = prices[~prices.name.str.contains('Fabio ChinaFabio China')]
prices = prices[~prices.name.str.contains('Fabio SantosFabio Santos')]
prices = prices[~prices.name.str.contains('Fabio Vazquez')]
prices = prices[~prices.name.str.contains('Fabricio Dominguez')]
prices = prices[~prices.name.str.contains('Facundo Fernandez')]
prices = prices[~prices.name.str.contains('Facundo Garces')]
prices = prices[~prices.name.str.contains('Facundo Sanchez')]
prices = prices[~prices.name.str.contains('Faiz Selemani')]
prices = prices[~prices.name.str.contains('Federico Martinez')]
prices = prices[~prices.name.str.contains('Fehmi KocFehmi Koc')]
prices = prices[~prices.name.str.contains('Fehmi Mert Gunok')]
prices = prices[~prices.name.str.contains('Fernando Roman')]
prices = prices[~prices.name.str.contains('Ferris N\'Goma')]
prices = prices[~prices.name.str.contains('Francesco D\'Alterio')]
prices = prices[~prices.name.str.contains('Francisco Ramirez')]
prices = prices[~prices.name.str.contains('Franco Gonzalez')]
prices = prices[~prices.name.str.contains('Franco Perez')]
prices = prices[~prices.name.str.contains('Gabriel AlvesGabriel Alves')]
prices = prices[~prices.name.str.contains('Gabriel Chapeco')]
prices = prices[~prices.name.str.contains('Gabriel Diaz')]
prices = prices[~prices.name.str.contains('Gabriel Falcao')]
prices = prices[~prices.name.str.contains('Gabriel Graciani')]
prices = prices[~prices.name.str.contains('Gabriel NogaGabriel Noga')]
prices = prices[~prices.name.str.contains('Gabriel PereiraGabriel Pereira')]
prices = prices[~prices.name.str.contains('Gaetan Poussin')]
prices = prices[~prices.name.str.contains('Gaston Benavidez')]
prices = prices[~prices.name.str.contains('Gaston Diaz')]
prices = prices[~prices.name.str.contains('Gaston Fernandez')]
prices = prices[~prices.name.str.contains('Gaston Gerzel')]
prices = prices[~prices.name.str.contains('Gaston Gomez')]
prices = prices[~prices.name.str.contains('Gaston Gonzalez')]
prices = prices[~prices.name.str.contains('Gaston Suso')]
prices = prices[~prices.name.str.contains('Gaston Torres')]
prices = prices[~prices.name.str.contains('George Nurse')]
prices = prices[~prices.name.str.contains('Gerald Sithole')]
prices = prices[~prices.name.str.contains('Giles Phillips')]
prices = prices[~prices.name.str.contains('Gonzalo Abrego')]
prices = prices[~prices.name.str.contains('Gonzalo Asis')]
prices = prices[~prices.name.str.contains('Gonzalo Rodriguez')]
prices = prices[~prices.name.str.contains('Gonzalo Veron')]
prices = prices[~prices.name.str.contains('Gorkem Can')]
prices = prices[~prices.name.str.contains('Guido Vadala')]
prices = prices[~prices.name.str.contains('Guillermo Fernandez')]
prices = prices[~prices.name.str.contains('Gustavo Ramalho')]
prices = prices[~prices.name.str.contains('Guven Yalcin')]
prices = prices[~prices.name.str.contains('Habib Keita')]
prices = prices[~prices.name.str.contains('Hamza Masoudi')]
prices = prices[~prices.name.str.contains('Hannibal Mejbri')]
prices = prices[~prices.name.str.contains('Hans Nicolussi Caviglia')]
prices = prices[~prices.name.str.contains('Harrison Paton')]
prices = prices[~prices.name.str.contains('Hatem Elhamed')]
prices = prices[~prices.name.str.contains('HeberHeber')]
prices = prices[~prices.name.str.contains('Hector Canteros')]
prices = prices[~prices.name.str.contains('HelioHelio')]
prices = prices[~prices.name.str.contains('HenriHenri')]
prices = prices[~prices.name.str.contains('Henry Burnett')]
prices = prices[~prices.name.str.contains('Hernan Burbano')]
prices = prices[~prices.name.str.contains('Hernan Lamberti')]
prices = prices[~prices.name.str.contains('Hernan Lopez')]
prices = prices[~prices.name.str.contains('Heung-min Son')]
prices = prices[~prices.name.str.contains('HiagoHiago')]
prices = prices[~prices.name.str.contains('Hugo Ekitike')]
prices = prices[~prices.name.str.contains('Hugo Silva')]
prices = prices[~prices.name.str.contains('Huseyin Biler')]
prices = prices[~prices.name.str.contains('Ibrahima Camara')]
prices = prices[~prices.name.str.contains('Ignacio Cechi')]
prices = prices[~prices.name.str.contains('Ignacio Gariglio')]
prices = prices[~prices.name.str.contains('Ignacio Lago')]
prices = prices[~prices.name.str.contains('Ignacio Maestro Puch')]
prices = prices[~prices.name.str.contains('Ignacio Miramon')]
prices = prices[~prices.name.str.contains('Igor CariusIgor Carius')]
prices = prices[~prices.name.str.contains('Igor JuliaoIgor Juliao')]
prices = prices[~prices.name.str.contains('Igor Julio')]
prices = prices[~prices.name.str.contains('Igor TorresIgor Torres')]
prices = prices[~prices.name.str.contains('Ike OparaIke Opara')]
prices = prices[~prices.name.str.contains('Il-lok Yun')]
prices = prices[~prices.name.str.contains('Ilya Shkurin')]
prices = prices[~prices.name.str.contains('Ilya Vorobjov')]
prices = prices[~prices.name.str.contains('Imanol Machuca')]
prices = prices[~prices.name.str.contains('In-beom Hwang')]
prices = prices[~prices.name.str.contains('Indiana Vassilev')]
prices = prices[~prices.name.str.contains('Innes Cameron')]
prices = prices[~prices.name.str.contains('Ionut Nedelcearu')]
prices = prices[~prices.name.str.contains('Irakoze Donasiyano')]
prices = prices[~prices.name.str.contains('Isaac Boehmer')]
prices = prices[~prices.name.str.contains('Isaac Hutchinson')]
prices = prices[~prices.name.str.contains('Isaac Karamoko')]
prices = prices[~prices.name.str.contains('IsaqueIsaque')]
prices = prices[~prices.name.str.contains('Islamzhan Nasyrov')]
prices = prices[~prices.name.str.contains('Isma Ruiz')]
prices = prices[~prices.name.str.contains('Ismael Saibari')]
prices = prices[~prices.name.str.contains('Ismael Sow')]
prices = prices[~prices.name.str.contains('Ismail Aissati')]
prices = prices[~prices.name.str.contains('Ismail Yuksek')]
prices = prices[~prices.name.str.contains('Israel Damonte')]
prices = prices[~prices.name.str.contains('Ivan Angulo')]
prices = prices[~prices.name.str.contains('Ivan Arboleda')]
prices = prices[~prices.name.str.contains('Ivan Budachev')]
prices = prices[~prices.name.str.contains('Ivan Kuzmichev')]
prices = prices[~prices.name.str.contains('IvoneiIvonei')]
prices = prices[~prices.name.str.contains('Iyenoma Udogie')]
prices = prices[~prices.name.str.contains('Jabez Makanda')]
prices = prices[~prices.name.str.contains('Jack Bolton')]
prices = prices[~prices.name.str.contains('Jack Burroughs')]
prices = prices[~prices.name.str.contains('Jack Chambers')]
prices = prices[~prices.name.str.contains('Jack Currie')]
prices = prices[~prices.name.str.contains('Jack Davies')]
prices = prices[~prices.name.str.contains('Jack de Vries')]
prices = prices[~prices.name.str.contains('Jack Hindle')]
prices = prices[~prices.name.str.contains('Jack Madelin')]
prices = prices[~prices.name.str.contains('Jack Newman')]
prices = prices[~prices.name.str.contains('Jack Rodwell')]
prices = prices[~prices.name.str.contains('Jack Roles')]
prices = prices[~prices.name.str.contains('Jack Ruddy')]
prices = prices[~prices.name.str.contains('Jack SimsJack Sims')]
prices = prices[~prices.name.str.contains('Jack Stretton')]
prices = prices[~prices.name.str.contains('Jack Vale')]
prices = prices[~prices.name.str.contains('Jack Young')]
prices = prices[~prices.name.str.contains('Jackson')]
prices = prices[~prices.name.str.contains('Jackson Conway')]
prices = prices[~prices.name.str.contains('Jacob Akanyirige')]
prices = prices[~prices.name.str.contains('Jacob Bancroft')]
prices = prices[~prices.name.str.contains('Jacob Barrett Laursen')]
prices = prices[~prices.name.str.contains('Jacob Gratton')]
prices = prices[~prices.name.str.contains('Jacob Greene')]
prices = prices[~prices.name.str.contains('Jacob Laursen')]
prices = prices[~prices.name.str.contains('Jacopo Dezi')]
prices = prices[~prices.name.str.contains('Jacopo Segre')]
prices = prices[~prices.name.str.contains('Jaden Philogene-Bidace')]
prices = prices[~prices.name.str.contains('JadsomJadsom')]
prices = prices[~prices.name.str.contains('Jae-ik Lee')]
prices = prices[~prices.name.str.contains('JailsonJailson')]
prices = prices[~prices.name.str.contains('Jaime Alvarado')]
prices = prices[~prices.name.str.contains('Jaimy Kroesen')]
prices = prices[~prices.name.str.contains('JairJair')]
prices = prices[~prices.name.str.contains('Jake Davidson')]
prices = prices[~prices.name.str.contains('Jake HullJake Hull')]
prices = prices[~prices.name.str.contains('Jake PeckJake Peck')]
prices = prices[~prices.name.str.contains('Jake Scrimshaw')]
prices = prices[~prices.name.str.contains('Jake Turner')]
prices = prices[~prices.name.str.contains('Jake Vokins')]
prices = prices[~prices.name.str.contains('Jakob Busk')]
prices = prices[~prices.name.str.contains('Jan Olschowsky')]
prices = prices[~prices.name.str.contains('Jan RasJan Ras')]
prices = prices[~prices.name.str.contains('JandersonJanderson')]
prices = prices[~prices.name.str.contains('JandreiJandrei')]
prices = prices[~prices.name.str.contains('Janeiler Rivas')]
prices = prices[~prices.name.str.contains('Jaquite')]
prices = prices[~prices.name.str.contains('Jason Meerstadt')]
prices = prices[~prices.name.str.contains('Jason Pendant')]
prices = prices[~prices.name.str.contains('Jason Steele')]
prices = prices[~prices.name.str.contains('Jay Henderson')]
prices = prices[~prices.name.str.contains('Jean Zimmer')]
prices = prices[~prices.name.str.contains('Jean-Aniel Assi')]
prices = prices[~prices.name.str.contains('Jean-Eudes Aholou')]
prices = prices[~prices.name.str.contains('Jeff Dewsnup')]
prices = prices[~prices.name.str.contains('Jeffrey de Lange')]
prices = prices[~prices.name.str.contains('Jeison Chala')]
prices = prices[~prices.name.str.contains('Jeison Lucumi')]
prices = prices[~prices.name.str.contains('Jeizon Ramirez')]
prices = prices[~prices.name.str.contains('JemersonJemerson')]
prices = prices[~prices.name.str.contains('JenisonJenison')]
prices = prices[~prices.name.str.contains('Jens Castrop')]
prices = prices[~prices.name.str.contains('Jens Craenmehr')]
prices = prices[~prices.name.str.contains('Jens Grahl')]
prices = prices[~prices.name.str.contains('Jeremy Perbet')]
prices = prices[~prices.name.str.contains('Jeronimo Pourtau')]
prices = prices[~prices.name.str.contains('Jesus Ferreira')]
prices = prices[~prices.name.str.contains('Jetro Willems')]
prices = prices[~prices.name.str.contains('Jevan Anderson')]
prices = prices[~prices.name.str.contains('Jhonatan Candia')]
prices = prices[~prices.name.str.contains('Jhonathan Agudelo')]
prices = prices[~prices.name.str.contains('JhonnathanJhonnathan')]
prices = prices[~prices.name.str.contains('Jimmy Adrian Kaparos')]
prices = prices[~prices.name.str.contains('Jimmy Kaparos')]
prices = prices[~prices.name.str.contains('Joachim Imbrechts')]
prices = prices[~prices.name.str.contains('Joan Mazzaco')]
prices = prices[~prices.name.str.contains('Joao Carlos Teixeira')]
prices = prices[~prices.name.str.contains('Joao CarlosJoao Carlos')]
prices = prices[~prices.name.str.contains('Joao Ferreira')]
prices = prices[~prices.name.str.contains('Joao GomesJoao Gomes')]
prices = prices[~prices.name.str.contains('Joao Jaquite')]
prices = prices[~prices.name.str.contains('Joao Klauss')]
prices = prices[~prices.name.str.contains('Joao LopesJoao Lopes')]
prices = prices[~prices.name.str.contains('Joao MarioJoao MarioRight Midfiel')]
prices = prices[~prices.name.str.contains('Joao Monteiro')]
prices = prices[~prices.name.str.contains('Joao Queiros')]
prices = prices[~prices.name.str.contains('Joao Rodriguez')]
prices = prices[~prices.name.str.contains('Joao Teixeira')]
prices = prices[~prices.name.str.contains('Joao VictorJoao Victor')]
prices = prices[~prices.name.str.contains('Joaquin Arzura')]
prices = prices[~prices.name.str.contains('Joaquin Blazquez')]
prices = prices[~prices.name.str.contains('Joaquin Hass')]
prices = prices[~prices.name.str.contains('Joaquin Mendive')]
prices = prices[~prices.name.str.contains('Joaquin Novillo')]
prices = prices[~prices.name.str.contains('Joaquin Varela')]
prices = prices[~prices.name.str.contains('JobsonJobson')]
prices = prices[~prices.name.str.contains('Joe Bendik')]
prices = prices[~prices.name.str.contains('Joe DodooJoe Dodoo')]
prices = prices[~prices.name.str.contains('Joe DunneJoe Dunne')]
prices = prices[~prices.name.str.contains('Joe HartJoe Hart')]
prices = prices[~prices.name.str.contains('Joe Maguire')]
prices = prices[~prices.name.str.contains('Joe Mason')]
prices = prices[~prices.name.str.contains('Joe NealJoe Neal')]
prices = prices[~prices.name.str.contains('Joe Rafferty')]
prices = prices[~prices.name.str.contains('Joe Scally')]
prices = prices[~prices.name.str.contains('Joe Willock')]
prices = prices[~prices.name.str.contains('Joe Woodiwiss')]
prices = prices[~prices.name.str.contains('Joel Tagueu')]
prices = prices[~prices.name.str.contains('Joey DeZart')]
prices = prices[~prices.name.str.contains('Johannes Eggestein')]
prices = prices[~prices.name.str.contains('John Anthony Brooks')]
prices = prices[~prices.name.str.contains('John Joe Patrick Finn')]
prices = prices[~prices.name.str.contains('John KennedyJohn Kennedy')]
prices = prices[~prices.name.str.contains('JohnJohn')]
prices = prices[~prices.name.str.contains('JohnnyJohnny')]
prices = prices[~prices.name.str.contains('JoilsonJoilson')]
prices = prices[~prices.name.str.contains('JoJo')]
prices = prices[~prices.name.str.contains('Jomaine Consbruch')]
prices = prices[~prices.name.str.contains('Jon BellJon Bell')]
prices = prices[~prices.name.str.contains('Jon Flanagan')]
prices = prices[~prices.name.str.contains('Jon Pacheco Dozagarat')]
prices = prices[~prices.name.str.contains('Jon Stead')]
prices = prices[~prices.name.str.contains('Jonah Osabutey')]
prices = prices[~prices.name.str.contains('Jonas Aguirre')]
prices = prices[~prices.name.str.contains('Jonas Carls')]
prices = prices[~prices.name.str.contains('JonasJonas')]
prices = prices[~prices.name.str.contains('Jonatan Gomez')]
prices = prices[~prices.name.str.contains('Jonatan Maidana')]
prices = prices[~prices.name.str.contains('Jonathan Bell')]
prices = prices[~prices.name.str.contains('Jonathan Bolingi')]
prices = prices[~prices.name.str.contains('Jonathan Cabral')]
prices = prices[~prices.name.str.contains('Jonathan Howson')]
prices = prices[~prices.name.str.contains('Jonathan Klinsmann')]
prices = prices[~prices.name.str.contains('Jonathan Mulder')]
prices = prices[~prices.name.str.contains('Jonathan Russell')]
prices = prices[~prices.name.str.contains('Jonathan Torres')]
prices = prices[~prices.name.str.contains('Jonathan Zacaria')]
prices = prices[~prices.name.str.contains('Jonny Howson')]
prices = prices[~prices.name.str.contains('Jonny Otto')]
prices = prices[~prices.name.str.contains('Jorben Vanhulle')]
prices = prices[~prices.name.str.contains('Jordan Barnett')]
prices = prices[~prices.name.str.contains('Jordan Beyer')]
prices = prices[~prices.name.str.contains('Jordan Eastham')]
prices = prices[~prices.name.str.contains('Jordan Flores')]
prices = prices[~prices.name.str.contains('Jordan Garrick')]
prices = prices[~prices.name.str.contains('Jordan Green')]
prices = prices[~prices.name.str.contains('Jordan Harvey')]
prices = prices[~prices.name.str.contains('Jordan Holsgrove')]
prices = prices[~prices.name.str.contains('Jordan Michael Flores')]
prices = prices[~prices.name.str.contains('Jordan Perruzza')]
prices = prices[~prices.name.str.contains('Jordan Roberts')]
prices = prices[~prices.name.str.contains('Jordan Thomas')]
prices = prices[~prices.name.str.contains('Jordan van der Gaag')]
prices = prices[~prices.name.str.contains('JordanJordan')]
prices = prices[~prices.name.str.contains('Jordon Garrick')]
prices = prices[~prices.name.str.contains('Jordon Thompson')]
prices = prices[~prices.name.str.contains('Jordy Schelfhout')]
prices = prices[~prices.name.str.contains('Jordy Wehrmann')]
prices = prices[~prices.name.str.contains('Joren DomJoren Dom')]
prices = prices[~prices.name.str.contains('Jores Rahou')]
prices = prices[~prices.name.str.contains('Jorge Aruga')]
prices = prices[~prices.name.str.contains('Jorge Caceres')]
prices = prices[~prices.name.str.contains('Jorge De Olivera')]
prices = prices[~prices.name.str.contains('Jorge Figal')]
prices = prices[~prices.name.str.contains('Jorge Moreira')]
prices = prices[~prices.name.str.contains('Jorge Ortega')]
prices = prices[~prices.name.str.contains('Jorge Ortiz')]
prices = prices[~prices.name.str.contains('Jorge Pereira')]
prices = prices[~prices.name.str.contains('Jorge Pereyra Diaz')]
prices = prices[~prices.name.str.contains('Jorge Saenz')]
prices = prices[~prices.name.str.contains('Joris Gnagnon')]
prices = prices[~prices.name.str.contains('Jorn Berkhout')]
prices = prices[~prices.name.str.contains('Joroen Dom')]
prices = prices[~prices.name.str.contains('Jorrit Smeets')]
prices = prices[~prices.name.str.contains('Jose Cifuentes')]
prices = prices[~prices.name.str.contains('Jose Devecchi')]
prices = prices[~prices.name.str.contains('Jose Gimenez')]
prices = prices[~prices.name.str.contains('Jose Gonzalez')]
prices = prices[~prices.name.str.contains('Jose Luis Fernandez')]
prices = prices[~prices.name.str.contains('Jose Luis Gomez')]
prices = prices[~prices.name.str.contains('Jose Luis Rodriguez')]
prices = prices[~prices.name.str.contains('Jose Luis Sinisterra')]
prices = prices[~prices.name.str.contains('Jose Manuel Fontan')]
prices = prices[~prices.name.str.contains('Jose Manuel Lopez')]
prices = prices[~prices.name.str.contains('Jose Marcelo')]
prices = prices[~prices.name.str.contains('Jose Mari')]
prices = prices[~prices.name.str.contains('Jose Maria Gimenez')]
prices = prices[~prices.name.str.contains('Jose MariJose Mari')]
prices = prices[~prices.name.str.contains('Jose Martinez')]
prices = prices[~prices.name.str.contains('Jose Moya')]
prices = prices[~prices.name.str.contains('JoryJory')]
prices = prices[~prices.name.str.contains('JosefJosef')]
prices = prices[~prices.name.str.contains('Joseph Amoah')]
prices = prices[~prices.name.str.contains('Joseph Collinge')]
prices = prices[~prices.name.str.contains('Joseph Dodoo')]
prices = prices[~prices.name.str.contains('Joseph Hungbo')]
prices = prices[~prices.name.str.contains('Joseph Maguire')]
prices = prices[~prices.name.str.contains('Joseph Marie Minala')]
prices = prices[~prices.name.str.contains('Joseph Martin Paintsil')]
prices = prices[~prices.name.str.contains('Joseph Paintsil')]
prices = prices[~prices.name.str.contains('Joseph Rafferty')]
prices = prices[~prices.name.str.contains('Joseph Willock')]
prices = prices[~prices.name.str.contains('Josete Miranda')]
prices = prices[~prices.name.str.contains('Josh Andrews')]
prices = prices[~prices.name.str.contains('Josh Atencio')]
prices = prices[~prices.name.str.contains('Josh Bauer')]
prices = prices[~prices.name.str.contains('Josh Coburn')]
prices = prices[~prices.name.str.contains('Josh EarlJosh Earl')]
prices = prices[~prices.name.str.contains('Josh Emmanuel')]
prices = prices[~prices.name.str.contains('Josh Ginnelly')]
prices = prices[~prices.name.str.contains('Josh Gould')]
prices = prices[~prices.name.str.contains('Josh KeyJosh Key')]
prices = prices[~prices.name.str.contains('Josh Knight')]
prices = prices[~prices.name.str.contains('Josh Lillis')]
prices = prices[~prices.name.str.contains('Josh Sargent')]
prices = prices[~prices.name.str.contains('Josh Vickers')]
prices = prices[~prices.name.str.contains('Joshua Atencio')]
prices = prices[~prices.name.str.contains('Joshua Earl')]
prices = prices[~prices.name.str.contains('Joshua Emmanuel')]
prices = prices[~prices.name.str.contains('Joshua Griffiths')]
prices = prices[~prices.name.str.contains('Joshua Key')]
prices = prices[~prices.name.str.contains('Joshua McPake')]
prices = prices[~prices.name.str.contains('Joshua Render')]
prices = prices[~prices.name.str.contains('Joshua Sargent')]
prices = prices[~prices.name.str.contains('Joshua Wehking')]
prices = prices[~prices.name.str.contains('Jospin Nshimirimana')]
prices = prices[~prices.name.str.contains('Josue Ayala')]
prices = prices[~prices.name.str.contains('Jota Peleteiro')]
prices = prices[~prices.name.str.contains('Jovanny Bolivar')]
prices = prices[~prices.name.str.contains('Joye Dezart')]
prices = prices[~prices.name.str.contains('JT Marcinkowski')]
prices = prices[~prices.name.str.contains('Juan Ãlvarez')]
prices = prices[~prices.name.str.contains('Juan Antonini')]
prices = prices[~prices.name.str.contains('Juan Bolado')]
prices = prices[~prices.name.str.contains('Juan Brunetta')]
prices = prices[~prices.name.str.contains('Juan Carlos Portillo')]
prices = prices[~prices.name.str.contains('Juan Castilla')]
prices = prices[~prices.name.str.contains('Juan Cavallaro')]
prices = prices[~prices.name.str.contains('Juan Caviglia')]
prices = prices[~prices.name.str.contains('Juan Cruz Komar')]
prices = prices[~prices.name.str.contains('Juan Espinola')]
prices = prices[~prices.name.str.contains('Juan Franzoni')]
prices = prices[~prices.name.str.contains('Juan Fuentes')]
prices = prices[~prices.name.str.contains('Juan Gonzalez')]
prices = prices[~prices.name.str.contains('Juan Iglesias')]
prices = prices[~prices.name.str.contains('Juan Ignacio Mendez')]
prices = prices[~prices.name.str.contains('Juan Ignacio Nardoni')]
prices = prices[~prices.name.str.contains('Juan Infante')]
prices = prices[~prices.name.str.contains('Juan Jose Caceres')]
prices = prices[~prices.name.str.contains('Juan Kaprof')]
prices = prices[~prices.name.str.contains('Juan Komar')]
prices = prices[~prices.name.str.contains('Juan Lucero')]
prices = prices[~prices.name.str.contains('Juan Manuel Cruz')]
prices = prices[~prices.name.str.contains('Juan Manuel Cuesta')]
prices = prices[~prices.name.str.contains('Juan Manuel Insaurralde')]
prices = prices[~prices.name.str.contains('Juan Martin Lucero')]
prices = prices[~prices.name.str.contains('Juan Miritello')]
prices = prices[~prices.name.str.contains('Juan Nardoni')]
prices = prices[~prices.name.str.contains('Juan Pablo Barinaga')]
prices = prices[~prices.name.str.contains('Juan Pablo Freytes')]
prices = prices[~prices.name.str.contains('Juan Pablo Ramirez')]
prices = prices[~prices.name.str.contains('Juan Pablo Romero')]
prices = prices[~prices.name.str.contains('Juan Pacchini')]
prices = prices[~prices.name.str.contains('Juan Roman Pucheta')]
prices = prices[~prices.name.str.contains('Juan Sforza')]
prices = prices[~prices.name.str.contains('Juan Vieyra')]
prices = prices[~prices.name.str.contains('Juan Zurbriggen')]
prices = prices[~prices.name.str.contains('Jugurtha Hamroun')]
prices = prices[~prices.name.str.contains('Jules-Anthony Vilsaint')]
prices = prices[~prices.name.str.contains('Julian Aude')]
prices = prices[~prices.name.str.contains('Julian Chicco')]
prices = prices[~prices.name.str.contains('Julian Dunn')]
prices = prices[~prices.name.str.contains('Julian Frommann')]
prices = prices[~prices.name.str.contains('Julian Krahl')]
prices = prices[~prices.name.str.contains('Julian Lopez')]
prices = prices[~prices.name.str.contains('Julian Malatini')]
prices = prices[~prices.name.str.contains('Julian Marcioni')]
prices = prices[~prices.name.str.contains('Julian Navas')]
prices = prices[~prices.name.str.contains('Julian Pollersbeck')]
prices = prices[~prices.name.str.contains('Julian Schieber')]
prices = prices[~prices.name.str.contains('Julian Velazquez')]
prices = prices[~prices.name.str.contains('Julio Gonzalez')]
prices = prices[~prices.name.str.contains('Julio Villalba')]
prices = prices[~prices.name.str.contains('Juninho Capixaba')]
prices = prices[~prices.name.str.contains('JunioJunio')]
prices = prices[~prices.name.str.contains('Junior Alonso')]
prices = prices[~prices.name.str.contains('Junior AlonsoJunior Alonso')]
prices = prices[~prices.name.str.contains('Junior Dina Ebimbe')]
prices = prices[~prices.name.str.contains('Junior Hoilett')]
prices = prices[~prices.name.str.contains('Justin Che')]
prices = prices[~prices.name.str.contains('Justin Haak')]
prices = prices[~prices.name.str.contains('Justin Johnson')]
prices = prices[~prices.name.str.contains('Justin Rennicks')]
prices = prices[~prices.name.str.contains('Justin vom Steeg')]
prices = prices[~prices.name.str.contains('Justo Giani')]
prices = prices[~prices.name.str.contains('Karim Zedadka')]
prices = prices[~prices.name.str.contains('Kartal Kayra Yilmaz')]
prices = prices[~prices.name.str.contains('Kasper Lunding')]
prices = prices[~prices.name.str.contains('Kasper Lunding Jakobsen')]
prices = prices[~prices.name.str.contains('Kemal Ruzgar')]
prices = prices[~prices.name.str.contains('KennedyKennedy')]
prices = prices[~prices.name.str.contains('Kevin Boma')]
prices = prices[~prices.name.str.contains('Kevin MalthusKevin Malthus')]
prices = prices[~prices.name.str.contains('Kevin Zenon')]
prices = prices[~prices.name.str.contains('Kibe')]
prices = prices[~prices.name.str.contains('Kiernan Dewsbury-Hall')]
prices = prices[~prices.name.str.contains('KikasKikas')]
prices = prices[~prices.name.str.contains('Koke Vegas')]
prices = prices[~prices.name.str.contains('Kosuke Nakamura')]
prices = prices[~prices.name.str.contains('Kouadio Dabila')]
prices = prices[~prices.name.str.contains('Kouadio-Yves Dabila')]
prices = prices[~prices.name.str.contains('Kuki Zalazar')]
prices = prices[~prices.name.str.contains('Kuku Emmanuel Fidelis')]
prices = prices[~prices.name.str.contains('Kwasi Okyere Wriedt')]
prices = prices[~prices.name.str.contains('LaercioLaercio')]
prices = prices[~prices.name.str.contains('Lars Lukas Mai')]
prices = prices[~prices.name.str.contains('Lautaro de Leon')]
prices = prices[~prices.name.str.contains('LazaroLazaro')]
prices = prices[~prices.name.str.contains('Leandro Contin')]
prices = prices[~prices.name.str.contains('Leandro Fernandez')]
prices = prices[~prices.name.str.contains('Lee GrantLee Grant')]
prices = prices[~prices.name.str.contains('Lee Seung-Woo')]
prices = prices[~prices.name.str.contains('Leo BorgesLeo Borges')]
prices = prices[~prices.name.str.contains('Leo Gomes')]
prices = prices[~prices.name.str.contains('Leo NatelLeo Natel')]
prices = prices[~prices.name.str.contains('Leo Ostigard')]
prices = prices[~prices.name.str.contains('Leo PassosLeo Passos')]
prices = prices[~prices.name.str.contains('Leo SantosLeo Santos')]
prices = prices[~prices.name.str.contains('Leon Chambers-Parillon')]
prices = prices[~prices.name.str.contains('Leon Sabua')]
prices = prices[~prices.name.str.contains('Leon Schaffran')]
prices = prices[~prices.name.str.contains('Leon Thomson King')]
prices = prices[~prices.name.str.contains('Leonel Gonzalez')]
prices = prices[~prices.name.str.contains('Levi Ntumba')]
prices = prices[~prices.name.str.contains('LiggerLigger')]
prices = prices[~prices.name.str.contains('Lillo Guarneri')]
prices = prices[~prices.name.str.contains('LinconLincon')]
prices = prices[~prices.name.str.contains('Lino Kasten')]
prices = prices[~prices.name.str.contains('Lion Lauberbach')]
prices = prices[~prices.name.str.contains('Lisandro Alzugaray')]
prices = prices[~prices.name.str.contains('Logan Delaurier-Chaubet')]
prices = prices[~prices.name.str.contains('LohanLohan')]
prices = prices[~prices.name.str.contains('Lois Diony')]
prices = prices[~prices.name.str.contains('Luan PolliLuan Polli')]
prices = prices[~prices.name.str.contains('Luca Martinez Dupuy')]
prices = prices[~prices.name.str.contains('Ludvig Ohman')]
prices = prices[~prices.name.str.contains('Luis Leal')]
prices = prices[~prices.name.str.contains('Luis Miguel Rodriguez')]
prices = prices[~prices.name.str.contains('Luis Ojeda')]
prices = prices[~prices.name.str.contains('Luis PhelipeLuis Phelipe')]
prices = prices[~prices.name.str.contains('Luis Rodriguez')]
prices = prices[~prices.name.str.contains('Luis Vazquez')]
prices = prices[~prices.name.str.contains('Luiz Otavio')]
prices = prices[~prices.name.str.contains('Lukas Fahrnberger')]
prices = prices[~prices.name.str.contains('Lukas Hinterseer')]
prices = prices[~prices.name.str.contains('Lukas Hornicek')]
prices = prices[~prices.name.str.contains('LumorLumor')]
prices = prices[~prices.name.str.contains('Mads Bidstrup')]
prices = prices[~prices.name.str.contains('Maikel van der Werff')]
prices = prices[~prices.name.str.contains('Majid Hosseini')]
prices = prices[~prices.name.str.contains('Maksim Kutovoi')]
prices = prices[~prices.name.str.contains('Maksim Martusevich')]
prices = prices[~prices.name.str.contains('Maksim Turischev')]
prices = prices[~prices.name.str.contains('Maksim Volodko')]
prices = prices[~prices.name.str.contains('Marc Auge')]
prices = prices[~prices.name.str.contains('Marc Baro')]
prices = prices[~prices.name.str.contains('Marc Olivier Doue')]
prices = prices[~prices.name.str.contains('Marcelo Meli')]
prices = prices[~prices.name.str.contains('Marcelo RyanMarcelo Ryan')]
prices = prices[~prices.name.str.contains('Marcio AzevedoMarcio Azevedo')]
prices = prices[~prices.name.str.contains('Marcio Mossoro')]
prices = prices[~prices.name.str.contains('Marco Torsiglieri')]
prices = prices[~prices.name.str.contains('Marcos Abreliano')]
prices = prices[~prices.name.str.contains('Marcos AndreMarcos Andre')]
prices = prices[~prices.name.str.contains('Marcos Angeleri')]
prices = prices[~prices.name.str.contains('Marcos BritoMarcos Brito')]
prices = prices[~prices.name.str.contains('Marcos de Sousa')]
prices = prices[~prices.name.str.contains('Marcos Landaburu')]
prices = prices[~prices.name.str.contains('Marcos Ledesma')]
prices = prices[~prices.name.str.contains('Marcos PauloMarcos Paulo')]
prices = prices[~prices.name.str.contains('Marcos Peano')]
prices = prices[~prices.name.str.contains('Marcos RochaMarcos Rocha')]
prices = prices[~prices.name.str.contains('Marcos Sanchez')]
prices = prices[~prices.name.str.contains('Mariano Diaz')]
prices = prices[~prices.name.str.contains('Mariano Vazquez')]
prices = prices[~prices.name.str.contains('Mark Kosznovszky')]
prices = prices[~prices.name.str.contains('Markus Schubert')]
prices = prices[~prices.name.str.contains('Marky Delgado')]
prices = prices[~prices.name.str.contains('Marley Ake')]
prices = prices[~prices.name.str.contains('Marley Marshall')]
prices = prices[~prices.name.str.contains('Martin Garay')]
prices = prices[~prices.name.str.contains('Martin Garcia')]
prices = prices[~prices.name.str.contains('Matheo Vroman')]
prices = prices[~prices.name.str.contains('Matheus FerrazMatheus Ferraz')]
prices = prices[~prices.name.str.contains('Matheus Kayser')]
prices = prices[~prices.name.str.contains('Matheus Martins')]
prices = prices[~prices.name.str.contains('Matheus Mascarenhas')]
prices = prices[~prices.name.str.contains('Matheus MendesMatheus Mendes')]
prices = prices[~prices.name.str.contains('Matheus NogueiraMatheus Nogueira')]
prices = prices[~prices.name.str.contains('Matheus Teixeira')]
prices = prices[~prices.name.str.contains('Mathias Abero')]
prices = prices[~prices.name.str.contains('Mathias Olivera')]
prices = prices[~prices.name.str.contains('Mathias Suarez')]
prices = prices[~prices.name.str.contains('Matias Belloso')]
prices = prices[~prices.name.str.contains('Matias Borgogno')]
prices = prices[~prices.name.str.contains('Matias Cordoba')]
prices = prices[~prices.name.str.contains('Matias Escudero')]
prices = prices[~prices.name.str.contains('Matias Fritzler')]
prices = prices[~prices.name.str.contains('Matias Garcia')]
prices = prices[~prices.name.str.contains('Matias Gomez')]
prices = prices[~prices.name.str.contains('Matias Gonzalez')]
prices = prices[~prices.name.str.contains('Matias Laba')]
prices = prices[~prices.name.str.contains('Matias Moya')]
prices = prices[~prices.name.str.contains('Matias Palavecino')]
prices = prices[~prices.name.str.contains('Matias Pellegrini')]
prices = prices[~prices.name.str.contains('Matias Pisano')]
prices = prices[~prices.name.str.contains('Matias Ramirez')]
prices = prices[~prices.name.str.contains('Matias Rodriguez')]
prices = prices[~prices.name.str.contains('Matias Sosa')]
prices = prices[~prices.name.str.contains('Matias Tagliamonte')]
prices = prices[~prices.name.str.contains('Matias Tissera')]
prices = prices[~prices.name.str.contains('Matias Villarreal')]
prices = prices[~prices.name.str.contains('Mauro Ortiz')]
prices = prices[~prices.name.str.contains('Max ClarkMax Clark')]
prices = prices[~prices.name.str.contains('Maxwell Haygarth')]
prices = prices[~prices.name.str.contains('Maycon CleitonMaycon Cleiton')]
prices = prices[~prices.name.str.contains('Mehmet Can Aydin')]
prices = prices[~prices.name.str.contains('Mehmet Erdem Ugurlu')]
prices = prices[~prices.name.str.contains('Mehmet Goktug Bakirbas')]
prices = prices[~prices.name.str.contains('Mehmet Umut Nayir')]
prices = prices[~prices.name.str.contains('Melvin Minter')]
prices = prices[~prices.name.str.contains('Melvin Ramusovic')]
prices = prices[~prices.name.str.contains('Melvin Sitti')]
prices = prices[~prices.name.str.contains('Meme Placca Fessou')]
prices = prices[~prices.name.str.contains('Menno Koch')]
prices = prices[~prices.name.str.contains('Mert Cetin')]
prices = prices[~prices.name.str.contains('Mert Gunok')]
prices = prices[~prices.name.str.contains('Mert KulaMert Kula')]
prices = prices[~prices.name.str.contains('Mert Sarikus')]
prices = prices[~prices.name.str.contains('Merveille Goblet')]
prices = prices[~prices.name.str.contains('Micah Obiero')]
prices = prices[~prices.name.str.contains('Michael Chacon')]
prices = prices[~prices.name.str.contains('Michael Lang')]
prices = prices[~prices.name.str.contains('Michel Araujo')]
prices = prices[~prices.name.str.contains('Michel Herrero')]
prices = prices[~prices.name.str.contains('Michel Macedo')]
prices = prices[~prices.name.str.contains('MichelMichel')]
prices = prices[~prices.name.str.contains('Miguel Barbieri')]
prices = prices[~prices.name.str.contains('Miguel Berry')]
prices = prices[~prices.name.str.contains('Mikey Ambrose')]
prices = prices[~prices.name.str.contains('Mikey Johnston')]
prices = prices[~prices.name.str.contains('Mohamed-Ali Cho')]
prices = prices[~prices.name.str.contains('Mohammed Kamara')]
prices = prices[~prices.name.str.contains('Mohammed Kesselly Kamara')]
prices = prices[~prices.name.str.contains('Mohammed Rabiu Al-Hassan')]
prices = prices[~prices.name.str.contains('Moises Caicedo')]
prices = prices[~prices.name.str.contains('Morgan Boyes')]
prices = prices[~prices.name.str.contains('Morgan Feeney')]
prices = prices[~prices.name.str.contains('Morgan Ferrier')]
prices = prices[~prices.name.str.contains('Morgan Penfold')]
prices = prices[~prices.name.str.contains('Mustafa Seyhan')]
prices = prices[~prices.name.str.contains('Mustafa Yumlu')]
prices = prices[~prices.name.str.contains('Mustapha Bundu')]
prices = prices[~prices.name.str.contains('Mustapha Olagunju')]
prices = prices[~prices.name.str.contains('Muzaffer Kablan')]
prices = prices[~prices.name.str.contains('Myles Judd')]
prices = prices[~prices.name.str.contains('Nacho Fernandez')]
prices = prices[~prices.name.str.contains('Nacho Martinez')]
prices = prices[~prices.name.str.contains('Nadir Ciftci')]
prices = prices[~prices.name.str.contains('Nadir Zeineddin')]
prices = prices[~prices.name.str.contains('Nahuel Arena')]
prices = prices[~prices.name.str.contains('Nahuel Banegas')]
prices = prices[~prices.name.str.contains('Nahuel Bustos')]
prices = prices[~prices.name.str.contains('Nahuel Iribarren')]
prices = prices[~prices.name.str.contains('Nair Tiknizyan')]
prices = prices[~prices.name.str.contains('Nando Pijnaker')]
prices = prices[~prices.name.str.contains('Nelson Iseguan')]
prices = prices[~prices.name.str.contains('Nestor Ortigoza')]
prices = prices[~prices.name.str.contains('Nico Gordon')]
prices = prices[~prices.name.str.contains('Nico Williams')]
prices = prices[~prices.name.str.contains('Nicola Borghetto')]
prices = prices[~prices.name.str.contains('Nicola Ravaglia')]
prices = prices[~prices.name.str.contains('Nicola Zalewski')]
prices = prices[~prices.name.str.contains('Nicolas Andereggen')]
prices = prices[~prices.name.str.contains('Nicolas Avellaneda')]
prices = prices[~prices.name.str.contains('Nicolas Blandi')]
prices = prices[~prices.name.str.contains('Nicolas Burgy')]
prices = prices[~prices.name.str.contains('Nicolas Contin')]
prices = prices[~prices.name.str.contains('Nicolas Correa')]
prices = prices[~prices.name.str.contains('Nicolas Domingo')]
prices = prices[~prices.name.str.contains('Nicolas Femia')]
prices = prices[~prices.name.str.contains('Nicolas Fernandez Mercau')]
prices = prices[~prices.name.str.contains('Nicolas Ferreyra')]
prices = prices[~prices.name.str.contains('Nicolas Figal')]
prices = prices[~prices.name.str.contains('Nicolas Gimenez')]
prices = prices[~prices.name.str.contains('Nicolas Lamendola')]
prices = prices[~prices.name.str.contains('Nicolas Linares')]
prices = prices[~prices.name.str.contains('Nicolas Mana')]
prices = prices[~prices.name.str.contains('Nicolas Mazzola')]
prices = prices[~prices.name.str.contains('Nicolas Messiniti')]
prices = prices[~prices.name.str.contains('Nicolas Morgantini')]
prices = prices[~prices.name.str.contains('Nicolas Romat')]
prices = prices[~prices.name.str.contains('Nicolas Romero')]
prices = prices[~prices.name.str.contains('Nicolas Schiappacasse')]
prices = prices[~prices.name.str.contains('Nicolas Thaller')]
prices = prices[~prices.name.str.contains('Nicolas Zalazar')]
prices = prices[~prices.name.str.contains('Nicolo Zaniolo')]
prices = prices[~prices.name.str.contains('Nigel Bertrams')]
prices = prices[~prices.name.str.contains('Nino ParaibaNino Paraiba')]
prices = prices[~prices.name.str.contains('O\'Shane Stewart')]
prices = prices[~prices.name.str.contains('Obite Evan Ndicka')]
prices = prices[~prices.name.str.contains('Odion Ighalo')]
prices = prices[~prices.name.str.contains('Odise Roshi')]
prices = prices[~prices.name.str.contains('Ody AlfaOdy Alfa')]
prices = prices[~prices.name.str.contains('Oguzhan Capar')]
prices = prices[~prices.name.str.contains('Oier Sanjurjo')]
prices = prices[~prices.name.str.contains('OierOier')]
prices = prices[~prices.name.str.contains('Okechukwu Azubuike')]
prices = prices[~prices.name.str.contains('Owen Windsor')]
prices = prices[~prices.name.str.contains('Owura Edwards')]
prices = prices[~prices.name.str.contains('Ozan Can Oruc')]
prices = prices[~prices.name.str.contains('Ozzie Cisneros')]
prices = prices[~prices.name.str.contains('Pablo DyegoPablo Dyego')]
prices = prices[~prices.name.str.contains('Pablo MaiaPablo Maia')]
prices = prices[~prices.name.str.contains('Pablo Minissale')]
prices = prices[~prices.name.str.contains('Pablo OroPablo Oro')]
prices = prices[~prices.name.str.contains('Pablo PardalPablo Pardal')]
prices = prices[~prices.name.str.contains('Pablo Velazquez')]
prices = prices[~prices.name.str.contains('Paulo MirandaPaulo Miranda')]
prices = prices[~prices.name.str.contains('Paulo VictorPaulo Victor')]
prices = prices[~prices.name.str.contains('Pedro RangelPedro Rangel')]
prices = prices[~prices.name.str.contains('Pedro Rocha')]
prices = prices[~prices.name.str.contains('PeglowPeglow')]
prices = prices[~prices.name.str.contains('PerottiPerotti')]
prices = prices[~prices.name.str.contains('Philip Turpitz')]
prices = prices[~prices.name.str.contains('Piero Hincapie')]
prices = prices[~prices.name.str.contains('Pol Garcia')]
prices = prices[~prices.name.str.contains('PopPop')]
prices = prices[~prices.name.str.contains('Praxedes')]
prices = prices[~prices.name.str.contains('Quentin Westberg')]
prices = prices[~prices.name.str.contains('Radoslaw Murawski')]
prices = prices[~prices.name.str.contains('Rafa SilvaRafa SilvaRight Midfiel')]
prices = prices[~prices.name.str.contains('Rafael Borre')]
prices = prices[~prices.name.str.contains('Rafael Garcia')]
prices = prices[~prices.name.str.contains('Rafael GrampolaRafael Grampola')]
prices = prices[~prices.name.str.contains('Rafael Longuine')]
prices = prices[~prices.name.str.contains('Ramon Arias')]
prices = prices[~prices.name.str.contains('Ramon Azeez')]
prices = prices[~prices.name.str.contains('Ramon Folch')]
prices = prices[~prices.name.str.contains('Ramon ten Hove')]
prices = prices[~prices.name.str.contains('RanielRaniel')]
prices = prices[~prices.name.str.contains('Raul Carnero')]
prices = prices[~prices.name.str.contains('Raul Lozano')]
prices = prices[~prices.name.str.contains('Raul Navas')]
prices = prices[~prices.name.str.contains('Raul Silva')]
prices = prices[~prices.name.str.contains('Ravel Morrison')]
prices = prices[~prices.name.str.contains('Reinaldo Lenis')]
prices = prices[~prices.name.str.contains('Renan Ribeiro')]
prices = prices[~prices.name.str.contains('Reynaldo')]
prices = prices[~prices.name.str.contains('Ricardo Velho')]
prices = prices[~prices.name.str.contains('Richecard Richard')]
prices = prices[~prices.name.str.contains('Richie Towell')]
prices = prices[~prices.name.str.contains('Rob Dickie')]
prices = prices[~prices.name.str.contains('Rob Elliot')]
prices = prices[~prices.name.str.contains('Rob HallRob Hall')]
prices = prices[~prices.name.str.contains('Robby McCrorie')]
prices = prices[~prices.name.str.contains('Rober Correa')]
prices = prices[~prices.name.str.contains('RobersonRoberson')]
prices = prices[~prices.name.str.contains('Roberto Jimenez')]
prices = prices[~prices.name.str.contains('Robinho')]
prices = prices[~prices.name.str.contains('Robson ReisRobson Reis')]
prices = prices[~prices.name.str.contains('RodineiRodinei')]
prices = prices[~prices.name.str.contains('Rodri Sanchez')]
prices = prices[~prices.name.str.contains('Rodrigo Arciero')]
prices = prices[~prices.name.str.contains('Rodrigo Gomez')]
prices = prices[~prices.name.str.contains('Rodrigo Herrera')]
prices = prices[~prices.name.str.contains('Rodrigo LindosoRodrigo Lindoso')]
prices = prices[~prices.name.str.contains('Rodrigo MoledoRodrigo Moledo')]
prices = prices[~prices.name.str.contains('Rodrigo Nestor')]
prices = prices[~prices.name.str.contains('Rodrigo Salinas')]
prices = prices[~prices.name.str.contains('Rodrigo Sanchez')]
prices = prices[~prices.name.str.contains('Rodrigo VarandaRodrigo Varanda')]
prices = prices[~prices.name.str.contains('Rogerio Santos')]
prices = prices[~prices.name.str.contains('Rolando Garcia')]
prices = prices[~prices.name.str.contains('Roman Eremenko')]
prices = prices[~prices.name.str.contains('Roman Martinez')]
prices = prices[~prices.name.str.contains('Romulo Otero')]
prices = prices[~prices.name.str.contains('Ron-Thorben Hoffmann')]
prices = prices[~prices.name.str.contains('Ruben Botta')]
prices = prices[~prices.name.str.contains('Ruben Fonseca')]
prices = prices[~prices.name.str.contains('Ruben Pardo')]
prices = prices[~prices.name.str.contains('Ruwen Werthmuller')]
prices = prices[~prices.name.str.contains('Saba Lobjanidze')]
prices = prices[~prices.name.str.contains('Saba Lobzhanidze')]
prices = prices[~prices.name.str.contains('Said Bakari')]
prices = prices[~prices.name.str.contains('Saidou Sow')]
prices = prices[~prices.name.str.contains('Saliou Sane')]
prices = prices[~prices.name.str.contains('Salva Ferrer')]
prices = prices[~prices.name.str.contains('Salvador Ferrer')]
prices = prices[~prices.name.str.contains('Sam BoothSam Booth')]
prices = prices[~prices.name.str.contains('Sam ByramSam Byram')]
prices = prices[~prices.name.str.contains('Sammie McLeod')]
prices = prices[~prices.name.str.contains('Sammy Bossut')]
prices = prices[~prices.name.str.contains('Sander Coopman')]
prices = prices[~prices.name.str.contains('Santiago Caseres')]
prices = prices[~prices.name.str.contains('Santiago Garcia')]
prices = prices[~prices.name.str.contains('Santiago Rodriguez')]
prices = prices[~prices.name.str.contains('Santiago Simon')]
prices = prices[~prices.name.str.contains('Santiago Trellez')]
prices = prices[~prices.name.str.contains('Santos Borre')]
prices = prices[~prices.name.str.contains('SaraviaSaravia')]
prices = prices[~prices.name.str.contains('Sarpreet Singh')]
prices = prices[~prices.name.str.contains('Saul Salcedo')]
prices = prices[~prices.name.str.contains('Sean Slaven')]
prices = prices[~prices.name.str.contains('Sebastian Assis')]
prices = prices[~prices.name.str.contains('Sebastian Blanco')]
prices = prices[~prices.name.str.contains('Sebastian Dubarbier')]
prices = prices[~prices.name.str.contains('Sebastian Meza')]
prices = prices[~prices.name.str.contains('Sebastian Ramirez')]
prices = prices[~prices.name.str.contains('Sebastian Sosa')]
prices = prices[~prices.name.str.contains('Sebastien Thill')]
prices = prices[~prices.name.str.contains('Sega Coulibaly')]
prices = prices[~prices.name.str.contains('Senne Lammens')]
prices = prices[~prices.name.str.contains('Sergi Garcia')]
prices = prices[~prices.name.str.contains('Sergio Benito')]
prices = prices[~prices.name.str.contains('Sergio Gonzalez')]
prices = prices[~prices.name.str.contains('Seydou SySeydou Sy')]
prices = prices[~prices.name.str.contains('Seyed Majid Hosseini')]
prices = prices[~prices.name.str.contains('Shadrach Ogie')]
prices = prices[~prices.name.str.contains('Shahriar Moghanlou')]
prices = prices[~prices.name.str.contains('Sidnei Tavares')]
prices = prices[~prices.name.str.contains('Siro Rosane')]
prices = prices[~prices.name.str.contains('SivaldoSivaldo')]
prices = prices[~prices.name.str.contains('Slobodan Rajkovic')]
prices = prices[~prices.name.str.contains('Son Heung-Min')]
prices = prices[~prices.name.str.contains('Sonny CoxSonny Cox')]
prices = prices[~prices.name.str.contains('Sonny Graham')]
prices = prices[~prices.name.str.contains('Sori ManeSori Mane')]
prices = prices[~prices.name.str.contains('Soslan Takazov')]
prices = prices[~prices.name.str.contains('Suleyman Cebeci')]
prices = prices[~prices.name.str.contains('Sylian Mokono')]
prices = prices[~prices.name.str.contains('TabataTabata')]
prices = prices[~prices.name.str.contains('Tahvon Campbell')]
prices = prices[~prices.name.str.contains('TallesTalles')]
prices = prices[~prices.name.str.contains('Tariq Uwakwe')]
prices = prices[~prices.name.str.contains('Tein Troost')]
prices = prices[~prices.name.str.contains('Telmo ArcanjoTelmo Arcanjo')]
prices = prices[~prices.name.str.contains('Theo Leoni')]
prices = prices[~prices.name.str.contains('Thiago RibeiroThiago Ribeiro')]
prices = prices[~prices.name.str.contains('Thomas O\'Connor')]
prices = prices[~prices.name.str.contains('Thomas Ortega')]
prices = prices[~prices.name.str.contains('Thomas Ouwejan')]
prices = prices[~prices.name.str.contains('Thomas Toure')]
prices = prices[~prices.name.str.contains('ThulerThuler')]
prices = prices[~prices.name.str.contains('Thuram')]
prices = prices[~prices.name.str.contains('ThyllerThyller')]
prices = prices[~prices.name.str.contains('TiagoTiago')]
prices = prices[~prices.name.str.contains('Tibet Oniz')]
prices = prices[~prices.name.str.contains('Timmy Abraham')]
prices = prices[~prices.name.str.contains('Timothe Nkada')]
prices = prices[~prices.name.str.contains('Tobias Zarate')]
prices = prices[~prices.name.str.contains('Tom HamerTom Hamer')]
prices = prices[~prices.name.str.contains('Tomas Andrade')]
prices = prices[~prices.name.str.contains('Tomas Cardona')]
prices = prices[~prices.name.str.contains('Tomas Cavanagh')]
prices = prices[~prices.name.str.contains('Tomas Durso')]
prices = prices[~prices.name.str.contains('Tomas Escalante')]
prices = prices[~prices.name.str.contains('Tomas Fernandez')]
prices = prices[~prices.name.str.contains('Tomas Marchiori')]
prices = prices[~prices.name.str.contains('Tomas Martinez')]
prices = prices[~prices.name.str.contains('Tomas Ortiz')]
prices = prices[~prices.name.str.contains('Tomas Romero')]
prices = prices[~prices.name.str.contains('Tomas Sandoval')]
prices = prices[~prices.name.str.contains('Tomas Silva')]
prices = prices[~prices.name.str.contains('TQ AddyTQ Addy')]
prices = prices[~prices.name.str.contains('Tyreece John-Jules')]
prices = prices[~prices.name.str.contains('Ugo Bertelli')]
prices = prices[~prices.name.str.contains('Uriel Ramirez')]
prices = prices[~prices.name.str.contains('Vani Da Silva')]
prices = prices[~prices.name.str.contains('Vaso Vasic')]
prices = prices[~prices.name.str.contains('Vaughn Covil')]
prices = prices[~prices.name.str.contains('Victor Camarasa')]
prices = prices[~prices.name.str.contains('Victor FerrazVictor Ferraz')]
prices = prices[~prices.name.str.contains('Victor KawaVictor Kawaattac')]
prices = prices[~prices.name.str.contains('Victor YanVictor Yan')]
prices = prices[~prices.name.str.contains('VinaVina')]
prices = prices[~prices.name.str.contains('Vincent Muller')]
prices = prices[~prices.name.str.contains('Vinicius BalieiroVinicius Balieiro')]
prices = prices[~prices.name.str.contains('Vinicius FoguinhoVinicius Foguinho')]
prices = prices[~prices.name.str.contains('Vinicius MingottiVinicius Mingotti')]
prices = prices[~prices.name.str.contains('Vinicius SilvestreVinicius Silvestre')]
prices = prices[~prices.name.str.contains('Vitor Hugodefenc')]
prices = prices[~prices.name.str.contains('Vitor JacareVitor Jacare')]
prices = prices[~prices.name.str.contains('Vitor RicardoVitor Ricardo')]
prices = prices[~prices.name.str.contains('WalberWalber')]
prices = prices[~prices.name.str.contains('WalceWalce')]
prices = prices[~prices.name.str.contains('WallisonWallison')]
prices = prices[~prices.name.str.contains('Walter Andrade')]
prices = prices[~prices.name.str.contains('Walter Kannemann')]
prices = prices[~prices.name.str.contains('Warner Hahn')]
prices = prices[~prices.name.str.contains('Wayne Hennessey')]
prices = prices[~prices.name.str.contains('Wayne Rooney')]
prices = prices[~prices.name.str.contains('Welinton Junior')]
prices = prices[~prices.name.str.contains('Will VintWill Vint')]
prices = prices[~prices.name.str.contains('William MatheusWilliam Matheus')]
prices = prices[~prices.name.str.contains('William OliveiraWilliam Oliveira')]
prices = prices[~prices.name.str.contains('William Simba')]
prices = prices[~prices.name.str.contains('William Smallbone')]
prices = prices[~prices.name.str.contains('William Togui')]
prices = prices[~prices.name.str.contains('Williams Barlasina')]
prices = prices[~prices.name.str.contains('Willian MaranhaoWillian Maranhao')]
prices = prices[~prices.name.str.contains('Wim Vanmarsenille')]
prices = prices[~prices.name.str.contains('Wiris Gustavo')]
prices = prices[~prices.name.str.contains('Wisdom Amey')]
prices = prices[~prices.name.str.contains('Woo-yeong Jeong')]
prices = prices[~prices.name.str.contains('XadasXadas')]
prices = prices[~prices.name.str.contains('Xander Severina')]
prices = prices[~prices.name.str.contains('Xandro Schenk')]
prices = prices[~prices.name.str.contains('XavierXavier')]
prices = prices[~prices.name.str.contains('Yahya Soumare')]
prices = prices[~prices.name.str.contains('Yair Arismendi')]
prices = prices[~prices.name.str.contains('Yan SasseYan Sasse')]
prices = prices[~prices.name.str.contains('Yanis Lhery')]
prices = prices[~prices.name.str.contains('Yann M\'Vila')]
prices = prices[~prices.name.str.contains('Yasin Oztekin')]
prices = prices[~prices.name.str.contains('Yaya Soumare')]
prices = prices[~prices.name.str.contains('Yildirim Mert Ã‡etin')]
prices = prices[~prices.name.str.contains('Younes Belhanda')]
prices = prices[~prices.name.str.contains('Youssef Ait Bennasser')]
prices = prices[~prices.name.str.contains('Youssuf Sylla')]
prices = prices[~prices.name.str.contains('Yuki Kobayashi')]
prices = prices[~prices.name.str.contains('Yun Il-Lok')]
prices = prices[~prices.name.str.contains('Yunus Bahadir')]
prices = prices[~prices.name.str.contains('Yusuf Ozdemir')]

df = df[~df.Name.str.contains('Alfon')]
df = df[~df.Name.str.contains('Carlos Domínguez')]
df = df[~df.Name.str.contains('Costel Pantilimon')]
df = df[~df.Name.str.contains('C.J. Sapong')]
df = df[~df.Name.str.contains('Carlos Dominguez')]
df = df[~df.Name.str.contains('Dan Ndoye')]
df = df[~df.Name.str.contains('Daniel Iversen')]
df = df[~df.Name.str.contains('Daniel Sappa')]
df = df[~df.Name.str.contains('Daniele De Rossi')]
df = df[~df.Name.str.contains('Dario Essugo')]
df = df[~df.Name.str.contains('Darnell Johnson')]
df = df[~df.Name.str.contains('Darren Keet')]
df = df[~df.Name.str.contains('David Abraham')]
df = df[~df.Name.str.contains('David Barbona')]
df = df[~df.Name.str.contains('Allan-Romeo Nyom')]
df = df[~df.Name.str.contains('Braian Cufre')]
df = df[~df.Name.str.contains('Cesar Ibañez')]
df = df[~df.Name.str.contains('Charalambos Makridis')]
df = df[~df.Name.str.contains('Charlie O\'Connell')]
df = df[~df.Name.str.contains('Danilo Ortiz')]
df = df[~df.Name.str.contains('David Henen')]
df = df[~df.Name.str.contains('Di\'Shon Bernard')]
df = df[~df.Name.str.contains('Alex Bono')]
df = df[~df.Name.str.contains('Alex Martin')]
df = df[~df.Name.str.contains('Alex Millan')]
df = df[~df.Name.str.contains('Alex Vallejo')]
df = df[~df.Name.str.contains('Alvaro Bravo')]
df = df[~df.Name.str.contains('Alvaro Fernandez')]
df = df[~df.Name.str.contains('Alvaro Gimenez')]
df = df[~df.Name.str.contains('Angelo Preciado')]
df = df[~df.Name.str.contains('Angelo Sagal')]
df = df[~df.Name.str.contains('Aykut Ozer')]
df = df[~df.Name.str.contains('Casper Staring')]
df = df[~df.Name.str.contains('Cesar Ibanez')]
df = df[~df.Name.str.contains('Djibril Dianessy')]
df = df[~df.Name.str.contains('Dogan Alemdar')]
df = df[~df.Name.str.contains('Dogan Erdogan')]
df = df[~df.Name.str.contains('Dominic Telford')]
df = df[~df.Name.str.contains('Elias Sierra-Cappelleti')]
df = df[~df.Name.str.contains('Enaut Mendia')]
df = df[~df.Name.str.contains('Enrico Hernandez')]
df = df[~df.Name.str.contains('Evgeni Gapon')]
df = df[~df.Name.str.contains('Evgeniy Tatarinov')]
df = df[~df.Name.str.contains('Evgeny Kharin')]
df = df[~df.Name.str.contains('Ewan Henderson')]
df = df[~df.Name.str.contains('Fabian Cubero')]
df = df[~df.Name.str.contains('Fabio Abreu')]
df = df[~df.Name.str.contains('Facundo Sanchez')]
df = df[~df.Name.str.contains('Federico Martinez')]
df = df[~df.Name.str.contains('Fehmi Mert Gunok')]
df = df[~df.Name.str.contains('Fernando Nino')]
df = df[~df.Name.str.contains('Filip Moller Delaveris')]
df = df[~df.Name.str.contains('Fin Bartels')]
df = df[~df.Name.str.contains('Francisco Ramirez')]
df = df[~df.Name.str.contains('Franck-Yves Bambock')]
df = df[~df.Name.str.contains('Franco Gonzalez')]
df = df[~df.Name.str.contains('Franco Perez')]
df = df[~df.Name.str.contains('Frederik Sörensen')]
df = df[~df.Name.str.contains('Gabriel Diaz')]
df = df[~df.Name.str.contains('Gabriel Menino')]
df = df[~df.Name.str.contains('Gaston Alvarez Suarez')]
df = df[~df.Name.str.contains('Gervasio Nunez')]
df = df[~df.Name.str.contains('Gonzalo Rodriguez')]
df = df[~df.Name.str.contains('Gonzalo Veron')]
df = df[~df.Name.str.contains('Guillermo Fernandez')]
df = df[~df.Name.str.contains('Halil Pehlivan')]
df = df[~df.Name.str.contains('Hasan Huseyin Acar')]
df = df[~df.Name.str.contains('Hernan Burbano')]
df = df[~df.Name.str.contains('Hwang In-Beom')]
df = df[~df.Name.str.contains('Hwang Ui-Jo')]
df = df[~df.Name.str.contains('Ibrahim Sissoko')]
df = df[~df.Name.str.contains('Ibrahim Tsallagov')]
df = df[~df.Name.str.contains('Ibrahima Camara')]
df = df[~df.Name.str.contains('Igor Bezdenezhnykh')]
df = df[~df.Name.str.contains('Ilker Karakas')]
df = df[~df.Name.str.contains('Islambek Kuat')]
df = df[~df.Name.str.contains('Ismail Yuksek')]
df = df[~df.Name.str.contains('Moritz Römling')]
df = df[~df.Name.str.contains('Mustafa Çecenoglu')]
df = df[~df.Name.str.contains('RegisRegis')]
df = df[~df.Name.str.contains('Jonas Aguirre')]
df = df[~df.Name.str.contains('Jonas Sterner')]
df = df[~df.Name.str.contains('Jonas Toro')]
df = df[~df.Name.str.contains('Jonatan Gomez')]
df = df[~df.Name.str.contains('Jonathan Zacaria')]
df = df[~df.Name.str.contains('Jony Alamo')]
df = df[~df.Name.str.contains('Jop van den Avert')]
df = df[~df.Name.str.contains('Jordi Osei-Tutu')]
df = df[~df.Name.str.contains('Jores Rahou')]
df = df[~df.Name.str.contains('Jorge Caceres')]
df = df[~df.Name.str.contains('Jorge Pereyra Diaz')]
df = df[~df.Name.str.contains('Jorge Saenz')]
df = df[~df.Name.str.contains('Jorgen Strand Larsen')]
df = df[~df.Name.str.contains('Jose Adolfo Valencia')]
df = df[~df.Name.str.contains('Jose Andres Martinez')]
df = df[~df.Name.str.contains('Jose AngelJose Angel')]
df = df[~df.Name.str.contains('Jose Cifuentes')]
df = df[~df.Name.str.contains('Jose Devecchi')]
df = df[~df.Name.str.contains('Jose Gimenez')]
df = df[~df.Name.str.contains('Jose Gonzalez')]
df = df[~df.Name.str.contains('Jose Luis Fernandez')]
df = df[~df.Name.str.contains('Jose Luis Gomez')]
df = df[~df.Name.str.contains('Jose Luis Rodriguez')]
df = df[~df.Name.str.contains('Jose Luis Sinisterra')]
df = df[~df.Name.str.contains('Jose Manuel Fontan')]
df = df[~df.Name.str.contains('Jose Manuel Lopez')]
df = df[~df.Name.str.contains('Jose Marcelo')]
df = df[~df.Name.str.contains('Jose Maria Gimenez')]
df = df[~df.Name.str.contains('Jose Martinez')]
df = df[~df.Name.str.contains('Jose Moya')]
df = df[~df.Name.str.contains('Josh JackJosh Jack')]
df = df[~df.Name.str.contains('Josh Neufville')]
df = df[~df.Name.str.contains('Josh Penn')]
df = df[~df.Name.str.contains('Josue Ayala')]
df = df[~df.Name.str.contains('Jovanny Bolivar')]
df = df[~df.Name.str.contains('Juan Alvarez')]
df = df[~df.Name.str.contains('Juan Cruz')]
df = df[~df.Name.str.contains('Juan Espinola')]
df = df[~df.Name.str.contains('Juan Gonzalez')]
df = df[~df.Name.str.contains('Juan Ignacio Mendez')]
df = df[~df.Name.str.contains('Juan Jose Caceres')]
df = df[~df.Name.str.contains('Juan Martin Lucero')]
df = df[~df.Name.str.contains('Juan Mendez')]
df = df[~df.Name.str.contains('Juan Pablo Alvarez')]
df = df[~df.Name.str.contains('Juan Pablo Ramirez')]
df = df[~df.Name.str.contains('Juan Roman Pucheta')]
df = df[~df.Name.str.contains('JuanJuan')]
df = df[~df.Name.str.contains('Jubril Okedina')]
df = df[~df.Name.str.contains('Julian Aude')]
df = df[~df.Name.str.contains('Julian Chicco')]
df = df[~df.Name.str.contains('Julian Lopez')]
df = df[~df.Name.str.contains('Julian Malatini')]
df = df[~df.Name.str.contains('Julian Marcioni')]
df = df[~df.Name.str.contains('Julian Navas')]
df = df[~df.Name.str.contains('Julian Velazquez')]
df = df[~df.Name.str.contains('Julio Gonzalez')]
df = df[~df.Name.str.contains('JuniorJunior')]
df = df[~df.Name.str.contains('KaduKadu')]
df = df[~df.Name.str.contains('Kaide Gordon')]
df = df[~df.Name.str.contains('Kaique RochaKaique Rocha')]
df = df[~df.Name.str.contains('KeillerKeiller')]
df = df[~df.Name.str.contains('KelvinKelvin')]
df = df[~df.Name.str.contains('Kemal Ruzgar')]
df = df[~df.Name.str.contains('KenoKeno')]
df = df[~df.Name.str.contains('KenoKeno')]
df = df[~df.Name.str.contains('Kerim Calhanoglu')]
df = df[~df.Name.str.contains('Kevin Boma')]
df = df[~df.Name.str.contains('Kevin Zenon')]
df = df[~df.Name.str.contains('Kian Slor')]
df = df[~df.Name.str.contains('Koffi')]
df = df[~df.Name.str.contains('Konstantin Kovalev')]
df = df[~df.Name.str.contains('Konstantin Rausch')]
df = df[~df.Name.str.contains('Kornell MacDonald')]
df = df[~df.Name.str.contains('Kortne Ford')]
df = df[~df.Name.str.contains('Kostas Stafylidis')]
df = df[~df.Name.str.contains('Laercio Solda')]
df = df[~df.Name.str.contains('LaercioLaercio')]
df = df[~df.Name.str.contains('Lars MaiLars Mai')]
df = df[~df.Name.str.contains('Lasse Sorensen')]
df = df[~df.Name.str.contains('Laureano Punet')]
df = df[~df.Name.str.contains('Lautaro Chavez')]
df = df[~df.Name.str.contains('Lautaro de Leon')]
df = df[~df.Name.str.contains('LazaroLazaro')]
df = df[~df.Name.str.contains('Leandro Contin')]
df = df[~df.Name.str.contains('Leandro Fernandez')]
df = df[~df.Name.str.contains('Lennart Schulze')]
df = df[~df.Name.str.contains('Lee O\'Connor')]
df = df[~df.Name.str.contains('Leo BorgesLeo Borges')]
df = df[~df.Name.str.contains('Leo Gomes')]
df = df[~df.Name.str.contains('Leo NatelLeo Natel')]
df = df[~df.Name.str.contains('Leonel Gonzalez')]
df = df[~df.Name.str.contains('Levi Ntumba')]
df = df[~df.Name.str.contains('Lois Diony')]
df = df[~df.Name.str.contains('Luca Martinez Dupuy')]
df = df[~df.Name.str.contains('Ludvig Ohman')]
df = df[~df.Name.str.contains('Luis Leal')]
df = df[~df.Name.str.contains('Luis Miguel Rodriguez')]
df = df[~df.Name.str.contains('Luis Rodriguez')]
df = df[~df.Name.str.contains('Luis Vazquez')]
df = df[~df.Name.str.contains('Luiz Fernando')]
df = df[~df.Name.str.contains('Luiz JuniorLuiz Junior')]
df = df[~df.Name.str.contains('Luiz Lucio Reis Junior')]
df = df[~df.Name.str.contains('Luiz Otavio')]
df = df[~df.Name.str.contains('Luka Zarandia')]
df = df[~df.Name.str.contains('MaiconMaicon')]
df = df[~df.Name.str.contains('Malik Sellouki')]
df = df[~df.Name.str.contains('Malvind Benning')]
df = df[~df.Name.str.contains('Mamadi Camara')]
df = df[~df.Name.str.contains('Mamadou Coulibaly')]
df = df[~df.Name.str.contains('Marc Auge')]
df = df[~df.Name.str.contains('Marc Baro')]
df = df[~df.Name.str.contains('Marc-Olivier Doue')]
df = df[~df.Name.str.contains('Marcelinho')]
df = df[~df.Name.str.contains('Marcio AzevedoMarcio Azevedo')]
df = df[~df.Name.str.contains('Marcio Mossoro')]
df = df[~df.Name.str.contains('Marcos AndreMarcos Andre')]
df = df[~df.Name.str.contains('Marcos Sanchez')]
df = df[~df.Name.str.contains('Mariano Diaz')]
df = df[~df.Name.str.contains('Mariano Vazquez')]
df = df[~df.Name.str.contains('Mark Kosznovszky')]
df = df[~df.Name.str.contains('Markus Fjortoft')]
df = df[~df.Name.str.contains('Marley Ake')]
df = df[~df.Name.str.contains('Martin Campana')]
df = df[~df.Name.str.contains('Martin Garay')]
df = df[~df.Name.str.contains('Martin Garcia')]
df = df[~df.Name.str.contains('Martin Rabunal')]
df = df[~df.Name.str.contains('Mateus BrunMateus Brun')]
df = df[~df.Name.str.contains('Matheo Vroman')]
df = df[~df.Name.str.contains('Mathias Abero')]
df = df[~df.Name.str.contains('Mathias Kjolo')]
df = df[~df.Name.str.contains('Mathias Suarez')]
df = df[~df.Name.str.contains('Matias Belloso')]
df = df[~df.Name.str.contains('Matias Borgogno')]
df = df[~df.Name.str.contains('Matias Cordoba')]
df = df[~df.Name.str.contains('Matias Escudero')]
df = df[~df.Name.str.contains('Matias Fritzler')]
df = df[~df.Name.str.contains('Matias Garcia')]
df = df[~df.Name.str.contains('Matias Gomez')]
df = df[~df.Name.str.contains('Matias Gonzalez')]
df = df[~df.Name.str.contains('Matias Laba')]
df = df[~df.Name.str.contains('Matias Moya')]
df = df[~df.Name.str.contains('Matias Palavecino')]
df = df[~df.Name.str.contains('Matias Pellegrini')]
df = df[~df.Name.str.contains('Matias Pisano')]
df = df[~df.Name.str.contains('Matias Ramirez')]
df = df[~df.Name.str.contains('Matias Rodriguez')]
df = df[~df.Name.str.contains('Matias Sosa')]
df = df[~df.Name.str.contains('Matias Tagliamonte')]
df = df[~df.Name.str.contains('Matias Tissera')]
df = df[~df.Name.str.contains('Matias Villarreal')]
df = df[~df.Name.str.contains('Mauro Ortiz')]
df = df[~df.Name.str.contains('Max GrunMax Grun')]
df = df[~df.Name.str.contains('Max WalefMax Walef')]
df = df[~df.Name.str.contains('Meme Placca Fessou')]
df = df[~df.Name.str.contains('Mert Gunok')]
df = df[~df.Name.str.contains('Metin Yildirim')]
df = df[~df.Name.str.contains('MetinhoMetinho')]
df = df[~df.Name.str.contains('Michael Chacon')]
df = df[~df.Name.str.contains('Michel Araujo')]
df = df[~df.Name.str.contains('Michel Herrero')]
df = df[~df.Name.str.contains('Michel Macedo')]
df = df[~df.Name.str.contains('MichelMichel')]
df = df[~df.Name.str.contains('Miguel Angel Atienza')]
df = df[~df.Name.str.contains('Miguel Angel Guerrero')]
df = df[~df.Name.str.contains('Miguel Angel Moya')]
df = df[~df.Name.str.contains('Miguel Angel Rubio')]
df = df[~df.Name.str.contains('Mike GrimMike Grim')]
df = df[~df.Name.str.contains('Milton Alvarez')]
df = df[~df.Name.str.contains('Milton Gimenez')]
df = df[~df.Name.str.contains('Miquel Scarlett')]
df = df[~df.Name.str.contains('Mohamed Traore')]
df = df[~df.Name.str.contains('Moises Caicedo')]
df = df[~df.Name.str.contains('Moon-hwan Kim')]
df = df[~df.Name.str.contains('MoraesMoraes')]
df = df[~df.Name.str.contains('Morato')]
df = df[~df.Name.str.contains('Moussa Wague')]
df = df[~df.Name.str.contains('Murat Akca')]
df = df[~df.Name.str.contains('Murat Aksit')]
df = df[~df.Name.str.contains('Murat Sipahioglu')]
df = df[~df.Name.str.contains('MurielMuriel')]
df = df[~df.Name.str.contains('Murilo RangelMurilo Rangel')]
df = df[~df.Name.str.contains('Mustafa Cecenoglu')]
df = df[~df.Name.str.contains('Mustafa Huseyin Seyhan')]
df = df[~df.Name.str.contains('NatanaelNatanael')]
df = df[~df.Name.str.contains('NatanNatan')]
df = df[~df.Name.str.contains('Nathan-Dylan Saliba')]
df = df[~df.Name.str.contains('Nathangelo Markelo')]
df = df[~df.Name.str.contains('Nazareno Solis')]
df = df[~df.Name.str.contains('NeiltonNeilton')]
df = df[~df.Name.str.contains('Nelson Ibanez')]
df = df[~df.Name.str.contains('Nemanja Antonov')]
df = df[~df.Name.str.contains('Nestor Ortigoza')]
df = df[~df.Name.str.contains('Nicolas Andereggen')]
df = df[~df.Name.str.contains('Nicolas Avellaneda')]
df = df[~df.Name.str.contains('Nicolas Blandi')]
df = df[~df.Name.str.contains('Nicolas Burgy')]
df = df[~df.Name.str.contains('Nicolas Contin')]
df = df[~df.Name.str.contains('Nicolas Correa')]
df = df[~df.Name.str.contains('Nicolas Domingo')]
df = df[~df.Name.str.contains('Nicolas Femia')]
df = df[~df.Name.str.contains('Nicolas Fernandez Mercau')]
df = df[~df.Name.str.contains('Nicolas Ferreyra')]
df = df[~df.Name.str.contains('Nicolas Figal')]
df = df[~df.Name.str.contains('Nicolas Gimenez')]
df = df[~df.Name.str.contains('Nicolas Lamendola')]
df = df[~df.Name.str.contains('Nicolas Linares')]
df = df[~df.Name.str.contains('Nicolas Mana')]
df = df[~df.Name.str.contains('Nicolas Mazzola')]
df = df[~df.Name.str.contains('Nicolas Messiniti')]
df = df[~df.Name.str.contains('Nicolas Morgantini')]
df = df[~df.Name.str.contains('Nicolas Olsak')]
df = df[~df.Name.str.contains('Nicolas Penailillo')]
df = df[~df.Name.str.contains('Nicolas Romat')]
df = df[~df.Name.str.contains('Nicolas Romero')]
df = df[~df.Name.str.contains('Nicolas Schiappacasse')]
df = df[~df.Name.str.contains('Nicolas Thaller')]
df = df[~df.Name.str.contains('Nicolas Zalazar')]
df = df[~df.Name.str.contains('Nicolo Zaniolo')]
df = df[~df.Name.str.contains('Nikola Pejčić')]
df = df[~df.Name.str.contains('Nino ParaibaNino Paraiba')]
df = df[~df.Name.str.contains('Omer Beyaz')]
df = df[~df.Name.str.contains('Omer Sismanoglu')]
df = df[~df.Name.str.contains('Oscar Benitez')]
df = df[~df.Name.str.contains('Oscar Linner')]
df = df[~df.Name.str.contains('Oscar Schönfelder')]
df = df[~df.Name.str.contains('OsmanOsman')]
df = df[~df.Name.str.contains('OsvaldoOsvaldo')]
df = df[~df.Name.str.contains('Pablo Velazquez')]
df = df[~df.Name.str.contains('PaulaoPaulao')]
df = df[~df.Name.str.contains('Paulinho BoiaPaulinho Boia')]
df = df[~df.Name.str.contains('Pedro Goncalves')]
df = df[~df.Name.str.contains('Pepê')]
df = df[~df.Name.str.contains('Pepe Bonet')]
df = df[~df.Name.str.contains('Philip Turpitz')]
df = df[~df.Name.str.contains('Piero Hincapie')]
df = df[~df.Name.str.contains('Piris da MottaPiris da Motta')]
df = df[~df.Name.str.contains('Pol Garcia')]
df = df[~df.Name.str.contains('Pote')]
df = df[~df.Name.str.contains('Rafael Borre')]
df = df[~df.Name.str.contains('Rafael Garcia')]
df = df[~df.Name.str.contains('Rahman Bugra Cagiran')]
df = df[~df.Name.str.contains('Ramon Arias')]
df = df[~df.Name.str.contains('Ramon Azeez')]
df = df[~df.Name.str.contains('Ramon Folch')]
df = df[~df.Name.str.contains('Ramon ten Hove')]
df = df[~df.Name.str.contains('Rashad Muhammed')]
df = df[~df.Name.str.contains('Rashid')]
df = df[~df.Name.str.contains('Raul Carnero')]
df = df[~df.Name.str.contains('Raul Lozano')]
df = df[~df.Name.str.contains('Raul Navas')]
df = df[~df.Name.str.contains('Raul Silva')]
df = df[~df.Name.str.contains('Raul SilvaRaul Silva')]
df = df[~df.Name.str.contains('Remeao Hutton')]
df = df[~df.Name.str.contains('RenêRenê')]
df = df[~df.Name.str.contains('Rhys Fenlon')]
df = df[~df.Name.str.contains('Rhys-James Fenlon')]
df = df[~df.Name.str.contains('RibamarRibamar')]
df = df[~df.Name.str.contains('Ricard Sanchez')]
df = df[~df.Name.str.contains('Ricardo Gomes')]
df = df[~df.Name.str.contains('Ricardo SilvaRicardo Silva')]
df = df[~df.Name.str.contains('RildoRildo')]
df = df[~df.Name.str.contains('Roberto Jimenez')]
df = df[~df.Name.str.contains('Rodri Sanchez')]
df = df[~df.Name.str.contains('Rodrigo Sanchez')]
df = df[~df.Name.str.contains('Rodrigues')]
df = df[~df.Name.str.contains('Rogerio Santos')]
df = df[~df.Name.str.contains('Roger IbanezRoger Ibanez')]
df = df[~df.Name.str.contains('Rolando Garcia')]
df = df[~df.Name.str.contains('Roman Martinez')]
df = df[~df.Name.str.contains('Romulo Otero')]
df = df[~df.Name.str.contains('Ruben Botta')]
df = df[~df.Name.str.contains('Ruben Fonseca')]
df = df[~df.Name.str.contains('Ruben Pardo')]
df = df[~df.Name.str.contains('Rui EncarnacaoRui Encarnacao')]
df = df[~df.Name.str.contains('Ruwen Werthmuller')]
df = df[~df.Name.str.contains('Sören Dieckmann')]
df = df[~df.Name.str.contains('Saîf-Eddine Khaoui')]
df = df[~df.Name.str.contains('Said Bakari')]
df = df[~df.Name.str.contains('Saidou Sow')]
df = df[~df.Name.str.contains('Saliou Sane')]
df = df[~df.Name.str.contains('Sandro PerpetuoSandro Perpetuo')]
df = df[~df.Name.str.contains('SandrySandry')]
df = df[~df.Name.str.contains('Santiago Brinone')]
df = df[~df.Name.str.contains('Santiago Caseres')]
df = df[~df.Name.str.contains('Santiago Garcia')]
df = df[~df.Name.str.contains('Santiago Rodriguez')]
df = df[~df.Name.str.contains('Santiago Simon')]
df = df[~df.Name.str.contains('Santiago Trellez')]
df = df[~df.Name.str.contains('Santos Borre')]
df = df[~df.Name.str.contains('Saul Salcedo')]
df = df[~df.Name.str.contains('Savio')]
df = df[~df.Name.str.contains('Sebastian Assis')]
df = df[~df.Name.str.contains('Sebastian Blanco')]
df = df[~df.Name.str.contains('Sebastian Dubarbier')]
df = df[~df.Name.str.contains('Sebastian Meza')]
df = df[~df.Name.str.contains('Sebastian Ramirez')]
df = df[~df.Name.str.contains('Sebastian Sosa')]
df = df[~df.Name.str.contains('Sebastien Thill')]
df = df[~df.Name.str.contains('Sega Coulibaly')]
df = df[~df.Name.str.contains('Sebastian Prödl')]
df = df[~df.Name.str.contains('Sergi Garcia')]
df = df[~df.Name.str.contains('Sergio Gonzalez')]
df = df[~df.Name.str.contains('Siro Rosane')]
df = df[~df.Name.str.contains('Suleyman Cebeci')]
df = df[~df.Name.str.contains('Sven Köhler')]
df = df[~df.Name.str.contains('Theo Leoni')]
df = df[~df.Name.str.contains('Thiago Andrade')]
df = df[~df.Name.str.contains('Thiago CoutoThiago Couto')]
df = df[~df.Name.str.contains('ThiaguinhoThiaguinho')]
df = df[~df.Name.str.contains('Thomas Toure')]
df = df[~df.Name.str.contains('Tibet Oniz')]
df = df[~df.Name.str.contains('Timothe Nkada')]
df = df[~df.Name.str.contains('Tobias Zarate')]
df = df[~df.Name.str.contains('Tomas Andrade')]
df = df[~df.Name.str.contains('Tomas Cardona')]
df = df[~df.Name.str.contains('Tomas Cavanagh')]
df = df[~df.Name.str.contains('Tomas Durso')]
df = df[~df.Name.str.contains('Tomas Escalante')]
df = df[~df.Name.str.contains('Tomas Fernandez')]
df = df[~df.Name.str.contains('Tomas Marchiori')]
df = df[~df.Name.str.contains('Tomas Martinez')]
df = df[~df.Name.str.contains('Tomas Ortiz')]
df = df[~df.Name.str.contains('Tomas Romero')]
df = df[~df.Name.str.contains('Tomas Sandoval')]
df = df[~df.Name.str.contains('Tomas Silva')]
df = df[~df.Name.str.contains('Torben Musel')]
df = df[~df.Name.str.contains('Uriel Ramirez')]
df = df[~df.Name.str.contains('Valentin Larralde')]
df = df[~df.Name.str.contains('Victor Camarasa')]
df = df[~df.Name.str.contains('Victor KawaVictor Kawaattac')]
df = df[~df.Name.str.contains('Vincent Muller')]
df = df[~df.Name.str.contains('ViniciusVinicius')]
df = df[~df.Name.str.contains('Vitor JacareVitor Jacare')]
df = df[~df.Name.str.contains('Vitor Oliveira')]
df = df[~df.Name.str.contains('WagnerWagner')]
df = df[~df.Name.str.contains('WalberWalber')]
df = df[~df.Name.str.contains('Walter Andrade')]
df = df[~df.Name.str.contains('Walter Kannemann')]
df = df[~df.Name.str.contains('WandsonWandson')]
df = df[~df.Name.str.contains('Welinton Junior')]
df = df[~df.Name.str.contains('Wellington NemWellington Nem')]
df = df[~df.Name.str.contains('Wellington TimWellington Tim')]
df = df[~df.Name.str.contains('YagoYago')]
df = df[~df.Name.str.contains('Yahya Soumare')]
df = df[~df.Name.str.contains('Yanis Lhery')]
df = df[~df.Name.str.contains('Yasin Oztekin')]
df = df[~df.Name.str.contains('Yeiler Goez')]
df = df[~df.Name.str.contains('Yildirim Mert Çetin')]
df = df[~df.Name.str.contains('Younes Belhanda')]
df = df[~df.Name.str.contains('Youssef Ait Bennasser')]
df = df[~df.Name.str.contains('Yuri Shafinskiy')]
df = df[~df.Name.str.contains('YuriYuri')]
df = df[~df.Name.str.contains('Yusuf Eren Göktas')]
df = df[~df.Name.str.contains('Ze GabrielZe Gabriel')]
df = df[~df.Name.str.contains('Ze IvaldoZe Ivaldo')]
df = df[~df.Name.str.contains('Ze Oliveira')]
df = df[~df.Name.str.contains('Angel Romero')]
df = df[~df.Name.str.contains('Buiu')]
df = df[~df.Name.str.contains('Dion Lopy')]
df = df[~df.Name.str.contains('Fedor Smolov')]
df = df[~df.Name.str.contains('Karadeniz')]
df = df[~df.Name.str.contains('Haji Mnoga')]
df = df[~df.Name.str.contains('Hakan Arslan')]
df = df[~df.Name.str.contains('Hakan Ozmert')]
df = df[~df.Name.str.contains('Hakan Yesil')]
df = df[~df.Name.str.contains('Hernani Infande')]
df = df[~df.Name.str.contains('Hugo Duro')]
df = df[~df.Name.str.contains('Igor Vinicius')]
df = df[~df.Name.str.contains('Ilaix Moriba')]
df = df[~df.Name.str.contains('Jacques Francois Moubandje')]
df = df[~df.Name.str.contains('Jaquite')]
df = df[~df.Name.str.contains('Javier Gandolfi')]
df = df[~df.Name.str.contains('Javier Mascherano')]
df = df[~df.Name.str.contains('Javier Mendez')]
df = df[~df.Name.str.contains('Javier Mendoza')]
df = df[~df.Name.str.contains('Jean Cleber')]
df = df[~df.Name.str.contains('Jean Eudes Aholou')]
df = df[~df.Name.str.contains('Jed Ward')]
df = df[~df.Name.str.contains('Jesus Vargas')]
df = df[~df.Name.str.contains('Joao Paulo da Silva')]
df = df[~df.Name.str.contains('Joao Teixeira')]
df = df[~df.Name.str.contains('Joaquin Munoz')]
df = df[~df.Name.str.contains('Jimmy Tabidze')]
df = df[~df.Name.str.contains('Jonathan dos Santos')]
df = df[~df.Name.str.contains('Vitali Balashov')]
df = df[~df.Name.str.contains('ngelo da Costa')]
df = df[~df.Name.str.contains('yuncu')]
df = df[~df.Name.str.contains('Lekue')]
df = df[~df.Name.str.contains('igo Martinez')]
df = df[~df.Name.str.contains('Thorvaldsson')]



prices = prices[~prices.name.str.contains('JoilsonJoilson')]
prices = prices[~prices.name.str.contains('Johan Mojica')]
prices = prices[~prices.name.str.contains('Joe WardJoe WardRight Midfiel')]
prices = prices[~prices.name.str.contains('Joaquin Mendive')]
prices = prices[~prices.name.str.contains('Joaquin Novillo')]
prices = prices[~prices.name.str.contains('Joaquin Hass')]
prices = prices[~prices.name.str.contains('Jean-Francois Gillet')]
prices = prices[~prices.name.str.contains('Javier Iritier')]
prices = prices[~prices.name.str.contains('Jaquite')]
prices = prices[~prices.name.str.contains('JaelJael')]
prices = prices[~prices.name.str.contains('Izzy Brow')]
prices = prices[~prices.name.str.contains('Jaakko Oksanen')]
prices = prices[~prices.name.str.contains('Strand Larsen')]
prices = prices[~prices.name.str.contains('Ivo GrbicIvo Grbic')]
prices = prices[~prices.name.str.contains('Ivan Angulo')]
prices = prices[~prices.name.str.contains('Ivan Arboleda')]
prices = prices[~prices.name.str.contains('Igor ViniciusIgor Vinicius')]
prices = prices[~prices.name.str.contains('Habib Diallo')]
prices = prices[~prices.name.str.contains('rkem Can')]
prices = prices[~prices.name.str.contains('Bakirbas')]
prices = prices[~prices.name.str.contains('khan Gul')]
prices = prices[~prices.name.str.contains('Ismael Sow')]
prices = prices[~prices.name.str.contains('Inigo Lekue')]
prices = prices[~prices.name.str.contains('Inigo Martinez')]
prices = prices[~prices.name.str.contains('Igor JuliaoIgor Juliao')]
prices = prices[~prices.name.str.contains('Igor CariusIgor Carius')]
prices = prices[~prices.name.str.contains('Inaki Pena')]
prices = prices[~prices.name.str.contains('Ilker Karakas')]
prices = prices[~prices.name.str.contains('Ignacio Miramon')]
prices = prices[~prices.name.str.contains('Ibrahim Sangare')]
prices = prices[~prices.name.str.contains('Ibragim Tsallagov')]
prices = prices[~prices.name.str.contains('Iago Herrerin')]
prices = prices[~prices.name.str.contains('HudsonHudson')]
prices = prices[~prices.name.str.contains('Hernan Lamberti')]
prices = prices[~prices.name.str.contains('Hernan Lopez')]
prices = prices[~prices.name.str.contains('HernaneHernane')]
prices = prices[~prices.name.str.contains('HernanesHernanes')]
prices = prices[~prices.name.str.contains('HelioHelio')]
prices = prices[~prices.name.str.contains('Hector Canteros')]
prices = prices[~prices.name.str.contains('HeberHeber')]
prices = prices[~prices.name.str.contains('Haji Mnoga')]
prices = prices[~prices.name.str.contains('Hakan Arslan')]
prices = prices[~prices.name.str.contains('Hakan Ozmert')]
prices = prices[~prices.name.str.contains('Hakan Yesil')]
prices = prices[~prices.name.str.contains('Guido Vadala')]
prices = prices[~prices.name.str.contains('Giangiacomo Magnani')]
prices = prices[~prices.name.str.contains('Georgi Makhatadze')]
prices = prices[~prices.name.str.contains('Gaston Torres')]
prices = prices[~prices.name.str.contains('Gaston Suso')]
prices = prices[~prices.name.str.contains('Gaston Gomez')]
prices = prices[~prices.name.str.contains('Gaston Gonzalez')]
prices = prices[~prices.name.str.contains('Gaetan Poussin')]
prices = prices[~prices.name.str.contains('Alfonso Espino')]
prices = prices[~prices.name.str.contains('Alfonso Pedraza')]
prices = prices[~prices.name.str.contains('AlêAlê')]
prices = prices[~prices.name.str.contains('Bjorn Engels')]
prices = prices[~prices.name.str.contains('Daniel Csoka')]
prices = prices[~prices.name.str.contains('Daniel Gazdag')]
prices = prices[~prices.name.str.contains('Daniel Gyollai')]
prices = prices[~prices.name.str.contains('Daniel Juarez')]
prices = prices[~prices.name.str.contains('Daniel Rios')]
prices = prices[~prices.name.str.contains('Daniel Williams')]
prices = prices[~prices.name.str.contains('Danieldoye')]
prices = prices[~prices.name.str.contains('Dario Sarmiento')]
prices = prices[~prices.name.str.contains('Darren Yapi')]
prices = prices[~prices.name.str.contains('David BrazDavid Braz')]
prices = prices[~prices.name.str.contains('David Buchholz')]
prices = prices[~prices.name.str.contains('David Da Costas')]
prices = prices[~prices.name.str.contains('Alan Cantero')]
prices = prices[~prices.name.str.contains('Aristoteles Romero')]
prices = prices[~prices.name.str.contains('Ben JonesBen Jones')]
prices = prices[~prices.name.str.contains('Davor Matijas')]
prices = prices[~prices.name.str.contains('Deian Veron')]
prices = prices[~prices.name.str.contains('Diego SouzaDiego Souza')]
prices = prices[~prices.name.str.contains('iego ÂngeloDiego Ângelo')]
prices = prices[~prices.name.str.contains('DiegoDiego')]
prices = prices[~prices.name.str.contains('Dino Visser')]
prices = prices[~prices.name.str.contains('Diogo BarbosaDiogo Barbosa')]
prices = prices[~prices.name.str.contains('Dixon Renteria')]
prices = prices[~prices.name.str.contains('Angelo Martino')]
prices = prices[~prices.name.str.contains('Ben Liddle')]
prices = prices[~prices.name.str.contains('Beni Makouana')]
prices = prices[~prices.name.str.contains('Bruno MendezBruno Mendez')]
prices = prices[~prices.name.str.contains('Buiu')]
prices = prices[~prices.name.str.contains('CauêCauê')]
prices = prices[~prices.name.str.contains('Dimitri Poloz')]
prices = prices[~prices.name.str.contains('Dimitri Rybchinskiy')]
prices = prices[~prices.name.str.contains('Dimitri Shomko')]
prices = prices[~prices.name.str.contains('Dimitri Skopintsev')]
prices = prices[~prices.name.str.contains('Dimitri Stotskiy')]
prices = prices[~prices.name.str.contains('Dimitri Sysuev')]
prices = prices[~prices.name.str.contains('Dimitri Tikhiy')]
prices = prices[~prices.name.str.contains('Dimitri Zhivoglyadov')]
prices = prices[~prices.name.str.contains('DodoDodo')]
prices = prices[~prices.name.str.contains('Dogan Alemdar')]
prices = prices[~prices.name.str.contains('Dogukan Ozkan')]
prices = prices[~prices.name.str.contains('Dom Telford')]
prices = prices[~prices.name.str.contains('Edson Montano')]
prices = prices[~prices.name.str.contains('Eduardo KauEduardo Kau')]
prices = prices[~prices.name.str.contains('Emanuel Insua')]
prices = prices[~prices.name.str.contains('Emiliano Gonzalez')]
prices = prices[~prices.name.str.contains('Emiliano Insua')]
prices = prices[~prices.name.str.contains('Enzo Fernandez')]
prices = prices[~prices.name.str.contains('Erik Duenas')]
prices = prices[~prices.name.str.contains('Esteban Rolon')]
prices = prices[~prices.name.str.contains('EvertonEverton')]
prices = prices[~prices.name.str.contains('Evgeni Kharin')]
prices = prices[~prices.name.str.contains('Ezequiel Fernandez')]
prices = prices[~prices.name.str.contains('Fabian Noguera')]
prices = prices[~prices.name.str.contains('Fabio ChinaFabio China')]
prices = prices[~prices.name.str.contains('Fabio SantosFabio Santos')]
prices = prices[~prices.name.str.contains('Fabio Vazquez')]
prices = prices[~prices.name.str.contains('Fabricio Dominguez')]
prices = prices[~prices.name.str.contains('Facundo Fernandez')]
prices = prices[~prices.name.str.contains('Facundo Garces')]
prices = prices[~prices.name.str.contains('Faiz Selemani')]
prices = prices[~prices.name.str.contains('Fedor Smolov')]
prices = prices[~prices.name.str.contains('Felipe Pena Biafore')]
prices = prices[~prices.name.str.contains('Fer NinoFer Nino')]
prices = prices[~prices.name.str.contains('Fernando Roman')]
prices = prices[~prices.name.str.contains('Filip Stankovic')]
prices = prices[~prices.name.str.contains('Filip Stuparevic')]
prices = prices[~prices.name.str.contains('Finn Ole Becker')]
prices = prices[~prices.name.str.contains('Florian Monzon')]
prices = prices[~prices.name.str.contains('Franck Bambock')]
prices = prices[~prices.name.str.contains('Francois Moubandje')]
prices = prices[~prices.name.str.contains('Gabriel Chapeco')]
prices = prices[~prices.name.str.contains('Gabriel Falcao')]
prices = prices[~prices.name.str.contains('Gabriel Gudino')]
prices = prices[~prices.name.str.contains('Giles Coke')]
prices = prices[~prices.name.str.contains('Gonzalo Alvez')]
prices = prices[~prices.name.str.contains('Gonzalo Asis')]
prices = prices[~prices.name.str.contains('Gonzalo Goni')]
prices = prices[~prices.name.str.contains('\*')]
prices = prices [~prices .name.str.contains('RegisRegis')]
prices = prices [~prices .name.str.contains('Jonas Aguirre')]
prices = prices [~prices .name.str.contains('Jonas Sterner')]
prices = prices [~prices .name.str.contains('Jonas Toro')]
prices = prices [~prices .name.str.contains('Jonatan Gomez')]
prices = prices [~prices .name.str.contains('Jonathan Zacaria')]
prices = prices [~prices .name.str.contains('Jony Alamo')]
prices = prices [~prices .name.str.contains('Jop van den Avert')]
prices = prices [~prices .name.str.contains('Jordi Osei-Tutu')]
prices = prices [~prices .name.str.contains('Jores Rahou')]
prices = prices [~prices .name.str.contains('Jorge Caceres')]
prices = prices [~prices .name.str.contains('Jorge Pereyra Diaz')]
prices = prices [~prices .name.str.contains('Jorge Saenz')]
prices = prices [~prices .name.str.contains('Jorgen Strand Larsen')]
prices = prices [~prices .name.str.contains('Jose Adolfo Valencia')]
prices = prices [~prices .name.str.contains('Jose Andres Martinez')]
prices = prices [~prices .name.str.contains('Jose AngelJose Angel')]
prices = prices [~prices .name.str.contains('Jose Cifuentes')]
prices = prices [~prices .name.str.contains('Jose Devecchi')]
prices = prices [~prices .name.str.contains('Jose Gimenez')]
prices = prices [~prices .name.str.contains('Jose Gonzalez')]
prices = prices [~prices .name.str.contains('Jose Luis Fernandez')]
prices = prices [~prices .name.str.contains('Jose Luis Gomez')]
prices = prices [~prices .name.str.contains('Jose Luis Rodriguez')]
prices = prices [~prices .name.str.contains('Jose Luis Sinisterra')]
prices = prices [~prices .name.str.contains('Jose Manuel Fontan')]
prices = prices [~prices .name.str.contains('Jose Manuel Lopez')]
prices = prices [~prices .name.str.contains('Jose Marcelo')]
prices = prices [~prices .name.str.contains('Jose Maria Gimenez')]
prices = prices [~prices .name.str.contains('Jose Martinez')]
prices = prices [~prices .name.str.contains('Jose Moya')]
prices = prices [~prices .name.str.contains('Josh JackJosh Jack')]
prices = prices [~prices .name.str.contains('Josh Neufville')]
prices = prices [~prices .name.str.contains('Josh Penn')]
prices = prices [~prices .name.str.contains('Josue Ayala')]
prices = prices [~prices .name.str.contains('Jovanny Bolivar')]
prices = prices [~prices .name.str.contains('Juan Alvarez')]
prices = prices [~prices .name.str.contains('Juan Cruz')]
prices = prices [~prices .name.str.contains('Juan Espinola')]
prices = prices [~prices .name.str.contains('Juan Gonzalez')]
prices = prices [~prices .name.str.contains('Juan Ignacio Mendez')]
prices = prices [~prices .name.str.contains('Juan Jose Caceres')]
prices = prices [~prices .name.str.contains('Juan Martin Lucero')]
prices = prices [~prices .name.str.contains('Juan Mendez')]
prices = prices [~prices .name.str.contains('Juan Pablo Alvarez')]
prices = prices [~prices .name.str.contains('Juan Pablo Ramirez')]
prices = prices [~prices .name.str.contains('Juan Roman Pucheta')]
prices = prices [~prices .name.str.contains('JuanJuan')]
prices = prices [~prices .name.str.contains('Jubril Okedina')]
prices = prices [~prices .name.str.contains('Julian Aude')]
prices = prices [~prices .name.str.contains('Julian Chicco')]
prices = prices [~prices .name.str.contains('Julian Lopez')]
prices = prices [~prices .name.str.contains('Julian Malatini')]
prices = prices [~prices .name.str.contains('Julian Marcioni')]
prices = prices [~prices .name.str.contains('Julian Navas')]
prices = prices [~prices .name.str.contains('Julian Velazquez')]
prices = prices [~prices .name.str.contains('Julio Gonzalez')]
prices = prices [~prices .name.str.contains('JuniorJunior')]
prices = prices [~prices .name.str.contains('KaduKadu')]
prices = prices [~prices .name.str.contains('Kaide Gordon')]
prices = prices [~prices .name.str.contains('Kaique RochaKaique Rocha')]
prices = prices [~prices .name.str.contains('KeillerKeiller')]
prices = prices [~prices .name.str.contains('KelvinKelvin')]
prices = prices [~prices .name.str.contains('Kemal Ruzgar')]
prices = prices [~prices .name.str.contains('KenoKeno')]
prices = prices [~prices .name.str.contains('KenoKeno')]
prices = prices [~prices .name.str.contains('Kerim Calhanoglu')]
prices = prices [~prices .name.str.contains('Kevin Boma')]
prices = prices [~prices .name.str.contains('Kevin Zenon')]
prices = prices [~prices .name.str.contains('Kian Slor')]
prices = prices [~prices .name.str.contains('Koffi')]
prices = prices [~prices .name.str.contains('Konstantin Kovalev')]
prices = prices [~prices .name.str.contains('Konstantin Rausch')]
prices = prices [~prices .name.str.contains('Kornell MacDonald')]
prices = prices [~prices .name.str.contains('Kortne Ford')]
prices = prices [~prices .name.str.contains('Kostas Stafylidis')]
prices = prices [~prices .name.str.contains('Laercio Solda')]
prices = prices [~prices .name.str.contains('LaercioLaercio')]
prices = prices [~prices .name.str.contains('Lars MaiLars Mai')]
prices = prices [~prices .name.str.contains('Lasse Sorensen')]
prices = prices [~prices .name.str.contains('Laureano Punet')]
prices = prices [~prices .name.str.contains('Lautaro Chavez')]
prices = prices [~prices .name.str.contains('Lautaro de Leon')]
prices = prices [~prices .name.str.contains('LazaroLazaro')]
prices = prices [~prices .name.str.contains('Leandro Contin')]
prices = prices [~prices .name.str.contains('Leandro Fernandez')]
prices = prices [~prices .name.str.contains('Lennart Schulze')]
prices = prices [~prices .name.str.contains('Lee O\'Connor')]
prices = prices [~prices .name.str.contains('Leo BorgesLeo Borges')]
prices = prices [~prices .name.str.contains('Leo Gomes')]
prices = prices [~prices .name.str.contains('Leo NatelLeo Natel')]
prices = prices [~prices .name.str.contains('Leonel Gonzalez')]
prices = prices [~prices .name.str.contains('Levi Ntumba')]
prices = prices [~prices .name.str.contains('Lois Diony')]
prices = prices [~prices .name.str.contains('Luca Martinez Dupuy')]
prices = prices [~prices .name.str.contains('Ludvig Ohman')]
prices = prices [~prices .name.str.contains('Luis Leal')]
prices = prices [~prices .name.str.contains('Luis Miguel Rodriguez')]
prices = prices [~prices .name.str.contains('Luis Rodriguez')]
prices = prices [~prices .name.str.contains('Luis Vazquez')]
prices = prices [~prices .name.str.contains('Luiz Fernando')]
prices = prices [~prices .name.str.contains('Luiz JuniorLuiz Junior')]
prices = prices [~prices .name.str.contains('Luiz Lucio Reis Junior')]
prices = prices [~prices .name.str.contains('Luiz Otavio')]
prices = prices [~prices .name.str.contains('Luka Zarandia')]
prices = prices [~prices .name.str.contains('MaiconMaicon')]
prices = prices [~prices .name.str.contains('Malik Sellouki')]
prices = prices [~prices .name.str.contains('Malvind Benning')]
prices = prices [~prices .name.str.contains('Mamadi Camara')]
prices = prices [~prices .name.str.contains('Mamadou Coulibaly')]
prices = prices [~prices .name.str.contains('Marc Auge')]
prices = prices [~prices .name.str.contains('Marc Baro')]
prices = prices [~prices .name.str.contains('Marc-Olivier Doue')]
prices = prices [~prices .name.str.contains('Marcelinho')]
prices = prices [~prices .name.str.contains('Marcio AzevedoMarcio Azevedo')]
prices = prices [~prices .name.str.contains('Marcio Mossoro')]
prices = prices [~prices .name.str.contains('Marcos AndreMarcos Andre')]
prices = prices [~prices .name.str.contains('Marcos Sanchez')]
prices = prices [~prices .name.str.contains('Mariano Diaz')]
prices = prices [~prices .name.str.contains('Mariano Vazquez')]
prices = prices [~prices .name.str.contains('Mark Kosznovszky')]
prices = prices [~prices .name.str.contains('Markus Fjortoft')]
prices = prices [~prices .name.str.contains('Marley Ake')]
prices = prices [~prices .name.str.contains('Martin Campana')]
prices = prices [~prices .name.str.contains('Martin Garay')]
prices = prices [~prices .name.str.contains('Martin Garcia')]
prices = prices [~prices .name.str.contains('Martin Rabunal')]
prices = prices [~prices .name.str.contains('Mateus BrunMateus Brun')]
prices = prices [~prices .name.str.contains('Matheo Vroman')]
prices = prices [~prices .name.str.contains('Mathias Abero')]
prices = prices [~prices .name.str.contains('Mathias Kjolo')]
prices = prices [~prices .name.str.contains('Mathias Suarez')]
prices = prices [~prices .name.str.contains('Matias Belloso')]
prices = prices [~prices .name.str.contains('Matias Borgogno')]
prices = prices [~prices .name.str.contains('Matias Cordoba')]
prices = prices [~prices .name.str.contains('Matias Escudero')]
prices = prices [~prices .name.str.contains('Matias Fritzler')]
prices = prices [~prices .name.str.contains('Matias Garcia')]
prices = prices [~prices .name.str.contains('Matias Gomez')]
prices = prices [~prices .name.str.contains('Matias Gonzalez')]
prices = prices [~prices .name.str.contains('Matias Laba')]
prices = prices [~prices .name.str.contains('Matias Moya')]
prices = prices [~prices .name.str.contains('Matias Palavecino')]
prices = prices [~prices .name.str.contains('Matias Pellegrini')]
prices = prices [~prices .name.str.contains('Matias Pisano')]
prices = prices [~prices .name.str.contains('Matias Ramirez')]
prices = prices [~prices .name.str.contains('Matias Rodriguez')]
prices = prices [~prices .name.str.contains('Matias Sosa')]
prices = prices [~prices .name.str.contains('Matias Tagliamonte')]
prices = prices [~prices .name.str.contains('Matias Tissera')]
prices = prices [~prices .name.str.contains('Matias Villarreal')]
prices = prices [~prices .name.str.contains('Mauro Ortiz')]
prices = prices [~prices .name.str.contains('Max GrunMax Grun')]
prices = prices [~prices .name.str.contains('Max WalefMax Walef')]
prices = prices [~prices .name.str.contains('Meme Placca Fessou')]
prices = prices [~prices .name.str.contains('Mert Gunok')]
prices = prices [~prices .name.str.contains('Metin Yildirim')]
prices = prices [~prices .name.str.contains('MetinhoMetinho')]
prices = prices [~prices .name.str.contains('Michael Chacon')]
prices = prices [~prices .name.str.contains('Michel Araujo')]
prices = prices [~prices .name.str.contains('Michel Herrero')]
prices = prices [~prices .name.str.contains('Michel Macedo')]
prices = prices [~prices .name.str.contains('MichelMichel')]
prices = prices [~prices .name.str.contains('Miguel Angel Atienza')]
prices = prices [~prices .name.str.contains('Miguel Angel Guerrero')]
prices = prices [~prices .name.str.contains('Miguel Angel Moya')]
prices = prices [~prices .name.str.contains('Miguel Angel Rubio')]
prices = prices [~prices .name.str.contains('Mike GrimMike Grim')]
prices = prices [~prices .name.str.contains('Milton Alvarez')]
prices = prices [~prices .name.str.contains('Milton Gimenez')]
prices = prices [~prices .name.str.contains('Miquel Scarlett')]
prices = prices [~prices .name.str.contains('Mohamed Traore')]
prices = prices [~prices .name.str.contains('Moises Caicedo')]
prices = prices [~prices .name.str.contains('Moon-hwan Kim')]
prices = prices [~prices .name.str.contains('MoraesMoraes')]
prices = prices [~prices .name.str.contains('Morato')]
prices = prices [~prices .name.str.contains('Moussa Wague')]
prices = prices [~prices .name.str.contains('Murat Akca')]
prices = prices [~prices .name.str.contains('Murat Aksit')]
prices = prices [~prices .name.str.contains('Murat Sipahioglu')]
prices = prices [~prices .name.str.contains('MurielMuriel')]
prices = prices [~prices .name.str.contains('Murilo RangelMurilo Rangel')]
prices = prices [~prices .name.str.contains('Mustafa Cecenoglu')]
prices = prices [~prices .name.str.contains('Mustafa Huseyin Seyhan')]
prices = prices [~prices .name.str.contains('NatanaelNatanael')]
prices = prices [~prices .name.str.contains('NatanNatan')]
prices = prices [~prices .name.str.contains('Nathan-Dylan Saliba')]
prices = prices [~prices .name.str.contains('Nathangelo Markelo')]
prices = prices [~prices .name.str.contains('Nazareno Solis')]
prices = prices [~prices .name.str.contains('NeiltonNeilton')]
prices = prices [~prices .name.str.contains('Nelson Ibanez')]
prices = prices [~prices .name.str.contains('Nemanja Antonov')]
prices = prices [~prices .name.str.contains('Nestor Ortigoza')]
prices = prices [~prices .name.str.contains('Nicolas Andereggen')]
prices = prices [~prices .name.str.contains('Nicolas Avellaneda')]
prices = prices [~prices .name.str.contains('Nicolas Blandi')]
prices = prices [~prices .name.str.contains('Nicolas Burgy')]
prices = prices [~prices .name.str.contains('Nicolas Contin')]
prices = prices [~prices .name.str.contains('Nicolas Correa')]
prices = prices [~prices .name.str.contains('Nicolas Domingo')]
prices = prices [~prices .name.str.contains('Nicolas Femia')]
prices = prices [~prices .name.str.contains('Nicolas Fernandez Mercau')]
prices = prices [~prices .name.str.contains('Nicolas Ferreyra')]
prices = prices [~prices .name.str.contains('Nicolas Figal')]
prices = prices [~prices .name.str.contains('Nicolas Gimenez')]
prices = prices [~prices .name.str.contains('Nicolas Lamendola')]
prices = prices [~prices .name.str.contains('Nicolas Linares')]
prices = prices [~prices .name.str.contains('Nicolas Mana')]
prices = prices [~prices .name.str.contains('Nicolas Mazzola')]
prices = prices [~prices .name.str.contains('Nicolas Messiniti')]
prices = prices [~prices .name.str.contains('Nicolas Morgantini')]
prices = prices [~prices .name.str.contains('Nicolas Olsak')]
prices = prices [~prices .name.str.contains('Nicolas Penailillo')]
prices = prices [~prices .name.str.contains('Nicolas Romat')]
prices = prices [~prices .name.str.contains('Nicolas Romero')]
prices = prices [~prices .name.str.contains('Nicolas Schiappacasse')]
prices = prices [~prices .name.str.contains('Nicolas Thaller')]
prices = prices [~prices .name.str.contains('Nicolas Zalazar')]
prices = prices [~prices .name.str.contains('Nicolo Zaniolo')]
prices = prices [~prices .name.str.contains('Nikola Pejčić')]
prices = prices [~prices .name.str.contains('Nino ParaibaNino Paraiba')]
prices = prices [~prices .name.str.contains('Omer Beyaz')]
prices = prices [~prices .name.str.contains('Omer Sismanoglu')]
prices = prices [~prices .name.str.contains('Oscar Benitez')]
prices = prices [~prices .name.str.contains('Oscar Linner')]
prices = prices [~prices .name.str.contains('Oscar Schönfelder')]
prices = prices [~prices .name.str.contains('OsmanOsman')]
prices = prices [~prices .name.str.contains('OsvaldoOsvaldo')]
prices = prices [~prices .name.str.contains('Pablo Velazquez')]
prices = prices [~prices .name.str.contains('PaulaoPaulao')]
prices = prices [~prices .name.str.contains('Paulinho BoiaPaulinho Boia')]
prices = prices [~prices .name.str.contains('Pedro Goncalves')]
prices = prices [~prices .name.str.contains('Pepê')]
prices = prices [~prices .name.str.contains('Pepe Bonet')]
prices = prices [~prices .name.str.contains('Philip Turpitz')]
prices = prices [~prices .name.str.contains('Piero Hincapie')]
prices = prices [~prices .name.str.contains('Piris da MottaPiris da Motta')]
prices = prices [~prices .name.str.contains('Pol Garcia')]
prices = prices [~prices .name.str.contains('Pote')]
prices = prices [~prices .name.str.contains('Rafael Borre')]
prices = prices [~prices .name.str.contains('Rafael Garcia')]
prices = prices [~prices .name.str.contains('Rahman Bugra Cagiran')]
prices = prices [~prices .name.str.contains('Ramon Arias')]
prices = prices [~prices .name.str.contains('Ramon Azeez')]
prices = prices [~prices .name.str.contains('Ramon Folch')]
prices = prices [~prices .name.str.contains('Ramon ten Hove')]
prices = prices [~prices .name.str.contains('Rashad Muhammed')]
prices = prices [~prices .name.str.contains('Rashid')]
prices = prices [~prices .name.str.contains('Raul Carnero')]
prices = prices [~prices .name.str.contains('Raul Lozano')]
prices = prices [~prices .name.str.contains('Raul Navas')]
prices = prices [~prices .name.str.contains('Raul Silva')]
prices = prices [~prices .name.str.contains('Raul SilvaRaul Silva')]
prices = prices [~prices .name.str.contains('Remeao Hutton')]
prices = prices [~prices .name.str.contains('RenêRenê')]
prices = prices [~prices .name.str.contains('Rhys Fenlon')]
prices = prices [~prices .name.str.contains('Rhys-James Fenlon')]
prices = prices [~prices .name.str.contains('RibamarRibamar')]
prices = prices [~prices .name.str.contains('Ricard Sanchez')]
prices = prices [~prices .name.str.contains('Ricardo Gomes')]
prices = prices [~prices .name.str.contains('Ricardo SilvaRicardo Silva')]
prices = prices [~prices .name.str.contains('RildoRildo')]
prices = prices [~prices .name.str.contains('Roberto Jimenez')]
prices = prices [~prices .name.str.contains('Rodri Sanchez')]
prices = prices [~prices .name.str.contains('Rodrigo Sanchez')]
prices = prices [~prices .name.str.contains('Rodrigues')]
prices = prices [~prices .name.str.contains('Rogerio Santos')]
prices = prices [~prices .name.str.contains('Roger IbanezRoger Ibanez')]
prices = prices [~prices .name.str.contains('Rolando Garcia')]
prices = prices [~prices .name.str.contains('Roman Martinez')]
prices = prices [~prices .name.str.contains('Romulo Otero')]
prices = prices [~prices .name.str.contains('Ruben Botta')]
prices = prices [~prices .name.str.contains('Ruben Fonseca')]
prices = prices [~prices .name.str.contains('Ruben Pardo')]
prices = prices [~prices .name.str.contains('Rui EncarnacaoRui Encarnacao')]
prices = prices [~prices .name.str.contains('Ruwen Werthmuller')]
prices = prices [~prices .name.str.contains('Sören Dieckmann')]
prices = prices [~prices .name.str.contains('Saîf-Eddine Khaoui')]
prices = prices [~prices .name.str.contains('Said Bakari')]
prices = prices [~prices .name.str.contains('Saidou Sow')]
prices = prices [~prices .name.str.contains('Saliou Sane')]
prices = prices [~prices .name.str.contains('Sandro PerpetuoSandro Perpetuo')]
prices = prices [~prices .name.str.contains('SandrySandry')]
prices = prices [~prices .name.str.contains('Santiago Brinone')]
prices = prices [~prices .name.str.contains('Santiago Caseres')]
prices = prices [~prices .name.str.contains('Santiago Garcia')]
prices = prices [~prices .name.str.contains('Santiago Rodriguez')]
prices = prices [~prices .name.str.contains('Santiago Simon')]
prices = prices [~prices .name.str.contains('Santiago Trellez')]
prices = prices [~prices .name.str.contains('Santos Borre')]
prices = prices [~prices .name.str.contains('Saul Salcedo')]
prices = prices [~prices .name.str.contains('Savio')]
prices = prices [~prices .name.str.contains('Sebastian Assis')]
prices = prices [~prices .name.str.contains('Sebastian Blanco')]
prices = prices [~prices .name.str.contains('Sebastian Dubarbier')]
prices = prices [~prices .name.str.contains('Sebastian Meza')]
prices = prices [~prices .name.str.contains('Sebastian Ramirez')]
prices = prices [~prices .name.str.contains('Sebastian Sosa')]
prices = prices [~prices .name.str.contains('Sebastien Thill')]
prices = prices [~prices .name.str.contains('Sega Coulibaly')]
prices = prices [~prices .name.str.contains('Sebastian Prödl')]
prices = prices [~prices .name.str.contains('Sergi Garcia')]
prices = prices [~prices .name.str.contains('Sergio Gonzalez')]
prices = prices [~prices .name.str.contains('Siro Rosane')]
prices = prices [~prices .name.str.contains('Suleyman Cebeci')]
prices = prices [~prices .name.str.contains('Sven Köhler')]
prices = prices [~prices .name.str.contains('Theo Leoni')]
prices = prices [~prices .name.str.contains('Thiago Andrade')]
prices = prices [~prices .name.str.contains('Thiago CoutoThiago Couto')]
prices = prices [~prices .name.str.contains('ThiaguinhoThiaguinho')]
prices = prices [~prices .name.str.contains('Thomas Toure')]
prices = prices [~prices .name.str.contains('Tibet Oniz')]
prices = prices [~prices .name.str.contains('Timothe Nkada')]
prices = prices [~prices .name.str.contains('Tobias Zarate')]
prices = prices [~prices .name.str.contains('Tomas Andrade')]
prices = prices [~prices .name.str.contains('Tomas Cardona')]
prices = prices [~prices .name.str.contains('Tomas Cavanagh')]
prices = prices [~prices .name.str.contains('Tomas Durso')]
prices = prices [~prices .name.str.contains('Tomas Escalante')]
prices = prices [~prices .name.str.contains('Tomas Fernandez')]
prices = prices [~prices .name.str.contains('Tomas Marchiori')]
prices = prices [~prices .name.str.contains('Tomas Martinez')]
prices = prices [~prices .name.str.contains('Tomas Ortiz')]
prices = prices [~prices .name.str.contains('Tomas Romero')]
prices = prices [~prices .name.str.contains('Tomas Sandoval')]
prices = prices [~prices .name.str.contains('Tomas Silva')]
prices = prices [~prices .name.str.contains('Torben Musel')]
prices = prices [~prices .name.str.contains('Uriel Ramirez')]
prices = prices [~prices .name.str.contains('Valentin Larralde')]
prices = prices [~prices .name.str.contains('Victor Camarasa')]
prices = prices [~prices .name.str.contains('Victor KawaVictor Kawaattac')]
prices = prices [~prices .name.str.contains('Vincent Muller')]
prices = prices [~prices .name.str.contains('ViniciusVinicius')]
prices = prices [~prices .name.str.contains('Vitor JacareVitor Jacare')]
prices = prices [~prices .name.str.contains('Vitor Oliveira')]
prices = prices [~prices .name.str.contains('WagnerWagner')]
prices = prices [~prices .name.str.contains('WalberWalber')]
prices = prices [~prices .name.str.contains('Walter Andrade')]
prices = prices [~prices .name.str.contains('Walter Kannemann')]
prices = prices [~prices .name.str.contains('WandsonWandson')]
prices = prices [~prices .name.str.contains('Welinton Junior')]
prices = prices [~prices .name.str.contains('Wellington NemWellington Nem')]
prices = prices [~prices .name.str.contains('Wellington TimWellington Tim')]
prices = prices [~prices .name.str.contains('YagoYago')]
prices = prices [~prices .name.str.contains('Yahya Soumare')]
prices = prices [~prices .name.str.contains('Yanis Lhery')]
prices = prices [~prices .name.str.contains('Yasin Oztekin')]
prices = prices [~prices .name.str.contains('Yeiler Goez')]
prices = prices [~prices .name.str.contains('Yildirim Mert Çetin')]
prices = prices [~prices .name.str.contains('Younes Belhanda')]
prices = prices [~prices .name.str.contains('Youssef Ait Bennasser')]
prices = prices [~prices .name.str.contains('Yuri Shafinskiy')]
prices = prices [~prices .name.str.contains('YuriYuri')]
prices = prices [~prices .name.str.contains('Yusuf Eren Göktas')]
prices = prices [~prices .name.str.contains('Ze GabrielZe Gabriel')]
prices = prices [~prices .name.str.contains('Ze IvaldoZe Ivaldo')]
prices = prices [~prices .name.str.contains('Ze Oliveira')]
prices = prices [~prices .name.str.contains('Dimitri German')]
prices = prices [~prices .name.str.contains('Dimitri Chistyakov')]
prices = prices [~prices .name.str.contains('Dimitri Khomich')]
prices = prices [~prices .name.str.contains('Dimitri Poloz')]
prices = prices [~prices .name.str.contains('Dimitri Rybchinskiy')]
prices = prices [~prices .name.str.contains('Dimitri Shomko')]
prices = prices [~prices .name.str.contains('Dimitri Skopintsev')]
prices = prices [~prices .name.str.contains('Dimitri Stotskiy')]
prices = prices [~prices .name.str.contains('Dimitri Sysuev')]
prices = prices [~prices .name.str.contains('Dimitri Tikhiy')]
prices = prices [~prices .name.str.contains('Dimitri Zhivoglyadov')]
prices = prices [~prices .name.str.contains('Altiparmak')]
prices = prices [~prices .name.str.contains('Bayrakdar')]
prices = prices [~prices .name.str.contains('Kardes')]
prices = prices [~prices .name.str.contains('Georgios Vagiannidis')]
prices = prices [~prices .name.str.contains('Georgiy Dzhikiya')]
prices = prices [~prices .name.str.contains('Javi Lopez')]
prices = prices [~prices .name.str.contains('Jed SteerJed Steer')]
prices = prices [~prices .name.str.contains('Jeffrey Bruma')]
prices = prices [~prices .name.str.contains('Jeizon Ramirez')]
prices = prices [~prices .name.str.contains('Jens J')]
prices = prices [~prices .name.str.contains('Jesus Ferreira')]
prices = prices [~prices .name.str.contains('Joao Carlos Teixeira')]
prices = prices [~prices .name.str.contains('Joao CarlosJoao Carlos')]
prices = prices [~prices .name.str.contains('Joao GomesJoao Gomes')]
prices = prices [~prices .name.str.contains('Joao IgorJoao Igor')]
prices = prices [~prices .name.str.contains('Joao Jaquite')]
prices = prices [~prices .name.str.contains('Joao Klauss')]
prices = prices [~prices .name.str.contains('Joao LopesJoao Lopes')]
prices = prices [~prices .name.str.contains('Joao MarioJoao MarioRight Midfiel')]
prices = prices [~prices .name.str.contains('Joao Monteiro')]
prices = prices [~prices .name.str.contains('Yaya SitholeYaya Sithole')]





df['Name'] = df['Name'].str.replace('Georgy','Georgi')
df['Name'] = df['Name'].str.replace('Dimitrios','Dimitri')
df['Name'] = df['Name'].str.replace('Dimitrij','Dimitri')
df['Name'] = df['Name'].str.replace('Max-Alain Gradel','Max Gradel')
df['Name'] = df['Name'].str.replace('Vitaly','Vitali')
df['Name'] = df['Name'].str.replace('Vitaliy','Vitali')




prices['name'] = prices['name'].str.replace('Dmitriy','Dimitri ')
prices['name'] = prices['name'].str.replace('Evgeniy','Evgeni')
prices['name'] = prices['name'].str.replace('Fede Valverde','Federico Valverde')
prices['name'] = prices['name'].str.replace('Georgy','Georgi')
prices['name'] = prices['name'].str.replace('Konstantinos','Kostas')
prices['name'] = prices['name'].str.replace('Daniel\'Lundulu','Daniel N\'Lundulu')
prices['name'] = prices['name'].str.replace('Dimitrios','Dimitri')
prices['name'] = prices['name'].str.replace('Dimitris','Dimitri')
prices['name'] = prices['name'].str.replace('GilGil','Gil')
prices['name'] = prices['name'].str.replace('IgorIgor','Igor')
prices['name'] = prices['name'].str.replace('RiccieliRiccieli','Riccieli')
prices['name'] = prices['name'].str.replace('RiccieliRiccieli','Riccieli')
prices['name'] = prices['name'].str.replace('RiccieliRiccieli','Riccieli')
prices['name'] = prices['name'].str.replace('Erkan KasErkan Kas','Erkan Kas')
prices['name'] = prices['name'].str.replace('Bruno RamiresBruno Ramires','Bruno Ramires')
prices['name'] = prices['name'].str.replace('Juan CruzJuan Cruz','Juan Cruz')
prices['name'] = prices['name'].str.replace('Leo','Leo')
prices['name'] = prices['name'].str.replace('Mahamadou Doucoure','Mamadou Doucoure')
prices['name'] = prices['name'].str.replace('MoisesMoises','Moises')
prices['name'] = prices['name'].str.replace('MunirMunir','Munir')
prices['name'] = prices['name'].str.replace('NatanNatan','Natan')
prices['name'] = prices['name'].str.replace('NatanNatan','Natan')
prices['name'] = prices['name'].str.replace('Nico Gaitan','Nicolas Gaitan')
prices['name'] = prices['name'].str.replace('ThiagoThiago','Thiago')
prices['name'] = prices['name'].str.replace('Tom RogicTom Rogic','Tom Rogic')
prices['name'] = prices['name'].str.replace('WendellWendell','Wendell')
prices['name'] = prices['name'].str.replace('WendelWendel','Wendel')
prices['name'] = prices['name'].str.replace('WillianWillian','Willian')
prices['name'] = prices['name'].str.replace('Jason Dimitriou','Jason Demetriou')
prices['name'] = prices['name'].str.replace('Javi Ontiveros','Javier Ontiveros')
prices['name'] = prices['name'].str.replace('Jean Manuel Mbom','Jean/Manuel Mbom')
prices['name'] = prices['name'].str.replace('JeanJean','Jean')
prices['name'] = prices['name'].str.replace('JeffersonJefferson','Jefferson')
prices['name'] = prices['name'].str.replace('Jesus Murillo','Jesus David Murillo')
prices['name'] = prices['name'].str.replace('JoJo','Jo')
prices['name'] = prices['name'].str.replace('JonyJony','Jony')
prices['name'] = prices['name'].str.replace('JordiJordi','Jordi')
prices['name'] = prices['name'].str.replace('Jorge FelixJorge Felix','Jorge Felix')
prices['name'] = prices['name'].str.replace('JosanJosan','Josan')
prices['name'] = prices['name'].str.replace('Jose WelisonJose Welison','Jose Welison')
prices['name'] = prices['name'].str.replace('KokeKoke','Koke')
prices['name'] = prices['name'].str.replace('Ângelo GabrielÂngelo Gabriel','Ângelo Gabriel')
prices['name'] = prices['name'].str.replace('Vinicius JuniorVinicius Junior','Vinicius Junior')
prices['name'] = prices['name'].str.replace('Vinicius MachadoVinicius Machado','Vinicius Machado')
prices['name'] = prices['name'].str.replace('WescleyWescley','Wescley')
prices['name'] = prices['name'].str.replace('WelintonWelinton','Welinton')
prices['name'] = prices['name'].str.replace('WellingtonWellington','Wellington')
prices['name'] = prices['name'].str.replace('WendsonWendsonattac','Wendson')
prices['name'] = prices['name'].str.replace('WevertonWeverton','Weverton')

In [429]:
prices = prices[~prices.name.str.contains('ÃÃ±igo Lekue')]
prices = prices[~prices.name.str.contains('ÃÃ±igo Martinez')]
prices = prices[~prices.name.str.contains('Adam Phillips')]
prices = prices[~prices.name.str.contains('Adam Randell')]
prices = prices[~prices.name.str.contains('Ahmet Calik')]
prices = prices[~prices.name.str.contains('Ahmet Canbaz')]
prices = prices[~prices.name.str.contains('Alan Carius')]
prices = prices[~prices.name.str.contains('Alejandro Cabrera')]
prices = prices[~prices.name.str.contains('Alejandro Cantero')]
prices = prices[~prices.name.str.contains('Aleksandr Golovnya')]
prices = prices[~prices.name.str.contains('Aleksandr Lomovitskiy')]
prices = prices[~prices.name.str.contains('Aleksandr Smirnov')]
prices = prices[~prices.name.str.contains('Aleksandr Sobolev')]
prices = prices[~prices.name.str.contains('Ãlex Blesa')]
prices = prices[~prices.name.str.contains('Alfie McCalmont')]
prices = prices[~prices.name.str.contains('Alfie Whiteman')]
prices = prices[~prices.name.str.contains('Alireza Beiranvand')]
prices = prices[~prices.name.str.contains('Alireza Jahanbakhsh')]
prices = prices[~prices.name.str.contains('Allan-Romeo Nyom')]
prices = prices[~prices.name.str.contains('Alonzo Engwanda')]
prices = prices[~prices.name.str.contains('Ãlvaro Bastida')]
prices = prices[~prices.name.str.contains('Ãlvaro Bravo')]
prices = prices[~prices.name.str.contains('Ãlvaro Gimenez')]
prices = prices[~prices.name.str.contains('Ãlvaro Quezada')]
prices = prices[~prices.name.str.contains('Ãlvaro Tejero')]
prices = prices[~prices.name.str.contains('Alvas Powell')]
prices = prices[~prices.name.str.contains('Alvi Adilkhanov')]
prices = prices[~prices.name.str.contains('Amir Abrashi')]
prices = prices[~prices.name.str.contains('Anders Trondsen')]
prices = prices[~prices.name.str.contains('Ãngelo Martino')]
prices = prices[~prices.name.str.contains('Ãngelo Preciado')]
prices = prices[~prices.name.str.contains('Ãngelo Sagal')]
prices = prices[~prices.name.str.contains('Anthony Ujah')]
prices = prices[~prices.name.str.contains('Anthony Uzodimma')]
prices = prices[~prices.name.str.contains('Anton Mitryushkin')]
prices = prices[~prices.name.str.contains('Anton Piskunov')]
prices = prices[~prices.name.str.contains('Antonio Raimondo')]
prices = prices[~prices.name.str.contains('Antonio Rosati')]
prices = prices[~prices.name.str.contains('Archie Meekison')]
prices = prices[~prices.name.str.contains('Arda Kizildag')]
prices = prices[~prices.name.str.contains('Aristote Nsiala')]
prices = prices[~prices.name.str.contains('Aristoteles Romero')]
prices = prices[~prices.name.str.contains('Ãsak Thorvaldsson')]
prices = prices[~prices.name.str.contains('Ayaz Guliev')]
prices = prices[~prices.name.str.contains('Aykut Ozer')]
prices = prices[~prices.name.str.contains('Batuhan Ãœnsal')]
prices = prices[~prices.name.str.contains('Batuhan Unsal')]
prices = prices[~prices.name.str.contains('Beckham Sunderland')]
prices = prices[~prices.name.str.contains('Beka Mikeltadze')]
prices = prices[~prices.name.str.contains('Ben Jackson')]
prices = prices[~prices.name.str.contains('Beni Makouana')]
prices = prices[~prices.name.str.contains('Berk Cetin')]
prices = prices[~prices.name.str.contains('Berk Yildiz')]
prices = prices[~prices.name.str.contains('Bernardo ADDDDDDDDDDD')]
prices = prices[~prices.name.str.contains('Braian Aguirre')]
prices = prices[~prices.name.str.contains('Braian Ãlvarez')]
prices = prices[~prices.name.str.contains('Braian Cufre')]
prices = prices[~prices.name.str.contains('Braian Sanchez')]
prices = prices[~prices.name.str.contains('BrayanBrayan')]
prices = prices[~prices.name.str.contains('Braydon Manu')]
prices = prices[~prices.name.str.contains('Bret Halsey')]
prices = prices[~prices.name.str.contains('Brett McGavin')]
prices = prices[~prices.name.str.contains('Brian Idowu')]
prices = prices[~prices.name.str.contains('Brian Lluy')]
prices = prices[~prices.name.str.contains('Brian Negro')]
prices = prices[~prices.name.str.contains('Brian Nievas')]
prices = prices[~prices.name.str.contains('BruninhoBruninho')]
prices = prices[~prices.name.str.contains('Bruno Andrade')]
prices = prices[~prices.name.str.contains('Bruno MendezBruno Mendez')]
prices = prices[~prices.name.str.contains('Bruno Miguel Rodrigues')]
prices = prices[~prices.name.str.contains('Bryce Hosannah')]
prices = prices[~prices.name.str.contains('Bryce Washington')]
prices = prices[~prices.name.str.contains('BuenoBueno')]
prices = prices[~prices.name.str.contains('Buiu')]
prices = prices[~prices.name.str.contains('CaduCadu*')]
prices = prices[~prices.name.str.contains('CafuCafu')]
prices = prices[~prices.name.str.contains('Cain Fara')]
prices = prices[~prices.name.str.contains('Callum Jones')]
prices = prices[~prices.name.str.contains('Callum King-Harmes')]
prices = prices[~prices.name.str.contains('Camacho')]
prices = prices[~prices.name.str.contains('Cameron Antwi')]
prices = prices[~prices.name.str.contains('Cammy Ballantyne')]
prices = prices[~prices.name.str.contains('Cammy Smith')]
prices = prices[~prices.name.str.contains('Cenk Sahin')]
prices = prices[~prices.name.str.contains('Cesar IbaÃ±ez')]
prices = prices[~prices.name.str.contains('Charalampos Makridis')]
prices = prices[~prices.name.str.contains('Charlie Owens')]
prices = prices[~prices.name.str.contains('Charly Jan')]
prices = prices[~prices.name.str.contains('Chase Gasper')]
prices = prices[~prices.name.str.contains('Christopher Garcia')]
prices = prices[~prices.name.str.contains('Christopher Gloster')]
prices = prices[~prices.name.str.contains('Clint Irwin')]
prices = prices[~prices.name.str.contains('Clint Leemans')]
prices = prices[~prices.name.str.contains('Connor Shanks')]
prices = prices[~prices.name.str.contains('Connor Taylor')]
prices = prices[~prices.name.str.contains('Couhaib Driouech')]
prices = prices[~prices.name.str.contains('Courtney Baker-Richardson')]
prices = prices[~prices.name.str.contains('Damian Perez')]
prices = prices[~prices.name.str.contains('Damian Schmidt')]
prices = prices[~prices.name.str.contains('Daniel Kemp')]
prices = prices[~prices.name.str.contains('Daniel Leadbitter')]
prices = prices[~prices.name.str.contains('Daniel Vega')]
prices = prices[~prices.name.str.contains('Daniel Vukovic')]
prices = prices[~prices.name.str.contains('Danil Prucev')]
prices = prices[~prices.name.str.contains('Danil Prutsev')]
prices = prices[~prices.name.str.contains('Danilo Fernandes')]
prices = prices[~prices.name.str.contains('Danilo OrtizDavid Henen')]
prices = prices[~prices.name.str.contains('David Jensen')]
prices = prices[~prices.name.str.contains('David Junior Hoilett')]
prices = prices[~prices.name.str.contains('David Karaev')]
prices = prices[~prices.name.str.contains('David Kong')]
prices = prices[~prices.name.str.contains('David Lennart Philipp')]
prices = prices[~prices.name.str.contains('David Loera')]
prices = prices[~prices.name.str.contains('David Martin')]
prices = prices[~prices.name.str.contains('David Pavelka')]
prices = prices[~prices.name.str.contains('David Philipp')]
prices = prices[~prices.name.str.contains('David Sualehe')]
prices = prices[~prices.name.str.contains('David Tavares')]
prices = prices[~prices.name.str.contains('Davide Faraoni')]
prices = prices[~prices.name.str.contains('Davide Ghislandi')]
prices = prices[~prices.name.str.contains('Davide Masella')]
prices = prices[~prices.name.str.contains('Dayro Moreno')]
prices = prices[~prices.name.str.contains('Declan Drysdale')]
prices = prices[~prices.name.str.contains('Deian Veron')]
prices = prices[~prices.name.str.contains('Dejan Galjen')]
prices = prices[~prices.name.str.contains('Dejan Kerkez')]
prices = prices[~prices.name.str.contains('Deji Sotona')]
prices = prices[~prices.name.str.contains('Delano Vianello')]
prices = prices[~prices.name.str.contains('Demba Ba')]
prices = prices[~prices.name.str.contains('Denis Adamov')]
prices = prices[~prices.name.str.contains('Denis Franchi')]
prices = prices[~prices.name.str.contains('Denis Petric')]
prices = prices[~prices.name.str.contains('Dennis Adeniran')]
prices = prices[~prices.name.str.contains('Dennis Gorka')]
prices = prices[~prices.name.str.contains('Dennis Jastrzembski')]
prices = prices[~prices.name.str.contains('Dennis Politic')]
prices = prices[~prices.name.str.contains('Denys Bain')]
prices = prices[~prices.name.str.contains('Denzel Jubitana')]
prices = prices[~prices.name.str.contains('Derek Dodson')]
prices = prices[~prices.name.str.contains('DerickDerick')]
prices = prices[~prices.name.str.contains('Derrick Tshimanga')]
prices = prices[~prices.name.str.contains('Desevio Payne')]
prices = prices[~prices.name.str.contains('Destiny Udogie')]
prices = prices[~prices.name.str.contains('Dialy Ndiaye')]
prices = prices[~prices.name.str.contains('Diant Ramaj')]
prices = prices[~prices.name.str.contains('DidiDidi')]
prices = prices[~prices.name.str.contains('Diego Braghieri')]
prices = prices[~prices.name.str.contains('Diego Calcaterra')]
prices = prices[~prices.name.str.contains('Diego Churin')]
prices = prices[~prices.name.str.contains('Diego FerreiraDiego Ferreira')]
prices = prices[~prices.name.str.contains('Diego Lopes')]
prices = prices[~prices.name.str.contains('Diego Mendoza')]
prices = prices[~prices.name.str.contains('Diego Novaretti')]
prices = prices[~prices.name.str.contains('Diogo Teixeira')]
prices = prices[~prices.name.str.contains('Dion McGhee')]
prices = prices[~prices.name.str.contains('Dion Moise Sahi')]
prices = prices[~prices.name.str.contains('Dion Sanderson')]
prices = prices[~prices.name.str.contains('Dior Angus')]
prices = prices[~prices.name.str.contains('Dixon Renteria')]
prices = prices[~prices.name.str.contains('Donis Avdijaj')]
prices = prices[~prices.name.str.contains('Douglas Friedrich')]
prices = prices[~prices.name.str.contains('Duncan Turnbull')]
prices = prices[~prices.name.str.contains('Duncan Turnbull')]
prices = prices[~prices.name.str.contains('Dusan Jovancic')]
prices = prices[~prices.name.str.contains('Dyego Sousa')]
prices = prices[~prices.name.str.contains('Dylan Asonganyi')]
prices = prices[~prices.name.str.contains('Dylan Borrero')]
prices = prices[~prices.name.str.contains('Dylan Castanheira')]
prices = prices[~prices.name.str.contains('Dylan Levitt')]
prices = prices[~prices.name.str.contains('Dylan Mottley-Henry')]
prices = prices[~prices.name.str.contains('Dylan Reid')]
prices = prices[~prices.name.str.contains('Dylan Vente')]
prices = prices[~prices.name.str.contains('EÃ±aut Mendia')]
prices = prices[~prices.name.str.contains('Ebrima Darboe')]
prices = prices[~prices.name.str.contains('Eddie Nolan')]
prices = prices[~prices.name.str.contains('Edgar Castillo')]
prices = prices[~prices.name.str.contains('Edison Duarte')]
prices = prices[~prices.name.str.contains('Edoardo Bove')]
prices = prices[~prices.name.str.contains('Edwin Banguera')]
prices = prices[~prices.name.str.contains('Edwin Cardona')]
prices = prices[~prices.name.str.contains('Edwin Mosquera')]
prices = prices[~prices.name.str.contains('Egemen Pehlivan')]
prices = prices[~prices.name.str.contains('Egor Generalov')]
prices = prices[~prices.name.str.contains('Egor Prutsev')]
prices = prices[~prices.name.str.contains('Ekrem Kilicarslan')]
prices = prices[~prices.name.str.contains('El Mami Tetah')]
prices = prices[~prices.name.str.contains('Elbasan Rashani')]
prices = prices[~prices.name.str.contains('Eldin Jakupovic')]
prices = prices[~prices.name.str.contains('Elia Caprile')]
prices = prices[~prices.name.str.contains('Elias Herzig')]
prices = prices[~prices.name.str.contains('Elias Manoel')]
prices = prices[~prices.name.str.contains('Elias Oubella')]
prices = prices[~prices.name.str.contains('Elias Pereyra')]
prices = prices[~prices.name.str.contains('Elliot Anderson')]
prices = prices[~prices.name.str.contains('Elliott Bolton')]
prices = prices[~prices.name.str.contains('Elliott Bonds')]
prices = prices[~prices.name.str.contains('Ellis Simms')]
prices = prices[~prices.name.str.contains('ElzoElzo')]
prices = prices[~prices.name.str.contains('Emanuel Herrera')]
prices = prices[~prices.name.str.contains('Emanuel Insua')]
prices = prices[~prices.name.str.contains('Emanuel Olivera')]
prices = prices[~prices.name.str.contains('Emanuel Rosales')]
prices = prices[~prices.name.str.contains('Emir Senocak')]
prices = prices[~prices.name.str.contains('Emirhan Kascioglu')]
prices = prices[~prices.name.str.contains('Emmanuel Boateng')]
prices = prices[~prices.name.str.contains('Emmanuel Gyabuaa')]
prices = prices[~prices.name.str.contains('Emmanuel Hackman')]
prices = prices[~prices.name.str.contains('Emmanuel Idem')]
prices = prices[~prices.name.str.contains('Emmanuel Monthe')]
prices = prices[~prices.name.str.contains('Emmanuel Ochoa')]
prices = prices[~prices.name.str.contains('Emmanuel Ojeda')]
prices = prices[~prices.name.str.contains('Emeka Obi')]
prices = prices[~prices.name.str.contains('Emerson Royal')]
prices = prices[~prices.name.str.contains('Emiliano Insua')]
prices = prices[~prices.name.str.contains('Emiliano Vecchio')]
prices = prices[~prices.name.str.contains('Emin Bayram')]
prices = prices[~prices.name.str.contains('Emrah Bassan')]
prices = prices[~prices.name.str.contains('Emre Bilgin')]
prices = prices[~prices.name.str.contains('Emre Colak')]
prices = prices[~prices.name.str.contains('Emre Karaal')]
prices = prices[~prices.name.str.contains('Enes Karakus')]
prices = prices[~prices.name.str.contains('Enes Sancar Sahin')]
prices = prices[~prices.name.str.contains('Ennio van der Gouw')]
prices = prices[~prices.name.str.contains('Enrico Hernandez')]
prices = prices[~prices.name.str.contains('Enver Cenk Sahin')]
prices = prices[~prices.name.str.contains('Enzo Coacci')]
prices = prices[~prices.name.str.contains('Enzo Copetti')]
prices = prices[~prices.name.str.contains('Enzo Fernandez')]
prices = prices[~prices.name.str.contains('Eray Iscan')]
prices = prices[~prices.name.str.contains('Erdin Demir')]
prices = prices[~prices.name.str.contains('Erdogan Kaya')]
prices = prices[~prices.name.str.contains('Eren Elmali')]
prices = prices[~prices.name.str.contains('Eren Kiryolcu')]
prices = prices[~prices.name.str.contains('Eric Bocat')]
prices = prices[~prices.name.str.contains('Eric Calvillo')]
prices = prices[~prices.name.str.contains('Eric DickEric Dick')]
prices = prices[~prices.name.str.contains('Eric Bocat')]
prices = prices[~prices.name.str.contains('Eric Calvillo')]
prices = prices[~prices.name.str.contains('Eric Ebimbe')]
prices = prices[~prices.name.str.contains('Eric Junior Bocat')]
prices = prices[~prices.name.str.contains('Eric Lichaj')]
prices = prices[~prices.name.str.contains('Eric Lopez')]
prices = prices[~prices.name.str.contains('Eric MezaEric Meza')]
prices = prices[~prices.name.str.contains('Eric Miller')]
prices = prices[~prices.name.str.contains('Erick Ferigra')]
prices = prices[~prices.name.str.contains('Ernest Boahene')]
prices = prices[~prices.name.str.contains('Erwan Belhadji')]
prices = prices[~prices.name.str.contains('Esteban Conde')]
prices = prices[~prices.name.str.contains('Esteban Rolon')]
prices = prices[~prices.name.str.contains('Ethan Bartlow')]
prices = prices[~prices.name.str.contains('Ethan Johnston')]
prices = prices[~prices.name.str.contains('Ethan Laird')]
prices = prices[~prices.name.str.contains('Etienne Vaessen')]
prices = prices[~prices.name.str.contains('Eugenio Pizzuto')]
prices = prices[~prices.name.str.contains('Evan BushEvan Bush')]
prices = prices[~prices.name.str.contains('Evan Newton')]
prices = prices[~prices.name.str.contains('Evann Guessand')]
prices = prices[~prices.name.str.contains('EveraldoEveraldo')]
prices = prices[~prices.name.str.contains('Evert Linthorst')]
prices = prices[~prices.name.str.contains('Everton Felipe')]
prices = prices[~prices.name.str.contains('Ezequiel Cerica')]
prices = prices[~prices.name.str.contains('Ezequiel Cirigliano')]
prices = prices[~prices.name.str.contains('Ezequiel Edison Rescaldani')]
prices = prices[~prices.name.str.contains('Ezequiel Fernandez')]
prices = prices[~prices.name.str.contains('Ezequiel Piovi')]
prices = prices[~prices.name.str.contains('Ezequiel Schelotto')]
prices = prices[~prices.name.str.contains('Ezequiel Unsain')]
prices = prices[~prices.name.str.contains('FaÃ¯z Selemani')]
prices = prices[~prices.name.str.contains('Fabian Cubero')]
prices = prices[~prices.name.str.contains('Fabian de Keijzer')]
prices = prices[~prices.name.str.contains('Fabian Kalig')]
prices = prices[~prices.name.str.contains('Fabian Noguera')]
prices = prices[~prices.name.str.contains('Fabien Farnolle')]
prices = prices[~prices.name.str.contains('Fabio Abreu')]
prices = prices[~prices.name.str.contains('Fabio Carvalho')]
prices = prices[~prices.name.str.contains('Fabio China')]
prices = prices[~prices.name.str.contains('Fabio ChinaFabio China')]
prices = prices[~prices.name.str.contains('Fabio Pereyra')]
prices = prices[~prices.name.str.contains('Fabio Pisacane')]
prices = prices[~prices.name.str.contains('Fabio SantosFabio Santos')]
prices = prices[~prices.name.str.contains('Fabio Schneider')]
prices = prices[~prices.name.str.contains('Fabio Vazquez')]
prices = prices[~prices.name.str.contains('Fabrice Hartmann')]
prices = prices[~prices.name.str.contains('Fabricio Dominguez')]
prices = prices[~prices.name.str.contains('Fabricio Fontanini')]
prices = prices[~prices.name.str.contains('FabriFabri')]
prices = prices[~prices.name.str.contains('Fabrizio Caligara')]
prices = prices[~prices.name.str.contains('Facundo Altamirano')]
prices = prices[~prices.name.str.contains('Facundo Barboza')]
prices = prices[~prices.name.str.contains('Facundo Bertoglio')]
prices = prices[~prices.name.str.contains('Facundo Cambeses')]
prices = prices[~prices.name.str.contains('Facundo Castet')]
prices = prices[~prices.name.str.contains('Facundo Curuchet')]
prices = prices[~prices.name.str.contains('Facundo Fernandez')]
prices = prices[~prices.name.str.contains('Facundo Ferrero')]
prices = prices[~prices.name.str.contains('Facundo Garces')]
prices = prices[~prices.name.str.contains('Facundo Mater')]
prices = prices[~prices.name.str.contains('Facundo Melivilo')]
prices = prices[~prices.name.str.contains('Facundo Pons')]
prices = prices[~prices.name.str.contains('Facundo Sanchez')]
prices = prices[~prices.name.str.contains('Facundo Tobares')]
prices = prices[~prices.name.str.contains('Faiq Bolkiah')]
prices = prices[~prices.name.str.contains('Faiz Selemani')]
prices = prices[~prices.name.str.contains('Falcao')]
prices = prices[~prices.name.str.contains('Fali')]
prices = prices[~prices.name.str.contains('FaliFali')]
prices = prices[~prices.name.str.contains('Famana Quizera')]
prices = prices[~prices.name.str.contains('Fatih Yigit Sanliturk')]
prices = prices[~prices.name.str.contains('Favio Cabral')]
prices = prices[~prices.name.str.contains('Fede Varela')]
prices = prices[~prices.name.str.contains('Federico Andueza')]
prices = prices[~prices.name.str.contains('Federico Bonansea')]
prices = prices[~prices.name.str.contains('Federico Brancolini')]
prices = prices[~prices.name.str.contains('Federico Gattoni')]
prices = prices[~prices.name.str.contains('Federico Gino')]
prices = prices[~prices.name.str.contains('Federico Girotti')]
prices = prices[~prices.name.str.contains('Federico Lanzillotta')]
prices = prices[~prices.name.str.contains('Federico Mancuello')]
prices = prices[~prices.name.str.contains('Federico Martinez')]
prices = prices[~prices.name.str.contains('Federico Palacios')]
prices = prices[~prices.name.str.contains('Federico Ricci')]
prices = prices[~prices.name.str.contains('Federico Vera')]
prices = prices[~prices.name.str.contains('Federico Vietto')]
prices = prices[~prices.name.str.contains('Federico Vismara')]
prices = prices[~prices.name.str.contains('Felipe Aguilar')]
prices = prices[~prices.name.str.contains('Felipe Martins')]
prices = prices[~prices.name.str.contains('Felipe PeÃ±a Biafore')]
prices = prices[~prices.name.str.contains('Felipe Santana')]
prices = prices[~prices.name.str.contains('Felipe Valencia')]
prices = prices[~prices.name.str.contains('Felipe VizeuFelipe Vizeu')]
prices = prices[~prices.name.str.contains('Felix Correia')]
prices = prices[~prices.name.str.contains('Felix Irorere')]
prices = prices[~prices.name.str.contains('Femi Azeez')]
prices = prices[~prices.name.str.contains('Femi Seriki')]
prices = prices[~prices.name.str.contains('Fer NiÃ±oFer NiÃ±o')]
prices = prices[~prices.name.str.contains('Ferhat Erdogen')]
prices = prices[~prices.name.str.contains('Ferhat GÃ¶rgulu')]
prices = prices[~prices.name.str.contains('Ferhat Kaplan')]
prices = prices[~prices.name.str.contains('Fernando Andrade')]
prices = prices[~prices.name.str.contains('Fernando Caixeta')]
prices = prices[~prices.name.str.contains('Fernando Canesin')]
prices = prices[~prices.name.str.contains('Fernando Coniglio')]
prices = prices[~prices.name.str.contains('Fernando Cosciuc')]
prices = prices[~prices.name.str.contains('Fernando Evangelista')]
prices = prices[~prices.name.str.contains('Fernando Gago')]
prices = prices[~prices.name.str.contains('Fernando Luna')]
prices = prices[~prices.name.str.contains('Fernando Meza')]
prices = prices[~prices.name.str.contains('Fernando Monetti')]
prices = prices[~prices.name.str.contains('Fernando NiÃ±o')]
prices = prices[~prices.name.str.contains('Fernando Pellegrino')]
prices = prices[~prices.name.str.contains('Fernando Reges')]
prices = prices[~prices.name.str.contains('Fernando Roman')]
prices = prices[~prices.name.str.contains('Fernando Sobral')]
prices = prices[~prices.name.str.contains('Fernando Tobio')]
prices = prices[~prices.name.str.contains('FerreiraFerreira')]
prices = prices[~prices.name.str.contains('Festus Arthur')]
prices = prices[~prices.name.str.contains('Festy Ebosele')]
prices = prices[~prices.name.str.contains('Filippo Rinaldi')]
prices = prices[~prices.name.str.contains('Filippo Romagna')]
prices = prices[~prices.name.str.contains('Fin Stevens')]
prices = prices[~prices.name.str.contains('Finlay Hurford-Lockett')]
prices = prices[~prices.name.str.contains('Finley Craske')]
prices = prices[~prices.name.str.contains('Finley Shrimpton')]
prices = prices[~prices.name.str.contains('Finley Stevens')]
prices = prices[~prices.name.str.contains('Finn Azaz')]
prices = prices[~prices.name.str.contains('Florent da Silva')]
prices = prices[~prices.name.str.contains('Florent Hoti')]
prices = prices[~prices.name.str.contains('Florian Flecker')]
prices = prices[~prices.name.str.contains('Florian Kamberi')]
prices = prices[~prices.name.str.contains('Florian Monzon')]
prices = prices[~prices.name.str.contains('Florian Stritzel')]
prices = prices[~prices.name.str.contains('Florin Andone')]
prices = prices[~prices.name.str.contains('Foster Langsdorf')]
prices = prices[~prices.name.str.contains('Francesco Rossi')]
prices = prices[~prices.name.str.contains('Francisco Apaolaza')]
prices = prices[~prices.name.str.contains('Francisco Flores')]
prices = prices[~prices.name.str.contains('Francisco Fydriszewski')]
prices = prices[~prices.name.str.contains('Francisco Geraldes')]
prices = prices[~prices.name.str.contains('Francisco Ilarregui')]
prices = prices[~prices.name.str.contains('Francisco Manenti')]
prices = prices[~prices.name.str.contains('Francisco Meixedo')]
prices = prices[~prices.name.str.contains('Francisco Montero')]
prices = prices[~prices.name.str.contains('Francisco Ramirez')]
prices = prices[~prices.name.str.contains('Francisco Silva')]
prices = prices[~prices.name.str.contains('Franco Baldassarra')]
prices = prices[~prices.name.str.contains('Franco Di Santo')]
prices = prices[~prices.name.str.contains('Franco Escobar')]
prices = prices[~prices.name.str.contains('Franco Godoy')]
prices = prices[~prices.name.str.contains('Franco Gonzalez')]
prices = prices[~prices.name.str.contains('Franco Leys')]
prices = prices[~prices.name.str.contains('Franco Negri')]
prices = prices[~prices.name.str.contains('Franco Orozco')]
prices = prices[~prices.name.str.contains('Franco Paredes')]
prices = prices[~prices.name.str.contains('Franco Perez')]
prices = prices[~prices.name.str.contains('Franco Perinciolo')]
prices = prices[~prices.name.str.contains('Franco Pizzicanella')]
prices = prices[~prices.name.str.contains('Franco Quiroz')]
prices = prices[~prices.name.str.contains('Franco Rivasseau')]
prices = prices[~prices.name.str.contains('Franco Tongya')]
prices = prices[~prices.name.str.contains('Francois Marquet')]
prices = prices[~prices.name.str.contains('Francois-Xavier Fumu Tamuzo')]
prices = prices[~prices.name.str.contains('Frank Fielding')]
prices = prices[~prices.name.str.contains('Frank Tsadjout')]
prices = prices[~prices.name.str.contains('Frank Vincent')]
prices = prices[~prices.name.str.contains('Franko Kovacevic')]
prices = prices[~prices.name.str.contains('Fraser Murray')]
prices = prices[~prices.name.str.contains('Fred Emmings')]
prices = prices[~prices.name.str.contains('Frederik Alves')]
prices = prices[~prices.name.str.contains('Fredrik Oppegard')]
prices = prices[~prices.name.str.contains('Furkan Beklevic')]
prices = prices[~prices.name.str.contains('Furkan Kara')]
prices = prices[~prices.name.str.contains('Fynn OttoFynn Otto')]
prices = prices[~prices.name.str.contains('Gabriel Barbosa')]
prices = prices[~prices.name.str.contains('Gabriel Batista')]
prices = prices[~prices.name.str.contains('Gabriel Boschilia')]
prices = prices[~prices.name.str.contains('Gabriel Carrasco')]
prices = prices[~prices.name.str.contains('Gabriel Chapeco')]
prices = prices[~prices.name.str.contains('Gabriel Compagnucci')]
prices = prices[~prices.name.str.contains('Gabriel Dias')]
prices = prices[~prices.name.str.contains('Gabriel Diaz')]
prices = prices[~prices.name.str.contains('Gabriel Esparza')]
prices = prices[~prices.name.str.contains('Gabriel Falcao')]
prices = prices[~prices.name.str.contains('Gabriel Hachen')]
prices = prices[~prices.name.str.contains('Gabriel Pires')]
prices = prices[~prices.name.str.contains('Gabriel Rojas')]
prices = prices[~prices.name.str.contains('Gabriel Slonina')]
prices = prices[~prices.name.str.contains('Gabriel Veiga')]
prices = prices[~prices.name.str.contains('Gabriel VeronGabriel Veron')]
prices = prices[~prices.name.str.contains('Gaetan Poussin')]
prices = prices[~prices.name.str.contains('GalenoGalenoLeft Midfiel')]
prices = prices[~prices.name.str.contains('Gamid Agalarov')]
prices = prices[~prices.name.str.contains('GansoGanso')]
prices = prices[~prices.name.str.contains('Gary Maley')]
prices = prices[~prices.name.str.contains('Gary Roberts')]
prices = prices[~prices.name.str.contains('Gary Sawyer')]
prices = prices[~prices.name.str.contains('Gaston Ãlvarez Suarez')]
prices = prices[~prices.name.str.contains('Gaston Benavidez')]
prices = prices[~prices.name.str.contains('Gaston Diaz')]
prices = prices[~prices.name.str.contains('Gaston Fernandez')]
prices = prices[~prices.name.str.contains('Gaston Gerzel')]
prices = prices[~prices.name.str.contains('Gaston Gomez')]
prices = prices[~prices.name.str.contains('Gaston Gonzalez')]
prices = prices[~prices.name.str.contains('Gaston Suso')]
prices = prices[~prices.name.str.contains('Gaston Torres')]
prices = prices[~prices.name.str.contains('Gavin Bazunu')]
prices = prices[~prices.name.str.contains('Gedion Zelalem')]
prices = prices[~prices.name.str.contains('GeilsonGeilson')]
prices = prices[~prices.name.str.contains('Gela Zaseev')]
prices = prices[~prices.name.str.contains('George Acosta')]
prices = prices[~prices.name.str.contains('George Asomani')]
prices = prices[~prices.name.str.contains('George Campbell')]
prices = prices[~prices.name.str.contains('George Fochive')]
prices = prices[~prices.name.str.contains('George Hirst')]
prices = prices[~prices.name.str.contains('George Melkadze')]
prices = prices[~prices.name.str.contains('George Nugent')]
prices = prices[~prices.name.str.contains('Georges Mandjeck')]
prices = prices[~prices.name.str.contains('Georgi Dzhikiya')]
prices = prices[~prices.name.str.contains('Getterson')]
prices = prices[~prices.name.str.contains('Giacomo Ricci')]
prices = prices[~prices.name.str.contains('Giacomo Vrioni')]
prices = prices[~prices.name.str.contains('Gian-Luca Itter')]
prices = prices[~prices.name.str.contains('Gian-Luca Waldschmidt')]
prices = prices[~prices.name.str.contains('Giancarlo Gonzalez')]
prices = prices[~prices.name.str.contains('Gianfranco Facchineri')]
prices = prices[~prices.name.str.contains('Giannelli Imbula')]
prices = prices[~prices.name.str.contains('Gianni Mollo')]
prices = prices[~prices.name.str.contains('Giorgi Kochorashvili')]
prices = prices[~prices.name.str.contains('Giovanni Crociata')]
prices = prices[~prices.name.str.contains('Giuseppe Mastinu')]
prices = prices[~prices.name.str.contains('Gjoko Zajkov')]
prices = prices[~prices.name.str.contains('Gokhan Kardes')]
prices = prices[~prices.name.str.contains('Gonzalo Asis')]
prices = prices[~prices.name.str.contains('Gonzalo Castellani')]
prices = prices[~prices.name.str.contains('Gonzalo GoÃ±i')]
prices = prices[~prices.name.str.contains('Gonzalo Jara')]
prices = prices[~prices.name.str.contains('Gonzalo Maroni')]
prices = prices[~prices.name.str.contains('Gonzalo Rodriguez')]
prices = prices[~prices.name.str.contains('Gonzalo Veron')]
prices = prices[~prices.name.str.contains('Grace Bokake')]
prices = prices[~prices.name.str.contains('Graham Smith')]
prices = prices[~prices.name.str.contains('Grant Lillard')]
prices = prices[~prices.name.str.contains('Grayson Barber')]
prices = prices[~prices.name.str.contains('Greg Ranjitsingh')]
prices = prices[~prices.name.str.contains('Griffin Yow')]
prices = prices[~prices.name.str.contains('Guido Mainero')]
prices = prices[~prices.name.str.contains('Guido Vadala')]
prices = prices[~prices.name.str.contains('Guilherme Camacho')]
prices = prices[~prices.name.str.contains('Guilherme GuedesGuilherme Guedes')]
prices = prices[~prices.name.str.contains('Guilherme Lazaroni')]
prices = prices[~prices.name.str.contains('Guilherme Mantuan')]
prices = prices[~prices.name.str.contains('Guilherme Oliveira')]
prices = prices[~prices.name.str.contains('Guillem Molina')]
prices = prices[~prices.name.str.contains('Guillermo Celis')]
prices = prices[~prices.name.str.contains('Guillermo Fernandez')]
prices = prices[~prices.name.str.contains('Guillermo Gozalvez')]
prices = prices[~prices.name.str.contains('Guram Tetrashvili')]
prices = prices[~prices.name.str.contains('GustavinhoGustavinho')]
prices = prices[~prices.name.str.contains('Gustavo ApisGustavo Apis')]
prices = prices[~prices.name.str.contains('Gustavo BlancoGustavo Blanco')]
prices = prices[~prices.name.str.contains('Gustavo Canto')]
prices = prices[~prices.name.str.contains('Gustavo CoutinhoGustavo Coutinho')]
prices = prices[~prices.name.str.contains('Gustavo Garcia')]
prices = prices[~prices.name.str.contains('Gustavo Henrique')]
prices = prices[~prices.name.str.contains('Gustavo Mosquito')]
prices = prices[~prices.name.str.contains('Gustavo NescauGustavo Nescau')]
prices = prices[~prices.name.str.contains('Gustavo SilvaGustavo Silva')]
prices = prices[~prices.name.str.contains('Guy Mbenza')]
prices = prices[~prices.name.str.contains('Haqi Osman')]
prices = prices[~prices.name.str.contains('Harlem Gnohere')]
prices = prices[~prices.name.str.contains('Harry Clarke')]
prices = prices[~prices.name.str.contains('Harry Draper')]
prices = prices[~prices.name.str.contains('Harry Isted')]
prices = prices[~prices.name.str.contains('Harry Lewis')]
prices = prices[~prices.name.str.contains('Harry McHugh')]
prices = prices[~prices.name.str.contains('Harry Paton')]
prices = prices[~prices.name.str.contains('Harry Vaughan')]
prices = prices[~prices.name.str.contains('Harry Wood')]
prices = prices[~prices.name.str.contains('Harry Wright')]
prices = prices[~prices.name.str.contains('Harvey Bunker')]
prices = prices[~prices.name.str.contains('Harvey Cartwright')]
prices = prices[~prices.name.str.contains('Harvey Elliott')]
prices = prices[~prices.name.str.contains('Harvey Rew')]
prices = prices[~prices.name.str.contains('Harvey White')]
prices = prices[~prices.name.str.contains('Hayden Carter')]
prices = prices[~prices.name.str.contains('Hayden Hackney')]
prices = prices[~prices.name.str.contains('Hayden Muller')]
prices = prices[~prices.name.str.contains('Haydon Roberts')]
prices = prices[~prices.name.str.contains('Helder Barbosa')]
prices = prices[~prices.name.str.contains('Henning Matriciani')]
prices = prices[~prices.name.str.contains('Henri Saivet')]
prices = prices[~prices.name.str.contains('Henrich Ravas')]
prices = prices[~prices.name.str.contains('Henrique Trevisan')]
prices = prices[~prices.name.str.contains('Hernan Burbano')]
prices = prices[~prices.name.str.contains('Hernan Lamberti')]
prices = prices[~prices.name.str.contains('Hernan Lopez')]
prices = prices[~prices.name.str.contains('Hicham Mahou')]
prices = prices[~prices.name.str.contains('Hugo Novoa')]
prices = prices[~prices.name.str.contains('Hugo Rodallega')]
prices = prices[~prices.name.str.contains('Hugo Silveira')]
prices = prices[~prices.name.str.contains('Hugo Sotelo')]
prices = prices[~prices.name.str.contains('Hugo SouzaHugo Souza')]
prices = prices[~prices.name.str.contains('Iago MaidanaIago Maidana')]
prices = prices[~prices.name.str.contains('Ian Escobar')]
prices = prices[~prices.name.str.contains('Ian FrayIan Fray')]
prices = prices[~prices.name.str.contains('Ian Hoffmann')]
prices = prices[~prices.name.str.contains('Ian Maatsen')]
prices = prices[~prices.name.str.contains('Ian Poveda')]
prices = prices[~prices.name.str.contains('Ian Poveda-Ocampo')]
prices = prices[~prices.name.str.contains('IbrahimaIbrahima')]
prices = prices[~prices.name.str.contains('Idris Umaev')]
prices = prices[~prices.name.str.contains('Idrissa Dioh')]
prices = prices[~prices.name.str.contains('Idrissa Sylla')]
prices = prices[~prices.name.str.contains('Ifunanyachi Achara')]
prices = prices[~prices.name.str.contains('Ignacio Miramon')]
prices = prices[~prices.name.str.contains('Ignacio Russo')]
prices = prices[~prices.name.str.contains('Ignacio Schor')]
prices = prices[~prices.name.str.contains('Igor Lucatelli')]
prices = prices[~prices.name.str.contains('Igor Obukhov')]
prices = prices[~prices.name.str.contains('Ilhan Depe')]
prices = prices[~prices.name.str.contains('Ilia Shkurin')]
prices = prices[~prices.name.str.contains('Iliman Ndiaye')]
prices = prices[~prices.name.str.contains('Isaac Olaofe')]
prices = prices[~prices.name.str.contains('Isaac Vassell')]
prices = prices[~prices.name.str.contains('Isaiah Brown')]
prices = prices[~prices.name.str.contains('Jakub Niemczyk')]
prices = prices[~prices.name.str.contains('Jakub Szumski')]
prices = prices[~prices.name.str.contains('Jalen Neal')]
prices = prices[~prices.name.str.contains('James Clark')]
prices = prices[~prices.name.str.contains('James Dodd')]
prices = prices[~prices.name.str.contains('James Garner')]
prices = prices[~prices.name.str.contains('James Gullan')]
prices = prices[~prices.name.str.contains('James Marcinkowski')]
prices = prices[~prices.name.str.contains('James Montgomery')]
prices = prices[~prices.name.str.contains('James Morton')]
prices = prices[~prices.name.str.contains('James Olayinka')]
prices = prices[~prices.name.str.contains('James Pardington')]
prices = prices[~prices.name.str.contains('James Rowland')]
prices = prices[~prices.name.str.contains('James Vennings')]
prices = prices[~prices.name.str.contains('Jamie Armstrong')]
prices = prices[~prices.name.str.contains('Jamie Barjonas')]
prices = prices[~prices.name.str.contains('Jamie Cumming')]
prices = prices[~prices.name.str.contains('Jamie Fielding')]
prices = prices[~prices.name.str.contains('Jamie Gullan')]
prices = prices[~prices.name.str.contains('Jamie Mascoll')]
prices = prices[~prices.name.str.contains('Jamie Semple')]
prices = prices[~prices.name.str.contains('Jamie Smith')]
prices = prices[~prices.name.str.contains('Jamie Soule')]
prices = prices[~prices.name.str.contains('Jamil Awoyejo')]
prices = prices[~prices.name.str.contains('Jan Bekkema')]
prices = prices[~prices.name.str.contains('Jan Carlos Hurtado')]
prices = prices[~prices.name.str.contains('Jan de Boer')]
prices = prices[~prices.name.str.contains('Jan Hurtado')]
prices = prices[~prices.name.str.contains('Jaquite')]
prices = prices[~prices.name.str.contains('Jarno Westerman')]
prices = prices[~prices.name.str.contains('Jaroslaw Jach')]
prices = prices[~prices.name.str.contains('Jaroslaw Niezgoda')]
prices = prices[~prices.name.str.contains('Jarvis Cleal')]
prices = prices[~prices.name.str.contains('Jason Cummings')]
prices = prices[~prices.name.str.contains('Jay-Dee Geusens')]
prices = prices[~prices.name.str.contains('Jayden Bennetts')]
prices = prices[~prices.name.str.contains('Jayden Campbell')]
prices = prices[~prices.name.str.contains('Jayden Mitchell-Lawson')]
prices = prices[~prices.name.str.contains('Jayden Onen')]
prices = prices[~prices.name.str.contains('Jayden Reid')]
prices = prices[~prices.name.str.contains('Jayden Richardson')]
prices = prices[~prices.name.str.contains('Jayson Molumby')]
prices = prices[~prices.name.str.contains('Jeison Chala')]
prices = prices[~prices.name.str.contains('Jeison Lucumi')]
prices = prices[~prices.name.str.contains('Jeizon Ramirez')]
prices = prices[~prices.name.str.contains('Jelle Duin')]
prices = prices[~prices.name.str.contains('Jens Jonsson')]
prices = prices[~prices.name.str.contains('Jeong Woo-Yeong')]
prices = prices[~prices.name.str.contains('Jeredy Hilterman')]
prices = prices[~prices.name.str.contains('Jeremy Antonisse')]
prices = prices[~prices.name.str.contains('Jeremy Helmer')]
prices = prices[~prices.name.str.contains('Jeremy Kelly')]
prices = prices[~prices.name.str.contains('Jeremy Perbet')]
prices = prices[~prices.name.str.contains('Jeremy Van Mullem')]
prices = prices[~prices.name.str.contains('Jerome Greaves')]
prices = prices[~prices.name.str.contains('Jerome Opoku')]
prices = prices[~prices.name.str.contains('Jerome Sinclair')]
prices = prices[~prices.name.str.contains('Jeronimo Pourtau')]
prices = prices[~prices.name.str.contains('Jesus Ferreira')]
prices = prices[~prices.name.str.contains('Jim McNulty')]
prices = prices[~prices.name.str.contains('Jimmy Knowles')]
prices = prices[~prices.name.str.contains('Jimmy McNulty')]
prices = prices[~prices.name.str.contains('Jiri Skalak')]
prices = prices[~prices.name.str.contains('Joao Jaquite')]
prices = prices[~prices.name.str.contains('Joao Klauss')]
prices = prices[~prices.name.str.contains('Joao Mendes')]
prices = prices[~prices.name.str.contains('Joao Monteiro')]
prices = prices[~prices.name.str.contains('Joao Queiros')]
prices = prices[~prices.name.str.contains('Joaquin Arzura')]
prices = prices[~prices.name.str.contains('Joaquin Blazquez')]
prices = prices[~prices.name.str.contains('Joaquin Hass')]
prices = prices[~prices.name.str.contains('Joaquin Mendive')]
prices = prices[~prices.name.str.contains('Joaquin Varela')]
prices = prices[~prices.name.str.contains('Jobi McAnuff')]
prices = prices[~prices.name.str.contains('Jock Curran')]
prices = prices[~prices.name.str.contains('Jodi Jones')]
prices = prices[~prices.name.str.contains('Jody Lukoki')]
prices = prices[~prices.name.str.contains('Joe Adams')]
prices = prices[~prices.name.str.contains('Joe Foulkes')]
prices = prices[~prices.name.str.contains('Joe Fryer')]
prices = prices[~prices.name.str.contains('Joe Garner')]
prices = prices[~prices.name.str.contains('Joe Gelhardt')]
prices = prices[~prices.name.str.contains('Joe Grayson')]
prices = prices[~prices.name.str.contains('Joe Hilton')]
prices = prices[~prices.name.str.contains('Joe Leesley')]
prices = prices[~prices.name.str.contains('Joel Coleman')]
prices = prices[~prices.name.str.contains('Joey Jones')]
prices = prices[~prices.name.str.contains('Johan Carbonero')]
prices = prices[~prices.name.str.contains('John Brooks')]
prices = prices[~prices.name.str.contains('John Donald')]
prices = prices[~prices.name.str.contains('John Nekadio')]
prices = prices[~prices.name.str.contains('John Patrick')]
prices = prices[~prices.name.str.contains('John Robertson')]
prices = prices[~prices.name.str.contains('John Yeboah')]
prices = prices[~prices.name.str.contains('Kacper Mialkowski')]
prices = prices[~prices.name.str.contains('Kacper Urbanski')]
prices = prices[~prices.name.str.contains('Kagan Moradaoglu')]
prices = prices[~prices.name.str.contains('Kai Koreniuk')]
prices = prices[~prices.name.str.contains('Kai McKenzie-Lyle')]
prices = prices[~prices.name.str.contains('Kane Crichlow')]
prices = prices[~prices.name.str.contains('Kang-in Lee')]
prices = prices[~prices.name.str.contains('Karamoko Dembele')]
prices = prices[~prices.name.str.contains('Kastriot Dermaku')]
prices = prices[~prices.name.str.contains('Katuku Tshimanga')]
prices = prices[~prices.name.str.contains('Kayden Pierre')]
prices = prices[~prices.name.str.contains('Kayne Ramsay')]
prices = prices[~prices.name.str.contains('Kazaiah Sterling')]
prices = prices[~prices.name.str.contains('Keaghan Jacobs')]
prices = prices[~prices.name.str.contains('Keanu Staude')]
prices = prices[~prices.name.str.contains('Keito Nakamura')]
prices = prices[~prices.name.str.contains('Kelechi Nwakali')]
prices = prices[~prices.name.str.contains('Kelland Watts')]
prices = prices[~prices.name.str.contains('Kelsey Mooney')]
prices = prices[~prices.name.str.contains('Kemal Ruzgar')]
prices = prices[~prices.name.str.contains('Kendall McIntosh')]
prices = prices[~prices.name.str.contains('Kenneth Dougall')]
prices = prices[~prices.name.str.contains('Kenneth Kronholm')]
prices = prices[~prices.name.str.contains('Kenny Coker')]
prices = prices[~prices.name.str.contains('Kenny Dougall')]
prices = prices[~prices.name.str.contains('Kenny Lala')]
prices = prices[~prices.name.str.contains('Kenny Nagera')]
prices = prices[~prices.name.str.contains('Kevin Berkoe')]
prices = prices[~prices.name.str.contains('Kevin Boma')]
prices = prices[~prices.name.str.contains('Kevin Goden')]
prices = prices[~prices.name.str.contains('Kevin Harr')]
prices = prices[~prices.name.str.contains('Kevin Ibrahim')]
prices = prices[~prices.name.str.contains('Kevin Isa Luna')]
prices = prices[~prices.name.str.contains('Kevin Lankford')]
prices = prices[~prices.name.str.contains('Kevin McDonald')]
prices = prices[~prices.name.str.contains('Kevin Molino')]
prices = prices[~prices.name.str.contains('Kevin Nickler')]
prices = prices[~prices.name.str.contains('Kevin Silva')]
prices = prices[~prices.name.str.contains('Kevin Zenon')]
prices = prices[~prices.name.str.contains('Khalil Caraballo')]
prices = prices[~prices.name.str.contains('Kieran Agard')]
prices = prices[~prices.name.str.contains('Kieran Ngwenya')]
prices = prices[~prices.name.str.contains('Kim Moon-Hwan')]
prices = prices[~prices.name.str.contains('Kingsley Fobi')]
prices = prices[~prices.name.str.contains('Kiril Despodov')]
prices = prices[~prices.name.str.contains('Kirill Dontsov')]
prices = prices[~prices.name.str.contains('Kirill Kolesnichenko')]
prices = prices[~prices.name.str.contains('Kirill Kosarev')]
prices = prices[~prices.name.str.contains('Kirill Kravtsov')]
prices = prices[~prices.name.str.contains('Koba Koindredi')]
prices = prices[~prices.name.str.contains('Kwasi Wriedt')]
prices = prices[~prices.name.str.contains('Kyle Connell')]
prices = prices[~prices.name.str.contains('Kyle Hayde')]
prices = prices[~prices.name.str.contains('Kyle Morton')]
prices = prices[~prices.name.str.contains('Kyle Taylor')]
prices = prices[~prices.name.str.contains('Kyle Zobeck')]
prices = prices[~prices.name.str.contains('Laercio Solda')]
prices = prices[~prices.name.str.contains('Lameck Banda')]
prices = prices[~prices.name.str.contains('Lamine Sherif')]
prices = prices[~prices.name.str.contains('Landry Dimata')]
prices = prices[~prices.name.str.contains('Lars Coveliers')]
prices = prices[~prices.name.str.contains('Lars Lokotsch')]
prices = prices[~prices.name.str.contains('Laurenz Beckemeyer')]
prices = prices[~prices.name.str.contains('Lautaro Blanco')]
prices = prices[~prices.name.str.contains('Lautaro Chavez')]
prices = prices[~prices.name.str.contains('Lautaro de Leon')]
prices = prices[~prices.name.str.contains('Lautaro Escalante')]
prices = prices[~prices.name.str.contains('Lautaro Geminiani')]
prices = prices[~prices.name.str.contains('Lautaro Giaccone')]
prices = prices[~prices.name.str.contains('Lautaro Montoya')]
prices = prices[~prices.name.str.contains('Lautaro Morales')]
prices = prices[~prices.name.str.contains('Lautaro Parisi')]
prices = prices[~prices.name.str.contains('Lautaro Rinaldi')]
prices = prices[~prices.name.str.contains('Lazaros Lamprou')]
prices = prices[~prices.name.str.contains('Leandro Barcia')]
prices = prices[~prices.name.str.contains('Leandro Barreiro')]
prices = prices[~prices.name.str.contains('Leandro Barreiro Martins')]
prices = prices[~prices.name.str.contains('Leandro CarvalhoLeandro Carvalho')]
prices = prices[~prices.name.str.contains('Leandro Contin')]
prices = prices[~prices.name.str.contains('Leandro Fernandez')]
prices = prices[~prices.name.str.contains('Leandro Finochietto')]
prices = prices[~prices.name.str.contains('Leandro Grimi')]
prices = prices[~prices.name.str.contains('Leandro MatheusLeandro Matheus')]
prices = prices[~prices.name.str.contains('Leandro Putaro')]
prices = prices[~prices.name.str.contains('Leandro Silva')]
prices = prices[~prices.name.str.contains('Leandro Vella')]
prices = prices[~prices.name.str.contains('Lebo Mothiba')]
prices = prices[~prices.name.str.contains('Lee Kang-In')]
prices = prices[~prices.name.str.contains('Lenny Lacroix')]
prices = prices[~prices.name.str.contains('Lentini Caciano')]
prices = prices[~prices.name.str.contains('Leo Fuhr Hjelde')]
prices = prices[~prices.name.str.contains('Leo Gomes')]
prices = prices[~prices.name.str.contains('Leonardo Gil')]
prices = prices[~prices.name.str.contains('Leonardo Jara')]
prices = prices[~prices.name.str.contains('Leonardo Realpe')]
prices = prices[~prices.name.str.contains('Leonardo Sequeira')]
prices = prices[~prices.name.str.contains('Leonardo Villalba')]
prices = prices[~prices.name.str.contains('LeonardoLeonardo')]
prices = prices[~prices.name.str.contains('Leonel Ferroni')]
prices = prices[~prices.name.str.contains('Leonel Galeano')]
prices = prices[~prices.name.str.contains('Leonel Gonzalez')]
prices = prices[~prices.name.str.contains('Leroy Kwadwo')]
prices = prices[~prices.name.str.contains('Levi Malangu')]
prices = prices[~prices.name.str.contains('Levi Ntumba')]
prices = prices[~prices.name.str.contains('Lewis Bradley')]
prices = prices[~prices.name.str.contains('Lewis Gibson')]
prices = prices[~prices.name.str.contains('Lewis Hardcastle')]
prices = prices[~prices.name.str.contains('Lewis Johnson')]
prices = prices[~prices.name.str.contains('Lewis Richardson')]
prices = prices[~prices.name.str.contains('Lewis Simper')]
prices = prices[~prices.name.str.contains('Lewis Wilson')]
prices = prices[~prices.name.str.contains('Lewis Young')]
prices = prices[~prices.name.str.contains('Lex-Tyger Lobinger')]
prices = prices[~prices.name.str.contains('Liam Armstrong')]
prices = prices[~prices.name.str.contains('Liam Cross')]
prices = prices[~prices.name.str.contains('Liam Delap')]
prices = prices[~prices.name.str.contains('Liam Edwards')]
prices = prices[~prices.name.str.contains('Liam Gibbs')]
prices = prices[~prices.name.str.contains('Liam Lightfoot')]
prices = prices[~prices.name.str.contains('Liam Millar')]
prices = prices[~prices.name.str.contains('Liam Waldock')]
prices = prices[~prices.name.str.contains('Logan Ketterer')]
prices = prices[~prices.name.str.contains('Logan Ross')]
prices = prices[~prices.name.str.contains('Loic Ritiere')]
prices = prices[~prices.name.str.contains('Lois Diony')]
prices = prices[~prices.name.str.contains('Lorenzo Colombo')]
prices = prices[~prices.name.str.contains('Lorenzo Del Pinto')]
prices = prices[~prices.name.str.contains('Lorenzo Faravelli')]
prices = prices[~prices.name.str.contains('Lorenzo Melgarejo')]
prices = prices[~prices.name.str.contains('Lorhan')]
prices = prices[~prices.name.str.contains('Louie Watson')]
prices = prices[~prices.name.str.contains('Louis Boyd')]
prices = prices[~prices.name.str.contains('Louis Britton')]
prices = prices[~prices.name.str.contains('Louis Jordan Beyer')]
prices = prices[~prices.name.str.contains('Louis Morison')]
prices = prices[~prices.name.str.contains('Louis Moult')]
prices = prices[~prices.name.str.contains('Lovro Bizjak')]
prices = prices[~prices.name.str.contains('Luan Garcia')]
prices = prices[~prices.name.str.contains('Luan Santos')]
prices = prices[~prices.name.str.contains('Luan Silva')]
prices = prices[~prices.name.str.contains('Lubomir Tupta')]
prices = prices[~prices.name.str.contains('Luca Andrada')]
prices = prices[~prices.name.str.contains('Luca Antei')]
prices = prices[~prices.name.str.contains('Luca Connell')]
prices = prices[~prices.name.str.contains('Luca Itter')]
prices = prices[~prices.name.str.contains('Luca Lewis')]
prices = prices[~prices.name.str.contains('Luca Mora')]
prices = prices[~prices.name.str.contains('Luca Orellano')]
prices = prices[~prices.name.str.contains('Luca Pfeiffer')]
prices = prices[~prices.name.str.contains('Luca Philipp')]
prices = prices[~prices.name.str.contains('Luca Plogmann')]
prices = prices[~prices.name.str.contains('Luca Sangalli')]
prices = prices[~prices.name.str.contains('Luca Schuler')]
prices = prices[~prices.name.str.contains('Luca Siligardi')]
prices = prices[~prices.name.str.contains('Luca Unbehaun')]
prices = prices[~prices.name.str.contains('Luca Waldschmidt')]
prices = prices[~prices.name.str.contains('Luca Zander')]
prices = prices[~prices.name.str.contains('Luca-Milan Zander')]
prices = prices[~prices.name.str.contains('Lucao')]
prices = prices[~prices.name.str.contains('LucaoLucaoattac')]
prices = prices[~prices.name.str.contains('Lucas')]
prices = prices[~prices.name.str.contains('Lucas Beltran')]
prices = prices[~prices.name.str.contains('Lucas Besozzi')]
prices = prices[~prices.name.str.contains('Lucas Brochero')]
prices = prices[~prices.name.str.contains('Lucas Chevalier')]
prices = prices[~prices.name.str.contains('Lucas Di Yorio')]
prices = prices[~prices.name.str.contains('Lucas Dias')]
prices = prices[~prices.name.str.contains('Lucas EstevesLucas Esteves')]
prices = prices[~prices.name.str.contains('Lucas FonsecaLucas Fonseca')]
prices = prices[~prices.name.str.contains('Lucas GabrielLucas Gabriel')]
prices = prices[~prices.name.str.contains('Lucas HalterLucas Halter')]
prices = prices[~prices.name.str.contains('Lucas Kal')]
prices = prices[~prices.name.str.contains('Lucas KalLucas Kal')]
prices = prices[~prices.name.str.contains('Lucas Kruspzky')]
prices = prices[~prices.name.str.contains('Lucas Lima')]
prices = prices[~prices.name.str.contains('Lucas LourenÃ§oLucas LourenÃ§o')]
prices = prices[~prices.name.str.contains('Lucas Mancinelli')]
prices = prices[~prices.name.str.contains('Lucas Menossi')]
prices = prices[~prices.name.str.contains('Lucas Orellano')]
prices = prices[~prices.name.str.contains('Lucas Perri')]
prices = prices[~prices.name.str.contains('Lucas Pirard')]
prices = prices[~prices.name.str.contains('Lucas Rios')]
prices = prices[~prices.name.str.contains('Lucas Robertone')]
prices = prices[~prices.name.str.contains('Lucas Suarez')]
prices = prices[~prices.name.str.contains('Lucas Tagliapietra')]
prices = prices[~prices.name.str.contains('Lucas Tomlinson')]
prices = prices[~prices.name.str.contains('Lucas VenutoLucas Venuto')]
prices = prices[~prices.name.str.contains('Lucas Viatri')]
prices = prices[~prices.name.str.contains('Lucas Villarruel')]
prices = prices[~prices.name.str.contains('LuccaLucca')]
prices = prices[~prices.name.str.contains('Luciano Cingolani')]
prices = prices[~prices.name.str.contains('Luciano Ferreyra')]
prices = prices[~prices.name.str.contains('Luciano Gondou')]
prices = prices[~prices.name.str.contains('Luciano Pons')]
prices = prices[~prices.name.str.contains('Luciano Recalde')]
prices = prices[~prices.name.str.contains('Luciano Squadrone')]
prices = prices[~prices.name.str.contains('Luciano Vietto')]
prices = prices[~prices.name.str.contains('Ludovic Butelle')]
prices = prices[~prices.name.str.contains('Ludvig Ohman')]
prices = prices[~prices.name.str.contains('Luis Adlard')]
prices = prices[~prices.name.str.contains('Luis Binks')]
prices = prices[~prices.name.str.contains('Luis Caicedo')]
prices = prices[~prices.name.str.contains('Luis Coordes')]
prices = prices[~prices.name.str.contains('Luis Felipe')]
prices = prices[~prices.name.str.contains('Luis Garnier')]
prices = prices[~prices.name.str.contains('Luis Leal')]
prices = prices[~prices.name.str.contains('Luis Manuel Orejuela')]
prices = prices[~prices.name.str.contains('Luis Sequeira')]
prices = prices[~prices.name.str.contains('Luis Unsain')]
prices = prices[~prices.name.str.contains('Luis Vazquez')]
prices = prices[~prices.name.str.contains('Luiz Otavio')]
prices = prices[~prices.name.str.contains('Luiz PhellypeLuiz Phellype')]
prices = prices[~prices.name.str.contains('Lukas Lerager')]
prices = prices[~prices.name.str.contains('Lukas Zima')]
prices = prices[~prices.name.str.contains('Luke Bolton')]
prices = prices[~prices.name.str.contains('Luke Brennan')]
prices = prices[~prices.name.str.contains('Luke Haines')]
prices = prices[~prices.name.str.contains('Luke Hallett')]
prices = prices[~prices.name.str.contains('Luke Matheson')]
prices = prices[~prices.name.str.contains('Luke McNally')]
prices = prices[~prices.name.str.contains('Mace Goodridge')]
prices = prices[~prices.name.str.contains('Maciej Dabrowski')]
prices = prices[~prices.name.str.contains('Mal Benning')]
prices = prices[~prices.name.str.contains('Malamine Doumbouya')]
prices = prices[~prices.name.str.contains('Malcolm Barcola')]
prices = prices[~prices.name.str.contains('Malcom Braida')]
prices = prices[~prices.name.str.contains('Mamor Niang')]
prices = prices[~prices.name.str.contains('ManchaMancha')]
prices = prices[~prices.name.str.contains('Mandela Egbo')]
prices = prices[~prices.name.str.contains('Mandela Keita')]
prices = prices[~prices.name.str.contains('Manny Monthe')]
prices = prices[~prices.name.str.contains('Manny Oyeleke')]
prices = prices[~prices.name.str.contains('ManoelManoel')]
prices = prices[~prices.name.str.contains('Manuel Cabit')]
prices = prices[~prices.name.str.contains('Manuel Capasso')]
prices = prices[~prices.name.str.contains('Manuel da Costa')]
prices = prices[~prices.name.str.contains('Manuel Llano')]
prices = prices[~prices.name.str.contains('Manuel Namora')]
prices = prices[~prices.name.str.contains('Manuel Nieto')]
prices = prices[~prices.name.str.contains('Manuel Panaro')]
prices = prices[~prices.name.str.contains('Manuel Perez')]
prices = prices[~prices.name.str.contains('Manuel Vicentini')]
prices = prices[~prices.name.str.contains('Marc Auge')]
prices = prices[~prices.name.str.contains('Marc Baro')]
prices = prices[~prices.name.str.contains('Marc Cardona')]
prices = prices[~prices.name.str.contains('Marc Guehi')]
prices = prices[~prices.name.str.contains('Marc Pugh')]
prices = prices[~prices.name.str.contains('Marcel Johnen')]
prices = prices[~prices.name.str.contains('Marcel Ritzmaier')]
prices = prices[~prices.name.str.contains('Marcel Schmelzer')]
prices = prices[~prices.name.str.contains('Marcello Trotta')]
prices = prices[~prices.name.str.contains('Marcelo Boeck')]
prices = prices[~prices.name.str.contains('Marcelo Estigarribia')]
prices = prices[~prices.name.str.contains('Marcelo Freites')]
prices = prices[~prices.name.str.contains('Marcelo Goiano')]
prices = prices[~prices.name.str.contains('Marcelo Toscano')]
prices = prices[~prices.name.str.contains('Marciano Aziz')]
prices = prices[~prices.name.str.contains('Marcin Bulka')]
prices = prices[~prices.name.str.contains('Marcio Mossoro')]
prices = prices[~prices.name.str.contains('Marco Alia')]
prices = prices[~prices.name.str.contains('Marco Aratore')]
prices = prices[~prices.name.str.contains('Marco Benassi')]
prices = prices[~prices.name.str.contains('Marco Bertini')]
prices = prices[~prices.name.str.contains('Marco Borgnino')]
prices = prices[~prices.name.str.contains('Marco Delgado')]
prices = prices[~prices.name.str.contains('Marco Di Cesare')]
prices = prices[~prices.name.str.contains('Marco Djuricin')]
prices = prices[~prices.name.str.contains('Marco Faraoni')]
prices = prices[~prices.name.str.contains('Marco Ludivin Majouga')]
prices = prices[~prices.name.str.contains('Marco Majouga')]
prices = prices[~prices.name.str.contains('Marcos Rouzies')]
prices = prices[~prices.name.str.contains('Marcos Sanchez')]
prices = prices[~prices.name.str.contains('Marcus Ferkranus')]
prices = prices[~prices.name.str.contains('Marcus McGuane')]
prices = prices[~prices.name.str.contains('Marian Shved')]
prices = prices[~prices.name.str.contains('Mariano Bareiro')]
prices = prices[~prices.name.str.contains('Mariano Diaz')]
prices = prices[~prices.name.str.contains('Mariano Vazquez')]
prices = prices[~prices.name.str.contains('Mario Situm')]
prices = prices[~prices.name.str.contains('Mario Suver')]
prices = prices[~prices.name.str.contains('Marius Liesegang')]
prices = prices[~prices.name.str.contains('Mark Gillespie')]
prices = prices[~prices.name.str.contains('Mark Kosznovszky')]
prices = prices[~prices.name.str.contains('Mark McGuinness')]
prices = prices[~prices.name.str.contains('Marlon Fossey')]
prices = prices[~prices.name.str.contains('Marlon Hairston')]
prices = prices[~prices.name.str.contains('Marlon Mustapha')]
prices = prices[~prices.name.str.contains('Marlon Santos')]
prices = prices[~prices.name.str.contains('Marouan Azarkan')]
prices = prices[~prices.name.str.contains('Marquinhos Calazans')]
prices = prices[~prices.name.str.contains('Martin Samuelsen')]
prices = prices[~prices.name.str.contains('Martin Skrtel')]
prices = prices[~prices.name.str.contains('Martin Smith')]
prices = prices[~prices.name.str.contains('Marvin Dewaele')]
prices = prices[~prices.name.str.contains('Marvin Obuz')]
prices = prices[~prices.name.str.contains('Marvin Park')]
prices = prices[~prices.name.str.contains('Marvin Tshibuabua')]
prices = prices[~prices.name.str.contains('Maryan Shved')]
prices = prices[~prices.name.str.contains('Mason Stajduhar')]
prices = prices[~prices.name.str.contains('Massimo Giamattei')]
prices = prices[~prices.name.str.contains('Mat Ryan')]
prices = prices[~prices.name.str.contains('Matej Kovar')]
prices = prices[~prices.name.str.contains('Matej Oravec')]
prices = prices[~prices.name.str.contains('Mateo Coronel')]
prices = prices[~prices.name.str.contains('Mateo Mamani')]
prices = prices[~prices.name.str.contains('Mateo Tanlongo')]
prices = prices[~prices.name.str.contains('Matheo Vroman')]
prices = prices[~prices.name.str.contains('Matheus Clemente')]
prices = prices[~prices.name.str.contains('Matheus Donelli')]
prices = prices[~prices.name.str.contains('Matheus Fernandes')]
prices = prices[~prices.name.str.contains('Matheus Oliveira')]
prices = prices[~prices.name.str.contains('Matheuzinho')]
prices = prices[~prices.name.str.contains('Mathew Hudson')]
prices = prices[~prices.name.str.contains('Mathew Ryan')]
prices = prices[~prices.name.str.contains('Mathias Suarez')]
prices = prices[~prices.name.str.contains('Mathias Wittek')]
prices = prices[~prices.name.str.contains('Mathieu Dossevi')]
prices = prices[~prices.name.str.contains('Mathys Saban')]
prices = prices[~prices.name.str.contains('Matias Belloso')]
prices = prices[~prices.name.str.contains('Matias Borgogno')]
prices = prices[~prices.name.str.contains('Matias Cordoba')]
prices = prices[~prices.name.str.contains('Matias De los Santos')]
prices = prices[~prices.name.str.contains('Matias de los Santos')]
prices = prices[~prices.name.str.contains('Matias Escudero')]
prices = prices[~prices.name.str.contains('Matias Fritzler')]
prices = prices[~prices.name.str.contains('Matias Garcia')]
prices = prices[~prices.name.str.contains('Matias Gomez')]
prices = prices[~prices.name.str.contains('Matias Gonzalez')]
prices = prices[~prices.name.str.contains('Matias Laba')]
prices = prices[~prices.name.str.contains('Matias Molina')]
prices = prices[~prices.name.str.contains('Matias Moya')]
prices = prices[~prices.name.str.contains('Matias Palavecino')]
prices = prices[~prices.name.str.contains('Matias Pellegrini')]
prices = prices[~prices.name.str.contains('Matias Pisano')]
prices = prices[~prices.name.str.contains('Matias Ramirez')]
prices = prices[~prices.name.str.contains('Matias Rodriguez')]
prices = prices[~prices.name.str.contains('Matias Sosa')]
prices = prices[~prices.name.str.contains('Matias Tagliamonte')]
prices = prices[~prices.name.str.contains('Matias Tissera')]
prices = prices[~prices.name.str.contains('Matias Villarreal')]
prices = prices[~prices.name.str.contains('Matis Carvalho')]
prices = prices[~prices.name.str.contains('Matisse Didden')]
prices = prices[~prices.name.str.contains('Matko Miljevic')]
prices = prices[~prices.name.str.contains('Matt Bersano')]
prices = prices[~prices.name.str.contains('Matt Clarke')]
prices = prices[~prices.name.str.contains('Matt Dolan')]
prices = prices[~prices.name.str.contains('Matt Freese')]
prices = prices[~prices.name.str.contains('Matt Gilks')]
prices = prices[~prices.name.str.contains('Matt Hundley')]
prices = prices[~prices.name.str.contains('Matt Lampson')]
prices = prices[~prices.name.str.contains('Matt Palmer')]
prices = prices[~prices.name.str.contains('Matt Warburton')]
prices = prices[~prices.name.str.contains('Matt Young')]
prices = prices[~prices.name.str.contains('Matteo Campagna')]
prices = prices[~prices.name.str.contains('Matteo Scozzarella')]
prices = prices[~prices.name.str.contains('Matthew Alexander')]
prices = prices[~prices.name.str.contains('Matthew Cash')]
prices = prices[~prices.name.str.contains('Matthew Clarke')]
prices = prices[~prices.name.str.contains('Matthew Dolan')]
prices = prices[~prices.name.str.contains('Matthew Foulds')]
prices = prices[~prices.name.str.contains('Matthew Gilks')]
prices = prices[~prices.name.str.contains('Matthew Millar')]
prices = prices[~prices.name.str.contains('Matthew Palmer')]
prices = prices[~prices.name.str.contains('Matthew Platt')]
prices = prices[~prices.name.str.contains('Matthew Pollock')]
prices = prices[~prices.name.str.contains('Matthew Smith')]
prices = prices[~prices.name.str.contains('Matthew Virtue-Thick')]
prices = prices[~prices.name.str.contains('Matthew Warburton')]
prices = prices[~prices.name.str.contains('Matthias Heiland')]
prices = prices[~prices.name.str.contains('Matthias Verreth')]
prices = prices[~prices.name.str.contains('Matthieu Dossevi')]
prices = prices[~prices.name.str.contains('Matthieu Saunier')]
prices = prices[~prices.name.str.contains('Matthis Abline')]
prices = prices[~prices.name.str.contains('Mattia Pagliuca')]
prices = prices[~prices.name.str.contains('Mattias Johansson')]
prices = prices[~prices.name.str.contains('Mattie Pollock')]
prices = prices[~prices.name.str.contains('Matty Cash')]
prices = prices[~prices.name.str.contains('Matty Foulds')]
prices = prices[~prices.name.str.contains('Matty Platt')]
prices = prices[~prices.name.str.contains('Matty Virtue')]
prices = prices[~prices.name.str.contains('Matty Wolfe')]
prices = prices[~prices.name.str.contains('Matz Sels')]
prices = prices[~prices.name.str.contains('Matz SelsMatz Sels')]
prices = prices[~prices.name.str.contains('Mauricio')]
prices = prices[~prices.name.str.contains('Mauricio Arboleda')]
prices = prices[~prices.name.str.contains('Mauricio Caranta')]
prices = prices[~prices.name.str.contains('Mauricio Cuero')]
prices = prices[~prices.name.str.contains('Mauricio Duarte')]
prices = prices[~prices.name.str.contains('Mauricio Guzman')]
prices = prices[~prices.name.str.contains('Mauricio Kozlinski')]
prices = prices[~prices.name.str.contains('MauricioMauricio')]
prices = prices[~prices.name.str.contains('Maurizio Pochettino')]
prices = prices[~prices.name.str.contains('Mauro Barraza')]
prices = prices[~prices.name.str.contains('Mauro Da Luz')]
prices = prices[~prices.name.str.contains('Mauro dos Santos')]
prices = prices[~prices.name.str.contains('Mauro Luna Diale')]
prices = prices[~prices.name.str.contains('Mauro Ortiz')]
prices = prices[~prices.name.str.contains('Mauro Osores')]
prices = prices[~prices.name.str.contains('Mauro Valiente')]
prices = prices[~prices.name.str.contains('Max Harris')]
prices = prices[~prices.name.str.contains('Max Hunt')]
prices = prices[~prices.name.str.contains('Max Kilman')]
prices = prices[~prices.name.str.contains('Maxi Gomez')]
prices = prices[~prices.name.str.contains('Maxi Moralez')]
prices = prices[~prices.name.str.contains('Maxim De Cuyper')]
prices = prices[~prices.name.str.contains('Maxim Deman')]
prices = prices[~prices.name.str.contains('Maxime Delanghe')]
prices = prices[~prices.name.str.contains('Maxime Wenssens')]
prices = prices[~prices.name.str.contains('Maximilian Kilman')]
prices = prices[~prices.name.str.contains('Maximiliano Caire')]
prices = prices[~prices.name.str.contains('Maximiliano Cavallotti')]
prices = prices[~prices.name.str.contains('Maximiliano Cuadra')]
prices = prices[~prices.name.str.contains('Maximiliano Gagliardo')]
prices = prices[~prices.name.str.contains('Maximiliano Gomez')]
prices = prices[~prices.name.str.contains('Maximiliano Luayza')]
prices = prices[~prices.name.str.contains('Maximiliano Moralez')]
prices = prices[~prices.name.str.contains('Maximiliano Rogoski')]
prices = prices[~prices.name.str.contains('MaykeMayke')]
prices = prices[~prices.name.str.contains('Mees Bakker')]
prices = prices[~prices.name.str.contains('Mehdi Abeid')]
prices = prices[~prices.name.str.contains('Mehmet Ali Ulaman')]
prices = prices[~prices.name.str.contains('Michael Chacon')]
prices = prices[~prices.name.str.contains('Michael Dawson')]
prices = prices[~prices.name.str.contains('Michael DeShields')]
prices = prices[~prices.name.str.contains('Michael Edwards')]
prices = prices[~prices.name.str.contains('Michael Fabrie')]
prices = prices[~prices.name.str.contains('Michael Hefele')]
prices = prices[~prices.name.str.contains('Michael Johnston')]
prices = prices[~prices.name.str.contains('Michael Jones')]
prices = prices[~prices.name.str.contains('Michael Kelly')]
prices = prices[~prices.name.str.contains('Michael Klass')]
prices = prices[~prices.name.str.contains('Michael Krmencik')]
prices = prices[~prices.name.str.contains('Michael Lennon')]
prices = prices[~prices.name.str.contains('Michael Nelson')]
prices = prices[~prices.name.str.contains('Michael Ruth')]
prices = prices[~prices.name.str.contains('Michael Santos')]
prices = prices[~prices.name.str.contains('Michael Schultz')]
prices = prices[~prices.name.str.contains('Michal Karbownik')]
prices = prices[~prices.name.str.contains('Michal Sadilek')]
prices = prices[~prices.name.str.contains('Michel Araujo')]
prices = prices[~prices.name.str.contains('Michel Balikwisha')]
prices = prices[~prices.name.str.contains('Michel Herrero')]
prices = prices[~prices.name.str.contains('Michel Macedo')]
prices = prices[~prices.name.str.contains('Michel-Ange Balikwisha')]
prices = prices[~prices.name.str.contains('Miguel Atienza')]
prices = prices[~prices.name.str.contains('Miguel Cifuentes')]
prices = prices[~prices.name.str.contains('Miguel Jacquet')]
prices = prices[~prices.name.str.contains('Miguel Van Damme')]
prices = prices[~prices.name.str.contains('MiguelMiguel')]
prices = prices[~prices.name.str.contains('Mihai-Alexandru Dobre')]
prices = prices[~prices.name.str.contains('Mihail Levashov')]
prices = prices[~prices.name.str.contains('Mikael Soisalo')]
prices = prices[~prices.name.str.contains('Mikalay Signevich')]
prices = prices[~prices.name.str.contains('Mike Hauptmeijer')]
prices = prices[~prices.name.str.contains('Mike Jones')]
prices = prices[~prices.name.str.contains('Mike Tresor')]
prices = prices[~prices.name.str.contains('Mike Tresor Ndayishimiye')]
prices = prices[~prices.name.str.contains('Mikhail Ageev')]
prices = prices[~prices.name.str.contains('Mikhail Levashov')]
prices = prices[~prices.name.str.contains('Mikkel Diskerud')]
prices = prices[~prices.name.str.contains('Milan De Mey')]
prices = prices[~prices.name.str.contains('Milan Iloski')]
prices = prices[~prices.name.str.contains('Milan Tucic')]
prices = prices[~prices.name.str.contains('Miles Mitchell-Nelson')]
prices = prices[~prices.name.str.contains('Mirkan Aydin')]
prices = prices[~prices.name.str.contains('Mitch Pinnock')]
prices = prices[~prices.name.str.contains('Mitchell Clark')]
prices = prices[~prices.name.str.contains('Mitchell Curry')]
prices = prices[~prices.name.str.contains('Mitchell Pinnock')]
prices = prices[~prices.name.str.contains('Mitchell Roberts')]
prices = prices[~prices.name.str.contains('Mitchy Ntelo')]
prices = prices[~prices.name.str.contains('Mitsuki Saito')]
prices = prices[~prices.name.str.contains('Mix Diskerud')]
prices = prices[~prices.name.str.contains('Mo AdamsMo Adams')]
prices = prices[~prices.name.str.contains('Modou Diagne')]
prices = prices[~prices.name.str.contains('Mohamed Ali-Cho')]
prices = prices[~prices.name.str.contains('Mohamed Badamosi')]
prices = prices[~prices.name.str.contains('Mohamed Bouchouari')]
prices = prices[~prices.name.str.contains('Mohamed Maouche')]
prices = prices[~prices.name.str.contains('Mohamed Rayhi')]
prices = prices[~prices.name.str.contains('Mohamed Taabouni')]
prices = prices[~prices.name.str.contains('Mohammed Sangare')]
prices = prices[~prices.name.str.contains('Moise Sakava')]
prices = prices[~prices.name.str.contains('Moises Caicedo')]
prices = prices[~prices.name.str.contains('Morgan Rogers')]
prices = prices[~prices.name.str.contains('Moritz Leitner')]
prices = prices[~prices.name.str.contains('Moritz RÃ¶mling')]
prices = prices[~prices.name.str.contains('Moritz Schulze')]
prices = prices[~prices.name.str.contains('Moritz-Broni Kwarteng')]
prices = prices[~prices.name.str.contains('Mostapha El Kabir')]
prices = prices[~prices.name.str.contains('Mouez Hassen')]
prices = prices[~prices.name.str.contains('Mousa Al-Tamari')]
prices = prices[~prices.name.str.contains('Mousa Suleiman')]
prices = prices[~prices.name.str.contains('Moussa Wague')]
prices = prices[~prices.name.str.contains('Moustafa Ava Dongo')]
prices = prices[~prices.name.str.contains('Mucahit Albayrak')]
prices = prices[~prices.name.str.contains('Mucahit Can Akcay')]
prices = prices[~prices.name.str.contains('Muhamed Badamosi')]
prices = prices[~prices.name.str.contains('Muhamed Besic')]
prices = prices[~prices.name.str.contains('Muhammadu Faal')]
prices = prices[~prices.name.str.contains('Muhammed Badamosi')]
prices = prices[~prices.name.str.contains('Muhammed Egemen Pehlivan')]
prices = prices[~prices.name.str.contains('Muhammed Emin Sarikaya')]
prices = prices[~prices.name.str.contains('Muhammed Eren Kiryolcu')]
prices = prices[~prices.name.str.contains('Muhammed Kerem Akturkoglu')]
prices = prices[~prices.name.str.contains('Muhammed Sarikaya')]
prices = prices[~prices.name.str.contains('Muhammed Uzun')]
prices = prices[~prices.name.str.contains('Muhayer Oktay')]
prices = prices[~prices.name.str.contains('Mukhammad Sultanov')]
prices = prices[~prices.name.str.contains('Munir Mohamedi')]
prices = prices[~prices.name.str.contains('Nano Mesa')]
prices = prices[~prices.name.str.contains('Naoufal Bannis')]
prices = prices[~prices.name.str.contains('Nassim Ahmed')]
prices = prices[~prices.name.str.contains('Nassim Chadli')]
prices = prices[~prices.name.str.contains('Nathan Baxter')]
prices = prices[~prices.name.str.contains('Nathan Broadhead')]
prices = prices[~prices.name.str.contains('Nathan Cameron')]
prices = prices[~prices.name.str.contains('Nathan Harriel')]
prices = prices[~prices.name.str.contains('Nathan Markelo')]
prices = prices[~prices.name.str.contains('Navajo Bakboord')]
prices = prices[~prices.name.str.contains('Nayair Tiknizyan')]
prices = prices[~prices.name.str.contains('Nayel Mehssatou')]
prices = prices[~prices.name.str.contains('Nazareno Solis')]
prices = prices[~prices.name.str.contains('NeiltonNeilton')]
prices = prices[~prices.name.str.contains('Neri Bandiera')]
prices = prices[~prices.name.str.contains('Neri Cardozo')]
prices = prices[~prices.name.str.contains('Nestor Ortigoza')]
prices = prices[~prices.name.str.contains('Neven Subotic')]
prices = prices[~prices.name.str.contains('Neyder Lozano')]
prices = prices[~prices.name.str.contains('Nezar S')]
prices = prices[~prices.name.str.contains('Niall Huggins')]
prices = prices[~prices.name.str.contains('Nicholas DePuy')]
prices = prices[~prices.name.str.contains('Nicholas Ioannou')]
prices = prices[~prices.name.str.contains('Nicholas Lawrence Anwan')]
prices = prices[~prices.name.str.contains('Nicholas Slonina')]
prices = prices[~prices.name.str.contains('Nick DePuy')]
prices = prices[~prices.name.str.contains('Nick Shinton')]
prices = prices[~prices.name.str.contains('Nicky Hernandez')]
prices = prices[~prices.name.str.contains('Nicky Law')]
prices = prices[~prices.name.str.contains('Niclas Thiede')]
prices = prices[~prices.name.str.contains('Nicolas Ferreyra')]
prices = prices[~prices.name.str.contains('Nicolas Figal')]
prices = prices[~prices.name.str.contains('Nicolas Franco')]
prices = prices[~prices.name.str.contains('Nicolas Gaitan')]
prices = prices[~prices.name.str.contains('Nicolas Gimenez')]
prices = prices[~prices.name.str.contains('Nicolas Lamendola')]
prices = prices[~prices.name.str.contains('Nicolas Linares')]
prices = prices[~prices.name.str.contains('Nicolas Mana')]
prices = prices[~prices.name.str.contains('Nicolas Mazzola')]
prices = prices[~prices.name.str.contains('Nicolas Messiniti')]
prices = prices[~prices.name.str.contains('Nicolas Morgantini')]
prices = prices[~prices.name.str.contains('Nicolas Romat')]
prices = prices[~prices.name.str.contains('Nicolas Romero')]
prices = prices[~prices.name.str.contains('Nicolas Schiappacasse')]
prices = prices[~prices.name.str.contains('Nicolas Thaller')]
prices = prices[~prices.name.str.contains('Nicolas Tie')]
prices = prices[~prices.name.str.contains('Nikell Touglo')]
prices = prices[~prices.name.str.contains('Nikita Belousov')]
prices = prices[~prices.name.str.contains('Nikita Chagrov')]
prices = prices[~prices.name.str.contains('Nikita Chicherin')]
prices = prices[~prices.name.str.contains('Nikita Iosifov')]
prices = prices[~prices.name.str.contains('Nikita Kalugin')]
prices = prices[~prices.name.str.contains('Nikita Repin')]
prices = prices[~prices.name.str.contains('Niklas Klinger')]
prices = prices[~prices.name.str.contains('Niklo Dailly')]
prices = prices[~prices.name.str.contains('Nikola Katic')]
prices = prices[~prices.name.str.contains('Nikolai Kemlein')]
prices = prices[~prices.name.str.contains('Nikolai Rasskazov')]
prices = prices[~prices.name.str.contains('Nikolai Rehnen')]
prices = prices[~prices.name.str.contains('Nikolai Zolotov')]
prices = prices[~prices.name.str.contains('Nikolaos Kainourgios')]
prices = prices[~prices.name.str.contains('Nikolaos Karelis')]
prices = prices[~prices.name.str.contains('Nikolaos Zografakis')]
prices = prices[~prices.name.str.contains('Nikolay Giorgobiani')]
prices = prices[~prices.name.str.contains('Nikolay Rasskazov')]
prices = prices[~prices.name.str.contains('Nikolay Titkov')]
prices = prices[~prices.name.str.contains('Nikolay Zlobin')]
prices = prices[~prices.name.str.contains('Nikos Kainourgios')]
prices = prices[~prices.name.str.contains('Nils De Wilde')]
prices = prices[~prices.name.str.contains('Nino ParaibaNino Paraiba')]
prices = prices[~prices.name.str.contains('Nkosi Burgess')]
prices = prices[~prices.name.str.contains('Nnamdi Chinonso Offor')]
prices = prices[~prices.name.str.contains('Noah Awuku')]
prices = prices[~prices.name.str.contains('Noah Baumann')]
prices = prices[~prices.name.str.contains('Noah Ohio')]
prices = prices[~prices.name.str.contains('Noah Smerdon')]
prices = prices[~prices.name.str.contains('Nunoo Sarpei')]
prices = prices[~prices.name.str.contains('Nya Kirby')]
prices = prices[~prices.name.str.contains('Oladapo Afolayan')]
prices = prices[~prices.name.str.contains('Oleg Baklov')]
prices = prices[~prices.name.str.contains('Oleg Danchenko')]
prices = prices[~prices.name.str.contains('Oleg Dzantiev')]
prices = prices[~prices.name.str.contains('Oleg Nikolaev')]
prices = prices[~prices.name.str.contains('Oleg Reabciuk')]
prices = prices[~prices.name.str.contains('Oleksandr Kaplienko')]
prices = prices[~prices.name.str.contains('Oli Finney')]
prices = prices[~prices.name.str.contains('Oliver Banks')]
prices = prices[~prices.name.str.contains('Oliver Coker')]
prices = prices[~prices.name.str.contains('Oliver Finney')]
prices = prices[~prices.name.str.contains('Oliver Lancashire')]
prices = prices[~prices.name.str.contains('Oliver Myny')]
prices = prices[~prices.name.str.contains('Oliver Norburn')]
prices = prices[~prices.name.str.contains('Oliver Turton')]
prices = prices[~prices.name.str.contains('Olivier Myny')]
prices = prices[~prices.name.str.contains('Olivier Thill')]
prices = prices[~prices.name.str.contains('Ollie Banks')]
prices = prices[~prices.name.str.contains('Ollie Battersby')]
prices = prices[~prices.name.str.contains('Ollie Norburn')]
prices = prices[~prices.name.str.contains('Ollie Turton')]
prices = prices[~prices.name.str.contains('Ollie Younger')]
prices = prices[~prices.name.str.contains('Olly Lancashire')]
prices = prices[~prices.name.str.contains('Olly Lee')]
prices = prices[~prices.name.str.contains('Olufela Olomola')]
prices = prices[~prices.name.str.contains('Omar Khailoti')]
prices = prices[~prices.name.str.contains('Omer Beyaz')]
prices = prices[~prices.name.str.contains('Omer Faruk Beyaz')]
prices = prices[~prices.name.str.contains('Omer Hanin')]
prices = prices[~prices.name.str.contains('Omer Sismanoglu')]
prices = prices[~prices.name.str.contains('Onur Kaya')]
prices = prices[~prices.name.str.contains('Onur KayaOnur Kaya')]
prices = prices[~prices.name.str.contains('Opi Edwards')]
prices = prices[~prices.name.str.contains('Oregan Ravet')]
prices = prices[~prices.name.str.contains('Orestis Karnezis')]
prices = prices[~prices.name.str.contains('Oriol Rey')]
prices = prices[~prices.name.str.contains('Osama Rashid')]
prices = prices[~prices.name.str.contains('Oscar Benitez')]
prices = prices[~prices.name.str.contains('Oscar Linner')]
prices = prices[~prices.name.str.contains('Oscar Ruiz')]
prices = prices[~prices.name.str.contains('Oscar SchÃ¶nfelder')]
prices = prices[~prices.name.str.contains('Oskar Buur')]
prices = prices[~prices.name.str.contains('Ouasim Bouy')]
prices = prices[~prices.name.str.contains('Oumar Niasse')]
prices = prices[~prices.name.str.contains('Pablo Aranda')]
prices = prices[~prices.name.str.contains('Pablo Cortizo')]
prices = prices[~prices.name.str.contains('Pablo Cuadra')]
prices = prices[~prices.name.str.contains('Pablo Insua')]
prices = prices[~prices.name.str.contains('Pablo Rosales')]
prices = prices[~prices.name.str.contains('Pablo Sabbag')]
prices = prices[~prices.name.str.contains('Pablo Velazquez')]
prices = prices[~prices.name.str.contains('Paddy Madden')]
prices = prices[~prices.name.str.contains('Paddy McNair')]
prices = prices[~prices.name.str.contains('Panagiotis Armenakas')]
prices = prices[~prices.name.str.contains('Panos Armenakas')]
prices = prices[~prices.name.str.contains('Pantelis Chatzidiakos')]
prices = prices[~prices.name.str.contains('Pantelis Hatzidiakos')]
prices = prices[~prices.name.str.contains('Paolo Bartolomei')]
prices = prices[~prices.name.str.contains('Paolo Guerrero')]
prices = prices[~prices.name.str.contains('Paolo Hurtado')]
prices = prices[~prices.name.str.contains('Parker Siegfried')]
prices = prices[~prices.name.str.contains('Pascal Steinwender')]
prices = prices[~prices.name.str.contains('Patric Klandt')]
prices = prices[~prices.name.str.contains('Patricio Cucchi')]
prices = prices[~prices.name.str.contains('Patricio Matricardi')]
prices = prices[~prices.name.str.contains('Patricio Ostachuk')]
prices = prices[~prices.name.str.contains('Patricio Toranzo')]
prices = prices[~prices.name.str.contains('Patrick Jones')]
prices = prices[~prices.name.str.contains('Patrick McNair')]
prices = prices[~prices.name.str.contains('Paul Grave')]
prices = prices[~prices.name.str.contains('Paul-Philipp Besong')]
prices = prices[~prices.name.str.contains('Pavel Kaloshin')]
prices = prices[~prices.name.str.contains('Pavel Karasev')]
prices = prices[~prices.name.str.contains('Pedro Lucas')]
prices = prices[~prices.name.str.contains('Pedro Ojeda')]
prices = prices[~prices.name.str.contains('Pedro Perotti')]
prices = prices[~prices.name.str.contains('Pepe Bonet')]
prices = prices[~prices.name.str.contains('Per Kristian Bratveit')]
prices = prices[~prices.name.str.contains('Peru Nolaskoain')]
prices = prices[~prices.name.str.contains('Peter Urminsky')]
prices = prices[~prices.name.str.contains('Petrus Boumal')]
prices = prices[~prices.name.str.contains('Phil Jones')]
prices = prices[~prices.name.str.contains('Philip Gameli Awuku')]
prices = prices[~prices.name.str.contains('Philip Mayaka')]
prices = prices[~prices.name.str.contains('Philip Turpitz')]
prices = prices[~prices.name.str.contains('Philip Yeboah Ankrah')]
prices = prices[~prices.name.str.contains('Philipp Heerwagen')]
prices = prices[~prices.name.str.contains('Philipp Mwene')]
prices = prices[~prices.name.str.contains('Philipp Tschauner')]
prices = prices[~prices.name.str.contains('Philippe Sandler')]
prices = prices[~prices.name.str.contains('Phillipp Mwene')]
prices = prices[~prices.name.str.contains('Pier Graziano Gori')]
prices = prices[~prices.name.str.contains('Piero Hincapie')]
prices = prices[~prices.name.str.contains('Pierre-Yves Polomat')]
prices = prices[~prices.name.str.contains('PJ Morrison')]
prices = prices[~prices.name.str.contains('Pol Garcia')]
prices = prices[~prices.name.str.contains('Racine Coly')]
prices = prices[~prices.name.str.contains('Rafa De Vita')]
prices = prices[~prices.name.str.contains('Rafael Borre')]
prices = prices[~prices.name.str.contains('Rafael Cabral')]
prices = prices[~prices.name.str.contains('Rafael Ferrario')]
prices = prices[~prices.name.str.contains('Rafael Ribeiro')]
prices = prices[~prices.name.str.contains('Rafael Sangiovani')]
prices = prices[~prices.name.str.contains('Ramiro')]
prices = prices[~prices.name.str.contains('Ramiro Enrique')]
prices = prices[~prices.name.str.contains('Ramiro Luna')]
prices = prices[~prices.name.str.contains('Ramiro Macagno')]
prices = prices[~prices.name.str.contains('Ramon Arias')]
prices = prices[~prices.name.str.contains('Ramon Azeez')]
prices = prices[~prices.name.str.contains('Ramon Folch')]
prices = prices[~prices.name.str.contains('Ramon ten Hove')]
prices = prices[~prices.name.str.contains('Raul Aguilera')]
prices = prices[~prices.name.str.contains('Raul Carnero')]
prices = prices[~prices.name.str.contains('Raul Lozano')]
prices = prices[~prices.name.str.contains('Raul Moro')]
prices = prices[~prices.name.str.contains('Raul Navas')]
prices = prices[~prices.name.str.contains('Raulzinho')]
prices = prices[~prices.name.str.contains('Rayan Philippe')]
prices = prices[~prices.name.str.contains('Rayan Raveloson')]
prices = prices[~prices.name.str.contains('Rayan Souici')]
prices = prices[~prices.name.str.contains('Rayhaan Tulloch')]
prices = prices[~prices.name.str.contains('Raynner Silva')]
prices = prices[~prices.name.str.contains('Reagan Ogle')]
prices = prices[~prices.name.str.contains('Recep Niyaz')]
prices = prices[~prices.name.str.contains('Reda Akbib')]
prices = prices[~prices.name.str.contains('Reda Khadra')]
prices = prices[~prices.name.str.contains('Reece Grego-Cox')]
prices = prices[~prices.name.str.contains('Reece McAlear')]
prices = prices[~prices.name.str.contains('Regan Slater')]
prices = prices[~prices.name.str.contains('Renat Dadashov')]
prices = prices[~prices.name.str.contains('Rene Gilmartin')]
prices = prices[~prices.name.str.contains('Renzo Bacchia')]
prices = prices[~prices.name.str.contains('Renzo Giampaoli')]
prices = prices[~prices.name.str.contains('Renzo Saravia')]
prices = prices[~prices.name.str.contains('Renzo Tesuri')]
prices = prices[~prices.name.str.contains('Reuben Reid')]
prices = prices[~prices.name.str.contains('Richard Prieto')]
prices = prices[~prices.name.str.contains('Richard Rodrigues')]
prices = prices[~prices.name.str.contains('Richard Towell')]
prices = prices[~prices.name.str.contains('Rida Zouhir')]
prices = prices[~prices.name.str.contains('Rik Vercauteren')]
prices = prices[~prices.name.str.contains('Rio Hope-Gund')]
prices = prices[~prices.name.str.contains('Robert Apter')]
prices = prices[~prices.name.str.contains('Robert Arboleda')]
prices = prices[~prices.name.str.contains('Robert Castellanos')]
prices = prices[~prices.name.str.contains('Robert Dickie')]
prices = prices[~prices.name.str.contains('Robert Hall')]
prices = prices[~prices.name.str.contains('Robert Piris da Motta')]
prices = prices[~prices.name.str.contains('Robert Voloder')]
prices = prices[~prices.name.str.contains('Robert Zwinkels')]
prices = prices[~prices.name.str.contains('Roberto Bochi')]
prices = prices[~prices.name.str.contains('Roberto Correa')]
prices = prices[~prices.name.str.contains('Roberto Jimenez')]
prices = prices[~prices.name.str.contains('Roberto Navarro')]
prices = prices[~prices.name.str.contains('Robin Jalving')]
prices = prices[~prices.name.str.contains('Rodolfo Cardoso')]
prices = prices[~prices.name.str.contains('RodolfoRodolfo')]
prices = prices[~prices.name.str.contains('Rodri Sanchez')]
prices = prices[~prices.name.str.contains('Rodrigo DouradoRodrigo Dourado')]
prices = prices[~prices.name.str.contains('Rogerio Santos')]
prices = prices[~prices.name.str.contains('Romain Amalfitano')]
prices = prices[~prices.name.str.contains('Roman Martinez')]
prices = prices[~prices.name.str.contains('Romano Postema')]
prices = prices[~prices.name.str.contains('Ross Doohan')]
prices = prices[~prices.name.str.contains('Ross Graham')]
prices = prices[~prices.name.str.contains('Ross MacIver')]
prices = prices[~prices.name.str.contains('Ross Munro')]
prices = prices[~prices.name.str.contains('Ross Sinclair')]
prices = prices[~prices.name.str.contains('Ruben Botta')]
prices = prices[~prices.name.str.contains('Ruben Del Campo')]
prices = prices[~prices.name.str.contains('Ruben Fonseca')]
prices = prices[~prices.name.str.contains('Ruben Pardo')]
prices = prices[~prices.name.str.contains('Rubin Wilson')]
prices = prices[~prices.name.str.contains('Rumarn Burrell')]
prices = prices[~prices.name.str.contains('Ruwen Werthmuller')]
prices = prices[~prices.name.str.contains('Ryan Campbell-Gordon')]
prices = prices[~prices.name.str.contains('Ryan Cassidy')]
prices = prices[~prices.name.str.contains('Ryan Cooney')]
prices = prices[~prices.name.str.contains('Ryan Duncan')]
prices = prices[~prices.name.str.contains('Ryan Edmondson')]
prices = prices[~prices.name.str.contains('Ryan Giles')]
prices = prices[~prices.name.str.contains('Ryan Hillier')]
prices = prices[~prices.name.str.contains('Ryan Jones')]
prices = prices[~prices.name.str.contains('Ryan Longman')]
prices = prices[~prices.name.str.contains('Ryan Meara')]
prices = prices[~prices.name.str.contains('Ryan Rydel')]
prices = prices[~prices.name.str.contains('Ryan Scully')]
prices = prices[~prices.name.str.contains('Ryan Stirk')]
prices = prices[~prices.name.str.contains('Saba Sazonov')]
prices = prices[~prices.name.str.contains('Sabit Abdulai')]
prices = prices[~prices.name.str.contains('Sacha Bastien')]
prices = prices[~prices.name.str.contains('Sacha Delaye')]
prices = prices[~prices.name.str.contains('Said Ahmed Said')]
prices = prices[~prices.name.str.contains('Said Akhmaev')]
prices = prices[~prices.name.str.contains('Said Bakari')]
prices = prices[~prices.name.str.contains('Said-Ali Akhmaev')]
prices = prices[~prices.name.str.contains('Saidou Sow')]
prices = prices[~prices.name.str.contains('Saif-Eddine Khaoui')]
prices = prices[~prices.name.str.contains('Saikou Janneh')]
prices = prices[~prices.name.str.contains('Salim Ben Seghir')]
prices = prices[~prices.name.str.contains('Saliou Sane')]
prices = prices[~prices.name.str.contains('Sam Bell')]
prices = prices[~prices.name.str.contains('Sam Cartwright')]
prices = prices[~prices.name.str.contains('Sam Folarin')]
prices = prices[~prices.name.str.contains('Sam Junqua')]
prices = prices[~prices.name.str.contains('Sam McQueen')]
prices = prices[~prices.name.str.contains('Sam Morsy')]
prices = prices[~prices.name.str.contains('Sam Pearson')]
prices = prices[~prices.name.str.contains('Sam Sargeant')]
prices = prices[~prices.name.str.contains('Samed Onur')]
prices = prices[~prices.name.str.contains('Sami Gokhan Altiparmak')]
prices = prices[~prices.name.str.contains('Samy Bourard')]
prices = prices[~prices.name.str.contains('Samy Kehli')]
prices = prices[~prices.name.str.contains('Samy Mmaee')]
prices = prices[~prices.name.str.contains('Samy Morsy')]
prices = prices[~prices.name.str.contains('Santiago Caseres')]
prices = prices[~prices.name.str.contains('Santiago Colombatto')]
prices = prices[~prices.name.str.contains('Santiago Gallucci')]
prices = prices[~prices.name.str.contains('Santiago Garcia')]
prices = prices[~prices.name.str.contains('Santiago Rodriguez')]
prices = prices[~prices.name.str.contains('Santiago Silva')]
prices = prices[~prices.name.str.contains('Santiago Simon')]
prices = prices[~prices.name.str.contains('Santiago Trellez')]
prices = prices[~prices.name.str.contains('Scott Caldwell')]
prices = prices[~prices.name.str.contains('Scott Fox')]
prices = prices[~prices.name.str.contains('Scott Moloney')]
prices = prices[~prices.name.str.contains('Scott Pollock')]
prices = prices[~prices.name.str.contains('Scott Robertson')]
prices = prices[~prices.name.str.contains('Scott Smith')]
prices = prices[~prices.name.str.contains('Sebastian Anderson')]
prices = prices[~prices.name.str.contains('Sebastian Assis')]
prices = prices[~prices.name.str.contains('Sebastian Blanco')]
prices = prices[~prices.name.str.contains('Sebastian Breza')]
prices = prices[~prices.name.str.contains('Sebastian Dubarbier')]
prices = prices[~prices.name.str.contains('Sebastian Meza')]
prices = prices[~prices.name.str.contains('Sebastian Ramirez')]
prices = prices[~prices.name.str.contains('Sebastian Sosa')]
prices = prices[~prices.name.str.contains('Sebastian Soto')]
prices = prices[~prices.name.str.contains('Sebastien Thill')]
prices = prices[~prices.name.str.contains('Sega Coulibaly')]
prices = prices[~prices.name.str.contains('Sekou Diawara')]
prices = prices[~prices.name.str.contains('Sepe Elye Wahi')]
prices = prices[~prices.name.str.contains('Serdar Poyraz')]
prices = prices[~prices.name.str.contains('Serdar Saatci')]
prices = prices[~prices.name.str.contains('Sergei Borodin')]
prices = prices[~prices.name.str.contains('Sergei Makarov')]
prices = prices[~prices.name.str.contains('Sergei Ryzhikov')]
prices = prices[~prices.name.str.contains('Sergey Chibisov')]
prices = prices[~prices.name.str.contains('Sergey Makarov')]
prices = prices[~prices.name.str.contains('Sergi Garcia')]
prices = prices[~prices.name.str.contains('Sergio Barcia')]
prices = prices[~prices.name.str.contains('Sergio Carreira')]
prices = prices[~prices.name.str.contains('Sergio Cubero')]
prices = prices[~prices.name.str.contains('Sergio Gonzalez')]
prices = prices[~prices.name.str.contains('Sergio Ojeda')]
prices = prices[~prices.name.str.contains('Sergio Quiroga')]
prices = prices[~prices.name.str.contains('Sergio Romero')]
prices = prices[~prices.name.str.contains('Sergio Vittor')]
prices = prices[~prices.name.str.contains('Sergiy Rybalka')]
prices = prices[~prices.name.str.contains('Seung-Ho Paik')]
prices = prices[~prices.name.str.contains('Seung-woo Lee')]
prices = prices[~prices.name.str.contains('Shay Logan')]
prices = prices[~prices.name.str.contains('Shilow Tracey')]
prices = prices[~prices.name.str.contains('Shola Shoretire')]
prices = prices[~prices.name.str.contains('Shoya Nakajima')]
prices = prices[~prices.name.str.contains('Shurandy Sambo')]
prices = prices[~prices.name.str.contains('Sibiry Keita')]
prices = prices[~prices.name.str.contains('Siebren Lathouwers')]
prices = prices[~prices.name.str.contains('Silas Gnaka')]
prices = prices[~prices.name.str.contains('Silas Katompa Mvumpa')]
prices = prices[~prices.name.str.contains('Silas Wamangituka')]
prices = prices[~prices.name.str.contains('Silas Zehnder')]
prices = prices[~prices.name.str.contains('Silvinho Esajas')]
prices = prices[~prices.name.str.contains('Simon Deli')]
prices = prices[~prices.name.str.contains('Simon Moore')]
prices = prices[~prices.name.str.contains('Simon Ngapandouetnbu')]
prices = prices[~prices.name.str.contains('Simon Rhein')]
prices = prices[~prices.name.str.contains('Simone Aresti')]
prices = prices[~prices.name.str.contains('Simone Edera')]
prices = prices[~prices.name.str.contains('Simone Farelli')]
prices = prices[~prices.name.str.contains('Simone Rabbi')]
prices = prices[~prices.name.str.contains('Siro Rosane')]
prices = prices[~prices.name.str.contains('Sofian Chakla')]
prices = prices[~prices.name.str.contains('Sofiane Bouzian')]
prices = prices[~prices.name.str.contains('Soufiane Chakla')]
prices = prices[~prices.name.str.contains('Souleymane Aw')]
prices = prices[~prices.name.str.contains('Spencer Richey')]
prices = prices[~prices.name.str.contains('Stan van Dijck')]
prices = prices[~prices.name.str.contains('Stanislav Ivanov')]
prices = prices[~prices.name.str.contains('Stanislav Krapukhin')]
prices = prices[~prices.name.str.contains('Stanislav Shopov')]
prices = prices[~prices.name.str.contains('Steeve-Mike Eboa Ebongue')]
prices = prices[~prices.name.str.contains('Stefan Drljaca')]
prices = prices[~prices.name.str.contains('Stefan Milosevic')]
prices = prices[~prices.name.str.contains('Stefan van der Lei')]
prices = prices[~prices.name.str.contains('Stefano Callegari')]
prices = prices[~prices.name.str.contains('Stepan Oganesyan')]
prices = prices[~prices.name.str.contains('Stephane Paul Keller')]
prices = prices[~prices.name.str.contains('Stephen Duke-McKenna')]
prices = prices[~prices.name.str.contains('Stephen Henderson')]
prices = prices[~prices.name.str.contains('Stephen Kelly')]
prices = prices[~prices.name.str.contains('Stephen Mallan')]
prices = prices[~prices.name.str.contains('Steve Birnbaum')]
prices = prices[~prices.name.str.contains('Steven Beitashour')]
prices = prices[~prices.name.str.contains('Steven Defour')]
prices = prices[~prices.name.str.contains('Stiven Mendoza')]
prices = prices[~prices.name.str.contains('Stiven Plaza')]
prices = prices[~prices.name.str.contains('Suk-jae Lee')]
prices = prices[~prices.name.str.contains('Suleyman Cebeci')]
prices = prices[~prices.name.str.contains('Taylor Harwood-Bellis')]
prices = prices[~prices.name.str.contains('Taylor Richards')]
prices = prices[~prices.name.str.contains('Taylor Seymour')]
prices = prices[~prices.name.str.contains('Teddy Bartouche')]
prices = prices[~prices.name.str.contains('Teddy Boulhendi')]
prices = prices[~prices.name.str.contains('Teden Mengi')]
prices = prices[~prices.name.str.contains('Tennai Watson')]
prices = prices[~prices.name.str.contains('Thakgalo Leshabela')]
prices = prices[~prices.name.str.contains('TharlisTharlis')]
prices = prices[~prices.name.str.contains('Theo Corbeanu')]
prices = prices[~prices.name.str.contains('Theo Leoni')]
prices = prices[~prices.name.str.contains('Thiago Lopes')]
prices = prices[~prices.name.str.contains('Thiago Maia')]
prices = prices[~prices.name.str.contains('Thiago Santana')]
prices = prices[~prices.name.str.contains('Thibang Phete')]
prices = prices[~prices.name.str.contains('Thibault Tamas')]
prices = prices[~prices.name.str.contains('Thomas Allan')]
prices = prices[~prices.name.str.contains('Thomas Basila')]
prices = prices[~prices.name.str.contains('Thomas Callens')]
prices = prices[~prices.name.str.contains('Thomas Eastman')]
prices = prices[~prices.name.str.contains('Thomas Edwards')]
prices = prices[~prices.name.str.contains('Thomas Hamer')]
prices = prices[~prices.name.str.contains('Thomas Hasal')]
prices = prices[~prices.name.str.contains('Thomas McGill')]
prices = prices[~prices.name.str.contains('Thomas Mehew')]
prices = prices[~prices.name.str.contains('Thomas O')]
prices = prices[~prices.name.str.contains('Thomas Poll')]
prices = prices[~prices.name.str.contains('Thomas Roberts')]
prices = prices[~prices.name.str.contains('Thomas Stagg')]
prices = prices[~prices.name.str.contains('Thomas Toure')]
prices = prices[~prices.name.str.contains('Thomas Williams')]
prices = prices[~prices.name.str.contains('Tiago Banega')]
prices = prices[~prices.name.str.contains('Tiago Ilori')]
prices = prices[~prices.name.str.contains('Tiago Lopes')]
prices = prices[~prices.name.str.contains('Tiago Palacios')]
prices = prices[~prices.name.str.contains('Tibet Oniz')]
prices = prices[~prices.name.str.contains('Tiernan Parker')]
prices = prices[~prices.name.str.contains('Tim Coremans')]
prices = prices[~prices.name.str.contains('Tim Hall')]
prices = prices[~prices.name.str.contains('Tim Lemperle')]
prices = prices[~prices.name.str.contains('Tim Maciejewski')]
prices = prices[~prices.name.str.contains('Timothe Nkada')]
prices = prices[~prices.name.str.contains('Timothy Eyoma')]
prices = prices[~prices.name.str.contains('Titas Krapikas')]
prices = prices[~prices.name.str.contains('Tivonge Rushesha')]
prices = prices[~prices.name.str.contains('Tobe Leysen')]
prices = prices[~prices.name.str.contains('Tobias Raschl')]
prices = prices[~prices.name.str.contains('Tobias Zarate')]
prices = prices[~prices.name.str.contains('Toby Bancroft')]
prices = prices[~prices.name.str.contains('Tolaji Bola')]
prices = prices[~prices.name.str.contains('Tolga Topaloglu')]
prices = prices[~prices.name.str.contains('Tom Bloxham')]
prices = prices[~prices.name.str.contains('Tom Chamberlain')]
prices = prices[~prices.name.str.contains('Tom Collins')]
prices = prices[~prices.name.str.contains('Tom Eastman')]
prices = prices[~prices.name.str.contains('Tom Edwards')]
prices = prices[~prices.name.str.contains('Tom Heaton')]
prices = prices[~prices.name.str.contains('Tom Hendriks')]
prices = prices[~prices.name.str.contains('Tom Mickel')]
prices = prices[~prices.name.str.contains('Tom ScottTom Scott')]
prices = prices[~prices.name.str.contains('Tom Van Hyfte')]
prices = prices[~prices.name.str.contains('Tom Weilandt')]
prices = prices[~prices.name.str.contains('Tom White')]
prices = prices[~prices.name.str.contains('Tom Wilson')]
prices = prices[~prices.name.str.contains('Tomas Andrade')]
prices = prices[~prices.name.str.contains('Tomas Cardona')]
prices = prices[~prices.name.str.contains('Tomas Cavanagh')]
prices = prices[~prices.name.str.contains('Tomas Durso')]
prices = prices[~prices.name.str.contains('Tomas Escalante')]
prices = prices[~prices.name.str.contains('Tomas Fernandez')]
prices = prices[~prices.name.str.contains('Tomas Koubek')]
prices = prices[~prices.name.str.contains('Tomas Marchiori')]
prices = prices[~prices.name.str.contains('Tomas Martinez')]
prices = prices[~prices.name.str.contains('Tomas Ortiz')]
prices = prices[~prices.name.str.contains('Tomas Romero')]
prices = prices[~prices.name.str.contains('Tomas Rukas')]
prices = prices[~prices.name.str.contains('Tomas Sandoval')]
prices = prices[~prices.name.str.contains('Tomas Silva')]
prices = prices[~prices.name.str.contains('Tommy Conway')]
prices = prices[~prices.name.str.contains('Tommy Elphick')]
prices = prices[~prices.name.str.contains('Toni Datkovic')]
prices = prices[~prices.name.str.contains('Toni Sunjic')]
prices = prices[~prices.name.str.contains('Tony Alfaro')]
prices = prices[~prices.name.str.contains('Torben Musel')]
prices = prices[~prices.name.str.contains('Toto Nsiala')]
prices = prices[~prices.name.str.contains('Trae Coyle')]
prices = prices[~prices.name.str.contains('Tristan Dingome')]
prices = prices[~prices.name.str.contains('Trova Boni')]
prices = prices[~prices.name.str.contains('Troy Parrott')]
prices = prices[~prices.name.str.contains('Tunde Owolabi')]
prices = prices[~prices.name.str.contains('Tunji Akinola')]
prices = prices[~prices.name.str.contains('Tyler Burey')]
prices = prices[~prices.name.str.contains('Tyler Cordner')]
prices = prices[~prices.name.str.contains('Tyler Freeman')]
prices = prices[~prices.name.str.contains('Tyler French')]
prices = prices[~prices.name.str.contains('Tyler Magloire')]
prices = prices[~prices.name.str.contains('Tyler Smith')]
prices = prices[~prices.name.str.contains('Tyler Wolff')]
prices = prices[~prices.name.str.contains('Tyreik Wright')]
prices = prices[~prices.name.str.contains('Tyrese Fornah')]
prices = prices[~prices.name.str.contains('Tyrese Francois')]
prices = prices[~prices.name.str.contains('Tyrese Omotoye')]
prices = prices[~prices.name.str.contains('Tyrone Marsh')]
prices = prices[~prices.name.str.contains('Tyrone Tormin')]
prices = prices[~prices.name.str.contains('Tyreece Simpson')]
prices = prices[~prices.name.str.contains('Ui-jo Hwang')]
prices = prices[~prices.name.str.contains('Uillian Correia')]
prices = prices[~prices.name.str.contains('Ulises Segura')]
prices = prices[~prices.name.str.contains('Umut Bozok')]
prices = prices[~prices.name.str.contains('Unai Arietaleanizbeaskoa')]
prices = prices[~prices.name.str.contains('Unai Dufur')]
prices = prices[~prices.name.str.contains('Uri Rosell')]
prices = prices[~prices.name.str.contains('Uriel Ramirez')]
prices = prices[~prices.name.str.contains('Urko Gonzalez')]
prices = prices[~prices.name.str.contains('Valentin Larralde')]
prices = prices[~prices.name.str.contains('Valentin Mihaila')]
prices = prices[~prices.name.str.contains('Valeriu Ciuperca')]
prices = prices[~prices.name.str.contains('Varazdat Haroyan')]
prices = prices[~prices.name.str.contains('Vedad Ibisevic')]
prices = prices[~prices.name.str.contains('Veron Parkes')]
prices = prices[~prices.name.str.contains('Vic Chambaere')]
prices = prices[~prices.name.str.contains('Vicente Esquerdo')]
prices = prices[~prices.name.str.contains('Victor Bobsin')]
prices = prices[~prices.name.str.contains('Victor Camarasa')]
prices = prices[~prices.name.str.contains('Victor Chust')]
prices = prices[~prices.name.str.contains('Victor Jensen')]
prices = prices[~prices.name.str.contains('Victor Mollejo')]
prices = prices[~prices.name.str.contains('Victor Ramis')]
prices = prices[~prices.name.str.contains('Vincent Muller')]
prices = prices[~prices.name.str.contains('Vincent Schippers')]
prices = prices[~prices.name.str.contains('Vincenzo Millico')]
prices = prices[~prices.name.str.contains('Vinicius Zanocelo')]
prices = prices[~prices.name.str.contains('Virgil Gomis')]
prices = prices[~prices.name.str.contains('Virgiliu Postolachi')]
prices = prices[~prices.name.str.contains('Vitor Bueno')]
prices = prices[~prices.name.str.contains('Vittorio Parigini')]
prices = prices[~prices.name.str.contains('VladimirVladimir')]
prices = prices[~prices.name.str.contains('Vladislav Cherny')]
prices = prices[~prices.name.str.contains('Vladislav Torop')]
prices = prices[~prices.name.str.contains('Vlasdilav Yakovlev')]
prices = prices[~prices.name.str.contains('Vukasin Jovanovic')]
prices = prices[~prices.name.str.contains('Vyacheslav Litvinov')]
prices = prices[~prices.name.str.contains('Walter Kannemann')]
prices = prices[~prices.name.str.contains('Walter Montoya')]
prices = prices[~prices.name.str.contains('Welinton Junior')]
prices = prices[~prices.name.str.contains('Wenderson Galeno')]
prices = prices[~prices.name.str.contains('Wes Foderingham')]
prices = prices[~prices.name.str.contains('Wes McDonald')]
prices = prices[~prices.name.str.contains('Wesley McDonald')]
prices = prices[~prices.name.str.contains('Wesley Ribeiro')]
prices = prices[~prices.name.str.contains('Wilfredo Rivera')]
prices = prices[~prices.name.str.contains('Will Bapaga')]
prices = prices[~prices.name.str.contains('Will Collar')]
prices = prices[~prices.name.str.contains('Will Dennis')]
prices = prices[~prices.name.str.contains('Will Grigg')]
prices = prices[~prices.name.str.contains('Will Huffer')]
prices = prices[~prices.name.str.contains('Will Pulisic')]
prices = prices[~prices.name.str.contains('Will Smallbone')]
prices = prices[~prices.name.str.contains('Will Sutton')]
prices = prices[~prices.name.str.contains('Will Swan')]
prices = prices[~prices.name.str.contains('William Evans')]
prices = prices[~prices.name.str.contains('William Fish')]
prices = prices[~prices.name.str.contains('William Forrester')]
prices = prices[~prices.name.str.contains('William Grigg')]
prices = prices[~prices.name.str.contains('William Hondermarck')]
prices = prices[~prices.name.str.contains('Wouter Biebauw')]
prices = prices[~prices.name.str.contains('Wouter George')]
prices = prices[~prices.name.str.contains('Yanis Ammour')]
prices = prices[~prices.name.str.contains('Yanis Lhery')]
prices = prices[~prices.name.str.contains('Yanis Saidani')]
prices = prices[~prices.name.str.contains('Yann Bodiger')]
prices = prices[~prices.name.str.contains('Yann M')]
prices = prices[~prices.name.str.contains('Yannick Brugger')]
prices = prices[~prices.name.str.contains('YanYan')]
prices = prices[~prices.name.str.contains('Yasin Davus')]
prices = prices[~prices.name.str.contains('Yasin Oztekin')]
prices = prices[~prices.name.str.contains('Yassin Fekir')]
prices = prices[~prices.name.str.contains('Yassin Oukili')]
prices = prices[~prices.name.str.contains('Yassine Bounou')]
prices = prices[~prices.name.str.contains('Yavuz Bugra Boyar')]
prices = prices[~prices.name.str.contains('Yavuz Ulas Genc')]
prices = prices[~prices.name.str.contains('Yaya Toure')]
prices = prices[~prices.name.str.contains('Yayah Kallon')]
prices = prices[~prices.name.str.contains('Yeiler Goez')]
prices = prices[~prices.name.str.contains('Yeison Gordillo')]
prices = prices[~prices.name.str.contains('Yildirim Mert Ã‡etin')]
prices = prices[~prices.name.str.contains('Yoan Cardinale')]
prices = prices[~prices.name.str.contains('Yoann Etienne')]
prices = prices[~prices.name.str.contains('Yohan Croizet')]
prices = prices[~prices.name.str.contains('Yonathan Cabral')]
prices = prices[~prices.name.str.contains('Younes Belhanda')]
prices = prices[~prices.name.str.contains('Youri Schoonderwaldt')]
prices = prices[~prices.name.str.contains('Zvonimir Sarlija')]
prices = prices[~prices.name.str.contains('Zlatko Tripic')]
prices = prices[~prices.name.str.contains('Ziyad Al-Oyouni')]
prices = prices[~prices.name.str.contains('Zito Luvumbo')]
prices = prices[~prices.name.str.contains('Zeno Rossi')]
prices = prices[~prices.name.str.contains('Zeki Yildirim')]
prices = prices[~prices.name.str.contains('Zeki Fryers')]
prices = prices[~prices.name.str.contains('Zeki Celik')]
prices = prices[~prices.name.str.contains('Zehn Mohammed')]
prices = prices[~prices.name.str.contains('Zech Medley')]
prices = prices[~prices.name.str.contains('Ze Oliveira')]
prices = prices[~prices.name.str.contains('Ze IvaldoZe Ivaldo')]
prices = prices[~prices.name.str.contains('Ze GabrielZe Gabriel')]
prices = prices[~prices.name.str.contains('Zdenek Zlamal')]
prices = prices[~prices.name.str.contains('Zanka')]
prices = prices[~prices.name.str.contains('Zakaria Diallo')]
prices = prices[~prices.name.str.contains('Zak Dearnley')]
prices = prices[~prices.name.str.contains('Zaid Romero')]
prices = prices[~prices.name.str.contains('Zaid Al Hussaini')]
prices = prices[~prices.name.str.contains('Zachary Dearnley')]
prices = prices[~prices.name.str.contains('Zach Jeacock')]
prices = prices[~prices.name.str.contains('Zac Williams')]
prices = prices[~prices.name.str.contains('Zac SmithZac Smith')]
prices = prices[~prices.name.str.contains('Dimitri Chistyakov')]
prices = prices[~prices.name.str.contains('Dimitri German')]
prices = prices[~prices.name.str.contains('Dimitri Khomich')]
prices = prices[~prices.name.str.contains('Dimitri Nazarov')]
prices = prices[~prices.name.str.contains('Dimitri Poloz')]
prices = prices[~prices.name.str.contains('Dimitri Rybchinskiy')]
prices = prices[~prices.name.str.contains('Dimitri Shomko')]
prices = prices[~prices.name.str.contains('Dimitri Skopintsev')]
prices = prices[~prices.name.str.contains('Dimitri Stotskiy')]
prices = prices[~prices.name.str.contains('Dimitri Sysuev')]
prices = prices[~prices.name.str.contains('Dimitri Tikhiy')]
prices = prices[~prices.name.str.contains('Dimitri Zhivoglyadov')]
prices = prices[~prices.name.str.contains('Dino Visser')]
prices = prices[~prices.name.str.contains('Diogo BarbosaDiogo Barbosa')]
prices = prices[~prices.name.str.contains('Dixon Renteria')]
prices = prices[~prices.name.str.contains('Djibril Diani')]
prices = prices[~prices.name.str.contains('Dmitri Efremov')]
prices = prices[~prices.name.str.contains('Dmitri German')]
prices = prices[~prices.name.str.contains('Dmitri Markitesov')]
prices = prices[~prices.name.str.contains('Dmitri Rybchinsky')]
prices = prices[~prices.name.str.contains('Dmitri Skopintsev')]
prices = prices[~prices.name.str.contains('Dmitri Sysuev')]
prices = prices[~prices.name.str.contains('Dmitri Tikhiy')]
prices = prices[~prices.name.str.contains('Dmitri Vershkov')]
prices = prices[~prices.name.str.contains('Dmitrii Merenchukov')]
prices = prices[~prices.name.str.contains('Dmitriy Chistyakov')]
prices = prices[~prices.name.str.contains('Dmitriy Poloz')]
prices = prices[~prices.name.str.contains('Dmitriy Shomko')]
prices = prices[~prices.name.str.contains('Dmitriy Stotskiy')]
prices = prices[~prices.name.str.contains('Dmitriy Tarasov')]
prices = prices[~prices.name.str.contains('Dmitry Zhivoglyadov')]
prices = prices[~prices.name.str.contains('DodoDodo')]
prices = prices[~prices.name.str.contains('Dominik Reimann')]
prices = prices[~prices.name.str.contains('Dominik Szoboszlai')]
prices = prices[~prices.name.str.contains('Dragan Lausberg')]
prices = prices[~prices.name.str.contains('Drake Callender')]
prices = prices[~prices.name.str.contains('Drissa Camara')]
prices = prices[~prices.name.str.contains('EÃ±aut Mendia')]
prices = prices[~prices.name.str.contains('Edgar Pacheco')]
prices = prices[~prices.name.str.contains('Edi Semedo')]
prices = prices[~prices.name.str.contains('EdimarEdimar')]
prices = prices[~prices.name.str.contains('Edson MontaÃ±o')]
prices = prices[~prices.name.str.contains('EdsonEdson')]
prices = prices[~prices.name.str.contains('Eduardo dos Santos Haesler')]
prices = prices[~prices.name.str.contains('Edwar Lopez')]
prices = prices[~prices.name.str.contains('Elias Pereyra')]
prices = prices[~prices.name.str.contains('EliasElias')]
prices = prices[~prices.name.str.contains('Elton Kabangu')]
prices = prices[~prices.name.str.contains('Elye WahiElye Wahi')]
prices = prices[~prices.name.str.contains('Emanuel Insua')]
prices = prices[~prices.name.str.contains('EmanuelEmanuel')]
prices = prices[~prices.name.str.contains('Emil Riis Jakobsen')]
prices = prices[~prices.name.str.contains('Emil RiisEmil Riis')]
prices = prices[~prices.name.str.contains('Emiliano Gonzalez')]
prices = prices[~prices.name.str.contains('Emiliano Insua')]
prices = prices[~prices.name.str.contains('Emre Pehlivan')]
prices = prices[~prices.name.str.contains('Enzo YbaÃ±ez')]
prices = prices[~prices.name.str.contains('Eric Lopez')]
prices = prices[~prices.name.str.contains('ErickErick')]
prices = prices[~prices.name.str.contains('Erickson Gallardo')]
prices = prices[~prices.name.str.contains('Erik DueÃ±as')]
prices = prices[~prices.name.str.contains('Ertugrul Cetin')]
prices = prices[~prices.name.str.contains('Esteban Rolon')]
prices = prices[~prices.name.str.contains('Eunan O\'Kane')]
prices = prices[~prices.name.str.contains('Evan N\'Dicka')]
prices = prices[~prices.name.str.contains('Exequiel Zeballos')]
prices = prices[~prices.name.str.contains('Ezekiel Fryers')]
prices = prices[~prices.name.str.contains('Ezequiel Cerica')]
prices = prices[~prices.name.str.contains('Ezequiel Fernandez')]
prices = prices[~prices.name.str.contains('Fabian Cubero')]
prices = prices[~prices.name.str.contains('Fabian Noguera')]
prices = prices[~prices.name.str.contains('Fabio Abreu')]
prices = prices[~prices.name.str.contains('Fabio ChinaFabio China')]
prices = prices[~prices.name.str.contains('Fabio SantosFabio Santos')]
prices = prices[~prices.name.str.contains('Fabio Vazquez')]
prices = prices[~prices.name.str.contains('Fabricio Dominguez')]
prices = prices[~prices.name.str.contains('Facundo Fernandez')]
prices = prices[~prices.name.str.contains('Facundo Garces')]
prices = prices[~prices.name.str.contains('Facundo Sanchez')]
prices = prices[~prices.name.str.contains('Faiz Selemani')]
prices = prices[~prices.name.str.contains('Federico Martinez')]
prices = prices[~prices.name.str.contains('Fehmi KocFehmi Koc')]
prices = prices[~prices.name.str.contains('Fehmi Mert Gunok')]
prices = prices[~prices.name.str.contains('Fernando Roman')]
prices = prices[~prices.name.str.contains('Ferris N\'Goma')]
prices = prices[~prices.name.str.contains('Francesco D\'Alterio')]
prices = prices[~prices.name.str.contains('Francisco Ramirez')]
prices = prices[~prices.name.str.contains('Franco Gonzalez')]
prices = prices[~prices.name.str.contains('Franco Perez')]
prices = prices[~prices.name.str.contains('Gabriel AlvesGabriel Alves')]
prices = prices[~prices.name.str.contains('Gabriel Chapeco')]
prices = prices[~prices.name.str.contains('Gabriel Diaz')]
prices = prices[~prices.name.str.contains('Gabriel Falcao')]
prices = prices[~prices.name.str.contains('Gabriel Graciani')]
prices = prices[~prices.name.str.contains('Gabriel NogaGabriel Noga')]
prices = prices[~prices.name.str.contains('Gabriel PereiraGabriel Pereira')]
prices = prices[~prices.name.str.contains('Gaetan Poussin')]
prices = prices[~prices.name.str.contains('Gaston Benavidez')]
prices = prices[~prices.name.str.contains('Gaston Diaz')]
prices = prices[~prices.name.str.contains('Gaston Fernandez')]
prices = prices[~prices.name.str.contains('Gaston Gerzel')]
prices = prices[~prices.name.str.contains('Gaston Gomez')]
prices = prices[~prices.name.str.contains('Gaston Gonzalez')]
prices = prices[~prices.name.str.contains('Gaston Suso')]
prices = prices[~prices.name.str.contains('Gaston Torres')]
prices = prices[~prices.name.str.contains('George Nurse')]
prices = prices[~prices.name.str.contains('Gerald Sithole')]
prices = prices[~prices.name.str.contains('Giles Phillips')]
prices = prices[~prices.name.str.contains('Gonzalo Abrego')]
prices = prices[~prices.name.str.contains('Gonzalo Asis')]
prices = prices[~prices.name.str.contains('Gonzalo Rodriguez')]
prices = prices[~prices.name.str.contains('Gonzalo Veron')]
prices = prices[~prices.name.str.contains('Gorkem Can')]
prices = prices[~prices.name.str.contains('Guido Vadala')]
prices = prices[~prices.name.str.contains('Guillermo Fernandez')]
prices = prices[~prices.name.str.contains('Gustavo Ramalho')]
prices = prices[~prices.name.str.contains('Guven Yalcin')]
prices = prices[~prices.name.str.contains('Habib Keita')]
prices = prices[~prices.name.str.contains('Hamza Masoudi')]
prices = prices[~prices.name.str.contains('Hannibal Mejbri')]
prices = prices[~prices.name.str.contains('Hans Nicolussi Caviglia')]
prices = prices[~prices.name.str.contains('Harrison Paton')]
prices = prices[~prices.name.str.contains('Hatem Elhamed')]
prices = prices[~prices.name.str.contains('HeberHeber')]
prices = prices[~prices.name.str.contains('Hector Canteros')]
prices = prices[~prices.name.str.contains('HelioHelio')]
prices = prices[~prices.name.str.contains('HenriHenri')]
prices = prices[~prices.name.str.contains('Henry Burnett')]
prices = prices[~prices.name.str.contains('Hernan Burbano')]
prices = prices[~prices.name.str.contains('Hernan Lamberti')]
prices = prices[~prices.name.str.contains('Hernan Lopez')]
prices = prices[~prices.name.str.contains('Heung-min Son')]
prices = prices[~prices.name.str.contains('HiagoHiago')]
prices = prices[~prices.name.str.contains('Hugo Ekitike')]
prices = prices[~prices.name.str.contains('Hugo Silva')]
prices = prices[~prices.name.str.contains('Huseyin Biler')]
prices = prices[~prices.name.str.contains('Ibrahima Camara')]
prices = prices[~prices.name.str.contains('Ignacio Cechi')]
prices = prices[~prices.name.str.contains('Ignacio Gariglio')]
prices = prices[~prices.name.str.contains('Ignacio Lago')]
prices = prices[~prices.name.str.contains('Ignacio Maestro Puch')]
prices = prices[~prices.name.str.contains('Ignacio Miramon')]
prices = prices[~prices.name.str.contains('Igor CariusIgor Carius')]
prices = prices[~prices.name.str.contains('Igor JuliaoIgor Juliao')]
prices = prices[~prices.name.str.contains('Igor Julio')]
prices = prices[~prices.name.str.contains('Igor TorresIgor Torres')]
prices = prices[~prices.name.str.contains('Ike OparaIke Opara')]
prices = prices[~prices.name.str.contains('Il-lok Yun')]
prices = prices[~prices.name.str.contains('Ilya Shkurin')]
prices = prices[~prices.name.str.contains('Ilya Vorobjov')]
prices = prices[~prices.name.str.contains('Imanol Machuca')]
prices = prices[~prices.name.str.contains('In-beom Hwang')]
prices = prices[~prices.name.str.contains('Indiana Vassilev')]
prices = prices[~prices.name.str.contains('Innes Cameron')]
prices = prices[~prices.name.str.contains('Ionut Nedelcearu')]
prices = prices[~prices.name.str.contains('Irakoze Donasiyano')]
prices = prices[~prices.name.str.contains('Isaac Boehmer')]
prices = prices[~prices.name.str.contains('Isaac Hutchinson')]
prices = prices[~prices.name.str.contains('Isaac Karamoko')]
prices = prices[~prices.name.str.contains('IsaqueIsaque')]
prices = prices[~prices.name.str.contains('Islamzhan Nasyrov')]
prices = prices[~prices.name.str.contains('Isma Ruiz')]
prices = prices[~prices.name.str.contains('Ismael Saibari')]
prices = prices[~prices.name.str.contains('Ismael Sow')]
prices = prices[~prices.name.str.contains('Ismail Aissati')]
prices = prices[~prices.name.str.contains('Ismail Yuksek')]
prices = prices[~prices.name.str.contains('Israel Damonte')]
prices = prices[~prices.name.str.contains('Ivan Angulo')]
prices = prices[~prices.name.str.contains('Ivan Arboleda')]
prices = prices[~prices.name.str.contains('Ivan Budachev')]
prices = prices[~prices.name.str.contains('Ivan Kuzmichev')]
prices = prices[~prices.name.str.contains('IvoneiIvonei')]
prices = prices[~prices.name.str.contains('Iyenoma Udogie')]
prices = prices[~prices.name.str.contains('Jabez Makanda')]
prices = prices[~prices.name.str.contains('Jack Bolton')]
prices = prices[~prices.name.str.contains('Jack Burroughs')]
prices = prices[~prices.name.str.contains('Jack Chambers')]
prices = prices[~prices.name.str.contains('Jack Currie')]
prices = prices[~prices.name.str.contains('Jack Davies')]
prices = prices[~prices.name.str.contains('Jack de Vries')]
prices = prices[~prices.name.str.contains('Jack Hindle')]
prices = prices[~prices.name.str.contains('Jack Madelin')]
prices = prices[~prices.name.str.contains('Jack Newman')]
prices = prices[~prices.name.str.contains('Jack Rodwell')]
prices = prices[~prices.name.str.contains('Jack Roles')]
prices = prices[~prices.name.str.contains('Jack Ruddy')]
prices = prices[~prices.name.str.contains('Jack SimsJack Sims')]
prices = prices[~prices.name.str.contains('Jack Stretton')]
prices = prices[~prices.name.str.contains('Jack Vale')]
prices = prices[~prices.name.str.contains('Jack Young')]
prices = prices[~prices.name.str.contains('Jackson')]
prices = prices[~prices.name.str.contains('Jackson Conway')]
prices = prices[~prices.name.str.contains('Jacob Akanyirige')]
prices = prices[~prices.name.str.contains('Jacob Bancroft')]
prices = prices[~prices.name.str.contains('Jacob Barrett Laursen')]
prices = prices[~prices.name.str.contains('Jacob Gratton')]
prices = prices[~prices.name.str.contains('Jacob Greene')]
prices = prices[~prices.name.str.contains('Jacob Laursen')]
prices = prices[~prices.name.str.contains('Jacopo Dezi')]
prices = prices[~prices.name.str.contains('Jacopo Segre')]
prices = prices[~prices.name.str.contains('Jaden Philogene-Bidace')]
prices = prices[~prices.name.str.contains('JadsomJadsom')]
prices = prices[~prices.name.str.contains('Jae-ik Lee')]
prices = prices[~prices.name.str.contains('JailsonJailson')]
prices = prices[~prices.name.str.contains('Jaime Alvarado')]
prices = prices[~prices.name.str.contains('Jaimy Kroesen')]
prices = prices[~prices.name.str.contains('JairJair')]
prices = prices[~prices.name.str.contains('Jake Davidson')]
prices = prices[~prices.name.str.contains('Jake HullJake Hull')]
prices = prices[~prices.name.str.contains('Jake PeckJake Peck')]
prices = prices[~prices.name.str.contains('Jake Scrimshaw')]
prices = prices[~prices.name.str.contains('Jake Turner')]
prices = prices[~prices.name.str.contains('Jake Vokins')]
prices = prices[~prices.name.str.contains('Jakob Busk')]
prices = prices[~prices.name.str.contains('Jan Olschowsky')]
prices = prices[~prices.name.str.contains('Jan RasJan Ras')]
prices = prices[~prices.name.str.contains('JandersonJanderson')]
prices = prices[~prices.name.str.contains('JandreiJandrei')]
prices = prices[~prices.name.str.contains('Janeiler Rivas')]
prices = prices[~prices.name.str.contains('Jaquite')]
prices = prices[~prices.name.str.contains('Jason Meerstadt')]
prices = prices[~prices.name.str.contains('Jason Pendant')]
prices = prices[~prices.name.str.contains('Jason Steele')]
prices = prices[~prices.name.str.contains('Jay Henderson')]
prices = prices[~prices.name.str.contains('Jean Zimmer')]
prices = prices[~prices.name.str.contains('Jean-Aniel Assi')]
prices = prices[~prices.name.str.contains('Jean-Eudes Aholou')]
prices = prices[~prices.name.str.contains('Jeff Dewsnup')]
prices = prices[~prices.name.str.contains('Jeffrey de Lange')]
prices = prices[~prices.name.str.contains('Jeison Chala')]
prices = prices[~prices.name.str.contains('Jeison Lucumi')]
prices = prices[~prices.name.str.contains('Jeizon Ramirez')]
prices = prices[~prices.name.str.contains('JemersonJemerson')]
prices = prices[~prices.name.str.contains('JenisonJenison')]
prices = prices[~prices.name.str.contains('Jens Castrop')]
prices = prices[~prices.name.str.contains('Jens Craenmehr')]
prices = prices[~prices.name.str.contains('Jens Grahl')]
prices = prices[~prices.name.str.contains('Jeremy Perbet')]
prices = prices[~prices.name.str.contains('Jeronimo Pourtau')]
prices = prices[~prices.name.str.contains('Jesus Ferreira')]
prices = prices[~prices.name.str.contains('Jetro Willems')]
prices = prices[~prices.name.str.contains('Jevan Anderson')]
prices = prices[~prices.name.str.contains('Jhonatan Candia')]
prices = prices[~prices.name.str.contains('Jhonathan Agudelo')]
prices = prices[~prices.name.str.contains('JhonnathanJhonnathan')]
prices = prices[~prices.name.str.contains('Jimmy Adrian Kaparos')]
prices = prices[~prices.name.str.contains('Jimmy Kaparos')]
prices = prices[~prices.name.str.contains('Joachim Imbrechts')]
prices = prices[~prices.name.str.contains('Joan Mazzaco')]
prices = prices[~prices.name.str.contains('Joao Carlos Teixeira')]
prices = prices[~prices.name.str.contains('Joao CarlosJoao Carlos')]
prices = prices[~prices.name.str.contains('Joao Ferreira')]
prices = prices[~prices.name.str.contains('Joao GomesJoao Gomes')]
prices = prices[~prices.name.str.contains('Joao Jaquite')]
prices = prices[~prices.name.str.contains('Joao Klauss')]
prices = prices[~prices.name.str.contains('Joao LopesJoao Lopes')]
prices = prices[~prices.name.str.contains('Joao MarioJoao MarioRight Midfiel')]
prices = prices[~prices.name.str.contains('Joao Monteiro')]
prices = prices[~prices.name.str.contains('Joao Queiros')]
prices = prices[~prices.name.str.contains('Joao Rodriguez')]
prices = prices[~prices.name.str.contains('Joao Teixeira')]
prices = prices[~prices.name.str.contains('Joao VictorJoao Victor')]
prices = prices[~prices.name.str.contains('Joaquin Arzura')]
prices = prices[~prices.name.str.contains('Joaquin Blazquez')]
prices = prices[~prices.name.str.contains('Joaquin Hass')]
prices = prices[~prices.name.str.contains('Joaquin Mendive')]
prices = prices[~prices.name.str.contains('Joaquin Novillo')]
prices = prices[~prices.name.str.contains('Joaquin Varela')]
prices = prices[~prices.name.str.contains('JobsonJobson')]
prices = prices[~prices.name.str.contains('Joe Bendik')]
prices = prices[~prices.name.str.contains('Joe DodooJoe Dodoo')]
prices = prices[~prices.name.str.contains('Joe DunneJoe Dunne')]
prices = prices[~prices.name.str.contains('Joe HartJoe Hart')]
prices = prices[~prices.name.str.contains('Joe Maguire')]
prices = prices[~prices.name.str.contains('Joe Mason')]
prices = prices[~prices.name.str.contains('Joe NealJoe Neal')]
prices = prices[~prices.name.str.contains('Joe Rafferty')]
prices = prices[~prices.name.str.contains('Joe Scally')]
prices = prices[~prices.name.str.contains('Joe Willock')]
prices = prices[~prices.name.str.contains('Joe Woodiwiss')]
prices = prices[~prices.name.str.contains('Joel Tagueu')]
prices = prices[~prices.name.str.contains('Joey DeZart')]
prices = prices[~prices.name.str.contains('Johannes Eggestein')]
prices = prices[~prices.name.str.contains('John Anthony Brooks')]
prices = prices[~prices.name.str.contains('John Joe Patrick Finn')]
prices = prices[~prices.name.str.contains('John KennedyJohn Kennedy')]
prices = prices[~prices.name.str.contains('JohnJohn')]
prices = prices[~prices.name.str.contains('JohnnyJohnny')]
prices = prices[~prices.name.str.contains('JoilsonJoilson')]
prices = prices[~prices.name.str.contains('JoJo')]
prices = prices[~prices.name.str.contains('Jomaine Consbruch')]
prices = prices[~prices.name.str.contains('Jon BellJon Bell')]
prices = prices[~prices.name.str.contains('Jon Flanagan')]
prices = prices[~prices.name.str.contains('Jon Pacheco Dozagarat')]
prices = prices[~prices.name.str.contains('Jon Stead')]
prices = prices[~prices.name.str.contains('Jonah Osabutey')]
prices = prices[~prices.name.str.contains('Jonas Aguirre')]
prices = prices[~prices.name.str.contains('Jonas Carls')]
prices = prices[~prices.name.str.contains('JonasJonas')]
prices = prices[~prices.name.str.contains('Jonatan Gomez')]
prices = prices[~prices.name.str.contains('Jonatan Maidana')]
prices = prices[~prices.name.str.contains('Jonathan Bell')]
prices = prices[~prices.name.str.contains('Jonathan Bolingi')]
prices = prices[~prices.name.str.contains('Jonathan Cabral')]
prices = prices[~prices.name.str.contains('Jonathan Howson')]
prices = prices[~prices.name.str.contains('Jonathan Klinsmann')]
prices = prices[~prices.name.str.contains('Jonathan Mulder')]
prices = prices[~prices.name.str.contains('Jonathan Russell')]
prices = prices[~prices.name.str.contains('Jonathan Torres')]
prices = prices[~prices.name.str.contains('Jonathan Zacaria')]
prices = prices[~prices.name.str.contains('Jonny Howson')]
prices = prices[~prices.name.str.contains('Jonny Otto')]
prices = prices[~prices.name.str.contains('Jorben Vanhulle')]
prices = prices[~prices.name.str.contains('Jordan Barnett')]
prices = prices[~prices.name.str.contains('Jordan Beyer')]
prices = prices[~prices.name.str.contains('Jordan Eastham')]
prices = prices[~prices.name.str.contains('Jordan Flores')]
prices = prices[~prices.name.str.contains('Jordan Garrick')]
prices = prices[~prices.name.str.contains('Jordan Green')]
prices = prices[~prices.name.str.contains('Jordan Harvey')]
prices = prices[~prices.name.str.contains('Jordan Holsgrove')]
prices = prices[~prices.name.str.contains('Jordan Michael Flores')]
prices = prices[~prices.name.str.contains('Jordan Perruzza')]
prices = prices[~prices.name.str.contains('Jordan Roberts')]
prices = prices[~prices.name.str.contains('Jordan Thomas')]
prices = prices[~prices.name.str.contains('Jordan van der Gaag')]
prices = prices[~prices.name.str.contains('JordanJordan')]
prices = prices[~prices.name.str.contains('Jordon Garrick')]
prices = prices[~prices.name.str.contains('Jordon Thompson')]
prices = prices[~prices.name.str.contains('Jordy Schelfhout')]
prices = prices[~prices.name.str.contains('Jordy Wehrmann')]
prices = prices[~prices.name.str.contains('Joren DomJoren Dom')]
prices = prices[~prices.name.str.contains('Jores Rahou')]
prices = prices[~prices.name.str.contains('Jorge Aruga')]
prices = prices[~prices.name.str.contains('Jorge Caceres')]
prices = prices[~prices.name.str.contains('Jorge De Olivera')]
prices = prices[~prices.name.str.contains('Jorge Figal')]
prices = prices[~prices.name.str.contains('Jorge Moreira')]
prices = prices[~prices.name.str.contains('Jorge Ortega')]
prices = prices[~prices.name.str.contains('Jorge Ortiz')]
prices = prices[~prices.name.str.contains('Jorge Pereira')]
prices = prices[~prices.name.str.contains('Jorge Pereyra Diaz')]
prices = prices[~prices.name.str.contains('Jorge Saenz')]
prices = prices[~prices.name.str.contains('Joris Gnagnon')]
prices = prices[~prices.name.str.contains('Jorn Berkhout')]
prices = prices[~prices.name.str.contains('Joroen Dom')]
prices = prices[~prices.name.str.contains('Jorrit Smeets')]
prices = prices[~prices.name.str.contains('Jose Cifuentes')]
prices = prices[~prices.name.str.contains('Jose Devecchi')]
prices = prices[~prices.name.str.contains('Jose Gimenez')]
prices = prices[~prices.name.str.contains('Jose Gonzalez')]
prices = prices[~prices.name.str.contains('Jose Luis Fernandez')]
prices = prices[~prices.name.str.contains('Jose Luis Gomez')]
prices = prices[~prices.name.str.contains('Jose Luis Rodriguez')]
prices = prices[~prices.name.str.contains('Jose Luis Sinisterra')]
prices = prices[~prices.name.str.contains('Jose Manuel Fontan')]
prices = prices[~prices.name.str.contains('Jose Manuel Lopez')]
prices = prices[~prices.name.str.contains('Jose Marcelo')]
prices = prices[~prices.name.str.contains('Jose Mari')]
prices = prices[~prices.name.str.contains('Jose Maria Gimenez')]
prices = prices[~prices.name.str.contains('Jose MariJose Mari')]
prices = prices[~prices.name.str.contains('Jose Martinez')]
prices = prices[~prices.name.str.contains('Jose Moya')]
prices = prices[~prices.name.str.contains('JoryJory')]
prices = prices[~prices.name.str.contains('JosefJosef')]
prices = prices[~prices.name.str.contains('Joseph Amoah')]
prices = prices[~prices.name.str.contains('Joseph Collinge')]
prices = prices[~prices.name.str.contains('Joseph Dodoo')]
prices = prices[~prices.name.str.contains('Joseph Hungbo')]
prices = prices[~prices.name.str.contains('Joseph Maguire')]
prices = prices[~prices.name.str.contains('Joseph Marie Minala')]
prices = prices[~prices.name.str.contains('Joseph Martin Paintsil')]
prices = prices[~prices.name.str.contains('Joseph Paintsil')]
prices = prices[~prices.name.str.contains('Joseph Rafferty')]
prices = prices[~prices.name.str.contains('Joseph Willock')]
prices = prices[~prices.name.str.contains('Josete Miranda')]
prices = prices[~prices.name.str.contains('Josh Andrews')]
prices = prices[~prices.name.str.contains('Josh Atencio')]
prices = prices[~prices.name.str.contains('Josh Bauer')]
prices = prices[~prices.name.str.contains('Josh Coburn')]
prices = prices[~prices.name.str.contains('Josh EarlJosh Earl')]
prices = prices[~prices.name.str.contains('Josh Emmanuel')]
prices = prices[~prices.name.str.contains('Josh Ginnelly')]
prices = prices[~prices.name.str.contains('Josh Gould')]
prices = prices[~prices.name.str.contains('Josh KeyJosh Key')]
prices = prices[~prices.name.str.contains('Josh Knight')]
prices = prices[~prices.name.str.contains('Josh Lillis')]
prices = prices[~prices.name.str.contains('Josh Sargent')]
prices = prices[~prices.name.str.contains('Josh Vickers')]
prices = prices[~prices.name.str.contains('Joshua Atencio')]
prices = prices[~prices.name.str.contains('Joshua Earl')]
prices = prices[~prices.name.str.contains('Joshua Emmanuel')]
prices = prices[~prices.name.str.contains('Joshua Griffiths')]
prices = prices[~prices.name.str.contains('Joshua Key')]
prices = prices[~prices.name.str.contains('Joshua McPake')]
prices = prices[~prices.name.str.contains('Joshua Render')]
prices = prices[~prices.name.str.contains('Joshua Sargent')]
prices = prices[~prices.name.str.contains('Joshua Wehking')]
prices = prices[~prices.name.str.contains('Jospin Nshimirimana')]
prices = prices[~prices.name.str.contains('Josue Ayala')]
prices = prices[~prices.name.str.contains('Jota Peleteiro')]
prices = prices[~prices.name.str.contains('Jovanny Bolivar')]
prices = prices[~prices.name.str.contains('Joye Dezart')]
prices = prices[~prices.name.str.contains('JT Marcinkowski')]
prices = prices[~prices.name.str.contains('Juan Ãlvarez')]
prices = prices[~prices.name.str.contains('Juan Antonini')]
prices = prices[~prices.name.str.contains('Juan Bolado')]
prices = prices[~prices.name.str.contains('Juan Brunetta')]
prices = prices[~prices.name.str.contains('Juan Carlos Portillo')]
prices = prices[~prices.name.str.contains('Juan Castilla')]
prices = prices[~prices.name.str.contains('Juan Cavallaro')]
prices = prices[~prices.name.str.contains('Juan Caviglia')]
prices = prices[~prices.name.str.contains('Juan Cruz Komar')]
prices = prices[~prices.name.str.contains('Juan Espinola')]
prices = prices[~prices.name.str.contains('Juan Franzoni')]
prices = prices[~prices.name.str.contains('Juan Fuentes')]
prices = prices[~prices.name.str.contains('Juan Gonzalez')]
prices = prices[~prices.name.str.contains('Juan Iglesias')]
prices = prices[~prices.name.str.contains('Juan Ignacio Mendez')]
prices = prices[~prices.name.str.contains('Juan Ignacio Nardoni')]
prices = prices[~prices.name.str.contains('Juan Infante')]
prices = prices[~prices.name.str.contains('Juan Jose Caceres')]
prices = prices[~prices.name.str.contains('Juan Kaprof')]
prices = prices[~prices.name.str.contains('Juan Komar')]
prices = prices[~prices.name.str.contains('Juan Lucero')]
prices = prices[~prices.name.str.contains('Juan Manuel Cruz')]
prices = prices[~prices.name.str.contains('Juan Manuel Cuesta')]
prices = prices[~prices.name.str.contains('Juan Manuel Insaurralde')]
prices = prices[~prices.name.str.contains('Juan Martin Lucero')]
prices = prices[~prices.name.str.contains('Juan Miritello')]
prices = prices[~prices.name.str.contains('Juan Nardoni')]
prices = prices[~prices.name.str.contains('Juan Pablo Barinaga')]
prices = prices[~prices.name.str.contains('Juan Pablo Freytes')]
prices = prices[~prices.name.str.contains('Juan Pablo Ramirez')]
prices = prices[~prices.name.str.contains('Juan Pablo Romero')]
prices = prices[~prices.name.str.contains('Juan Pacchini')]
prices = prices[~prices.name.str.contains('Juan Roman Pucheta')]
prices = prices[~prices.name.str.contains('Juan Sforza')]
prices = prices[~prices.name.str.contains('Juan Vieyra')]
prices = prices[~prices.name.str.contains('Juan Zurbriggen')]
prices = prices[~prices.name.str.contains('Jugurtha Hamroun')]
prices = prices[~prices.name.str.contains('Jules-Anthony Vilsaint')]
prices = prices[~prices.name.str.contains('Julian Aude')]
prices = prices[~prices.name.str.contains('Julian Chicco')]
prices = prices[~prices.name.str.contains('Julian Dunn')]
prices = prices[~prices.name.str.contains('Julian Frommann')]
prices = prices[~prices.name.str.contains('Julian Krahl')]
prices = prices[~prices.name.str.contains('Julian Lopez')]
prices = prices[~prices.name.str.contains('Julian Malatini')]
prices = prices[~prices.name.str.contains('Julian Marcioni')]
prices = prices[~prices.name.str.contains('Julian Navas')]
prices = prices[~prices.name.str.contains('Julian Pollersbeck')]
prices = prices[~prices.name.str.contains('Julian Schieber')]
prices = prices[~prices.name.str.contains('Julian Velazquez')]
prices = prices[~prices.name.str.contains('Julio Gonzalez')]
prices = prices[~prices.name.str.contains('Julio Villalba')]
prices = prices[~prices.name.str.contains('Juninho Capixaba')]
prices = prices[~prices.name.str.contains('JunioJunio')]
prices = prices[~prices.name.str.contains('Junior Alonso')]
prices = prices[~prices.name.str.contains('Junior AlonsoJunior Alonso')]
prices = prices[~prices.name.str.contains('Junior Dina Ebimbe')]
prices = prices[~prices.name.str.contains('Junior Hoilett')]
prices = prices[~prices.name.str.contains('Justin Che')]
prices = prices[~prices.name.str.contains('Justin Haak')]
prices = prices[~prices.name.str.contains('Justin Johnson')]
prices = prices[~prices.name.str.contains('Justin Rennicks')]
prices = prices[~prices.name.str.contains('Justin vom Steeg')]
prices = prices[~prices.name.str.contains('Justo Giani')]
prices = prices[~prices.name.str.contains('Karim Zedadka')]
prices = prices[~prices.name.str.contains('Kartal Kayra Yilmaz')]
prices = prices[~prices.name.str.contains('Kasper Lunding')]
prices = prices[~prices.name.str.contains('Kasper Lunding Jakobsen')]
prices = prices[~prices.name.str.contains('Kemal Ruzgar')]
prices = prices[~prices.name.str.contains('KennedyKennedy')]
prices = prices[~prices.name.str.contains('Kevin Boma')]
prices = prices[~prices.name.str.contains('Kevin MalthusKevin Malthus')]
prices = prices[~prices.name.str.contains('Kevin Zenon')]
prices = prices[~prices.name.str.contains('Kibe')]
prices = prices[~prices.name.str.contains('Kiernan Dewsbury-Hall')]
prices = prices[~prices.name.str.contains('KikasKikas')]
prices = prices[~prices.name.str.contains('Koke Vegas')]
prices = prices[~prices.name.str.contains('Kosuke Nakamura')]
prices = prices[~prices.name.str.contains('Kouadio Dabila')]
prices = prices[~prices.name.str.contains('Kouadio-Yves Dabila')]
prices = prices[~prices.name.str.contains('Kuki Zalazar')]
prices = prices[~prices.name.str.contains('Kuku Emmanuel Fidelis')]
prices = prices[~prices.name.str.contains('Kwasi Okyere Wriedt')]
prices = prices[~prices.name.str.contains('LaercioLaercio')]
prices = prices[~prices.name.str.contains('Lars Lukas Mai')]
prices = prices[~prices.name.str.contains('Lautaro de Leon')]
prices = prices[~prices.name.str.contains('LazaroLazaro')]
prices = prices[~prices.name.str.contains('Leandro Contin')]
prices = prices[~prices.name.str.contains('Leandro Fernandez')]
prices = prices[~prices.name.str.contains('Lee GrantLee Grant')]
prices = prices[~prices.name.str.contains('Lee Seung-Woo')]
prices = prices[~prices.name.str.contains('Leo BorgesLeo Borges')]
prices = prices[~prices.name.str.contains('Leo Gomes')]
prices = prices[~prices.name.str.contains('Leo NatelLeo Natel')]
prices = prices[~prices.name.str.contains('Leo Ostigard')]
prices = prices[~prices.name.str.contains('Leo PassosLeo Passos')]
prices = prices[~prices.name.str.contains('Leo SantosLeo Santos')]
prices = prices[~prices.name.str.contains('Leon Chambers-Parillon')]
prices = prices[~prices.name.str.contains('Leon Sabua')]
prices = prices[~prices.name.str.contains('Leon Schaffran')]
prices = prices[~prices.name.str.contains('Leon Thomson King')]
prices = prices[~prices.name.str.contains('Leonel Gonzalez')]
prices = prices[~prices.name.str.contains('Levi Ntumba')]
prices = prices[~prices.name.str.contains('LiggerLigger')]
prices = prices[~prices.name.str.contains('Lillo Guarneri')]
prices = prices[~prices.name.str.contains('LinconLincon')]
prices = prices[~prices.name.str.contains('Lino Kasten')]
prices = prices[~prices.name.str.contains('Lion Lauberbach')]
prices = prices[~prices.name.str.contains('Lisandro Alzugaray')]
prices = prices[~prices.name.str.contains('Logan Delaurier-Chaubet')]
prices = prices[~prices.name.str.contains('LohanLohan')]
prices = prices[~prices.name.str.contains('Lois Diony')]
prices = prices[~prices.name.str.contains('Luan PolliLuan Polli')]
prices = prices[~prices.name.str.contains('Luca Martinez Dupuy')]
prices = prices[~prices.name.str.contains('Ludvig Ohman')]
prices = prices[~prices.name.str.contains('Luis Leal')]
prices = prices[~prices.name.str.contains('Luis Miguel Rodriguez')]
prices = prices[~prices.name.str.contains('Luis Ojeda')]
prices = prices[~prices.name.str.contains('Luis PhelipeLuis Phelipe')]
prices = prices[~prices.name.str.contains('Luis Rodriguez')]
prices = prices[~prices.name.str.contains('Luis Vazquez')]
prices = prices[~prices.name.str.contains('Luiz Otavio')]
prices = prices[~prices.name.str.contains('Lukas Fahrnberger')]
prices = prices[~prices.name.str.contains('Lukas Hinterseer')]
prices = prices[~prices.name.str.contains('Lukas Hornicek')]
prices = prices[~prices.name.str.contains('LumorLumor')]
prices = prices[~prices.name.str.contains('Mads Bidstrup')]
prices = prices[~prices.name.str.contains('Maikel van der Werff')]
prices = prices[~prices.name.str.contains('Majid Hosseini')]
prices = prices[~prices.name.str.contains('Maksim Kutovoi')]
prices = prices[~prices.name.str.contains('Maksim Martusevich')]
prices = prices[~prices.name.str.contains('Maksim Turischev')]
prices = prices[~prices.name.str.contains('Maksim Volodko')]
prices = prices[~prices.name.str.contains('Marc Auge')]
prices = prices[~prices.name.str.contains('Marc Baro')]
prices = prices[~prices.name.str.contains('Marc Olivier Doue')]
prices = prices[~prices.name.str.contains('Marcelo Meli')]
prices = prices[~prices.name.str.contains('Marcelo RyanMarcelo Ryan')]
prices = prices[~prices.name.str.contains('Marcio AzevedoMarcio Azevedo')]
prices = prices[~prices.name.str.contains('Marcio Mossoro')]
prices = prices[~prices.name.str.contains('Marco Torsiglieri')]
prices = prices[~prices.name.str.contains('Marcos Abreliano')]
prices = prices[~prices.name.str.contains('Marcos AndreMarcos Andre')]
prices = prices[~prices.name.str.contains('Marcos Angeleri')]
prices = prices[~prices.name.str.contains('Marcos BritoMarcos Brito')]
prices = prices[~prices.name.str.contains('Marcos de Sousa')]
prices = prices[~prices.name.str.contains('Marcos Landaburu')]
prices = prices[~prices.name.str.contains('Marcos Ledesma')]
prices = prices[~prices.name.str.contains('Marcos PauloMarcos Paulo')]
prices = prices[~prices.name.str.contains('Marcos Peano')]
prices = prices[~prices.name.str.contains('Marcos RochaMarcos Rocha')]
prices = prices[~prices.name.str.contains('Marcos Sanchez')]
prices = prices[~prices.name.str.contains('Mariano Diaz')]
prices = prices[~prices.name.str.contains('Mariano Vazquez')]
prices = prices[~prices.name.str.contains('Mark Kosznovszky')]
prices = prices[~prices.name.str.contains('Markus Schubert')]
prices = prices[~prices.name.str.contains('Marky Delgado')]
prices = prices[~prices.name.str.contains('Marley Ake')]
prices = prices[~prices.name.str.contains('Marley Marshall')]
prices = prices[~prices.name.str.contains('Martin Garay')]
prices = prices[~prices.name.str.contains('Martin Garcia')]
prices = prices[~prices.name.str.contains('Matheo Vroman')]
prices = prices[~prices.name.str.contains('Matheus FerrazMatheus Ferraz')]
prices = prices[~prices.name.str.contains('Matheus Kayser')]
prices = prices[~prices.name.str.contains('Matheus Martins')]
prices = prices[~prices.name.str.contains('Matheus Mascarenhas')]
prices = prices[~prices.name.str.contains('Matheus MendesMatheus Mendes')]
prices = prices[~prices.name.str.contains('Matheus NogueiraMatheus Nogueira')]
prices = prices[~prices.name.str.contains('Matheus Teixeira')]
prices = prices[~prices.name.str.contains('Mathias Abero')]
prices = prices[~prices.name.str.contains('Mathias Olivera')]
prices = prices[~prices.name.str.contains('Mathias Suarez')]
prices = prices[~prices.name.str.contains('Matias Belloso')]
prices = prices[~prices.name.str.contains('Matias Borgogno')]
prices = prices[~prices.name.str.contains('Matias Cordoba')]
prices = prices[~prices.name.str.contains('Matias Escudero')]
prices = prices[~prices.name.str.contains('Matias Fritzler')]
prices = prices[~prices.name.str.contains('Matias Garcia')]
prices = prices[~prices.name.str.contains('Matias Gomez')]
prices = prices[~prices.name.str.contains('Matias Gonzalez')]
prices = prices[~prices.name.str.contains('Matias Laba')]
prices = prices[~prices.name.str.contains('Matias Moya')]
prices = prices[~prices.name.str.contains('Matias Palavecino')]
prices = prices[~prices.name.str.contains('Matias Pellegrini')]
prices = prices[~prices.name.str.contains('Matias Pisano')]
prices = prices[~prices.name.str.contains('Matias Ramirez')]
prices = prices[~prices.name.str.contains('Matias Rodriguez')]
prices = prices[~prices.name.str.contains('Matias Sosa')]
prices = prices[~prices.name.str.contains('Matias Tagliamonte')]
prices = prices[~prices.name.str.contains('Matias Tissera')]
prices = prices[~prices.name.str.contains('Matias Villarreal')]
prices = prices[~prices.name.str.contains('Mauro Ortiz')]
prices = prices[~prices.name.str.contains('Max ClarkMax Clark')]
prices = prices[~prices.name.str.contains('Maxwell Haygarth')]
prices = prices[~prices.name.str.contains('Maycon CleitonMaycon Cleiton')]
prices = prices[~prices.name.str.contains('Mehmet Can Aydin')]
prices = prices[~prices.name.str.contains('Mehmet Erdem Ugurlu')]
prices = prices[~prices.name.str.contains('Mehmet Goktug Bakirbas')]
prices = prices[~prices.name.str.contains('Mehmet Umut Nayir')]
prices = prices[~prices.name.str.contains('Melvin Minter')]
prices = prices[~prices.name.str.contains('Melvin Ramusovic')]
prices = prices[~prices.name.str.contains('Melvin Sitti')]
prices = prices[~prices.name.str.contains('Meme Placca Fessou')]
prices = prices[~prices.name.str.contains('Menno Koch')]
prices = prices[~prices.name.str.contains('Mert Cetin')]
prices = prices[~prices.name.str.contains('Mert Gunok')]
prices = prices[~prices.name.str.contains('Mert KulaMert Kula')]
prices = prices[~prices.name.str.contains('Mert Sarikus')]
prices = prices[~prices.name.str.contains('Merveille Goblet')]
prices = prices[~prices.name.str.contains('Micah Obiero')]
prices = prices[~prices.name.str.contains('Michael Chacon')]
prices = prices[~prices.name.str.contains('Michael Lang')]
prices = prices[~prices.name.str.contains('Michel Araujo')]
prices = prices[~prices.name.str.contains('Michel Herrero')]
prices = prices[~prices.name.str.contains('Michel Macedo')]
prices = prices[~prices.name.str.contains('MichelMichel')]
prices = prices[~prices.name.str.contains('Miguel Barbieri')]
prices = prices[~prices.name.str.contains('Miguel Berry')]
prices = prices[~prices.name.str.contains('Mikey Ambrose')]
prices = prices[~prices.name.str.contains('Mikey Johnston')]
prices = prices[~prices.name.str.contains('Mohamed-Ali Cho')]
prices = prices[~prices.name.str.contains('Mohammed Kamara')]
prices = prices[~prices.name.str.contains('Mohammed Kesselly Kamara')]
prices = prices[~prices.name.str.contains('Mohammed Rabiu Al-Hassan')]
prices = prices[~prices.name.str.contains('Moises Caicedo')]
prices = prices[~prices.name.str.contains('Morgan Boyes')]
prices = prices[~prices.name.str.contains('Morgan Feeney')]
prices = prices[~prices.name.str.contains('Morgan Ferrier')]
prices = prices[~prices.name.str.contains('Morgan Penfold')]
prices = prices[~prices.name.str.contains('Mustafa Seyhan')]
prices = prices[~prices.name.str.contains('Mustafa Yumlu')]
prices = prices[~prices.name.str.contains('Mustapha Bundu')]
prices = prices[~prices.name.str.contains('Mustapha Olagunju')]
prices = prices[~prices.name.str.contains('Muzaffer Kablan')]
prices = prices[~prices.name.str.contains('Myles Judd')]
prices = prices[~prices.name.str.contains('Nacho Fernandez')]
prices = prices[~prices.name.str.contains('Nacho Martinez')]
prices = prices[~prices.name.str.contains('Nadir Ciftci')]
prices = prices[~prices.name.str.contains('Nadir Zeineddin')]
prices = prices[~prices.name.str.contains('Nahuel Arena')]
prices = prices[~prices.name.str.contains('Nahuel Banegas')]
prices = prices[~prices.name.str.contains('Nahuel Bustos')]
prices = prices[~prices.name.str.contains('Nahuel Iribarren')]
prices = prices[~prices.name.str.contains('Nair Tiknizyan')]
prices = prices[~prices.name.str.contains('Nando Pijnaker')]
prices = prices[~prices.name.str.contains('Nelson Iseguan')]
prices = prices[~prices.name.str.contains('Nestor Ortigoza')]
prices = prices[~prices.name.str.contains('Nico Gordon')]
prices = prices[~prices.name.str.contains('Nico Williams')]
prices = prices[~prices.name.str.contains('Nicola Borghetto')]
prices = prices[~prices.name.str.contains('Nicola Ravaglia')]
prices = prices[~prices.name.str.contains('Nicola Zalewski')]
prices = prices[~prices.name.str.contains('Nicolas Andereggen')]
prices = prices[~prices.name.str.contains('Nicolas Avellaneda')]
prices = prices[~prices.name.str.contains('Nicolas Blandi')]
prices = prices[~prices.name.str.contains('Nicolas Burgy')]
prices = prices[~prices.name.str.contains('Nicolas Contin')]
prices = prices[~prices.name.str.contains('Nicolas Correa')]
prices = prices[~prices.name.str.contains('Nicolas Domingo')]
prices = prices[~prices.name.str.contains('Nicolas Femia')]
prices = prices[~prices.name.str.contains('Nicolas Fernandez Mercau')]
prices = prices[~prices.name.str.contains('Nicolas Ferreyra')]
prices = prices[~prices.name.str.contains('Nicolas Figal')]
prices = prices[~prices.name.str.contains('Nicolas Gimenez')]
prices = prices[~prices.name.str.contains('Nicolas Lamendola')]
prices = prices[~prices.name.str.contains('Nicolas Linares')]
prices = prices[~prices.name.str.contains('Nicolas Mana')]
prices = prices[~prices.name.str.contains('Nicolas Mazzola')]
prices = prices[~prices.name.str.contains('Nicolas Messiniti')]
prices = prices[~prices.name.str.contains('Nicolas Morgantini')]
prices = prices[~prices.name.str.contains('Nicolas Romat')]
prices = prices[~prices.name.str.contains('Nicolas Romero')]
prices = prices[~prices.name.str.contains('Nicolas Schiappacasse')]
prices = prices[~prices.name.str.contains('Nicolas Thaller')]
prices = prices[~prices.name.str.contains('Nicolas Zalazar')]
prices = prices[~prices.name.str.contains('Nicolo Zaniolo')]
prices = prices[~prices.name.str.contains('Nigel Bertrams')]
prices = prices[~prices.name.str.contains('Nino ParaibaNino Paraiba')]
prices = prices[~prices.name.str.contains('O\'Shane Stewart')]
prices = prices[~prices.name.str.contains('Obite Evan Ndicka')]
prices = prices[~prices.name.str.contains('Odion Ighalo')]
prices = prices[~prices.name.str.contains('Odise Roshi')]
prices = prices[~prices.name.str.contains('Ody AlfaOdy Alfa')]
prices = prices[~prices.name.str.contains('Oguzhan Capar')]
prices = prices[~prices.name.str.contains('Oier Sanjurjo')]
prices = prices[~prices.name.str.contains('OierOier')]
prices = prices[~prices.name.str.contains('Okechukwu Azubuike')]
prices = prices[~prices.name.str.contains('Owen Windsor')]
prices = prices[~prices.name.str.contains('Owura Edwards')]
prices = prices[~prices.name.str.contains('Ozan Can Oruc')]
prices = prices[~prices.name.str.contains('Ozzie Cisneros')]
prices = prices[~prices.name.str.contains('Pablo DyegoPablo Dyego')]
prices = prices[~prices.name.str.contains('Pablo MaiaPablo Maia')]
prices = prices[~prices.name.str.contains('Pablo Minissale')]
prices = prices[~prices.name.str.contains('Pablo OroPablo Oro')]
prices = prices[~prices.name.str.contains('Pablo PardalPablo Pardal')]
prices = prices[~prices.name.str.contains('Pablo Velazquez')]
prices = prices[~prices.name.str.contains('Paulo MirandaPaulo Miranda')]
prices = prices[~prices.name.str.contains('Paulo VictorPaulo Victor')]
prices = prices[~prices.name.str.contains('Pedro RangelPedro Rangel')]
prices = prices[~prices.name.str.contains('Pedro Rocha')]
prices = prices[~prices.name.str.contains('PeglowPeglow')]
prices = prices[~prices.name.str.contains('PerottiPerotti')]
prices = prices[~prices.name.str.contains('Philip Turpitz')]
prices = prices[~prices.name.str.contains('Piero Hincapie')]
prices = prices[~prices.name.str.contains('Pol Garcia')]
prices = prices[~prices.name.str.contains('PopPop')]
prices = prices[~prices.name.str.contains('Praxedes')]
prices = prices[~prices.name.str.contains('Quentin Westberg')]
prices = prices[~prices.name.str.contains('Radoslaw Murawski')]
prices = prices[~prices.name.str.contains('Rafa SilvaRafa SilvaRight Midfiel')]
prices = prices[~prices.name.str.contains('Rafael Borre')]
prices = prices[~prices.name.str.contains('Rafael Garcia')]
prices = prices[~prices.name.str.contains('Rafael GrampolaRafael Grampola')]
prices = prices[~prices.name.str.contains('Rafael Longuine')]
prices = prices[~prices.name.str.contains('Ramon Arias')]
prices = prices[~prices.name.str.contains('Ramon Azeez')]
prices = prices[~prices.name.str.contains('Ramon Folch')]
prices = prices[~prices.name.str.contains('Ramon ten Hove')]
prices = prices[~prices.name.str.contains('RanielRaniel')]
prices = prices[~prices.name.str.contains('Raul Carnero')]
prices = prices[~prices.name.str.contains('Raul Lozano')]
prices = prices[~prices.name.str.contains('Raul Navas')]
prices = prices[~prices.name.str.contains('Raul Silva')]
prices = prices[~prices.name.str.contains('Ravel Morrison')]
prices = prices[~prices.name.str.contains('Reinaldo Lenis')]
prices = prices[~prices.name.str.contains('Renan Ribeiro')]
prices = prices[~prices.name.str.contains('Reynaldo')]
prices = prices[~prices.name.str.contains('Ricardo Velho')]
prices = prices[~prices.name.str.contains('Richecard Richard')]
prices = prices[~prices.name.str.contains('Richie Towell')]
prices = prices[~prices.name.str.contains('Rob Dickie')]
prices = prices[~prices.name.str.contains('Rob Elliot')]
prices = prices[~prices.name.str.contains('Rob HallRob Hall')]
prices = prices[~prices.name.str.contains('Robby McCrorie')]
prices = prices[~prices.name.str.contains('Rober Correa')]
prices = prices[~prices.name.str.contains('RobersonRoberson')]
prices = prices[~prices.name.str.contains('Roberto Jimenez')]
prices = prices[~prices.name.str.contains('Robinho')]
prices = prices[~prices.name.str.contains('Robson ReisRobson Reis')]
prices = prices[~prices.name.str.contains('RodineiRodinei')]
prices = prices[~prices.name.str.contains('Rodri Sanchez')]
prices = prices[~prices.name.str.contains('Rodrigo Arciero')]
prices = prices[~prices.name.str.contains('Rodrigo Gomez')]
prices = prices[~prices.name.str.contains('Rodrigo Herrera')]
prices = prices[~prices.name.str.contains('Rodrigo LindosoRodrigo Lindoso')]
prices = prices[~prices.name.str.contains('Rodrigo MoledoRodrigo Moledo')]
prices = prices[~prices.name.str.contains('Rodrigo Nestor')]
prices = prices[~prices.name.str.contains('Rodrigo Salinas')]
prices = prices[~prices.name.str.contains('Rodrigo Sanchez')]
prices = prices[~prices.name.str.contains('Rodrigo VarandaRodrigo Varanda')]
prices = prices[~prices.name.str.contains('Rogerio Santos')]
prices = prices[~prices.name.str.contains('Rolando Garcia')]
prices = prices[~prices.name.str.contains('Roman Eremenko')]
prices = prices[~prices.name.str.contains('Roman Martinez')]
prices = prices[~prices.name.str.contains('Romulo Otero')]
prices = prices[~prices.name.str.contains('Ron-Thorben Hoffmann')]
prices = prices[~prices.name.str.contains('Ruben Botta')]
prices = prices[~prices.name.str.contains('Ruben Fonseca')]
prices = prices[~prices.name.str.contains('Ruben Pardo')]
prices = prices[~prices.name.str.contains('Ruwen Werthmuller')]
prices = prices[~prices.name.str.contains('Saba Lobjanidze')]
prices = prices[~prices.name.str.contains('Saba Lobzhanidze')]
prices = prices[~prices.name.str.contains('Said Bakari')]
prices = prices[~prices.name.str.contains('Saidou Sow')]
prices = prices[~prices.name.str.contains('Saliou Sane')]
prices = prices[~prices.name.str.contains('Salva Ferrer')]
prices = prices[~prices.name.str.contains('Salvador Ferrer')]
prices = prices[~prices.name.str.contains('Sam BoothSam Booth')]
prices = prices[~prices.name.str.contains('Sam ByramSam Byram')]
prices = prices[~prices.name.str.contains('Sammie McLeod')]
prices = prices[~prices.name.str.contains('Sammy Bossut')]
prices = prices[~prices.name.str.contains('Sander Coopman')]
prices = prices[~prices.name.str.contains('Santiago Caseres')]
prices = prices[~prices.name.str.contains('Santiago Garcia')]
prices = prices[~prices.name.str.contains('Santiago Rodriguez')]
prices = prices[~prices.name.str.contains('Santiago Simon')]
prices = prices[~prices.name.str.contains('Santiago Trellez')]
prices = prices[~prices.name.str.contains('Santos Borre')]
prices = prices[~prices.name.str.contains('SaraviaSaravia')]
prices = prices[~prices.name.str.contains('Sarpreet Singh')]
prices = prices[~prices.name.str.contains('Saul Salcedo')]
prices = prices[~prices.name.str.contains('Sean Slaven')]
prices = prices[~prices.name.str.contains('Sebastian Assis')]
prices = prices[~prices.name.str.contains('Sebastian Blanco')]
prices = prices[~prices.name.str.contains('Sebastian Dubarbier')]
prices = prices[~prices.name.str.contains('Sebastian Meza')]
prices = prices[~prices.name.str.contains('Sebastian Ramirez')]
prices = prices[~prices.name.str.contains('Sebastian Sosa')]
prices = prices[~prices.name.str.contains('Sebastien Thill')]
prices = prices[~prices.name.str.contains('Sega Coulibaly')]
prices = prices[~prices.name.str.contains('Senne Lammens')]
prices = prices[~prices.name.str.contains('Sergi Garcia')]
prices = prices[~prices.name.str.contains('Sergio Benito')]
prices = prices[~prices.name.str.contains('Sergio Gonzalez')]
prices = prices[~prices.name.str.contains('Seydou SySeydou Sy')]
prices = prices[~prices.name.str.contains('Seyed Majid Hosseini')]
prices = prices[~prices.name.str.contains('Shadrach Ogie')]
prices = prices[~prices.name.str.contains('Shahriar Moghanlou')]
prices = prices[~prices.name.str.contains('Sidnei Tavares')]
prices = prices[~prices.name.str.contains('Siro Rosane')]
prices = prices[~prices.name.str.contains('SivaldoSivaldo')]
prices = prices[~prices.name.str.contains('Slobodan Rajkovic')]
prices = prices[~prices.name.str.contains('Son Heung-Min')]
prices = prices[~prices.name.str.contains('Sonny CoxSonny Cox')]
prices = prices[~prices.name.str.contains('Sonny Graham')]
prices = prices[~prices.name.str.contains('Sori ManeSori Mane')]
prices = prices[~prices.name.str.contains('Soslan Takazov')]
prices = prices[~prices.name.str.contains('Suleyman Cebeci')]
prices = prices[~prices.name.str.contains('Sylian Mokono')]
prices = prices[~prices.name.str.contains('TabataTabata')]
prices = prices[~prices.name.str.contains('Tahvon Campbell')]
prices = prices[~prices.name.str.contains('TallesTalles')]
prices = prices[~prices.name.str.contains('Tariq Uwakwe')]
prices = prices[~prices.name.str.contains('Tein Troost')]
prices = prices[~prices.name.str.contains('Telmo ArcanjoTelmo Arcanjo')]
prices = prices[~prices.name.str.contains('Theo Leoni')]
prices = prices[~prices.name.str.contains('Thiago RibeiroThiago Ribeiro')]
prices = prices[~prices.name.str.contains('Thomas O\'Connor')]
prices = prices[~prices.name.str.contains('Thomas Ortega')]
prices = prices[~prices.name.str.contains('Thomas Ouwejan')]
prices = prices[~prices.name.str.contains('Thomas Toure')]
prices = prices[~prices.name.str.contains('ThulerThuler')]
prices = prices[~prices.name.str.contains('Thuram')]
prices = prices[~prices.name.str.contains('ThyllerThyller')]
prices = prices[~prices.name.str.contains('TiagoTiago')]
prices = prices[~prices.name.str.contains('Tibet Oniz')]
prices = prices[~prices.name.str.contains('Timmy Abraham')]
prices = prices[~prices.name.str.contains('Timothe Nkada')]
prices = prices[~prices.name.str.contains('Tobias Zarate')]
prices = prices[~prices.name.str.contains('Tom HamerTom Hamer')]
prices = prices[~prices.name.str.contains('Tomas Andrade')]
prices = prices[~prices.name.str.contains('Tomas Cardona')]
prices = prices[~prices.name.str.contains('Tomas Cavanagh')]
prices = prices[~prices.name.str.contains('Tomas Durso')]
prices = prices[~prices.name.str.contains('Tomas Escalante')]
prices = prices[~prices.name.str.contains('Tomas Fernandez')]
prices = prices[~prices.name.str.contains('Tomas Marchiori')]
prices = prices[~prices.name.str.contains('Tomas Martinez')]
prices = prices[~prices.name.str.contains('Tomas Ortiz')]
prices = prices[~prices.name.str.contains('Tomas Romero')]
prices = prices[~prices.name.str.contains('Tomas Sandoval')]
prices = prices[~prices.name.str.contains('Tomas Silva')]
prices = prices[~prices.name.str.contains('TQ AddyTQ Addy')]
prices = prices[~prices.name.str.contains('Tyreece John-Jules')]
prices = prices[~prices.name.str.contains('Ugo Bertelli')]
prices = prices[~prices.name.str.contains('Uriel Ramirez')]
prices = prices[~prices.name.str.contains('Vani Da Silva')]
prices = prices[~prices.name.str.contains('Vaso Vasic')]
prices = prices[~prices.name.str.contains('Vaughn Covil')]
prices = prices[~prices.name.str.contains('Victor Camarasa')]
prices = prices[~prices.name.str.contains('Victor FerrazVictor Ferraz')]
prices = prices[~prices.name.str.contains('Victor KawaVictor Kawaattac')]
prices = prices[~prices.name.str.contains('Victor YanVictor Yan')]
prices = prices[~prices.name.str.contains('VinaVina')]
prices = prices[~prices.name.str.contains('Vincent Muller')]
prices = prices[~prices.name.str.contains('Vinicius BalieiroVinicius Balieiro')]
prices = prices[~prices.name.str.contains('Vinicius FoguinhoVinicius Foguinho')]
prices = prices[~prices.name.str.contains('Vinicius MingottiVinicius Mingotti')]
prices = prices[~prices.name.str.contains('Vinicius SilvestreVinicius Silvestre')]
prices = prices[~prices.name.str.contains('Vitor Hugodefenc')]
prices = prices[~prices.name.str.contains('Vitor JacareVitor Jacare')]
prices = prices[~prices.name.str.contains('Vitor RicardoVitor Ricardo')]
prices = prices[~prices.name.str.contains('WalberWalber')]
prices = prices[~prices.name.str.contains('WalceWalce')]
prices = prices[~prices.name.str.contains('WallisonWallison')]
prices = prices[~prices.name.str.contains('Walter Andrade')]
prices = prices[~prices.name.str.contains('Walter Kannemann')]
prices = prices[~prices.name.str.contains('Warner Hahn')]
prices = prices[~prices.name.str.contains('Wayne Hennessey')]
prices = prices[~prices.name.str.contains('Wayne Rooney')]
prices = prices[~prices.name.str.contains('Welinton Junior')]
prices = prices[~prices.name.str.contains('Will VintWill Vint')]
prices = prices[~prices.name.str.contains('William MatheusWilliam Matheus')]
prices = prices[~prices.name.str.contains('William OliveiraWilliam Oliveira')]
prices = prices[~prices.name.str.contains('William Simba')]
prices = prices[~prices.name.str.contains('William Smallbone')]
prices = prices[~prices.name.str.contains('William Togui')]
prices = prices[~prices.name.str.contains('Williams Barlasina')]
prices = prices[~prices.name.str.contains('Willian MaranhaoWillian Maranhao')]
prices = prices[~prices.name.str.contains('Wim Vanmarsenille')]
prices = prices[~prices.name.str.contains('Wiris Gustavo')]
prices = prices[~prices.name.str.contains('Wisdom Amey')]
prices = prices[~prices.name.str.contains('Woo-yeong Jeong')]
prices = prices[~prices.name.str.contains('XadasXadas')]
prices = prices[~prices.name.str.contains('Xander Severina')]
prices = prices[~prices.name.str.contains('Xandro Schenk')]
prices = prices[~prices.name.str.contains('XavierXavier')]
prices = prices[~prices.name.str.contains('Yahya Soumare')]
prices = prices[~prices.name.str.contains('Yair Arismendi')]
prices = prices[~prices.name.str.contains('Yan SasseYan Sasse')]
prices = prices[~prices.name.str.contains('Yanis Lhery')]
prices = prices[~prices.name.str.contains('Yann M\'Vila')]
prices = prices[~prices.name.str.contains('Yasin Oztekin')]
prices = prices[~prices.name.str.contains('Yaya Soumare')]
prices = prices[~prices.name.str.contains('Yildirim Mert Ã‡etin')]
prices = prices[~prices.name.str.contains('Younes Belhanda')]
prices = prices[~prices.name.str.contains('Youssef Ait Bennasser')]
prices = prices[~prices.name.str.contains('Youssuf Sylla')]
prices = prices[~prices.name.str.contains('Yuki Kobayashi')]
prices = prices[~prices.name.str.contains('Yun Il-Lok')]
prices = prices[~prices.name.str.contains('Yunus Bahadir')]
prices = prices[~prices.name.str.contains('Yusuf Ozdemir')]

In [430]:
#remove all the accents
prices['name'] = prices['name'].str.replace('ã','a')
df['Name'] = df['Name'].str.replace('ã','a')

prices['name'] = prices['name'].str.replace('â','a')
df['Name'] = df['Name'].str.replace('â','a')

prices['name'] = prices['name'].str.replace('á','a')
df['Name'] = df['Name'].str.replace('á','a')

prices['name'] = prices['name'].str.replace('ä','a')
df['Name'] = df['Name'].str.replace('ä','a')

prices['name'] = prices['name'].str.replace('à','a')
df['Name'] = df['Name'].str.replace('à','a')

prices['name'] = prices['name'].str.replace('å','a')
df['Name'] = df['Name'].str.replace('å','a')

prices['name'] = prices['name'].str.replace('è','e')
df['Name'] = df['Name'].str.replace('è','e')

prices['name'] = prices['name'].str.replace('é','e')
df['Name'] = df['Name'].str.replace('é','e')

prices['name'] = prices['name'].str.replace('ë','e')
df['Name'] = df['Name'].str.replace('ë','e')

prices['name'] = prices['name'].str.replace('ẽ','e')
df['Name'] = df['Name'].str.replace('ẽ','e')

prices['name'] = prices['name'].str.replace('ẽ','e')
df['Name'] = df['Name'].str.replace('ê','e')

prices['name'] = prices['name'].str.replace('É','E')
df['Name'] = df['Name'].str.replace('É','E')

prices['name'] = prices['name'].str.replace('í','i')
df['Name'] = df['Name'].str.replace('í','i')

prices['name'] = prices['name'].str.replace('ì','i')
df['Name'] = df['Name'].str.replace('ì','i')

prices['name'] = prices['name'].str.replace('ĩ','i')
df['Name'] = df['Name'].str.replace('ĩ','i')

prices['name'] = prices['name'].str.replace('ï','i')
df['Name'] = df['Name'].str.replace('ï','i')

prices['name'] = prices['name'].str.replace('ó','o')
df['Name'] = df['Name'].str.replace('ó','o')

prices['name'] = prices['name'].str.replace('ò','o')
df['Name'] = df['Name'].str.replace('ò','o')

prices['name'] = prices['name'].str.replace('õ','o')
df['Name'] = df['Name'].str.replace('õ','o')

prices['name'] = prices['name'].str.replace('ô','o')
df['Name'] = df['Name'].str.replace('ô','o')

prices['name'] = prices['name'].str.replace('ô','o')
df['Name'] = df['Name'].str.replace('ô','o')

prices['name'] = prices['name'].str.replace('ø','o')
df['Name'] = df['Name'].str.replace('ø','o')

prices['name'] = prices['name'].str.replace('Ö','O')
df['Name'] = df['Name'].str.replace('Ö','O')

prices['name'] = prices['name'].str.replace('Ø','O')
df['Name'] = df['Name'].str.replace('Ø','O')

prices['name'] = prices['name'].str.replace('Ó','O')
df['Name'] = df['Name'].str.replace('Ó','O')

prices['name'] = prices['name'].str.replace('ú','u')
df['Name'] = df['Name'].str.replace('ú','u')

prices['name'] = prices['name'].str.replace(' ù','u')
df['Name'] = df['Name'].str.replace(' ù','u')

prices['name'] = prices['name'].str.replace('ũ','u')
df['Name'] = df['Name'].str.replace('ũ','u')

prices['name'] = prices['name'].str.replace('ü','u')
df['Name'] = df['Name'].str.replace('ü','u')

prices['name'] = prices['name'].str.replace('ç','c')
df['Name'] = df['Name'].str.replace('ç','c')

prices['name'] = prices['name'].str.replace('ñ','n')
df['Name'] = df['Name'].str.replace('ñ','n')

prices['name'] = prices['name'].str.replace('Á','A')
df['Name'] = df['Name'].str.replace('Á','A')

prices['name'] = prices['name'].str.replace('š','s')
df['Name'] = df['Name'].str.replace('š','s')


In [431]:
df = df[~df.Name.str.contains('Alfon')]
df = df[~df.Name.str.contains('Carlos Domínguez')]
df = df[~df.Name.str.contains('Costel Pantilimon')]
df = df[~df.Name.str.contains('C.J. Sapong')]
df = df[~df.Name.str.contains('Carlos Dominguez')]
df = df[~df.Name.str.contains('Dan Ndoye')]
df = df[~df.Name.str.contains('Daniel Iversen')]
df = df[~df.Name.str.contains('Daniel Sappa')]
df = df[~df.Name.str.contains('Daniele De Rossi')]
df = df[~df.Name.str.contains('Dario Essugo')]
df = df[~df.Name.str.contains('Darnell Johnson')]
df = df[~df.Name.str.contains('Darren Keet')]
df = df[~df.Name.str.contains('David Abraham')]
df = df[~df.Name.str.contains('David Barbona')]
df = df[~df.Name.str.contains('Allan-Romeo Nyom')]
df = df[~df.Name.str.contains('Braian Cufre')]
df = df[~df.Name.str.contains('Cesar Ibañez')]
df = df[~df.Name.str.contains('Charalambos Makridis')]
df = df[~df.Name.str.contains('Charlie O\'Connell')]
df = df[~df.Name.str.contains('Danilo Ortiz')]
df = df[~df.Name.str.contains('David Henen')]
df = df[~df.Name.str.contains('Di\'Shon Bernard')]
df = df[~df.Name.str.contains('Alex Bono')]
df = df[~df.Name.str.contains('Alex Martin')]
df = df[~df.Name.str.contains('Alex Millan')]
df = df[~df.Name.str.contains('Alex Vallejo')]
df = df[~df.Name.str.contains('Alvaro Bravo')]
df = df[~df.Name.str.contains('Alvaro Fernandez')]
df = df[~df.Name.str.contains('Alvaro Gimenez')]
df = df[~df.Name.str.contains('Angelo Preciado')]
df = df[~df.Name.str.contains('Angelo Sagal')]
df = df[~df.Name.str.contains('Aykut Ozer')]
df = df[~df.Name.str.contains('Casper Staring')]
df = df[~df.Name.str.contains('Cesar Ibanez')]
df = df[~df.Name.str.contains('Djibril Dianessy')]
df = df[~df.Name.str.contains('Dogan Alemdar')]
df = df[~df.Name.str.contains('Dogan Erdogan')]
df = df[~df.Name.str.contains('Dominic Telford')]
df = df[~df.Name.str.contains('Elias Sierra-Cappelleti')]
df = df[~df.Name.str.contains('Enaut Mendia')]
df = df[~df.Name.str.contains('Enrico Hernandez')]
df = df[~df.Name.str.contains('Evgeni Gapon')]
df = df[~df.Name.str.contains('Evgeniy Tatarinov')]
df = df[~df.Name.str.contains('Evgeny Kharin')]
df = df[~df.Name.str.contains('Ewan Henderson')]
df = df[~df.Name.str.contains('Fabian Cubero')]
df = df[~df.Name.str.contains('Fabio Abreu')]
df = df[~df.Name.str.contains('Facundo Sanchez')]
df = df[~df.Name.str.contains('Federico Martinez')]
df = df[~df.Name.str.contains('Fehmi Mert Gunok')]
df = df[~df.Name.str.contains('Fernando Nino')]
df = df[~df.Name.str.contains('Filip Moller Delaveris')]
df = df[~df.Name.str.contains('Fin Bartels')]
df = df[~df.Name.str.contains('Francisco Ramirez')]
df = df[~df.Name.str.contains('Franck-Yves Bambock')]
df = df[~df.Name.str.contains('Franco Gonzalez')]
df = df[~df.Name.str.contains('Franco Perez')]
df = df[~df.Name.str.contains('Frederik Sörensen')]
df = df[~df.Name.str.contains('Gabriel Diaz')]
df = df[~df.Name.str.contains('Gabriel Menino')]
df = df[~df.Name.str.contains('Gaston Alvarez Suarez')]
df = df[~df.Name.str.contains('Gervasio Nunez')]
df = df[~df.Name.str.contains('Gonzalo Rodriguez')]
df = df[~df.Name.str.contains('Gonzalo Veron')]
df = df[~df.Name.str.contains('Guillermo Fernandez')]
df = df[~df.Name.str.contains('Halil Pehlivan')]
df = df[~df.Name.str.contains('Hasan Huseyin Acar')]
df = df[~df.Name.str.contains('Hernan Burbano')]
df = df[~df.Name.str.contains('Hwang In-Beom')]
df = df[~df.Name.str.contains('Hwang Ui-Jo')]
df = df[~df.Name.str.contains('Ibrahim Sissoko')]
df = df[~df.Name.str.contains('Ibrahim Tsallagov')]
df = df[~df.Name.str.contains('Ibrahima Camara')]
df = df[~df.Name.str.contains('Igor Bezdenezhnykh')]
df = df[~df.Name.str.contains('Ilker Karakas')]
df = df[~df.Name.str.contains('Islambek Kuat')]
df = df[~df.Name.str.contains('Ismail Yuksek')]
df = df[~df.Name.str.contains('Moritz Römling')]
df = df[~df.Name.str.contains('Mustafa Çecenoglu')]
df = df[~df.Name.str.contains('RegisRegis')]
df = df[~df.Name.str.contains('Jonas Aguirre')]
df = df[~df.Name.str.contains('Jonas Sterner')]
df = df[~df.Name.str.contains('Jonas Toro')]
df = df[~df.Name.str.contains('Jonatan Gomez')]
df = df[~df.Name.str.contains('Jonathan Zacaria')]
df = df[~df.Name.str.contains('Jony Alamo')]
df = df[~df.Name.str.contains('Jop van den Avert')]
df = df[~df.Name.str.contains('Jordi Osei-Tutu')]
df = df[~df.Name.str.contains('Jores Rahou')]
df = df[~df.Name.str.contains('Jorge Caceres')]
df = df[~df.Name.str.contains('Jorge Pereyra Diaz')]
df = df[~df.Name.str.contains('Jorge Saenz')]
df = df[~df.Name.str.contains('Jorgen Strand Larsen')]
df = df[~df.Name.str.contains('Jose Adolfo Valencia')]
df = df[~df.Name.str.contains('Jose Andres Martinez')]
df = df[~df.Name.str.contains('Jose AngelJose Angel')]
df = df[~df.Name.str.contains('Jose Cifuentes')]
df = df[~df.Name.str.contains('Jose Devecchi')]
df = df[~df.Name.str.contains('Jose Gimenez')]
df = df[~df.Name.str.contains('Jose Gonzalez')]
df = df[~df.Name.str.contains('Jose Luis Fernandez')]
df = df[~df.Name.str.contains('Jose Luis Gomez')]
df = df[~df.Name.str.contains('Jose Luis Rodriguez')]
df = df[~df.Name.str.contains('Jose Luis Sinisterra')]
df = df[~df.Name.str.contains('Jose Manuel Fontan')]
df = df[~df.Name.str.contains('Jose Manuel Lopez')]
df = df[~df.Name.str.contains('Jose Marcelo')]
df = df[~df.Name.str.contains('Jose Maria Gimenez')]
df = df[~df.Name.str.contains('Jose Martinez')]
df = df[~df.Name.str.contains('Jose Moya')]
df = df[~df.Name.str.contains('Josh JackJosh Jack')]
df = df[~df.Name.str.contains('Josh Neufville')]
df = df[~df.Name.str.contains('Josh Penn')]
df = df[~df.Name.str.contains('Josue Ayala')]
df = df[~df.Name.str.contains('Jovanny Bolivar')]
df = df[~df.Name.str.contains('Juan Alvarez')]
df = df[~df.Name.str.contains('Juan Cruz')]
df = df[~df.Name.str.contains('Juan Espinola')]
df = df[~df.Name.str.contains('Juan Gonzalez')]
df = df[~df.Name.str.contains('Juan Ignacio Mendez')]
df = df[~df.Name.str.contains('Juan Jose Caceres')]
df = df[~df.Name.str.contains('Juan Martin Lucero')]
df = df[~df.Name.str.contains('Juan Mendez')]
df = df[~df.Name.str.contains('Juan Pablo Alvarez')]
df = df[~df.Name.str.contains('Juan Pablo Ramirez')]
df = df[~df.Name.str.contains('Juan Roman Pucheta')]
df = df[~df.Name.str.contains('JuanJuan')]
df = df[~df.Name.str.contains('Jubril Okedina')]
df = df[~df.Name.str.contains('Julian Aude')]
df = df[~df.Name.str.contains('Julian Chicco')]
df = df[~df.Name.str.contains('Julian Lopez')]
df = df[~df.Name.str.contains('Julian Malatini')]
df = df[~df.Name.str.contains('Julian Marcioni')]
df = df[~df.Name.str.contains('Julian Navas')]
df = df[~df.Name.str.contains('Julian Velazquez')]
df = df[~df.Name.str.contains('Julio Gonzalez')]
df = df[~df.Name.str.contains('JuniorJunior')]
df = df[~df.Name.str.contains('KaduKadu')]
df = df[~df.Name.str.contains('Kaide Gordon')]
df = df[~df.Name.str.contains('Kaique RochaKaique Rocha')]
df = df[~df.Name.str.contains('KeillerKeiller')]
df = df[~df.Name.str.contains('KelvinKelvin')]
df = df[~df.Name.str.contains('Kemal Ruzgar')]
df = df[~df.Name.str.contains('KenoKeno')]
df = df[~df.Name.str.contains('KenoKeno')]
df = df[~df.Name.str.contains('Kerim Calhanoglu')]
df = df[~df.Name.str.contains('Kevin Boma')]
df = df[~df.Name.str.contains('Kevin Zenon')]
df = df[~df.Name.str.contains('Kian Slor')]
df = df[~df.Name.str.contains('Koffi')]
df = df[~df.Name.str.contains('Konstantin Kovalev')]
df = df[~df.Name.str.contains('Konstantin Rausch')]
df = df[~df.Name.str.contains('Kornell MacDonald')]
df = df[~df.Name.str.contains('Kortne Ford')]
df = df[~df.Name.str.contains('Kostas Stafylidis')]
df = df[~df.Name.str.contains('Laercio Solda')]
df = df[~df.Name.str.contains('LaercioLaercio')]
df = df[~df.Name.str.contains('Lars MaiLars Mai')]
df = df[~df.Name.str.contains('Lasse Sorensen')]
df = df[~df.Name.str.contains('Laureano Punet')]
df = df[~df.Name.str.contains('Lautaro Chavez')]
df = df[~df.Name.str.contains('Lautaro de Leon')]
df = df[~df.Name.str.contains('LazaroLazaro')]
df = df[~df.Name.str.contains('Leandro Contin')]
df = df[~df.Name.str.contains('Leandro Fernandez')]
df = df[~df.Name.str.contains('Lennart Schulze')]
df = df[~df.Name.str.contains('Lee O\'Connor')]
df = df[~df.Name.str.contains('Leo BorgesLeo Borges')]
df = df[~df.Name.str.contains('Leo Gomes')]
df = df[~df.Name.str.contains('Leo NatelLeo Natel')]
df = df[~df.Name.str.contains('Leonel Gonzalez')]
df = df[~df.Name.str.contains('Levi Ntumba')]
df = df[~df.Name.str.contains('Lois Diony')]
df = df[~df.Name.str.contains('Luca Martinez Dupuy')]
df = df[~df.Name.str.contains('Ludvig Ohman')]
df = df[~df.Name.str.contains('Luis Leal')]
df = df[~df.Name.str.contains('Luis Miguel Rodriguez')]
df = df[~df.Name.str.contains('Luis Rodriguez')]
df = df[~df.Name.str.contains('Luis Vazquez')]
df = df[~df.Name.str.contains('Luiz Fernando')]
df = df[~df.Name.str.contains('Luiz JuniorLuiz Junior')]
df = df[~df.Name.str.contains('Luiz Lucio Reis Junior')]
df = df[~df.Name.str.contains('Luiz Otavio')]
df = df[~df.Name.str.contains('Luka Zarandia')]
df = df[~df.Name.str.contains('MaiconMaicon')]
df = df[~df.Name.str.contains('Malik Sellouki')]
df = df[~df.Name.str.contains('Malvind Benning')]
df = df[~df.Name.str.contains('Mamadi Camara')]
df = df[~df.Name.str.contains('Mamadou Coulibaly')]
df = df[~df.Name.str.contains('Marc Auge')]
df = df[~df.Name.str.contains('Marc Baro')]
df = df[~df.Name.str.contains('Marc-Olivier Doue')]
df = df[~df.Name.str.contains('Marcelinho')]
df = df[~df.Name.str.contains('Marcio AzevedoMarcio Azevedo')]
df = df[~df.Name.str.contains('Marcio Mossoro')]
df = df[~df.Name.str.contains('Marcos AndreMarcos Andre')]
df = df[~df.Name.str.contains('Marcos Sanchez')]
df = df[~df.Name.str.contains('Mariano Diaz')]
df = df[~df.Name.str.contains('Mariano Vazquez')]
df = df[~df.Name.str.contains('Mark Kosznovszky')]
df = df[~df.Name.str.contains('Markus Fjortoft')]
df = df[~df.Name.str.contains('Marley Ake')]
df = df[~df.Name.str.contains('Martin Campana')]
df = df[~df.Name.str.contains('Martin Garay')]
df = df[~df.Name.str.contains('Martin Garcia')]
df = df[~df.Name.str.contains('Martin Rabunal')]
df = df[~df.Name.str.contains('Mateus BrunMateus Brun')]
df = df[~df.Name.str.contains('Matheo Vroman')]
df = df[~df.Name.str.contains('Mathias Abero')]
df = df[~df.Name.str.contains('Mathias Kjolo')]
df = df[~df.Name.str.contains('Mathias Suarez')]
df = df[~df.Name.str.contains('Matias Belloso')]
df = df[~df.Name.str.contains('Matias Borgogno')]
df = df[~df.Name.str.contains('Matias Cordoba')]
df = df[~df.Name.str.contains('Matias Escudero')]
df = df[~df.Name.str.contains('Matias Fritzler')]
df = df[~df.Name.str.contains('Matias Garcia')]
df = df[~df.Name.str.contains('Matias Gomez')]
df = df[~df.Name.str.contains('Matias Gonzalez')]
df = df[~df.Name.str.contains('Matias Laba')]
df = df[~df.Name.str.contains('Matias Moya')]
df = df[~df.Name.str.contains('Matias Palavecino')]
df = df[~df.Name.str.contains('Matias Pellegrini')]
df = df[~df.Name.str.contains('Matias Pisano')]
df = df[~df.Name.str.contains('Matias Ramirez')]
df = df[~df.Name.str.contains('Matias Rodriguez')]
df = df[~df.Name.str.contains('Matias Sosa')]
df = df[~df.Name.str.contains('Matias Tagliamonte')]
df = df[~df.Name.str.contains('Matias Tissera')]
df = df[~df.Name.str.contains('Matias Villarreal')]
df = df[~df.Name.str.contains('Mauro Ortiz')]
df = df[~df.Name.str.contains('Max GrunMax Grun')]
df = df[~df.Name.str.contains('Max WalefMax Walef')]
df = df[~df.Name.str.contains('Meme Placca Fessou')]
df = df[~df.Name.str.contains('Mert Gunok')]
df = df[~df.Name.str.contains('Metin Yildirim')]
df = df[~df.Name.str.contains('MetinhoMetinho')]
df = df[~df.Name.str.contains('Michael Chacon')]
df = df[~df.Name.str.contains('Michel Araujo')]
df = df[~df.Name.str.contains('Michel Herrero')]
df = df[~df.Name.str.contains('Michel Macedo')]
df = df[~df.Name.str.contains('MichelMichel')]
df = df[~df.Name.str.contains('Miguel Angel Atienza')]
df = df[~df.Name.str.contains('Miguel Angel Guerrero')]
df = df[~df.Name.str.contains('Miguel Angel Moya')]
df = df[~df.Name.str.contains('Miguel Angel Rubio')]
df = df[~df.Name.str.contains('Mike GrimMike Grim')]
df = df[~df.Name.str.contains('Milton Alvarez')]
df = df[~df.Name.str.contains('Milton Gimenez')]
df = df[~df.Name.str.contains('Miquel Scarlett')]
df = df[~df.Name.str.contains('Mohamed Traore')]
df = df[~df.Name.str.contains('Moises Caicedo')]
df = df[~df.Name.str.contains('Moon-hwan Kim')]
df = df[~df.Name.str.contains('MoraesMoraes')]
df = df[~df.Name.str.contains('Morato')]
df = df[~df.Name.str.contains('Moussa Wague')]
df = df[~df.Name.str.contains('Murat Akca')]
df = df[~df.Name.str.contains('Murat Aksit')]
df = df[~df.Name.str.contains('Murat Sipahioglu')]
df = df[~df.Name.str.contains('MurielMuriel')]
df = df[~df.Name.str.contains('Murilo RangelMurilo Rangel')]
df = df[~df.Name.str.contains('Mustafa Cecenoglu')]
df = df[~df.Name.str.contains('Mustafa Huseyin Seyhan')]
df = df[~df.Name.str.contains('NatanaelNatanael')]
df = df[~df.Name.str.contains('NatanNatan')]
df = df[~df.Name.str.contains('Nathan-Dylan Saliba')]
df = df[~df.Name.str.contains('Nathangelo Markelo')]
df = df[~df.Name.str.contains('Nazareno Solis')]
df = df[~df.Name.str.contains('NeiltonNeilton')]
df = df[~df.Name.str.contains('Nelson Ibanez')]
df = df[~df.Name.str.contains('Nemanja Antonov')]
df = df[~df.Name.str.contains('Nestor Ortigoza')]
df = df[~df.Name.str.contains('Nicolas Andereggen')]
df = df[~df.Name.str.contains('Nicolas Avellaneda')]
df = df[~df.Name.str.contains('Nicolas Blandi')]
df = df[~df.Name.str.contains('Nicolas Burgy')]
df = df[~df.Name.str.contains('Nicolas Contin')]
df = df[~df.Name.str.contains('Nicolas Correa')]
df = df[~df.Name.str.contains('Nicolas Domingo')]
df = df[~df.Name.str.contains('Nicolas Femia')]
df = df[~df.Name.str.contains('Nicolas Fernandez Mercau')]
df = df[~df.Name.str.contains('Nicolas Ferreyra')]
df = df[~df.Name.str.contains('Nicolas Figal')]
df = df[~df.Name.str.contains('Nicolas Gimenez')]
df = df[~df.Name.str.contains('Nicolas Lamendola')]
df = df[~df.Name.str.contains('Nicolas Linares')]
df = df[~df.Name.str.contains('Nicolas Mana')]
df = df[~df.Name.str.contains('Nicolas Mazzola')]
df = df[~df.Name.str.contains('Nicolas Messiniti')]
df = df[~df.Name.str.contains('Nicolas Morgantini')]
df = df[~df.Name.str.contains('Nicolas Olsak')]
df = df[~df.Name.str.contains('Nicolas Penailillo')]
df = df[~df.Name.str.contains('Nicolas Romat')]
df = df[~df.Name.str.contains('Nicolas Romero')]
df = df[~df.Name.str.contains('Nicolas Schiappacasse')]
df = df[~df.Name.str.contains('Nicolas Thaller')]
df = df[~df.Name.str.contains('Nicolas Zalazar')]
df = df[~df.Name.str.contains('Nicolo Zaniolo')]
df = df[~df.Name.str.contains('Nikola Pejčić')]
df = df[~df.Name.str.contains('Nino ParaibaNino Paraiba')]
df = df[~df.Name.str.contains('Omer Beyaz')]
df = df[~df.Name.str.contains('Omer Sismanoglu')]
df = df[~df.Name.str.contains('Oscar Benitez')]
df = df[~df.Name.str.contains('Oscar Linner')]
df = df[~df.Name.str.contains('Oscar Schönfelder')]
df = df[~df.Name.str.contains('OsmanOsman')]
df = df[~df.Name.str.contains('OsvaldoOsvaldo')]
df = df[~df.Name.str.contains('Pablo Velazquez')]
df = df[~df.Name.str.contains('PaulaoPaulao')]
df = df[~df.Name.str.contains('Paulinho BoiaPaulinho Boia')]
df = df[~df.Name.str.contains('Pedro Goncalves')]
df = df[~df.Name.str.contains('Pepê')]
df = df[~df.Name.str.contains('Pepe Bonet')]
df = df[~df.Name.str.contains('Philip Turpitz')]
df = df[~df.Name.str.contains('Piero Hincapie')]
df = df[~df.Name.str.contains('Piris da MottaPiris da Motta')]
df = df[~df.Name.str.contains('Pol Garcia')]
df = df[~df.Name.str.contains('Pote')]
df = df[~df.Name.str.contains('Rafael Borre')]
df = df[~df.Name.str.contains('Rafael Garcia')]
df = df[~df.Name.str.contains('Rahman Bugra Cagiran')]
df = df[~df.Name.str.contains('Ramon Arias')]
df = df[~df.Name.str.contains('Ramon Azeez')]
df = df[~df.Name.str.contains('Ramon Folch')]
df = df[~df.Name.str.contains('Ramon ten Hove')]
df = df[~df.Name.str.contains('Rashad Muhammed')]
df = df[~df.Name.str.contains('Rashid')]
df = df[~df.Name.str.contains('Raul Carnero')]
df = df[~df.Name.str.contains('Raul Lozano')]
df = df[~df.Name.str.contains('Raul Navas')]
df = df[~df.Name.str.contains('Raul Silva')]
df = df[~df.Name.str.contains('Raul SilvaRaul Silva')]
df = df[~df.Name.str.contains('Remeao Hutton')]
df = df[~df.Name.str.contains('RenêRenê')]
df = df[~df.Name.str.contains('Rhys Fenlon')]
df = df[~df.Name.str.contains('Rhys-James Fenlon')]
df = df[~df.Name.str.contains('RibamarRibamar')]
df = df[~df.Name.str.contains('Ricard Sanchez')]
df = df[~df.Name.str.contains('Ricardo Gomes')]
df = df[~df.Name.str.contains('Ricardo SilvaRicardo Silva')]
df = df[~df.Name.str.contains('RildoRildo')]
df = df[~df.Name.str.contains('Roberto Jimenez')]
df = df[~df.Name.str.contains('Rodri Sanchez')]
df = df[~df.Name.str.contains('Rodrigo Sanchez')]
df = df[~df.Name.str.contains('Rodrigues')]
df = df[~df.Name.str.contains('Rogerio Santos')]
df = df[~df.Name.str.contains('Roger IbanezRoger Ibanez')]
df = df[~df.Name.str.contains('Rolando Garcia')]
df = df[~df.Name.str.contains('Roman Martinez')]
df = df[~df.Name.str.contains('Romulo Otero')]
df = df[~df.Name.str.contains('Ruben Botta')]
df = df[~df.Name.str.contains('Ruben Fonseca')]
df = df[~df.Name.str.contains('Ruben Pardo')]
df = df[~df.Name.str.contains('Rui EncarnacaoRui Encarnacao')]
df = df[~df.Name.str.contains('Ruwen Werthmuller')]
df = df[~df.Name.str.contains('Sören Dieckmann')]
df = df[~df.Name.str.contains('Saîf-Eddine Khaoui')]
df = df[~df.Name.str.contains('Said Bakari')]
df = df[~df.Name.str.contains('Saidou Sow')]
df = df[~df.Name.str.contains('Saliou Sane')]
df = df[~df.Name.str.contains('Sandro PerpetuoSandro Perpetuo')]
df = df[~df.Name.str.contains('SandrySandry')]
df = df[~df.Name.str.contains('Santiago Brinone')]
df = df[~df.Name.str.contains('Santiago Caseres')]
df = df[~df.Name.str.contains('Santiago Garcia')]
df = df[~df.Name.str.contains('Santiago Rodriguez')]
df = df[~df.Name.str.contains('Santiago Simon')]
df = df[~df.Name.str.contains('Santiago Trellez')]
df = df[~df.Name.str.contains('Santos Borre')]
df = df[~df.Name.str.contains('Saul Salcedo')]
df = df[~df.Name.str.contains('Savio')]
df = df[~df.Name.str.contains('Sebastian Assis')]
df = df[~df.Name.str.contains('Sebastian Blanco')]
df = df[~df.Name.str.contains('Sebastian Dubarbier')]
df = df[~df.Name.str.contains('Sebastian Meza')]
df = df[~df.Name.str.contains('Sebastian Ramirez')]
df = df[~df.Name.str.contains('Sebastian Sosa')]
df = df[~df.Name.str.contains('Sebastien Thill')]
df = df[~df.Name.str.contains('Sega Coulibaly')]
df = df[~df.Name.str.contains('Sebastian Prödl')]
df = df[~df.Name.str.contains('Sergi Garcia')]
df = df[~df.Name.str.contains('Sergio Gonzalez')]
df = df[~df.Name.str.contains('Siro Rosane')]
df = df[~df.Name.str.contains('Suleyman Cebeci')]
df = df[~df.Name.str.contains('Sven Köhler')]
df = df[~df.Name.str.contains('Theo Leoni')]
df = df[~df.Name.str.contains('Thiago Andrade')]
df = df[~df.Name.str.contains('Thiago CoutoThiago Couto')]
df = df[~df.Name.str.contains('ThiaguinhoThiaguinho')]
df = df[~df.Name.str.contains('Thomas Toure')]
df = df[~df.Name.str.contains('Tibet Oniz')]
df = df[~df.Name.str.contains('Timothe Nkada')]
df = df[~df.Name.str.contains('Tobias Zarate')]
df = df[~df.Name.str.contains('Tomas Andrade')]
df = df[~df.Name.str.contains('Tomas Cardona')]
df = df[~df.Name.str.contains('Tomas Cavanagh')]
df = df[~df.Name.str.contains('Tomas Durso')]
df = df[~df.Name.str.contains('Tomas Escalante')]
df = df[~df.Name.str.contains('Tomas Fernandez')]
df = df[~df.Name.str.contains('Tomas Marchiori')]
df = df[~df.Name.str.contains('Tomas Martinez')]
df = df[~df.Name.str.contains('Tomas Ortiz')]
df = df[~df.Name.str.contains('Tomas Romero')]
df = df[~df.Name.str.contains('Tomas Sandoval')]
df = df[~df.Name.str.contains('Tomas Silva')]
df = df[~df.Name.str.contains('Torben Musel')]
df = df[~df.Name.str.contains('Uriel Ramirez')]
df = df[~df.Name.str.contains('Valentin Larralde')]
df = df[~df.Name.str.contains('Victor Camarasa')]
df = df[~df.Name.str.contains('Victor KawaVictor Kawaattac')]
df = df[~df.Name.str.contains('Vincent Muller')]
df = df[~df.Name.str.contains('ViniciusVinicius')]
df = df[~df.Name.str.contains('Vitor JacareVitor Jacare')]
df = df[~df.Name.str.contains('Vitor Oliveira')]
df = df[~df.Name.str.contains('WagnerWagner')]
df = df[~df.Name.str.contains('WalberWalber')]
df = df[~df.Name.str.contains('Walter Andrade')]
df = df[~df.Name.str.contains('Walter Kannemann')]
df = df[~df.Name.str.contains('WandsonWandson')]
df = df[~df.Name.str.contains('Welinton Junior')]
df = df[~df.Name.str.contains('Wellington NemWellington Nem')]
df = df[~df.Name.str.contains('Wellington TimWellington Tim')]
df = df[~df.Name.str.contains('YagoYago')]
df = df[~df.Name.str.contains('Yahya Soumare')]
df = df[~df.Name.str.contains('Yanis Lhery')]
df = df[~df.Name.str.contains('Yasin Oztekin')]
df = df[~df.Name.str.contains('Yeiler Goez')]
df = df[~df.Name.str.contains('Yildirim Mert Çetin')]
df = df[~df.Name.str.contains('Younes Belhanda')]
df = df[~df.Name.str.contains('Youssef Ait Bennasser')]
df = df[~df.Name.str.contains('Yuri Shafinskiy')]
df = df[~df.Name.str.contains('YuriYuri')]
df = df[~df.Name.str.contains('Yusuf Eren Göktas')]
df = df[~df.Name.str.contains('Ze GabrielZe Gabriel')]
df = df[~df.Name.str.contains('Ze IvaldoZe Ivaldo')]
df = df[~df.Name.str.contains('Ze Oliveira')]
df = df[~df.Name.str.contains('Angel Romero')]
df = df[~df.Name.str.contains('Buiu')]
df = df[~df.Name.str.contains('Dion Lopy')]
df = df[~df.Name.str.contains('Fedor Smolov')]
df = df[~df.Name.str.contains('Karadeniz')]
df = df[~df.Name.str.contains('Haji Mnoga')]
df = df[~df.Name.str.contains('Hakan Arslan')]
df = df[~df.Name.str.contains('Hakan Ozmert')]
df = df[~df.Name.str.contains('Hakan Yesil')]
df = df[~df.Name.str.contains('Hernani Infande')]
df = df[~df.Name.str.contains('Hugo Duro')]
df = df[~df.Name.str.contains('Igor Vinicius')]
df = df[~df.Name.str.contains('Ilaix Moriba')]
df = df[~df.Name.str.contains('Jacques Francois Moubandje')]
df = df[~df.Name.str.contains('Jaquite')]
df = df[~df.Name.str.contains('Javier Gandolfi')]
df = df[~df.Name.str.contains('Javier Mascherano')]
df = df[~df.Name.str.contains('Javier Mendez')]
df = df[~df.Name.str.contains('Javier Mendoza')]
df = df[~df.Name.str.contains('Jean Cleber')]
df = df[~df.Name.str.contains('Jean Eudes Aholou')]
df = df[~df.Name.str.contains('Jed Ward')]
df = df[~df.Name.str.contains('Jesus Vargas')]
df = df[~df.Name.str.contains('Joao Paulo da Silva')]
df = df[~df.Name.str.contains('Joao Teixeira')]
df = df[~df.Name.str.contains('Joaquin Munoz')]
df = df[~df.Name.str.contains('Jimmy Tabidze')]
df = df[~df.Name.str.contains('Jonathan dos Santos')]
df = df[~df.Name.str.contains('Vitali Balashov')]
df = df[~df.Name.str.contains('ngelo da Costa')]
df = df[~df.Name.str.contains('yuncu')]
df = df[~df.Name.str.contains('Lekue')]
df = df[~df.Name.str.contains('igo Martinez')]
df = df[~df.Name.str.contains('Thorvaldsson')]



prices = prices[~prices.name.str.contains('JoilsonJoilson')]
prices = prices[~prices.name.str.contains('Johan Mojica')]
prices = prices[~prices.name.str.contains('Joe WardJoe WardRight Midfiel')]
prices = prices[~prices.name.str.contains('Joaquin Mendive')]
prices = prices[~prices.name.str.contains('Joaquin Novillo')]
prices = prices[~prices.name.str.contains('Joaquin Hass')]
prices = prices[~prices.name.str.contains('Jean-Francois Gillet')]
prices = prices[~prices.name.str.contains('Javier Iritier')]
prices = prices[~prices.name.str.contains('Jaquite')]
prices = prices[~prices.name.str.contains('JaelJael')]
prices = prices[~prices.name.str.contains('Izzy Brow')]
prices = prices[~prices.name.str.contains('Jaakko Oksanen')]
prices = prices[~prices.name.str.contains('Strand Larsen')]
prices = prices[~prices.name.str.contains('Ivo GrbicIvo Grbic')]
prices = prices[~prices.name.str.contains('Ivan Angulo')]
prices = prices[~prices.name.str.contains('Ivan Arboleda')]
prices = prices[~prices.name.str.contains('Igor ViniciusIgor Vinicius')]
prices = prices[~prices.name.str.contains('Habib Diallo')]
prices = prices[~prices.name.str.contains('rkem Can')]
prices = prices[~prices.name.str.contains('Bakirbas')]
prices = prices[~prices.name.str.contains('khan Gul')]
prices = prices[~prices.name.str.contains('Ismael Sow')]
prices = prices[~prices.name.str.contains('Inigo Lekue')]
prices = prices[~prices.name.str.contains('Inigo Martinez')]
prices = prices[~prices.name.str.contains('Igor JuliaoIgor Juliao')]
prices = prices[~prices.name.str.contains('Igor CariusIgor Carius')]
prices = prices[~prices.name.str.contains('Inaki Pena')]
prices = prices[~prices.name.str.contains('Ilker Karakas')]
prices = prices[~prices.name.str.contains('Ignacio Miramon')]
prices = prices[~prices.name.str.contains('Ibrahim Sangare')]
prices = prices[~prices.name.str.contains('Ibragim Tsallagov')]
prices = prices[~prices.name.str.contains('Iago Herrerin')]
prices = prices[~prices.name.str.contains('HudsonHudson')]
prices = prices[~prices.name.str.contains('Hernan Lamberti')]
prices = prices[~prices.name.str.contains('Hernan Lopez')]
prices = prices[~prices.name.str.contains('HernaneHernane')]
prices = prices[~prices.name.str.contains('HernanesHernanes')]
prices = prices[~prices.name.str.contains('HelioHelio')]
prices = prices[~prices.name.str.contains('Hector Canteros')]
prices = prices[~prices.name.str.contains('HeberHeber')]
prices = prices[~prices.name.str.contains('Haji Mnoga')]
prices = prices[~prices.name.str.contains('Hakan Arslan')]
prices = prices[~prices.name.str.contains('Hakan Ozmert')]
prices = prices[~prices.name.str.contains('Hakan Yesil')]
prices = prices[~prices.name.str.contains('Guido Vadala')]
prices = prices[~prices.name.str.contains('Giangiacomo Magnani')]
prices = prices[~prices.name.str.contains('Georgi Makhatadze')]
prices = prices[~prices.name.str.contains('Gaston Torres')]
prices = prices[~prices.name.str.contains('Gaston Suso')]
prices = prices[~prices.name.str.contains('Gaston Gomez')]
prices = prices[~prices.name.str.contains('Gaston Gonzalez')]
prices = prices[~prices.name.str.contains('Gaetan Poussin')]
prices = prices[~prices.name.str.contains('Alfonso Espino')]
prices = prices[~prices.name.str.contains('Alfonso Pedraza')]
prices = prices[~prices.name.str.contains('AlêAlê')]
prices = prices[~prices.name.str.contains('Bjorn Engels')]
prices = prices[~prices.name.str.contains('Daniel Csoka')]
prices = prices[~prices.name.str.contains('Daniel Gazdag')]
prices = prices[~prices.name.str.contains('Daniel Gyollai')]
prices = prices[~prices.name.str.contains('Daniel Juarez')]
prices = prices[~prices.name.str.contains('Daniel Rios')]
prices = prices[~prices.name.str.contains('Daniel Williams')]
prices = prices[~prices.name.str.contains('Danieldoye')]
prices = prices[~prices.name.str.contains('Dario Sarmiento')]
prices = prices[~prices.name.str.contains('Darren Yapi')]
prices = prices[~prices.name.str.contains('David BrazDavid Braz')]
prices = prices[~prices.name.str.contains('David Buchholz')]
prices = prices[~prices.name.str.contains('David Da Costas')]
prices = prices[~prices.name.str.contains('Alan Cantero')]
prices = prices[~prices.name.str.contains('Aristoteles Romero')]
prices = prices[~prices.name.str.contains('Ben JonesBen Jones')]
prices = prices[~prices.name.str.contains('Davor Matijas')]
prices = prices[~prices.name.str.contains('Deian Veron')]
prices = prices[~prices.name.str.contains('Diego SouzaDiego Souza')]
prices = prices[~prices.name.str.contains('iego ÂngeloDiego Ângelo')]
prices = prices[~prices.name.str.contains('DiegoDiego')]
prices = prices[~prices.name.str.contains('Dino Visser')]
prices = prices[~prices.name.str.contains('Diogo BarbosaDiogo Barbosa')]
prices = prices[~prices.name.str.contains('Dixon Renteria')]
prices = prices[~prices.name.str.contains('Angelo Martino')]
prices = prices[~prices.name.str.contains('Ben Liddle')]
prices = prices[~prices.name.str.contains('Beni Makouana')]
prices = prices[~prices.name.str.contains('Bruno MendezBruno Mendez')]
prices = prices[~prices.name.str.contains('Buiu')]
prices = prices[~prices.name.str.contains('CauêCauê')]
prices = prices[~prices.name.str.contains('Dimitri Poloz')]
prices = prices[~prices.name.str.contains('Dimitri Rybchinskiy')]
prices = prices[~prices.name.str.contains('Dimitri Shomko')]
prices = prices[~prices.name.str.contains('Dimitri Skopintsev')]
prices = prices[~prices.name.str.contains('Dimitri Stotskiy')]
prices = prices[~prices.name.str.contains('Dimitri Sysuev')]
prices = prices[~prices.name.str.contains('Dimitri Tikhiy')]
prices = prices[~prices.name.str.contains('Dimitri Zhivoglyadov')]
prices = prices[~prices.name.str.contains('DodoDodo')]
prices = prices[~prices.name.str.contains('Dogan Alemdar')]
prices = prices[~prices.name.str.contains('Dogukan Ozkan')]
prices = prices[~prices.name.str.contains('Dom Telford')]
prices = prices[~prices.name.str.contains('Edson Montano')]
prices = prices[~prices.name.str.contains('Eduardo KauEduardo Kau')]
prices = prices[~prices.name.str.contains('Emanuel Insua')]
prices = prices[~prices.name.str.contains('Emiliano Gonzalez')]
prices = prices[~prices.name.str.contains('Emiliano Insua')]
prices = prices[~prices.name.str.contains('Enzo Fernandez')]
prices = prices[~prices.name.str.contains('Erik Duenas')]
prices = prices[~prices.name.str.contains('Esteban Rolon')]
prices = prices[~prices.name.str.contains('EvertonEverton')]
prices = prices[~prices.name.str.contains('Evgeni Kharin')]
prices = prices[~prices.name.str.contains('Ezequiel Fernandez')]
prices = prices[~prices.name.str.contains('Fabian Noguera')]
prices = prices[~prices.name.str.contains('Fabio ChinaFabio China')]
prices = prices[~prices.name.str.contains('Fabio SantosFabio Santos')]
prices = prices[~prices.name.str.contains('Fabio Vazquez')]
prices = prices[~prices.name.str.contains('Fabricio Dominguez')]
prices = prices[~prices.name.str.contains('Facundo Fernandez')]
prices = prices[~prices.name.str.contains('Facundo Garces')]
prices = prices[~prices.name.str.contains('Faiz Selemani')]
prices = prices[~prices.name.str.contains('Fedor Smolov')]
prices = prices[~prices.name.str.contains('Felipe Pena Biafore')]
prices = prices[~prices.name.str.contains('Fer NinoFer Nino')]
prices = prices[~prices.name.str.contains('Fernando Roman')]
prices = prices[~prices.name.str.contains('Filip Stankovic')]
prices = prices[~prices.name.str.contains('Filip Stuparevic')]
prices = prices[~prices.name.str.contains('Finn Ole Becker')]
prices = prices[~prices.name.str.contains('Florian Monzon')]
prices = prices[~prices.name.str.contains('Franck Bambock')]
prices = prices[~prices.name.str.contains('Francois Moubandje')]
prices = prices[~prices.name.str.contains('Gabriel Chapeco')]
prices = prices[~prices.name.str.contains('Gabriel Falcao')]
prices = prices[~prices.name.str.contains('Gabriel Gudino')]
prices = prices[~prices.name.str.contains('Giles Coke')]
prices = prices[~prices.name.str.contains('Gonzalo Alvez')]
prices = prices[~prices.name.str.contains('Gonzalo Asis')]
prices = prices[~prices.name.str.contains('Gonzalo Goni')]
prices = prices[~prices.name.str.contains('\*')]
prices = prices [~prices .name.str.contains('RegisRegis')]
prices = prices [~prices .name.str.contains('Jonas Aguirre')]
prices = prices [~prices .name.str.contains('Jonas Sterner')]
prices = prices [~prices .name.str.contains('Jonas Toro')]
prices = prices [~prices .name.str.contains('Jonatan Gomez')]
prices = prices [~prices .name.str.contains('Jonathan Zacaria')]
prices = prices [~prices .name.str.contains('Jony Alamo')]
prices = prices [~prices .name.str.contains('Jop van den Avert')]
prices = prices [~prices .name.str.contains('Jordi Osei-Tutu')]
prices = prices [~prices .name.str.contains('Jores Rahou')]
prices = prices [~prices .name.str.contains('Jorge Caceres')]
prices = prices [~prices .name.str.contains('Jorge Pereyra Diaz')]
prices = prices [~prices .name.str.contains('Jorge Saenz')]
prices = prices [~prices .name.str.contains('Jorgen Strand Larsen')]
prices = prices [~prices .name.str.contains('Jose Adolfo Valencia')]
prices = prices [~prices .name.str.contains('Jose Andres Martinez')]
prices = prices [~prices .name.str.contains('Jose AngelJose Angel')]
prices = prices [~prices .name.str.contains('Jose Cifuentes')]
prices = prices [~prices .name.str.contains('Jose Devecchi')]
prices = prices [~prices .name.str.contains('Jose Gimenez')]
prices = prices [~prices .name.str.contains('Jose Gonzalez')]
prices = prices [~prices .name.str.contains('Jose Luis Fernandez')]
prices = prices [~prices .name.str.contains('Jose Luis Gomez')]
prices = prices [~prices .name.str.contains('Jose Luis Rodriguez')]
prices = prices [~prices .name.str.contains('Jose Luis Sinisterra')]
prices = prices [~prices .name.str.contains('Jose Manuel Fontan')]
prices = prices [~prices .name.str.contains('Jose Manuel Lopez')]
prices = prices [~prices .name.str.contains('Jose Marcelo')]
prices = prices [~prices .name.str.contains('Jose Maria Gimenez')]
prices = prices [~prices .name.str.contains('Jose Martinez')]
prices = prices [~prices .name.str.contains('Jose Moya')]
prices = prices [~prices .name.str.contains('Josh JackJosh Jack')]
prices = prices [~prices .name.str.contains('Josh Neufville')]
prices = prices [~prices .name.str.contains('Josh Penn')]
prices = prices [~prices .name.str.contains('Josue Ayala')]
prices = prices [~prices .name.str.contains('Jovanny Bolivar')]
prices = prices [~prices .name.str.contains('Juan Alvarez')]
prices = prices [~prices .name.str.contains('Juan Cruz')]
prices = prices [~prices .name.str.contains('Juan Espinola')]
prices = prices [~prices .name.str.contains('Juan Gonzalez')]
prices = prices [~prices .name.str.contains('Juan Ignacio Mendez')]
prices = prices [~prices .name.str.contains('Juan Jose Caceres')]
prices = prices [~prices .name.str.contains('Juan Martin Lucero')]
prices = prices [~prices .name.str.contains('Juan Mendez')]
prices = prices [~prices .name.str.contains('Juan Pablo Alvarez')]
prices = prices [~prices .name.str.contains('Juan Pablo Ramirez')]
prices = prices [~prices .name.str.contains('Juan Roman Pucheta')]
prices = prices [~prices .name.str.contains('JuanJuan')]
prices = prices [~prices .name.str.contains('Jubril Okedina')]
prices = prices [~prices .name.str.contains('Julian Aude')]
prices = prices [~prices .name.str.contains('Julian Chicco')]
prices = prices [~prices .name.str.contains('Julian Lopez')]
prices = prices [~prices .name.str.contains('Julian Malatini')]
prices = prices [~prices .name.str.contains('Julian Marcioni')]
prices = prices [~prices .name.str.contains('Julian Navas')]
prices = prices [~prices .name.str.contains('Julian Velazquez')]
prices = prices [~prices .name.str.contains('Julio Gonzalez')]
prices = prices [~prices .name.str.contains('JuniorJunior')]
prices = prices [~prices .name.str.contains('KaduKadu')]
prices = prices [~prices .name.str.contains('Kaide Gordon')]
prices = prices [~prices .name.str.contains('Kaique RochaKaique Rocha')]
prices = prices [~prices .name.str.contains('KeillerKeiller')]
prices = prices [~prices .name.str.contains('KelvinKelvin')]
prices = prices [~prices .name.str.contains('Kemal Ruzgar')]
prices = prices [~prices .name.str.contains('KenoKeno')]
prices = prices [~prices .name.str.contains('KenoKeno')]
prices = prices [~prices .name.str.contains('Kerim Calhanoglu')]
prices = prices [~prices .name.str.contains('Kevin Boma')]
prices = prices [~prices .name.str.contains('Kevin Zenon')]
prices = prices [~prices .name.str.contains('Kian Slor')]
prices = prices [~prices .name.str.contains('Koffi')]
prices = prices [~prices .name.str.contains('Konstantin Kovalev')]
prices = prices [~prices .name.str.contains('Konstantin Rausch')]
prices = prices [~prices .name.str.contains('Kornell MacDonald')]
prices = prices [~prices .name.str.contains('Kortne Ford')]
prices = prices [~prices .name.str.contains('Kostas Stafylidis')]
prices = prices [~prices .name.str.contains('Laercio Solda')]
prices = prices [~prices .name.str.contains('LaercioLaercio')]
prices = prices [~prices .name.str.contains('Lars MaiLars Mai')]
prices = prices [~prices .name.str.contains('Lasse Sorensen')]
prices = prices [~prices .name.str.contains('Laureano Punet')]
prices = prices [~prices .name.str.contains('Lautaro Chavez')]
prices = prices [~prices .name.str.contains('Lautaro de Leon')]
prices = prices [~prices .name.str.contains('LazaroLazaro')]
prices = prices [~prices .name.str.contains('Leandro Contin')]
prices = prices [~prices .name.str.contains('Leandro Fernandez')]
prices = prices [~prices .name.str.contains('Lennart Schulze')]
prices = prices [~prices .name.str.contains('Lee O\'Connor')]
prices = prices [~prices .name.str.contains('Leo BorgesLeo Borges')]
prices = prices [~prices .name.str.contains('Leo Gomes')]
prices = prices [~prices .name.str.contains('Leo NatelLeo Natel')]
prices = prices [~prices .name.str.contains('Leonel Gonzalez')]
prices = prices [~prices .name.str.contains('Levi Ntumba')]
prices = prices [~prices .name.str.contains('Lois Diony')]
prices = prices [~prices .name.str.contains('Luca Martinez Dupuy')]
prices = prices [~prices .name.str.contains('Ludvig Ohman')]
prices = prices [~prices .name.str.contains('Luis Leal')]
prices = prices [~prices .name.str.contains('Luis Miguel Rodriguez')]
prices = prices [~prices .name.str.contains('Luis Rodriguez')]
prices = prices [~prices .name.str.contains('Luis Vazquez')]
prices = prices [~prices .name.str.contains('Luiz Fernando')]
prices = prices [~prices .name.str.contains('Luiz JuniorLuiz Junior')]
prices = prices [~prices .name.str.contains('Luiz Lucio Reis Junior')]
prices = prices [~prices .name.str.contains('Luiz Otavio')]
prices = prices [~prices .name.str.contains('Luka Zarandia')]
prices = prices [~prices .name.str.contains('MaiconMaicon')]
prices = prices [~prices .name.str.contains('Malik Sellouki')]
prices = prices [~prices .name.str.contains('Malvind Benning')]
prices = prices [~prices .name.str.contains('Mamadi Camara')]
prices = prices [~prices .name.str.contains('Mamadou Coulibaly')]
prices = prices [~prices .name.str.contains('Marc Auge')]
prices = prices [~prices .name.str.contains('Marc Baro')]
prices = prices [~prices .name.str.contains('Marc-Olivier Doue')]
prices = prices [~prices .name.str.contains('Marcelinho')]
prices = prices [~prices .name.str.contains('Marcio AzevedoMarcio Azevedo')]
prices = prices [~prices .name.str.contains('Marcio Mossoro')]
prices = prices [~prices .name.str.contains('Marcos AndreMarcos Andre')]
prices = prices [~prices .name.str.contains('Marcos Sanchez')]
prices = prices [~prices .name.str.contains('Mariano Diaz')]
prices = prices [~prices .name.str.contains('Mariano Vazquez')]
prices = prices [~prices .name.str.contains('Mark Kosznovszky')]
prices = prices [~prices .name.str.contains('Markus Fjortoft')]
prices = prices [~prices .name.str.contains('Marley Ake')]
prices = prices [~prices .name.str.contains('Martin Campana')]
prices = prices [~prices .name.str.contains('Martin Garay')]
prices = prices [~prices .name.str.contains('Martin Garcia')]
prices = prices [~prices .name.str.contains('Martin Rabunal')]
prices = prices [~prices .name.str.contains('Mateus BrunMateus Brun')]
prices = prices [~prices .name.str.contains('Matheo Vroman')]
prices = prices [~prices .name.str.contains('Mathias Abero')]
prices = prices [~prices .name.str.contains('Mathias Kjolo')]
prices = prices [~prices .name.str.contains('Mathias Suarez')]
prices = prices [~prices .name.str.contains('Matias Belloso')]
prices = prices [~prices .name.str.contains('Matias Borgogno')]
prices = prices [~prices .name.str.contains('Matias Cordoba')]
prices = prices [~prices .name.str.contains('Matias Escudero')]
prices = prices [~prices .name.str.contains('Matias Fritzler')]
prices = prices [~prices .name.str.contains('Matias Garcia')]
prices = prices [~prices .name.str.contains('Matias Gomez')]
prices = prices [~prices .name.str.contains('Matias Gonzalez')]
prices = prices [~prices .name.str.contains('Matias Laba')]
prices = prices [~prices .name.str.contains('Matias Moya')]
prices = prices [~prices .name.str.contains('Matias Palavecino')]
prices = prices [~prices .name.str.contains('Matias Pellegrini')]
prices = prices [~prices .name.str.contains('Matias Pisano')]
prices = prices [~prices .name.str.contains('Matias Ramirez')]
prices = prices [~prices .name.str.contains('Matias Rodriguez')]
prices = prices [~prices .name.str.contains('Matias Sosa')]
prices = prices [~prices .name.str.contains('Matias Tagliamonte')]
prices = prices [~prices .name.str.contains('Matias Tissera')]
prices = prices [~prices .name.str.contains('Matias Villarreal')]
prices = prices [~prices .name.str.contains('Mauro Ortiz')]
prices = prices [~prices .name.str.contains('Max GrunMax Grun')]
prices = prices [~prices .name.str.contains('Max WalefMax Walef')]
prices = prices [~prices .name.str.contains('Meme Placca Fessou')]
prices = prices [~prices .name.str.contains('Mert Gunok')]
prices = prices [~prices .name.str.contains('Metin Yildirim')]
prices = prices [~prices .name.str.contains('MetinhoMetinho')]
prices = prices [~prices .name.str.contains('Michael Chacon')]
prices = prices [~prices .name.str.contains('Michel Araujo')]
prices = prices [~prices .name.str.contains('Michel Herrero')]
prices = prices [~prices .name.str.contains('Michel Macedo')]
prices = prices [~prices .name.str.contains('MichelMichel')]
prices = prices [~prices .name.str.contains('Miguel Angel Atienza')]
prices = prices [~prices .name.str.contains('Miguel Angel Guerrero')]
prices = prices [~prices .name.str.contains('Miguel Angel Moya')]
prices = prices [~prices .name.str.contains('Miguel Angel Rubio')]
prices = prices [~prices .name.str.contains('Mike GrimMike Grim')]
prices = prices [~prices .name.str.contains('Milton Alvarez')]
prices = prices [~prices .name.str.contains('Milton Gimenez')]
prices = prices [~prices .name.str.contains('Miquel Scarlett')]
prices = prices [~prices .name.str.contains('Mohamed Traore')]
prices = prices [~prices .name.str.contains('Moises Caicedo')]
prices = prices [~prices .name.str.contains('Moon-hwan Kim')]
prices = prices [~prices .name.str.contains('MoraesMoraes')]
prices = prices [~prices .name.str.contains('Morato')]
prices = prices [~prices .name.str.contains('Moussa Wague')]
prices = prices [~prices .name.str.contains('Murat Akca')]
prices = prices [~prices .name.str.contains('Murat Aksit')]
prices = prices [~prices .name.str.contains('Murat Sipahioglu')]
prices = prices [~prices .name.str.contains('MurielMuriel')]
prices = prices [~prices .name.str.contains('Murilo RangelMurilo Rangel')]
prices = prices [~prices .name.str.contains('Mustafa Cecenoglu')]
prices = prices [~prices .name.str.contains('Mustafa Huseyin Seyhan')]
prices = prices [~prices .name.str.contains('NatanaelNatanael')]
prices = prices [~prices .name.str.contains('NatanNatan')]
prices = prices [~prices .name.str.contains('Nathan-Dylan Saliba')]
prices = prices [~prices .name.str.contains('Nathangelo Markelo')]
prices = prices [~prices .name.str.contains('Nazareno Solis')]
prices = prices [~prices .name.str.contains('NeiltonNeilton')]
prices = prices [~prices .name.str.contains('Nelson Ibanez')]
prices = prices [~prices .name.str.contains('Nemanja Antonov')]
prices = prices [~prices .name.str.contains('Nestor Ortigoza')]
prices = prices [~prices .name.str.contains('Nicolas Andereggen')]
prices = prices [~prices .name.str.contains('Nicolas Avellaneda')]
prices = prices [~prices .name.str.contains('Nicolas Blandi')]
prices = prices [~prices .name.str.contains('Nicolas Burgy')]
prices = prices [~prices .name.str.contains('Nicolas Contin')]
prices = prices [~prices .name.str.contains('Nicolas Correa')]
prices = prices [~prices .name.str.contains('Nicolas Domingo')]
prices = prices [~prices .name.str.contains('Nicolas Femia')]
prices = prices [~prices .name.str.contains('Nicolas Fernandez Mercau')]
prices = prices [~prices .name.str.contains('Nicolas Ferreyra')]
prices = prices [~prices .name.str.contains('Nicolas Figal')]
prices = prices [~prices .name.str.contains('Nicolas Gimenez')]
prices = prices [~prices .name.str.contains('Nicolas Lamendola')]
prices = prices [~prices .name.str.contains('Nicolas Linares')]
prices = prices [~prices .name.str.contains('Nicolas Mana')]
prices = prices [~prices .name.str.contains('Nicolas Mazzola')]
prices = prices [~prices .name.str.contains('Nicolas Messiniti')]
prices = prices [~prices .name.str.contains('Nicolas Morgantini')]
prices = prices [~prices .name.str.contains('Nicolas Olsak')]
prices = prices [~prices .name.str.contains('Nicolas Penailillo')]
prices = prices [~prices .name.str.contains('Nicolas Romat')]
prices = prices [~prices .name.str.contains('Nicolas Romero')]
prices = prices [~prices .name.str.contains('Nicolas Schiappacasse')]
prices = prices [~prices .name.str.contains('Nicolas Thaller')]
prices = prices [~prices .name.str.contains('Nicolas Zalazar')]
prices = prices [~prices .name.str.contains('Nicolo Zaniolo')]
prices = prices [~prices .name.str.contains('Nikola Pejčić')]
prices = prices [~prices .name.str.contains('Nino ParaibaNino Paraiba')]
prices = prices [~prices .name.str.contains('Omer Beyaz')]
prices = prices [~prices .name.str.contains('Omer Sismanoglu')]
prices = prices [~prices .name.str.contains('Oscar Benitez')]
prices = prices [~prices .name.str.contains('Oscar Linner')]
prices = prices [~prices .name.str.contains('Oscar Schönfelder')]
prices = prices [~prices .name.str.contains('OsmanOsman')]
prices = prices [~prices .name.str.contains('OsvaldoOsvaldo')]
prices = prices [~prices .name.str.contains('Pablo Velazquez')]
prices = prices [~prices .name.str.contains('PaulaoPaulao')]
prices = prices [~prices .name.str.contains('Paulinho BoiaPaulinho Boia')]
prices = prices [~prices .name.str.contains('Pedro Goncalves')]
prices = prices [~prices .name.str.contains('Pepê')]
prices = prices [~prices .name.str.contains('Pepe Bonet')]
prices = prices [~prices .name.str.contains('Philip Turpitz')]
prices = prices [~prices .name.str.contains('Piero Hincapie')]
prices = prices [~prices .name.str.contains('Piris da MottaPiris da Motta')]
prices = prices [~prices .name.str.contains('Pol Garcia')]
prices = prices [~prices .name.str.contains('Pote')]
prices = prices [~prices .name.str.contains('Rafael Borre')]
prices = prices [~prices .name.str.contains('Rafael Garcia')]
prices = prices [~prices .name.str.contains('Rahman Bugra Cagiran')]
prices = prices [~prices .name.str.contains('Ramon Arias')]
prices = prices [~prices .name.str.contains('Ramon Azeez')]
prices = prices [~prices .name.str.contains('Ramon Folch')]
prices = prices [~prices .name.str.contains('Ramon ten Hove')]
prices = prices [~prices .name.str.contains('Rashad Muhammed')]
prices = prices [~prices .name.str.contains('Rashid')]
prices = prices [~prices .name.str.contains('Raul Carnero')]
prices = prices [~prices .name.str.contains('Raul Lozano')]
prices = prices [~prices .name.str.contains('Raul Navas')]
prices = prices [~prices .name.str.contains('Raul Silva')]
prices = prices [~prices .name.str.contains('Raul SilvaRaul Silva')]
prices = prices [~prices .name.str.contains('Remeao Hutton')]
prices = prices [~prices .name.str.contains('RenêRenê')]
prices = prices [~prices .name.str.contains('Rhys Fenlon')]
prices = prices [~prices .name.str.contains('Rhys-James Fenlon')]
prices = prices [~prices .name.str.contains('RibamarRibamar')]
prices = prices [~prices .name.str.contains('Ricard Sanchez')]
prices = prices [~prices .name.str.contains('Ricardo Gomes')]
prices = prices [~prices .name.str.contains('Ricardo SilvaRicardo Silva')]
prices = prices [~prices .name.str.contains('RildoRildo')]
prices = prices [~prices .name.str.contains('Roberto Jimenez')]
prices = prices [~prices .name.str.contains('Rodri Sanchez')]
prices = prices [~prices .name.str.contains('Rodrigo Sanchez')]
prices = prices [~prices .name.str.contains('Rodrigues')]
prices = prices [~prices .name.str.contains('Rogerio Santos')]
prices = prices [~prices .name.str.contains('Roger IbanezRoger Ibanez')]
prices = prices [~prices .name.str.contains('Rolando Garcia')]
prices = prices [~prices .name.str.contains('Roman Martinez')]
prices = prices [~prices .name.str.contains('Romulo Otero')]
prices = prices [~prices .name.str.contains('Ruben Botta')]
prices = prices [~prices .name.str.contains('Ruben Fonseca')]
prices = prices [~prices .name.str.contains('Ruben Pardo')]
prices = prices [~prices .name.str.contains('Rui EncarnacaoRui Encarnacao')]
prices = prices [~prices .name.str.contains('Ruwen Werthmuller')]
prices = prices [~prices .name.str.contains('Sören Dieckmann')]
prices = prices [~prices .name.str.contains('Saîf-Eddine Khaoui')]
prices = prices [~prices .name.str.contains('Said Bakari')]
prices = prices [~prices .name.str.contains('Saidou Sow')]
prices = prices [~prices .name.str.contains('Saliou Sane')]
prices = prices [~prices .name.str.contains('Sandro PerpetuoSandro Perpetuo')]
prices = prices [~prices .name.str.contains('SandrySandry')]
prices = prices [~prices .name.str.contains('Santiago Brinone')]
prices = prices [~prices .name.str.contains('Santiago Caseres')]
prices = prices [~prices .name.str.contains('Santiago Garcia')]
prices = prices [~prices .name.str.contains('Santiago Rodriguez')]
prices = prices [~prices .name.str.contains('Santiago Simon')]
prices = prices [~prices .name.str.contains('Santiago Trellez')]
prices = prices [~prices .name.str.contains('Santos Borre')]
prices = prices [~prices .name.str.contains('Saul Salcedo')]
prices = prices [~prices .name.str.contains('Savio')]
prices = prices [~prices .name.str.contains('Sebastian Assis')]
prices = prices [~prices .name.str.contains('Sebastian Blanco')]
prices = prices [~prices .name.str.contains('Sebastian Dubarbier')]
prices = prices [~prices .name.str.contains('Sebastian Meza')]
prices = prices [~prices .name.str.contains('Sebastian Ramirez')]
prices = prices [~prices .name.str.contains('Sebastian Sosa')]
prices = prices [~prices .name.str.contains('Sebastien Thill')]
prices = prices [~prices .name.str.contains('Sega Coulibaly')]
prices = prices [~prices .name.str.contains('Sebastian Prödl')]
prices = prices [~prices .name.str.contains('Sergi Garcia')]
prices = prices [~prices .name.str.contains('Sergio Gonzalez')]
prices = prices [~prices .name.str.contains('Siro Rosane')]
prices = prices [~prices .name.str.contains('Suleyman Cebeci')]
prices = prices [~prices .name.str.contains('Sven Köhler')]
prices = prices [~prices .name.str.contains('Theo Leoni')]
prices = prices [~prices .name.str.contains('Thiago Andrade')]
prices = prices [~prices .name.str.contains('Thiago CoutoThiago Couto')]
prices = prices [~prices .name.str.contains('ThiaguinhoThiaguinho')]
prices = prices [~prices .name.str.contains('Thomas Toure')]
prices = prices [~prices .name.str.contains('Tibet Oniz')]
prices = prices [~prices .name.str.contains('Timothe Nkada')]
prices = prices [~prices .name.str.contains('Tobias Zarate')]
prices = prices [~prices .name.str.contains('Tomas Andrade')]
prices = prices [~prices .name.str.contains('Tomas Cardona')]
prices = prices [~prices .name.str.contains('Tomas Cavanagh')]
prices = prices [~prices .name.str.contains('Tomas Durso')]
prices = prices [~prices .name.str.contains('Tomas Escalante')]
prices = prices [~prices .name.str.contains('Tomas Fernandez')]
prices = prices [~prices .name.str.contains('Tomas Marchiori')]
prices = prices [~prices .name.str.contains('Tomas Martinez')]
prices = prices [~prices .name.str.contains('Tomas Ortiz')]
prices = prices [~prices .name.str.contains('Tomas Romero')]
prices = prices [~prices .name.str.contains('Tomas Sandoval')]
prices = prices [~prices .name.str.contains('Tomas Silva')]
prices = prices [~prices .name.str.contains('Torben Musel')]
prices = prices [~prices .name.str.contains('Uriel Ramirez')]
prices = prices [~prices .name.str.contains('Valentin Larralde')]
prices = prices [~prices .name.str.contains('Victor Camarasa')]
prices = prices [~prices .name.str.contains('Victor KawaVictor Kawaattac')]
prices = prices [~prices .name.str.contains('Vincent Muller')]
prices = prices [~prices .name.str.contains('ViniciusVinicius')]
prices = prices [~prices .name.str.contains('Vitor JacareVitor Jacare')]
prices = prices [~prices .name.str.contains('Vitor Oliveira')]
prices = prices [~prices .name.str.contains('WagnerWagner')]
prices = prices [~prices .name.str.contains('WalberWalber')]
prices = prices [~prices .name.str.contains('Walter Andrade')]
prices = prices [~prices .name.str.contains('Walter Kannemann')]
prices = prices [~prices .name.str.contains('WandsonWandson')]
prices = prices [~prices .name.str.contains('Welinton Junior')]
prices = prices [~prices .name.str.contains('Wellington NemWellington Nem')]
prices = prices [~prices .name.str.contains('Wellington TimWellington Tim')]
prices = prices [~prices .name.str.contains('YagoYago')]
prices = prices [~prices .name.str.contains('Yahya Soumare')]
prices = prices [~prices .name.str.contains('Yanis Lhery')]
prices = prices [~prices .name.str.contains('Yasin Oztekin')]
prices = prices [~prices .name.str.contains('Yeiler Goez')]
prices = prices [~prices .name.str.contains('Yildirim Mert Çetin')]
prices = prices [~prices .name.str.contains('Younes Belhanda')]
prices = prices [~prices .name.str.contains('Youssef Ait Bennasser')]
prices = prices [~prices .name.str.contains('Yuri Shafinskiy')]
prices = prices [~prices .name.str.contains('YuriYuri')]
prices = prices [~prices .name.str.contains('Yusuf Eren Göktas')]
prices = prices [~prices .name.str.contains('Ze GabrielZe Gabriel')]
prices = prices [~prices .name.str.contains('Ze IvaldoZe Ivaldo')]
prices = prices [~prices .name.str.contains('Ze Oliveira')]
prices = prices [~prices .name.str.contains('Dimitri German')]
prices = prices [~prices .name.str.contains('Dimitri Chistyakov')]
prices = prices [~prices .name.str.contains('Dimitri Khomich')]
prices = prices [~prices .name.str.contains('Dimitri Poloz')]
prices = prices [~prices .name.str.contains('Dimitri Rybchinskiy')]
prices = prices [~prices .name.str.contains('Dimitri Shomko')]
prices = prices [~prices .name.str.contains('Dimitri Skopintsev')]
prices = prices [~prices .name.str.contains('Dimitri Stotskiy')]
prices = prices [~prices .name.str.contains('Dimitri Sysuev')]
prices = prices [~prices .name.str.contains('Dimitri Tikhiy')]
prices = prices [~prices .name.str.contains('Dimitri Zhivoglyadov')]
prices = prices [~prices .name.str.contains('Altiparmak')]
prices = prices [~prices .name.str.contains('Bayrakdar')]
prices = prices [~prices .name.str.contains('Kardes')]
prices = prices [~prices .name.str.contains('Georgios Vagiannidis')]
prices = prices [~prices .name.str.contains('Georgiy Dzhikiya')]
prices = prices [~prices .name.str.contains('Javi Lopez')]
prices = prices [~prices .name.str.contains('Jed SteerJed Steer')]
prices = prices [~prices .name.str.contains('Jeffrey Bruma')]
prices = prices [~prices .name.str.contains('Jeizon Ramirez')]
prices = prices [~prices .name.str.contains('Jens J')]
prices = prices [~prices .name.str.contains('Jesus Ferreira')]
prices = prices [~prices .name.str.contains('Joao Carlos Teixeira')]
prices = prices [~prices .name.str.contains('Joao CarlosJoao Carlos')]
prices = prices [~prices .name.str.contains('Joao GomesJoao Gomes')]
prices = prices [~prices .name.str.contains('Joao IgorJoao Igor')]
prices = prices [~prices .name.str.contains('Joao Jaquite')]
prices = prices [~prices .name.str.contains('Joao Klauss')]
prices = prices [~prices .name.str.contains('Joao LopesJoao Lopes')]
prices = prices [~prices .name.str.contains('Joao MarioJoao MarioRight Midfiel')]
prices = prices [~prices .name.str.contains('Joao Monteiro')]
prices = prices [~prices .name.str.contains('Yaya SitholeYaya Sithole')]





df['Name'] = df['Name'].str.replace('Georgy','Georgi')
df['Name'] = df['Name'].str.replace('Dimitrios','Dimitri')
df['Name'] = df['Name'].str.replace('Dimitrij','Dimitri')
df['Name'] = df['Name'].str.replace('Max-Alain Gradel','Max Gradel')
df['Name'] = df['Name'].str.replace('Vitaly','Vitali')
df['Name'] = df['Name'].str.replace('Vitaliy','Vitali')




prices['name'] = prices['name'].str.replace('Dmitriy','Dimitri ')
prices['name'] = prices['name'].str.replace('Evgeniy','Evgeni')
prices['name'] = prices['name'].str.replace('Fede Valverde','Federico Valverde')
prices['name'] = prices['name'].str.replace('Georgy','Georgi')
prices['name'] = prices['name'].str.replace('Konstantinos','Kostas')
prices['name'] = prices['name'].str.replace('Daniel\'Lundulu','Daniel N\'Lundulu')
prices['name'] = prices['name'].str.replace('Dimitrios','Dimitri')
prices['name'] = prices['name'].str.replace('Dimitris','Dimitri')
prices['name'] = prices['name'].str.replace('GilGil','Gil')
prices['name'] = prices['name'].str.replace('IgorIgor','Igor')
prices['name'] = prices['name'].str.replace('RiccieliRiccieli','Riccieli')
prices['name'] = prices['name'].str.replace('RiccieliRiccieli','Riccieli')
prices['name'] = prices['name'].str.replace('RiccieliRiccieli','Riccieli')
prices['name'] = prices['name'].str.replace('Erkan KasErkan Kas','Erkan Kas')
prices['name'] = prices['name'].str.replace('Bruno RamiresBruno Ramires','Bruno Ramires')
prices['name'] = prices['name'].str.replace('Juan CruzJuan Cruz','Juan Cruz')
prices['name'] = prices['name'].str.replace('Leo','Leo')
prices['name'] = prices['name'].str.replace('Mahamadou Doucoure','Mamadou Doucoure')
prices['name'] = prices['name'].str.replace('MoisesMoises','Moises')
prices['name'] = prices['name'].str.replace('MunirMunir','Munir')
prices['name'] = prices['name'].str.replace('NatanNatan','Natan')
prices['name'] = prices['name'].str.replace('NatanNatan','Natan')
prices['name'] = prices['name'].str.replace('Nico Gaitan','Nicolas Gaitan')
prices['name'] = prices['name'].str.replace('ThiagoThiago','Thiago')
prices['name'] = prices['name'].str.replace('Tom RogicTom Rogic','Tom Rogic')
prices['name'] = prices['name'].str.replace('WendellWendell','Wendell')
prices['name'] = prices['name'].str.replace('WendelWendel','Wendel')
prices['name'] = prices['name'].str.replace('WillianWillian','Willian')
prices['name'] = prices['name'].str.replace('Jason Dimitriou','Jason Demetriou')
prices['name'] = prices['name'].str.replace('Javi Ontiveros','Javier Ontiveros')
prices['name'] = prices['name'].str.replace('Jean Manuel Mbom','Jean/Manuel Mbom')
prices['name'] = prices['name'].str.replace('JeanJean','Jean')
prices['name'] = prices['name'].str.replace('JeffersonJefferson','Jefferson')
prices['name'] = prices['name'].str.replace('Jesus Murillo','Jesus David Murillo')
prices['name'] = prices['name'].str.replace('JoJo','Jo')
prices['name'] = prices['name'].str.replace('JonyJony','Jony')
prices['name'] = prices['name'].str.replace('JordiJordi','Jordi')
prices['name'] = prices['name'].str.replace('Jorge FelixJorge Felix','Jorge Felix')
prices['name'] = prices['name'].str.replace('JosanJosan','Josan')
prices['name'] = prices['name'].str.replace('Jose WelisonJose Welison','Jose Welison')
prices['name'] = prices['name'].str.replace('KokeKoke','Koke')
prices['name'] = prices['name'].str.replace('Ângelo GabrielÂngelo Gabriel','Ângelo Gabriel')
prices['name'] = prices['name'].str.replace('Vinicius JuniorVinicius Junior','Vinicius Junior')
prices['name'] = prices['name'].str.replace('Vinicius MachadoVinicius Machado','Vinicius Machado')
prices['name'] = prices['name'].str.replace('WescleyWescley','Wescley')
prices['name'] = prices['name'].str.replace('WelintonWelinton','Welinton')
prices['name'] = prices['name'].str.replace('WellingtonWellington','Wellington')
prices['name'] = prices['name'].str.replace('WendsonWendsonattac','Wendson')
prices['name'] = prices['name'].str.replace('WevertonWeverton','Weverton')

In [432]:
#delete some players who have only 1 name (I need to use the index becuse the operation would delete other player's name)
df = df.drop([4923, 7842, 7802, 6666, 7370, 7830, 7815, 7884, 6176, 1473, 6855, 8465, 1573, 9397, 3347, 2586])

prices = prices.drop([4310, 4664, 2820, 150, 4705])

df = df[~df.Name.str.contains('Bayrakdar')]

In [433]:
#one last cleaning

df = df[~df.Name.str.contains('Bayrakdar')]
df = df[~df.Name.str.contains('Dimitri Shomko')]
df = df[~df.Name.str.contains('Dimitri Skopintsev')]
df = df[~df.Name.str.contains('Dimitri Stotskiy')]
df = df[~df.Name.str.contains('Dimitri Sysuev')]
df = df[~df.Name.str.contains('Dimitri Tikhiy')]
df = df[~df.Name.str.contains('Dimitri Zhivoglyadov')]
df = df[~df.Name.str.contains('Erce Kardesler')]
df = df[~df.Name.str.contains('Eric Ebimbe')]
df = df[~df.Name.str.contains('Habib Diallo')]
df = df[~df.Name.str.contains('Jasper Dahlhaus')]
df = df[~df.Name.str.contains('Jasper Moon')]
df = df[~df.Name.str.contains('Javi Diaz')]
df = df[~df.Name.str.contains('Javi Montero')]
df = df[~df.Name.str.contains('Javi Moyano')]
df = df[~df.Name.str.contains('Jean-Phillipe Krasso')]
df = df[~df.Name.str.contains('Jeremy Perbet')]
df = df[~df.Name.str.contains('Johan Mojica')]
df = df[~df.Name.str.contains('ull Andresson')]
df = df[~df.Name.str.contains('Kerim Çalhanoglu')]
df = df[~df.Name.str.contains('Lyle Foster')]
df = df[~df.Name.str.contains('M\'Baye Niang')]
df = df[~df.Name.str.contains('Mahamadou Ba')]
df = df[~df.Name.str.contains('Maximiliano Gomez')]
df = df[~df.Name.str.contains('Natanael Pimienta')]
df = df[~df.Name.str.contains('Pablo Ínsua')]
df = df[~df.Name.str.contains('Rodrigo Gomes')]
df = df[~df.Name.str.contains('Samuel Emem Eduok')]
df = df[~df.Name.str.contains('Ryder')]
df = df[~df.Name.str.contains('Ryley Towler')]
df = df[~df.Name.str.contains('Rodrigo Gomez')]

df['Name'] = df['Name'].str.replace('Eren Sami Dinkci','Eren Dinkci')
df['Name'] = df['Name'].str.replace('Klauss','Klaus')
df['Name'] = df['Name'].str.replace('Konstantinos Laifis','Kostas Laifis')
df['Name'] = df['Name'].str.replace('Konstantinos Mavropanos','Kostas Mavropanos')
df['Name'] = df['Name'].str.replace('Konstantinos Tsimikas','Kostas Tsimikas')

prices = prices[~prices.name.str.contains('Kerem Akturkoglu')]
prices = prices[~prices.name.str.contains('Eric Lopez')]
prices = prices[~prices.name.str.contains('JasonJason')]
prices = prices[~prices.name.str.contains('Javier Cabrera')]
prices = prices[~prices.name.str.contains('Jeronimo Pourtau')]
prices = prices[~prices.name.str.contains('Jimmy Tabidze')]
prices = prices[~prices.name.str.contains('Shomko')]
prices = prices[~prices.name.str.contains('Skopintsev')]
prices = prices[~prices.name.str.contains('Stotskiy')]
prices = prices[~prices.name.str.contains('Sysuev')]
prices = prices[~prices.name.str.contains('Tikhiy')]
prices = prices[~prices.name.str.contains('Zhivoglyadov')]

prices['name'] = prices['name'].str.replace('EmersonEmerson','Emerson')
prices['name'] = prices['name'].str.replace('Jean/Manuel Mbom','Jean-Manuel Mbom')

In [434]:
prices = prices[~prices.name.str.contains('Javi Diaz')]
prices = prices[~prices.name.str.contains('Jean-Marie Plath')]
prices = prices[~prices.name.str.contains('Dimitri Chistyakov')]

prices['name'] = prices['name'].str.replace('Dimitri   Barinov','Dimitri Barinov')
prices['name'] = prices['name'].str.replace('Dimitri   Chistyakov','Dimitri Chistyakov')
prices['name'] = prices['name'].str.replace('Dimitri   German','Dimitri German')
prices['name'] = prices['name'].str.replace('Dimitri   Khomich','Dimitri Khomich')
prices['name'] = prices['name'].str.replace('Dimitri   Nazarov','Dimitri Nazarov')
prices['name'] = prices['name'].str.replace('Dimitri   Poloz','Dimitri Poloz')
prices['name'] = prices['name'].str.replace('Dimitri   Rybchinskiy','Dimitri Rybchinskiy')
prices['name'] = prices['name'].str.replace('Dimitri Giannoulis','Dimitris Giannoulis')
prices['name'] = prices['name'].str.replace('Dimitri Limnios','Dimitris Limnios')
prices['name'] = prices['name'].str.replace('Mamadou Doucoure','Mahamadou Doucoure')

prices = prices[~prices.name.str.contains('Dimitri German')]
prices = prices[~prices.name.str.contains('Dimitri Khomich')]
prices = prices[~prices.name.str.contains('Dimitri Poloz')]
prices = prices[~prices.name.str.contains('Dimitri Rybchinskiy')]
prices = prices[~prices.name.str.contains('Korede Adedoyin')]
prices = prices[~prices.name.str.contains('Leo PassosLeo Passos')]
prices = prices[~prices.name.str.contains('Leo SantosLeo Santos')]
prices = prices[~prices.name.str.contains('Mattheus')]
prices = prices[~prices.name.str.contains('Maxi Gomez')]
prices = prices[~prices.name.str.contains('Nacho Martinez')]
prices = prices[~prices.name.str.contains('OliveiraOliveira')]
prices = prices[~prices.name.str.contains('Oscar Ruiz')]
prices = prices[~prices.name.str.contains('ReginaldoReginaldo')]
prices = prices[~prices.name.str.contains('RikelmeRikelme')]
prices = prices[~prices.name.str.contains('RonanRonan')]
prices = prices[~prices.name.str.contains('Welington')]
prices = prices[~prices.name.str.contains('Matheus JesusMatheus Jesus')]

prices['name'] = prices['name'].str.replace('Hasan Emre Yesilyurt','Hasan Yesilyurt')
prices['name'] = prices['name'].str.replace('LeoLeo','Leo')
prices['name'] = prices['name'].str.replace('MarianoMariano','Mariano')
prices['name'] = prices['name'].str.replace('Mathi­as Olivera','Mathias Olivera')
prices['name'] = prices['name'].str.replace('Matias De los Santos','Matias de los Santos')
prices['name'] = prices['name'].str.replace('Oscar de Marcos','Oscar De Marcos')
prices['name'] = prices['name'].str.replace('Roberto FirminoRoberto Firmino','Roberto Firmino')
prices['name'] = prices['name'].str.replace('Rodrigo MunizRodrigo Muniz','Rodrigo Muniz')
prices['name'] = prices['name'].str.replace('SamirSamir','Samir')
prices['name'] = prices['name'].str.replace('SamuelSamuel','Samuel')
prices['name'] = prices['name'].str.replace('Samuel XavierSamuel Xavier','Samuel Xavier')
prices['name'] = prices['name'].str.replace('Serge-Philippe Raux Yao','Serge Raux Yao')
prices['name'] = prices['name'].str.replace('Tom SangTom SangRight Midfiel','Tom Sang')
prices['name'] = prices['name'].str.replace('Tom Rogic','Tomas Rogic')
prices['name'] = prices['name'].str.replace('Vitaly Janelt','Vitali Janelt')
prices['name'] = prices['name'].str.replace('Zurab Gigashvili','Zurab Gigashvili')

In [435]:
df = df.drop([2230, 8156, 820, 7017, 8057, 6618, 1756, 6618])
prices = prices.drop([5792, 4430, 4124, 4427, 1550, 4427, 4425])

In [436]:
prices = prices[~prices.name.str.contains('yuncu')]

In [437]:
#order the dataframes in ascending order to check the matches

df = df.sort_values(by=['Name'], ascending=True)
prices = prices.sort_values(by=['name'], ascending=True)

In [438]:
df = df[~df.Name.str.contains('Ben Liddle')]
df = df[~df.Name.str.contains('Carlos Araujo')]
df = df[~df.Name.str.contains('Casagrande')]
df = df[~df.Name.str.contains('Caue')]
df = df[~df.Name.str.contains('Diego Angelo')]
df = df[~df.Name.str.contains('Gabriel Magalhaes')]
df = df[~df.Name.str.contains('Gaston Fernandez')]
df = df[~df.Name.str.contains('Giles Coke')]
df = df[~df.Name.str.contains('Jahmal Hector-Ingram')]
df = df[~df.Name.str.contains('Batuhan Kirdaroglu')]
df = df[~df.Name.str.contains('Braian Alvarez')]
df = df[~df.Name.str.contains('Joe Ward')]
df = df[~df.Name.str.contains('Pedro Gallese')]

prices = prices[~prices.name.str.contains('Batuhan')]
prices = prices[~prices.name.str.contains('Braian Alvarez')]
prices = prices[~prices.name.str.contains('Braian Sanchez')]
prices = prices[~prices.name.str.contains('Carlos Auzqui')]
prices = prices[~prices.name.str.contains('Casper Staring')]
prices = prices[~prices.name.str.contains('CassioCassio')]
prices = prices[~prices.name.str.contains('Diego Churin')]
prices = prices[~prices.name.str.contains('Chistyakov')]
prices = prices[~prices.name.str.contains('Ferhat Görgulu')]
prices = prices[~prices.name.str.contains('Gabriel MeninoGabriel Menino')]
prices = prices[~prices.name.str.contains('Gaston Benavidez')]
prices = prices[~prices.name.str.contains('GiovaniGiovani')]
prices = prices[~prices.name.str.contains('JailsonJailson')]
prices = prices[~prices.name.str.contains('Joe Rodwell-Grant')]
prices = prices[~prices.name.str.contains('Jonathan dos Santos')]
prices = prices[~prices.name.str.contains('Pedro Henrique')]
prices = prices[~prices.name.str.contains('Pedro Gallese')]
prices = prices[~prices.name.str.contains('PêpêPêpê')]
prices = prices[~prices.name.str.contains('ReverRever')]

df['Name'] = df['Name'].str.replace('Ezequiel Cerica','Ezequiel')
df['Name'] = df['Name'].str.replace('Jonas Michel Dirkner','Jonas Dirkner')
df['Name'] = df['Name'].str.replace('Jota Goncalves', 'Goncalves Jota')
df['Name'] = df['Name'].str.replace('Jota Goncalves', 'Goncalves Jota')
df['Name'] = df['Name'].str.replace('Mehmet Zeki Çelik', 'Mehmet Aydin')
df['Name'] = df['Name'].str.replace('Mesut Çaytemel', 'Mesut Caytemel')

prices['name'] = prices['name'].str.replace('BebetoBebeto','Bebeto')
prices['name'] = prices['name'].str.replace('Ben AmosBen Amos','Ben Amos')
prices['name'] = prices['name'].str.replace('Ben MeeBen Mee','Ben Mee')
prices['name'] = prices['name'].str.replace('Brek SheaBrek SheaLeft Midfiel','Brek Shea')
prices['name'] = prices['name'].str.replace('CasemiroCasemiro', 'Casemiro')
prices['name'] = prices['name'].str.replace('FagnerFagner', 'Fagner')
prices['name'] = prices['name'].str.replace('Gelson DalaGelson Dala', 'Gelson Dala')
prices['name'] = prices['name'].str.replace('GiulianoGiuliano', 'Giuliano')
prices['name'] = prices['name'].str.replace('Henrique GomesHenrique Gomes', 'Henrique Gomes')
prices['name'] = prices['name'].str.replace('HenriqueHenrique', 'Henrique')
prices['name'] = prices['name'].str.replace('Jacob Lungi', 'Jacob')
prices['name'] = prices['name'].str.replace('Jonas Dirkner', 'Jonas Michel Dirkner')
prices['name'] = prices['name'].str.replace('Jonas Dirkner', 'Jonas Michel Dirkner')
prices['name'] = prices['name'].str.replace('Josh Eppiah', 'Josh Felix-Eppiah')
prices['name'] = prices['name'].str.replace('JotaJota', 'Jota')
prices['name'] = prices['name'].str.replace('Nuno Da Costa', 'Nuno da Costa')
prices['name'] = prices['name'].str.replace('Pedro de La Vega', 'Pedro De La Vega')
prices['name'] = prices['name'].str.replace('QuiniQuini', 'Quini')
prices['name'] = prices['name'].str.replace('Joel WardJoel Ward', 'Joel Ward')
prices['name'] = prices['name'].str.replace('John Mikel Obi', 'John Obi Mikel')
prices['name'] = prices['name'].str.replace('Jonas Michel Dirkner', 'Jonas Dirkner')
prices['name'] = prices['name'].str.replace('Jota Goncalves', 'Goncalves Jota')
prices['name'] = prices['name'].str.replace('Marcao SilvaMarcao Silva', 'Marcao Silva')
prices['name'] = prices['name'].str.replace('Nils-Jonathan Körber', 'Nils Schouterden')
prices['name'] = prices['name'].str.replace('Pedro NunoPedro Nuno', 'Pedro Nuno')
prices['name'] = prices['name'].str.replace('RickRick', 'Rick')
prices['name'] = prices['name'].str.replace('SanderSander', 'Sander')
prices['name'] = prices['name'].str.replace('SilvioSilvio', 'Silvio')
prices['name'] = prices['name'].str.replace('Vasilios Pavlidis', 'Vasilis Pavlidis')
prices['name'] = prices['name'].str.replace('Zurab Davitashvili', 'Zuriko Davitashvili')
prices['name'] = prices['name'].str.replace('Joao RicardoJoao Ricardo', 'Joao Ricardo')
prices['name'] = prices['name'].str.replace('MarcaoMarcao', 'Marcao')

In [439]:
df = df.drop([1510, 1015, 5815, 1565, 8134])

In [440]:
df = df[~df.Name.str.contains('Angel Jimenez')]
df = df[~df.Name.str.contains('Pedro Henrique')]

prices = prices[~prices.name.str.contains('Angel Romero')]
prices = prices[~prices.name.str.contains('Joao Queiros')]

In [441]:
df = df.drop([3470])

In [442]:
df = df[~df.Name.str.contains('Joao Rodriguez')]

In [443]:
prices['name'] = prices['name'].str.replace('Efe Eric Ambrose', 'Efe Ambrose')
prices['name'] = prices['name'].str.replace('Geoffry Hairemans', 'Geoffrey Hairemans')
prices['name'] = prices['name'].str.replace('RolandoRolando', 'Rolando')

In [444]:
# save the final dataframe
df_final = pd.concat([df.reset_index(drop=True), prices.reset_index(drop=True)], axis=1) 
df_final.to_csv('df_final.csv', index=True)

In [445]:
df_final.shape

(7960, 60)

Now I just reload the dataframe and remove the last non-correspoding rows

In [446]:
df_final = pd.read_csv(os.path.join('df_final.csv')) 

In [448]:
df_final = df_final.drop([293, 1052, 2130, 2521, 2695, 2771, 2903])
df_final.to_csv('df_final.csv', index=True)